In [31]:
from datetime import datetime
import funcoes_eua as f_eua # noqa: F401
import funcoes_sec as f_sec # noqa: F401
import pandas as pd
import yfinance as yf

# Alterando a exibição dos números para não usar notação científica
# pd.set_option('display.float_format', '{:,.2f}'.format)

## Funções de Scraping SEC

Canal Sunrise Analysis:
* Part 1 - https://www.youtube.com/watch?v=y_tvyJCdCBo
* Part 2 - https://www.youtube.com/watch?v=viibnZRIiYc
* Part 3 - https://www.youtube.com/watch?v=wvqfHWaSuJQ&t
* Part 4 - https://www.youtube.com/watch?v=gpvG9vYBzwc
* Part 5 - https://www.youtube.com/watch?v=kJ0VJrJYB48
* Part 6 - https://www.youtube.com/watch?v=SneBju9SgLs

Para fazer o scraping dos principais documentos (balance sheet, income statement e cash flow statement) utilizar as funções:
* **cik_matching_ticker**: mostra qual é o número CIK da empresa;
* **f_sec.get_filtered_filings**: retorna os números dos documentos que contém a DRE, BP e FC de cada ano/trimestre;
* **labels_dict**: retorna os rótulos formatados dos documentos;
* **f_sec.process_one_statement**: faz o scraping do documento;
* **f_sec.rename_statement**: aplica a formatação dos rótulos no documento.

In [ ]:
# Algumas vezes, a função **f_sec.process_one_statement** não consegue fazer o scraping do documento
# É muito provavelmente, porque a string do documento não está no dicionário 'statement_keys_map'
# Para ver qual é a string do documento rodar a função **get_statement_file_names_in_filing_summary**
# Ela retorna um dicionário {'string do documento': 'número do documento'}:
{
    'condensed consolidated statements of income': 'R2.htm',
    'condensed consolidated statement of comprehensive income': 'R3.htm',
    'condensed consolidated balance sheets': 'R4.htm',
    "condensed consolidated statements of shareholders' equity": 'R5.htm',
    'condensed consolidated statements of shareholders??? equity (parenthetical)': 'R6.htm',
    'condensed consolidated statements of cash flows': 'R7.htm'
}

# Verificar se a 'string do documento' está no dicionário 'statement_keys_map'. Caso não esteja, adicionar no 'statement_keys_map'

In [ ]:
# 'https://www.sec.gov/Archives/edgar/data/0001326801/000162828026003942'

# get_statement_file_names_in_filing_summary(ticker, accession_number='000162828026003942', headers=header)

# **Setor de semicondutores**

## NVIDIA

* 1Q -> Lançamento em Abril (04);
* 2Q -> Lançamento em Julho (07);
* 3Q -> Lançamento em Outubro (10);
* 10K -> Lançamento em Janeiro (01).

In [ ]:
ticker = 'nvda'
f_sec.cik_matching_ticker(ticker)

### Scraping NVIDIA 10-K

In [ ]:
# Número dos documentos que contém a DRE, BP e FC 
acc_documents = f_sec.get_filtered_filings(ticker=ticker, ten_k=True, just_accession_numbers=True)

# Trocando os '-' para '' das strings ('0001045810-24-000264' -> '000104581024000264')
dict_acc_num = {k: v.replace('-', '') for k, v in acc_documents.items()}

# Transformando o dicionário em um df
df_lst_acc_num_nvda = pd.DataFrame.from_dict(dict_acc_num, orient='index', columns=['acc_num'])

# Adicionando os 'acc_num' de 2017, 2018
df_acc_num = pd.DataFrame(
    {
        'acc_num': [
            '000104581019000023', # 2019
            '000104581018000010', # 2018
            '000104581017000027', # 2017
        ]
    },
    index=pd.to_datetime([
        '2019-01-27',
        '2018-01-28',
        '2017-01-29',
    ])
)

# Concatenando os dfs
df_lst_acc_num_nvda = pd.concat([df_lst_acc_num_nvda, df_acc_num])

# Transformando em datetime o index
df_lst_acc_num_nvda.index = pd.to_datetime(df_lst_acc_num_nvda.index, format='%Y-%m-%d')

df_lst_acc_num_nvda

In [ ]:
# Rótulos
labels_dict = f_sec.get_label_dictionary(ticker, headers=f_sec.header)

In [ ]:
# Fazendo o scraping do balance sheet (10-K)
balance_sheet_lst = []

for acc_num in df_lst_acc_num_nvda['acc_num']:

    # Balance sheet
    balance_sheet = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='balance_sheet')
    balance_sheet = f_sec.rename_statement(balance_sheet, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    balance_sheet = balance_sheet.iloc[:, 0]
    balance_sheet_lst.append(balance_sheet)

# Juntando todos os dfs que estão em listas em apenas um df
df_balance_sheet_10k = pd.concat(balance_sheet_lst, axis=1)

# Substituindo os NaN por 0
df_balance_sheet_10k = df_balance_sheet_10k.fillna(0)

# A linha 'Deferred Income Tax Assets, Net' na coluna '2020-01-26' está zerada
# A informação está na linha 'Deferred Tax Assets, Gross, Noncurrent'
df_balance_sheet_10k.loc['Deferred Income Tax Assets, Net', '2020-01-26'] = df_balance_sheet_10k.loc['Deferred Tax Assets, Gross, Noncurrent', '2020-01-26'] 

# A linha 'Long-term Debt, Excluding Current Maturities' nas colunas '2020-01-26', '2019-01-27', '2018-01-28' e '2017-01-29' estão zeradas
# As informações estão na linha 'Long-term Debt'
df_balance_sheet_10k.loc['Long-term Debt, Excluding Current Maturities', '2020-01-26'] = df_balance_sheet_10k.loc['Long-term Debt', '2020-01-26'] 
df_balance_sheet_10k.loc['Long-term Debt, Excluding Current Maturities', '2019-01-27'] = df_balance_sheet_10k.loc['Long-term Debt', '2019-01-27'] 
df_balance_sheet_10k.loc['Long-term Debt, Excluding Current Maturities', '2018-01-28'] = df_balance_sheet_10k.loc['Long-term Debt', '2018-01-28'] 
df_balance_sheet_10k.loc['Long-term Debt, Excluding Current Maturities', '2017-01-29'] = df_balance_sheet_10k.loc['Long-term Debt', '2017-01-29'] 

# Removendo as linhas que movemos os seus itens para as linhas corretas
df_balance_sheet_10k = df_balance_sheet_10k.drop([
    'Deferred Tax Assets, Gross, Noncurrent',
    'Long-term Debt'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_balance_sheet_10k = df_balance_sheet_10k.drop([
    'us-gaap_AssetsCurrentAbstract',
    'Preferred Stock, Value, Outstanding',
    'Commitments and Contingencies',
    'Convertible Debt, Current',
    'Temporary Equity, Par Value',
    'Capital Lease Obligations, Noncurrent'
])

df_balance_sheet_10k

In [ ]:
# Fazendo o scraping do income statement (10-K)
income_statement_lst = []

for acc_num in df_lst_acc_num_nvda['acc_num']:

    # Income statement
    income_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='income_statement')
    income_statement = f_sec.rename_statement(income_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    income_statement = income_statement.iloc[:, 0]
    income_statement_lst.append(income_statement)

# Juntando todos os dfs que estão em listas em apenas um df
df_income_statement_10k = pd.concat(income_statement_lst, axis=1)

# Substituindo os NaN por 0
df_income_statement_10k = df_income_statement_10k.fillna(0)

# A linha 'InterestExpenseNonoperating' nas colunas '2024-01-28', '2023-01-29', '2022-01-30', '2021-01-31', '2020-01-26', '2019-01-27', '2018-01-28' e '2017-01-29' estão zeradas
# As informações estão na linha 'Interest Expense'
df_income_statement_10k.loc['InterestExpenseNonoperating', '2024-01-28'] = df_income_statement_10k.loc['Interest Expense', '2024-01-28'] 
df_income_statement_10k.loc['InterestExpenseNonoperating', '2023-01-29'] = df_income_statement_10k.loc['Interest Expense', '2023-01-29'] 
df_income_statement_10k.loc['InterestExpenseNonoperating', '2022-01-30'] = df_income_statement_10k.loc['Interest Expense', '2022-01-30'] 
df_income_statement_10k.loc['InterestExpenseNonoperating', '2021-01-31'] = df_income_statement_10k.loc['Interest Expense', '2021-01-31'] 
df_income_statement_10k.loc['InterestExpenseNonoperating', '2020-01-26'] = df_income_statement_10k.loc['Interest Expense', '2020-01-26'] 
df_income_statement_10k.loc['InterestExpenseNonoperating', '2019-01-27'] = df_income_statement_10k.loc['Interest Expense', '2019-01-27'] 
df_income_statement_10k.loc['InterestExpenseNonoperating', '2019-01-27'] = df_income_statement_10k.loc['Interest Expense', '2019-01-27'] 
df_income_statement_10k.loc['InterestExpenseNonoperating', '2018-01-28'] = df_income_statement_10k.loc['Interest Expense', '2018-01-28'] 
df_income_statement_10k.loc['InterestExpenseNonoperating', '2017-01-29'] = df_income_statement_10k.loc['Interest Expense', '2017-01-29'] 

# A linha 'Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest' nas colunas '2021-01-31', '2020-01-26', '2019-01-27', '2018-01-28' e '2017-01-29' estão zeradas
# As informações estão na linha 'Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest'
df_income_statement_10k.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2021-01-31'] = df_income_statement_10k.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2021-01-31'] 
df_income_statement_10k.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2020-01-26'] = df_income_statement_10k.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2020-01-26'] 
df_income_statement_10k.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2019-01-27'] = df_income_statement_10k.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2019-01-27'] 
df_income_statement_10k.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2019-01-27'] = df_income_statement_10k.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2019-01-27'] 
df_income_statement_10k.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2018-01-28'] = df_income_statement_10k.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2018-01-28'] 
df_income_statement_10k.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2017-01-29'] = df_income_statement_10k.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2017-01-29'] 

# A linha 'Reveneus' nas colunas '2021-01-31', '2020-01-26'	e '2019-01-27' estão zeradas
# As informações estão na linha 'Revenue from Contract with Customer, Excluding Assessed Tax'
df_income_statement_10k.loc['Revenues', '2021-01-31'] = df_income_statement_10k.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2021-01-31'] 
df_income_statement_10k.loc['Revenues', '2020-01-26'] = df_income_statement_10k.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2020-01-26'] 
df_income_statement_10k.loc['Revenues', '2019-01-27'] = df_income_statement_10k.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2019-01-27'] 

# A linha 'Cost of Revenue' nas colunas '2021-01-31', '2020-01-26'	e '2019-01-27' estão zeradas
# As informações estão na linha 'Cost of Goods and Services Sold'
df_income_statement_10k.loc['Cost of Revenue', '2021-01-31'] = df_income_statement_10k.loc['Cost of Goods and Services Sold', '2021-01-31'] 
df_income_statement_10k.loc['Cost of Revenue', '2020-01-26'] = df_income_statement_10k.loc['Cost of Goods and Services Sold', '2020-01-26'] 
df_income_statement_10k.loc['Cost of Revenue', '2019-01-27'] = df_income_statement_10k.loc['Cost of Goods and Services Sold', '2019-01-27'] 

# Removendo as linhas que movemos os seus itens para as linhas corretas
df_income_statement_10k = df_income_statement_10k.drop([
    'Interest Expense',
    'Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest',
    'Revenue from Contract with Customer, Excluding Assessed Tax',
    'Cost of Goods and Services Sold',
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_income_statement_10k = df_income_statement_10k.drop([
    'us-gaap_IncomeStatementAbstract',
    'us-gaap_OperatingExpensesAbstract',
    'Restructuring Charges',
    'Common Stock, Dividends, Per Share, Cash Paid'
])

# Renomeando a linha do índice que está sem formato 
df_income_statement_10k = df_income_statement_10k.rename(index={
    'nvda_BusinessCombinationAdvancedConsiderationWrittenOff': 'Business Combination Advanced Consideration Written Off',
    'InterestExpenseNonoperating': 'Interest Expense'
})

# Formatando o número para ficar positivo
df_income_statement_10k.loc['Interest Expense'] = df_income_statement_10k.loc['Interest Expense'] * -1

df_income_statement_10k

In [ ]:
# Fazendo o scraping do cash flow statement (10-K)
cash_flow_statement_lst = []

for acc_num in df_lst_acc_num_nvda['acc_num']:

    # Cash flow statement
    cash_flow_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='cash_flow_statement')
    cash_flow_statement = f_sec.rename_statement(cash_flow_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    cash_flow_statement = cash_flow_statement.iloc[:, 0]
    cash_flow_statement_lst.append(cash_flow_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_cash_flow_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in cash_flow_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_cash_flow_10k = pd.concat([df_cash_flow_10k, df], axis=1)

# Substituindo os NaN por 0
df_cash_flow_10k = df_cash_flow_10k.fillna(0)

# A linha 'Depreciation, Depletion and Amortization' nas colunas '2021-01-31', '2020-01-26', '2019-01-27', '2018-01-28' e '2017-01-29' estão zeradas
# As informações estão na linha 'Depreciation, Depletion and Amortization, Nonproduction'
df_cash_flow_10k.loc['Depreciation, Depletion and Amortization', '2021-01-31'] = df_cash_flow_10k.loc['Depreciation, Depletion and Amortization, Nonproduction', '2021-01-31'] 
df_cash_flow_10k.loc['Depreciation, Depletion and Amortization', '2020-01-26'] = df_cash_flow_10k.loc['Depreciation, Depletion and Amortization, Nonproduction', '2020-01-26'] 
df_cash_flow_10k.loc['Depreciation, Depletion and Amortization', '2019-01-27'] = df_cash_flow_10k.loc['Depreciation, Depletion and Amortization, Nonproduction', '2019-01-27'] 
df_cash_flow_10k.loc['Depreciation, Depletion and Amortization', '2018-01-28'] = df_cash_flow_10k.loc['Depreciation, Depletion and Amortization, Nonproduction', '2018-01-28'] 
df_cash_flow_10k.loc['Depreciation, Depletion and Amortization', '2017-01-29'] = df_cash_flow_10k.loc['Depreciation, Depletion and Amortization, Nonproduction', '2017-01-29'] 

# A linha 'Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale' nas colunas '2021-01-31', '2020-01-26', '2019-01-27', '2018-01-28' e '2017-01-29' estão zeradas
# As informações estão na linha 'Proceeds from Sale and Maturity of Marketable Securities'
df_cash_flow_10k.loc['Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale', '2021-01-31'] = df_cash_flow_10k.loc['Proceeds from Sale and Maturity of Marketable Securities', '2021-01-31'] 
df_cash_flow_10k.loc['Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale', '2020-01-26'] = df_cash_flow_10k.loc['Proceeds from Sale and Maturity of Marketable Securities', '2020-01-26'] 
df_cash_flow_10k.loc['Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale', '2019-01-27'] = df_cash_flow_10k.loc['Proceeds from Sale and Maturity of Marketable Securities', '2019-01-27'] 
df_cash_flow_10k.loc['Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale', '2018-01-28'] = df_cash_flow_10k.loc['Proceeds from Sale and Maturity of Marketable Securities', '2018-01-28'] 
df_cash_flow_10k.loc['Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale', '2017-01-29'] = df_cash_flow_10k.loc['Proceeds from Sale and Maturity of Marketable Securities', '2017-01-29'] 

# A linha 'Payments to Acquire Productive Assets' nas colunas '2023-01-29', '2022-01-30', '2021-01-31', '2020-01-26', '2019-01-27', '2018-01-28' e '2017-01-29' estão zeradas
# As informações estão na linha 'nvda_PurchasesOfPropertyAndEquipmentAndIntangibleAssets'
df_cash_flow_10k.loc['Payments to Acquire Productive Assets', '2023-01-29'] = df_cash_flow_10k.loc['nvda_PurchasesOfPropertyAndEquipmentAndIntangibleAssets', '2023-01-29'] 
df_cash_flow_10k.loc['Payments to Acquire Productive Assets', '2022-01-30'] = df_cash_flow_10k.loc['nvda_PurchasesOfPropertyAndEquipmentAndIntangibleAssets', '2022-01-30'] 
df_cash_flow_10k.loc['Payments to Acquire Productive Assets', '2021-01-31'] = df_cash_flow_10k.loc['nvda_PurchasesOfPropertyAndEquipmentAndIntangibleAssets', '2021-01-31'] 
df_cash_flow_10k.loc['Payments to Acquire Productive Assets', '2020-01-26'] = df_cash_flow_10k.loc['nvda_PurchasesOfPropertyAndEquipmentAndIntangibleAssets', '2020-01-26'] 
df_cash_flow_10k.loc['Payments to Acquire Productive Assets', '2019-01-27'] = df_cash_flow_10k.loc['nvda_PurchasesOfPropertyAndEquipmentAndIntangibleAssets', '2019-01-27'] 
df_cash_flow_10k.loc['Payments to Acquire Productive Assets', '2018-01-28'] = df_cash_flow_10k.loc['nvda_PurchasesOfPropertyAndEquipmentAndIntangibleAssets', '2018-01-28'] 
df_cash_flow_10k.loc['Payments to Acquire Productive Assets', '2017-01-29'] = df_cash_flow_10k.loc['nvda_PurchasesOfPropertyAndEquipmentAndIntangibleAssets', '2017-01-29'] 

# A linha 'Proceeds from Stock Plans' nas colunas '2023-01-29', '2022-01-30', '2021-01-31', '2020-01-26', '2019-01-27', '2018-01-28' e '2017-01-29' estão zeradas
# As informações estão na linha 'nvda_Netproceedspaymentsrelatedtoemployeestockplans'
df_cash_flow_10k.loc['Proceeds from Stock Plans', '2023-01-29'] = df_cash_flow_10k.loc['nvda_Netproceedspaymentsrelatedtoemployeestockplans', '2023-01-29'] 
df_cash_flow_10k.loc['Proceeds from Stock Plans', '2022-01-30'] = df_cash_flow_10k.loc['nvda_Netproceedspaymentsrelatedtoemployeestockplans', '2022-01-30'] 
df_cash_flow_10k.loc['Proceeds from Stock Plans', '2021-01-31'] = df_cash_flow_10k.loc['nvda_Netproceedspaymentsrelatedtoemployeestockplans', '2021-01-31'] 
df_cash_flow_10k.loc['Proceeds from Stock Plans', '2020-01-26'] = df_cash_flow_10k.loc['nvda_Netproceedspaymentsrelatedtoemployeestockplans', '2020-01-26'] 
df_cash_flow_10k.loc['Proceeds from Stock Plans', '2019-01-27'] = df_cash_flow_10k.loc['nvda_Netproceedspaymentsrelatedtoemployeestockplans', '2019-01-27'] 
df_cash_flow_10k.loc['Proceeds from Stock Plans', '2018-01-28'] = df_cash_flow_10k.loc['nvda_Netproceedspaymentsrelatedtoemployeestockplans', '2018-01-28'] 
df_cash_flow_10k.loc['Proceeds from Stock Plans', '2017-01-29'] = df_cash_flow_10k.loc['nvda_Netproceedspaymentsrelatedtoemployeestockplans', '2017-01-29'] 

# A linha 'nvda_PaymentsForFinancedPropertyPlantAndEquipmentAndIntangibleAssetsFinancingActivities' nas colunas '2023-01-29', '2022-01-30' e '2021-01-31' estão zeradas
# As informações estão na linha 'nvda_PaymentsForFinancedPropertyPlantAndEquipmentFinancingActivities'
df_cash_flow_10k.loc['nvda_PaymentsForFinancedPropertyPlantAndEquipmentAndIntangibleAssetsFinancingActivities', '2023-01-29'] = df_cash_flow_10k.loc['nvda_PaymentsForFinancedPropertyPlantAndEquipmentFinancingActivities', '2023-01-29'] 
df_cash_flow_10k.loc['nvda_PaymentsForFinancedPropertyPlantAndEquipmentAndIntangibleAssetsFinancingActivities', '2022-01-30'] = df_cash_flow_10k.loc['nvda_PaymentsForFinancedPropertyPlantAndEquipmentFinancingActivities', '2022-01-30'] 
df_cash_flow_10k.loc['nvda_PaymentsForFinancedPropertyPlantAndEquipmentAndIntangibleAssetsFinancingActivities', '2021-01-31'] = df_cash_flow_10k.loc['nvda_PaymentsForFinancedPropertyPlantAndEquipmentFinancingActivities', '2021-01-31'] 

# A linha 'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect' nas colunas '2019-01-27', '2018-01-28' e '2017-01-29' estão zeradas
# As informações estão na linha 'Cash and Cash Equivalents, Period Increase (Decrease)'
df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect', '2019-01-27'] = df_cash_flow_10k.loc['Cash and Cash Equivalents, Period Increase (Decrease)', '2019-01-27'] 
df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect', '2018-01-28'] = df_cash_flow_10k.loc['Cash and Cash Equivalents, Period Increase (Decrease)', '2018-01-28'] 
df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect', '2017-01-29'] = df_cash_flow_10k.loc['Cash and Cash Equivalents, Period Increase (Decrease)', '2017-01-29'] 

# A linha 'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents' nas colunas '2019-01-27', '2018-01-28' e '2017-01-29' estão zeradas
# As informações estão na linha 'Cash and Cash Equivalents, at Carrying Value'
df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2019-01-27'] = df_cash_flow_10k.loc['Cash and Cash Equivalents, at Carrying Value', '2019-01-27'] 
df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2018-01-28'] = df_cash_flow_10k.loc['Cash and Cash Equivalents, at Carrying Value', '2018-01-28'] 
df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2017-01-29'] = df_cash_flow_10k.loc['Cash and Cash Equivalents, at Carrying Value', '2017-01-29'] 

# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_cash_flow_10k = df_cash_flow_10k.drop([
    'Depreciation, Depletion and Amortization, Nonproduction',
    'Proceeds from Sale and Maturity of Marketable Securities',
    'nvda_PurchasesOfPropertyAndEquipmentAndIntangibleAssets',
    'nvda_Netproceedspaymentsrelatedtoemployeestockplans',
    'nvda_PaymentsForFinancedPropertyPlantAndEquipmentFinancingActivities',
    'Cash and Cash Equivalents, Period Increase (Decrease)',
    'Cash and Cash Equivalents, at Carrying Value'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_cash_flow_10k = df_cash_flow_10k.drop([
    'us-gaap_NetCashProvidedByUsedInOperatingActivitiesAbstract',
    'nvda_RestructuringchargesNoncash',
    'Excess Tax Benefit from Share-based Compensation, Operating Activities',
    'nvda_Reimbursementofheadquartersbuildingdevelopmentcostsfromlessor',
    'Excess Tax Benefit from Share-based Compensation, Financing Activities',
    'nvda_Goodwilladjustmentrelatedtopreviouslyacquiredbusiness3dfx'
])

# Arrumando e adicionando alguns números no df
df_cash_flow_10k.loc['ProceedsFromSaleOfEquitySecuritiesFvNi', '2024-01-28'] = 1
df_cash_flow_10k.loc['ProceedsFromSaleOfEquitySecuritiesFvNi', '2023-01-29'] = 8

df_cash_flow_10k.loc['PaymentsToAcquireEquitySecuritiesFvNi', '2024-01-28'] = -862
df_cash_flow_10k.loc['PaymentsToAcquireEquitySecuritiesFvNi', '2023-01-29'] = -85

# Renomeando as linhas do índice que estão sem formato 
df_cash_flow_10k = df_cash_flow_10k.rename(index={
    'nvda_BusinessCombinationAdvancedConsiderationWrittenOff': 'Business Combination Advanced Consideration Written Off',
    'nvda_PaymentsForFinancedPropertyPlantAndEquipmentAndIntangibleAssetsFinancingActivities':'Payments For Financed Property Plant And Equipment And Intangible Assets Financing Activities',
    'nvda_Proceedsfromsaleoflonglivedassetsandinvestments': 'Proceeds From Sale Of Long Lived Assets And Investments',
    'nvda_GainsonSalesoflonglivedassetsandinvestment': 'Gains on Sales of Long Lived Assets And Investment'
})

df_cash_flow_10k

In [ ]:
# Arrumando alguns index dos dfs para ficar padronizado com os index utilizados nas funções p/ calcular os indicadores fundamentalistas
df_balance_sheet_10k = df_balance_sheet_10k.rename(index={
    'Cash and Cash Equivalents, at Carrying Value': 'Cash and Cash Equivalents',
    'Marketable Securities, Current': 'Marketable Securities',
    'Debt, Current': 'Short-term Debt',
    'Long-term Debt, Excluding Current Maturities': 'Long-term Debt',
    'Operating Lease, Liability, Noncurrent': 'Long-term Operating Lease Liabilities',
    "Stockholders' Equity Attributable to Parent": 'Total Shareholders Equity'
})

df_income_statement_10k = df_income_statement_10k.rename(index={
    'Research and Development Expense': 'Research and Development',
    'Operating Income (Loss)': 'Operating Income',
    'Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest': 'Income Before Provision for Income Taxes',
    'Income Tax Expense (Benefit)': 'Provision for Income Taxes',
    'Net Income (Loss) Attributable to Parent': 'Net Income',
    'Weighted Average Number of Shares Outstanding, Basic': 'Number of Shares - Basic',
    'Weighted Average Number of Shares Outstanding, Diluted': 'Number of Shares - Diluted'
})

df_cash_flow_10k = df_cash_flow_10k.rename(index={
    'Payments to Acquire Productive Assets': 'Purchases Of Property And Equipment And Intangible Assets'
})

In [ ]:
# 'acc_num' entre 2022 e 2025
df_lst_acc_num_nvda_sorted = df_lst_acc_num_nvda.sort_index()
acc_num_list_nvda_sliced = df_lst_acc_num_nvda_sorted.loc['2022':'2025', 'acc_num']

# Fazendo o scraping do leases (10-K)
leases_lst = []

for acc_num in acc_num_list_nvda_sliced:

    # Fazendo o scraping do leases (10-K) -> 'Note 17 – Leases' -> 'short-term operating lease liabilities'
    leases_10k = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='leases')
    leases_10k = f_sec.rename_statement(leases_10k, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    leases_10k = leases_10k.iloc[:, 0]
    leases_lst.append(leases_10k)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_leases_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in leases_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_leases_10k = pd.concat([df_leases_10k, df], axis=1)
    
# Renomeando a linha do índice
df_leases_10k = df_leases_10k.rename(index={
    'Operating Lease, Liability, Current': 'Short-term Operating Lease Liabilities',

})

# Selecionando a linha desejada
df_leases_10k = df_leases_10k.loc[['Short-term Operating Lease Liabilities']]

# Criando as colunas dos anos de 2017, 2018, 2019, 2020 e 2021
df_leases_10k.loc['Short-term Operating Lease Liabilities', '2017-01-29'] = 0
df_leases_10k.loc['Short-term Operating Lease Liabilities', '2018-01-28'] = 0
df_leases_10k.loc['Short-term Operating Lease Liabilities', '2019-01-27'] = 0
df_leases_10k.loc['Short-term Operating Lease Liabilities', '2021-01-31'] = 121
df_leases_10k.loc['Short-term Operating Lease Liabilities', '2020-01-26'] = 91

# Ordenando o df
df_leases_10k = df_leases_10k[sorted(df_leases_10k.columns, reverse=True)]

df_leases_10k

In [ ]:
# Selecionando o 'acc_num' do ano de 2024 - Nvidia tem 'Investments in non-affiliated entities' em 2024. Em 2025, ela não possui
acc_num_2024 = '000104581024000029'

# Fazendo o scraping do 'Investments in non-affiliated entities' (10-K) -> 'Note 10 - Balance Sheet Components' → 'Other assets' 
cross_holding_10k = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num_2024, statement_name='cross_holding')
cross_holding_10k = f_sec.rename_statement(cross_holding_10k, label_dictionary=labels_dict)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
cross_holding_10k = cross_holding_10k.drop('us-gaap_SupplyCommitmentLineItems')

# Renomeando a linha do índice
cross_holding_10k  = cross_holding_10k .rename(index={
    'Other Long-term Investments': 'Investments in Non-affiliated Entities',

})

# Selecionando a linha desejada
cross_holding_10k  = cross_holding_10k .loc[['Investments in Non-affiliated Entities']]

cross_holding_10k 

In [ ]:
last_acc_num = df_lst_acc_num_nvda.iloc[0][0]

# Fazendo o scraping das receitas por segmento geográfico
geographic_revenue_10k = f_sec.process_one_statement(ticker=ticker, accession_number=last_acc_num, statement_name='geographic_revenue')
geographic_revenue_10k = f_sec.rename_statement(geographic_revenue_10k, label_dictionary=labels_dict)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
geographic_revenue_10k = geographic_revenue_10k.drop('us-gaap_RevenuesFromExternalCustomersAndLongLivedAssetsLineItems')

# Resetando o index do df, porque ele possui vários index igauis
geographic_revenue_10k = geographic_revenue_10k.reset_index(drop=True)

# Renomeado o index do df
geographic_revenue_10k = geographic_revenue_10k.rename(index={
    0: 'Total revenue',
    2: 'Revenue - United States',
    4: 'Revenue - Singapore',
    5: 'Revenue - Taiwan',
    7: 'Revenue - China',
    9: 'Revenue - Other',
})

# Selecionando as linhas desejadas
geographic_revenue_10k  = geographic_revenue_10k .loc[[
    'Total revenue',
    'Revenue - United States',
    'Revenue - Singapore',
    'Revenue - Taiwan',
    'Revenue - China',
    'Revenue - Other'
]]

geographic_revenue_10k

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//semicondutores//nvidia//nvda_10k.xlsx') as writer:
    df_balance_sheet_10k.to_excel(writer, sheet_name='balance_sheet')
    df_income_statement_10k.to_excel(writer, sheet_name='income_statement')
    df_cash_flow_10k.to_excel(writer, sheet_name='cash_flow')
    df_leases_10k.to_excel(writer, sheet_name='leases')
    cross_holding_10k.to_excel(writer, sheet_name='cross_holding')
    geographic_revenue_10k.to_excel(writer, sheet_name='geographic_revenue')

### Scraping NVIDIA 10-Q

In [ ]:
# Número dos documentos que contém a DRE, BP e FC 
acc_documents_10q = f_sec.get_filtered_filings(ticker=ticker, ten_k=False, just_accession_numbers=True)

# Trocando os '-' para '' das strings ('0001045810-24-000264' -> '000104581024000264')
dict_acc_num_10q = {k: v.replace('-', '') for k, v in acc_documents_10q.items()}

# Transformando o dicionário em um df
df_lst_acc_num_10q_nvda = pd.DataFrame.from_dict(dict_acc_num_10q, orient='index', columns=['acc_num'])

# Transformando em datetime o index
df_lst_acc_num_10q_nvda.index = pd.to_datetime(df_lst_acc_num_10q_nvda.index, format='%Y-%m-%d')
df_lst_acc_num_10q_nvda

In [ ]:
# Selecionando os 'acc_num' dos 10-Q a partir de 2020 p/ iteração
df_lst_acc_num_10q_nvda_sliced = df_lst_acc_num_10q_nvda.loc['2020-04-26':]
df_lst_acc_num_10q_nvda_sliced

In [ ]:
# Rótulos
labels_dict = f_sec.get_label_dictionary(ticker, headers=f_sec.header)

In [ ]:
# Fazendo o scraping do balance sheet (10-Q)
balance_sheet_lst = []

for acc_num in df_lst_acc_num_10q_nvda_sliced['acc_num']:

    # Balance sheet
    balance_sheet = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='balance_sheet')
    balance_sheet = f_sec.rename_statement(balance_sheet, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    balance_sheet = balance_sheet.iloc[:, 0]
    balance_sheet_lst.append(balance_sheet)

# Juntando todos os dfs que estão em listas em apenas um df
df_balance_sheet_10q = pd.concat(balance_sheet_lst, axis=1)

# Substituindo os NaN por 0
df_balance_sheet_10q = df_balance_sheet_10q.fillna(0)

# A linha 'Long-term Debt, Excluding Current Maturities' nas colunas '2020-07-26' e '2020-04-26' estão zeradas
# As informações estão na linha 'Long-term Debt'
df_balance_sheet_10q.loc['Long-term Debt, Excluding Current Maturities', '2020-07-26'] = df_balance_sheet_10q.loc['Long-term Debt', '2020-07-26'] 
df_balance_sheet_10q.loc['Long-term Debt, Excluding Current Maturities', '2020-04-26'] = df_balance_sheet_10q.loc['Long-term Debt', '2020-04-26'] 

# A linha 'Other Assets, Noncurrent' nas colunas '2020-07-26' e '2020-04-26' estão zeradas
# As informações estão na linha 'Other Assets'
df_balance_sheet_10q.loc['Other Assets, Noncurrent', '2020-07-26'] = df_balance_sheet_10q.loc['Other Assets', '2020-07-26'] 
df_balance_sheet_10q.loc['Other Assets, Noncurrent', '2020-04-26'] = df_balance_sheet_10q.loc['Other Assets', '2020-04-26'] 

# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_balance_sheet_10q = df_balance_sheet_10q.drop([
    'Long-term Debt',
    'Other Assets'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_balance_sheet_10q = df_balance_sheet_10q.drop([
    'us-gaap_AssetsCurrentAbstract',
    'Preferred Stock, Value, Outstanding'

])

df_balance_sheet_10q

In [ ]:
# Fazendo o scraping do income statement (10-Q)
income_statement_lst = []

for acc_num in df_lst_acc_num_10q_nvda_sliced['acc_num']:

    # Income statement
    income_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='income_statement')
    income_statement = f_sec.rename_statement(income_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    income_statement = income_statement.iloc[:, 0]
    income_statement_lst.append(income_statement)

# Juntando todos os dfs que estão em listas em apenas um df
df_income_statement_10q = pd.concat(income_statement_lst, axis=1)

# Substituindo os NaN por 0
df_income_statement_10q = df_income_statement_10q.fillna(0)

# A linha 'InterestExpenseNonoperating' nas colunas '2024-04-28', '2023-10-29', '2023-07-30', '2023-04-30', '2022-10-30', '2022-07-31', '2022-05-01', '2021-10-31', '2021-08-01', '2021-05-02', '2020-10-25',	'2020-07-26' e '2020-04-26' estão zeradas
# As informações estão na linha 'Interest Expense'
df_income_statement_10q.loc['InterestExpenseNonoperating', '2024-04-28'] = df_income_statement_10q.loc['Interest Expense', '2024-04-28'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2023-10-29'] = df_income_statement_10q.loc['Interest Expense', '2023-10-29'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2023-07-30'] = df_income_statement_10q.loc['Interest Expense', '2023-07-30'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2023-04-30'] = df_income_statement_10q.loc['Interest Expense', '2023-04-30'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2022-10-30'] = df_income_statement_10q.loc['Interest Expense', '2022-10-30'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2022-07-31'] = df_income_statement_10q.loc['Interest Expense', '2022-07-31'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2022-05-01'] = df_income_statement_10q.loc['Interest Expense', '2022-05-01'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2021-10-31'] = df_income_statement_10q.loc['Interest Expense', '2021-10-31'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2021-08-01'] = df_income_statement_10q.loc['Interest Expense', '2021-08-01'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2021-05-02'] = df_income_statement_10q.loc['Interest Expense', '2021-05-02'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2020-10-25'] = df_income_statement_10q.loc['Interest Expense', '2020-10-25'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2020-07-26'] = df_income_statement_10q.loc['Interest Expense', '2020-07-26'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2020-04-26'] = df_income_statement_10q.loc['Interest Expense', '2020-04-26'] 

# A linha 'Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest' nas colunas '2021-08-01', '2021-05-02', '2020-10-25',	'2020-07-26' e '2020-04-26' estão zeradas
# As informações estão na linha 'Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest'
df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2021-08-01'] = df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2021-08-01'] 
df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2021-05-02'] = df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2021-05-02'] 
df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2020-10-25'] = df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2020-10-25'] 
df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2020-07-26'] = df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2020-07-26'] 
df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2020-04-26'] = df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2020-04-26'] 

# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_income_statement_10q = df_income_statement_10q.drop([
    'Interest Expense',
    'Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest',
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_income_statement_10q = df_income_statement_10q.drop('us-gaap_IncomeStatementAbstract')

# Renomeando a linha do índice 'nvda_BusinessCombinationAdvancedConsiderationWrittenOff'
df_income_statement_10q = df_income_statement_10q.rename(index={
    'nvda_BusinessCombinationAdvancedConsiderationWrittenOff': 'Business Combination Advanced Consideration Written Off',
    'InterestExpenseNonoperating': 'Interest Expense'
})

# Formatando o número para ficar positivo
df_income_statement_10q.loc['Interest Expense'] = df_income_statement_10q.loc['Interest Expense'] * -1

df_income_statement_10q

In [ ]:
# Fazendo o scraping do cash flow statement (10-Q)
cash_flow_statement_lst = []

for acc_num in df_lst_acc_num_10q_nvda_sliced['acc_num']:

    # Cash flow statement
    cash_flow_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='cash_flow_statement')
    cash_flow_statement = f_sec.rename_statement(cash_flow_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    cash_flow_statement = cash_flow_statement.iloc[:, 0]
    cash_flow_statement_lst.append(cash_flow_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_cash_flow_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in cash_flow_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_cash_flow_10q = pd.concat([df_cash_flow_10q, df], axis=1)

# Substituindo os NaN por 0
df_cash_flow_10q = df_cash_flow_10q.fillna(0)

# A linha 'Depreciation, Depletion and Amortization' nas colunas '2021-08-01', '2021-05-02', '2020-10-25', '2020-07-26' e '2020-04-26' estão zeradas	
# As informações estão na linha 'Depreciation, Depletion and Amortization, Nonproduction'
df_cash_flow_10q.loc['Depreciation, Depletion and Amortization', '2021-08-01'] = df_cash_flow_10q.loc['Depreciation, Depletion and Amortization, Nonproduction', '2021-08-01'] 
df_cash_flow_10q.loc['Depreciation, Depletion and Amortization', '2021-05-02'] = df_cash_flow_10q.loc['Depreciation, Depletion and Amortization, Nonproduction', '2021-05-02'] 
df_cash_flow_10q.loc['Depreciation, Depletion and Amortization', '2020-10-25'] = df_cash_flow_10q.loc['Depreciation, Depletion and Amortization, Nonproduction', '2020-10-25'] 
df_cash_flow_10q.loc['Depreciation, Depletion and Amortization', '2020-07-26'] = df_cash_flow_10q.loc['Depreciation, Depletion and Amortization, Nonproduction', '2020-07-26'] 
df_cash_flow_10q.loc['Depreciation, Depletion and Amortization', '2020-04-26'] = df_cash_flow_10q.loc['Depreciation, Depletion and Amortization, Nonproduction', '2020-04-26'] 

# A linha 'Share-based Payment Arrangement, Noncash Expense' na coluna '2023-04-30' estão zeradas
# As informações estão na linha 'Share-based Payment Arrangement, Expense'
df_cash_flow_10q.loc['Share-based Payment Arrangement, Noncash Expense', '2023-04-30'] = df_cash_flow_10q.loc['Share-based Payment Arrangement, Expense', '2023-04-30'] 

# A linha 'nvda_NetProceedsPaymentsRelatedToEmployeeStockPlans' nas colunas '2022-05-01', '2021-10-31', '2021-08-01', '2021-05-02', '2020-10-25', '2020-07-26' e '2020-04-26' estão zeradas
# As informações estão na linha 'nvda_Netproceedspaymentsrelatedtoemployeestockplans'
df_cash_flow_10q.loc['nvda_NetProceedsPaymentsRelatedToEmployeeStockPlans', '2022-05-01'] = df_cash_flow_10q.loc['nvda_Netproceedspaymentsrelatedtoemployeestockplans', '2022-05-01'] 
df_cash_flow_10q.loc['nvda_NetProceedsPaymentsRelatedToEmployeeStockPlans', '2021-10-31'] = df_cash_flow_10q.loc['nvda_Netproceedspaymentsrelatedtoemployeestockplans', '2021-10-31'] 
df_cash_flow_10q.loc['nvda_NetProceedsPaymentsRelatedToEmployeeStockPlans', '2021-08-01'] = df_cash_flow_10q.loc['nvda_Netproceedspaymentsrelatedtoemployeestockplans', '2021-08-01'] 
df_cash_flow_10q.loc['nvda_NetProceedsPaymentsRelatedToEmployeeStockPlans', '2021-05-02'] = df_cash_flow_10q.loc['nvda_Netproceedspaymentsrelatedtoemployeestockplans', '2021-05-02']
df_cash_flow_10q.loc['nvda_NetProceedsPaymentsRelatedToEmployeeStockPlans', '2020-10-25'] = df_cash_flow_10q.loc['nvda_Netproceedspaymentsrelatedtoemployeestockplans', '2020-10-25']
df_cash_flow_10q.loc['nvda_NetProceedsPaymentsRelatedToEmployeeStockPlans', '2020-07-26'] = df_cash_flow_10q.loc['nvda_Netproceedspaymentsrelatedtoemployeestockplans', '2020-07-26']
df_cash_flow_10q.loc['nvda_NetProceedsPaymentsRelatedToEmployeeStockPlans', '2020-04-26'] = df_cash_flow_10q.loc['nvda_Netproceedspaymentsrelatedtoemployeestockplans', '2020-04-26']

# A linha 'nvda_PaymentsForFinancedPropertyPlantAndEquipmentAndIntangibleAssetsFinancingActivities' nas colunas '2021-10-31', '2021-08-01', '2021-05-02', '2020-10-25', '2020-07-26' e '2020-04-26' estão zeradas
# As informações estão na linha 'nvda_PaymentsForFinancedPropertyPlantAndEquipmentFinancingActivities'
df_cash_flow_10q.loc['nvda_PaymentsForFinancedPropertyPlantAndEquipmentAndIntangibleAssetsFinancingActivities', '2021-10-31'] = df_cash_flow_10q.loc['nvda_PaymentsForFinancedPropertyPlantAndEquipmentFinancingActivities', '2021-10-31'] 
df_cash_flow_10q.loc['nvda_PaymentsForFinancedPropertyPlantAndEquipmentAndIntangibleAssetsFinancingActivities', '2021-08-01'] = df_cash_flow_10q.loc['nvda_PaymentsForFinancedPropertyPlantAndEquipmentFinancingActivities', '2021-08-01'] 
df_cash_flow_10q.loc['nvda_PaymentsForFinancedPropertyPlantAndEquipmentAndIntangibleAssetsFinancingActivities', '2021-05-02'] = df_cash_flow_10q.loc['nvda_PaymentsForFinancedPropertyPlantAndEquipmentFinancingActivities', '2021-05-02']
df_cash_flow_10q.loc['nvda_PaymentsForFinancedPropertyPlantAndEquipmentAndIntangibleAssetsFinancingActivities', '2020-10-25'] = df_cash_flow_10q.loc['nvda_PaymentsForFinancedPropertyPlantAndEquipmentFinancingActivities', '2020-10-25']
df_cash_flow_10q.loc['nvda_PaymentsForFinancedPropertyPlantAndEquipmentAndIntangibleAssetsFinancingActivities', '2020-07-26'] = df_cash_flow_10q.loc['nvda_PaymentsForFinancedPropertyPlantAndEquipmentFinancingActivities', '2020-07-26']
df_cash_flow_10q.loc['nvda_PaymentsForFinancedPropertyPlantAndEquipmentAndIntangibleAssetsFinancingActivities', '2020-04-26'] = df_cash_flow_10q.loc['nvda_PaymentsForFinancedPropertyPlantAndEquipmentFinancingActivities', '2020-04-26']

# A linha 'Payments to Acquire Productive Assets' nas colunas '2023-07-30', '2023-04-30', '2022-10-30', '2022-07-31' e '2022-05-01' estão zeradas
# As informações estão na linha 'nvda_PurchasesOfPropertyAndEquipmentAndIntangibleAssets'
df_cash_flow_10q.loc['Payments to Acquire Productive Assets', '2023-07-30'] = df_cash_flow_10q.loc['nvda_PurchasesOfPropertyAndEquipmentAndIntangibleAssets', '2023-07-30']
df_cash_flow_10q.loc['Payments to Acquire Productive Assets', '2023-04-30'] = df_cash_flow_10q.loc['nvda_PurchasesOfPropertyAndEquipmentAndIntangibleAssets', '2023-04-30']
df_cash_flow_10q.loc['Payments to Acquire Productive Assets', '2022-10-30'] = df_cash_flow_10q.loc['nvda_PurchasesOfPropertyAndEquipmentAndIntangibleAssets', '2022-10-30']
df_cash_flow_10q.loc['Payments to Acquire Productive Assets', '2022-07-31'] = df_cash_flow_10q.loc['nvda_PurchasesOfPropertyAndEquipmentAndIntangibleAssets', '2022-07-31']
df_cash_flow_10q.loc['Payments to Acquire Productive Assets', '2022-05-01'] = df_cash_flow_10q.loc['nvda_PurchasesOfPropertyAndEquipmentAndIntangibleAssets', '2022-05-01']

# A linha 'Payments for (Proceeds from) Other Investing Activities' nas colunas '2020-10-25', '2020-07-26' e '2020-04-26' estão zeradas
# As informações estão na linha 'Payments to Acquire Other Investments'
df_cash_flow_10q.loc['Payments for (Proceeds from) Other Investing Activities', '2020-10-25'] = df_cash_flow_10q.loc['Payments to Acquire Other Investments', '2020-10-25']
df_cash_flow_10q.loc['Payments for (Proceeds from) Other Investing Activities', '2020-07-26'] = df_cash_flow_10q.loc['Payments to Acquire Other Investments', '2020-07-26']
df_cash_flow_10q.loc['Payments for (Proceeds from) Other Investing Activities', '2020-04-26'] = df_cash_flow_10q.loc['Payments to Acquire Other Investments', '2020-04-26']

# A linha 'nvda_PaymentsForFinancedPropertyPlantAndEquipmentAndIntangibleAssetsFinancingActivities' nas colunas '2020-07-26' e '2020-04-26' estão zeradas
# As informações estão na linha 'Payments to Acquire Property, Plant, and Equipment'
df_cash_flow_10q.loc['nvda_PaymentsForFinancedPropertyPlantAndEquipmentAndIntangibleAssetsFinancingActivities', '2020-07-26'] = df_cash_flow_10q.loc['Payments to Acquire Property, Plant, and Equipment', '2020-07-26']
df_cash_flow_10q.loc['nvda_PaymentsForFinancedPropertyPlantAndEquipmentAndIntangibleAssetsFinancingActivities', '2020-04-26'] = df_cash_flow_10q.loc['Payments to Acquire Property, Plant, and Equipment', '2020-04-26']

# A linha 'Proceeds from Stock Plans' nas colunas '2023-10-29', '2023-07-30', '2023-04-30',	'2022-10-30', '2022-07-31', '2022-05-01', '2021-10-31', '2021-08-01', '2021-05-02', '2020-10-25', '2020-07-26' e '2020-04-26' estão zeradas
# As informações estão na linha 'nvda_NetProceedsPaymentsRelatedToEmployeeStockPlans'
df_cash_flow_10q.loc['Proceeds from Stock Plans', '2023-10-29'] = df_cash_flow_10q.loc['nvda_NetProceedsPaymentsRelatedToEmployeeStockPlans', '2023-10-29']
df_cash_flow_10q.loc['Proceeds from Stock Plans', '2023-07-30'] = df_cash_flow_10q.loc['nvda_NetProceedsPaymentsRelatedToEmployeeStockPlans', '2023-07-30']
df_cash_flow_10q.loc['Proceeds from Stock Plans', '2023-04-30'] = df_cash_flow_10q.loc['nvda_NetProceedsPaymentsRelatedToEmployeeStockPlans', '2023-04-30']
df_cash_flow_10q.loc['Proceeds from Stock Plans', '2022-10-30'] = df_cash_flow_10q.loc['nvda_NetProceedsPaymentsRelatedToEmployeeStockPlans', '2022-10-30']
df_cash_flow_10q.loc['Proceeds from Stock Plans', '2022-07-31'] = df_cash_flow_10q.loc['nvda_NetProceedsPaymentsRelatedToEmployeeStockPlans', '2022-07-31']
df_cash_flow_10q.loc['Proceeds from Stock Plans', '2022-05-01'] = df_cash_flow_10q.loc['nvda_NetProceedsPaymentsRelatedToEmployeeStockPlans', '2022-05-01']
df_cash_flow_10q.loc['Proceeds from Stock Plans', '2021-10-31'] = df_cash_flow_10q.loc['nvda_NetProceedsPaymentsRelatedToEmployeeStockPlans', '2021-10-31']
df_cash_flow_10q.loc['Proceeds from Stock Plans', '2021-08-01'] = df_cash_flow_10q.loc['nvda_NetProceedsPaymentsRelatedToEmployeeStockPlans', '2021-08-01']
df_cash_flow_10q.loc['Proceeds from Stock Plans', '2021-05-02'] = df_cash_flow_10q.loc['nvda_NetProceedsPaymentsRelatedToEmployeeStockPlans', '2021-05-02']
df_cash_flow_10q.loc['Proceeds from Stock Plans', '2020-10-25'] = df_cash_flow_10q.loc['nvda_NetProceedsPaymentsRelatedToEmployeeStockPlans', '2020-10-25']
df_cash_flow_10q.loc['Proceeds from Stock Plans', '2020-07-26'] = df_cash_flow_10q.loc['nvda_NetProceedsPaymentsRelatedToEmployeeStockPlans', '2020-07-26']
df_cash_flow_10q.loc['Proceeds from Stock Plans', '2020-04-26'] = df_cash_flow_10q.loc['nvda_NetProceedsPaymentsRelatedToEmployeeStockPlans', '2020-04-26']

# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_cash_flow_10q = df_cash_flow_10q.drop([
    'Depreciation, Depletion and Amortization, Nonproduction',
    'Share-based Payment Arrangement, Expense',
    'nvda_Netproceedspaymentsrelatedtoemployeestockplans',
    'nvda_PaymentsForFinancedPropertyPlantAndEquipmentFinancingActivities',
    'nvda_PurchasesOfPropertyAndEquipmentAndIntangibleAssets',
    'Payments to Acquire Other Investments',
    'Payments to Acquire Property, Plant, and Equipment',
    'nvda_NetProceedsPaymentsRelatedToEmployeeStockPlans'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_cash_flow_10q = df_cash_flow_10q.drop([
    'us-gaap_NetCashProvidedByUsedInOperatingActivitiesAbstract',
    'Proceeds from Sale and Maturity of Marketable Securities',
    'Cash and Cash Equivalents, at Carrying Value',
    'Restricted Cash, Current'
])

# Renomeando algumas linhas do índice
df_cash_flow_10q = df_cash_flow_10q.rename(index={
    'nvda_BusinessCombinationAdvancedConsiderationWrittenOff': 'Business Combination Advanced Consideration Written Off',
    'nvda_PaymentsForFinancedPropertyPlantAndEquipmentAndIntangibleAssetsFinancingActivities': 'Payments For Financed Property Plant And Equipment And Intangible Assets Financing Activities',
    'PaymentsToAcquireEquitySecuritiesFvNi': 'Payments To Acquire Equity Securities FvNi',
    'ProceedsFromSaleOfEquitySecuritiesFvNi': 'Proceeds From Sale Of Equity Securities FvNi'
})

df_cash_flow_10q

In [ ]:
# Arrumando alguns index dos dfs para ficar padronizado com os index utilizados nas funções p/ calcular os indicadores fundamentalistas
df_balance_sheet_10q = df_balance_sheet_10q.rename(index={
    'Cash and Cash Equivalents, at Carrying Value': 'Cash and Cash Equivalents',
    'Marketable Securities, Current': 'Marketable Securities',
    'Debt, Current': 'Short-term Debt',
    'Long-term Debt, Excluding Current Maturities': 'Long-term Debt',
    'Operating Lease, Liability, Noncurrent': 'Long-term Operating Lease Liabilities',
    "Stockholders' Equity Attributable to Parent": 'Total Shareholders Equity'
})

df_income_statement_10q = df_income_statement_10q.rename(index={
    'Research and Development Expense': 'Research and Development',
    'Operating Income (Loss)': 'Operating Income',
    'Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest': 'Income Before Provision for Income Taxes',
    'Income Tax Expense (Benefit)': 'Provision for Income Taxes',
    'Net Income (Loss) Attributable to Parent': 'Net Income',
    'Weighted Average Number of Shares Outstanding, Basic': 'Number of Shares - Basic',
    'Weighted Average Number of Shares Outstanding, Diluted': 'Number of Shares - Diluted'
})

df_cash_flow_10q = df_cash_flow_10q.rename(index={
    'Payments to Acquire Productive Assets': 'Purchases Of Property And Equipment And Intangible Assets',
})

In [ ]:
# 'acc_num' entre '2021-10-31' e '2025-07-27'
df_lst_acc_num_10q_nvda_sorted = df_lst_acc_num_10q_nvda_sliced['acc_num'].sort_index()
df_lst_acc_num_10q_nvda_sliced_2 = df_lst_acc_num_10q_nvda_sorted.loc['2021-10-31':'2025-10-26']

# Fazendo o scraping do leases (10-Q)
leases_lst = []

for acc_num in df_lst_acc_num_10q_nvda_sliced_2:

    # Fazendo o scraping do leases (10-Q) -> 'Note 17 – Leases' -> 'short-term operating lease liabilities'
    leases_10q = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='leases')
    leases_10q = f_sec.rename_statement(leases_10q, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    leases_10q = leases_10q.iloc[:, 0]
    leases_lst.append(leases_10q)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_leases_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in leases_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_leases_10q = pd.concat([df_leases_10q, df], axis=1)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_leases_10q = df_leases_10q.drop('us-gaap_LeasesAbstract')

# Renomeando a linha do índice
df_leases_10q = df_leases_10q.rename(index={
    'Operating Lease, Liability, Current': 'Short-term Operating Lease Liabilities',

})

# Selecionando a linha desejada
df_leases_10q = df_leases_10q.loc[['Short-term Operating Lease Liabilities']]

# Criando as colunas dos trimestre de '2020-04-26', '2020-07-26', '2020-10-25', '2021-05-02' e '2021-08-01'
df_leases_10q.loc['Short-term Operating Lease Liabilities', '2020-04-26'] = 100
df_leases_10q.loc['Short-term Operating Lease Liabilities', '2020-07-26'] = 124
df_leases_10q.loc['Short-term Operating Lease Liabilities', '2020-10-25'] = 116
df_leases_10q.loc['Short-term Operating Lease Liabilities', '2021-05-02'] = 135
df_leases_10q.loc['Short-term Operating Lease Liabilities', '2021-08-01'] = 132

# Ordenando o df
df_leases_10q = df_leases_10q[sorted(df_leases_10q.columns, reverse=True)]

df_leases_10q 

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//semicondutores//nvidia//nvda_10q.xlsx') as writer:
    df_balance_sheet_10q.to_excel(writer, sheet_name='balance_sheet')
    df_income_statement_10q.to_excel(writer, sheet_name='income_statement')
    df_cash_flow_10q.to_excel(writer, sheet_name='cash_flow')
    df_leases_10q.to_excel(writer, sheet_name='leases')

### Indicadores Fundamentalistas

#### Indicadores 10-K

In [ ]:
# Lendo os arquivos 10k
nvda_bs = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//nvidia//nvda_10k.xlsx', 
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

nvda_is = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//nvidia//nvda_10k.xlsx', 
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

nvda_cf = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//nvidia//nvda_10k.xlsx', 
    sheet_name='cash_flow', 
    index_col='Unnamed: 0'
)

nvda_leases = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//nvidia//nvda_10k.xlsx', 
    sheet_name='leases', 
    index_col='Unnamed: 0'
)

# Transformando as colunas em datetime
nvda_bs.columns = pd.to_datetime(nvda_bs.columns)
nvda_is.columns = pd.to_datetime(nvda_is.columns)
nvda_cf.columns = pd.to_datetime(nvda_cf.columns)
nvda_leases.columns = pd.to_datetime(nvda_leases.columns)

# Transformando as colunas em apenas o ano
nvda_bs.columns = nvda_bs.columns.year
nvda_is.columns = nvda_is.columns.year
nvda_cf.columns = nvda_cf.columns.year
nvda_leases.columns = nvda_leases.columns.year

In [ ]:
# Preços da NVDA
nvda_close = yf.download('NVDA', auto_adjust=True, multi_level_index=False, start='2017-01-01')['Close']

# Selecionando o último preço do mês de Janeiro de cada ano 
lst_years = [
    '2017', '2018', '2019', '2020', 
    '2021', '2022', '2023', '2024',
    '2025'
]
last_jan_close = []

for year in lst_years:
    last_close = nvda_close[f'{year}-01'].iloc[-1]
    last_jan_close.append(last_close)

# Transformando em uma série pandas
nvda_close_jan = pd.Series(last_jan_close, index=lst_years)

# Revertendo o index da série p/ ficar igual com os dos indicadores
nvda_close_jan = nvda_close_jan.reindex(index=nvda_close_jan.index[::-1])

# Arredondamento de duas casas decimais
nvda_close_jan = round(nvda_close_jan, 2)

# Teve um split de 10:1 em 10/06/2024 (https://br.investing.com/equities/nvidia-corp-historical-data-splits)
nvda_close_jan.loc['2024'] = nvda_close_jan.loc['2024'] * 10
nvda_close_jan.loc['2023'] = nvda_close_jan.loc['2023'] * 10
nvda_close_jan.loc['2022'] = nvda_close_jan.loc['2022'] * 10
# Teve um split de 4:1 em 20/07/2021 (https://br.investing.com/equities/nvidia-corp-historical-data-splits) 
# Tenho que multiplcar os dois splits (4*10=40)
nvda_close_jan.loc['2021'] = nvda_close_jan.loc['2021'] * 40
nvda_close_jan.loc['2020'] = nvda_close_jan.loc['2020'] * 40
nvda_close_jan.loc['2019'] = nvda_close_jan.loc['2019'] * 40
nvda_close_jan.loc['2018'] = nvda_close_jan.loc['2018'] * 40
nvda_close_jan.loc['2017'] = nvda_close_jan.loc['2017'] * 40

# Transformando o index em int
nvda_close_jan .index = nvda_close_jan .index.astype(int)

nvda_close_jan 

In [ ]:
# Criando uma lista só para fazer a indexação em que o ano mais recente está no começo
idx = nvda_bs.columns

# Valor de Mercado
vm = f_eua.indicador_vm(ten_k=True, price=nvda_close_jan, df_is=nvda_is) 

# LPA
lpa = f_eua.indicador_lpa(ten_k=True, df_is=nvda_is, idx=None)

# P/L Damodaran
p_lpa_damodaran = f_eua.indicador_p_lpa_2(ten_k=True, vm=vm, df_is=nvda_is)

# P/L
p_lpa = f_eua.indicador_p_lpa(ten_k=True, price=nvda_close_jan, lpa=lpa, idx=idx)

# L/P
lpa_p = f_eua.indicador_lpa_p(ten_k=True, price=nvda_close_jan, lpa=lpa, idx=idx) 

# Depreciação
depreciacao = f_eua.indicador_depreciacao(ten_k=True, df_cf=nvda_cf, first_quarter=None, idx=idx)

# EBITDA
ebitda = f_eua.indicador_ebitda(ten_k=True, df_is=nvda_is, idx=None, depreciacao=depreciacao)

# Margem Líquida
m_liq = f_eua.indicador_ml(ten_k=True, df_is=nvda_is)

# VPA
vpa = f_eua.indicador_vpa(ten_k=True, df_bs=nvda_bs, df_is=nvda_is)

# P/VPA
p_vpa = f_eua.indicador_p_vpa(ten_k=True, price=nvda_close_jan, vpa=vpa, idx=idx)

# Caixa e Equivalentes
lst_caixa = ['Cash and Cash Equivalents', 'Marketable Securities']
caixa = f_eua.indicador_caixa(ten_k=True, df_bs=nvda_bs, lst_itens_caixa=lst_caixa)

# Dívida Bruta
lst_divida = ['Short-term Debt', 'Long-term Debt', 'Long-term Operating Lease Liabilities']
div_bruta = f_eua.indicador_divida_bruta(ten_k=True, df_bs=nvda_bs, lst_itens_divida=lst_divida)
div_bruta_total = div_bruta.values + nvda_leases.values
div_bruta_total = pd.Series(div_bruta_total[0], index=idx)

# Dívida Líquida
div_liq = f_eua.indicador_divida_liquida(ten_k=True, caixa=caixa, div_bruta=div_bruta_total, idx=idx)

# Dívida Líquida / EBITDA
dl_ebitda =  f_eua.indicador_dl_ebitda(ten_k=True, ebitda=ebitda, div_liq=div_liq, idx=None)

# Dívida Líquida / PL
dl_pl = f_eua.indicador_dl_pl(ten_k=True, div_liq=div_liq, df_bs=nvda_bs) 

# EV/EBITDA
ev_ebitda = f_eua.indicador_ev_ebitda(vm=vm, div_liq=div_liq, ebitda=ebitda)

# ROE
roe = f_eua.indicador_roe(ten_k=True, df_is=nvda_is, df_bs=nvda_bs, idx=None)

# ROIC
roic = f_eua.indicador_roic(ten_k=True, df_is=nvda_is, df_bs=nvda_bs, div_bruta=div_bruta_total, idx=idx)

# FCO
fco = f_eua.indicador_fco(ten_k=True, df_cf=nvda_cf, first_quarter=None) 

# FCI
fci = f_eua.indicador_fci(ten_k=True, df_cf=nvda_cf, first_quarter=None) 

# FCF
fcf = f_eua.indicador_fcf(ten_k=True, df_cf=nvda_cf, first_quarter=None) 

# CAPEX
lst_capex = ['Purchases Of Property And Equipment And Intangible Assets']

capex= f_eua.indicador_capex(ten_k=True, df_cf=nvda_cf, first_quarter=None, lst_itens=lst_capex, idx=idx)

# Free Cash Flow
free_cash_flow = f_eua.indicador_free_cash_flow(fco=fco, capex=capex)

# Net CAPEX
net_capex = f_eua.indicador_net_capex(capex=capex, depreciation=depreciacao)

# R&D
rd_expense = f_eua.indicador_rd(ten_k=True, df_is=nvda_is, idx=idx)

# Net CAPEX ajustado
adj_net_capex = f_eua.indicador_adj_net_capex(net_capex=net_capex, rd_expense=rd_expense)

# Working capital
working_capital = f_eua.indicador_working_capital(ten_k=True, df_bs=nvda_bs)

# Change in non cash working capital
lst_itens_change_non_cash_wc = [
    'Deferred Income Tax Expense (Benefit)',
    'Increase (Decrease) in Accounts Receivable',
    'Increase (Decrease) in Inventories',
    'Increase (Decrease) in Prepaid Expense and Other Assets',
    'Increase (Decrease) in Accounts Payable',
    'Increase (Decrease) in Accrued Liabilities and Other Operating Liabilities',
    'Increase (Decrease) in Other Noncurrent Liabilities'
]

change_non_cash_wc = f_eua.indicador_change_non_cash_wc(ten_k=True, df_cf=nvda_cf, lst_itens=lst_itens_change_non_cash_wc, first_quarter=None)

# Reinvestment rate
reinvestment_rate = f_eua.indicador_reinvestment_rate(ten_k=True, net_capex=net_capex, change_non_cash_wc=change_non_cash_wc, df_is=nvda_is, idx=idx)

# New borrowing
lst_itens_new_borrowing = ['Proceeds from Debt, Net of Issuance Costs']

new_borrowing = f_eua.indicador_new_borrowing(ten_k=True, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=nvda_cf, first_quarter=None, idx=idx)

# Debt paid
lst_itens_debt_paid = [
    'Repayments of Debt',
    'Payments of Debt Issuance Costs'
]

debt_paid = f_eua.indicador_debt_paid(ten_k=True, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=nvda_cf, first_quarter=None, idx=idx)

# Taxes on operating income
taxes_operating_income = f_eua.indicador_taxes_operating_income(ten_k=True, df_is=nvda_is, idx=idx)

# FCFE
fcfe_10k = f_eua.indicador_fcfe(
    ten_k=True, 
    depreciacao=depreciacao, 
    capex=capex, 
    change_non_cash_wc=change_non_cash_wc, 
    new_borrowing=new_borrowing,
    debt_paid=debt_paid,
    df_is=nvda_is,
    idx=idx
)

# FCFF
fcff_10k = f_eua.indicador_fcff(
        ten_k=True, 
        taxes_operating_income=taxes_operating_income,
        depreciacao=depreciacao, 
        capex=capex, 
        change_non_cash_wc=change_non_cash_wc, 
        df_is=nvda_is,
        idx=idx
)

# Dividendo por Ação
dpa = f_eua.indicador_dpa(ten_k=True, df_cf=nvda_cf, df_is=nvda_is, div_nao_acum=None)

# Payout
payout = f_eua.indicador_payout(ten_k=True, df_cf=nvda_cf, df_is=nvda_is, div_nao_acum=None)

# Buyback
buyback = f_eua.indicador_buyback(ten_k=True, df_cf=nvda_cf, first_quarter=None)

# Colocando todos os indicadores em uma lista
lst_indicadores = [
    p_lpa_damodaran,
    p_lpa,
    lpa_p,  
    ebitda, 
    m_liq,
    p_vpa,
    vm,
    div_bruta_total,
    caixa,
    div_liq,
    dl_ebitda,
    dl_pl,
    ev_ebitda,
    roe,
    roic,
    fco,
    fci,
    fcf,
    free_cash_flow,
    capex,
    net_capex,
    rd_expense,
    adj_net_capex,
    working_capital,
    reinvestment_rate,
    fcfe_10k,
    fcff_10k,
    dpa,
    payout,
    buyback
]

# Juntando todos os indicadores em um df
nvda_indicadores_10k = pd.concat(lst_indicadores, axis=1)

# Nomes das colunas
nomes_col = [
    'P/L Damodaran', 
    'P/L', 
    'L/P', 
    'EBITDA', 
    'Margem Líquida',
    'P/VPA',
    'Valor de Mercado',
    'Dívida Bruta',
    'Caixa e Equivalentes',
    'Dívida Líquida',
    'Dívida Líquida/EBITDA',
    'Dívida Líquida/PL',
    'EV/EBITDA',
    'ROE',
    'ROIC',
    'FCO',
    'FCI',
    'FCF',
    'Free Cash Flow',
    'Capex',
    'Net Capex',
    'R&D',
    'Adj Net Capex',
    'Working Capital',
    'Reinvestment Rate',
    'FCFE',
    'FCFF',
    'DPA',
    'Payout',
    'Buyback'
]

# Atribuindo os novos nomes das colunas do df
nvda_indicadores_10k.columns = nomes_col
nvda_indicadores_10k

#### Indicadores 10-Q

In [ ]:
# Lendo os arquivos 10q
nvda_bs_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//nvidia//nvda_10q.xlsx',
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

nvda_is_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//nvidia//nvda_10q.xlsx',
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

nvda_cf_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//nvidia//nvda_10q.xlsx',
    sheet_name='cash_flow',  
    index_col='Unnamed: 0'
)

nvda_leases_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//nvidia//nvda_10q.xlsx',
    sheet_name='leases',  
    index_col='Unnamed: 0'
)

# Aplicando o ajuste nas colunas
nvda_bs_10q.columns = [f_sec.ajustar_mes_04_07_10(col) for col in nvda_bs_10q.columns]
nvda_is_10q.columns = [f_sec.ajustar_mes_04_07_10(col) for col in nvda_is_10q.columns]
nvda_cf_10q.columns = [f_sec.ajustar_mes_04_07_10(col) for col in nvda_cf_10q.columns]
nvda_leases_10q.columns = [f_sec.ajustar_mes_04_07_10(col) for col in nvda_leases_10q.columns]

In [ ]:
# Preços da NVDA
nvda_close = yf.download('NVDA', auto_adjust=True, multi_level_index=False, start='2021-01-01')['Close']

# Selecionando o último preço dos meses de Abril, Julho e Outubro
lst_years = [
    '2021', '2022', '2023', '2024',
    '2025',
]
lst_months = ['04', '07', '10']
last_close_months = []

for year in lst_years:
    for month in lst_months:
        last_close = nvda_close[f'{year}-{month}'].dropna().iloc[-1]  # Dropna para lidar com meses sem dados
        last_close_months.append(last_close)

# Criando uma lista em que junte os meses e os anos para que seja o index da serie pandas
lst_idx = []

for year in lst_years:
    for month in lst_months:
        lst_idx.append(f"{month}-{year}")

# Transformando em uma série pandas
nvda_close_months = pd.Series(last_close_months, index=lst_idx)

# Revertendo o index da série p/ ficar igual com os dos indicadores
nvda_close_months = nvda_close_months.reindex(index=nvda_close_months.index[::-1])

# No ano mais atual, como ainda ele não acabou, não consigo fazer o loop com todos meses (Fevereiro, Maio e Agosto)
actual_year_int = datetime.now().year  
actual_year_str = str(actual_year_int) 

lst_actual_months = [
    # '04', 
    # '07', 
    # '10'
] 

last_close_actual_months = []

for month in lst_actual_months:
    last_close = nvda_close[f'{actual_year_str}-{month}'].dropna().iloc[-1]  # Dropna para lidar com meses sem dados
    last_close_actual_months.append(last_close)

# Criando uma lista em que junte os meses e os anos para que seja o index da serie pandas
lst_idx_actual = []

for month in lst_actual_months:
    lst_idx_actual.append(f"{month}-{actual_year_str}")

# Transformando em uma série pandas
nvda_close_months_atual = pd.Series(last_close_actual_months, index=lst_idx_actual)

# Revertendo o index da série p/ ficar igual com os dos indicadores
nvda_close_months_atual = nvda_close_months_atual.reindex(nvda_close_months_atual.index[::-1])

# Juntando as duas series pandas
nvda_close_months = pd.concat([nvda_close_months_atual, nvda_close_months])

# Arredondamento de duas casas decimais
nvda_close_months = round(nvda_close_months, 2)

# Teve um split de 10:1 em 10/06/2024 (https://br.investing.com/equities/nvidia-corp-historical-data-splits)
nvda_close_months.loc['04-2024'] = nvda_close_months.loc['04-2024'] * 10
nvda_close_months.loc['10-2023'] = nvda_close_months.loc['10-2023'] * 10
nvda_close_months.loc['07-2023'] = nvda_close_months.loc['07-2023'] * 10
nvda_close_months.loc['04-2023'] = nvda_close_months.loc['04-2023'] * 10
nvda_close_months.loc['10-2022'] = nvda_close_months.loc['10-2022'] * 10
nvda_close_months.loc['07-2022'] = nvda_close_months.loc['07-2022'] * 10
nvda_close_months.loc['04-2022'] = nvda_close_months.loc['04-2022'] * 10
nvda_close_months.loc['10-2021'] = nvda_close_months.loc['10-2021'] * 10
nvda_close_months.loc['07-2021'] = nvda_close_months.loc['07-2021'] * 10
# Teve um split de 4:1 em 20/07/2021 (https://br.investing.com/equities/nvidia-corp-historical-data-splits)
# Tenho que multiplcar os dois splits (4*10=40)
nvda_close_months.loc['04-2021'] = nvda_close_months.loc['04-2021'] * 40

# Substituindo o index para ficar igual ao dos dados fundamentalistas
nvda_close_months.index = nvda_bs_10q.columns[:-3]

nvda_close_months

In [ ]:
# Criando uma lista só para fazer a indexação em que o ano mais recente está no começo
idx_10q = nvda_bs_10q.columns

# Valor de Mercado
vm_10q = f_eua.indicador_vm(ten_k=False, price=nvda_close_months, df_is=nvda_is_10q)

# LPA
lpa_acum = f_eua.indicador_lpa(ten_k=False, df_is=nvda_is_10q, idx=idx_10q)[1]
lpa_10q = f_eua.indicador_lpa(ten_k=False, df_is=nvda_is_10q, idx=idx_10q)[0]

# P/L Damodaran
p_lpa_10q_damodaran = f_eua.indicador_p_lpa_2(ten_k=False, vm=vm_10q, df_is=nvda_is_10q)

# P/L
p_lpa_10q = f_eua.indicador_p_lpa(ten_k=False, price=nvda_close_months, lpa=lpa_acum, idx=idx_10q)

# L/P
lpa_p_10q = f_eua.indicador_lpa_p(ten_k=False, price=nvda_close_months, lpa=lpa_acum, idx=idx_10q) 

# Depreciação
depreciacao_10q = f_eua.indicador_depreciacao(ten_k=False, df_cf=nvda_cf_10q, first_quarter='04', idx=idx_10q)[0]
depreciacao_10q_sliced = f_eua.indicador_depreciacao(ten_k=False, df_cf=nvda_cf_10q, first_quarter='04', idx=idx_10q)[1]
depreciacao_10q_acum = f_eua.indicador_depreciacao(ten_k=False, df_cf=nvda_cf_10q, first_quarter='04', idx=idx_10q)[2]

# EBITDA
ebitda_10q = f_eua.indicador_ebitda(ten_k=False, df_is=nvda_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[0]
ebitda_10q_sliced = f_eua.indicador_ebitda(ten_k=False, df_is=nvda_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[1]
ebitda_10q_acum = f_eua.indicador_ebitda(ten_k=False, df_is=nvda_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[2]

# Margem Líquida
m_liq_10q = f_eua.indicador_ml(ten_k=False, df_is=nvda_is_10q)

# VPA
vpa_10q = f_eua.indicador_vpa(ten_k=False, df_bs=nvda_bs_10q, df_is=nvda_is_10q)

# P/VPA
p_vpa_10q = f_eua.indicador_p_vpa(ten_k=False, price=nvda_close_months, vpa=vpa_10q, idx=idx_10q)

# Caixa e Equivalentes
lst_caixa = ['Cash and Cash Equivalents', 'Marketable Securities']
caixa_10q = f_eua.indicador_caixa(ten_k=False, df_bs=nvda_bs_10q, lst_itens_caixa=lst_caixa)

# Dívida Bruta
lst_divida = ['Short-term Debt', 'Long-term Debt', 'Long-term Operating Lease Liabilities']
div_bruta_10q = f_eua.indicador_divida_bruta(ten_k=False, df_bs=nvda_bs_10q, lst_itens_divida=lst_divida)
div_bruta_total_10q = div_bruta_10q.values + nvda_leases_10q.loc['Short-term Operating Lease Liabilities'].values[0:-3]
div_bruta_total_10q = pd.Series(div_bruta_total_10q, index=idx_10q[0:-3])

# Dívida Líquida
div_liq_10q = f_eua.indicador_divida_liquida(ten_k=False, caixa=caixa_10q, div_bruta=div_bruta_total_10q, idx=idx_10q)

# Dívida Líquida / EBITDA
dl_ebitda_10q =  f_eua.indicador_dl_ebitda(ten_k=False, ebitda=ebitda_10q_acum, div_liq=div_liq_10q, idx=idx_10q)

# Dívida Líquida / PL
dl_pl_10q = f_eua.indicador_dl_pl(ten_k=False, div_liq=div_liq_10q, df_bs=nvda_bs_10q) 

# EV/EBITDA
ev_ebitda_10q = f_eua.indicador_ev_ebitda(vm=vm_10q, div_liq=div_liq_10q, ebitda=ebitda_10q_acum)

# ROE
roe_10q = f_eua.indicador_roe(ten_k=False, df_is=nvda_is_10q, df_bs=nvda_bs_10q, idx=idx_10q)

# ROIC 
roic_10q = f_eua.indicador_roic(ten_k=False, df_is=nvda_is_10q, df_bs=nvda_bs_10q, div_bruta=div_bruta_total_10q, idx=idx_10q)

# FCO
fco_10q = f_eua.indicador_fco(ten_k=False, df_cf=nvda_cf_10q, first_quarter='04') 

# FCI
fci_10q = f_eua.indicador_fci(ten_k=False, df_cf=nvda_cf_10q, first_quarter='04') 

# FCF
fcf_10q = f_eua.indicador_fcf(ten_k=False, df_cf=nvda_cf_10q, first_quarter='04') 

# CAPEX
lst_capex_10q = ['Purchases Of Property And Equipment And Intangible Assets']

capex_10q = f_eua.indicador_capex(ten_k=False, df_cf=nvda_cf_10q, first_quarter='04', lst_itens=lst_capex_10q, idx=idx_10q)[0]
capex_10q_sliced = f_eua.indicador_capex(ten_k=False, df_cf=nvda_cf_10q, first_quarter='04', lst_itens=lst_capex_10q, idx=idx_10q)[1]
capex_10q_acum = f_eua.indicador_capex(ten_k=False, df_cf=nvda_cf_10q, first_quarter='04', lst_itens=lst_capex_10q, idx=idx_10q)[2]

# Free Cash Flow
free_cash_flow_10q = f_eua.indicador_free_cash_flow(fco=fco_10q, capex=capex_10q_sliced)

# Net CAPEX
net_capex_10q = f_eua.indicador_net_capex(capex=capex_10q_acum, depreciation=depreciacao_10q_acum)

# R&D
rd_expense_10q = f_eua.indicador_rd(ten_k=False, df_is=nvda_is_10q, idx=idx_10q)[0]
rd_expense_10q_sliced = f_eua.indicador_rd(ten_k=False, df_is=nvda_is_10q, idx=idx_10q)[1]
rd_expense_10q_acum = f_eua.indicador_rd(ten_k=False, df_is=nvda_is_10q, idx=idx_10q)[2]

# Net CAPEX ajustado
adj_net_capex_10q = f_eua.indicador_adj_net_capex(net_capex=net_capex_10q, rd_expense=rd_expense_10q_acum)

# Working capital
working_capital_10q = f_eua.indicador_working_capital(ten_k=False, df_bs=nvda_bs_10q)

# Change in non cash working capital
lst_itens_change_non_cash_wc = [
    'Deferred Income Tax Expense (Benefit)',
    'Increase (Decrease) in Accounts Receivable',
    'Increase (Decrease) in Inventories',
    'Increase (Decrease) in Prepaid Expense and Other Assets',
    'Increase (Decrease) in Accounts Payable',
    'Increase (Decrease) in Accrued Liabilities and Other Operating Liabilities',
    'Increase (Decrease) in Other Noncurrent Liabilities'
]

change_non_cash_wc_10q = f_eua.indicador_change_non_cash_wc(ten_k=False, df_cf=nvda_cf_10q, lst_itens=lst_itens_change_non_cash_wc, first_quarter='04')[0]
change_non_cash_wc_10q_sliced = f_eua.indicador_change_non_cash_wc(ten_k=False, df_cf=nvda_cf_10q, lst_itens=lst_itens_change_non_cash_wc, first_quarter='04')[1]

# Reinvestment rate
reinvestment_rate_10q = f_eua.indicador_reinvestment_rate(ten_k=False, net_capex=adj_net_capex_10q, change_non_cash_wc=change_non_cash_wc_10q, df_is=nvda_is_10q, idx=idx_10q)

# New borrowing
lst_itens_new_borrowing = ['Proceeds from Debt, Net of Issuance Costs']

new_borrowing_10q = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=nvda_cf_10q, first_quarter='04', idx=idx_10q)[0]
new_borrowing_10q_sliced = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=nvda_cf_10q, first_quarter='04', idx=idx_10q)[1]
new_borrowing_10q_acum = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=nvda_cf_10q, first_quarter='04', idx=idx_10q)[2]

# Debt paid
lst_itens_debt_paid = ['Repayments of Debt']

debt_paid_10q = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=nvda_cf_10q, first_quarter='04', idx=idx_10q)[0]
debt_paid_10q_sliced = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=nvda_cf_10q, first_quarter='04', idx=idx_10q)[1]
debt_paid_10q_acum = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=nvda_cf_10q, first_quarter='04', idx=idx_10q)[2]

# Taxes on operating income
taxes_operating_income_10q = f_eua.indicador_taxes_operating_income(ten_k=False, df_is=nvda_is_10q, idx=idx_10q)

# FCFE
fcfe_10q = f_eua.indicador_fcfe(
    ten_k=False, 
    depreciacao=depreciacao_10q_acum, 
    capex=capex_10q_acum, 
    change_non_cash_wc=change_non_cash_wc_10q_sliced, 
    new_borrowing=new_borrowing_10q_acum, 
    debt_paid=debt_paid_10q_acum, 
    df_is=nvda_is_10q,
    idx=idx_10q
)

# FCFF
fcff_10q = f_eua.indicador_fcff(
        ten_k=False, 
        taxes_operating_income=taxes_operating_income_10q,
        depreciacao=depreciacao_10q_acum, 
        capex=capex_10q_acum, 
        change_non_cash_wc=change_non_cash_wc_10q_sliced, 
        df_is=nvda_is_10q,
        idx=idx_10q
)

# Dividendo não acumulado
div_nao_acum = f_eua.indicador_div_nao_acum(df_cf=nvda_cf_10q, first_quarter='04')

# Dividendo por Ação
dpa_10q = f_eua.indicador_dpa(ten_k=False, df_cf=nvda_cf_10q, df_is=nvda_is_10q, div_nao_acum=div_nao_acum)

# Payout
payout_10q = f_eua.indicador_payout(ten_k=False, df_cf=nvda_cf_10q, df_is=nvda_is_10q, div_nao_acum=div_nao_acum)

# Buyback
buyback_10q = f_eua.indicador_buyback(ten_k=False, df_cf=nvda_cf_10q,  first_quarter='04')

# Lista com todos os indicadores
lst_indicadores_10q = [
    p_lpa_10q_damodaran,
    p_lpa_10q,
    lpa_p_10q,  
    ebitda_10q_sliced, 
    m_liq_10q,
    p_vpa_10q,
    vm_10q,
    div_bruta_total_10q,
    caixa_10q,
    div_liq_10q,
    dl_ebitda_10q,
    dl_pl_10q,
    ev_ebitda_10q,
    roe_10q,
    roic_10q,
    fco_10q,
    fci_10q,
    fcf_10q,
    free_cash_flow_10q,
    capex_10q_sliced,
    net_capex_10q,
    rd_expense_10q_sliced,
    adj_net_capex_10q,
    working_capital_10q,
    reinvestment_rate_10q,
    fcfe_10q,
    fcff_10q,
    dpa_10q,
    payout_10q,
    buyback_10q
]

# Juntando todos os indicadores em um df
nvda_indicadores_10q = pd.concat(lst_indicadores_10q, axis=1)

# Nomes das colunas
nomes_col = [
    'P/L Damodaran', 
    'P/L', 
    'L/P', 
    'EBITDA', 
    'Margem Líquida',
    'P/VPA',
    'Valor de Mercado',
    'Dívida Bruta',
    'Caixa e Equivalentes',
    'Dívida Líquida',
    'Dívida Líquida/EBITDA',
    'Dívida Líquida/PL',
    'EV/EBITDA',
    'ROE',
    'ROIC',
    'FCO',
    'FCI',
    'FCF',
    'Free Cash Flow',
    'Capex',
    'Net Capex',
    'R&D',
    'Adj Net Capex',
    'Working Capital',
    'Reinvestment Rate',
    'FCFE',
    'FCFF',
    'DPA',
    'Payout',
    'Buyback'
]

# Atribuindo os novos nomes das colunas do df
nvda_indicadores_10q.columns = nomes_col

# Preenchendo os NaN por zero
nvda_indicadores_10q = nvda_indicadores_10q.fillna(0)

nvda_indicadores_10q

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//semicondutores//nvidia//nvda_indicators.xlsx') as writer:
    nvda_indicadores_10k.to_excel(writer, sheet_name='indicators_10k')
    nvda_indicadores_10q.to_excel(writer, sheet_name='indicators_10q')

## AMD

* 1Q -> Lançamento em Março (03);
* 2Q -> Lançamento em Junho (06);
* 3Q -> Lançamento em Setembro (09);
* 10K -> Lançamento em Dezembro (12).

In [ ]:
ticker = 'amd'
f_sec.cik_matching_ticker(ticker)

### Scraping AMD 10-K

In [ ]:
# Número dos documentos que contém a DRE, BP e FC 
acc_documents = f_sec.get_filtered_filings(ticker=ticker, ten_k=True, just_accession_numbers=True)

# Trocando os '-' para '' das strings ('0001045810-24-000264' -> '000104581024000264')
dict_acc_num = {k: v.replace('-', '') for k, v in acc_documents.items()}

# Transformando o dicionário em um df
df_lst_acc_num_amd = pd.DataFrame.from_dict(dict_acc_num, orient='index', columns=['acc_num'])

# Transformando em datetime o index
df_lst_acc_num_amd.index = pd.to_datetime(df_lst_acc_num_amd.index, format='%Y-%m-%d')
df_lst_acc_num_amd

In [ ]:
# Selecionando os 'acc_num' dos 10-Q a partir de 2017 p/ iteração
df_lst_acc_num_amd_sliced = df_lst_acc_num_amd.loc['2017-12-30':]
df_lst_acc_num_amd_sliced

In [ ]:
# Rótulos
labels_dict = f_sec.get_label_dictionary(ticker, headers=f_sec.header)

In [ ]:
# Fazendo o scraping do balance sheet (10-K)
balance_sheet_lst = []

for acc_num in df_lst_acc_num_amd_sliced['acc_num']:

    # Balance sheet
    balance_sheet = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='balance_sheet')
    balance_sheet = f_sec.rename_statement(balance_sheet, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    balance_sheet = balance_sheet.iloc[:, 0]
    balance_sheet_lst.append(balance_sheet)

# Juntando todos os dfs que estão em listas em apenas um df
df_balance_sheet_10k = pd.concat(balance_sheet_lst, axis=1)

# Substituindo os NaN por 0
df_balance_sheet_10k = df_balance_sheet_10k.fillna(0)

# A linha 'Short-term Investments' nas colunas '2019-12-28' e '2018-12-29' estão vazias
# A informação está na linha 'Marketable Securities, Current'
df_balance_sheet_10k.loc['Short-term Investments', '2019-12-28'] = df_balance_sheet_10k.loc['Marketable Securities, Current', '2019-12-28'] 
df_balance_sheet_10k.loc['Short-term Investments', '2018-12-29'] = df_balance_sheet_10k.loc['Marketable Securities, Current', '2018-12-29'] 

# A linha 'Accounts Payable, Other' na coluna '2022-12-31', '2021-12-25', '2020-12-26', '2019-12-28', '2018-12-29' e '2017-12-30' estão zeradas
# A informação está na linha 'Accounts Payable, Related Parties, Current'
df_balance_sheet_10k.loc['Accounts Payable, Other', '2022-12-31'] = df_balance_sheet_10k.loc['Accounts Payable, Related Parties, Current', '2022-12-31'] 
df_balance_sheet_10k.loc['Accounts Payable, Other', '2021-12-25'] = df_balance_sheet_10k.loc['Accounts Payable, Related Parties, Current', '2021-12-25'] 
df_balance_sheet_10k.loc['Accounts Payable, Other', '2020-12-26'] = df_balance_sheet_10k.loc['Accounts Payable, Related Parties, Current', '2020-12-26'] 
df_balance_sheet_10k.loc['Accounts Payable, Other', '2019-12-28'] = df_balance_sheet_10k.loc['Accounts Payable, Related Parties, Current', '2019-12-28'] 
df_balance_sheet_10k.loc['Accounts Payable, Other', '2018-12-29'] = df_balance_sheet_10k.loc['Accounts Payable, Related Parties, Current', '2018-12-29'] 
df_balance_sheet_10k.loc['Accounts Payable, Other', '2017-12-30'] = df_balance_sheet_10k.loc['Accounts Payable, Related Parties, Current', '2017-12-30'] 

# A linha 'Treasury Stock, Common, Shares' nas colunas '2022-12-31', '2021-12-25' e '2020-12-26' estão zeradas
# As informações estão na linha 'Treasury Stock, Shares'
df_balance_sheet_10k.loc['Treasury Stock, Common, Shares', '2022-12-31'] = df_balance_sheet_10k.loc['Treasury Stock, Shares', '2022-12-31'] 
df_balance_sheet_10k.loc['Treasury Stock, Common, Shares', '2021-12-25'] = df_balance_sheet_10k.loc['Treasury Stock, Shares', '2021-12-25'] 
df_balance_sheet_10k.loc['Treasury Stock, Common, Shares', '2020-12-26'] = df_balance_sheet_10k.loc['Treasury Stock, Shares', '2020-12-26'] 

# A linha 'Other Receivables' nas colunas '2022-12-31', '2021-12-25', '2020-12-26', '2019-12-28', '2018-12-29' e '2017-12-30' estão zeradas
# As informações estão na linha 'Related Party Transaction, Due from (to) Related Party, Current' e 'amd_RelatedPartyTransactionPrepaidExpenseCurrent'
df_balance_sheet_10k.loc['Other Receivables', '2022-12-31'] = df_balance_sheet_10k.loc['Related Party Transaction, Due from (to) Related Party, Current', '2022-12-31'] 
df_balance_sheet_10k.loc['Other Receivables', '2021-12-25'] = df_balance_sheet_10k.loc['Related Party Transaction, Due from (to) Related Party, Current', '2021-12-25'] 
df_balance_sheet_10k.loc['Other Receivables', '2020-12-26'] = df_balance_sheet_10k.loc['Related Party Transaction, Due from (to) Related Party, Current', '2020-12-26'] 
df_balance_sheet_10k.loc['Other Receivables', '2019-12-28'] = df_balance_sheet_10k.loc['amd_RelatedPartyTransactionPrepaidExpenseCurrent', '2019-12-28'] 
df_balance_sheet_10k.loc['Other Receivables', '2018-12-29'] = df_balance_sheet_10k.loc['amd_RelatedPartyTransactionPrepaidExpenseCurrent', '2018-12-29'] 
df_balance_sheet_10k.loc['Other Receivables', '2017-12-30'] = df_balance_sheet_10k.loc['amd_RelatedPartyTransactionPrepaidExpenseCurrent', '2017-12-30'] 

# A linha 'Prepaid Expense and Other Assets, Current' nas colunas '2018-12-29' e '2017-12-30' estão zeradas
# As informações estão na linha 'Prepaid Expense, Current'
df_balance_sheet_10k.loc['Prepaid Expense and Other Assets, Current', '2018-12-29'] = df_balance_sheet_10k.loc['Prepaid Expense, Current', '2018-12-29'] 
df_balance_sheet_10k.loc['Prepaid Expense and Other Assets, Current', '2017-12-30'] = df_balance_sheet_10k.loc['Prepaid Expense, Current', '2017-12-30'] 

# A linha 'Short-term Debt' nas colunas '2018-12-29' e '2017-12-30' estão zeradas
# As informações estão na linha 'Line of Credit, Current'
df_balance_sheet_10k.loc['Short-term Debt', '2018-12-29'] = df_balance_sheet_10k.loc['Line of Credit, Current', '2018-12-29'] 
df_balance_sheet_10k.loc['Short-term Debt', '2017-12-30'] = df_balance_sheet_10k.loc['Line of Credit, Current', '2017-12-30'] 

# Removendo as linhas que movemos os seus itens para as linhas corretas
df_balance_sheet_10k = df_balance_sheet_10k.drop([
    'Marketable Securities, Current',
    'Accounts Payable, Related Parties, Current',
    'Treasury Stock, Shares',
    'Related Party Transaction, Due from (to) Related Party, Current',
    'amd_RelatedPartyTransactionPrepaidExpenseCurrent',
    'Prepaid Expense, Current',
    'Line of Credit, Current'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_balance_sheet_10k = df_balance_sheet_10k.drop([
    'us-gaap_AssetsCurrentAbstract',
    'Debt, Current',
    'Treasury Stock, Common, Shares',
    'Other Assets, Current',
    'Deferred Revenue, Current'
])

df_balance_sheet_10k

In [ ]:
# Os dados do income statement da AMD de 2017 não estão consolidados em uma coluna. Eles estão separados em quatro colunas '2017-12-30', '2017-09-30', '2017-07-01' e '2017-04-01'
income_statement_2017 = f_sec.process_one_statement(ticker='amd', accession_number='000000248818000042', statement_name='income_statement')
income_statement_2017 = f_sec.rename_statement(income_statement_2017, label_dictionary=labels_dict)

# Selecionando apenas as quatros primeiras colunas do df
income_statement_2017 = income_statement_2017.iloc[:, 0:4]

# Substituindo os NaN por 0
income_statement_2017 = income_statement_2017.fillna(0)

# Somando as quatro colunas para dar o resultado do income statement de 2017 
is_2017 = income_statement_2017.sum(axis=1)

# Transformando em um df
amd_is_2017 = pd.DataFrame({'2017-12-30':is_2017})

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
amd_is_2017 = amd_is_2017.drop([
    'us-gaap_IncomeStatementAbstract',
    'Amortization of Intangible Assets',
    'amd_RestructuringAndOtherSpecialChargesNet'

])

# Arrumando a escala do df
amd_is_2017.loc['Revenue, Net (Deprecated 2018-01-31)'] = amd_is_2017.loc['Revenue, Net (Deprecated 2018-01-31)'] / 1000000
amd_is_2017.loc['Cost of Goods Sold (Deprecated 2018-01-31)'] = amd_is_2017.loc['Cost of Goods Sold (Deprecated 2018-01-31)'] / 1000000
amd_is_2017.loc['Gross Profit'] = amd_is_2017.loc['Gross Profit'] / 1000000
amd_is_2017.loc['Research and Development Expense'] = amd_is_2017.loc['Research and Development Expense'] / 1000000
amd_is_2017.loc['Selling, General and Administrative Expense'] = amd_is_2017.loc['Selling, General and Administrative Expense'] / 1000000
amd_is_2017.loc['Other Operating Income'] = amd_is_2017.loc['Other Operating Income'] / 1000000
amd_is_2017.loc['Operating Income (Loss)'] = amd_is_2017.loc['Operating Income (Loss)'] / 1000000
amd_is_2017.loc['Interest Expense'] = amd_is_2017.loc['Interest Expense'] / 1000000
amd_is_2017.loc['Other Nonoperating Income (Expense)'] = amd_is_2017.loc['Other Nonoperating Income (Expense)'] / 1000000
amd_is_2017.loc['Income Tax Expense (Benefit)'] = amd_is_2017.loc['Income Tax Expense (Benefit)'] / 1000000
amd_is_2017.loc['Income (Loss) from Equity Method Investments'] = amd_is_2017.loc['Income (Loss) from Equity Method Investments'] / 1000000
amd_is_2017.loc['Net Income (Loss) Attributable to Parent'] = amd_is_2017.loc['Net Income (Loss) Attributable to Parent'] / 1000000

# Arrumando e adicionando alguns números no df
amd_is_2017.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2017-12-30'] = 69
amd_is_2017.loc['Earnings Per Share, Basic'] = 0.04
amd_is_2017.loc['Earnings Per Share, Diluted'] = 0.04
amd_is_2017.loc['Weighted Average Number of Shares Outstanding, Basic'] = 952
amd_is_2017.loc['Weighted Average Number of Shares Outstanding, Diluted'] = 1039

##############################################################################################

# Os dados do income statement da AMD de 2018 não estão consolidados em uma coluna. Eles estão separados em quatro colunas '2018-12-30', '2018-09-30', '2018-07-01' e '2018-04-01'
income_statement_2018 = f_sec.process_one_statement(ticker='amd', accession_number='000000248819000011', statement_name='income_statement')
income_statement_2018 = f_sec.rename_statement(income_statement_2018, label_dictionary=labels_dict)

# Selecionando apenas as quatros primeiras colunas do df
income_statement_2018 = income_statement_2018.iloc[:, 0:4]

# Substituindo os NaN por 0
income_statement_2018 = income_statement_2018.fillna(0)

# Somando as quatro colunas para dar o resultado do income statement de 2017
is_2018 = income_statement_2018.sum(axis=1)

# Transformando em um df
amd_is_2018 = pd.DataFrame({'2018-12-29':is_2018})

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
amd_is_2018 = amd_is_2018.drop([
    'us-gaap_IncomeStatementAbstract',
    'amd_RestructuringAndOtherSpecialChargesNet'
])

# Arrumando e adicionando alguns números no df
amd_is_2018.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2018-12-29'] = 330
amd_is_2018.loc['Weighted Average Number of Shares Outstanding, Basic', '2018-12-29'] = 982
amd_is_2018.loc['Weighted Average Number of Shares Outstanding, Diluted', '2018-12-29'] = 1064

# Renomeando o index do df do income statement de 2017
idx_renamed = {
    'Revenue, Net (Deprecated 2018-01-31)': 'Revenue from Contract with Customer, Excluding Assessed Tax',
    'Cost of Goods Sold (Deprecated 2018-01-31)': 'Cost of Goods and Services Sold', 
}
amd_is_2017 = amd_is_2017.rename(index=idx_renamed)

# Juntandos os dfs de 2017 e 2018 
df_is_1718 = pd.concat([amd_is_2018, amd_is_2017], axis=1)

# Adicionando as linhas ('Amortization Of Acquisition Related Intangibles' e 'Gain Loss On Licensing Agreement') p/ ficar do mesmo tamanho do 'df_income_statement_10k'
novas_linhas = pd.DataFrame({'2018-12-29': [0], '2017-12-30': [0]}, index=['Amortization Of Acquisition Related Intangibles', 'Gain Loss On Licensing Agreement'])
df_is_1718 = pd.concat([df_is_1718, novas_linhas])

# Arrumando a formatação da coluna '2017-12-30' (os nº estão com zeros a mais e fica mostrando em notação científica)
df_is_1718.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2017-12-30'] = df_is_1718.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2017-12-30'] 
df_is_1718.loc['Cost of Goods and Services Sold', '2017-12-30'] = df_is_1718.loc['Cost of Goods and Services Sold', '2017-12-30'] 
df_is_1718.loc['Gross Profit', '2017-12-30'] = df_is_1718.loc['Gross Profit', '2017-12-30'] 
df_is_1718.loc['Research and Development Expense', '2017-12-30'] = df_is_1718.loc['Research and Development Expense', '2017-12-30'] 
df_is_1718.loc['Selling, General and Administrative Expense', '2017-12-30'] = df_is_1718.loc['Selling, General and Administrative Expense', '2017-12-30'] 
df_is_1718.loc['Operating Income (Loss)', '2017-12-30'] = df_is_1718.loc['Operating Income (Loss)', '2017-12-30']
df_is_1718.loc['Interest Expense', '2017-12-30'] = df_is_1718.loc['Interest Expense', '2017-12-30']
df_is_1718.loc['Other Nonoperating Income (Expense)', '2017-12-30'] = df_is_1718.loc['Other Nonoperating Income (Expense)', '2017-12-30']
df_is_1718.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2017-12-30'] = df_is_1718.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2017-12-30']
df_is_1718.loc['Income Tax Expense (Benefit)', '2017-12-30'] = df_is_1718.loc['Income Tax Expense (Benefit)', '2017-12-30'] 
df_is_1718.loc['Income (Loss) from Equity Method Investments', '2017-12-30'] = df_is_1718.loc['Income (Loss) from Equity Method Investments', '2017-12-30'] 
df_is_1718.loc['Net Income (Loss) Attributable to Parent', '2017-12-30'] = df_is_1718.loc['Net Income (Loss) Attributable to Parent', '2017-12-30']
df_is_1718.loc['Other Operating Income', '2017-12-30'] = df_is_1718.loc['Other Operating Income', '2017-12-30']

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_is_1718 = df_is_1718.drop([
    'Amortization Of Acquisition Related Intangibles',
    'Gain Loss On Licensing Agreement',
    'Other Operating Income'
])

##############################################################################################

# Fazendo o scraping do income statement (10-K)
income_statement_lst = []

for acc_num in df_lst_acc_num_amd_sliced['acc_num']:

    # Income statement
    income_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='income_statement')
    income_statement = f_sec.rename_statement(income_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    income_statement = income_statement.iloc[:, 0]
    income_statement_lst.append(income_statement)

# Juntando todos os dfs que estão em listas em apenas um df
df_income_statement_10k = pd.concat(income_statement_lst, axis=1)

# Substituindo os NaN por 0
df_income_statement_10k = df_income_statement_10k.fillna(0)

# A linha 'amd_GainLossOnLicensingAgreement'  na coluna '2019-12-28' está zerada
# A informação está na linha 'amd_GainOnLicensingAgreement'
df_income_statement_10k.loc['amd_GainLossOnLicensingAgreement', '2019-12-28'] = df_income_statement_10k.loc['amd_GainOnLicensingAgreement', '2019-12-28'] 

# Removendo as linhas que movemos os seus itens para as linhas corretas
df_income_statement_10k = df_income_statement_10k.drop('amd_GainOnLicensingAgreement')

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_income_statement_10k = df_income_statement_10k.drop([
    'us-gaap_IncomeStatementAbstract',
    'amd_RestructuringAndOtherSpecialChargesNet',
    'Other Operating Income',
    'Amortization of Intangible Assets',
    'Revenue, Net (Deprecated 2018-01-31)',
    'Cost of Goods Sold (Deprecated 2018-01-31)',
    'Cost of Goods and Service, Excluding Depreciation, Depletion, and Amortization',
    'amd_AmortizationOfAcquisitionRelatedIntangibles',
    'Cost, Depreciation, Amortization and Depletion',
    'amd_AmortizationOfAcquisitionRelatedIntangiblesCOGS'
])

# Excluindo a coluna do ano de '2017' e '2018', pois ela não está com os dados consolidados do período
df_income_statement_10k = df_income_statement_10k.drop('2017-12-30', axis=1)
df_income_statement_10k = df_income_statement_10k.drop('2018-12-29', axis=1)

# Renomeando a linha do índice que está sem formato 
df_income_statement_10k = df_income_statement_10k.rename(index={
    'amd_AmortizationOfAcquisitionRelatedIntangibles_Opex': 'Amortization Of Acquisition Related Intangibles',
    'amd_GainLossOnLicensingAgreement': 'Gain Loss On Licensing Agreement'
})

# Arrumando alguns números no df
df_income_statement_10k.loc['Earnings Per Share, Basic', '2024-12-28'] = 1.01
df_income_statement_10k.loc['Earnings Per Share, Diluted', '2024-12-28'] = 1.00
df_income_statement_10k.loc['Earnings Per Share, Basic', '2023-12-30'] = 0.53
df_income_statement_10k.loc['Earnings Per Share, Diluted', '2023-12-30'] = 0.53
df_income_statement_10k.loc['Earnings Per Share, Basic', '2022-12-31'] = 0.85
df_income_statement_10k.loc['Earnings Per Share, Diluted', '2022-12-31'] = 0.84
df_income_statement_10k.loc['Earnings Per Share, Basic', '2021-12-25'] = 2.61
df_income_statement_10k.loc['Earnings Per Share, Diluted', '2021-12-25'] = 2.57
df_income_statement_10k.loc['Earnings Per Share, Basic', '2020-12-26'] = 2.10
df_income_statement_10k.loc['Earnings Per Share, Diluted', '2020-12-26'] = 2.06
df_income_statement_10k.loc['Earnings Per Share, Basic', '2019-12-28'] = 0.31
df_income_statement_10k.loc['Earnings Per Share, Diluted', '2019-12-28'] = 0.30

# Adicionando as colunas dos anos de 2017 e 2018 no df
df_income_statement_10k_final = pd.concat([df_income_statement_10k, df_is_1718], axis=1)

# Transformando o tipo das colunas de str p/ datetime
df_income_statement_10k_final.columns = pd.to_datetime(df_income_statement_10k_final.columns, format='%Y-%m-%d')

# Substituindo os NaN por 0
df_income_statement_10k_final = df_income_statement_10k_final.fillna(0)

df_income_statement_10k_final

In [ ]:
# Fazendo o scraping do cash flow statement (10-K)
cash_flow_statement_lst = []

for acc_num in df_lst_acc_num_amd_sliced['acc_num']:

    # Cash flow statement
    cash_flow_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='cash_flow_statement')
    cash_flow_statement = f_sec.rename_statement(cash_flow_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    cash_flow_statement = cash_flow_statement.iloc[:, 0]
    cash_flow_statement_lst.append(cash_flow_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_cash_flow_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in cash_flow_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_cash_flow_10k = pd.concat([df_cash_flow_10k, df], axis=1)

# Substituindo os NaN por 0
df_cash_flow_10k = df_cash_flow_10k.fillna(0)

# A linha 'Proceeds from Maturities, Prepayments and Calls of Short-term Investments' nas colunas '2022-12-31', '2021-12-25' e '2020-12-26' estão zeradas
# As informações estão na linha 'Proceeds from Sale, Maturity and Collection of Short-term Investments'
df_cash_flow_10k.loc['Proceeds from Maturities, Prepayments and Calls of Short-term Investments', '2022-12-31'] = df_cash_flow_10k.loc['Proceeds from Sale, Maturity and Collection of Short-term Investments', '2022-12-31'] 
df_cash_flow_10k.loc['Proceeds from Maturities, Prepayments and Calls of Short-term Investments', '2021-12-25'] = df_cash_flow_10k.loc['Proceeds from Sale, Maturity and Collection of Short-term Investments', '2021-12-25'] 
df_cash_flow_10k.loc['Proceeds from Maturities, Prepayments and Calls of Short-term Investments', '2020-12-26'] = df_cash_flow_10k.loc['Proceeds from Sale, Maturity and Collection of Short-term Investments', '2020-12-26'] 

# A linha 'Other Depreciation and Amortization' nas colunas '2019-12-28', '2018-12-29' e '2017-12-30' estão zeradas
# As informações estão na linha 'Depreciation, Depletion and Amortization'
df_cash_flow_10k.loc['Other Depreciation and Amortization', '2019-12-28'] = df_cash_flow_10k.loc['Depreciation, Depletion and Amortization', '2019-12-28'] 
df_cash_flow_10k.loc['Other Depreciation and Amortization', '2018-12-29'] = df_cash_flow_10k.loc['Depreciation, Depletion and Amortization', '2018-12-29'] 
df_cash_flow_10k.loc['Other Depreciation and Amortization', '2017-12-30'] = df_cash_flow_10k.loc['Depreciation, Depletion and Amortization', '2017-12-30'] 

# A linha 'amd_AcquisitionsNetOfCashAcquiredAllOthers' na coluna '2022-12-31' está zerada
# As informações estão na linha 'Payments to Acquire Businesses, Net of Cash Acquired'
df_cash_flow_10k.loc['amd_AcquisitionsNetOfCashAcquiredAllOthers', '2022-12-31'] = df_cash_flow_10k.loc['Payments to Acquire Businesses, Net of Cash Acquired', '2022-12-31']

# A linha 'Operating Lease, Right-of-Use Asset, Amortization Expense' na coluna '2019-12-28' está zerada
# As informações estão na linha 'amd_OperatingLeaseRightofUseAssetAmortization'                                        
df_cash_flow_10k.loc['Operating Lease, Right-of-Use Asset, Amortization Expense', '2019-12-28'] = df_cash_flow_10k.loc['amd_OperatingLeaseRightofUseAssetAmortization', '2019-12-28'] 

#  A linha 'Increase (Decrease) in Due to Related Parties' nas colunas '2019-12-28', '2018-12-29' e '2017-12-30' estão zeradas
# As informações estão na linha 'Increase (Decrease) in Accounts Payable, Related Parties'
df_cash_flow_10k.loc['Increase (Decrease) in Due to Related Parties', '2019-12-28'] = df_cash_flow_10k.loc['Increase (Decrease) in Accounts Payable, Related Parties', '2019-12-28'] 
df_cash_flow_10k.loc['Increase (Decrease) in Due to Related Parties', '2018-12-29'] = df_cash_flow_10k.loc['Increase (Decrease) in Accounts Payable, Related Parties', '2018-12-29'] 
df_cash_flow_10k.loc['Increase (Decrease) in Due to Related Parties', '2017-12-30'] = df_cash_flow_10k.loc['Increase (Decrease) in Accounts Payable, Related Parties', '2017-12-30'] 

# A linha 'Increase (Decrease) in Accounts Payable' nas colunas '2019-12-28', '2018-12-29' e '2017-12-30' estão zeradas
# As informações estão na linha 'Increase (Decrease) in Accounts Payable and Accrued Liabilities'
df_cash_flow_10k.loc['Increase (Decrease) in Accounts Payable', '2019-12-28'] = df_cash_flow_10k.loc['Increase (Decrease) in Accounts Payable and Accrued Liabilities', '2019-12-28'] 
df_cash_flow_10k.loc['Increase (Decrease) in Accounts Payable', '2018-12-29'] = df_cash_flow_10k.loc['Increase (Decrease) in Accounts Payable and Accrued Liabilities', '2018-12-29'] 
df_cash_flow_10k.loc['Increase (Decrease) in Accounts Payable', '2017-12-30'] = df_cash_flow_10k.loc['Increase (Decrease) in Accounts Payable and Accrued Liabilities', '2017-12-30'] 

# A linha 'Payments to Acquire Short-term Investments' na coluna '2019-12-28' está zerada
# As informações estão na linha 'Payments to Acquire Debt Securities, Available-for-sale'
df_cash_flow_10k.loc['Payments to Acquire Short-term Investments', '2019-12-28'] = df_cash_flow_10k.loc['Payments to Acquire Debt Securities, Available-for-sale', '2019-12-28'] 

# A linha 'Proceeds from Maturities, Prepayments and Calls of Short-term Investments' nas colunas '2019-12-28', '2018-12-29' e '2017-12-30' estão zeradas
# As informações estão na linha 'Proceeds from Sale and Maturity of Debt Securities, Available-for-sale'
df_cash_flow_10k.loc['Proceeds from Maturities, Prepayments and Calls of Short-term Investments', '2019-12-28'] = df_cash_flow_10k.loc['Proceeds from Sale and Maturity of Debt Securities, Available-for-sale', '2019-12-28'] 
df_cash_flow_10k.loc['Proceeds from Maturities, Prepayments and Calls of Short-term Investments', '2018-12-29'] = df_cash_flow_10k.loc['Proceeds from Sale and Maturity of Debt Securities, Available-for-sale', '2018-12-29'] 
df_cash_flow_10k.loc['Proceeds from Maturities, Prepayments and Calls of Short-term Investments', '2017-12-30'] = df_cash_flow_10k.loc['Proceeds from Sale and Maturity of Debt Securities, Available-for-sale', '2017-12-30'] 

#  A linha 'Proceeds from Short-term Debt' na coluna '2019-12-28' está zerada
# As informações estão na linha 'Proceeds from (Repayments of) Short-term Debt'
df_cash_flow_10k.loc['Proceeds from Short-term Debt', '2019-12-28'] = df_cash_flow_10k.loc['Proceeds from (Repayments of) Short-term Debt', '2019-12-28'] 

#  A linha 'Gain (Loss) on Disposition of Property Plant Equipment' na coluna '2018-12-29' está zerada
# As informações estão na linha 'Gain (Loss) on Disposition of Assets'
df_cash_flow_10k.loc['Gain (Loss) on Disposition of Property Plant Equipment', '2018-12-29'] = df_cash_flow_10k.loc['Gain (Loss) on Disposition of Assets', '2018-12-29'] 

#  A linha 'Payments to Acquire Short-term Investments' na coluna '2018-12-29' está zerada
# As informações estão na linha 'Payments to Acquire Available-for-sale Securities'
df_cash_flow_10k.loc['Payments to Acquire Short-term Investments', '2018-12-29'] = df_cash_flow_10k.loc['Payments to Acquire Available-for-sale Securities', '2018-12-29'] 

#  A linha 'Repayments of Debt' nas colunas '2019-12-28', '2018-12-29' e '2017-12-30' estão zeradas
# As informações estão na linha 'Repayments of Long-term Debt' e 'Repayment of Long-term Debt, Long-term Lease Obligation, and Capital Security'
df_cash_flow_10k.loc['Repayments of Debt', '2019-12-28'] = df_cash_flow_10k.loc['Repayments of Long-term Debt', '2019-12-28'] 
df_cash_flow_10k.loc['Repayments of Debt', '2018-12-29'] = df_cash_flow_10k.loc['Repayment of Long-term Debt, Long-term Lease Obligation, and Capital Security', '2018-12-29'] 
df_cash_flow_10k.loc['Repayments of Debt', '2017-12-30'] = df_cash_flow_10k.loc['Repayment of Long-term Debt, Long-term Lease Obligation, and Capital Security', '2017-12-30'] 

#  A linha 'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect' na coluna '2017-12-30' está zerada
# As informações estão na linha 'Cash and Cash Equivalents, Period Increase (Decrease)'
df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect', '2017-12-30'] = df_cash_flow_10k.loc['Cash and Cash Equivalents, Period Increase (Decrease)', '2017-12-30'] 

#  A linha 'Proceeds from Short-term Debt' na coluna '2017-12-30' está zerada
# As informações estão na linha 'Proceeds from (Repayments of) Lines of Credit'
df_cash_flow_10k.loc['Proceeds from Short-term Debt', '2017-12-30'] = df_cash_flow_10k.loc['Proceeds from (Repayments of) Lines of Credit', '2017-12-30'] 

# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_cash_flow_10k = df_cash_flow_10k.drop([
    'Proceeds from Sale, Maturity and Collection of Short-term Investments',
    'Depreciation, Depletion and Amortization',
    'Payments to Acquire Businesses, Net of Cash Acquired',
    'amd_OperatingLeaseRightofUseAssetAmortization',
    'Increase (Decrease) in Accounts Payable, Related Parties',
    'Increase (Decrease) in Accounts Payable and Accrued Liabilities',
    'Payments to Acquire Debt Securities, Available-for-sale',
    'Proceeds from Sale and Maturity of Debt Securities, Available-for-sale',
    'Proceeds from (Repayments of) Short-term Debt',
    'Gain (Loss) on Disposition of Assets',
    'Payments to Acquire Available-for-sale Securities',
    'Repayments of Long-term Debt',
    'Repayment of Long-term Debt, Long-term Lease Obligation, and Capital Security',
    'Cash and Cash Equivalents, Period Increase (Decrease)',
    'Proceeds from (Repayments of) Lines of Credit',
    'Proceeds from Divestiture of Interest in Consolidated Subsidiaries'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_cash_flow_10k = df_cash_flow_10k.drop([
    'us-gaap_NetCashProvidedByUsedInOperatingActivitiesAbstract',
    'Other Adjustments to Income, Discontinued Operations',
    'amd_AmortizationOfInventoryFairValueAdjustment',
    'amd_IssuanceOfTreasuryStockToPartiallySettleDebt',
    'Restricted Cash, Current',
    'amd_DeferredProceedsOnSaleOfReceivable',
    'amd_DeferredCashOnSoldReceivables',
    'Issuance of Stock and Warrants for Services or Claims',
    'Proceeds from Issuance of Common Stock',
    'Proceeds from Issuance of Long-term Debt',
    'amd_RestructuringandOtherSpecialChargesNetNoncash',
    'Proceeds from Sale of Property, Plant, and Equipment',
    'Other Significant Noncash Transaction, Value of Consideration Given',
    'Stock Issued',
    'amd_NoncashActivitiesOther',
    'Conversion of Stock, Amount Converted',
    'Cash Acquired from Acquisition',
    'Deconsolidation, Gain (Loss), Amount',
    'amd_Deferredcashonsoldreceivables',
    'Net Cash Provided by (Used in) Operating Activities, Continuing Operations',
    'Cash Provided by (Used in) Operating Activities, Discontinued Operations',
    'Net Cash Provided by (Used in) Investing Activities, Continuing Operations',
    'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations',
    'amd_InventoryLossAtRecoveryFromContractManufacturer',
    'Asset Impairment Charges',
    'Proceeds from Warrant Exercises',
])

# Arrumando alguns números no df
df_cash_flow_10k.loc['amd_AcquisitionsNetOfCashAcquiredAllOthers', '2023-12-30'] = -131
df_cash_flow_10k.loc['Amortization', '2023-12-30'] = 2811 	
df_cash_flow_10k.loc['Amortization', '2022-12-31'] = 3548 	
df_cash_flow_10k.loc['Payments to Acquire Short-term Investments', '2017-12-30'] = -222 	
df_cash_flow_10k.loc['Increase (Decrease) in Accrued Liabilities and Other Operating Liabilities', '2019-12-28'] = 220 	 	
df_cash_flow_10k.loc['Increase (Decrease) in Accrued Liabilities and Other Operating Liabilities', '2018-12-29'] = 81 	
df_cash_flow_10k.loc['Increase (Decrease) in Accrued Liabilities and Other Operating Liabilities', '2017-12-30'] = -8 	
df_cash_flow_10k.loc['Payments to Acquire Other Investments', '2023-12-30'] = 11 	
df_cash_flow_10k.loc['Payments to Acquire Other Investments', '2022-12-31'] = 5 	
df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2017-12-30'] = 1266
	
# Renomeando as linhas do índice que estão sem formato 
df_cash_flow_10k = df_cash_flow_10k.rename(index={
    'amd_IncreaseDecreaseinPrepaymentsRelatedParties': 'Increase Decrease in Prepayments Related Parties',
    'amd_AcquisitionsNetOfCashAcquiredAllOthers': 'Acquisitions Net Of Cash Acquired All Others',
})

df_cash_flow_10k

In [ ]:
# Arrumando alguns index dos dfs para ficar padronizado com os index utilizados nas funções p/ calcular os indicadores fundamentalistas
df_balance_sheet_10k = df_balance_sheet_10k.rename(index={
    'Cash and Cash Equivalents, at Carrying Value': 'Cash and Cash Equivalents',
    'Short-term Investments': 'Marketable Securities',
    "Stockholders' Equity Attributable to Parent": 'Total Shareholders Equity'
})

df_income_statement_10k_final = df_income_statement_10k_final.rename(index={
    'Revenue from Contract with Customer, Excluding Assessed Tax': 'Revenues',
    'Research and Development Expense': 'Research and Development',
    'Operating Income (Loss)': 'Operating Income',
    'Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest': 'Income Before Provision for Income Taxes',
    'Income Tax Expense (Benefit)': 'Provision for Income Taxes',
    'Net Income (Loss) Attributable to Parent': 'Net Income',
    'Weighted Average Number of Shares Outstanding, Basic': 'Number of Shares - Basic',
    'Weighted Average Number of Shares Outstanding, Diluted': 'Number of Shares - Diluted'
})

df_cash_flow_10k = df_cash_flow_10k.rename(index={
    'Other Depreciation and Amortization': 'Depreciation, Depletion and Amortization',
    'Payments to Acquire Property, Plant, and Equipment': 'Purchases Of Property And Equipment And Intangible Assets',
})

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//semicondutores//amd//amd_10k.xlsx') as writer:
    df_balance_sheet_10k.to_excel(writer, sheet_name='balance_sheet')
    df_income_statement_10k_final.to_excel(writer, sheet_name='income_statement')
    df_cash_flow_10k.to_excel(writer, sheet_name='cash_flow')

### Scraping AMD 10-Q

In [ ]:
# Número dos documentos que contém a DRE, BP e FC 
acc_documents_10q = f_sec.get_filtered_filings(ticker=ticker, ten_k=False, just_accession_numbers=True)

# Trocando os '-' para '' das strings ('0001045810-24-000264' -> '000104581024000264')
dict_acc_num_10q = {k: v.replace('-', '') for k, v in acc_documents_10q.items()}

# Transformando o dicionário em um df
df_lst_acc_num_10q_amd = pd.DataFrame.from_dict(dict_acc_num_10q, orient='index', columns=['acc_num'])

# Transformando em datetime o index
df_lst_acc_num_10q_amd.index = pd.to_datetime(df_lst_acc_num_10q_amd.index, format='%Y-%m-%d')
df_lst_acc_num_10q_amd

In [ ]:
# Selecionando os 'acc_num' dos 10-Q a partir de 2020 p/ iteração
df_lst_acc_num_10q_amd_sliced = df_lst_acc_num_10q_amd.loc['2020-03-28':]
df_lst_acc_num_10q_amd_sliced

In [ ]:
# Rótulos
labels_dict = f_sec.get_label_dictionary(ticker, headers=f_sec.header)

In [ ]:
# Fazendo o scraping do cash flow statement (10-Q)
balance_sheet_lst = []

for acc_num in df_lst_acc_num_10q_amd_sliced['acc_num']:

    # Cash flow statement
    balance_sheet = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='balance_sheet')
    balance_sheet = f_sec.rename_statement(balance_sheet, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    balance_sheet = balance_sheet.iloc[:, 0]
    balance_sheet_lst.append(balance_sheet)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_balance_sheet_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in balance_sheet_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_balance_sheet_10q = pd.concat([df_balance_sheet_10q, df], axis=1)

# Substituindo os NaN por 0
df_balance_sheet_10q = df_balance_sheet_10q.fillna(0)

# A linha 'Other Receivables, Net, Current' nas colunas '2022-09-24', '2022-06-25', '2022-03-26', '2021-09-25', '2021-06-26', '2021-03-27', '2020-09-26', '2020-06-27' e '2020-03-28' estão zeradas
# As informações estão na linha 'Due from Related Parties, Current'
df_balance_sheet_10q.loc['Other Receivables, Net, Current', '2022-09-24'] = df_balance_sheet_10q.loc['Due from Related Parties, Current', '2022-09-24'] 
df_balance_sheet_10q.loc['Other Receivables, Net, Current', '2022-06-25'] = df_balance_sheet_10q.loc['Due from Related Parties, Current', '2022-06-25'] 
df_balance_sheet_10q.loc['Other Receivables, Net, Current', '2022-03-26'] = df_balance_sheet_10q.loc['Due from Related Parties, Current', '2022-03-26'] 
df_balance_sheet_10q.loc['Other Receivables, Net, Current', '2021-09-25'] = df_balance_sheet_10q.loc['Due from Related Parties, Current', '2021-09-25'] 
df_balance_sheet_10q.loc['Other Receivables, Net, Current', '2021-06-26'] = df_balance_sheet_10q.loc['Due from Related Parties, Current', '2021-06-26'] 
df_balance_sheet_10q.loc['Other Receivables, Net, Current', '2021-03-27'] = df_balance_sheet_10q.loc['Due from Related Parties, Current', '2021-03-27'] 
df_balance_sheet_10q.loc['Other Receivables, Net, Current', '2020-09-26'] = df_balance_sheet_10q.loc['Due from Related Parties, Current', '2020-09-26'] 
df_balance_sheet_10q.loc['Other Receivables, Net, Current', '2020-06-27'] = df_balance_sheet_10q.loc['Due from Related Parties, Current', '2020-06-27'] 
df_balance_sheet_10q.loc['Other Receivables, Net, Current', '2020-03-28'] = df_balance_sheet_10q.loc['Due from Related Parties, Current', '2020-03-28'] 

# A linha 'Accounts Payable, Other' nas colunas '2022-09-24', '2022-06-25', '2022-03-26', '2021-09-25', '2021-06-26', '2021-03-27', '2020-09-26', '2020-06-27' e '2020-03-28' estão zeradas
# As informações estão na linha 'Due to Related Parties, Current'
df_balance_sheet_10q.loc['Accounts Payable, Other', '2022-09-24'] = df_balance_sheet_10q.loc['Due to Related Parties, Current', '2022-09-24'] 
df_balance_sheet_10q.loc['Accounts Payable, Other', '2022-06-25'] = df_balance_sheet_10q.loc['Due to Related Parties, Current', '2022-06-25'] 
df_balance_sheet_10q.loc['Accounts Payable, Other', '2022-03-26'] = df_balance_sheet_10q.loc['Due to Related Parties, Current', '2022-03-26'] 
df_balance_sheet_10q.loc['Accounts Payable, Other', '2021-09-25'] = df_balance_sheet_10q.loc['Due to Related Parties, Current', '2021-09-25'] 
df_balance_sheet_10q.loc['Accounts Payable, Other', '2021-06-26'] = df_balance_sheet_10q.loc['Due to Related Parties, Current', '2021-06-26'] 
df_balance_sheet_10q.loc['Accounts Payable, Other', '2021-03-27'] = df_balance_sheet_10q.loc['Due to Related Parties, Current', '2021-03-27'] 
df_balance_sheet_10q.loc['Accounts Payable, Other', '2020-09-26'] = df_balance_sheet_10q.loc['Due to Related Parties, Current', '2020-09-26'] 
df_balance_sheet_10q.loc['Accounts Payable, Other', '2020-06-27'] = df_balance_sheet_10q.loc['Due to Related Parties, Current', '2020-06-27'] 
df_balance_sheet_10q.loc['Accounts Payable, Other', '2020-03-28'] = df_balance_sheet_10q.loc['Due to Related Parties, Current', '2020-03-28'] 

# A linha 'Short-term Investments' na coluna '2020-03-28' está zerada
# As informações estão na linha 'Marketable Securities, Current'
df_balance_sheet_10q.loc['Short-term Investments', '2020-03-28'] = df_balance_sheet_10q.loc['Marketable Securities, Current', '2020-03-28'] 

# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_balance_sheet_10q = df_balance_sheet_10q.drop([
    'Due from Related Parties, Current',
    'Due to Related Parties, Current',
    'Marketable Securities, Current'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_balance_sheet_10q = df_balance_sheet_10q.drop([
    'us-gaap_AssetsCurrentAbstract',
    'Common Stock, Shares, Outstanding',
    'Common Stock, Par or Stated Value Per Share',
    'Common Stock, Shares Authorized',
    'Common Stock, Shares, Issued'
])

df_balance_sheet_10q

In [ ]:
# Fazendo o scraping do income statement (10-Q)
income_statement_lst = []

for acc_num in df_lst_acc_num_10q_amd_sliced['acc_num']:

    # Income statement
    income_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='income_statement')
    income_statement = f_sec.rename_statement(income_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    income_statement = income_statement.iloc[:, 0]
    income_statement_lst.append(income_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_income_statement_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in income_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_income_statement_10q = pd.concat([df_income_statement_10q, df], axis=1)

# Substituindo os NaN por 0
df_income_statement_10q = df_income_statement_10q.fillna(0)

# A linha 'amd_AmortizationOfAcquisitionRelatedIntangiblesOpex' nas colunas '2022-06-25' e '2022-03-26' estão zeradas
# As informações estão na linha 'Amortization of Intangible Assets'
df_income_statement_10q.loc['amd_AmortizationOfAcquisitionRelatedIntangiblesOpex', '2022-06-25'] = df_income_statement_10q.loc['Amortization of Intangible Assets', '2022-06-25'] 
df_income_statement_10q.loc['amd_AmortizationOfAcquisitionRelatedIntangiblesOpex', '2022-03-26'] = df_income_statement_10q.loc['Amortization of Intangible Assets', '2022-03-26'] 

# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_income_statement_10q = df_income_statement_10q.drop('Amortization of Intangible Assets')

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_income_statement_10q = df_income_statement_10q.drop([
    'us-gaap_IncomeStatementAbstract',
    'Cost of Goods and Service, Excluding Depreciation, Depletion, and Amortization',
    'amd_AmortizationOfAcquisitionRelatedIntangiblesCOGS',
    'Income (Loss) from Continuing Operations, Net of Tax, Attributable to Parent',
    'Income (Loss) from Discontinued Operations, Net of Tax, Attributable to Parent',
    'Income (Loss) from Continuing Operations, Per Basic Share',
    'Income (Loss) from Discontinued Operations and Disposal of Discontinued Operations, Net of Tax, Per Basic Share',
    'Income (Loss) from Discontinued Operations and Disposal of Discontinued Operations, Net of Tax, Per Diluted Share',
])

# Arrumando e adicionando alguns números no df
df_income_statement_10q.loc['Earnings Per Share, Diluted', '2024-06-29'] = 0.16
df_income_statement_10q.loc['Earnings Per Share, Diluted', '2023-04-01'] = -0.09
df_income_statement_10q.loc['Earnings Per Share, Diluted', '2020-06-27'] = 0.13
df_income_statement_10q.loc['Earnings Per Share, Diluted', '2020-03-28'] = 0.14

# Renomeando algumas linhas do índice
df_income_statement_10q = df_income_statement_10q.rename(index={
    'amd_AmortizationOfAcquisitionRelatedIntangiblesOpex': 'Amortization Of Acquisition Related Intangibles - OPEX',
    'amd_GainOnLicensingAgreement': 'Gain On Licensing Agreement'
})

df_income_statement_10q

In [ ]:
# Fazendo o scraping do cash flow statement (10-Q)
cash_flow_statement_lst = []

for acc_num in df_lst_acc_num_10q_amd_sliced['acc_num']:

    # Cash flow statement
    cash_flow_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='cash_flow_statement')
    cash_flow_statement = f_sec.rename_statement(cash_flow_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    cash_flow_statement = cash_flow_statement.iloc[:, 0]
    cash_flow_statement_lst.append(cash_flow_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_cash_flow_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in cash_flow_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_cash_flow_10q = pd.concat([df_cash_flow_10q, df], axis=1)

# Substituindo os NaN por 0
df_cash_flow_10q = df_cash_flow_10q.fillna(0)

# A linha 'Other Depreciation and Amortization' na coluna '2023-09-30' está zerada
# As informações estão na linha 'Depreciation, Depletion and Amortization'
df_cash_flow_10q.loc['Other Depreciation and Amortization', '2023-09-30'] = df_cash_flow_10q.loc['Depreciation, Depletion and Amortization', '2023-09-30']

# A linha 'amd_InventoryLossAtContractManufacturer' na coluna '2024-03-30' está zerada
# As informações estão na linha 'Inventory Write-down'
df_cash_flow_10q.loc['amd_InventoryLossAtContractManufacturer', '2024-03-30'] = df_cash_flow_10q.loc['Inventory Write-down', '2024-03-30']

# A linha 'amd_AcquisitionsNetOfCashAcquiredAllOthers' nas colunas '2022-09-24' e '2022-06-25' estão zeradas
# As informações estão na linha 'Payments to Acquire Businesses, Gross' e 'Payments to Acquire Businesses, Net of Cash Acquired'
df_cash_flow_10q.loc['amd_AcquisitionsNetOfCashAcquiredAllOthers', '2022-09-24'] = df_cash_flow_10q.loc['Payments to Acquire Businesses, Gross', '2022-09-24']
df_cash_flow_10q.loc['amd_AcquisitionsNetOfCashAcquiredAllOthers', '2022-06-25'] = df_cash_flow_10q.loc['Payments to Acquire Businesses, Net of Cash Acquired', '2022-06-25']

# A linha 'Equity Securities, FV-NI, Gain (Loss)' nas colunas '2021-06-26' e '2021-03-27' estão zeradas
# As informações estão na linha 'Other than Temporary Impairment Losses, Investments'
df_cash_flow_10q.loc['Equity Securities, FV-NI, Gain (Loss)', '2021-06-26'] = df_cash_flow_10q.loc['Other than Temporary Impairment Losses, Investments', '2021-06-26']
df_cash_flow_10q.loc['Equity Securities, FV-NI, Gain (Loss)', '2021-03-27'] = df_cash_flow_10q.loc['Other than Temporary Impairment Losses, Investments', '2021-03-27']

# A linha 'Increase (Decrease) in Due from Related Parties' nas colunas '2020-09-26', '2020-06-27' e '2020-03-28' estão zeradas
# As informações estão na linha 'Increase (Decrease) in Due from Related Parties, Current'
df_cash_flow_10q.loc['Increase (Decrease) in Due from Related Parties', '2020-09-26'] = df_cash_flow_10q.loc['Increase (Decrease) in Due from Related Parties, Current', '2020-09-26']
df_cash_flow_10q.loc['Increase (Decrease) in Due from Related Parties', '2020-06-27'] = df_cash_flow_10q.loc['Increase (Decrease) in Due from Related Parties, Current', '2020-06-27']
df_cash_flow_10q.loc['Increase (Decrease) in Due from Related Parties', '2020-03-28'] = df_cash_flow_10q.loc['Increase (Decrease) in Due from Related Parties, Current', '2020-03-28']

# A linha 'Increase (Decrease) in Due to Related Parties' nas colunas '2020-09-26', '2020-06-27' e '2020-03-28' estão zeradas
# As informações estão na linha 'Increase (Decrease) in Due to Related Parties, Current'
df_cash_flow_10q.loc['Increase (Decrease) in Due to Related Parties', '2020-09-26'] = df_cash_flow_10q.loc['Increase (Decrease) in Due to Related Parties, Current', '2020-09-26']
df_cash_flow_10q.loc['Increase (Decrease) in Due to Related Parties', '2020-06-27'] = df_cash_flow_10q.loc['Increase (Decrease) in Due to Related Parties, Current', '2020-06-27']
df_cash_flow_10q.loc['Increase (Decrease) in Due to Related Parties', '2020-03-28'] = df_cash_flow_10q.loc['Increase (Decrease) in Due to Related Parties, Current', '2020-03-28']

# A linha 'Increase (Decrease) in Accounts Payable' nas colunas '2020-09-26', '2020-06-27' e '2020-03-28' estão zeradas
# As informações estão na linha 'Increase (Decrease) in Accounts Payable and Accrued Liabilities'
df_cash_flow_10q.loc['Increase (Decrease) in Accounts Payable', '2020-09-26'] = df_cash_flow_10q.loc['Increase (Decrease) in Accounts Payable and Accrued Liabilities', '2020-09-26']
df_cash_flow_10q.loc['Increase (Decrease) in Accounts Payable', '2020-06-27'] = df_cash_flow_10q.loc['Increase (Decrease) in Accounts Payable and Accrued Liabilities', '2020-06-27']
df_cash_flow_10q.loc['Increase (Decrease) in Accounts Payable', '2020-03-28'] = df_cash_flow_10q.loc['Increase (Decrease) in Accounts Payable and Accrued Liabilities', '2020-03-28']

# A linha 'Operating Lease, Right-of-Use Asset, Amortization Expense' nas colunas '2020-06-27' e '2020-03-28' estão zeradas
# As informações estão na linha 'amd_OperatingLeaseRightofUseAssetAmortization'
df_cash_flow_10q.loc['Operating Lease, Right-of-Use Asset, Amortization Expense', '2020-06-27'] = df_cash_flow_10q.loc['amd_OperatingLeaseRightofUseAssetAmortization', '2020-06-27']
df_cash_flow_10q.loc['Operating Lease, Right-of-Use Asset, Amortization Expense', '2020-03-28'] = df_cash_flow_10q.loc['amd_OperatingLeaseRightofUseAssetAmortization', '2020-03-28']

# A linha 'Payments to Acquire Short-term Investments' nas colunas '2020-06-27' e '2020-03-28' estão zeradas
# As informações estão na linha 'Payments to Acquire Debt Securities, Available-for-sale'
df_cash_flow_10q.loc['Payments to Acquire Short-term Investments', '2020-06-27'] = df_cash_flow_10q.loc['Payments to Acquire Debt Securities, Available-for-sale', '2020-06-27']
df_cash_flow_10q.loc['Payments to Acquire Short-term Investments', '2020-03-28'] = df_cash_flow_10q.loc['Payments to Acquire Debt Securities, Available-for-sale', '2020-03-28']

# A linha 'Proceeds from Maturities, Prepayments and Calls of Short-term Investments' nas colunas '2020-06-27' e '2020-03-28' estão zeradas
# As informações estão na linha 'Proceeds from Sale and Maturity of Debt Securities, Available-for-sale'
df_cash_flow_10q.loc['Proceeds from Maturities, Prepayments and Calls of Short-term Investments', '2020-06-27'] = df_cash_flow_10q.loc['Proceeds from Sale and Maturity of Debt Securities, Available-for-sale', '2020-06-27']
df_cash_flow_10q.loc['Proceeds from Maturities, Prepayments and Calls of Short-term Investments', '2020-03-28'] = df_cash_flow_10q.loc['Proceeds from Sale and Maturity of Debt Securities, Available-for-sale', '2020-03-28']

# A linha 'Repayments of Debt' na coluna '2022-09-24' está zerada
# As informações estão na linha 'Repayments of Short-term Debt'
df_cash_flow_10q.loc['Repayments of Debt', '2022-09-24'] = df_cash_flow_10q.loc['Repayments of Short-term Debt', '2022-09-24']

# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_cash_flow_10q = df_cash_flow_10q.drop([
    'Depreciation, Depletion and Amortization',
    'Inventory Write-down',
    'Payments to Acquire Businesses, Gross',
    'Payments to Acquire Businesses, Net of Cash Acquired',
    'Other than Temporary Impairment Losses, Investments',
    'Increase (Decrease) in Due from Related Parties, Current',
    'Increase (Decrease) in Due to Related Parties, Current',
    'Increase (Decrease) in Accounts Payable and Accrued Liabilities',
    'amd_OperatingLeaseRightofUseAssetAmortization',
    'Payments to Acquire Debt Securities, Available-for-sale',
    'Proceeds from Sale and Maturity of Debt Securities, Available-for-sale',
    'Repayments of Short-term Debt'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_cash_flow_10q = df_cash_flow_10q.drop([
    'us-gaap_NetCashProvidedByUsedInOperatingActivitiesAbstract',
    'Proceeds from Issuance of Unsecured Debt',
    'Proceeds from Sale and Collection of Receivables',
    'Proceeds from Warrant Exercises',
    'amd_IssuanceOfTreasuryStockToPartiallySettleDebt',
    'Stock Issued',
    'Repayments of Long-term Debt',
    'Other Adjustments to Income, Discontinued Operations',
    'Net Cash Provided by (Used in) Operating Activities, Continuing Operations',
    'Cash Provided by (Used in) Operating Activities, Discontinued Operations',
    'Net Cash Provided by (Used in) Investing Activities, Continuing Operations',
    'Cash Provided by (Used in) Investing Activities, Discontinued Operations',
    'Cash and Cash Equivalents, at Carrying Value, Including Discontinued Operations',
    'Restricted Cash',
    'Restricted Cash, Current',
    'Stock Issued During Period, Value, Treasury Stock Reissued',
    'AssetAcquisitionContingentConsiderationLiability',
    'Payments to Acquire Interest in Subsidiaries and Affiliates',
    'amd_LiabilitiesAssumedRSUTaxWithholding',
    'amd_InventoryLossAtContractManufacturer',
    'Cash Acquired from Acquisition',
    'amd_AcquisitionsNetOfCashAcquiredAllOthers',
    'Business Combination, Consideration Transferred, Equity Interests Issued and Issuable',
    'Gain (Loss) on Extinguishment of Debt',
    'Stock Issued During Period, Value, Conversion of Convertible Securities',
    'Noncash or Part Noncash Acquisition, Fixed Assets Acquired'
])

# Arrumando e adicionando alguns números no df
df_cash_flow_10q.loc['amd_AmortizationOfInventoryFairValueAdjustment', '2022-09-24'] = 187
df_cash_flow_10q.loc['Cash and Cash Equivalents, at Carrying Value', '2024-06-29'] = 4113
df_cash_flow_10q.loc['Other Depreciation and Amortization', '2020-06-27'] = 140
df_cash_flow_10q.loc['Other Depreciation and Amortization', '2020-03-28'] = 68

# Renomeando algumas linhas do índice
df_cash_flow_10q = df_cash_flow_10q.rename(index={
    'amd_ReleaseOfReservesForTaxLiabilities': 'Release Of Reserves For Tax Liabilities',
    'amd_AmortizationOfInventoryFairValueAdjustment': 'Amortization Of Inventory Fair Value Adjustment',
})

df_cash_flow_10q

In [ ]:
# Arrumando alguns index dos dfs para ficar padronizado com os index utilizados nas funções p/ calcular os indicadores fundamentalistas
df_balance_sheet_10q = df_balance_sheet_10q.rename(index={
    'Cash and Cash Equivalents, at Carrying Value': 'Cash and Cash Equivalents',
    'Short-term Investments': 'Marketable Securities',
    "Stockholders' Equity Attributable to Parent": 'Total Shareholders Equity'
})

df_income_statement_10q = df_income_statement_10q.rename(index={
    'Revenue from Contract with Customer, Excluding Assessed Tax': 'Revenues',
    'Research and Development Expense': 'Research and Development',
    'Operating Income (Loss)': 'Operating Income',
    'Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest': 'Income Before Provision for Income Taxes',
    'Income Tax Expense (Benefit)': 'Provision for Income Taxes',
    'Net Income (Loss) Attributable to Parent': 'Net Income',
    'Weighted Average Number of Shares Outstanding, Basic': 'Number of Shares - Basic',
    'Weighted Average Number of Shares Outstanding, Diluted': 'Number of Shares - Diluted'
})

df_cash_flow_10q = df_cash_flow_10q.rename(index={
    'Other Depreciation and Amortization': 'Depreciation, Depletion and Amortization',
    'Payments to Acquire Property, Plant, and Equipment': 'Purchases Of Property And Equipment And Intangible Assets',
})

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//semicondutores//amd//amd_10q.xlsx') as writer:
    df_balance_sheet_10q.to_excel(writer, sheet_name='balance_sheet')
    df_income_statement_10q.to_excel(writer, sheet_name='income_statement')
    df_cash_flow_10q.to_excel(writer, sheet_name='cash_flow')

### Indicadores Fundamentalistas

#### Indicadores 10-K

In [ ]:
# Lendo os arquivos 10k
amd_bs = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//amd//amd_10k.xlsx', 
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

amd_is = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//amd//amd_10k.xlsx', 
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

amd_cf = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//amd//amd_10k.xlsx', 
    sheet_name='cash_flow', 
    index_col='Unnamed: 0'
)

# Transformando as colunas em datetime
amd_bs.columns = pd.to_datetime(amd_bs.columns)
amd_is.columns = pd.to_datetime(amd_is.columns)
amd_cf.columns = pd.to_datetime(amd_cf.columns)

# Transformando as colunas em apenas o ano
amd_bs.columns = amd_bs.columns.year
amd_is.columns = amd_is.columns.year
amd_cf.columns = amd_cf.columns.year

In [ ]:
# Preços da AMD
amd_close = yf.download('AMD', auto_adjust=True, multi_level_index=False, start='2017-01-01')['Close']

# Selecionando o último preço do mês de Dezembro de cada ano 
lst_years = [
    '2017', '2018', '2019', '2020', 
    '2021', '2022', '2023', '2024',
    '2025',
]

last_dec_close = []

for year in lst_years:
    last_close = amd_close[f'{year}-12'].iloc[-1]
    last_dec_close.append(last_close)

# Transformando em uma série pandas
amd_close_dec = pd.Series(last_dec_close, index=lst_years)

# Revertendo o index da série p/ ficar igual com os dos indicadores
amd_close_dec = amd_close_dec.reindex(index=amd_close_dec.index[::-1])

# Arredondamento de duas casas decimais
amd_close_dec = round(amd_close_dec, 2)

# Não realizou split recentemente (https://br.investing.com/equities/adv-micro-device-historical-data-splits)

# Transformando o index em int
amd_close_dec.index = amd_close_dec.index.astype(int)

amd_close_dec

In [ ]:
# Criando uma lista só para fazer a indexação em que o ano mais recente está no começo
idx = amd_bs.columns

# Valor de Mercado
vm = f_eua.indicador_vm(ten_k=True, price=amd_close_dec, df_is=amd_is) 

# LPA
lpa = f_eua.indicador_lpa(ten_k=True, df_is=amd_is, idx=None)

# P/L Damodaran
p_lpa_damodaran = f_eua.indicador_p_lpa_2(ten_k=True, vm=vm, df_is=amd_is)

# P/L
p_lpa = f_eua.indicador_p_lpa(ten_k=True, price=amd_close_dec, lpa=lpa, idx=idx)

# L/P
lpa_p = f_eua.indicador_lpa_p(ten_k=True, price=amd_close_dec, lpa=lpa, idx=idx) 

# Depreciação
depreciacao = f_eua.indicador_depreciacao(ten_k=True, df_cf=amd_cf, first_quarter=None, idx=idx)

# EBITDA
ebitda = f_eua.indicador_ebitda(ten_k=True, df_is=amd_is, idx=None, depreciacao=depreciacao)

# Margem Líquida
m_liq = f_eua.indicador_ml(ten_k=True, df_is=amd_is)

# VPA
vpa = f_eua.indicador_vpa(ten_k=True, df_bs=amd_bs, df_is=amd_is)

# P/VPA
p_vpa = f_eua.indicador_p_vpa(ten_k=True, price=amd_close_dec, vpa=vpa, idx=idx)

# Caixa e Equivalentes
lst_caixa = ['Cash and Cash Equivalents', 'Marketable Securities']
caixa = f_eua.indicador_caixa(ten_k=True, df_bs=amd_bs, lst_itens_caixa=lst_caixa)

# Dívida Bruta
lst_divida = ['Short-term Debt', 'Long-term Debt, Excluding Current Maturities', 'Operating Lease, Liability, Noncurrent']
div_bruta = f_eua.indicador_divida_bruta(ten_k=True, df_bs=amd_bs, lst_itens_divida=lst_divida)

# Dívida Líquida
div_liq = f_eua.indicador_divida_liquida(ten_k=True, caixa=caixa, div_bruta=div_bruta, idx=idx)

# Dívida Líquida / EBITDA
dl_ebitda =  f_eua.indicador_dl_ebitda(ten_k=True, ebitda=ebitda, div_liq=div_liq, idx=None)

# Dívida Líquida / PL
dl_pl = f_eua.indicador_dl_pl(ten_k=True, div_liq=div_liq, df_bs=amd_bs) 

# EV/EBITDA
ev_ebitda = f_eua.indicador_ev_ebitda(vm=vm, div_liq=div_liq, ebitda=ebitda)

# ROE
roe = f_eua.indicador_roe(ten_k=True, df_is=amd_is, df_bs=amd_bs, idx=None)

# ROIC
roic = f_eua.indicador_roic(ten_k=True, df_is=amd_is, df_bs=amd_bs, div_bruta=div_bruta, idx=idx)

# FCO
fco = f_eua.indicador_fco(ten_k=True, df_cf=amd_cf, first_quarter=None) 

# FCI
fci = f_eua.indicador_fci(ten_k=True, df_cf=amd_cf, first_quarter=None) 

# FCF
fcf = f_eua.indicador_fcf(ten_k=True, df_cf=amd_cf, first_quarter=None) 

# CAPEX
lst_capex = ['Purchases Of Property And Equipment And Intangible Assets']

capex= f_eua.indicador_capex(ten_k=True, df_cf=amd_cf, first_quarter=None, lst_itens=lst_capex, idx=idx)

# Free Cash Flow
free_cash_flow = f_eua.indicador_free_cash_flow(fco=fco, capex=capex)

# Net CAPEX
net_capex = f_eua.indicador_net_capex(capex=capex, depreciation=depreciacao)

# R&D
rd_expense = f_eua.indicador_rd(ten_k=True, df_is=amd_is, idx=idx)

# Net CAPEX ajustado
adj_net_capex = f_eua.indicador_adj_net_capex(net_capex=net_capex, rd_expense=rd_expense)

# Working capital
working_capital = f_eua.indicador_working_capital(ten_k=True, df_bs=amd_bs)

# Change in non cash working capital
lst_itens_change_non_cash_wc = [
    'Deferred Income Taxes and Tax Credits',
    'Increase (Decrease) in Accounts Receivable',
    'Increase (Decrease) in Inventories',
    'Increase Decrease in Prepayments Related Parties',
    'Increase (Decrease) in Prepaid Expense and Other Assets',
    'Increase (Decrease) in Due to Related Parties',
    'Increase (Decrease) in Accounts Payable',
    'Increase (Decrease) in Accrued Liabilities and Other Operating Liabilities',
]

change_non_cash_wc = f_eua.indicador_change_non_cash_wc(ten_k=True, df_cf=amd_cf, lst_itens=lst_itens_change_non_cash_wc, first_quarter=None)

# Reinvestment rate
reinvestment_rate = f_eua.indicador_reinvestment_rate(ten_k=True, net_capex=net_capex, change_non_cash_wc=change_non_cash_wc, df_is=amd_is, idx=idx)

# New borrowing
lst_itens_new_borrowing = ['Proceeds from Short-term Debt']

new_borrowing = f_eua.indicador_new_borrowing(ten_k=True, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=amd_cf, first_quarter=None, idx=idx)

# Debt paid
lst_itens_debt_paid = ['Repayments of Debt']

debt_paid = f_eua.indicador_debt_paid(ten_k=True, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=amd_cf, first_quarter=None, idx=idx)

# Taxes on operating income
taxes_operating_income = f_eua.indicador_taxes_operating_income(ten_k=True, df_is=amd_is, idx=idx)

# FCFE
fcfe_10q = f_eua.indicador_fcfe(
    ten_k=True, 
    depreciacao=depreciacao, 
    capex=capex, 
    change_non_cash_wc=change_non_cash_wc, 
    new_borrowing=new_borrowing,
    debt_paid=debt_paid,
    df_is=amd_is,
    idx=idx
)

# FCFF
fcff_10q = f_eua.indicador_fcff(
        ten_k=True, 
        taxes_operating_income=taxes_operating_income,
        depreciacao=depreciacao, 
        capex=capex, 
        change_non_cash_wc=change_non_cash_wc, 
        df_is=amd_is,
        idx=idx
)

# DPA e Payout - AMD não paga dividendo

# Buyback
buyback = f_eua.indicador_buyback(ten_k=True, df_cf=amd_cf, first_quarter=None)

# Colocando todos os indicadores em uma lista
lst_indicadores = [
    p_lpa_damodaran,
    p_lpa,
    lpa_p,  
    ebitda, 
    m_liq,
    p_vpa,
    vm,
    div_bruta,
    caixa,
    div_liq,
    dl_ebitda,
    dl_pl,
    ev_ebitda,
    roe,
    roic,
    fco,
    fci,
    fcf,
    free_cash_flow,
    capex,
    net_capex,
    rd_expense,
    adj_net_capex,
    working_capital,
    reinvestment_rate,
    fcfe_10q,
    fcff_10q,
    buyback
]

# Juntando todos os indicadores em um df
amd_indicadores_10k = pd.concat(lst_indicadores, axis=1)

# Nomes das colunas
nomes_col = [
    'P/L Damodaran', 
    'P/L', 
    'L/P', 
    'EBITDA', 
    'Margem Líquida',
    'P/VPA',
    'Valor de Mercado',
    'Dívida Bruta',
    'Caixa e Equivalentes',
    'Dívida Líquida',
    'Dívida Líquida/EBITDA',
    'Dívida Líquida/PL',
    'EV/EBITDA',
    'ROE',
    'ROIC',
    'FCO',
    'FCI',
    'FCF',
    'Free Cash Flow',
    'Capex',
    'Net Capex',
    'R&D',
    'Adj Net Capex',
    'Working Capital',
    'Reinvestment Rate',
    'FCFE',
    'FCFF',
    'Buyback'
]

# Atribuindo os novos nomes das colunas do df
amd_indicadores_10k.columns = nomes_col
amd_indicadores_10k

#### Indicadores 10-Q

In [ ]:
# Lendo os arquivos 10q
amd_bs_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//amd//amd_10q.xlsx', 
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

amd_is_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//amd//amd_10q.xlsx', 
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

amd_cf_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//amd//amd_10q.xlsx', 
    sheet_name='cash_flow', 
    index_col='Unnamed: 0'
)

# Aplicando o ajuste nas colunas
amd_bs_10q.columns = [f_sec.ajustar_mes_03_06_09(col) for col in amd_bs_10q.columns]
amd_is_10q.columns = [f_sec.ajustar_mes_03_06_09(col) for col in amd_is_10q.columns]
amd_cf_10q.columns = [f_sec.ajustar_mes_03_06_09(col) for col in amd_cf_10q.columns]

In [ ]:
# Preços da AMD
amd_close = yf.download('AMD', auto_adjust=True, multi_level_index=False, start='2021-01-01')['Close']

# Selecionando o último preço dos meses de Março, Junho e Setembro
lst_years = [
    '2021', '2022', '2023', '2024',
    '2025',
]
lst_months = ['03', '06', '09']
last_close_months = []

for year in lst_years:
    for month in lst_months:
        last_close = amd_close[f'{year}-{month}'].dropna().iloc[-1]  # Dropna para lidar com meses sem dados
        last_close_months.append(last_close)

# Criando uma lista em que junte os meses e os anos para que seja o index da serie pandas
lst_idx = []

for year in lst_years:
    for month in lst_months:
        lst_idx.append(f"{month}-{year}")

# Transformando em uma série pandas
amd_close_months = pd.Series(last_close_months, index=lst_idx)

# Revertendo o index da série p/ ficar igual com os dos indicadores
amd_close_months = amd_close_months.reindex(index=amd_close_months.index[::-1])

# No ano mais atual, como ainda ele não acabou, não consigo fazer o loop com todos meses (Março, Junho e Setembro)
actual_year_int = datetime.now().year  
actual_year_str = str(actual_year_int) 

lst_actual_months = [
    # '03',
    # '06', 
    #'09'
] 

last_close_actual_months = []

for month in lst_actual_months:
    last_close = amd_close[f'{actual_year_str}-{month}'].dropna().iloc[-1]  # Dropna para lidar com meses sem dados
    last_close_actual_months.append(last_close)

# Criando uma lista em que junte os meses e os anos para que seja o index da serie pandas
lst_idx_actual = []

for month in lst_actual_months:
    lst_idx_actual.append(f"{month}-{actual_year_str}")

# Transformando em uma série pandas
amd_close_months_atual = pd.Series(last_close_actual_months, index=lst_idx_actual)

# Revertendo o index da série p/ ficar igual com os dos indicadores
amd_close_months_atual = amd_close_months_atual.reindex(index=amd_close_months_atual.index[::-1])

# Juntando as duas series pandas
amd_close_months = pd.concat([amd_close_months_atual, amd_close_months])

# Arredondamento de duas casas decimais
amd_close_months = round(amd_close_months, 2)

# Não realizou split recentemente (https://br.investing.com/equities/adv-micro-device-historical-data-splits)

# Substituindo o index para ficar igual ao dos dados fundamentalistas
amd_close_months.index = amd_bs_10q.columns[:-3]

amd_close_months

In [ ]:
# Criando uma lista só para fazer a indexação em que o ano mais recente está no começo
idx_10q = amd_bs_10q.columns

# Valor de Mercado
vm_10q = f_eua.indicador_vm(ten_k=False, price=amd_close_months, df_is=amd_is_10q)

# LPA
lpa_acum = f_eua.indicador_lpa(ten_k=False, df_is=amd_is_10q, idx=idx_10q)[1]
lpa_10q = f_eua.indicador_lpa(ten_k=False, df_is=amd_is_10q, idx=idx_10q)[0]

# P/L Damodaran
p_lpa_10q_damodaran = f_eua.indicador_p_lpa_2(ten_k=False, vm=vm_10q, df_is=amd_is_10q)

# P/L
p_lpa_10q = f_eua.indicador_p_lpa(ten_k=False, price=amd_close_months, lpa=lpa_acum, idx=idx_10q)

# L/P
lpa_p_10q = f_eua.indicador_lpa_p(ten_k=False, price=amd_close_months, lpa=lpa_acum, idx=idx_10q) 

# Depreciação
depreciacao_10q = f_eua.indicador_depreciacao(ten_k=False, df_cf=amd_cf_10q, first_quarter='03', idx=idx_10q)[0]
depreciacao_10q_sliced = f_eua.indicador_depreciacao(ten_k=False, df_cf=amd_cf_10q, first_quarter='03', idx=idx_10q)[1]
depreciacao_10q_acum = f_eua.indicador_depreciacao(ten_k=False, df_cf=amd_cf_10q, first_quarter='03', idx=idx_10q)[2]

# EBITDA
ebitda_10q = f_eua.indicador_ebitda(ten_k=False, df_is=amd_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[0]
ebitda_10q_sliced = f_eua.indicador_ebitda(ten_k=False, df_is=amd_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[1]
ebitda_10q_acum = f_eua.indicador_ebitda(ten_k=False, df_is=amd_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[2]

# Margem Líquida
m_liq_10q = f_eua.indicador_ml(ten_k=False, df_is=amd_is_10q)

# VPA
vpa_10q = f_eua.indicador_vpa(ten_k=False, df_bs=amd_bs_10q, df_is=amd_is_10q)

# P/VPA
p_vpa_10q = f_eua.indicador_p_vpa(ten_k=False, price=amd_close_months, vpa=vpa_10q, idx=idx_10q)

# Caixa e Equivalentes
lst_caixa = ['Cash and Cash Equivalents', 'Marketable Securities']
caixa_10q = f_eua.indicador_caixa(ten_k=False, df_bs=amd_bs_10q, lst_itens_caixa=lst_caixa)

# Dívida Bruta
lst_divida = ['Short-term Debt', 'Long-term Debt, Excluding Current Maturities', 'Operating Lease, Liability, Noncurrent']
div_bruta_10q = f_eua.indicador_divida_bruta(ten_k=False, df_bs=amd_bs_10q, lst_itens_divida=lst_divida)

# Dívida Líquida
div_liq_10q = f_eua.indicador_divida_liquida(ten_k=False, caixa=caixa_10q, div_bruta=div_bruta_10q, idx=idx_10q)

# Dívida Líquida / EBITDA
dl_ebitda_10q =  f_eua.indicador_dl_ebitda(ten_k=False, ebitda=ebitda_10q_acum, div_liq=div_liq_10q, idx=idx_10q)

# Dívida Líquida / PL
dl_pl_10q = f_eua.indicador_dl_pl(ten_k=False, div_liq=div_liq_10q, df_bs=amd_bs_10q) 

# EV/EBITDA
ev_ebitda_10q = f_eua.indicador_ev_ebitda(vm=vm_10q, div_liq=div_liq_10q, ebitda=ebitda_10q_acum)

# ROE
roe_10q = f_eua.indicador_roe(ten_k=False, df_is=amd_is_10q, df_bs=amd_bs_10q, idx=idx_10q)

# ROIC 
roic_10q = f_eua.indicador_roic(ten_k=False, df_is=amd_is_10q, df_bs=amd_bs_10q, div_bruta=div_bruta_10q, idx=idx_10q)

# FCO
fco_10q = f_eua.indicador_fco(ten_k=False, df_cf=amd_cf_10q, first_quarter='03') 

# FCI
fci_10q = f_eua.indicador_fci(ten_k=False, df_cf=amd_cf_10q, first_quarter='03') 

# FCF
fcf_10q = f_eua.indicador_fcf(ten_k=False, df_cf=amd_cf_10q, first_quarter='03') 

# CAPEX
lst_capex_10q = ['Purchases Of Property And Equipment And Intangible Assets']

capex_10q = f_eua.indicador_capex(ten_k=False, df_cf=amd_cf_10q, first_quarter='03', lst_itens=lst_capex_10q, idx=idx_10q)[0]
capex_10q_sliced = f_eua.indicador_capex(ten_k=False, df_cf=amd_cf_10q, first_quarter='03', lst_itens=lst_capex_10q, idx=idx_10q)[1]
capex_10q_acum = f_eua.indicador_capex(ten_k=False, df_cf=amd_cf_10q, first_quarter='03', lst_itens=lst_capex_10q, idx=idx_10q)[2]

# Free Cash Flow
free_cash_flow_10q = f_eua.indicador_free_cash_flow(fco=fco_10q, capex=capex_10q_sliced)

# Net CAPEX
net_capex_10q = f_eua.indicador_net_capex(capex=capex_10q_acum, depreciation=depreciacao_10q_acum)

# R&D
rd_expense_10q = f_eua.indicador_rd(ten_k=False, df_is=amd_is_10q, idx=idx_10q)[0]
rd_expense_10q_sliced = f_eua.indicador_rd(ten_k=False, df_is=amd_is_10q, idx=idx_10q)[1]
rd_expense_10q_acum = f_eua.indicador_rd(ten_k=False, df_is=amd_is_10q, idx=idx_10q)[2]

# Net CAPEX ajustado
adj_net_capex_10q = f_eua.indicador_adj_net_capex(net_capex=net_capex_10q, rd_expense=rd_expense_10q_acum)

# Working capital
working_capital_10q = f_eua.indicador_working_capital(ten_k=False, df_bs=amd_bs_10q)

# Change in non cash working capital
lst_itens_change_non_cash_wc = [
    'Deferred Income Taxes and Tax Credits',
    'Increase (Decrease) in Accounts Receivable',
    'Increase (Decrease) in Inventories',
    'Increase (Decrease) in Prepaid Expense and Other Assets',
    'Increase (Decrease) in Due to Related Parties',
    'Increase (Decrease) in Accounts Payable',
    'Increase (Decrease) in Accrued Liabilities and Other Operating Liabilities'
]

change_non_cash_wc_10q = f_eua.indicador_change_non_cash_wc(ten_k=False, df_cf=amd_cf_10q, lst_itens=lst_itens_change_non_cash_wc, first_quarter='03')[0]
change_non_cash_wc_10q_sliced = f_eua.indicador_change_non_cash_wc(ten_k=False, df_cf=amd_cf_10q, lst_itens=lst_itens_change_non_cash_wc, first_quarter='03')[1]

# Reinvestment rate
reinvestment_rate_10q = f_eua.indicador_reinvestment_rate(ten_k=False, net_capex=adj_net_capex_10q, change_non_cash_wc=change_non_cash_wc_10q, df_is=amd_is_10q, idx=idx_10q)

# New borrowing
lst_itens_new_borrowing = ['Proceeds from Issuance of Long-term Debt']

new_borrowing_10q = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=amd_cf_10q, first_quarter='03', idx=idx_10q)[0]
new_borrowing_10q_sliced = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=amd_cf_10q, first_quarter='03', idx=idx_10q)[1]
new_borrowing_10q_acum = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=amd_cf_10q, first_quarter='03', idx=idx_10q)[2]

# Debt paid
lst_itens_debt_paid = ['Repayments of Debt', 'Repayments of Commercial Paper']

debt_paid_10q = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=amd_cf_10q, first_quarter='03', idx=idx_10q)[0]
debt_paid_10q_sliced = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=amd_cf_10q, first_quarter='03', idx=idx_10q)[1]
debt_paid_10q_acum = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=amd_cf_10q, first_quarter='03', idx=idx_10q)[2]

# Taxes on operating income
taxes_operating_income_10q = f_eua.indicador_taxes_operating_income(ten_k=False, df_is=amd_is_10q, idx=idx_10q)

# FCFE
fcfe_10q = f_eua.indicador_fcfe(
    ten_k=False, 
    depreciacao=depreciacao_10q_acum, 
    capex=capex_10q_acum, 
    change_non_cash_wc=change_non_cash_wc_10q_sliced, 
    new_borrowing=new_borrowing_10q_acum, 
    debt_paid=debt_paid_10q_acum, 
    df_is=amd_is_10q,
    idx=idx_10q
)

# FCFF
fcff_10q = f_eua.indicador_fcff(
        ten_k=False, 
        taxes_operating_income=taxes_operating_income_10q,
        depreciacao=depreciacao_10q_acum, 
        capex=capex_10q_acum, 
        change_non_cash_wc=change_non_cash_wc_10q_sliced, 
        df_is=amd_is_10q,
        idx=idx_10q
)

# DPA e Payout - AMD não paga dividendo

# Buyback
buyback_10q = f_eua.indicador_buyback(ten_k=False, df_cf=amd_cf_10q,  first_quarter='03')

# Lista com todos os indicadores
lst_indicadores_10q = [
    p_lpa_10q_damodaran,
    p_lpa_10q,
    lpa_p_10q,  
    ebitda_10q_sliced, 
    m_liq_10q,
    p_vpa_10q,
    vm_10q,
    div_bruta_10q,
    caixa_10q,
    div_liq_10q,
    dl_ebitda_10q,
    dl_pl_10q,
    ev_ebitda_10q,
    roe_10q,
    roic_10q,
    fco_10q,
    fci_10q,
    fcf_10q,
    free_cash_flow_10q,
    capex_10q_sliced,
    net_capex_10q,
    rd_expense_10q_sliced,
    adj_net_capex_10q,
    working_capital_10q,
    reinvestment_rate_10q,
    fcfe_10q,
    fcff_10q,
    buyback_10q
]

# Juntando todos os indicadores em um df
amd_indicadores_10q = pd.concat(lst_indicadores_10q, axis=1)

# Nomes das colunas
nomes_col = [
    'P/L Damodaran', 
    'P/L', 
    'L/P', 
    'EBITDA', 
    'Margem Líquida',
    'P/VPA',
    'Valor de Mercado',
    'Dívida Bruta',
    'Caixa e Equivalentes',
    'Dívida Líquida',
    'Dívida Líquida/EBITDA',
    'Dívida Líquida/PL',
    'EV/EBITDA',
    'ROE',
    'ROIC',
    'FCO',
    'FCI',
    'FCF',
    'Free Cash Flow',
    'Capex',
    'Net Capex',
    'R&D',
    'Adj Net Capex',
    'Working Capital',
    'Reinvestment Rate',
    'FCFE',
    'FCFF',
    'Buyback'
]

# Atribuindo os novos nomes das colunas do df
amd_indicadores_10q.columns = nomes_col
amd_indicadores_10q

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//semicondutores//amd//amd_indicators.xlsx') as writer:
    amd_indicadores_10k.to_excel(writer, sheet_name='indicators_10k')
    amd_indicadores_10q.to_excel(writer, sheet_name='indicators_10q')

## Intel

* 1Q -> Lançamento em Março (03);
* 2Q -> Lançamento em Junho (06);
* 3Q -> Lançamento em Setembro (09);
* 10K -> Lançamento em Dezembro (12).

In [ ]:
ticker = 'intc'
f_sec.cik_matching_ticker(ticker)

### Scraping Intel 10-K

In [ ]:
# Número dos documentos que contém a DRE, BP e FC 
acc_documents = f_sec.get_filtered_filings(ticker=ticker, ten_k=True, just_accession_numbers=True)

# Trocando os '-' para '' das strings ('0001045810-24-000264' -> '000104581024000264')
dict_acc_num = {k: v.replace('-', '') for k, v in acc_documents.items()}

# Transformando o dicionário em um df
df_lst_acc_num_intel = pd.DataFrame.from_dict(dict_acc_num, orient='index', columns=['acc_num'])

# Adicionando o 'acc_num' de 2017
df_acc_num = pd.DataFrame(
    {
        'acc_num': [
            '000005086318000007', # 2017
        ]
    },
    index=pd.to_datetime([
        '2017-12-30',
    ])
)

# Concatenando os dfs
df_lst_acc_num_intel = pd.concat([df_lst_acc_num_intel, df_acc_num])

# Transformando em datetime o index
df_lst_acc_num_intel.index = pd.to_datetime(df_lst_acc_num_intel.index, format='%Y-%m-%d')

df_lst_acc_num_intel

In [ ]:
# Rótulos
labels_dict = f_sec.get_label_dictionary(ticker, headers=f_sec.header)

In [ ]:
# Fazendo o scraping do balance sheet (10-K)
balance_sheet_lst = []

for acc_num in df_lst_acc_num_intel['acc_num']:

    # Balance sheet
    balance_sheet = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='balance_sheet')
    balance_sheet = f_sec.rename_statement(balance_sheet, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    balance_sheet = balance_sheet.iloc[:, 0]
    balance_sheet_lst.append(balance_sheet)

# Juntando todos os dfs que estão em listas em apenas um df
df_balance_sheet_10k = pd.concat(balance_sheet_lst, axis=1)

# Substituindo os NaN por 0
df_balance_sheet_10k = df_balance_sheet_10k.fillna(0)

# A linha 'Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization' nas colunas '2022-12-31', '2021-12-25', '2020-12-26', '2019-12-28', '2018-12-29' e	'2017-12-30' estão zeradas
# A informação está na linha 'Property, Plant and Equipment, Net'
df_balance_sheet_10k.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2022-12-31'] = df_balance_sheet_10k.loc['Property, Plant and Equipment, Net', '2022-12-31'] 
df_balance_sheet_10k.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2021-12-25'] = df_balance_sheet_10k.loc['Property, Plant and Equipment, Net', '2021-12-25'] 
df_balance_sheet_10k.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2020-12-26'] = df_balance_sheet_10k.loc['Property, Plant and Equipment, Net', '2020-12-26'] 
df_balance_sheet_10k.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2019-12-28'] = df_balance_sheet_10k.loc['Property, Plant and Equipment, Net', '2019-12-28'] 
df_balance_sheet_10k.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2018-12-29'] = df_balance_sheet_10k.loc['Property, Plant and Equipment, Net', '2018-12-29'] 
df_balance_sheet_10k.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2017-12-30'] = df_balance_sheet_10k.loc['Property, Plant and Equipment, Net', '2017-12-30'] 

# A linha 'intc_EquityInvestments' nas colunas '2020-12-26', '2019-12-28', '2018-12-29' e	'2017-12-30' estão zeradas
# A informação está na linha 'Available-for-sale Securities, Equity Securities, Noncurrent'
df_balance_sheet_10k.loc['intc_EquityInvestments', '2020-12-26'] = df_balance_sheet_10k.loc['Available-for-sale Securities, Equity Securities, Noncurrent', '2020-12-26'] 
df_balance_sheet_10k.loc['intc_EquityInvestments', '2019-12-28'] = df_balance_sheet_10k.loc['Available-for-sale Securities, Equity Securities, Noncurrent', '2019-12-28'] 
df_balance_sheet_10k.loc['intc_EquityInvestments', '2018-12-29'] = df_balance_sheet_10k.loc['Available-for-sale Securities, Equity Securities, Noncurrent', '2018-12-29'] 
df_balance_sheet_10k.loc['intc_EquityInvestments', '2017-12-30'] = df_balance_sheet_10k.loc['Available-for-sale Securities, Equity Securities, Noncurrent', '2017-12-30'] 

# A linha 'Deferred Income Tax Liabilities, Net' nas colunas '2020-12-26', '2019-12-28', '2018-12-29' e	'2017-12-30' estão zeradas
# A informação está na linha 'Deferred Tax Liabilities, Net, Noncurrent'
df_balance_sheet_10k.loc['Deferred Income Tax Liabilities, Net', '2020-12-26'] = df_balance_sheet_10k.loc['Deferred Tax Liabilities, Net, Noncurrent', '2020-12-26'] 
df_balance_sheet_10k.loc['Deferred Income Tax Liabilities, Net', '2019-12-28'] = df_balance_sheet_10k.loc['Deferred Tax Liabilities, Net, Noncurrent', '2019-12-28'] 
df_balance_sheet_10k.loc['Deferred Income Tax Liabilities, Net', '2018-12-29'] = df_balance_sheet_10k.loc['Deferred Tax Liabilities, Net, Noncurrent', '2018-12-29'] 
df_balance_sheet_10k.loc['Deferred Income Tax Liabilities, Net', '2017-12-30'] = df_balance_sheet_10k.loc['Deferred Tax Liabilities, Net, Noncurrent', '2017-12-30'] 

# A linha 'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents' nas colunas '2018-12-29' e '2017-12-30' estão zeradas
# A informação está na linha 'Cash and Cash Equivalents, at Carrying Value'
df_balance_sheet_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2018-12-29'] = df_balance_sheet_10k.loc['Cash and Cash Equivalents, at Carrying Value', '2018-12-29'] 
df_balance_sheet_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2017-12-30'] = df_balance_sheet_10k.loc['Cash and Cash Equivalents, at Carrying Value', '2017-12-30'] 

# Removendo as linhas que movemos os seus itens para as linhas corretas
df_balance_sheet_10k = df_balance_sheet_10k.drop('Property, Plant and Equipment, Net')

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_balance_sheet_10k = df_balance_sheet_10k.drop([
    'us-gaap_AssetsCurrentAbstract',
    'Preferred Stock, Value, Issued',
    'Contract with Customer, Liability, Current',
    'Disposal Group, Including Discontinued Operation, Liabilities, Current',
    'Disposal Group, Including Discontinued Operation, Assets, Current',
    "Stockholders' Equity Attributable to Noncontrolling Interest",
    "Stockholders' Equity, Including Portion Attributable to Noncontrolling Interest",
    'Deferred Revenue, Current',
    'Cash and Cash Equivalents, at Carrying Value'
])

# Renomeando as linhas do índice que estão sem formato 
df_balance_sheet_10k = df_balance_sheet_10k.rename(index={
    'intc_EquityInvestments': 'Equity Investments'
})

df_balance_sheet_10k

In [ ]:
# Fazendo o scraping do income statement (10-K)
income_statement_lst = []

for acc_num in df_lst_acc_num_intel['acc_num']:

    # Income statement
    income_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='income_statement')
    income_statement = f_sec.rename_statement(income_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    income_statement = income_statement.iloc[:, 0]
    income_statement_lst.append(income_statement)

# Juntando todos os dfs que estão em listas em apenas um df
df_income_statement_10k = pd.concat(income_statement_lst, axis=1)

# Substituindo os NaN por 0
df_income_statement_10k = df_income_statement_10k.fillna(0)

# A linha 'Equity Securities, FV-NI, Gain (Loss)' na coluna '2024-12-28', '2021-12-25', '2020-12-26', '2019-12-28', '2018-12-29' e '2017-12-30' estão zeradas
# As informações estão nsa linhas 'intc_GainsLossesOnEquityInvestments' e 'Gain (Loss) on Investments' 
df_income_statement_10k.loc['Equity Securities, FV-NI, Gain (Loss)', '2024-12-28'] = df_income_statement_10k.loc['intc_GainsLossesOnEquityInvestments', '2024-12-28'] 
df_income_statement_10k.loc['Equity Securities, FV-NI, Gain (Loss)', '2021-12-25'] = df_income_statement_10k.loc['Gain (Loss) on Investments', '2021-12-25'] 
df_income_statement_10k.loc['Equity Securities, FV-NI, Gain (Loss)', '2021-12-25'] = df_income_statement_10k.loc['Gain (Loss) on Investments', '2021-12-25'] 
df_income_statement_10k.loc['Equity Securities, FV-NI, Gain (Loss)', '2020-12-26'] = df_income_statement_10k.loc['Gain (Loss) on Investments', '2020-12-26'] 
df_income_statement_10k.loc['Equity Securities, FV-NI, Gain (Loss)', '2019-12-28'] = df_income_statement_10k.loc['Gain (Loss) on Investments', '2019-12-28'] 
df_income_statement_10k.loc['Equity Securities, FV-NI, Gain (Loss)', '2018-12-29'] = df_income_statement_10k.loc['Gain (Loss) on Investments', '2018-12-29'] 
df_income_statement_10k.loc['Equity Securities, FV-NI, Gain (Loss)', '2017-12-30'] = df_income_statement_10k.loc['Gain (Loss) on Investments', '2017-12-30'] 

# A linha 'Revenue from Contract with Customer, Excluding Assessed Tax' na coluna '2017-12-30' está zerada
# As informações estão na linha 'Revenue, Net (Deprecated 2018-01-31)'
df_income_statement_10k.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2017-12-30'] = df_income_statement_10k.loc['Revenue, Net (Deprecated 2018-01-31)', '2017-12-30'] 

# Removendo as linhas que movemos os seus itens para as linhas corretas
df_income_statement_10k = df_income_statement_10k.drop([
    'intc_GainsLossesOnEquityInvestments',
    'Gain (Loss) on Investments',
    'Revenue, Net (Deprecated 2018-01-31)',
    'Net Income (Loss), Including Portion Attributable to Noncontrolling Interest',
    'Net Income (Loss) Attributable to Noncontrolling Interest',
    'Depreciation, Depletion and Amortization, Nonproduction'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_income_statement_10k = df_income_statement_10k.drop('us-gaap_IncomeStatementAbstract')

# Arrumando e adicionando alguns números no df
df_income_statement_10k.loc['Earnings Per Share, Diluted', '2025-12-27'] = -0.06
df_income_statement_10k.loc['Equity Securities, FV-NI, Gain (Loss)', '2025-12-27'] = 514

df_income_statement_10k

In [ ]:
# Fazendo o scraping do cash flow statement (10-K)
cash_flow_statement_lst = []

for acc_num in df_lst_acc_num_intel['acc_num']:

    # Cash flow statement
    cash_flow_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='cash_flow_statement')
    cash_flow_statement = f_sec.rename_statement(cash_flow_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    cash_flow_statement = cash_flow_statement.iloc[:, 0]
    cash_flow_statement_lst.append(cash_flow_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_cash_flow_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in cash_flow_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_cash_flow_10k = pd.concat([df_cash_flow_10k, df], axis=1)

# Substituindo os NaN por 0
df_cash_flow_10k = df_cash_flow_10k.fillna(0)

# A linha 'Net Income (Loss), Including Portion Attributable to Noncontrolling Interest' nas colunas '2021-12-25', '2020-12-26', '2019-12-28' e '2018-12-29' estão zeradas
# As informações estão na linha 'Net Income (Loss) Attributable to Parent'
df_cash_flow_10k.loc['Net Income (Loss), Including Portion Attributable to Noncontrolling Interest', '2021-12-25'] = df_cash_flow_10k.loc['Net Income (Loss) Attributable to Parent', '2021-12-25'] 
df_cash_flow_10k.loc['Net Income (Loss), Including Portion Attributable to Noncontrolling Interest', '2020-12-26'] = df_cash_flow_10k.loc['Net Income (Loss) Attributable to Parent', '2020-12-26'] 
df_cash_flow_10k.loc['Net Income (Loss), Including Portion Attributable to Noncontrolling Interest', '2019-12-28'] = df_cash_flow_10k.loc['Net Income (Loss) Attributable to Parent', '2019-12-28'] 
df_cash_flow_10k.loc['Net Income (Loss), Including Portion Attributable to Noncontrolling Interest', '2018-12-29'] = df_cash_flow_10k.loc['Net Income (Loss) Attributable to Parent', '2018-12-29'] 
df_cash_flow_10k.loc['Net Income (Loss), Including Portion Attributable to Noncontrolling Interest', '2017-12-30'] = df_cash_flow_10k.loc['Net Income (Loss) Attributable to Parent', '2017-12-30'] 

# A linha 'Restructuring Charges' nas colunas '2021-12-25' e '2017-12-30' estão zeradas
# As informações estão na linha 'Restructuring, Settlement and Impairment Provisions'
df_cash_flow_10k.loc['Restructuring Charges', '2021-12-25'] = df_cash_flow_10k.loc['Restructuring, Settlement and Impairment Provisions', '2021-12-25']  
df_cash_flow_10k.loc['Restructuring Charges', '2017-12-30'] = df_cash_flow_10k.loc['Restructuring, Settlement and Impairment Provisions', '2017-12-30']  

# A linha 'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents' nas colunas '2018-12-29' e '2017-12-30' estão zeradas
# As informações estão na linha 'Cash and Cash Equivalents, at Carrying Value'
df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2018-12-29'] = df_cash_flow_10k.loc['Cash and Cash Equivalents, at Carrying Value', '2018-12-29'] 
df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2017-12-30'] = df_cash_flow_10k.loc['Cash and Cash Equivalents, at Carrying Value', '2017-12-30'] 

# A linha 'Net Cash Provided by (Used in) Operating Activities' nas colunas '2018-12-29' e '2017-12-30' estão zeradas
# As informações estão na linha 'Net Cash Provided by (Used in) Operating Activities, Continuing Operations'
df_cash_flow_10k.loc['Net Cash Provided by (Used in) Operating Activities', '2018-12-29'] = df_cash_flow_10k.loc['Net Cash Provided by (Used in) Operating Activities, Continuing Operations', '2018-12-29'] 
df_cash_flow_10k.loc['Net Cash Provided by (Used in) Operating Activities', '2017-12-30'] = df_cash_flow_10k.loc['Net Cash Provided by (Used in) Operating Activities, Continuing Operations', '2017-12-30'] 

# A linha 'Net Cash Provided by (Used in) Investing Activities' nas colunas '2018-12-29' e '2017-12-30' estão zeradas
# As informações estão na linha 'Net Cash Provided by (Used in) Investing Activities, Continuing Operations'
df_cash_flow_10k.loc['Net Cash Provided by (Used in) Investing Activities', '2018-12-29'] = df_cash_flow_10k.loc['Net Cash Provided by (Used in) Investing Activities, Continuing Operations', '2018-12-29'] 
df_cash_flow_10k.loc['Net Cash Provided by (Used in) Investing Activities', '2017-12-30'] = df_cash_flow_10k.loc['Net Cash Provided by (Used in) Investing Activities, Continuing Operations', '2017-12-30'] 

# A linha 'Net Cash Provided by (Used in) Financing Activities' nas colunas '2018-12-29' e '2017-12-30' estão zeradas
# As informações estão na linha 'Net Cash Provided by (Used in) Financing Activities, Continuing Operations'
df_cash_flow_10k.loc['Net Cash Provided by (Used in) Financing Activities', '2018-12-29'] = df_cash_flow_10k.loc['Net Cash Provided by (Used in) Financing Activities, Continuing Operations', '2018-12-29'] 
df_cash_flow_10k.loc['Net Cash Provided by (Used in) Financing Activities', '2017-12-30'] = df_cash_flow_10k.loc['Net Cash Provided by (Used in) Financing Activities, Continuing Operations', '2017-12-30'] 

# A linha 'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect' nas colunas '2018-12-29' e '2017-12-30' estão zeradas
# As informações estão na linha 'Cash and Cash Equivalents, Period Increase (Decrease)'
df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect', '2018-12-29'] = df_cash_flow_10k.loc['Cash and Cash Equivalents, Period Increase (Decrease)', '2018-12-29'] 
df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect', '2017-12-30'] = df_cash_flow_10k.loc['Cash and Cash Equivalents, Period Increase (Decrease)', '2017-12-30'] 

# A linha 'Payments to Acquire Debt Securities, Available-for-sale' na coluna '2017-12-30' está zerada
# As informações estão na linha 'Payments to Acquire Available-for-sale Securities'
df_cash_flow_10k.loc['Payments to Acquire Debt Securities, Available-for-sale', '2017-12-30'] = df_cash_flow_10k.loc['Payments to Acquire Available-for-sale Securities', '2017-12-30'] 

# A linha 'Proceeds from Sale of Debt Securities, Available-for-sale' na coluna '2017-12-30' está zerada
# As informações estão na linha 'Proceeds from Sale of Available-for-sale Securities'
df_cash_flow_10k.loc['Proceeds from Sale of Debt Securities, Available-for-sale', '2017-12-30'] = df_cash_flow_10k.loc['Proceeds from Sale of Available-for-sale Securities', '2017-12-30'] 

# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_cash_flow_10k = df_cash_flow_10k.drop([
    'Net Income (Loss) Attributable to Parent',
    'Restructuring, Settlement and Impairment Provisions',
    'Cash and Cash Equivalents, at Carrying Value',
    'Payments to Acquire Available-for-sale Securities',
    'Proceeds from Sale of Available-for-sale Securities',
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_cash_flow_10k = df_cash_flow_10k.drop([
    'us-gaap_StatementOfCashFlowsAbstract',
    'Transfer to Investments',
    'Net Cash Provided by (Used in) Operating Activities, Continuing Operations',
    'Net Cash Provided by (Used in) Investing Activities, Continuing Operations',
    'Net Cash Provided by (Used in) Financing Activities, Continuing Operations',
    'Cash and Cash Equivalents, Period Increase (Decrease)',
    'Gain (Loss) on Sale of Equity Investments',
    'Payment, Tax Withholding, Share-based Payment Arrangement',
    'Gain (Loss) on Extinguishment of Debt',
    'Proceeds from (Repayments of) Short-term Debt',
])

# Arrumando e adicionando alguns números no df
df_cash_flow_10k.loc['Increase (Decrease) in Equity Securities, FV-NI', '2017-12-30'] = -2583
df_cash_flow_10k.loc['Deferred Income Taxes and Tax Credits', '2023-12-30'] = -2033
df_cash_flow_10k.loc['Deferred Income Taxes and Tax Credits', '2022-12-31'] = -5148
df_cash_flow_10k.loc['intc_AssetImpairmentChargesAndGainLossOnDisposalOfPropertyPlantAndEquipment', '2023-12-30'] = 33
df_cash_flow_10k.loc['intc_AssetImpairmentChargesAndGainLossOnDisposalOfPropertyPlantAndEquipment', '2022-12-31'] = 301
df_cash_flow_10k.loc['intc_ProceedsFromCapitalGrants', '2022-12-31'] = 246
df_cash_flow_10k.loc['Payments to Acquire Businesses, Net of Cash Acquired', '2023-12-30'] = -13
df_cash_flow_10k.loc['Payments to Acquire Businesses, Net of Cash Acquired', '2022-12-31'] = -681
df_cash_flow_10k.loc['Proceeds from Equity Method Investment, Distribution, Return of Capital', '2017-12-30'] = 5236
df_cash_flow_10k.loc['Increase (Decrease) in Contract with Customer, Liability', '2017-12-30'] = 1105

# Renomeando as linhas do índice que estão sem formato 
df_cash_flow_10k = df_cash_flow_10k.rename(index={
    'intc_AssetImpairmentChargesAndGainLossOnDisposalOfPropertyPlantAndEquipment': 'Asset Impairment Charges And Gain Loss On Disposal Of Property Plant And Equipment',
    'intc_ProceedsFromCapitalGrants': 'Proceeds From Capital Grants',
    'intc_DivestitureAdditionsToPropertyPlantAndEquipmentHeldForSale': 'Divestiture Additions To Property Plant And Equipment Held For Sale',
    'intc_PaymentsToAcquirePropertyPlantAndEquipmentFinancingActivities': 'Payments To Acquire Property Plant And Equipment Financing Activities',
    'intc_ProceedsFromIssuanceOfCommonStockWarrantsAndEscrowedShares': 'Proceeds From Issuance Of Common Stock Warrants And Escrowed Shares',
})

df_cash_flow_10k

In [ ]:
# Arrumando alguns index dos dfs para ficar padronizado com os index utilizados nas funções p/ calcular os indicadores fundamentalistas
df_balance_sheet_10k = df_balance_sheet_10k.rename(index={
    'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents': 'Cash and Cash Equivalents',
    'Debt Securities, Available-for-sale, Current': 'Marketable Securities',
    "Stockholders' Equity Attributable to Parent": 'Total Shareholders Equity'
})

df_income_statement_10k = df_income_statement_10k.rename(index={
    'Revenue from Contract with Customer, Excluding Assessed Tax': 'Revenues',
    'Research and Development Expense': 'Research and Development',
    'Operating Income (Loss)': 'Operating Income',
    'Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest': 'Income Before Provision for Income Taxes',
    'Income Tax Expense (Benefit)': 'Provision for Income Taxes',
    'Net Income (Loss) Attributable to Parent': 'Net Income',
    'Weighted Average Number of Shares Outstanding, Basic': 'Number of Shares - Basic',
    'Weighted Average Number of Shares Outstanding, Diluted': 'Number of Shares - Diluted'
})

df_cash_flow_10k = df_cash_flow_10k.rename(index={
    'Depreciation': 'Depreciation, Depletion and Amortization',
    'Payments to Acquire Property, Plant, and Equipment': 'Purchases Of Property And Equipment And Intangible Assets',
    'Payments of Ordinary Dividends, Common Stock': 'Payments of Dividends'
})

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//semicondutores//intel//intc_10k.xlsx') as writer:
    df_balance_sheet_10k.to_excel(writer, sheet_name='balance_sheet')
    df_income_statement_10k.to_excel(writer, sheet_name='income_statement')
    df_cash_flow_10k.to_excel(writer, sheet_name='cash_flow')

### Scraping Intel 10-Q

In [ ]:
# Número dos documentos que contém a DRE, BP e FC 
acc_documents_10q = f_sec.get_filtered_filings(ticker=ticker, ten_k=False, just_accession_numbers=True)

# Trocando os '-' para '' das strings ('0001045810-24-000264' -> '000104581024000264')
dict_acc_num_10q = {k: v.replace('-', '') for k, v in acc_documents_10q.items()}

# Transformando o dicionário em um df
df_lst_acc_num_10q_intel = pd.DataFrame.from_dict(dict_acc_num_10q, orient='index', columns=['acc_num'])

# Transformando em datetime o index
df_lst_acc_num_10q_intel.index = pd.to_datetime(df_lst_acc_num_10q_intel.index, format='%Y-%m-%d')
df_lst_acc_num_10q_intel

In [ ]:
# Selecionando os 'acc_num' dos 10-Q a partir de 2020 p/ iteração
df_lst_acc_num_10q_intel_sliced = df_lst_acc_num_10q_intel.loc['2020-03-28':]
df_lst_acc_num_10q_intel_sliced

In [ ]:
# Rótulos
labels_dict = f_sec.get_label_dictionary(ticker, headers=f_sec.header)

In [ ]:
# Fazendo o scraping do balance sheet (10-Q)
balance_sheet_lst = []

for acc_num in df_lst_acc_num_10q_intel_sliced['acc_num']:

    # Balance sheet
    balance_sheet = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='balance_sheet')
    balance_sheet = f_sec.rename_statement(balance_sheet, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    balance_sheet = balance_sheet.iloc[:, 0]
    balance_sheet_lst.append(balance_sheet)

# Juntando todos os dfs que estão em listas em apenas um df
df_balance_sheet_10q = pd.concat(balance_sheet_lst, axis=1)

# Substituindo os NaN por 0
df_balance_sheet_10q = df_balance_sheet_10q.fillna(0)

# A linha 'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents' nas colunas '2021-09-25', '2021-06-26', '2021-03-27', '2020-09-26', '2020-06-27' e '2020-03-28' estão zeradas
# As informações estão na linha 'Cash and Cash Equivalents, at Carrying Value'
df_balance_sheet_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2021-09-25'] = df_balance_sheet_10q.loc['Cash and Cash Equivalents, at Carrying Value', '2021-09-25'] 
df_balance_sheet_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2021-06-26'] = df_balance_sheet_10q.loc['Cash and Cash Equivalents, at Carrying Value', '2021-06-26'] 
df_balance_sheet_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2021-03-27'] = df_balance_sheet_10q.loc['Cash and Cash Equivalents, at Carrying Value', '2021-03-27'] 
df_balance_sheet_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2020-09-26'] = df_balance_sheet_10q.loc['Cash and Cash Equivalents, at Carrying Value', '2020-09-26'] 
df_balance_sheet_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2020-06-27'] = df_balance_sheet_10q.loc['Cash and Cash Equivalents, at Carrying Value', '2020-06-27'] 
df_balance_sheet_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2020-03-28'] = df_balance_sheet_10q.loc['Cash and Cash Equivalents, at Carrying Value', '2020-03-28'] 

# A linha 'Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization' nas colunas '2021-09-25', '2021-06-26', '2021-03-27', '2020-09-26', '2020-06-27' e '2020-03-28' estão zeradas
# As informações estão na linha 'Property, Plant and Equipment, Net'
df_balance_sheet_10q.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2021-09-25'] = df_balance_sheet_10q.loc['Property, Plant and Equipment, Net', '2021-09-25'] 
df_balance_sheet_10q.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2021-06-26'] = df_balance_sheet_10q.loc['Property, Plant and Equipment, Net', '2021-06-26'] 
df_balance_sheet_10q.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2021-03-27'] = df_balance_sheet_10q.loc['Property, Plant and Equipment, Net', '2021-03-27'] 
df_balance_sheet_10q.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2020-09-26'] = df_balance_sheet_10q.loc['Property, Plant and Equipment, Net', '2020-09-26'] 
df_balance_sheet_10q.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2020-06-27'] = df_balance_sheet_10q.loc['Property, Plant and Equipment, Net', '2020-06-27'] 
df_balance_sheet_10q.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2020-03-28'] = df_balance_sheet_10q.loc['Property, Plant and Equipment, Net', '2020-03-28'] 

# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_balance_sheet_10q = df_balance_sheet_10q.drop([
    'Cash and Cash Equivalents, at Carrying Value',
    'Property, Plant and Equipment, Net',
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_balance_sheet_10q = df_balance_sheet_10q.drop([
    'us-gaap_AssetsCurrentAbstract',
    'Temporary Equity, Carrying Amount, Attributable to Parent',
    'Preferred Stock, Value, Issued',
    "Stockholders' Equity Attributable to Noncontrolling Interest",
    "Stockholders' Equity, Including Portion Attributable to Noncontrolling Interest",
    
])

# Arrumando e adicionando alguns números no df
df_balance_sheet_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2025-06-28'] = 9643 	

# Renomeando as linhas do índice que estão sem formato 
df_balance_sheet_10q = df_balance_sheet_10q.rename(index={
    'intc_EquityInvestments': 'Equity Investments',
    'intc_AssetsHeldForSale': 'Assets Held For Sale',
})

df_balance_sheet_10q

In [ ]:
# Fazendo o scraping do income statement (10-Q)
income_statement_lst = []

for acc_num in df_lst_acc_num_10q_intel_sliced['acc_num']:

    # Income statement
    income_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='income_statement')
    income_statement = f_sec.rename_statement(income_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    income_statement = income_statement.iloc[:, 0]
    income_statement_lst.append(income_statement)

# Juntando todos os dfs que estão em listas em apenas um df
df_income_statement_10q = pd.concat(income_statement_lst, axis=1)

# Substituindo os NaN por 0
df_income_statement_10q = df_income_statement_10q.fillna(0)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_income_statement_10q = df_income_statement_10q.drop([
    'us-gaap_IncomeStatementAbstract',
    'Net Income (Loss), Including Portion Attributable to Noncontrolling Interest',
    'Net Income (Loss) Attributable to Noncontrolling Interest'
])

# Arrumando e adicionando alguns números no df
df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Diluted', '2025-06-28'] = 4369
df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Diluted', '2025-03-29'] = 4343
df_income_statement_10q.loc['Earnings Per Share, Diluted', '2025-09-27'] = 0.90
df_income_statement_10q.loc['Earnings Per Share, Diluted', '2025-06-28'] = -0.67
df_income_statement_10q.loc['Earnings Per Share, Diluted', '2025-03-29'] = -0.19
df_income_statement_10q.loc['Earnings Per Share, Diluted', '2024-06-29'] = -0.38
df_income_statement_10q.loc['Earnings Per Share, Diluted', '2024-03-30'] = -0.09
df_income_statement_10q.loc['Earnings Per Share, Diluted', '2024-09-28'] = -3.88

df_income_statement_10q

In [ ]:
# Fazendo o scraping do cash flow statement (10-Q)
cash_flow_statement_lst = []

for acc_num in df_lst_acc_num_10q_intel_sliced['acc_num']:

    # Cash flow statement
    cash_flow_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='cash_flow_statement')
    cash_flow_statement = f_sec.rename_statement(cash_flow_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    cash_flow_statement = cash_flow_statement.iloc[:, 0]
    cash_flow_statement_lst.append(cash_flow_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_cash_flow_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in cash_flow_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_cash_flow_10q = pd.concat([df_cash_flow_10q, df], axis=1)

# Substituindo os NaN por 0
df_cash_flow_10q = df_cash_flow_10q.fillna(0)

# A linha 'Net Income (Loss), Including Portion Attributable to Noncontrolling Interest' nas colunas '2022-10-01', '2022-07-02', '2022-04-02', '2021-09-25', '2021-06-26', '2021-03-27', '2020-09-26', '2020-06-27'	e '2020-03-28' estão zeradas
# As informações estão na linha 'Other Depreciation and Amortization'
df_cash_flow_10q.loc['Net Income (Loss), Including Portion Attributable to Noncontrolling Interest', '2022-10-01'] = df_cash_flow_10q.loc['Net Income (Loss) Attributable to Parent', '2022-10-01']
df_cash_flow_10q.loc['Net Income (Loss), Including Portion Attributable to Noncontrolling Interest', '2022-07-02'] = df_cash_flow_10q.loc['Net Income (Loss) Attributable to Parent', '2022-07-02']
df_cash_flow_10q.loc['Net Income (Loss), Including Portion Attributable to Noncontrolling Interest', '2022-04-02'] = df_cash_flow_10q.loc['Net Income (Loss) Attributable to Parent', '2022-04-02']
df_cash_flow_10q.loc['Net Income (Loss), Including Portion Attributable to Noncontrolling Interest', '2021-09-25'] = df_cash_flow_10q.loc['Net Income (Loss) Attributable to Parent', '2021-09-25']
df_cash_flow_10q.loc['Net Income (Loss), Including Portion Attributable to Noncontrolling Interest', '2021-06-26'] = df_cash_flow_10q.loc['Net Income (Loss) Attributable to Parent', '2021-06-26']
df_cash_flow_10q.loc['Net Income (Loss), Including Portion Attributable to Noncontrolling Interest', '2021-03-27'] = df_cash_flow_10q.loc['Net Income (Loss) Attributable to Parent', '2021-03-27']
df_cash_flow_10q.loc['Net Income (Loss), Including Portion Attributable to Noncontrolling Interest', '2020-09-26'] = df_cash_flow_10q.loc['Net Income (Loss) Attributable to Parent', '2020-09-26']
df_cash_flow_10q.loc['Net Income (Loss), Including Portion Attributable to Noncontrolling Interest', '2020-06-27'] = df_cash_flow_10q.loc['Net Income (Loss) Attributable to Parent', '2020-06-27']
df_cash_flow_10q.loc['Net Income (Loss), Including Portion Attributable to Noncontrolling Interest', '2020-03-28'] = df_cash_flow_10q.loc['Net Income (Loss) Attributable to Parent', '2020-03-28']

# A linha 'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents' nas colunas '2021-09-25', '2021-06-26', '2021-03-27', '2020-09-26', '2020-06-27' e '2020-03-28' estão zeradas
# As informações estão na linha 'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations' e 'Cash and Cash Equivalents, at Carrying Value' 
df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2025-06-28'] = df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations', '2025-06-28']
df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2021-09-25'] = df_cash_flow_10q.loc['Cash and Cash Equivalents, at Carrying Value', '2021-09-25']
df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2021-06-26'] = df_cash_flow_10q.loc['Cash and Cash Equivalents, at Carrying Value', '2021-06-26']
df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2021-03-27'] = df_cash_flow_10q.loc['Cash and Cash Equivalents, at Carrying Value', '2021-03-27']
df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2020-09-26'] = df_cash_flow_10q.loc['Cash and Cash Equivalents, at Carrying Value', '2020-09-26']
df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2020-06-27'] = df_cash_flow_10q.loc['Cash and Cash Equivalents, at Carrying Value', '2020-06-27']
df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2020-03-28'] = df_cash_flow_10q.loc['Cash and Cash Equivalents, at Carrying Value', '2020-03-28']

# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_cash_flow_10q = df_cash_flow_10q.drop([
    'Net Income (Loss) Attributable to Parent',
    'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations',
    'Cash and Cash Equivalents, at Carrying Value'

])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_cash_flow_10q = df_cash_flow_10q.drop([
    'us-gaap_StatementOfCashFlowsAbstract',
    'Proceeds from (Repayments of) Short-term Debt',
    'intc_PaymentsForAcceleratedShareRepurchaseForwardAgreements',
])

# Renomeando algumas linhas do índice
df_cash_flow_10q = df_cash_flow_10q.rename(index={
    'intc_AssetImpairmentChargesAndGainLossOnDisposalOfPropertyPlantAndEquipment': 'Asset Impairment Charges And Gain Loss On Disposal Of Property Plant And Equipment',
    'intc_NonCashGainsLossesOnEquityInvestmentsNet': 'Non Cash Gains Losses On Equity Investments Net',
    'intc_ProceedsFromPartnerContributions': 'Proceeds From Partner Contributions',
    'intc_DivestitureAdditionsToPropertyPlantAndEquipmentHeldForSale': 'Divestiture Additions To Property Plant And Equipment Held For Sale',
    'intc_ProceedsFromCapitalGrants': 'Proceeds From Capital Grants',
    'intc_NoncashCapitalRelatedGovernmentAssistance': 'Non cash Capital Related Government Assistance',
    'intc_PaymentsToAcquirePropertyPlantAndEquipmentFinancingActivities': 'Payments To Acquire Property Plant And Equipment Financing Activities',
})

df_cash_flow_10q

In [ ]:
# Arrumando alguns index dos dfs para ficar padronizado com os index utilizados nas funções p/ calcular os indicadores fundamentalistas
df_balance_sheet_10q = df_balance_sheet_10q.rename(index={
    'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents': 'Cash and Cash Equivalents',
    'Debt Securities, Available-for-sale, Current': 'Marketable Securities',
    "Stockholders' Equity Attributable to Parent": 'Total Shareholders Equity'
})

df_income_statement_10q = df_income_statement_10q.rename(index={
    'Revenue from Contract with Customer, Excluding Assessed Tax': 'Revenues',
    'Research and Development Expense': 'Research and Development',
    'Operating Income (Loss)': 'Operating Income',
    'Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest': 'Income Before Provision for Income Taxes',
    'Income Tax Expense (Benefit)': 'Provision for Income Taxes',
    'Net Income (Loss) Attributable to Parent': 'Net Income',
    'Weighted Average Number of Shares Outstanding, Basic': 'Number of Shares - Basic',
    'Weighted Average Number of Shares Outstanding, Diluted': 'Number of Shares - Diluted'

})

df_cash_flow_10q = df_cash_flow_10q.rename(index={
    'Depreciation': 'Depreciation, Depletion and Amortization',
     'Payments to Acquire Property, Plant, and Equipment': 'Purchases Of Property And Equipment And Intangible Assets',
     'Payments of Ordinary Dividends, Common Stock': 'Payments of Dividends'
})

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//semicondutores//intel//intc_10q.xlsx') as writer:
    df_balance_sheet_10q.to_excel(writer, sheet_name='balance_sheet')
    df_income_statement_10q.to_excel(writer, sheet_name='income_statement')
    df_cash_flow_10q.to_excel(writer, sheet_name='cash_flow')

### Indicadores Fundamentalistas

#### Indicadores 10-K

In [ ]:
# Lendo os arquivos 10k
intel_bs = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//intel//intc_10k.xlsx', 
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

intel_is = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//intel//intc_10k.xlsx', 
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

intel_cf = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//intel//intc_10k.xlsx', 
    sheet_name='cash_flow', 
    index_col='Unnamed: 0'
)

# Transformando as colunas em datetime
intel_bs.columns = pd.to_datetime(intel_bs.columns)
intel_is.columns = pd.to_datetime(intel_is.columns)
intel_cf.columns = pd.to_datetime(intel_cf.columns)

# Transformando as colunas em apenas o ano
intel_bs.columns = intel_bs.columns.year
intel_is.columns = intel_is.columns.year
intel_cf.columns = intel_cf.columns.year

In [ ]:
# Preços da INTC
intel_close = yf.download('INTC', auto_adjust=True, multi_level_index=False, start='2017-01-01')['Close']

# Selecionando o último preço do mês de Dezembro de cada ano 
lst_years = [
    '2017', '2018', '2019', '2020', 
    '2021', '2022', '2023', '2024',
    '2025',
]
last_dec_close = []

for year in lst_years:
    last_close = intel_close[f'{year}-12'].iloc[-1]
    last_dec_close.append(last_close)

# Transformando em uma série pandas
intel_close_dec = pd.Series(last_dec_close, index=lst_years)

# Revertendo o index da série p/ ficar igual com os dos indicadores
intel_close_dec = intel_close_dec.reindex(index=intel_close_dec.index[::-1])

# Arredondamento de duas casas decimais
intel_close_dec = round(intel_close_dec, 2)

# Não realizou split recentemente (https://br.investing.com/equities/intel-corp-historical-data-splits)

# Transformando o index em int
intel_close_dec.index = intel_close_dec.index.astype(int)

intel_close_dec

In [ ]:
# Criando uma lista só para fazer a indexação em que o ano mais recente está no começo
idx = intel_bs.columns

# Valor de Mercado
vm = f_eua.indicador_vm(ten_k=True, price=intel_close_dec, df_is=intel_is) 

# LPA
lpa = f_eua.indicador_lpa(ten_k=True, df_is=intel_is, idx=None)

# P/L Damodaran
p_lpa_damodaran = f_eua.indicador_p_lpa_2(ten_k=True, vm=vm, df_is=intel_is)

# P/L
p_lpa = f_eua.indicador_p_lpa(ten_k=True, price=intel_close_dec, lpa=lpa, idx=idx)

# L/P
lpa_p = f_eua.indicador_lpa_p(ten_k=True, price=intel_close_dec, lpa=lpa, idx=idx) 

# Depreciação
depreciacao = f_eua.indicador_depreciacao(ten_k=True, df_cf=intel_cf, first_quarter=None, idx=idx)

# EBITDA
ebitda = f_eua.indicador_ebitda(ten_k=True, df_is=intel_is, idx=None, depreciacao=depreciacao)

# Margem Líquida
m_liq = f_eua.indicador_ml(ten_k=True, df_is=intel_is)

# VPA
vpa = f_eua.indicador_vpa(ten_k=True, df_bs=intel_bs, df_is=intel_is)

# P/VPA
p_vpa = f_eua.indicador_p_vpa(ten_k=True, price=intel_close_dec, vpa=vpa, idx=idx)

# Caixa e Equivalentes
lst_caixa = ['Cash and Cash Equivalents', 'Marketable Securities']
caixa = f_eua.indicador_caixa(ten_k=True, df_bs=intel_bs, lst_itens_caixa=lst_caixa)

# Dívida Bruta
lst_divida = ['Debt, Current', 'Long-term Debt, Excluding Current Maturities', 'Other Liabilities, Noncurrent']
div_bruta = f_eua.indicador_divida_bruta(ten_k=True, df_bs=intel_bs, lst_itens_divida=lst_divida)

# Dívida Líquida
div_liq = f_eua.indicador_divida_liquida(ten_k=True, caixa=caixa, div_bruta=div_bruta, idx=idx)

# Dívida Líquida / EBITDA
dl_ebitda =  f_eua.indicador_dl_ebitda(ten_k=True, ebitda=ebitda, div_liq=div_liq, idx=None)

# Dívida Líquida / PL
dl_pl = f_eua.indicador_dl_pl(ten_k=True, div_liq=div_liq, df_bs=intel_bs) 

# EV/EBITDA
ev_ebitda = f_eua.indicador_ev_ebitda(vm=vm, div_liq=div_liq, ebitda=ebitda)

# ROE
roe = f_eua.indicador_roe(ten_k=True, df_is=intel_is, df_bs=intel_bs, idx=None)

# ROIC
roic = f_eua.indicador_roic(ten_k=True, df_is=intel_is, df_bs=intel_bs, div_bruta=div_bruta, idx=idx)

# FCO
fco = f_eua.indicador_fco(ten_k=True, df_cf=intel_cf, first_quarter=None) 

# FCI
fci = f_eua.indicador_fci(ten_k=True, df_cf=intel_cf, first_quarter=None) 

# FCF
fcf = f_eua.indicador_fcf(ten_k=True, df_cf=intel_cf, first_quarter=None) 

# CAPEX
lst_capex = ['Purchases Of Property And Equipment And Intangible Assets']

capex= f_eua.indicador_capex(ten_k=True, df_cf=intel_cf, first_quarter=None, lst_itens=lst_capex, idx=idx)

# Free Cash Flow
free_cash_flow = f_eua.indicador_free_cash_flow(fco=fco, capex=capex)

# Net CAPEX
net_capex = f_eua.indicador_net_capex(capex=capex, depreciation=depreciacao)

# R&D
rd_expense = f_eua.indicador_rd(ten_k=True, df_is=intel_is, idx=idx)

# Net CAPEX ajustado
adj_net_capex = f_eua.indicador_adj_net_capex(net_capex=net_capex, rd_expense=rd_expense)

# Working capital
working_capital = f_eua.indicador_working_capital(ten_k=True, df_bs=intel_bs)

# Change in non cash working capital
lst_itens_change_non_cash_wc = [
    'Increase (Decrease) in Accounts Receivable',
    'Increase (Decrease) in Inventories',
    'Increase (Decrease) in Accounts Payable',
    'Increase (Decrease) in Employee Related Liabilities',
    'Increase (Decrease) in Contract with Customer, Liability',
    'Increase (Decrease) in Income Taxes Payable, Net of Income Taxes Receivable',
    'Increase (Decrease) in Other Operating Assets and Liabilities, Net'
]

change_non_cash_wc = f_eua.indicador_change_non_cash_wc(ten_k=True, df_cf=intel_cf, lst_itens=lst_itens_change_non_cash_wc, first_quarter=None)

# Reinvestment rate
reinvestment_rate = f_eua.indicador_reinvestment_rate(ten_k=True, net_capex=net_capex, change_non_cash_wc=change_non_cash_wc, df_is=intel_is, idx=idx)

# New borrowing
lst_itens_new_borrowing = [
    'Proceeds from Issuance of Long-term Debt',
    'Proceeds from Issuance of Commercial Paper'	
]

new_borrowing = f_eua.indicador_new_borrowing(ten_k=True, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=intel_cf, first_quarter=None, idx=idx)

# Debt paid
lst_itens_debt_paid = [
    'Repayments of Long-term Debt',
    'Repayments of Commercial Paper',
    'Finance Lease, Principal Payments'
]

debt_paid = f_eua.indicador_debt_paid(ten_k=True, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=intel_cf, first_quarter=None, idx=idx)

# Taxes on operating income
taxes_operating_income =f_eua. indicador_taxes_operating_income(ten_k=True, df_is=intel_is, idx=idx)

# FCFE
fcfe_10q = f_eua.indicador_fcfe(
    ten_k=True, 
    depreciacao=depreciacao, 
    capex=capex, 
    change_non_cash_wc=change_non_cash_wc, 
    new_borrowing=new_borrowing,
    debt_paid=debt_paid,
    df_is=intel_is,
    idx=idx
)

# FCFF
fcff_10q = f_eua.indicador_fcff(
        ten_k=True, 
        taxes_operating_income=taxes_operating_income,
        depreciacao=depreciacao, 
        capex=capex, 
        change_non_cash_wc=change_non_cash_wc, 
        df_is=intel_is,
        idx=idx
)

# Dividendo por Ação
dpa = f_eua.indicador_dpa(ten_k=True, df_cf=intel_cf, df_is=intel_is, div_nao_acum=None)

# Payout
payout = f_eua.indicador_payout(ten_k=True, df_cf=intel_cf, df_is=intel_is, div_nao_acum=None)

# Buyback
buyback = f_eua.indicador_buyback(ten_k=True, df_cf=intel_cf, first_quarter=None)

# Colocando todos os indicadores em uma lista
lst_indicadores = [
    p_lpa_damodaran,
    p_lpa,
    lpa_p,  
    ebitda, 
    m_liq,
    p_vpa,
    vm,
    div_bruta,
    caixa,
    div_liq,
    dl_ebitda,
    dl_pl,
    ev_ebitda,
    roe,
    roic,
    fco,
    fci,
    fcf,
    free_cash_flow,
    capex,
    net_capex,
    rd_expense,
    adj_net_capex,
    working_capital,
    reinvestment_rate,
    fcfe_10q,
    fcff_10q,
    dpa,
    payout,
    buyback
]

# Juntando todos os indicadores em um df
intel_indicadores_10k = pd.concat(lst_indicadores, axis=1)

# Nomes das colunas
nomes_col = [
    'P/L Damodaran', 
    'P/L', 
    'L/P', 
    'EBITDA', 
    'Margem Líquida',
    'P/VPA',
    'Valor de Mercado',
    'Dívida Bruta',
    'Caixa e Equivalentes',
    'Dívida Líquida',
    'Dívida Líquida/EBITDA',
    'Dívida Líquida/PL',
    'EV/EBITDA',
    'ROE',
    'ROIC',
    'FCO',
    'FCI',
    'FCF',
    'Free Cash Flow',
    'Capex',
    'Net Capex',
    'R&D',
    'Adj Net Capex',
    'Working Capital',
    'Reinvestment Rate',
    'FCFE',
    'FCFF',
    'DPA',
    'Payout',
    'Buyback'
]

# Atribuindo os novos nomes das colunas do df
intel_indicadores_10k.columns = nomes_col
intel_indicadores_10k

#### Indicadores 10-Q

In [ ]:
# Lendo os arquivos 10q
intel_bs_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//intel//intc_10q.xlsx', 
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

intel_is_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//intel//intc_10q.xlsx', 
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

intel_cf_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//intel//intc_10q.xlsx', 
    sheet_name='cash_flow', 
    index_col='Unnamed: 0'
)

# Aplicando o ajuste nas colunas
intel_bs_10q.columns = [f_sec.ajustar_mes_03_06_09(col) for col in intel_bs_10q.columns]
intel_is_10q.columns = [f_sec.ajustar_mes_03_06_09(col) for col in intel_is_10q.columns]
intel_cf_10q.columns = [f_sec.ajustar_mes_03_06_09(col) for col in intel_cf_10q.columns]

In [ ]:
# Preços da INTC
intel_close = yf.download('INTC', auto_adjust=True, multi_level_index=False, start='2021-01-01')['Close']

# Selecionando o último preço dos meses de Março, Junho e Setembro
lst_years = [
    '2021', '2022', '2023', '2024', 
    '2025',
]
lst_months = ['03', '06', '09']
last_close_months = []

for year in lst_years:
    for month in lst_months:
        last_close = intel_close[f'{year}-{month}'].dropna().iloc[-1]  # Dropna para lidar com meses sem dados
        last_close_months.append(last_close)

# Criando uma lista em que junte os meses e os anos para que seja o index da serie pandas
lst_idx = []

for year in lst_years:
    for month in lst_months:
        lst_idx.append(f"{month}-{year}")

# Transformando em uma série pandas
intel_close_months = pd.Series(last_close_months, index=lst_idx)

# Revertendo o index da série p/ ficar igual com os dos indicadores
intel_close_months = intel_close_months.reindex(index=intel_close_months.index[::-1])

# No ano mais atual, como ainda ele não acabou, não consigo fazer o loop com todos meses (Fevereiro, Maio e Agosto)
actual_year_int = datetime.now().year  
actual_year_str = str(actual_year_int) 

lst_actual_months = [
    # '03', 
    # '06', 
    #'09'
] 

last_close_months_atual = []

for month in lst_actual_months:
    last_close = intel_close[f'{actual_year_str}-{month}'].dropna().iloc[-1]  # Dropna para lidar com meses sem dados
    last_close_months_atual.append(last_close)

# Criando uma lista em que junte os meses e os anos para que seja o index da serie pandas
lst_idx_actual = []

for month in lst_actual_months:
    lst_idx_actual.append(f"{month}-{actual_year_str}")

# Transformando em uma série pandas
intel_close_months_atual = pd.Series(last_close_months_atual, index=lst_idx_actual)

# Revertendo o index da série p/ ficar igual com os dos indicadores
intel_close_months_atual = intel_close_months_atual.reindex(index=intel_close_months_atual.index[::-1])

# Juntando as duas series pandas
intel_close_months = pd.concat([intel_close_months_atual, intel_close_months])

# Arredondamento de duas casas decimais
intel_close_months = round(intel_close_months, 2)

# Não realizou split recentemente (https://br.investing.com/equities/intel-corp-historical-data-splits)

# Substituindo o index para ficar igual ao dos dados fundamentalistas
intel_close_months.index = intel_bs_10q.columns[:-3]

intel_close_months

In [ ]:
# Criando uma lista só para fazer a indexação em que o ano mais recente está no começo
idx_10q = intel_bs_10q.columns

# Valor de Mercado
vm_10q = f_eua.indicador_vm(ten_k=False, price=intel_close_months, df_is=intel_is_10q)

# LPA
lpa_acum = f_eua.indicador_lpa(ten_k=False, df_is=intel_is_10q, idx=idx_10q)[1]
lpa_10q = f_eua.indicador_lpa(ten_k=False, df_is=intel_is_10q, idx=idx_10q)[0]

# P/L Damodaran
p_lpa_10q_damodaran = f_eua.indicador_p_lpa_2(ten_k=False, vm=vm_10q, df_is=intel_is_10q)

# P/L
p_lpa_10q = f_eua.indicador_p_lpa(ten_k=False, price=intel_close_months, lpa=lpa_acum, idx=idx_10q)

# L/P
lpa_p_10q = f_eua.indicador_lpa_p(ten_k=False, price=intel_close_months, lpa=lpa_acum, idx=idx_10q) 

# Depreciação
depreciacao_10q = f_eua.indicador_depreciacao(ten_k=False, df_cf=intel_cf_10q, first_quarter='03', idx=idx_10q)[0]
depreciacao_10q_sliced = f_eua.indicador_depreciacao(ten_k=False, df_cf=intel_cf_10q, first_quarter='03', idx=idx_10q)[1]
depreciacao_10q_acum = f_eua.indicador_depreciacao(ten_k=False, df_cf=intel_cf_10q, first_quarter='03', idx=idx_10q)[2]

# EBITDA
ebitda_10q = f_eua.indicador_ebitda(ten_k=False, df_is=intel_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[0]
ebitda_10q_sliced = f_eua.indicador_ebitda(ten_k=False, df_is=intel_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[1]
ebitda_10q_acum = f_eua.indicador_ebitda(ten_k=False, df_is=intel_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[2]

# Margem Líquida
m_liq_10q = f_eua.indicador_ml(ten_k=False, df_is=intel_is_10q)

# VPA
vpa_10q = f_eua.indicador_vpa(ten_k=False, df_bs=intel_bs_10q, df_is=intel_is_10q)

# P/VPA
p_vpa_10q = f_eua.indicador_p_vpa(ten_k=False, price=intel_close_months, vpa=vpa_10q, idx=idx_10q)

# Caixa e Equivalentes
lst_caixa = ['Cash and Cash Equivalents', 'Marketable Securities']
caixa_10q = f_eua.indicador_caixa(ten_k=False, df_bs=intel_bs_10q, lst_itens_caixa=lst_caixa)

# Dívida Bruta
lst_divida = ['Debt, Current', 'Long-term Debt, Excluding Current Maturities', 'Other Liabilities, Noncurrent']
div_bruta_10q = f_eua.indicador_divida_bruta(ten_k=False, df_bs=intel_bs_10q, lst_itens_divida=lst_divida)

# Dívida Líquida
div_liq_10q = f_eua.indicador_divida_liquida(ten_k=False, caixa=caixa_10q, div_bruta=div_bruta_10q, idx=idx_10q)

# Dívida Líquida / EBITDA
dl_ebitda_10q = f_eua.indicador_dl_ebitda(ten_k=False, ebitda=ebitda_10q_acum, div_liq=div_liq_10q, idx=idx_10q)

# Dívida Líquida / PL
dl_pl_10q = f_eua.indicador_dl_pl(ten_k=False, div_liq=div_liq_10q, df_bs=intel_bs_10q) 

# EV/EBITDA
ev_ebitda_10q = f_eua.indicador_ev_ebitda(vm=vm_10q, div_liq=div_liq_10q, ebitda=ebitda_10q_acum)

# ROE
roe_10q = f_eua.indicador_roe(ten_k=False, df_is=intel_is_10q, df_bs=intel_bs_10q, idx=idx_10q)

# ROIC 
roic_10q = f_eua.indicador_roic(ten_k=False, df_is=intel_is_10q, df_bs=intel_bs_10q, div_bruta=div_bruta_10q, idx=idx_10q)

# FCO
fco_10q = f_eua.indicador_fco(ten_k=False, df_cf=intel_cf_10q, first_quarter='03') 

# FCI
fci_10q = f_eua.indicador_fci(ten_k=False, df_cf=intel_cf_10q, first_quarter='03') 

# FCF
fcf_10q = f_eua.indicador_fcf(ten_k=False, df_cf=intel_cf_10q, first_quarter='03') 

# CAPEX
lst_capex_10q = ['Purchases Of Property And Equipment And Intangible Assets']

capex_10q = f_eua.indicador_capex(ten_k=False, df_cf=intel_cf_10q, first_quarter='03', lst_itens=lst_capex_10q, idx=idx_10q)[0]
capex_10q_sliced = f_eua.indicador_capex(ten_k=False, df_cf=intel_cf_10q, first_quarter='03', lst_itens=lst_capex_10q, idx=idx_10q)[1]
capex_10q_acum = f_eua.indicador_capex(ten_k=False, df_cf=intel_cf_10q, first_quarter='03', lst_itens=lst_capex_10q, idx=idx_10q)[2]

# Free Cash Flow
free_cash_flow_10q = f_eua.indicador_free_cash_flow(fco=fco_10q, capex=capex_10q_sliced)

# Net CAPEX
net_capex_10q = f_eua.indicador_net_capex(capex=capex_10q_acum, depreciation=depreciacao_10q_acum)

# R&D
rd_expense_10q = f_eua.indicador_rd(ten_k=False, df_is=intel_is_10q, idx=idx_10q)[0]
rd_expense_10q_sliced = f_eua.indicador_rd(ten_k=False, df_is=intel_is_10q, idx=idx_10q)[1]
rd_expense_10q_acum = f_eua.indicador_rd(ten_k=False, df_is=intel_is_10q, idx=idx_10q)[2]

# Net CAPEX ajustado
adj_net_capex_10q = f_eua.indicador_adj_net_capex(net_capex=net_capex_10q, rd_expense=rd_expense_10q_acum)

# Working capital
working_capital_10q = f_eua.indicador_working_capital(ten_k=False, df_bs=intel_bs_10q)

# Change in non cash working capital
lst_itens_change_non_cash_wc = [
    'Increase (Decrease) in Accounts Receivable',
    'Increase (Decrease) in Inventories',
    'Increase (Decrease) in Accounts Payable',
    'Increase (Decrease) in Employee Related Liabilities',
    'Increase (Decrease) in Contract with Customer, Liability',
    'Increase (Decrease) in Income Taxes Payable, Net of Income Taxes Receivable',
    'Increase (Decrease) in Other Operating Assets and Liabilities, Net'
]

change_non_cash_wc_10q = f_eua.indicador_change_non_cash_wc(ten_k=False, df_cf=intel_cf_10q, lst_itens=lst_itens_change_non_cash_wc, first_quarter='03')[0]
change_non_cash_wc_10q_sliced = f_eua.indicador_change_non_cash_wc(ten_k=False, df_cf=intel_cf_10q, lst_itens=lst_itens_change_non_cash_wc, first_quarter='03')[1]

# Reinvestment rate
reinvestment_rate_10q = f_eua.indicador_reinvestment_rate(ten_k=False, net_capex=adj_net_capex_10q, change_non_cash_wc=change_non_cash_wc_10q, df_is=intel_is_10q, idx=idx_10q)

# New borrowing
lst_itens_new_borrowing = [
    'Proceeds from Issuance of Long-term Debt',
    'Proceeds from Issuance of Commercial Paper'   
]

new_borrowing_10q = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=intel_cf_10q, first_quarter='03', idx=idx_10q)[0]
new_borrowing_10q_sliced = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=intel_cf_10q, first_quarter='03', idx=idx_10q)[1]
new_borrowing_10q_acum = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=intel_cf_10q, first_quarter='03', idx=idx_10q)[2]

# Debt paid
lst_itens_debt_paid = [
    'Repayments of Long-term Debt',
    'Repayments of Commercial Paper',
    'Finance Lease, Principal Payments'
]

debt_paid_10q = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=intel_cf_10q, first_quarter='03', idx=idx_10q)[0]
debt_paid_10q_sliced = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=intel_cf_10q, first_quarter='03', idx=idx_10q)[1]
debt_paid_10q_acum = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=intel_cf_10q, first_quarter='03', idx=idx_10q)[2]

# Taxes on operating income
taxes_operating_income_10q = f_eua.indicador_taxes_operating_income(ten_k=False, df_is=intel_is_10q, idx=idx_10q)

# FCFE
fcfe_10q = f_eua.indicador_fcfe(
    ten_k=False, 
    depreciacao=depreciacao_10q_acum, 
    capex=capex_10q_acum, 
    change_non_cash_wc=change_non_cash_wc_10q_sliced, 
    new_borrowing=new_borrowing_10q_acum, 
    debt_paid=debt_paid_10q_acum, 
    df_is=intel_is_10q,
    idx=idx_10q
)

# FCFF
fcff_10q = f_eua.indicador_fcff(
        ten_k=False, 
        taxes_operating_income=taxes_operating_income_10q,
        depreciacao=depreciacao_10q_acum, 
        capex=capex_10q_acum, 
        change_non_cash_wc=change_non_cash_wc_10q_sliced, 
        df_is=intel_is_10q,
        idx=idx_10q
)

# Dividendo não acumulado
div_nao_acum = f_eua.indicador_div_nao_acum(df_cf=intel_cf_10q, first_quarter='03')

# Dividendo por Ação
dpa_10q = f_eua.indicador_dpa(ten_k=False, df_cf=intel_cf_10q, df_is=intel_is_10q, div_nao_acum=div_nao_acum)

# Payout
payout_10q = f_eua.indicador_payout(ten_k=False, df_cf=intel_cf_10q, df_is=intel_is_10q, div_nao_acum=div_nao_acum)

# Buyback
buyback_10q = f_eua.indicador_buyback(ten_k=False, df_cf=intel_cf_10q,  first_quarter='03')

# Lista com todos os indicadores
lst_indicadores_10q = [
    p_lpa_10q_damodaran,
    p_lpa_10q,
    lpa_p_10q,  
    ebitda_10q_sliced, 
    m_liq_10q,
    p_vpa_10q,
    vm_10q,
    div_bruta_10q,
    caixa_10q,
    div_liq_10q,
    dl_ebitda_10q,
    dl_pl_10q,
    ev_ebitda_10q,
    roe_10q,
    roic_10q,
    fco_10q,
    fci_10q,
    fcf_10q,
    free_cash_flow_10q,
    capex_10q_sliced,
    net_capex_10q,
    rd_expense_10q_sliced,
    adj_net_capex_10q,
    working_capital_10q,
    reinvestment_rate_10q,
    fcfe_10q,
    fcff_10q,
    dpa_10q,
    payout_10q,
    buyback_10q
]

# Juntando todos os indicadores em um df
intel_indicadores_10q = pd.concat(lst_indicadores_10q, axis=1)

# Nomes das colunas
nomes_col = [
    'P/L Damodaran', 
    'P/L', 
    'L/P', 
    'EBITDA', 
    'Margem Líquida',
    'P/VPA',
    'Valor de Mercado',
    'Dívida Bruta',
    'Caixa e Equivalentes',
    'Dívida Líquida',
    'Dívida Líquida/EBITDA',
    'Dívida Líquida/PL',
    'EV/EBITDA',
    'ROE',
    'ROIC',
    'FCO',
    'FCI',
    'FCF',
    'Free Cash Flow',
    'Capex',
    'Net Capex',
    'R&D',
    'Adj Net Capex',
    'Working Capital',
    'Reinvestment Rate',
    'FCFE',
    'FCFF',
    'DPA',
    'Payout',
    'Buyback'
]

# Atribuindo os novos nomes das colunas do df
intel_indicadores_10q.columns = nomes_col
intel_indicadores_10q

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//semicondutores//intel//intc_indicators.xlsx') as writer:
    intel_indicadores_10k.to_excel(writer, sheet_name='indicators_10k')
    intel_indicadores_10q.to_excel(writer, sheet_name='indicators_10q')

## Micron

* 1Q -> Lançamento em Dezembro (12);
* 2Q -> Lançamento em Março (03);
* 3Q -> Lançamento em Junho (06);
* 10K -> Lançamento em Setembro (09).

In [ ]:
ticker = 'mu'
f_sec.cik_matching_ticker(ticker)

### Scraping Micron 10-K

In [ ]:
# Número dos documentos que contém a DRE, BP e FC 
acc_documents = f_sec.get_filtered_filings(ticker=ticker, ten_k=True, just_accession_numbers=True)

# Trocando os '-' para '' das strings ('0001045810-24-000264' -> '000104581024000264')
dict_acc_num = {k: v.replace('-', '') for k, v in acc_documents.items()}

# Transformando o dicionário em um df
df_lst_acc_num_micron = pd.DataFrame.from_dict(dict_acc_num, orient='index', columns=['acc_num'])

# Transformando em datetime o index
df_lst_acc_num_micron.index = pd.to_datetime(df_lst_acc_num_micron.index, format='%Y-%m-%d')
df_lst_acc_num_micron

In [ ]:
# Selecionando os 'acc_num' dos 10-Q a partir de 2017 p/ iteração
df_lst_acc_num_micron_sliced = df_lst_acc_num_micron.loc['2017-08-31':]
df_lst_acc_num_micron_sliced

In [ ]:
# Rótulos
labels_dict = f_sec.get_label_dictionary(ticker, headers=f_sec.header)

In [ ]:
# Fazendo o scraping do balance sheet (10-K)
balance_sheet_lst = []

for acc_num in df_lst_acc_num_micron_sliced['acc_num']:

    # Balance sheet
    balance_sheet = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='balance_sheet')
    balance_sheet = f_sec.rename_statement(balance_sheet, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    balance_sheet = balance_sheet.iloc[:, 0]
    balance_sheet_lst.append(balance_sheet)

# Juntando todos os dfs que estão em listas em apenas um df
df_balance_sheet_10k = pd.concat(balance_sheet_lst, axis=1)

# Substituindo os NaN por 0
df_balance_sheet_10k = df_balance_sheet_10k.fillna(0)

# A linha 'GovernmentAssistanceLiabilityNoncurrent' nas colunas '2023-08-31', '2022-09-01', '2021-09-02', '2020-09-03' e '2019-08-29' estão zeradas
# A informação estão nas linhas 'mu_UnearnedGovernmentAssistanceNoncurrent' e 'mu_UnearnedGovernmentIncentivesNoncurrent'
df_balance_sheet_10k.loc['GovernmentAssistanceLiabilityNoncurrent', '2023-08-31'] = df_balance_sheet_10k.loc['mu_UnearnedGovernmentAssistanceNoncurrent', '2023-08-31'] 
df_balance_sheet_10k.loc['GovernmentAssistanceLiabilityNoncurrent', '2022-09-01'] = df_balance_sheet_10k.loc['mu_UnearnedGovernmentIncentivesNoncurrent', '2022-09-01'] 
df_balance_sheet_10k.loc['GovernmentAssistanceLiabilityNoncurrent', '2021-09-02'] = df_balance_sheet_10k.loc['mu_UnearnedGovernmentIncentivesNoncurrent', '2021-09-02'] 
df_balance_sheet_10k.loc['GovernmentAssistanceLiabilityNoncurrent', '2020-09-03'] = df_balance_sheet_10k.loc['mu_UnearnedGovernmentIncentivesNoncurrent', '2020-09-03'] 
df_balance_sheet_10k.loc['GovernmentAssistanceLiabilityNoncurrent', '2019-08-29'] = df_balance_sheet_10k.loc['mu_UnearnedGovernmentIncentivesNoncurrent', '2019-08-29'] 

# A linha 'Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization' nas colunas '2019-08-29', '2018-08-30' e '2017-08-31' estão zeradas
# A informação está na linha 'Property, Plant and Equipment, Net'
df_balance_sheet_10k.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2019-08-29'] = df_balance_sheet_10k.loc['Property, Plant and Equipment, Net', '2019-08-29'] 
df_balance_sheet_10k.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2018-08-30'] = df_balance_sheet_10k.loc['Property, Plant and Equipment, Net', '2018-08-30'] 
df_balance_sheet_10k.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2017-08-31'] = df_balance_sheet_10k.loc['Property, Plant and Equipment, Net', '2017-08-31'] 

# A linha 'Debt Securities, Available-for-sale, Current' nas colunas '2018-08-30' e '2017-08-31' estão zeradas
# A informação está na linha 'Available-for-sale Securities, Current'
df_balance_sheet_10k.loc['Debt Securities, Available-for-sale, Current', '2018-08-30'] = df_balance_sheet_10k.loc['Available-for-sale Securities, Current', '2018-08-30'] 
df_balance_sheet_10k.loc['Debt Securities, Available-for-sale, Current', '2017-08-31'] = df_balance_sheet_10k.loc['Available-for-sale Securities, Current', '2017-08-31'] 

# A linha 'Debt Securities, Available-for-sale, Noncurrent' nas colunas '2018-08-30' e '2017-08-31' estão zeradas
# A informação está na linha 'Available-for-sale Securities, Noncurrent'
df_balance_sheet_10k.loc['Debt Securities, Available-for-sale, Noncurrent', '2018-08-30'] = df_balance_sheet_10k.loc['Available-for-sale Securities, Noncurrent', '2018-08-30'] 
df_balance_sheet_10k.loc['Debt Securities, Available-for-sale, Noncurrent', '2017-08-31'] = df_balance_sheet_10k.loc['Available-for-sale Securities, Noncurrent', '2017-08-31'] 

#  Removendo as linhas que movemos os seus itens para as linhas corretas
df_balance_sheet_10k = df_balance_sheet_10k.drop([
    'mu_UnearnedGovernmentAssistanceNoncurrent',
    'mu_UnearnedGovernmentIncentivesNoncurrent',
    'Property, Plant and Equipment, Net',
    'Available-for-sale Securities, Current',
    'Available-for-sale Securities, Noncurrent'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_balance_sheet_10k = df_balance_sheet_10k.drop([
    'us-gaap_AssetsAbstract',
    'mu_AssetsHeldForSaleCurrent',
    'Redeemable Noncontrolling Interest, Equity, Preferred, Carrying Amount',
    "Stockholders' Equity Attributable to Noncontrolling Interest",
    "Stockholders' Equity, Including Portion Attributable to Noncontrolling Interest",
    'Temporary Equity, Carrying Amount, Attributable to Parent',
    'Deferred Revenue, Current',
    'Equity Method Investments'
])

# Arrumando e adicionando alguns números no df
df_balance_sheet_10k.loc['Other Liabilities, Current', '2018-08-30'] = 521
df_balance_sheet_10k.loc['GovernmentAssistanceLiabilityNoncurrent', '2018-08-30'] = 227

# Renomeando as linhas do índice que estão sem formato 
df_balance_sheet_10k = df_balance_sheet_10k.rename(index={
    'Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization': 'Property, plant, and equipment',
    'GovernmentAssistanceLiabilityNoncurrent': 'Government Assistance Liability Noncurrent'
})

df_balance_sheet_10k

In [ ]:
# Fazendo o scraping do income statement (10-K)
income_statement_lst = []

for acc_num in df_lst_acc_num_micron_sliced['acc_num']:

    # Income statement
    income_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='income_statement')
    income_statement = f_sec.rename_statement(income_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    income_statement = income_statement.iloc[:, 0]
    income_statement_lst.append(income_statement)

# Juntando todos os dfs que estão em listas em apenas um df
df_income_statement_10k = pd.concat(income_statement_lst, axis=1)

# Substituindo os NaN por 0
df_income_statement_10k = df_income_statement_10k.fillna(0)

# A linha 'Revenue from Contract with Customer, Excluding Assessed Tax' na coluna '2018-08-30' e '2017-08-31' estão zeradas
# As informações estão nas linhas 'Revenues' e 'Revenue, Net (Deprecated 2018-01-31)'
df_income_statement_10k.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2018-08-30'] = df_income_statement_10k.loc['Revenues', '2018-08-30'] 
df_income_statement_10k.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2017-08-31'] = df_income_statement_10k.loc['Revenue, Net (Deprecated 2018-01-31)', '2017-08-31'] 

# A linha 'InterestExpenseNonoperating' na coluna '2023-08-31', '2022-09-01', '2021-09-02', '2020-09-03', '2019-08-29', '2018-08-30' e '2017-08-31' estão zeradas
# As informações estão na linha 'Interest Expense' 
df_income_statement_10k.loc['InterestExpenseNonoperating', '2023-08-31'] = df_income_statement_10k.loc['Interest Expense', '2023-08-31'] 
df_income_statement_10k.loc['InterestExpenseNonoperating', '2022-09-01'] = df_income_statement_10k.loc['Interest Expense', '2022-09-01'] 
df_income_statement_10k.loc['InterestExpenseNonoperating', '2021-09-02'] = df_income_statement_10k.loc['Interest Expense', '2021-09-02'] 
df_income_statement_10k.loc['InterestExpenseNonoperating', '2020-09-03'] = df_income_statement_10k.loc['Interest Expense', '2020-09-03'] 
df_income_statement_10k.loc['InterestExpenseNonoperating', '2019-08-29'] = df_income_statement_10k.loc['Interest Expense', '2019-08-29'] 
df_income_statement_10k.loc['InterestExpenseNonoperating', '2018-08-30'] = df_income_statement_10k.loc['Interest Expense', '2018-08-30'] 
df_income_statement_10k.loc['InterestExpenseNonoperating', '2017-08-31'] = df_income_statement_10k.loc['Interest Expense', '2017-08-31'] 


# Removendo as linhas que movemos os seus itens para as linhas corretas
df_income_statement_10k = df_income_statement_10k.drop([
    'Revenues',
    'Revenue, Net (Deprecated 2018-01-31)',
    'Interest Expense'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_income_statement_10k = df_income_statement_10k.drop([
    'us-gaap_IncomeStatementAbstract',
    'Net Income (Loss), Including Portion Attributable to Noncontrolling Interest',
    'Net Income (Loss) Attributable to Noncontrolling Interest'

])

# Renomeando as linhas do índice que estão sem formato 
df_income_statement_10k = df_income_statement_10k.rename(index={
    'InterestExpenseNonoperating': 'Interest Expense',
})

df_income_statement_10k

In [ ]:
# Fazendo o scraping do cash flow statement (10-K)
cash_flow_statement_lst = []

for acc_num in df_lst_acc_num_micron_sliced['acc_num']:

    # Cash flow statement
    cash_flow_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='cash_flow_statement')
    cash_flow_statement = f_sec.rename_statement(cash_flow_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    cash_flow_statement = cash_flow_statement.iloc[:, 0]
    cash_flow_statement_lst.append(cash_flow_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_cash_flow_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in cash_flow_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_cash_flow_10k = pd.concat([df_cash_flow_10k, df], axis=1)

# Substituindo os NaN por 0
df_cash_flow_10k = df_cash_flow_10k.fillna(0)

# A linha 'Net Income (Loss) Attributable to Parent' nas colunas '2022-09-01', '2021-09-02', '2020-09-03', '2019-08-29', '2018-08-30' e '2017-08-31' estão zeradas
# As informações estão na linha 'Net Income (Loss), Including Portion Attributable to Noncontrolling Interest'
df_cash_flow_10k.loc['Net Income (Loss) Attributable to Parent', '2022-09-01'] = df_cash_flow_10k.loc['Net Income (Loss), Including Portion Attributable to Noncontrolling Interest', '2022-09-01'] 
df_cash_flow_10k.loc['Net Income (Loss) Attributable to Parent', '2021-09-02'] = df_cash_flow_10k.loc['Net Income (Loss), Including Portion Attributable to Noncontrolling Interest', '2021-09-02'] 
df_cash_flow_10k.loc['Net Income (Loss) Attributable to Parent', '2020-09-03'] = df_cash_flow_10k.loc['Net Income (Loss), Including Portion Attributable to Noncontrolling Interest', '2020-09-03'] 
df_cash_flow_10k.loc['Net Income (Loss) Attributable to Parent', '2019-08-29'] = df_cash_flow_10k.loc['Net Income (Loss), Including Portion Attributable to Noncontrolling Interest', '2019-08-29'] 
df_cash_flow_10k.loc['Net Income (Loss) Attributable to Parent', '2018-08-30'] = df_cash_flow_10k.loc['Net Income (Loss), Including Portion Attributable to Noncontrolling Interest', '2018-08-30'] 
df_cash_flow_10k.loc['Net Income (Loss) Attributable to Parent', '2017-08-31'] = df_cash_flow_10k.loc['Net Income (Loss), Including Portion Attributable to Noncontrolling Interest', '2017-08-31'] 

# A linha 'mu_ProceedsFromGovernmentAssistance' nas colunas '2022-09-01', '2021-09-02', '2020-09-03', '2019-08-29', '2018-08-30' e '2017-08-31' estão zeradas
# As informações estão na linha 'mu_ProceedsFromGovernmentIncentives'
df_cash_flow_10k.loc['mu_ProceedsFromGovernmentAssistance', '2022-09-01'] = df_cash_flow_10k.loc['mu_ProceedsFromGovernmentIncentives', '2022-09-01'] 
df_cash_flow_10k.loc['mu_ProceedsFromGovernmentAssistance', '2021-09-02'] = df_cash_flow_10k.loc['mu_ProceedsFromGovernmentIncentives', '2021-09-02'] 
df_cash_flow_10k.loc['mu_ProceedsFromGovernmentAssistance', '2020-09-03'] = df_cash_flow_10k.loc['mu_ProceedsFromGovernmentIncentives', '2020-09-03'] 
df_cash_flow_10k.loc['mu_ProceedsFromGovernmentAssistance', '2019-08-29'] = df_cash_flow_10k.loc['mu_ProceedsFromGovernmentIncentives', '2019-08-29'] 
df_cash_flow_10k.loc['mu_ProceedsFromGovernmentAssistance', '2018-08-30'] = df_cash_flow_10k.loc['mu_ProceedsFromGovernmentIncentives', '2018-08-30'] 
df_cash_flow_10k.loc['mu_ProceedsFromGovernmentAssistance', '2017-08-31'] = df_cash_flow_10k.loc['mu_ProceedsFromGovernmentIncentives', '2017-08-31'] 

# A linha 'mu_CapitalExpendituresIncurredOnPurchaseContractsNotYetPaid' nas colunas '2021-09-02', '2020-09-03', '2019-08-29', '2018-08-30' e '2017-08-31' estão zeradas
# As informações estão na linha 'mu_CapitalExpendituresIncurredNotYetPaidAndFinanceLeaseObligationsIncurred' e 'Capital Expenditures Incurred but Not yet Paid'
df_cash_flow_10k.loc['mu_CapitalExpendituresIncurredOnPurchaseContractsNotYetPaid', '2021-09-02'] = df_cash_flow_10k.loc['mu_CapitalExpendituresIncurredNotYetPaidAndFinanceLeaseObligationsIncurred', '2021-09-02'] 
df_cash_flow_10k.loc['mu_CapitalExpendituresIncurredOnPurchaseContractsNotYetPaid', '2020-09-03'] = df_cash_flow_10k.loc['Capital Expenditures Incurred but Not yet Paid', '2020-09-03'] 
df_cash_flow_10k.loc['mu_CapitalExpendituresIncurredOnPurchaseContractsNotYetPaid', '2019-08-29'] = df_cash_flow_10k.loc['Capital Expenditures Incurred but Not yet Paid', '2019-08-29'] 
df_cash_flow_10k.loc['mu_CapitalExpendituresIncurredOnPurchaseContractsNotYetPaid', '2018-08-30'] = df_cash_flow_10k.loc['Capital Expenditures Incurred but Not yet Paid', '2018-08-30'] 
df_cash_flow_10k.loc['mu_CapitalExpendituresIncurredOnPurchaseContractsNotYetPaid', '2017-08-31'] = df_cash_flow_10k.loc['Capital Expenditures Incurred but Not yet Paid', '2017-08-31'] 

# A linha 'Payments to Acquire Debt Securities, Available-for-sale' nas colunas '2018-08-30' e '2017-08-31' estão zeradas
# As informações estão na linha 'Payments to Acquire Available-for-sale Securities' 
df_cash_flow_10k.loc['Payments to Acquire Debt Securities, Available-for-sale', '2018-08-30'] = df_cash_flow_10k.loc['Payments to Acquire Available-for-sale Securities', '2018-08-30'] 
df_cash_flow_10k.loc['Payments to Acquire Debt Securities, Available-for-sale', '2017-08-31'] = df_cash_flow_10k.loc['Payments to Acquire Available-for-sale Securities', '2017-08-31'] 

# A linha 'Proceeds from Sale of Debt Securities, Available-for-sale' nas colunas '2018-08-30' e '2017-08-31' estão zeradas
# As informações estão na linha 'Proceeds from Sale of Available-for-sale Securities' 
df_cash_flow_10k.loc['Proceeds from Sale of Debt Securities, Available-for-sale', '2018-08-30'] = df_cash_flow_10k.loc['Proceeds from Sale of Available-for-sale Securities', '2018-08-30'] 
df_cash_flow_10k.loc['Proceeds from Sale of Debt Securities, Available-for-sale', '2017-08-31'] = df_cash_flow_10k.loc['Proceeds from Sale of Available-for-sale Securities', '2017-08-31'] 

# A linha 'Depreciation, Depletion and Amortization' na coluna '2017-08-31' está zerada
# As informações estão na linha 'mu_DepreciationandAmortizationofIntangibleAssets' 
df_cash_flow_10k.loc['Depreciation, Depletion and Amortization', '2017-08-31'] = df_cash_flow_10k.loc['mu_DepreciationandAmortizationofIntangibleAssets', '2017-08-31'] 

# A linha 'Proceeds from Sale of Debt Securities, Available-for-sale' na coluna '2017-08-31' está zerada
# As informações estão na linha 'Proceeds from Sale and Maturity of Debt Securities, Available-for-sale' 
df_cash_flow_10k.loc['Proceeds from Sale of Debt Securities, Available-for-sale', '2017-08-31'] = df_cash_flow_10k.loc['Proceeds from Sale and Maturity of Debt Securities, Available-for-sale', '2017-08-31'] 

# A linha 'Proceeds from Issuance or Sale of Equity' na coluna '2017-08-31' está zerada
# As informações estão na linha 'Proceeds from Stock Options Exercised' 
df_cash_flow_10k.loc['Proceeds from Issuance or Sale of Equity', '2017-08-31'] = df_cash_flow_10k.loc['Proceeds from Stock Options Exercised', '2017-08-31'] 

# A linha 'Increase (Decrease) in Deferred Income Taxes' na coluna '2017-08-31' está zerada
# As informações estão na linha 'Increase (Decrease) in Deferred Revenue' 
df_cash_flow_10k.loc['Increase (Decrease) in Deferred Income Taxes', '2017-08-31'] = df_cash_flow_10k.loc['Increase (Decrease) in Deferred Revenue', '2017-08-31'] 

# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_cash_flow_10k = df_cash_flow_10k.drop([
    'Net Income (Loss), Including Portion Attributable to Noncontrolling Interest',
    'mu_ProceedsFromGovernmentIncentives',
    'mu_CapitalExpendituresIncurredNotYetPaidAndFinanceLeaseObligationsIncurred',
    'Capital Expenditures Incurred but Not yet Paid',
    'Payments to Acquire Available-for-sale Securities',
    'Proceeds from Sale of Available-for-sale Securities',
    'mu_DepreciationandAmortizationofIntangibleAssets',
    'Proceeds from Sale and Maturity of Debt Securities, Available-for-sale',
    'Proceeds from Stock Options Exercised',
    'Increase (Decrease) in Deferred Revenue'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_cash_flow_10k = df_cash_flow_10k.drop([
    'us-gaap_NetCashProvidedByUsedInOperatingActivitiesAbstract',
    'Sale Leaseback Transaction, Gross Proceeds, Financing Activities',
    'us-gaap_NetCashProvidedByUsedInOperatingActivitiesContinuingOperationsAbstract',
    'Payment, Tax Withholding, Share-based Payment Arrangement',
    'Payments to Noncontrolling Interests',
    'Business Combination, Step Acquisition, Equity Interest in Acquiree, Remeasurement Gain',
    'Income (Loss) from Equity Method Investments'
])

# Arrumando e adicionando alguns números no df
df_cash_flow_10k.loc['Increase (Decrease) in Other Current Liabilities', '2023-08-31'] = -817
df_cash_flow_10k.loc['Increase (Decrease) in Other Current Liabilities', '2022-09-01'] = 400
df_cash_flow_10k.loc['mu_ProceedsFromGovernmentAssistance', '2017-08-31'] = 21
df_cash_flow_10k.loc['Payments for Repurchase of Common Stock', '2018-08-30'] = -71
df_cash_flow_10k.loc['Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale', '2017-08-31'] = 194

# Renomeando as linhas do índice que estão sem formato 
df_cash_flow_10k = df_cash_flow_10k.rename(index={
    'mu_ProceedsFromGovernmentAssistance': 'Proceeds From Government Assistance',
    'mu_PaymentsOnEquipmentPurchaseContracts': 'Payments On Equipment Purchase Contracts',
    'mu_CapitalExpendituresIncurredOnPurchaseContractsNotYetPaid': 'Capital Expenditures Incurred On Purchase Contracts Not Yet Paid',
    'mu_PaymentsForDerivativeAndHedgeInvestingActivities': 'Payments For Derivative And Hedge Investing Activities',
    'mu_ProceedsForDerivativeAndHedgeInvestingActivities': 'Proceeds For Derivative And Hedge Investing Activities',
    'mu_BusinessCombinationConsiderationTransferredSettlementOfReceivablesPayablesWithAcquireeNet': 'Business Combination Consideration Transferred Settlement Of Receivables Payables With Acquiree Net',
})

df_cash_flow_10k

In [ ]:
# Arrumando alguns index dos dfs para ficar padronizado com os index utilizados nas funções p/ calcular os indicadores fundamentalistas
df_balance_sheet_10k = df_balance_sheet_10k.rename(index={
    'Debt Securities, Available-for-sale, Current': 'Marketable Securities, Current',
    'Debt Securities, Available-for-sale, Noncurrent': 'Long-term Marketable Investments, Non Current',
    "Stockholders' Equity Attributable to Parent": 'Total Shareholders Equity'
})

df_income_statement_10k = df_income_statement_10k.rename(index={
    'Revenue from Contract with Customer, Excluding Assessed Tax': 'Revenues',
    'Research and Development Expense': 'Research and Development',
    'Operating Income (Loss)': 'Operating Income',
    'Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest': 'Income Before Provision for Income Taxes',
    'Income Tax Expense (Benefit)': 'Provision for Income Taxes',
    'Net Income (Loss) Attributable to Parent': 'Net Income',
    'Weighted Average Number of Shares Outstanding, Basic': 'Number of Shares - Basic',
    'Weighted Average Number of Shares Outstanding, Diluted': 'Number of Shares - Diluted'
})

df_cash_flow_10k = df_cash_flow_10k.rename(index={
    'Payments to Acquire Property, Plant, and Equipment': 'Purchases Of Property And Equipment And Intangible Assets',
    'Payments of Ordinary Dividends, Common Stock': 'Payments of Dividends'
})

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//semicondutores//micron//mu_10k.xlsx') as writer:
    df_balance_sheet_10k.to_excel(writer, sheet_name='balance_sheet')
    df_income_statement_10k.to_excel(writer, sheet_name='income_statement')
    df_cash_flow_10k.to_excel(writer, sheet_name='cash_flow')

### Scraping Micron 10-Q

In [ ]:
# Número dos documentos que contém a DRE, BP e FC 
acc_documents_10q = f_sec.get_filtered_filings(ticker=ticker, ten_k=False, just_accession_numbers=True)

# Trocando os '-' para '' das strings ('0001045810-24-000264' -> '000104581024000264')
dict_acc_num_10q = {k: v.replace('-', '') for k, v in acc_documents_10q.items()}

# Transformando o dicionário em um df
df_lst_acc_num_10q_micron = pd.DataFrame.from_dict(dict_acc_num_10q, orient='index', columns=['acc_num'])

# Transformando em datetime o index
df_lst_acc_num_10q_micron.index = pd.to_datetime(df_lst_acc_num_10q_micron.index, format='%Y-%m-%d')
df_lst_acc_num_10q_micron

In [ ]:
# Selecionando os 'acc_num' dos 10-Q a partir de dez-2019 p/ iteração
df_lst_acc_num_10q_micron_sliced = df_lst_acc_num_10q_micron.loc['2019-11-28':]
df_lst_acc_num_10q_micron_sliced

In [ ]:
# Rótulos
labels_dict = f_sec.get_label_dictionary(ticker, headers=f_sec.header)

In [ ]:
# Fazendo o scraping do balance sheet (10-Q)
balance_sheet_lst = []

for acc_num in df_lst_acc_num_10q_micron_sliced['acc_num']:

    # Balance sheet
    balance_sheet = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='balance_sheet')
    balance_sheet = f_sec.rename_statement(balance_sheet, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    balance_sheet = balance_sheet.iloc[:, 0]
    balance_sheet_lst.append(balance_sheet)

# Juntando todos os dfs que estão em listas em apenas um df
df_balance_sheet_10q = pd.concat(balance_sheet_lst, axis=1)

# Substituindo os NaN por 0
df_balance_sheet_10q = df_balance_sheet_10q.fillna(0)

# A linha 'Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization' estão zeradas nas colunas '2020-05-28' e '2020-02-27' estão zeradas
# As informações estão na linha 'Property, Plant and Equipment, Net'
df_balance_sheet_10q.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2020-05-28'] = df_balance_sheet_10q.loc['Property, Plant and Equipment, Net', '2020-05-28'] 
df_balance_sheet_10q.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2020-02-27'] = df_balance_sheet_10q.loc['Property, Plant and Equipment, Net', '2020-02-27'] 
df_balance_sheet_10q.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2019-11-28'] = df_balance_sheet_10q.loc['Property, Plant and Equipment, Net', '2019-11-28'] 

# A linha 'mu_UnearnedGovernmentAssistanceNoncurrent' nas colunas '2025-05-29', '2025-02-27', '2024-11-28', '2023-06-01', '2023-03-02', '2022-12-01', '2022-06-02', '2022-03-03', '2021-12-02', '2021-06-03', '2021-03-04', '2020-12-03', '2020-05-28', '2020-02-27' e '2019-11-28' estão zeradas
# As informações estão nas linhas 'GovernmentAssistanceLiabilityNoncurrent' e 'mu_UnearnedGovernmentIncentivesNoncurrent'
df_balance_sheet_10q.loc['mu_UnearnedGovernmentAssistanceNoncurrent', '2025-05-29'] = df_balance_sheet_10q.loc['GovernmentAssistanceLiabilityNoncurrent', '2025-05-29'] 
df_balance_sheet_10q.loc['mu_UnearnedGovernmentAssistanceNoncurrent', '2025-02-27'] = df_balance_sheet_10q.loc['GovernmentAssistanceLiabilityNoncurrent', '2025-02-27'] 
df_balance_sheet_10q.loc['mu_UnearnedGovernmentAssistanceNoncurrent', '2024-11-28'] = df_balance_sheet_10q.loc['GovernmentAssistanceLiabilityNoncurrent', '2024-11-28'] 
df_balance_sheet_10q.loc['mu_UnearnedGovernmentAssistanceNoncurrent', '2023-06-01'] = df_balance_sheet_10q.loc['mu_UnearnedGovernmentIncentivesNoncurrent', '2023-06-01'] 
df_balance_sheet_10q.loc['mu_UnearnedGovernmentAssistanceNoncurrent', '2023-03-02'] = df_balance_sheet_10q.loc['mu_UnearnedGovernmentIncentivesNoncurrent', '2023-03-02'] 
df_balance_sheet_10q.loc['mu_UnearnedGovernmentAssistanceNoncurrent', '2022-12-01'] = df_balance_sheet_10q.loc['mu_UnearnedGovernmentIncentivesNoncurrent', '2022-12-01'] 
df_balance_sheet_10q.loc['mu_UnearnedGovernmentAssistanceNoncurrent', '2022-06-02'] = df_balance_sheet_10q.loc['mu_UnearnedGovernmentIncentivesNoncurrent', '2022-06-02'] 
df_balance_sheet_10q.loc['mu_UnearnedGovernmentAssistanceNoncurrent', '2022-03-03'] = df_balance_sheet_10q.loc['mu_UnearnedGovernmentIncentivesNoncurrent', '2022-03-03'] 
df_balance_sheet_10q.loc['mu_UnearnedGovernmentAssistanceNoncurrent', '2021-12-02'] = df_balance_sheet_10q.loc['mu_UnearnedGovernmentIncentivesNoncurrent', '2021-12-02'] 
df_balance_sheet_10q.loc['mu_UnearnedGovernmentAssistanceNoncurrent', '2021-06-03'] = df_balance_sheet_10q.loc['mu_UnearnedGovernmentIncentivesNoncurrent', '2021-06-03'] 
df_balance_sheet_10q.loc['mu_UnearnedGovernmentAssistanceNoncurrent', '2021-03-04'] = df_balance_sheet_10q.loc['mu_UnearnedGovernmentIncentivesNoncurrent', '2021-03-04'] 
df_balance_sheet_10q.loc['mu_UnearnedGovernmentAssistanceNoncurrent', '2020-12-03'] = df_balance_sheet_10q.loc['mu_UnearnedGovernmentIncentivesNoncurrent', '2020-12-03'] 
df_balance_sheet_10q.loc['mu_UnearnedGovernmentAssistanceNoncurrent', '2020-05-28'] = df_balance_sheet_10q.loc['mu_UnearnedGovernmentIncentivesNoncurrent', '2020-05-28'] 
df_balance_sheet_10q.loc['mu_UnearnedGovernmentAssistanceNoncurrent', '2020-02-27'] = df_balance_sheet_10q.loc['mu_UnearnedGovernmentIncentivesNoncurrent', '2020-02-27'] 
df_balance_sheet_10q.loc['mu_UnearnedGovernmentAssistanceNoncurrent', '2019-11-28'] = df_balance_sheet_10q.loc['mu_UnearnedGovernmentIncentivesNoncurrent', '2019-11-28'] 

# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_balance_sheet_10q = df_balance_sheet_10q.drop([
    'Property, Plant and Equipment, Net',
    'GovernmentAssistanceLiabilityNoncurrent',
    'mu_UnearnedGovernmentIncentivesNoncurrent',
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_balance_sheet_10q = df_balance_sheet_10q.drop([
    'us-gaap_AssetsAbstract',
    "Stockholders' Equity Attributable to Noncontrolling Interest",
    'mu_AssetsHeldForSaleCurrent',
    'Common Stock, Shares, Outstanding',
    'Common Stock, Par or Stated Value Per Share',
    'Common Stock, Shares Authorized',
    'Common Stock, Shares, Issued',
    'Redeemable Noncontrolling Interest, Equity, Preferred, Carrying Amount',
    "Stockholders' Equity, Including Portion Attributable to Noncontrolling Interest"
])
                                            
# Renomeando as linhas do índice que estão sem formato 
df_balance_sheet_10q = df_balance_sheet_10q.rename(index={
    'mu_UnearnedGovernmentAssistanceNoncurrent': 'Unearned Government Assistance Non Current'
})

df_balance_sheet_10q

In [ ]:
# Fazendo o scraping do income statement (10-Q)
income_statement_lst = []

for acc_num in df_lst_acc_num_10q_micron_sliced['acc_num']:

    # Income statement
    income_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='income_statement')
    income_statement = f_sec.rename_statement(income_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    income_statement = income_statement.iloc[:, 0]
    income_statement_lst.append(income_statement)

# Juntando todos os dfs que estão em listas em apenas um df
df_income_statement_10q = pd.concat(income_statement_lst, axis=1)

# Substituindo os NaN por 0
df_income_statement_10q = df_income_statement_10q.fillna(0)

# A linha 'InterestExpenseNonoperating' estão zeradas nas colunas '2024-05-30', '2024-02-29', '2023-11-30', '2023-06-01', '2023-03-02', '2022-12-01', '2022-06-02', '2022-03-03', '2021-12-02', '2021-06-03', '2021-03-04', 
# '2020-12-03', '2020-05-28', '2020-02-27' e '2019-11-28' estão zeradas
# As informações estão na linha 'Interest Expense'
df_income_statement_10q.loc['InterestExpenseNonoperating', '2024-05-30'] = df_income_statement_10q.loc['Interest Expense', '2024-05-30'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2024-02-29'] = df_income_statement_10q.loc['Interest Expense', '2024-02-29'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2023-11-30'] = df_income_statement_10q.loc['Interest Expense', '2023-11-30'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2023-06-01'] = df_income_statement_10q.loc['Interest Expense', '2023-06-01'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2023-03-02'] = df_income_statement_10q.loc['Interest Expense', '2023-03-02'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2022-12-01'] = df_income_statement_10q.loc['Interest Expense', '2022-12-01'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2022-06-02'] = df_income_statement_10q.loc['Interest Expense', '2022-06-02'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2022-03-03'] = df_income_statement_10q.loc['Interest Expense', '2022-03-03'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2021-12-02'] = df_income_statement_10q.loc['Interest Expense', '2021-12-02'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2021-06-03'] = df_income_statement_10q.loc['Interest Expense', '2021-06-03'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2021-03-04'] = df_income_statement_10q.loc['Interest Expense', '2021-03-04'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2020-12-03'] = df_income_statement_10q.loc['Interest Expense', '2020-12-03'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2020-05-28'] = df_income_statement_10q.loc['Interest Expense', '2020-05-28'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2020-02-27'] = df_income_statement_10q.loc['Interest Expense', '2020-02-27'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2019-11-28'] = df_income_statement_10q.loc['Interest Expense', '2019-11-28'] 


# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_income_statement_10q = df_income_statement_10q.drop([
    'us-gaap_IncomeStatementAbstract',
    'Interest Expense',
    'Net Income (Loss), Including Portion Attributable to Noncontrolling Interest',
    'Net Income (Loss) Attributable to Noncontrolling Interest',
    'Restructuring, Settlement and Impairment Provisions'
])

# Arrumando e adicionando alguns números no df
df_income_statement_10q.loc['Earnings Per Share, Diluted', '2024-05-30'] = 0.3
df_income_statement_10q.loc['Earnings Per Share, Diluted', '2023-11-30'] = -1.12
df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Diluted', '2023-11-30'] = 1100

# Renomeando as linhas do índice que estão sem formato 
df_balance_sheet_10q = df_balance_sheet_10q.rename(index={
    'mu_UnearnedGovernmentAssistanceNoncurrent': 'Unearned Government Assistance Non Current'
})

df_income_statement_10q

In [ ]:
# Fazendo o scraping do cash flow statement (10-Q)
cash_flow_statement_lst = []

for acc_num in df_lst_acc_num_10q_micron_sliced['acc_num']:

    # Cash flow statement
    cash_flow_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='cash_flow_statement')
    cash_flow_statement = f_sec.rename_statement(cash_flow_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    cash_flow_statement = cash_flow_statement.iloc[:, 0]
    cash_flow_statement_lst.append(cash_flow_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_cash_flow_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in cash_flow_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_cash_flow_10q = pd.concat([df_cash_flow_10q, df], axis=1)

# Substituindo os NaN por 0
df_cash_flow_10q = df_cash_flow_10q.fillna(0)

# A linha 'Proceeds from Sale and Maturity of Debt Securities, Available-for-sale' nas colunas '2023-06-01', '2023-03-02', '2022-12-01', '2022-06-02', '2022-03-03', '2021-12-02', '2021-06-03', '2021-03-04', '2020-12-03', '2020-05-28', '2020-02-27' e '2019-11-28' estão zeradas
# As informações estão na linha 'Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale'
df_cash_flow_10q.loc['Proceeds from Sale and Maturity of Debt Securities, Available-for-sale', '2023-06-01'] = df_cash_flow_10q.loc['Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale', '2023-06-01']
df_cash_flow_10q.loc['Proceeds from Sale and Maturity of Debt Securities, Available-for-sale', '2023-03-02'] = df_cash_flow_10q.loc['Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale', '2023-03-02']
df_cash_flow_10q.loc['Proceeds from Sale and Maturity of Debt Securities, Available-for-sale', '2022-12-01'] = df_cash_flow_10q.loc['Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale', '2022-12-01']
df_cash_flow_10q.loc['Proceeds from Sale and Maturity of Debt Securities, Available-for-sale', '2022-06-02'] = df_cash_flow_10q.loc['Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale', '2022-06-02']
df_cash_flow_10q.loc['Proceeds from Sale and Maturity of Debt Securities, Available-for-sale', '2022-03-03'] = df_cash_flow_10q.loc['Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale', '2022-03-03']
df_cash_flow_10q.loc['Proceeds from Sale and Maturity of Debt Securities, Available-for-sale', '2021-12-02'] = df_cash_flow_10q.loc['Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale', '2021-12-02']
df_cash_flow_10q.loc['Proceeds from Sale and Maturity of Debt Securities, Available-for-sale', '2021-06-03'] = df_cash_flow_10q.loc['Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale', '2021-06-03']
df_cash_flow_10q.loc['Proceeds from Sale and Maturity of Debt Securities, Available-for-sale', '2021-03-04'] = df_cash_flow_10q.loc['Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale', '2021-03-04']
df_cash_flow_10q.loc['Proceeds from Sale and Maturity of Debt Securities, Available-for-sale', '2020-12-03'] = df_cash_flow_10q.loc['Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale', '2020-12-03']
df_cash_flow_10q.loc['Proceeds from Sale and Maturity of Debt Securities, Available-for-sale', '2020-05-28'] = df_cash_flow_10q.loc['Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale', '2020-05-28']
df_cash_flow_10q.loc['Proceeds from Sale and Maturity of Debt Securities, Available-for-sale', '2020-02-27'] = df_cash_flow_10q.loc['Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale', '2020-02-27']
df_cash_flow_10q.loc['Proceeds from Sale and Maturity of Debt Securities, Available-for-sale', '2019-11-28'] = df_cash_flow_10q.loc['Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale', '2019-11-28']

# A linha 'mu_ProceedsFromGovernmentAssistance' nas colunas '2023-06-01', '2023-03-02', '2022-12-01', '2022-06-02', '2022-03-03', '2021-12-02', '2021-06-03', '2021-03-04', '2020-12-03', '2020-05-28', '2020-02-27' e '2019-11-28' estão zeradas
# As informações estão na linha 'mu_ProceedsFromGovernmentIncentives'
df_cash_flow_10q.loc['mu_ProceedsFromGovernmentAssistance', '2023-06-01'] = df_cash_flow_10q.loc['mu_ProceedsFromGovernmentIncentives', '2023-06-01']
df_cash_flow_10q.loc['mu_ProceedsFromGovernmentAssistance', '2023-03-02'] = df_cash_flow_10q.loc['mu_ProceedsFromGovernmentIncentives', '2023-03-02']
df_cash_flow_10q.loc['mu_ProceedsFromGovernmentAssistance', '2022-12-01'] = df_cash_flow_10q.loc['mu_ProceedsFromGovernmentIncentives', '2022-12-01']
df_cash_flow_10q.loc['mu_ProceedsFromGovernmentAssistance', '2022-06-02'] = df_cash_flow_10q.loc['mu_ProceedsFromGovernmentIncentives', '2022-06-02']
df_cash_flow_10q.loc['mu_ProceedsFromGovernmentAssistance', '2022-03-03'] = df_cash_flow_10q.loc['mu_ProceedsFromGovernmentIncentives', '2022-03-03']
df_cash_flow_10q.loc['mu_ProceedsFromGovernmentAssistance', '2021-12-02'] = df_cash_flow_10q.loc['mu_ProceedsFromGovernmentIncentives', '2021-12-02']
df_cash_flow_10q.loc['mu_ProceedsFromGovernmentAssistance', '2021-06-03'] = df_cash_flow_10q.loc['mu_ProceedsFromGovernmentIncentives', '2021-06-03']
df_cash_flow_10q.loc['mu_ProceedsFromGovernmentAssistance', '2021-03-04'] = df_cash_flow_10q.loc['mu_ProceedsFromGovernmentIncentives', '2021-03-04']
df_cash_flow_10q.loc['mu_ProceedsFromGovernmentAssistance', '2020-12-03'] = df_cash_flow_10q.loc['mu_ProceedsFromGovernmentIncentives', '2020-12-03']
df_cash_flow_10q.loc['mu_ProceedsFromGovernmentAssistance', '2020-05-28'] = df_cash_flow_10q.loc['mu_ProceedsFromGovernmentIncentives', '2020-05-28']
df_cash_flow_10q.loc['mu_ProceedsFromGovernmentAssistance', '2020-02-27'] = df_cash_flow_10q.loc['mu_ProceedsFromGovernmentIncentives', '2020-02-27']
df_cash_flow_10q.loc['mu_ProceedsFromGovernmentAssistance', '2019-11-28'] = df_cash_flow_10q.loc['mu_ProceedsFromGovernmentIncentives', '2019-11-28']

# A linha 'Net Income (Loss) Attributable to Parent' nas colunas '2021-06-03', '2021-03-04', '2020-12-03', '2020-05-28', '2020-02-27' e '2019-11-28' estão zeradas
# As informações estão na linha 'Net Income (Loss), Including Portion Attributable to Noncontrolling Interest'
df_cash_flow_10q.loc['Net Income (Loss) Attributable to Parent', '2021-06-03'] = df_cash_flow_10q.loc['Net Income (Loss), Including Portion Attributable to Noncontrolling Interest', '2021-06-03']
df_cash_flow_10q.loc['Net Income (Loss) Attributable to Parent', '2021-03-04'] = df_cash_flow_10q.loc['Net Income (Loss), Including Portion Attributable to Noncontrolling Interest', '2021-03-04']
df_cash_flow_10q.loc['Net Income (Loss) Attributable to Parent', '2020-12-03'] = df_cash_flow_10q.loc['Net Income (Loss), Including Portion Attributable to Noncontrolling Interest', '2020-12-03']
df_cash_flow_10q.loc['Net Income (Loss) Attributable to Parent', '2020-05-28'] = df_cash_flow_10q.loc['Net Income (Loss), Including Portion Attributable to Noncontrolling Interest', '2020-05-28']
df_cash_flow_10q.loc['Net Income (Loss) Attributable to Parent', '2020-02-27'] = df_cash_flow_10q.loc['Net Income (Loss), Including Portion Attributable to Noncontrolling Interest', '2020-02-27']
df_cash_flow_10q.loc['Net Income (Loss) Attributable to Parent', '2019-11-28'] = df_cash_flow_10q.loc['Net Income (Loss), Including Portion Attributable to Noncontrolling Interest', '2019-11-28']

# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_cash_flow_10q = df_cash_flow_10q.drop([
    'Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale',
    'mu_ProceedsFromGovernmentIncentives',
    'Net Income (Loss), Including Portion Attributable to Noncontrolling Interest'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_cash_flow_10q = df_cash_flow_10q.drop([
    'us-gaap_NetCashProvidedByUsedInOperatingActivitiesAbstract',
    'Restructuring Costs and Asset Impairment Charges'
])

# Arrumando e adicionando alguns números no df
df_cash_flow_10q.loc['Increase (Decrease) in Other Noncurrent Liabilities', '2025-05-29'] = -681
df_cash_flow_10q.loc['Increase (Decrease) in Other Noncurrent Liabilities', '2024-11-28'] = 132
df_cash_flow_10q.loc['Increase (Decrease) in Other Noncurrent Liabilities', '2024-05-30'] = 769


# Renomeando algumas linhas do índice
df_cash_flow_10q = df_cash_flow_10q.rename(index={
    'mu_ProceedsFromGovernmentAssistance': 'Proceeds From Government Assistance',
    'mu_PaymentsOnEquipmentPurchaseContracts': 'Payments On Equipment Purchase Contracts'
})

df_cash_flow_10q

In [ ]:
# Arrumando alguns index dos dfs para ficar padronizado com os index utilizados nas funções p/ calcular os indicadores fundamentalistas
df_balance_sheet_10q = df_balance_sheet_10q.rename(index={
    'Cash and Cash Equivalents, at Carrying Value': 'Cash and Cash Equivalents',
    'Debt Securities, Available-for-sale, Current': 'Marketable Securities, Current',
    "Stockholders' Equity Attributable to Parent": 'Total Shareholders Equity'
})

df_income_statement_10q = df_income_statement_10q.rename(index={
    'Revenue from Contract with Customer, Excluding Assessed Tax': 'Revenues',
    'Research and Development Expense': 'Research and Development',
    'Operating Income (Loss)': 'Operating Income',
    'Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest': 'Income Before Provision for Income Taxes',
    'Income Tax Expense (Benefit)': 'Provision for Income Taxes',
    'Net Income (Loss) Attributable to Parent': 'Net Income',
    'Weighted Average Number of Shares Outstanding, Basic': 'Number of Shares - Basic',
    'Weighted Average Number of Shares Outstanding, Diluted': 'Number of Shares - Diluted'
})

df_cash_flow_10q = df_cash_flow_10q.rename(index={
    'Payments to Acquire Property, Plant, and Equipment': 'Purchases Of Property And Equipment And Intangible Assets',
    'Payments of Ordinary Dividends, Common Stock': 'Payments of Dividends'
})

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//semicondutores//micron//mu_10q.xlsx') as writer:
    df_balance_sheet_10q.to_excel(writer, sheet_name='balance_sheet')
    df_income_statement_10q.to_excel(writer, sheet_name='income_statement')
    df_cash_flow_10q.to_excel(writer, sheet_name='cash_flow')

### Indicadores Fundamentalistas

#### Indicadores 10-K

In [ ]:
# Lendo os arquivos 10k
micron_bs = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//micron//mu_10k.xlsx', 
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

micron_is = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//micron//mu_10k.xlsx', 
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

micron_cf = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//micron//mu_10k.xlsx', 
    sheet_name='cash_flow', 
    index_col='Unnamed: 0'
)

# Transformando as colunas em datetime
micron_bs.columns = pd.to_datetime(micron_bs.columns)
micron_is.columns = pd.to_datetime(micron_is.columns)
micron_cf.columns = pd.to_datetime(micron_cf.columns)

# Transformando as colunas em apenas o ano
micron_bs.columns = micron_bs.columns.year
micron_is.columns = micron_is.columns.year
micron_cf.columns = micron_cf.columns.year

In [ ]:
# Preços da MU
micron_close = yf.download('MU', auto_adjust=True, multi_level_index=False, start='2017-01-01')['Close']

# Selecionando o último preço do mês de Janeiro de cada ano 
lst_years = [
    '2017', '2018', '2019', '2020', 
    '2021', '2022', '2023', '2024', 
    '2025',
]
last_sep_close = []

for year in lst_years:
    last_close = micron_close[f'{year}-09'].iloc[-1]
    last_sep_close.append(last_close)

# Transformando em uma série pandas
micron_close_sep = pd.Series(last_sep_close, index=lst_years)

# Revertendo o index da série p/ ficar igual com os dos indicadores
micron_close_sep = micron_close_sep.reindex(index=micron_close_sep.index[::-1])

# Arredondamento de duas casas decimais
micron_close_sep = round(micron_close_sep, 2)

# Não realizou split recentemente (https://br.investing.com/equities/micron-tech-historical-data-splits)

# Transformando o index em int
micron_close_sep.index = micron_close_sep.index.astype(int)

micron_close_sep

In [ ]:
# Criando uma lista só para fazer a indexação em que o ano mais recente está no começo
idx = micron_bs.columns

# Valor de Mercado
vm = f_eua.indicador_vm(ten_k=True, price=micron_close_sep, df_is=micron_is) 

# LPA
lpa = f_eua.indicador_lpa(ten_k=True, df_is=micron_is, idx=None)

# P/L Damodaran
p_lpa_damodaran = f_eua.indicador_p_lpa_2(ten_k=True, vm=vm, df_is=micron_is)

# P/L
p_lpa = f_eua.indicador_p_lpa(ten_k=True, price=micron_close_sep, lpa=lpa, idx=idx)

# L/P
lpa_p = f_eua.indicador_lpa_p(ten_k=True, price=micron_close_sep, lpa=lpa, idx=idx) 

# Depreciação
depreciacao = f_eua.indicador_depreciacao(ten_k=True, df_cf=micron_cf, first_quarter=None, idx=idx)

# EBITDA
ebitda = f_eua.indicador_ebitda(ten_k=True, df_is=micron_is, idx=None, depreciacao=depreciacao)

# Margem Líquida
m_liq = f_eua.indicador_ml(ten_k=True, df_is=micron_is)

# VPA
vpa = f_eua.indicador_vpa(ten_k=True, df_bs=micron_bs, df_is=micron_is)

# P/VPA
p_vpa = f_eua.indicador_p_vpa(ten_k=True, price=micron_close_sep, vpa=vpa, idx=idx)

# Caixa e Equivalentes
lst_caixa = ['Cash and Cash Equivalents, at Carrying Value', 'Marketable Securities, Current']
caixa = f_eua.indicador_caixa(ten_k=True, df_bs=micron_bs, lst_itens_caixa=lst_caixa)

# Dívida Bruta
lst_divida = ['Debt, Current', 'Long-term Debt and Lease Obligation', 'Operating Lease, Liability, Noncurrent']
div_bruta = f_eua.indicador_divida_bruta(ten_k=True, df_bs=micron_bs, lst_itens_divida=lst_divida)

# Dívida Líquida
div_liq = f_eua.indicador_divida_liquida(ten_k=True, caixa=caixa, div_bruta=div_bruta, idx=idx)

# Dívida Líquida / EBITDA
dl_ebitda = f_eua.indicador_dl_ebitda(ten_k=True, ebitda=ebitda, div_liq=div_liq, idx=None)

# Dívida Líquida / PL
dl_pl = f_eua.indicador_dl_pl(ten_k=True, div_liq=div_liq, df_bs=micron_bs) 

# EV/EBITDA
ev_ebitda = f_eua.indicador_ev_ebitda(vm=vm, div_liq=div_liq, ebitda=ebitda)

# ROE
roe = f_eua.indicador_roe(ten_k=True, df_is=micron_is, df_bs=micron_bs, idx=None)

# ROIC
roic = f_eua.indicador_roic(ten_k=True, df_is=micron_is, df_bs=micron_bs, div_bruta=div_bruta, idx=idx)

# FCO
fco = f_eua.indicador_fco(ten_k=True, df_cf=micron_cf, first_quarter=None) 

# FCI
fci = f_eua.indicador_fci(ten_k=True, df_cf=micron_cf, first_quarter=None) 

# FCF
fcf = f_eua.indicador_fcf(ten_k=True, df_cf=micron_cf, first_quarter=None) 

# CAPEX
lst_capex = ['Purchases Of Property And Equipment And Intangible Assets']

capex= f_eua.indicador_capex(ten_k=True, df_cf=micron_cf, first_quarter=None, lst_itens=lst_capex, idx=idx)

# Free Cash Flow
free_cash_flow = f_eua.indicador_free_cash_flow(fco=fco, capex=capex)

# Net CAPEX
net_capex = f_eua.indicador_net_capex(capex=capex, depreciation=depreciacao)

# R&D
rd_expense = f_eua.indicador_rd(ten_k=True, df_is=micron_is, idx=idx)

# Net CAPEX ajustado
adj_net_capex = f_eua.indicador_adj_net_capex(net_capex=net_capex, rd_expense=rd_expense)

# Working capital
working_capital = f_eua.indicador_working_capital(ten_k=True, df_bs=micron_bs)

# Change in non cash working capital
lst_itens_change_non_cash_wc = [
    'Increase (Decrease) in Receivables',
    'Increase (Decrease) in Inventories',
    'Increase (Decrease) in Accounts Payable and Accrued Liabilities',
    'Increase (Decrease) in Other Current Liabilities'
]

change_non_cash_wc = f_eua.indicador_change_non_cash_wc(ten_k=True, df_cf=micron_cf, lst_itens=lst_itens_change_non_cash_wc, first_quarter=None)

# Reinvestment rate
reinvestment_rate = f_eua.indicador_reinvestment_rate(ten_k=True, net_capex=net_capex, change_non_cash_wc=change_non_cash_wc, df_is=micron_is, idx=idx)

# New borrowing
lst_itens_new_borrowing = ['Proceeds from Issuance of Long-term Debt']

new_borrowing = f_eua.indicador_new_borrowing(ten_k=True, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=micron_cf, first_quarter=None, idx=idx)

# Debt paid
lst_itens_debt_paid = ['Repayment of Long-term Debt, Long-term Lease Obligation, and Capital Security']

debt_paid = f_eua.indicador_debt_paid(ten_k=True, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=micron_cf, first_quarter=None, idx=idx)

# Taxes on operating income
taxes_operating_income = f_eua.indicador_taxes_operating_income(ten_k=True, df_is=micron_is, idx=idx)

# FCFE
fcfe_10q = f_eua.indicador_fcfe(
    ten_k=True, 
    depreciacao=depreciacao, 
    capex=capex, 
    change_non_cash_wc=change_non_cash_wc, 
    new_borrowing=new_borrowing,
    debt_paid=debt_paid,
    df_is=micron_is,
    idx=idx
)

# FCFF
fcff_10q = f_eua.indicador_fcff(
        ten_k=True, 
        taxes_operating_income=taxes_operating_income,
        depreciacao=depreciacao, 
        capex=capex, 
        change_non_cash_wc=change_non_cash_wc, 
        df_is=micron_is,
        idx=idx
)

# Dividendo por Ação
dpa = f_eua.indicador_dpa(ten_k=True, df_cf=micron_cf, df_is=micron_is, div_nao_acum=None)

# Payout
payout = f_eua.indicador_payout(ten_k=True, df_cf=micron_cf, df_is=micron_is, div_nao_acum=None)

# Buyback
buyback = f_eua.indicador_buyback(ten_k=True, df_cf=micron_cf, first_quarter=None)

# Colocando todos os indicadores em uma lista
lst_indicadores = [
    p_lpa_damodaran,
    p_lpa,
    lpa_p,  
    ebitda, 
    m_liq,
    p_vpa,
    vm,
    div_bruta,
    caixa,
    div_liq,
    dl_ebitda,
    dl_pl,
    ev_ebitda,
    roe,
    roic,
    fco,
    fci,
    fcf,
    free_cash_flow,
    capex,
    net_capex,
    rd_expense,
    adj_net_capex,
    working_capital,
    reinvestment_rate,
    fcfe_10q,
    fcff_10q,
    dpa,
    payout,
    buyback
]

# Juntando todos os indicadores em um df
micron_indicadores_10k = pd.concat(lst_indicadores, axis=1)

# Nomes das colunas
nomes_col = [
    'P/L Damodaran', 
    'P/L', 
    'L/P', 
    'EBITDA', 
    'Margem Líquida',
    'P/VPA',
    'Valor de Mercado',
    'Dívida Bruta',
    'Caixa e Equivalentes',
    'Dívida Líquida',
    'Dívida Líquida/EBITDA',
    'Dívida Líquida/PL',
    'EV/EBITDA',
    'ROE',
    'ROIC',
    'FCO',
    'FCI',
    'FCF',
    'Free Cash Flow',
    'Capex',
    'Net Capex',
    'R&D',
    'Adj Net Capex',
    'Working Capital',
    'Reinvestment Rate',
    'FCFE',
    'FCFF',
    'DPA',
    'Payout',
    'Buyback'
]

# Atribuindo os novos nomes das colunas do df
micron_indicadores_10k.columns = nomes_col
micron_indicadores_10k

#### Indicadores 10-Q

In [ ]:
# Lendo os arquivos 10q
micron_bs_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//micron//mu_10q.xlsx', 
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

micron_is_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//micron//mu_10q.xlsx', 
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

micron_cf_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//micron//mu_10q.xlsx', 
    sheet_name='cash_flow', 
    index_col='Unnamed: 0'
)

# Aplicando o ajuste nas colunas
micron_bs_10q.columns = [f_sec.ajustar_mes_12_03_06(col) for col in micron_bs_10q.columns]
micron_is_10q.columns = [f_sec.ajustar_mes_12_03_06(col) for col in micron_is_10q.columns]
micron_cf_10q.columns = [f_sec.ajustar_mes_12_03_06(col) for col in micron_cf_10q.columns]

In [ ]:
# Preços da MU
micron_close = yf.download('MU', auto_adjust=True, multi_level_index=False, start='2020-01-01')['Close']

# Selecionando o último preço dos meses de Março, Junho e Dezembro
lst_years = [
    '2020', '2021', '2022', '2023', 
    '2024', '2025',
]
lst_months = ['03', '06', '12']
last_close_months = []

for year in lst_years:
    for month in lst_months:
        last_close = micron_close[f'{year}-{month}'].dropna().iloc[-1]  # Dropna para lidar com meses sem dados
        last_close_months.append(last_close)

# Criando uma lista em que junte os meses e os anos para que seja o index da serie pandas
lst_idx = []

for year in lst_years:
    for month in lst_months:
        lst_idx.append(f"{month}-{year}")

# Transformando em uma série pandas
micron_close_months = pd.Series(last_close_months, index=lst_idx)

# Revertendo o index da série p/ ficar igual com os dos indicadores
micron_close_months = micron_close_months.reindex(index=micron_close_months.index[::-1])

# Retirando os preços de '06-2020' e '03-2020'
micron_close_months = micron_close_months[:-2]


# No ano mais atual, como ainda ele não acabou, não consigo fazer o loop com todos meses (Março, Junho e Dezembro)
actual_year_int = datetime.now().year  
actual_year_str = str(actual_year_int) 

lst_actual_months = [
    # '03',
    # '06',
    # '12'
] 

last_close_actual_months = []

for month in lst_actual_months:
    last_close = micron_close[f'{actual_year_str}-{month}'].dropna().iloc[-1]  # Dropna para lidar com meses sem dados
    last_close_actual_months.append(last_close)

# Criando uma lista em que junte os meses e os anos para que seja o index da serie pandas
lst_idx_actual = []

for month in lst_actual_months:
    lst_idx_actual.append(f"{month}-{actual_year_str}")

# Transformando em uma série pandas
micron_close_months_atual = pd.Series(last_close_actual_months, index=lst_idx_actual)

# Revertendo o index da série p/ ficar igual com os dos indicadores
micron_close_months_atual = micron_close_months_atual.reindex(index=micron_close_months_atual.index[::-1])

# Juntando as duas series pandas
micron_close_months = pd.concat([micron_close_months_atual, micron_close_months])

# Arredondamento de duas casas decimais
micron_close_months = round(micron_close_months, 2)

# Não realizou split recentemente (https://br.investing.com/equities/micron-tech-historical-data-splits)

# Substituindo o index para ficar igual ao dos dados fundamentalistas
micron_close_months.index = micron_bs_10q.columns[:-3]

micron_close_months

In [ ]:
# Criando uma lista só para fazer a indexação em que o ano mais recente está no começo
idx_10q = micron_bs_10q.columns

# Valor de Mercado
vm_10q = f_eua.indicador_vm(ten_k=False, price=micron_close_months, df_is=micron_is_10q)

# LPA
lpa_acum = f_eua.indicador_lpa(ten_k=False, df_is=micron_is_10q, idx=idx_10q)[1]
lpa_10q = f_eua.indicador_lpa(ten_k=False, df_is=micron_is_10q, idx=idx_10q)[0]

# P/L Damodaran
p_lpa_10q_damodaran = f_eua.indicador_p_lpa_2(ten_k=False, vm=vm_10q, df_is=micron_is_10q)

# P/L
p_lpa_10q = f_eua.indicador_p_lpa(ten_k=False, price=micron_close_months, lpa=lpa_acum, idx=idx_10q)

# L/P
lpa_p_10q = f_eua.indicador_lpa_p(ten_k=False, price=micron_close_months, lpa=lpa_acum, idx=idx_10q) 

# Depreciação
depreciacao_10q = f_eua.indicador_depreciacao(ten_k=False, df_cf=micron_cf_10q, first_quarter='12', idx=idx_10q)[0]
depreciacao_10q_sliced = f_eua.indicador_depreciacao(ten_k=False, df_cf=micron_cf_10q, first_quarter='12', idx=idx_10q)[1]
depreciacao_10q_acum = f_eua.indicador_depreciacao(ten_k=False, df_cf=micron_cf_10q, first_quarter='12', idx=idx_10q)[2]

# EBITDA
ebitda_10q = f_eua.indicador_ebitda(ten_k=False, df_is=micron_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[0]
ebitda_10q_sliced = f_eua.indicador_ebitda(ten_k=False, df_is=micron_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[1]
ebitda_10q_acum = f_eua.indicador_ebitda(ten_k=False, df_is=micron_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[2]

# Margem Líquida
m_liq_10q = f_eua.indicador_ml(ten_k=False, df_is=micron_is_10q)

# VPA
vpa_10q = f_eua.indicador_vpa(ten_k=False, df_bs=micron_bs_10q, df_is=micron_is_10q)

# P/VPA
p_vpa_10q = f_eua.indicador_p_vpa(ten_k=False, price=micron_close_months, vpa=vpa_10q, idx=idx_10q)

# Caixa e Equivalentes
lst_caixa = ['Cash and Cash Equivalents', 'Marketable Securities, Current']
caixa_10q = f_eua.indicador_caixa(ten_k=False, df_bs=micron_bs_10q, lst_itens_caixa=lst_caixa)

# Dívida Bruta
lst_divida = ['Debt, Current', 'Long-term Debt and Lease Obligation', 'Operating Lease, Liability, Noncurrent']
div_bruta_10q = f_eua.indicador_divida_bruta(ten_k=False, df_bs=micron_bs_10q, lst_itens_divida=lst_divida)

# Dívida Líquida
div_liq_10q = f_eua.indicador_divida_liquida(ten_k=False, caixa=caixa_10q, div_bruta=div_bruta_10q, idx=idx_10q)

# Dívida Líquida / EBITDA
dl_ebitda_10q = f_eua.indicador_dl_ebitda(ten_k=False, ebitda=ebitda_10q_acum, div_liq=div_liq_10q, idx=idx_10q)

# Dívida Líquida / PL
dl_pl_10q = f_eua.indicador_dl_pl(ten_k=False, div_liq=div_liq_10q, df_bs=micron_bs_10q) 

# EV/EBITDA
ev_ebitda_10q = f_eua.indicador_ev_ebitda(vm=vm_10q, div_liq=div_liq_10q, ebitda=ebitda_10q_acum)

# ROE
roe_10q = f_eua.indicador_roe(ten_k=False, df_is=micron_is_10q, df_bs=micron_bs_10q, idx=idx_10q)

# ROIC 
roic_10q = f_eua.indicador_roic(ten_k=False, df_is=micron_is_10q, df_bs=micron_bs_10q, div_bruta=div_bruta_10q, idx=idx_10q)

# FCO
fco_10q = f_eua.indicador_fco(ten_k=False, df_cf=micron_cf_10q, first_quarter='12') 

# FCI
fci_10q = f_eua.indicador_fci(ten_k=False, df_cf=micron_cf_10q, first_quarter='12') 

# FCF
fcf_10q = f_eua.indicador_fcf(ten_k=False, df_cf=micron_cf_10q, first_quarter='12') 

# CAPEX
lst_capex_10q = ['Purchases Of Property And Equipment And Intangible Assets']

capex_10q = f_eua.indicador_capex(ten_k=False, df_cf=micron_cf_10q, first_quarter='12', lst_itens=lst_capex_10q, idx=idx_10q)[0]
capex_10q_sliced = f_eua.indicador_capex(ten_k=False, df_cf=micron_cf_10q, first_quarter='12', lst_itens=lst_capex_10q, idx=idx_10q)[1]
capex_10q_acum = f_eua.indicador_capex(ten_k=False, df_cf=micron_cf_10q, first_quarter='12', lst_itens=lst_capex_10q, idx=idx_10q)[2]

# Free Cash Flow
free_cash_flow_10q = f_eua.indicador_free_cash_flow(fco=fco_10q, capex=capex_10q_sliced)

# Net CAPEX
net_capex_10q = f_eua.indicador_net_capex(capex=capex_10q_acum, depreciation=depreciacao_10q_acum)

# R&D
rd_expense_10q = f_eua.indicador_rd(ten_k=False, df_is=micron_is_10q, idx=idx_10q)[0]
rd_expense_10q_sliced = f_eua.indicador_rd(ten_k=False, df_is=micron_is_10q, idx=idx_10q)[1]
rd_expense_10q_acum = f_eua.indicador_rd(ten_k=False, df_is=micron_is_10q, idx=idx_10q)[2]

# Net CAPEX ajustado
adj_net_capex_10q = f_eua.indicador_adj_net_capex(net_capex=net_capex_10q, rd_expense=rd_expense_10q_acum)

# Working capital
working_capital_10q = f_eua.indicador_working_capital(ten_k=False, df_bs=micron_bs_10q)

# Change in non cash working capital
lst_itens_change_non_cash_wc = [
    'Increase (Decrease) in Receivables',
    'Increase (Decrease) in Inventories',
    'Increase (Decrease) in Other Current Assets',
    'Increase (Decrease) in Accounts Payable and Accrued Liabilities',
    'Increase (Decrease) in Other Current Liabilities'
]

change_non_cash_wc_10q = f_eua.indicador_change_non_cash_wc(ten_k=False, df_cf=micron_cf_10q, lst_itens=lst_itens_change_non_cash_wc, first_quarter='12')[0]
change_non_cash_wc_10q_sliced = f_eua.indicador_change_non_cash_wc(ten_k=False, df_cf=micron_cf_10q, lst_itens=lst_itens_change_non_cash_wc, first_quarter='12')[1]

# Reinvestment rate
reinvestment_rate_10q = f_eua.indicador_reinvestment_rate(ten_k=False, net_capex=adj_net_capex_10q, change_non_cash_wc=change_non_cash_wc_10q, df_is=micron_is_10q, idx=idx_10q)

# New borrowing
lst_itens_new_borrowing = ['Proceeds from Issuance of Long-term Debt']

new_borrowing_10q = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=micron_cf_10q, first_quarter='12', idx=idx_10q)[0]
new_borrowing_10q_sliced = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=micron_cf_10q, first_quarter='12', idx=idx_10q)[1]
new_borrowing_10q_acum = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=micron_cf_10q, first_quarter='12', idx=idx_10q)[2]

# Debt paid
lst_itens_debt_paid = ['Repayment of Long-term Debt, Long-term Lease Obligation, and Capital Security']

debt_paid_10q = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=micron_cf_10q, first_quarter='12', idx=idx_10q)[0]
debt_paid_10q_sliced = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=micron_cf_10q, first_quarter='12', idx=idx_10q)[1]
debt_paid_10q_acum = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=micron_cf_10q, first_quarter='12', idx=idx_10q)[2]

# Taxes on operating income
taxes_operating_income_10q = f_eua.indicador_taxes_operating_income(ten_k=False, df_is=micron_is_10q, idx=idx_10q)

# FCFE
fcfe_10q = f_eua.indicador_fcfe(
    ten_k=False, 
    depreciacao=depreciacao_10q_acum, 
    capex=capex_10q_acum, 
    change_non_cash_wc=change_non_cash_wc_10q_sliced, 
    new_borrowing=new_borrowing_10q_acum, 
    debt_paid=debt_paid_10q_acum, 
    df_is=micron_is_10q,
    idx=idx_10q
)

# FCFF
fcff_10q = f_eua.indicador_fcff(
        ten_k=False, 
        taxes_operating_income=taxes_operating_income_10q,
        depreciacao=depreciacao_10q_acum, 
        capex=capex_10q_acum, 
        change_non_cash_wc=change_non_cash_wc_10q_sliced, 
        df_is=micron_is_10q,
        idx=idx_10q
)

# Dividendo não acumulado
div_nao_acum = f_eua.indicador_div_nao_acum(df_cf=micron_cf_10q, first_quarter='12')

# Dividendo por Ação
dpa_10q = f_eua.indicador_dpa(ten_k=False, df_cf=micron_cf_10q, df_is=micron_is_10q, div_nao_acum=div_nao_acum)

# Payout
payout_10q = f_eua.indicador_payout(ten_k=False, df_cf=micron_cf_10q, df_is=micron_is_10q, div_nao_acum=div_nao_acum)

# Buyback
buyback_10q = f_eua.indicador_buyback(ten_k=False, df_cf=micron_cf_10q,  first_quarter='12')

# Lista com todos os indicadores
lst_indicadores_10q = [
    p_lpa_10q_damodaran,
    p_lpa_10q,
    lpa_p_10q,  
    ebitda_10q_sliced, 
    m_liq_10q,
    p_vpa_10q,
    vm_10q,
    div_bruta_10q,
    caixa_10q,
    div_liq_10q,
    dl_ebitda_10q,
    dl_pl_10q,
    ev_ebitda_10q,
    roe_10q,
    roic_10q,
    fco_10q,
    fci_10q,
    fcf_10q,
    free_cash_flow_10q,
    capex_10q_sliced,
    net_capex_10q,
    rd_expense_10q_sliced,
    adj_net_capex_10q,
    working_capital_10q,
    reinvestment_rate_10q,
    fcfe_10q,
    fcff_10q,
    dpa_10q,
    payout_10q,
    buyback_10q
]

# Juntando todos os indicadores em um df
micron_indicadores_10q = pd.concat(lst_indicadores_10q, axis=1)

# Nomes das colunas
nomes_col = [
    'P/L Damodaran', 
    'P/L', 
    'L/P', 
    'EBITDA', 
    'Margem Líquida',
    'P/VPA',
    'Valor de Mercado',
    'Dívida Bruta',
    'Caixa e Equivalentes',
    'Dívida Líquida',
    'Dívida Líquida/EBITDA',
    'Dívida Líquida/PL',
    'EV/EBITDA',
    'ROE',
    'ROIC',
    'FCO',
    'FCI',
    'FCF',
    'Free Cash Flow',
    'Capex',
    'Net Capex',
    'R&D',
    'Adj Net Capex',
    'Working Capital',
    'Reinvestment Rate',
    'FCFE',
    'FCFF',
    'DPA',
    'Payout',
    'Buyback'
]

# Atribuindo os novos nomes das colunas do df
micron_indicadores_10q.columns = nomes_col
micron_indicadores_10q

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//semicondutores//micron//mu_indicators.xlsx') as writer:
    micron_indicadores_10k.to_excel(writer, sheet_name='indicators_10k')
    micron_indicadores_10q.to_excel(writer, sheet_name='indicators_10q')

## Broadcom

* 1Q -> Lançamento em Feveiro (02);
* 2Q -> Lançamento em Maio (05);
* 3Q -> Lançamento em Agosto (08);
* 10K -> Lançamento em Novembro (11).

In [ ]:
ticker = 'avgo'
f_sec.cik_matching_ticker(ticker)

### Scraping Broadcom 10-K

In [ ]:
# Número dos documentos que contém a DRE, BP e FC 
acc_documents = f_sec.get_filtered_filings(ticker=ticker, ten_k=True, just_accession_numbers=True)

# Trocando os '-' para '' das strings ('0001045810-24-000264' -> '000104581024000264')
dict_acc_num = {k: v.replace('-', '') for k, v in acc_documents.items()}

# Transformando o dicionário em um df
df_lst_acc_num_broadcom = pd.DataFrame.from_dict(dict_acc_num, orient='index', columns=['acc_num'])

# Transformando em datetime o index
df_lst_acc_num_broadcom.index = pd.to_datetime(df_lst_acc_num_broadcom.index, format='%Y-%m-%d')
df_lst_acc_num_broadcom

* No site da SEC não possui o 'acc_num' do ano de 2017. Para pegar os dados deste ano, eu tive que usar o 'accession_number' de 2018.

In [ ]:
# Rótulos
labels_dict = f_sec.get_label_dictionary(ticker, headers=f_sec.header)

In [ ]:
# Fazendo o scraping do balance sheet (10-K)
balance_sheet_lst_2 = []

for acc_num in df_lst_acc_num_broadcom['acc_num']:

    # Balance sheet
    balance_sheet = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='balance_sheet')
    balance_sheet = f_sec.rename_statement(balance_sheet, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    balance_sheet = balance_sheet.iloc[:, 0]
    balance_sheet_lst_2.append(balance_sheet)

# Juntando todos os dfs que estão em listas em apenas um df
df_balance_sheet_10k = pd.concat(balance_sheet_lst_2, axis=1)

# Substituindo os NaN por 0
df_balance_sheet_10k = df_balance_sheet_10k.fillna(0)

# A linha 'Additional Paid in Capital' na coluna '2018-11-04' está zerada
# A informação está na linha 'Common Stocks, Including Additional Paid in Capital'
df_balance_sheet_10k.loc['Additional Paid in Capital', '2018-11-04'] = df_balance_sheet_10k.loc['Common Stocks, Including Additional Paid in Capital', '2018-11-04'] 

# A linha 'Long-term Debt and Lease Obligation, Current' nas colunas '2021-10-31', '2020-11-01'	e '2019-11-03' estão zeradas
# A informação está na linha 'Long-term Debt, Current Maturities'
df_balance_sheet_10k.loc['Long-term Debt and Lease Obligation, Current', '2021-10-31'] = df_balance_sheet_10k.loc['Long-term Debt, Current Maturities', '2021-10-31'] 
df_balance_sheet_10k.loc['Long-term Debt and Lease Obligation, Current', '2020-11-01'] = df_balance_sheet_10k.loc['Long-term Debt, Current Maturities', '2020-11-01'] 
df_balance_sheet_10k.loc['Long-term Debt and Lease Obligation, Current', '2019-11-03'] = df_balance_sheet_10k.loc['Long-term Debt, Current Maturities', '2019-11-03'] 

# A linha 'Long-term Debt and Lease Obligation' nas colunas '2021-10-31', '2020-11-01'	e '2019-11-03' estão zeradas
# A informação está na linha 'Long-term Debt, Excluding Current Maturities'
df_balance_sheet_10k.loc['Long-term Debt and Lease Obligation', '2021-10-31'] = df_balance_sheet_10k.loc['Long-term Debt, Excluding Current Maturities', '2021-10-31'] 
df_balance_sheet_10k.loc['Long-term Debt and Lease Obligation', '2020-11-01'] = df_balance_sheet_10k.loc['Long-term Debt, Excluding Current Maturities', '2020-11-01'] 
df_balance_sheet_10k.loc['Long-term Debt and Lease Obligation', '2019-11-03'] = df_balance_sheet_10k.loc['Long-term Debt, Excluding Current Maturities', '2019-11-03'] 

# Removendo as linhas que movemos os seus itens para as linhas corretas
df_balance_sheet_10k = df_balance_sheet_10k.drop([
    'Common Stocks, Including Additional Paid in Capital',
    'Long-term Debt, Current Maturities',
    'Long-term Debt, Excluding Current Maturities'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_balance_sheet_10k = df_balance_sheet_10k.drop([
    'us-gaap_AssetsCurrentAbstract',
    'Preferred Stock, Value, Issued',
    "Stockholders' Equity Attributable to Parent",
    "Stockholders' Equity Attributable to Noncontrolling Interest"
])

# Renomeando as linhas do índice que estão sem formato 
df_balance_sheet_10k = df_balance_sheet_10k.rename(index={
    'avgo_TemporaryEquityPreferredStockDividendObligation': 'Temporary Equity Preferred Stock Dividend Obligation'
})


# Para pegar os dados de 2017, eu tive que usar o 'accession_number' de 2018
# Dados do 'balance_sheet' do ano de 2017
df_balance_sheet_2017 = f_sec.process_one_statement(ticker=ticker, accession_number='000173016818000084', statement_name='balance_sheet')
df_balance_sheet_2017  = f_sec.rename_statement(df_balance_sheet_2017, label_dictionary=labels_dict)
# Selecionando apenas a coluna de 2017
df_balance_sheet_2017 = df_balance_sheet_2017.iloc[:, 1:] 
# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_balance_sheet_2017 = df_balance_sheet_2017.drop([
    'us-gaap_AssetsCurrentAbstract',
    'Preferred Stock, Value, Issued'
])

# Lista com os dfs do 'balance_shhet'
balance_sheet_lst_2 = [
    df_balance_sheet_10k, 
    df_balance_sheet_2017
]

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_balance_sheet_10k_final = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in balance_sheet_lst_2:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_balance_sheet_10k_final = pd.concat([df_balance_sheet_10k_final, df], axis=1)

# Substituindo os NaN por 0
df_balance_sheet_10k_final  = df_balance_sheet_10k_final .fillna(0)

# A linha 'Long-term Debt and Lease Obligation, Current' na coluna '2017-10-29' está zerada
# A informação está na linha 'Long-term Debt, Current Maturities'
df_balance_sheet_10k_final.loc['Long-term Debt and Lease Obligation, Current', '2017-10-29'] = df_balance_sheet_10k_final.loc['Long-term Debt, Current Maturities', '2017-10-29'] 

# A linha 'Long-term Debt and Lease Obligation' na coluna '2017-10-29' está zerada
# A informação está na linha 'Long-term Debt, Excluding Current Maturities'
df_balance_sheet_10k_final.loc['Long-term Debt and Lease Obligation', '2017-10-29'] = df_balance_sheet_10k_final.loc['Long-term Debt, Excluding Current Maturities', '2017-10-29'] 

# A linha 'Additional Paid in Capital' na coluna '2017-10-29' está zerada
# A informação está na linha 'Common Stocks, Including Additional Paid in Capital'
df_balance_sheet_10k_final.loc['Additional Paid in Capital', '2017-10-29'] = df_balance_sheet_10k_final.loc['Common Stocks, Including Additional Paid in Capital', '2017-10-29'] 

# A linha 'Temporary Equity Preferred Stock Dividend Obligation' na coluna '2017-10-29' está zerada
# A informação está na linha 'Stockholders' Equity Attributable to Noncontrolling Interest'
df_balance_sheet_10k_final.loc['Temporary Equity Preferred Stock Dividend Obligation', '2017-10-29'] = df_balance_sheet_10k_final.loc["Stockholders' Equity Attributable to Noncontrolling Interest", '2017-10-29'] 

#  Removendo as linhas que movemos os seus itens para as linhas corretas
df_balance_sheet_10k_final = df_balance_sheet_10k_final.drop([
    'Long-term Debt, Current Maturities',
    'Long-term Debt, Excluding Current Maturities',
    'Common Stocks, Including Additional Paid in Capital',
    "Stockholders' Equity Attributable to Noncontrolling Interest"
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_balance_sheet_10k_final = df_balance_sheet_10k_final.drop([
    "Stockholders' Equity Attributable to Parent",
    'Common Stock, Value, Issued',
    'Temporary Equity Preferred Stock Dividend Obligation'
])

# Arrumando e adicionando alguns números no df
df_balance_sheet_10k_final.loc['Long-term Debt and Lease Obligation', '2018-11-04'] = 17493

df_balance_sheet_10k_final 

In [ ]:
# Nos anos de 2019 e 2018, o 'income_statement' retorna a tabela do 'R5.htm' que não é do 'income_statement'
# Para retornar os dados corretos do 'income_statement'- eu tenho que passar o statement_name='income_statement_2'. Isso faz com que retorne a tabela do 'R4.htm'
# Para que statement_name='income_statement_2' funcione, ...
# eu tive que adicionar essa chave e o valor 'consolidated statements of operations consolidated statements of operations' no dicionário 'statement_keys_map'

# Dados do 'income_statement' dos anos de 2018 e 2017
income_statement_2018_2017 = f_sec.process_one_statement(ticker='avgo', accession_number='000173016818000084', statement_name='income_statement_2')
income_statement_2018_2017 = f_sec.rename_statement(income_statement_2018_2017, label_dictionary=labels_dict)
# Selecionando as colunas dos anos de 2018 e 2017
income_statement_2018_2017 = income_statement_2018_2017.iloc[:,8:10]
# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
income_statement_2018_2017 = income_statement_2018_2017.drop('us-gaap_IncomeStatementAbstract')

# Dados do 'income_statement' do ano de 2021, 2020 e 2019
income_statement_2021_2020_2019 = f_sec.process_one_statement(ticker='avgo', accession_number='000173016821000153', statement_name='income_statement')
income_statement_2021_2020_2019 = f_sec.rename_statement(income_statement_2021_2020_2019, label_dictionary=labels_dict)
# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
income_statement_2021_2020_2019 = income_statement_2021_2020_2019.drop('srt_CondensedIncomeStatementsCaptionsLineItems')

# Dados do 'income_statement' do ano de 2024, 2023 e 2022
income_statement_2024_2023_2022 = f_sec.process_one_statement(ticker='avgo', accession_number='000173016824000139', statement_name='income_statement')
income_statement_2024_2023_2022 = f_sec.rename_statement(income_statement_2024_2023_2022, label_dictionary=labels_dict)
# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
income_statement_2024_2023_2022 = income_statement_2024_2023_2022.drop('srt_CondensedIncomeStatementsCaptionsLineItems')

# Dados do 'income_statement' do ano de 2025
income_statement_2025 = f_sec.process_one_statement(ticker='avgo', accession_number='000173016825000121', statement_name='income_statement')
income_statement_2025 = f_sec.rename_statement(income_statement_2025, label_dictionary=labels_dict)
# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
income_statement_2025 = income_statement_2025.drop('srt_CondensedIncomeStatementsCaptionsLineItems')
# Selecionando a coluna do ano de 2025
income_statement_2025 = income_statement_2025.iloc[:,0:1]

# Lista com os dfs do 'income_statement'
income_statement_lst = [
    income_statement_2025,
    income_statement_2024_2023_2022, 
    income_statement_2021_2020_2019, 
    income_statement_2018_2017
]

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_income_statement_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in income_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_income_statement_10k = pd.concat([df_income_statement_10k, df], axis=1)

# Substituindo os NaN por 0
df_income_statement_10k = df_income_statement_10k.fillna(0)

# A linha 'Revenue from Contract with Customer, Excluding Assessed Tax' nas colunas '2018-11-04' e '2017-10-29' estão zeradas
# A informação está na linha 'Long-term Debt, Current Maturities'
df_income_statement_10k.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2018-11-04'] = df_income_statement_10k.loc['Revenues', '2018-11-04'] 
df_income_statement_10k.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2017-10-29'] = df_income_statement_10k.loc['Revenues', '2017-10-29'] 

# A linha 'avgo_CostofProductsSold' nas colunas '2018-11-04' e '2017-10-29' estão zeradas
# A informação está na linha 'avgo_CostOfProductsSold'
df_income_statement_10k.loc['avgo_CostofProductsSold', '2018-11-04'] = df_income_statement_10k.loc['avgo_CostOfProductsSold', '2018-11-04'] 
df_income_statement_10k.loc['avgo_CostofProductsSold', '2017-10-29'] = df_income_statement_10k.loc['avgo_CostOfProductsSold', '2017-10-29'] 

# A linha 'Cost of Revenue' nas colunas '2018-11-04' e '2017-10-29' estão zeradas
# A informação está na linha 'Cost of Goods and Services Sold'
df_income_statement_10k.loc['Cost of Revenue', '2018-11-04'] = df_income_statement_10k.loc['Cost of Goods and Services Sold', '2018-11-04'] 
df_income_statement_10k.loc['Cost of Revenue', '2017-10-29'] = df_income_statement_10k.loc['Cost of Goods and Services Sold', '2017-10-29'] 

# A linha 'Net Income (Loss) Available to Common Stockholders, Basic' nas colunas '2018-11-04' e '2017-10-29' estão zeradas
# A informação está na linha 'Net Income (Loss) Attributable to Parent'
df_income_statement_10k.loc['Net Income (Loss) Available to Common Stockholders, Basic', '2018-11-04'] = df_income_statement_10k.loc['Net Income (Loss) Attributable to Parent', '2018-11-04'] 
df_income_statement_10k.loc['Net Income (Loss) Available to Common Stockholders, Basic', '2017-10-29'] = df_income_statement_10k.loc['Net Income (Loss) Attributable to Parent', '2017-10-29'] 

# A linha 'Interest Expense' na coluna '2025-11-02' está zerada
# A informação está na linha 'InterestExpenseNonoperating'
df_income_statement_10k.loc['Interest Expense', '2025-11-02'] = df_income_statement_10k.loc['InterestExpenseNonoperating', '2025-11-02'] 


#  Removendo as linhas que movemos os seus itens para as linhas corretas
df_income_statement_10k = df_income_statement_10k.drop([
    'Revenues',
    'avgo_CostOfProductsSold',
    'Cost of Goods and Services Sold',
    'Net Income (Loss) Attributable to Parent',
    'InterestExpenseNonoperating',
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_income_statement_10k = df_income_statement_10k.drop([
    'Income (Loss) from Discontinued Operations and Disposal of Discontinued Operations, Net of Tax, Per Diluted Share',
    'Income (Loss) from Continuing Operations, Per Basic Share',
    'Income (Loss) from Discontinued Operations and Disposal of Discontinued Operations, Net of Tax, Per Basic Share',
    'Income (Loss) from Continuing Operations, Per Diluted Share',
    'Income (Loss) from Continuing Operations, Net of Tax, Including Portion Attributable to Noncontrolling Interest',
    'Income (Loss) from Discontinued Operations, Net of Tax, Including Portion Attributable to Noncontrolling Interest',
    'Income (Loss) from Discontinued Operations, Net of Tax, Including Portion Attributable to Noncontrolling Interest',
    'avgo_Amortizationofinventoryvaluationstepup',
    'Litigation Settlement, Expense',
    'Cost-method Investments, Other than Temporary Impairment',
    'Gain (Loss) on Extinguishment of Debt',
    'Net Income (Loss) Attributable to Noncontrolling Interest'
])

# Arrumando e adicionando alguns números no df
df_income_statement_10k.loc['avgo_CostofSubscriptionsandServices', '2017-10-29'] = 44
df_income_statement_10k.loc['avgo_CostofSubscriptionsandServices', '2018-11-04'] = 97
df_income_statement_10k.loc['Weighted Average Number of Shares Outstanding, Basic', '2022-10-30'] = 409
df_income_statement_10k.loc['Weighted Average Number of Shares Outstanding, Diluted', '2022-10-30'] = 423
df_income_statement_10k.loc['Weighted Average Number of Shares Outstanding, Basic', '2023-10-29'] = 415
df_income_statement_10k.loc['Weighted Average Number of Shares Outstanding, Diluted', '2023-10-29'] = 427
df_income_statement_10k.loc['Net Income (Loss) Available to Common Stockholders, Basic', '2025-11-02'] = 23126		

# Renomeando as linhas do índice que estão sem formato 
df_income_statement_10k = df_income_statement_10k.rename(index={
    'avgo_CostofProductsSold': 'Cost Of Products Sold',
    'avgo_CostofSubscriptionsandServices': 'Cost Of Subscriptions And Services',
    'avgo_Amortizationofacquisitionrelatedintangibleassetscostofproductssold': 'Amortization Of Acquisition Related Intangible Assets Cost Of Products Sold',
    'avgo_RestructuringChargesCostOfProductsSold': 'Restructuring Charges Cost Of Products Sold',
    'avgo_Amortizationofacquisitionrelatedintangibleassetsoperatingexpenses': 'Amortization Of Acquisition Related Intangible Assets Operating Expenses',
})

df_income_statement_10k

In [ ]:
# Fazendo o scraping do cash flow statement (10-K)
cash_flow_statement_lst = []

for acc_num in df_lst_acc_num_broadcom['acc_num']:

    # Cash flow statement
    cash_flow_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='cash_flow_statement')
    cash_flow_statement = f_sec.rename_statement(cash_flow_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    cash_flow_statement = cash_flow_statement.iloc[:, 0]
    cash_flow_statement_lst.append(cash_flow_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_cash_flow_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in cash_flow_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_cash_flow_10k = pd.concat([df_cash_flow_10k, df], axis=1)

# Substituindo os NaN por 0
df_cash_flow_10k = df_cash_flow_10k.fillna(0)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_cash_flow_10k = df_cash_flow_10k.drop('us-gaap_NetCashProvidedByUsedInOperatingActivitiesAbstract')


# Para pegar os dados de 2017, eu tive que usar o 'accession_number' de 2018
# Dados do 'cash_flow_statement' do ano de 2017
df_cash_flow_statement_2017 = f_sec.process_one_statement(ticker=ticker, accession_number='000173016818000084', statement_name='cash_flow_statement')
df_cash_flow_statement_2017 = f_sec.rename_statement(df_cash_flow_statement_2017, label_dictionary=labels_dict)
# Selecionando a coluna de 2017
df_cash_flow_statement_2017 = df_cash_flow_statement_2017.iloc[:, 1:2]
# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_cash_flow_statement_2017 = df_cash_flow_statement_2017.drop('us-gaap_NetCashProvidedByUsedInOperatingActivitiesAbstract')


# Lista com os dfs do 'cash_flow_statement'
cash_flow_statement_lst_2 = [df_cash_flow_10k, df_cash_flow_statement_2017]

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_cash_flow_10k_final = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in cash_flow_statement_lst_2:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_cash_flow_10k_final = pd.concat([df_cash_flow_10k_final, df], axis=1)

# Substituindo os NaN por 0
df_cash_flow_10k_final = df_cash_flow_10k_final.fillna(0)

# A linha 'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents' nas colunas '2021-10-31', '2020-11-01', '2019-11-03', '2018-11-04'	e '2017-10-29' estão zeradas
# As informações estão na linha 'Cash and Cash Equivalents, at Carrying Value'
df_cash_flow_10k_final.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2021-10-31'] = df_cash_flow_10k_final.loc['Cash and Cash Equivalents, at Carrying Value', '2021-10-31'] 
df_cash_flow_10k_final.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2020-11-01'] = df_cash_flow_10k_final.loc['Cash and Cash Equivalents, at Carrying Value', '2020-11-01'] 
df_cash_flow_10k_final.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2019-11-03'] = df_cash_flow_10k_final.loc['Cash and Cash Equivalents, at Carrying Value', '2019-11-03'] 
df_cash_flow_10k_final.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2018-11-04'] = df_cash_flow_10k_final.loc['Cash and Cash Equivalents, at Carrying Value', '2018-11-04'] 
df_cash_flow_10k_final.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2017-10-29'] = df_cash_flow_10k_final.loc['Cash and Cash Equivalents, at Carrying Value', '2017-10-29'] 

# A linha 'avgo_Amortizationofintangibleandrightofuseassets' nas colunas '2019-11-03', '2018-11-04'	e '2017-10-29' estão zeradas
# As informações estão na linha 'Amortization of Intangible Assets'
df_cash_flow_10k_final.loc['avgo_Amortizationofintangibleandrightofuseassets', '2019-11-03'] = df_cash_flow_10k_final.loc['Amortization of Intangible Assets', '2019-11-03'] 
df_cash_flow_10k_final.loc['avgo_Amortizationofintangibleandrightofuseassets', '2018-11-04'] = df_cash_flow_10k_final.loc['Amortization of Intangible Assets', '2018-11-04'] 
df_cash_flow_10k_final.loc['avgo_Amortizationofintangibleandrightofuseassets', '2017-10-29'] = df_cash_flow_10k_final.loc['Amortization of Intangible Assets', '2017-10-29'] 

# A linha 'Proceeds from (Repayments of) Other Debt' na coluna '2019-11-03' está zerada
# As informações estão na linha 'Proceeds from Other Debt'
df_cash_flow_10k_final.loc['Proceeds from (Repayments of) Other Debt', '2019-11-03'] = df_cash_flow_10k_final.loc['Proceeds from Other Debt', '2019-11-03'] 

# A linha 'Proceeds from Issuance of Common Stock' nas colunas '2018-11-04' e '2017-10-29' estão zeradas
# As informações estão na linha 'Repayments of Long-term Capital Lease Obligations'
df_cash_flow_10k_final.loc['Proceeds from Issuance of Common Stock', '2018-11-04'] = df_cash_flow_10k_final.loc['Repayments of Long-term Capital Lease Obligations', '2018-11-04'] 
df_cash_flow_10k_final.loc['Proceeds from Issuance of Common Stock', '2017-10-29'] = df_cash_flow_10k_final.loc['Repayments of Long-term Capital Lease Obligations', '2017-10-29'] 

# A linha 'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect' nas colunas '2018-11-04' e '2017-10-29' estão zeradas
# As informações estão na linha 'Cash and Cash Equivalents, Period Increase (Decrease)'
df_cash_flow_10k_final.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect', '2018-11-04'] = df_cash_flow_10k_final.loc['Cash and Cash Equivalents, Period Increase (Decrease)', '2018-11-04'] 
df_cash_flow_10k_final.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect', '2017-10-29'] = df_cash_flow_10k_final.loc['Cash and Cash Equivalents, Period Increase (Decrease)', '2017-10-29'] 

# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_cash_flow_10k_final = df_cash_flow_10k_final.drop([
    'Amortization of Intangible Assets',
    'Cash and Cash Equivalents, at Carrying Value',
    'Proceeds from Other Debt',
    'Repayments of Long-term Capital Lease Obligations',
    'Cash and Cash Equivalents, Period Increase (Decrease)',
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_cash_flow_10k_final = df_cash_flow_10k_final.drop([
    'Excess Tax Benefit from Share-based Compensation, Operating Activities',
    'Excess Tax Benefit from Share-based Compensation, Financing Activities',
    'Unrealized Gain (Loss) on Investments',
    'avgo_ProceedsforStockIssuedDuringPeriodValueSharebasedCompensationNetofSharesWithheldforTaxes',
    'avgo_Noncashportionofdebtextinguishmentgainloss',
    'Proceeds from Issuance of Convertible Preferred Stock'
])

# Renomeando as linhas do índice que estão sem formato 
df_cash_flow_10k_final = df_cash_flow_10k_final.rename(index={
    'avgo_Amortizationofintangibleandrightofuseassets': 'Amortization Of Intangible And Right Of Use Assets',
    'avgo_DeferredIncomeTaxesandOtherNoncashTaxExpense': 'Deferred Income Taxes And Other Non Cash Tax Expense',
})

df_cash_flow_10k_final

In [ ]:
# Arrumando alguns index dos dfs para ficar padronizado com os index utilizados nas funções p/ calcular os indicadores fundamentalistas
df_balance_sheet_10k_final = df_balance_sheet_10k_final.rename(index={
    'Cash and Cash Equivalents, at Carrying Value': 'Cash and Cash Equivalents',
    "Stockholders' Equity, Including Portion Attributable to Noncontrolling Interest": 'Total Shareholders Equity'
})

df_income_statement_10k = df_income_statement_10k.rename(index={
    'Revenue from Contract with Customer, Excluding Assessed Tax': 'Revenues',
    'Research and Development Expense': 'Research and Development',
    'Operating Income (Loss)': 'Operating Income',
    'Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest': 'Income Before Provision for Income Taxes',
    'Income Tax Expense (Benefit)': 'Provision for Income Taxes',
    'Net Income (Loss) Available to Common Stockholders, Basic': 'Net Income',
    'Weighted Average Number of Shares Outstanding, Basic': 'Number of Shares - Basic',
    'Weighted Average Number of Shares Outstanding, Diluted': 'Number of Shares - Diluted'
})

df_cash_flow_10k_final = df_cash_flow_10k_final.rename(index={
    'Depreciation': 'Depreciation, Depletion and Amortization',
    'Payments to Acquire Property, Plant, and Equipment': 'Purchases Of Property And Equipment And Intangible Assets',
})

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//semicondutores//broadcom//avgo_10k.xlsx') as writer:
    df_balance_sheet_10k_final.to_excel(writer, sheet_name='balance_sheet')
    df_income_statement_10k.to_excel(writer, sheet_name='income_statement')
    df_cash_flow_10k_final.to_excel(writer, sheet_name='cash_flow')

### Scraping Broadcom 10-Q

In [ ]:
# Número dos documentos que contém a DRE, BP e FC 
acc_documents_10q = f_sec.get_filtered_filings(ticker=ticker, ten_k=False, just_accession_numbers=True)

# Trocando os '-' para '' das strings ('0001045810-24-000264' -> '000104581024000264')
dict_acc_num_10q = {k: v.replace('-', '') for k, v in acc_documents_10q.items()}

# Transformando o dicionário em um df
df_lst_acc_num_10q_broadcom = pd.DataFrame.from_dict(dict_acc_num_10q, orient='index', columns=['acc_num'])

# Transformando em datetime o index
df_lst_acc_num_10q_broadcom.index = pd.to_datetime(df_lst_acc_num_10q_broadcom.index, format='%Y-%m-%d')
df_lst_acc_num_10q_broadcom

In [ ]:
# Selecionando os 'acc_num' dos 10-Q a partir de 2020 p/ iteração
df_lst_acc_num_10q_broadcom_sliced = df_lst_acc_num_10q_broadcom.loc['2020-02-02':]
df_lst_acc_num_10q_broadcom_sliced

In [ ]:
# Rótulos
labels_dict = f_sec.get_label_dictionary(ticker, headers=f_sec.header)

In [ ]:
# Fazendo o scraping do balance sheet (10-Q)
balance_sheet_lst = []

for acc_num in df_lst_acc_num_10q_broadcom_sliced['acc_num']:

    # Balance sheet
    balance_sheet = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='balance_sheet')
    balance_sheet = f_sec.rename_statement(balance_sheet, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    balance_sheet = balance_sheet.iloc[:, 0]
    balance_sheet_lst.append(balance_sheet)

# Juntando todos os dfs que estão em listas em apenas um df
df_balance_sheet_10q = pd.concat(balance_sheet_lst, axis=1)

# Substituindo os NaN por 0
df_balance_sheet_10q = df_balance_sheet_10q.fillna(0)

# A linha 'Long-term Debt and Lease Obligation, Current' nas colunas '2022-01-30', '2021-08-01', '2021-01-31', '2020-08-02', '2020-05-03' e '2020-02-02' estão zeradas
# A informação está na linha 'Long-term Debt, Current Maturities'
df_balance_sheet_10q.loc['Long-term Debt and Lease Obligation, Current', '2022-01-30'] = df_balance_sheet_10q.loc['Long-term Debt, Current Maturities', '2022-01-30']
df_balance_sheet_10q.loc['Long-term Debt and Lease Obligation, Current', '2021-08-01'] = df_balance_sheet_10q.loc['Long-term Debt, Current Maturities', '2021-08-01']
df_balance_sheet_10q.loc['Long-term Debt and Lease Obligation, Current', '2021-01-31'] = df_balance_sheet_10q.loc['Long-term Debt, Current Maturities', '2021-01-31']
df_balance_sheet_10q.loc['Long-term Debt and Lease Obligation, Current', '2020-08-02'] = df_balance_sheet_10q.loc['Long-term Debt, Current Maturities', '2020-08-02']
df_balance_sheet_10q.loc['Long-term Debt and Lease Obligation, Current', '2020-05-03'] = df_balance_sheet_10q.loc['Long-term Debt, Current Maturities', '2020-05-03']
df_balance_sheet_10q.loc['Long-term Debt and Lease Obligation, Current', '2020-02-02'] = df_balance_sheet_10q.loc['Long-term Debt, Current Maturities', '2020-02-02']

# A linha 'Long-term Debt and Lease Obligation' nas colunas '2022-01-30', '2021-08-01', '2021-01-31', '2020-08-02', '2020-05-03' e '2020-02-02' estão zeradas
# A informação está na linha 'Long-term Debt, Excluding Current Maturities'
df_balance_sheet_10q.loc['Long-term Debt and Lease Obligation', '2022-01-30'] = df_balance_sheet_10q.loc['Long-term Debt, Excluding Current Maturities', '2022-01-30']
df_balance_sheet_10q.loc['Long-term Debt and Lease Obligation', '2021-08-01'] = df_balance_sheet_10q.loc['Long-term Debt, Excluding Current Maturities', '2021-08-01']
df_balance_sheet_10q.loc['Long-term Debt and Lease Obligation', '2021-01-31'] = df_balance_sheet_10q.loc['Long-term Debt, Excluding Current Maturities', '2021-01-31']
df_balance_sheet_10q.loc['Long-term Debt and Lease Obligation', '2020-08-02'] = df_balance_sheet_10q.loc['Long-term Debt, Excluding Current Maturities', '2020-08-02']
df_balance_sheet_10q.loc['Long-term Debt and Lease Obligation', '2020-05-03'] = df_balance_sheet_10q.loc['Long-term Debt, Excluding Current Maturities', '2020-05-03']
df_balance_sheet_10q.loc['Long-term Debt and Lease Obligation', '2020-02-02'] = df_balance_sheet_10q.loc['Long-term Debt, Excluding Current Maturities', '2020-02-02']

# Removendo as linhas que movemos os seus itens para as linhas corretas
df_balance_sheet_10q = df_balance_sheet_10q.drop([
    'Long-term Debt, Current Maturities',
    'Long-term Debt, Excluding Current Maturities'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_balance_sheet_10q = df_balance_sheet_10q.drop([
    'us-gaap_AssetsCurrentAbstract',
    'Preferred Stock, Value, Issued',
    'Preferred Stock, Shares Authorized',
    'Preferred Stock, Par or Stated Value Per Share',
    'Preferred Stock, Dividend Rate, Percentage',
    'Preferred Stock, Shares Issued',
    'Preferred Stock, Liquidation Preference, Value',
    'Common Stock, Shares Authorized',
    'Common Stock, Shares, Issued',
    'Common Stock, Value, Issued',
    'avgo_TemporaryEquityPreferredStockDividendObligation'

])

# Arrumando a formatação da coluna '2020-08-02' (os nº estão com zeros a mais)
df_balance_sheet_10q['2020-08-02'] = df_balance_sheet_10q['2020-08-02'] / 1000000

df_balance_sheet_10q

In [ ]:
# Fazendo o scraping do income statement (10-Q)
income_statement_lst = []

for acc_num in df_lst_acc_num_10q_broadcom_sliced['acc_num']:

    # Income statement
    income_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='income_statement')
    income_statement = f_sec.rename_statement(income_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    income_statement = income_statement.iloc[:, 0]
    income_statement_lst.append(income_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_income_statement_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in income_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_income_statement_10q = pd.concat([df_income_statement_10q, df], axis=1)

# Substituindo os NaN por 0
df_income_statement_10q = df_income_statement_10q.fillna(0)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_income_statement_10q = df_income_statement_10q.drop([
    'us-gaap_RevenuesAbstract',
    'Income (Loss) from Discontinued Operations and Disposal of Discontinued Operations, Net of Tax, Per Diluted Share',
    'Income (Loss) from Continuing Operations, Per Basic Share',
    'Income (Loss) from Continuing Operations, Per Diluted Share',
    'Income (Loss) from Discontinued Operations and Disposal of Discontinued Operations, Net of Tax, Per Basic Share',
    'Income (Loss) from Continuing Operations, Net of Tax, Including Portion Attributable to Noncontrolling Interest',
    'Income (Loss) from Discontinued Operations, Net of Tax, Including Portion Attributable to Noncontrolling Interest',
    'Preferred Stock Dividends, Income Statement Impact'
])

# Arrumando e adicionando alguns números no df
df_income_statement_10q.loc['Net Income (Loss) Available to Common Stockholders, Basic', '2025-08-03'] = 4140
df_income_statement_10q.loc['Net Income (Loss) Available to Common Stockholders, Basic', '2025-05-04'] = 4965
df_income_statement_10q.loc['Net Income (Loss) Available to Common Stockholders, Basic', '2025-02-02'] = 5503
df_income_statement_10q.loc['Net Income (Loss) Available to Common Stockholders, Basic', '2024-08-04'] = -1875
df_income_statement_10q.loc['Net Income (Loss) Available to Common Stockholders, Basic', '2024-05-05'] = 2121
df_income_statement_10q.loc['Net Income (Loss) Available to Common Stockholders, Basic', '2024-02-04'] = 1325

# Renomeando as linhas do índice que estão sem formato 
df_income_statement_10q = df_income_statement_10q.rename(index={
    'avgo_CostofProductsSold': 'Cost Of Products Sold',
    'avgo_CostofSubscriptionsandServices': 'Cost Of Subscriptions And Services',
    'avgo_Amortizationofacquisitionrelatedintangibleassetscostofproductssold': 'Amortization Of Acquisition Related Intangible Assets Cost Of Products Sold',
    'avgo_RestructuringChargesCostOfProductsSold': 'Restructuring Charges Cost Of Products Sold',
    'avgo_Amortizationofacquisitionrelatedintangibleassetsoperatingexpenses': 'Amortization Of Acquisition Related Intangible Assets Operating Expenses'
})

df_income_statement_10q

In [ ]:
# Fazendo o scraping do cash flow statement (10-Q)
cash_flow_statement_lst = []

for acc_num in df_lst_acc_num_10q_broadcom_sliced['acc_num']:

    # Cash flow statement
    cash_flow_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='cash_flow_statement')
    cash_flow_statement = f_sec.rename_statement(cash_flow_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    cash_flow_statement = cash_flow_statement.iloc[:, 0]
    cash_flow_statement_lst.append(cash_flow_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_cash_flow_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in cash_flow_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_cash_flow_10q = pd.concat([df_cash_flow_10q, df], axis=1)

# Substituindo os NaN por 0
df_cash_flow_10q = df_cash_flow_10q.fillna(0)

# A linha 'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents' nas colunas '2022-07-31', '2022-05-01', '2022-01-30', '2021-08-01', '2021-05-02', / 
# '2021-01-31', '2020-08-02', '2020-05-03' e '2020-02-02' estão zeradas
# A informação está na linha 'Cash and Cash Equivalents, at Carrying Value'
df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2022-07-31'] = df_cash_flow_10q.loc['Cash and Cash Equivalents, at Carrying Value', '2022-07-31'] 
df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2022-05-01'] = df_cash_flow_10q.loc['Cash and Cash Equivalents, at Carrying Value', '2022-05-01'] 
df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2022-01-30'] = df_cash_flow_10q.loc['Cash and Cash Equivalents, at Carrying Value', '2022-01-30'] 
df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2021-08-01'] = df_cash_flow_10q.loc['Cash and Cash Equivalents, at Carrying Value', '2021-08-01'] 
df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2021-05-02'] = df_cash_flow_10q.loc['Cash and Cash Equivalents, at Carrying Value', '2021-05-02'] 
df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2021-01-31'] = df_cash_flow_10q.loc['Cash and Cash Equivalents, at Carrying Value', '2021-01-31'] 
df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2020-08-02'] = df_cash_flow_10q.loc['Cash and Cash Equivalents, at Carrying Value', '2020-08-02'] 
df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2020-05-03'] = df_cash_flow_10q.loc['Cash and Cash Equivalents, at Carrying Value', '2020-05-03'] 
df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2020-02-02'] = df_cash_flow_10q.loc['Cash and Cash Equivalents, at Carrying Value', '2020-02-02'] 

# A linha 'Gain (Loss) on Investments' nas colunas '2021-05-02' e '2021-01-31' estão zeradas
# A informação está na linha 'Unrealized Gain (Loss) on Investments'
df_cash_flow_10q.loc['Gain (Loss) on Investments', '2021-05-02'] = df_cash_flow_10q.loc['Unrealized Gain (Loss) on Investments', '2021-05-02'] 
df_cash_flow_10q.loc['Gain (Loss) on Investments', '2021-01-31'] = df_cash_flow_10q.loc['Unrealized Gain (Loss) on Investments', '2021-01-31'] 

# A linha 'Proceeds from (Repayments of) Other Debt	' nas colunas '2020-05-03', '2020-02-02' estão zeradas
# A informação está na linha 'Proceeds from Other Debt'
df_cash_flow_10q.loc['Proceeds from (Repayments of) Other Debt', '2020-05-03'] = df_cash_flow_10q.loc['Proceeds from Other Debt', '2020-05-03'] 
df_cash_flow_10q.loc['Proceeds from (Repayments of) Other Debt', '2020-02-02'] = df_cash_flow_10q.loc['Proceeds from Other Debt', '2020-02-02'] 

# Removendo as linhas que movemos os seus itens para as linhas corretas
df_cash_flow_10q = df_cash_flow_10q.drop([
    'Cash and Cash Equivalents, at Carrying Value',
    'Unrealized Gain (Loss) on Investments',
    'Proceeds from Other Debt',
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_cash_flow_10q = df_cash_flow_10q.drop([
    'us-gaap_NetCashProvidedByUsedInOperatingActivitiesAbstract',
    'Proceeds from Divestiture of Businesses',
    'Gain (Loss) on Investments',
    'Restructuring Costs and Asset Impairment Charges',
    'Proceeds from Sale of Property, Plant, and Equipment',
    'Proceeds from (Repayments of) Other Debt'

])

# Renomeando as linhas do índice que estão sem formato 
df_cash_flow_10q = df_cash_flow_10q.rename(index={
    'avgo_Amortizationofintangibleandrightofuseassets': 'Amortization Of Intangible And Right Of Use Assets',
    'avgo_DeferredIncomeTaxesandOtherNoncashTaxExpense': 'Deferred Income Taxes And Other Non Cash Tax Expense'
})

df_cash_flow_10q 

In [ ]:
# Arrumando alguns index dos dfs para ficar padronizado com os index utilizados nas funções p/ calcular os indicadores fundamentalistas
df_balance_sheet_10q = df_balance_sheet_10q.rename(index={
    'Cash and Cash Equivalents, at Carrying Value': 'Cash and Cash Equivalents',
    "Stockholders' Equity, Including Portion Attributable to Noncontrolling Interest": 'Total Shareholders Equity'
})

df_income_statement_10q = df_income_statement_10q.rename(index={
    'Revenue from Contract with Customer, Excluding Assessed Tax': 'Revenues',
    'Research and Development Expense': 'Research and Development',
    'Operating Income (Loss)': 'Operating Income',
    'Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest': 'Income Before Provision for Income Taxes',
    'Income Tax Expense (Benefit)': 'Provision for Income Taxes',
    'Net Income (Loss) Available to Common Stockholders, Basic': 'Net Income',
    'Weighted Average Number of Shares Outstanding, Basic': 'Number of Shares - Basic',
    'Weighted Average Number of Shares Outstanding, Diluted': 'Number of Shares - Diluted'
})

df_cash_flow_10q = df_cash_flow_10q.rename(index={
    'Depreciation': 'Depreciation, Depletion and Amortization',
    'Payments to Acquire Businesses, Net of Cash Acquired': 'Purchases Of Property And Equipment And Intangible Assets',
})

# Confirmando que as colunas estão em datetime - esse passo é necessário p/ não dar problema na hora da aplicação da função 'ajustar_mes_02_05_08'
df_balance_sheet_10q.columns = pd.to_datetime(df_balance_sheet_10q.columns, format='%Y-%m-%d')
df_income_statement_10q.columns = pd.to_datetime(df_income_statement_10q.columns, format='%Y-%m-%d')
df_cash_flow_10q.columns = pd.to_datetime(df_cash_flow_10q.columns, format='%Y-%m-%d')

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//semicondutores//broadcom//avgo_10q.xlsx') as writer:
    df_balance_sheet_10q.to_excel(writer, sheet_name='balance_sheet')
    df_income_statement_10q.to_excel(writer, sheet_name='income_statement')
    df_cash_flow_10q.to_excel(writer, sheet_name='cash_flow')

### Indicadores Fundamentalistas

#### Indicadores 10-K

In [ ]:
# Lendo os arquivos 10k
broadcom_bs = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//broadcom//avgo_10k.xlsx', 
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

broadcom_is = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//broadcom//avgo_10k.xlsx', 
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

broadcom_cf = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//broadcom//avgo_10k.xlsx', 
    sheet_name='cash_flow', 
    index_col='Unnamed: 0'
)

# Transformando as colunas em datetime
broadcom_bs.columns = pd.to_datetime(broadcom_bs.columns)
broadcom_is.columns = pd.to_datetime(broadcom_is.columns)
broadcom_cf.columns = pd.to_datetime(broadcom_cf.columns)

# Transformando as colunas em apenas o ano
broadcom_bs.columns = broadcom_bs.columns.year
broadcom_is.columns = broadcom_is.columns.year
broadcom_cf.columns = broadcom_cf.columns.year

In [ ]:
# Preços da AVGO
broadcom_close = yf.download('AVGO', auto_adjust=True, multi_level_index=False, start='2017-01-01')['Close']

# Selecionando o último preço do mês de Novembro de cada ano 
lst_years = [
    '2017', '2018', '2019', '2020', 
    '2021', '2022', '2023', '2024',
    '2025'
]
last_nov_close = []

for year in lst_years:
    last_close = broadcom_close[f'{year}-11'].iloc[-1]
    last_nov_close.append(last_close)

# Transformando em uma série pandas
broadcom_close_nov = pd.Series(last_nov_close, index=lst_years)

# Revertendo o index da série p/ ficar igual com os dos indicadores
broadcom_close_nov = broadcom_close_nov.reindex(index=broadcom_close_nov.index[::-1])

# Arredondamento de duas casas decimais
broadcom_close_nov = round(broadcom_close_nov, 2)

# Teve um split de 10:1 em 15/07/2024 (https://br.investing.com/equities/avago-technologies-historical-data-splits)
broadcom_close_nov.loc['2023'] = broadcom_close_nov.loc['2023'] * 10
broadcom_close_nov.loc['2022'] = broadcom_close_nov.loc['2022'] * 10
broadcom_close_nov.loc['2021'] = broadcom_close_nov.loc['2021'] * 10
broadcom_close_nov.loc['2020'] = broadcom_close_nov.loc['2020'] * 10
broadcom_close_nov.loc['2019'] = broadcom_close_nov.loc['2019'] * 10
broadcom_close_nov.loc['2018'] = broadcom_close_nov.loc['2018'] * 10
broadcom_close_nov.loc['2017'] = broadcom_close_nov.loc['2017'] * 10

# Transformando o index em int
broadcom_close_nov.index = broadcom_close_nov.index.astype(int)

broadcom_close_nov

In [ ]:
# Criando uma lista só para fazer a indexação em que o ano mais recente está no começo
idx = broadcom_bs.columns

# Valor de Mercado
vm = f_eua.indicador_vm(ten_k=True, price=broadcom_close_nov, df_is=broadcom_is) 

# LPA
lpa = f_eua.indicador_lpa(ten_k=True, df_is=broadcom_is, idx=None)

# P/L Damodaran
p_lpa_damodaran = f_eua.indicador_p_lpa_2(ten_k=True, vm=vm, df_is=broadcom_is)

# P/L
p_lpa = f_eua.indicador_p_lpa(ten_k=True, price=broadcom_close_nov, lpa=lpa, idx=idx)

# L/P
lpa_p = f_eua.indicador_lpa_p(ten_k=True, price=broadcom_close_nov, lpa=lpa, idx=idx) 

# Depreciação
depreciacao = f_eua.indicador_depreciacao(ten_k=True, df_cf=broadcom_cf, first_quarter=None, idx=idx)

# EBITDA
ebitda = f_eua.indicador_ebitda(ten_k=True, df_is=broadcom_is, idx=None, depreciacao=depreciacao)

# Margem Líquida
m_liq = f_eua.indicador_ml(ten_k=True, df_is=broadcom_is)

# VPA
vpa = f_eua.indicador_vpa(ten_k=True, df_bs=broadcom_bs, df_is=broadcom_is)

# P/VPA
p_vpa = f_eua.indicador_p_vpa(ten_k=True, price=broadcom_close_nov, vpa=vpa, idx=idx)

# Caixa e Equivalentes
lst_caixa = ['Cash and Cash Equivalents']
caixa = f_eua.indicador_caixa(ten_k=True, df_bs=broadcom_bs, lst_itens_caixa=lst_caixa)

# Dívida Bruta
lst_divida = ['Long-term Debt and Lease Obligation, Current', 'Long-term Debt and Lease Obligation']
div_bruta = f_eua.indicador_divida_bruta(ten_k=True, df_bs=broadcom_bs, lst_itens_divida=lst_divida)

# Dívida Líquida
div_liq = f_eua.indicador_divida_liquida(ten_k=True, caixa=caixa, div_bruta=div_bruta, idx=idx)

# Dívida Líquida / EBITDA
dl_ebitda = f_eua.indicador_dl_ebitda(ten_k=True, ebitda=ebitda, div_liq=div_liq, idx=None)

# Dívida Líquida / PL
dl_pl = f_eua.indicador_dl_pl(ten_k=True, div_liq=div_liq, df_bs=broadcom_bs) 

# EV/EBITDA
ev_ebitda = f_eua.indicador_ev_ebitda(vm=vm, div_liq=div_liq, ebitda=ebitda)

# ROE
roe = f_eua.indicador_roe(ten_k=True, df_is=broadcom_is, df_bs=broadcom_bs, idx=None)

# ROIC
roic = f_eua.indicador_roic(ten_k=True, df_is=broadcom_is, df_bs=broadcom_bs, div_bruta=div_bruta, idx=idx)

# FCO
fco = f_eua.indicador_fco(ten_k=True, df_cf=broadcom_cf, first_quarter=None) 

# FCI
fci = f_eua.indicador_fci(ten_k=True, df_cf=broadcom_cf, first_quarter=None) 

# FCF
fcf = f_eua.indicador_fcf(ten_k=True, df_cf=broadcom_cf, first_quarter=None) 

# CAPEX
lst_capex = ['Purchases Of Property And Equipment And Intangible Assets']

capex= f_eua.indicador_capex(ten_k=True, df_cf=broadcom_cf, first_quarter=None, lst_itens=lst_capex, idx=idx)

# Free Cash Flow
free_cash_flow = f_eua.indicador_free_cash_flow(fco=fco, capex=capex)

# Net CAPEX
net_capex = f_eua.indicador_net_capex(capex=capex, depreciation=depreciacao)

# R&D
rd_expense = f_eua.indicador_rd(ten_k=True, df_is=broadcom_is, idx=idx)

# Net CAPEX ajustado
adj_net_capex = f_eua.indicador_adj_net_capex(net_capex=net_capex, rd_expense=rd_expense)

# Working capital
working_capital = f_eua.indicador_working_capital(ten_k=True, df_bs=broadcom_bs)

# Change in non cash working capital
lst_itens_change_non_cash_wc = [
    'Deferred Income Taxes And Other Non Cash Tax Expense',
    'Increase (Decrease) in Accounts Receivable',
    'Increase (Decrease) in Inventories',
    'Increase (Decrease) in Accounts Payable',
    'Increase (Decrease) in Employee Related Liabilities',
    'Increase (Decrease) in Other Current Assets and Liabilities, Net',
    'Increase (Decrease) in Other Noncurrent Assets and Liabilities, Net'
]

change_non_cash_wc = f_eua.indicador_change_non_cash_wc(ten_k=True, df_cf=broadcom_cf, lst_itens=lst_itens_change_non_cash_wc, first_quarter=None)

# Reinvestment rate
reinvestment_rate = f_eua.indicador_reinvestment_rate(ten_k=True, net_capex=net_capex, change_non_cash_wc=change_non_cash_wc, df_is=broadcom_is, idx=idx)

# New borrowing             
lst_itens_new_borrowing = ['Proceeds from Issuance of Long-term Debt']

new_borrowing = f_eua.indicador_new_borrowing(ten_k=True, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=broadcom_cf, first_quarter=None, idx=idx)

# Debt paid
lst_itens_debt_paid = ['Repayments of Debt']

debt_paid = f_eua.indicador_debt_paid(ten_k=True, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=broadcom_cf, first_quarter=None, idx=idx)

# Taxes on operating income
taxes_operating_income = f_eua.indicador_taxes_operating_income(ten_k=True, df_is=broadcom_is, idx=idx)

# FCFE
fcfe_10q = f_eua.indicador_fcfe(
    ten_k=True, 
    depreciacao=depreciacao, 
    capex=capex, 
    change_non_cash_wc=change_non_cash_wc, 
    new_borrowing=new_borrowing,
    debt_paid=debt_paid,
    df_is=broadcom_is,
    idx=idx
)

# FCFF
fcff_10q = f_eua.indicador_fcff(
        ten_k=True, 
        taxes_operating_income=taxes_operating_income,
        depreciacao=depreciacao, 
        capex=capex, 
        change_non_cash_wc=change_non_cash_wc, 
        df_is=broadcom_is,
        idx=idx
)

# Dividendo por Ação
dpa = f_eua.indicador_dpa(ten_k=True, df_cf=broadcom_cf, df_is=broadcom_is, div_nao_acum=None)

# Payout
payout = f_eua.indicador_payout(ten_k=True, df_cf=broadcom_cf, df_is=broadcom_is, div_nao_acum=None)

# Buyback
buyback = f_eua.indicador_buyback(ten_k=True, df_cf=broadcom_cf, first_quarter=None)

# Colocando todos os indicadores em uma lista
lst_indicadores = [
    p_lpa_damodaran,
    p_lpa,
    lpa_p,  
    ebitda, 
    m_liq,
    p_vpa,
    vm,
    div_bruta,
    caixa,
    div_liq,
    dl_ebitda,
    dl_pl,
    ev_ebitda,
    roe,
    roic,
    fco,
    fci,
    fcf,
    free_cash_flow,
    capex,
    net_capex,
    rd_expense,
    adj_net_capex,
    working_capital,
    reinvestment_rate,
    fcfe_10q,
    fcff_10q,
    dpa,
    payout,
    buyback
]

# Juntando todos os indicadores em um df
broadcom_indicadores_10k = pd.concat(lst_indicadores, axis=1)

# Nomes das colunas
nomes_col = [
    'P/L Damodaran', 
    'P/L', 
    'L/P', 
    'EBITDA', 
    'Margem Líquida',
    'P/VPA',
    'Valor de Mercado',
    'Dívida Bruta',
    'Caixa e Equivalentes',
    'Dívida Líquida',
    'Dívida Líquida/EBITDA',
    'Dívida Líquida/PL',
    'EV/EBITDA',
    'ROE',
    'ROIC',
    'FCO',
    'FCI',
    'FCF',
    'Free Cash Flow',
    'Capex',
    'Net Capex',
    'R&D',
    'Adj Net Capex',
    'Working Capital',
    'Reinvestment Rate',
    'FCFE',
    'FCFF',
    'DPA',
    'Payout',
    'Buyback'
]

# Atribuindo os novos nomes das colunas do df
broadcom_indicadores_10k.columns = nomes_col
broadcom_indicadores_10k

#### Indicadores 10-Q

In [ ]:
# Lendo os arquivos 10q
broadcom_bs_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//broadcom//avgo_10q.xlsx', 
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

broadcom_is_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//broadcom//avgo_10q.xlsx', 
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

broadcom_cf_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//semicondutores//broadcom//avgo_10q.xlsx', 
    sheet_name='cash_flow', 
    index_col='Unnamed: 0'
)

# Transformando o datetime em uma string para retirar os dias - 
# Faço isso para retirar os dias '20XX-01-31' p/ conseguir aplicar a função 'ajustar_mes_02_05_08'
# Se eu não faço isso, a função não funciona, porque não existe '20XX-02-31' (Fevereiro vai até 28 ou 29)
broadcom_bs_10q.columns = broadcom_bs_10q.columns.strftime('%Y-%m')
broadcom_is_10q.columns = broadcom_is_10q.columns.strftime('%Y-%m')
broadcom_cf_10q.columns = broadcom_cf_10q.columns.strftime('%Y-%m')

# Retransformando em datetime - agora todos os dias são '01'
broadcom_bs_10q.columns = pd.to_datetime(broadcom_bs_10q.columns, format='%Y-%m')
broadcom_is_10q.columns = pd.to_datetime(broadcom_is_10q.columns, format='%Y-%m')
broadcom_cf_10q.columns = pd.to_datetime(broadcom_cf_10q.columns, format='%Y-%m')

# Aplicando o ajuste nas colunas
broadcom_bs_10q.columns = [f_sec.ajustar_mes_02_05_08(col) for col in broadcom_bs_10q.columns]
broadcom_is_10q.columns = [f_sec.ajustar_mes_02_05_08(col) for col in broadcom_is_10q.columns]
broadcom_cf_10q.columns = [f_sec.ajustar_mes_02_05_08(col) for col in broadcom_cf_10q.columns]

In [ ]:
# Preços da AVGO
broadcom_close = yf.download('AVGO', auto_adjust=True, multi_level_index=False, start='2021-01-01')['Close']

# Selecionando o último preço dos meses de Fevereiro, Maio e Agosto
lst_years = [
    '2021', '2022', '2023', '2024', 
    '2025',
]
lst_months = ['02', '05', '08']
last_close_months = []

for year in lst_years:
    for month in lst_months:
        last_close = broadcom_close[f'{year}-{month}'].dropna().iloc[-1]  # Dropna para lidar com meses sem dados
        last_close_months.append(last_close)

# Criando uma lista em que junte os meses e os anos para que seja o index da serie pandas
lst_idx = []

for year in lst_years:
    for month in lst_months:
        lst_idx.append(f"{month}-{year}")

# Transformando em uma série pandas
broadcom_close_months = pd.Series(last_close_months, index=lst_idx)

# Revertendo o index da série p/ ficar igual com os dos indicadores
broadcom_close_months = broadcom_close_months.reindex(index=broadcom_close_months.index[::-1])

# No ano mais atual, como ainda ele não acabou, não consigo fazer o loop com todos meses (Fevereiro, Maio e Agosto)
actual_year_int = datetime.now().year  
actual_year_str = str(actual_year_int) 

lst_actual_months = [
    # '02',
    # '05',
    # '08'
] 

last_close_actual_months = []

for month in lst_actual_months:
    last_close = broadcom_close[f'{actual_year_str}-{month}'].dropna().iloc[-1]  # Dropna para lidar com meses sem dados
    last_close_actual_months.append(last_close)

# Criando uma lista em que junte os meses e os anos para que seja o index da serie pandas
lst_idx_actual = []

for month in lst_actual_months:
    lst_idx_actual.append(f"{month}-{actual_year_str}")

# Transformando em uma série pandas
broadcom_close_months_atual = pd.Series(last_close_actual_months, index=lst_idx_actual)

# Revertendo o index da série p/ ficar igual com os dos indicadores
broadcom_close_months_atual = broadcom_close_months_atual.reindex(index=broadcom_close_months_atual.index[::-1])

# Juntando as duas series pandas
broadcom_close_months = pd.concat([broadcom_close_months_atual, broadcom_close_months])

# Arredondamento de duas casas decimais
broadcom_close_months = round(broadcom_close_months, 2)

# Teve um split de 10:1 em 15/07/2024 (https://br.investing.com/equities/avago-technologies-historical-data-splits)
broadcom_close_months.loc['05-2024'] = broadcom_close_months.loc['05-2024'] * 10
broadcom_close_months.loc['02-2024'] = broadcom_close_months.loc['02-2024'] * 10
broadcom_close_months.loc['08-2023'] = broadcom_close_months.loc['08-2023'] * 10
broadcom_close_months.loc['05-2023'] = broadcom_close_months.loc['05-2023'] * 10
broadcom_close_months.loc['02-2023'] = broadcom_close_months.loc['02-2023'] * 10
broadcom_close_months.loc['08-2022'] = broadcom_close_months.loc['08-2022'] * 10
broadcom_close_months.loc['05-2022'] = broadcom_close_months.loc['05-2022'] * 10
broadcom_close_months.loc['02-2022'] = broadcom_close_months.loc['02-2022'] * 10
broadcom_close_months.loc['08-2021'] = broadcom_close_months.loc['08-2021'] * 10
broadcom_close_months.loc['05-2021'] = broadcom_close_months.loc['05-2021'] * 10
broadcom_close_months.loc['02-2021'] = broadcom_close_months.loc['02-2021'] * 10

# Substituindo o index para ficar igual ao dos dados fundamentalistas
broadcom_close_months.index = broadcom_bs_10q.columns[:-3]

broadcom_close_months

In [ ]:
# Criando uma lista só para fazer a indexação em que o ano mais recente está no começo
idx_10q = broadcom_bs_10q.columns

# Valor de Mercado
vm_10q = f_eua.indicador_vm(ten_k=False, price=broadcom_close_months, df_is=broadcom_is_10q)

# LPA
lpa_acum = f_eua.indicador_lpa(ten_k=False, df_is=broadcom_is_10q, idx=idx_10q)[1]
lpa_10q = f_eua.indicador_lpa(ten_k=False, df_is=broadcom_is_10q, idx=idx_10q)[0]

# P/L Damodaran
p_lpa_10q_damodaran = f_eua.indicador_p_lpa_2(ten_k=False, vm=vm_10q, df_is=broadcom_is_10q)

# P/L
p_lpa_10q = f_eua.indicador_p_lpa(ten_k=False, price=broadcom_close_months, lpa=lpa_acum, idx=idx_10q)

# L/P
lpa_p_10q = f_eua.indicador_lpa_p(ten_k=False, price=broadcom_close_months, lpa=lpa_acum, idx=idx_10q) 

# Depreciação
depreciacao_10q = f_eua.indicador_depreciacao(ten_k=False, df_cf=broadcom_cf_10q, first_quarter='02', idx=idx_10q)[0]
depreciacao_10q_sliced = f_eua.indicador_depreciacao(ten_k=False, df_cf=broadcom_cf_10q, first_quarter='02', idx=idx_10q)[1]
depreciacao_10q_acum = f_eua.indicador_depreciacao(ten_k=False, df_cf=broadcom_cf_10q, first_quarter='02', idx=idx_10q)[2]

# EBITDA
ebitda_10q = f_eua.indicador_ebitda(ten_k=False, df_is=broadcom_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[0]
ebitda_10q_sliced = f_eua.indicador_ebitda(ten_k=False, df_is=broadcom_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[1]
ebitda_10q_acum = f_eua.indicador_ebitda(ten_k=False, df_is=broadcom_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[2]

# Margem Líquida
m_liq_10q = f_eua.indicador_ml(ten_k=False, df_is=broadcom_is_10q)

# VPA
vpa_10q = f_eua.indicador_vpa(ten_k=False, df_bs=broadcom_bs_10q, df_is=broadcom_is_10q)

# P/VPA
p_vpa_10q = f_eua.indicador_p_vpa(ten_k=False, price=broadcom_close_months, vpa=vpa_10q, idx=idx_10q)

# Caixa e Equivalentes
lst_caixa = ['Cash and Cash Equivalents']
caixa_10q = f_eua.indicador_caixa(ten_k=False, df_bs=broadcom_bs_10q, lst_itens_caixa=lst_caixa)

# Dívida Bruta
lst_divida = ['Debt, Current', 'Long-term Debt and Lease Obligation']
div_bruta_10q = f_eua.indicador_divida_bruta(ten_k=False, df_bs=broadcom_bs_10q, lst_itens_divida=lst_divida)

# Dívida Líquida
div_liq_10q = f_eua.indicador_divida_liquida(ten_k=False, caixa=caixa_10q, div_bruta=div_bruta_10q, idx=idx_10q)

# Dívida Líquida / EBITDA
dl_ebitda_10q = f_eua.indicador_dl_ebitda(ten_k=False, ebitda=ebitda_10q_acum, div_liq=div_liq_10q, idx=idx_10q)

# Dívida Líquida / PL
dl_pl_10q = f_eua.indicador_dl_pl(ten_k=False, div_liq=div_liq_10q, df_bs=broadcom_bs_10q) 

# EV/EBITDA
ev_ebitda_10q = f_eua.indicador_ev_ebitda(vm=vm_10q, div_liq=div_liq_10q, ebitda=ebitda_10q_acum)

# ROE
roe_10q = f_eua.indicador_roe(ten_k=False, df_is=broadcom_is_10q, df_bs=broadcom_bs_10q, idx=idx_10q)

# ROIC 
roic_10q = f_eua.indicador_roic(ten_k=False, df_is=broadcom_is_10q, df_bs=broadcom_bs_10q, div_bruta=div_bruta_10q, idx=idx_10q)

# FCO
fco_10q = f_eua.indicador_fco(ten_k=False, df_cf=broadcom_cf_10q, first_quarter='02') 

# FCI
fci_10q = f_eua.indicador_fci(ten_k=False, df_cf=broadcom_cf_10q, first_quarter='02') 

# FCF
fcf_10q = f_eua.indicador_fcf(ten_k=False, df_cf=broadcom_cf_10q, first_quarter='02') 

# CAPEX
lst_capex_10q = ['Payments to Acquire Property, Plant, and Equipment']

capex_10q = f_eua.indicador_capex(ten_k=False, df_cf=broadcom_cf_10q, first_quarter='02', lst_itens=lst_capex_10q, idx=idx_10q)[0]
capex_10q_sliced = f_eua.indicador_capex(ten_k=False, df_cf=broadcom_cf_10q, first_quarter='02', lst_itens=lst_capex_10q, idx=idx_10q)[1]
capex_10q_acum = f_eua.indicador_capex(ten_k=False, df_cf=broadcom_cf_10q, first_quarter='02', lst_itens=lst_capex_10q, idx=idx_10q)[2]

# Free Cash Flow
free_cash_flow_10q = f_eua.indicador_free_cash_flow(fco=fco_10q, capex=capex_10q_sliced)

# Net CAPEX
net_capex_10q = f_eua.indicador_net_capex(capex=capex_10q_acum, depreciation=depreciacao_10q_acum)

# R&D
rd_expense_10q = f_eua.indicador_rd(ten_k=False, df_is=broadcom_is_10q, idx=idx_10q)[0]
rd_expense_10q_sliced = f_eua.indicador_rd(ten_k=False, df_is=broadcom_is_10q, idx=idx_10q)[1]
rd_expense_10q_acum = f_eua.indicador_rd(ten_k=False, df_is=broadcom_is_10q, idx=idx_10q)[2]

# Net CAPEX ajustado
adj_net_capex_10q = f_eua.indicador_adj_net_capex(net_capex=net_capex_10q, rd_expense=rd_expense_10q_acum)

# Working capital
working_capital_10q = f_eua.indicador_working_capital(ten_k=False, df_bs=broadcom_bs_10q)

# Change in non cash working capital
lst_itens_change_non_cash_wc = [
    'Deferred Income Taxes And Other Non Cash Tax Expense',
    'Increase (Decrease) in Accounts Receivable',
    'Increase (Decrease) in Inventories',
    'Increase (Decrease) in Accounts Payable',
    'Increase (Decrease) in Employee Related Liabilities',
    'Increase (Decrease) in Other Current Assets and Liabilities, Net',
    'Increase (Decrease) in Other Noncurrent Assets and Liabilities, Net'
]

change_non_cash_wc_10q = f_eua.indicador_change_non_cash_wc(ten_k=False, df_cf=broadcom_cf_10q, lst_itens=lst_itens_change_non_cash_wc, first_quarter='02')[0]
change_non_cash_wc_10q_sliced = f_eua.indicador_change_non_cash_wc(ten_k=False, df_cf=broadcom_cf_10q, lst_itens=lst_itens_change_non_cash_wc, first_quarter='02')[1]

# Reinvestment rate
reinvestment_rate_10q = f_eua.indicador_reinvestment_rate(ten_k=False, net_capex=adj_net_capex_10q, change_non_cash_wc=change_non_cash_wc_10q, df_is=broadcom_is_10q, idx=idx_10q)

# New borrowing
lst_itens_new_borrowing = ['Proceeds from Issuance of Long-term Debt']

new_borrowing_10q = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=broadcom_cf_10q, first_quarter='02', idx=idx_10q)[0]
new_borrowing_10q_sliced = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=broadcom_cf_10q, first_quarter='02', idx=idx_10q)[1]
new_borrowing_10q_acum = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=broadcom_cf_10q, first_quarter='02', idx=idx_10q)[2]

# Debt paid
lst_itens_debt_paid = ['Repayments of Debt']

debt_paid_10q = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=broadcom_cf_10q, first_quarter='02', idx=idx_10q)[0]
debt_paid_10q_sliced = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=broadcom_cf_10q, first_quarter='02', idx=idx_10q)[1]
debt_paid_10q_acum = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=broadcom_cf_10q, first_quarter='02', idx=idx_10q)[2]

# Taxes on operating income
taxes_operating_income_10q = f_eua.indicador_taxes_operating_income(ten_k=False, df_is=broadcom_is_10q, idx=idx_10q)

# FCFE
fcfe_10q = f_eua.indicador_fcfe(
    ten_k=False, 
    depreciacao=depreciacao_10q_acum, 
    capex=capex_10q_acum, 
    change_non_cash_wc=change_non_cash_wc_10q_sliced, 
    new_borrowing=new_borrowing_10q_acum, 
    debt_paid=debt_paid_10q_acum, 
    df_is=broadcom_is_10q,
    idx=idx_10q
)

# FCFF
fcff_10q = f_eua.indicador_fcff(
        ten_k=False, 
        taxes_operating_income=taxes_operating_income_10q,
        depreciacao=depreciacao_10q_acum, 
        capex=capex_10q_acum, 
        change_non_cash_wc=change_non_cash_wc_10q_sliced, 
        df_is=broadcom_is_10q,
        idx=idx_10q
)

# Dividendo não acumulado
div_nao_acum = f_eua.indicador_div_nao_acum(df_cf=broadcom_cf_10q, first_quarter='02')

# Dividendo por Ação
dpa_10q = f_eua.indicador_dpa(ten_k=False, df_cf=broadcom_cf_10q, df_is=broadcom_is_10q, div_nao_acum=div_nao_acum)

# Payout
payout_10q = f_eua.indicador_payout(ten_k=False, df_cf=broadcom_cf_10q, df_is=broadcom_is_10q, div_nao_acum=div_nao_acum)

# Buyback
buyback_10q = f_eua.indicador_buyback(ten_k=False, df_cf=broadcom_cf_10q,  first_quarter='02')

# Lista com todos os indicadores
lst_indicadores_10q = [
    p_lpa_10q_damodaran,
    p_lpa_10q,
    lpa_p_10q,  
    ebitda_10q_sliced, 
    m_liq_10q,
    p_vpa_10q,
    vm_10q,
    div_bruta_10q,
    caixa_10q,
    div_liq_10q,
    dl_ebitda_10q,
    dl_pl_10q,
    ev_ebitda_10q,
    roe_10q,
    roic_10q,
    fco_10q,
    fci_10q,
    fcf_10q,
    free_cash_flow_10q,
    capex_10q_sliced,
    net_capex_10q,
    rd_expense_10q_sliced,
    adj_net_capex_10q,
    working_capital_10q,
    reinvestment_rate_10q,
    fcfe_10q,
    fcff_10q,
    dpa_10q,
    payout_10q,
    buyback_10q
]

# Juntando todos os indicadores em um df
broadcom_indicadores_10q = pd.concat(lst_indicadores_10q, axis=1)

# Nomes das colunas
nomes_col = [
    'P/L Damodaran', 
    'P/L', 
    'L/P', 
    'EBITDA', 
    'Margem Líquida',
    'P/VPA',
    'Valor de Mercado',
    'Dívida Bruta',
    'Caixa e Equivalentes',
    'Dívida Líquida',
    'Dívida Líquida/EBITDA',
    'Dívida Líquida/PL',
    'EV/EBITDA',
    'ROE',
    'ROIC',
    'FCO',
    'FCI',
    'FCF',
    'Free Cash Flow',
    'Capex',
    'Net Capex',
    'R&D',
    'Adj Net Capex',
    'Working Capital',
    'Reinvestment Rate',
    'FCFE',
    'FCFF',
    'DPA',
    'Payout',
    'Buyback'
]

# Atribuindo os novos nomes das colunas do df
broadcom_indicadores_10q.columns = nomes_col
broadcom_indicadores_10q

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//semicondutores//broadcom//avgo_indicators.xlsx') as writer:
    broadcom_indicadores_10k.to_excel(writer, sheet_name='indicators_10k')
    broadcom_indicadores_10q.to_excel(writer, sheet_name='indicators_10q')

# **Setor de Energia Elétrica**

## Constellation Energy

* 1Q -> Lançamento em Março (03);
* 2Q -> Lançamento em Junho (06);
* 3Q -> Lançamento em Setembro (09);
* 10K -> Lançamento em Dezembro (12).

In [ ]:
ticker = 'ceg'
f_sec.cik_matching_ticker(ticker)

### Scraping Constellation 10-K

In [ ]:
# Número dos documentos que contém a DRE, BP e FC 
acc_documents = f_sec.get_filtered_filings(ticker=ticker, ten_k=True, just_accession_numbers=True)

# Trocando os '-' para '' das strings ('0001045810-24-000264' -> '000104581024000264')
dict_acc_num = {k: v.replace('-', '') for k, v in acc_documents.items()}

# Transformando o dicionário em um df
df_lst_acc_num_constellation = pd.DataFrame.from_dict(dict_acc_num, orient='index', columns=['acc_num'])

# Transformando em datetime o index
df_lst_acc_num_constellation.index = pd.to_datetime(df_lst_acc_num_constellation.index, format='%Y-%m-%d')

# Selecionando os 'acc_num' dos 10-k a partir de 2022 p/ iteração
df_lst_acc_num_constellation = df_lst_acc_num_constellation.loc['2022-12-31':]

df_lst_acc_num_constellation

In [ ]:
# Rótulos
labels_dict = f_sec.get_label_dictionary(ticker, headers=f_sec.header)

In [ ]:
# Fazendo o scraping do balance sheet (10-K)
balance_sheet_lst = []

for acc_num in df_lst_acc_num_constellation['acc_num']:

    # Balance sheet
    balance_sheet = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='balance_sheet')
    balance_sheet = f_sec.rename_statement(balance_sheet, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    balance_sheet = balance_sheet.iloc[:, 0]
    balance_sheet_lst.append(balance_sheet)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_balance_sheet_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in balance_sheet_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_balance_sheet_10k = pd.concat([df_balance_sheet_10k, df], axis=1)

# Substituindo os NaN por 0
df_balance_sheet_10k = df_balance_sheet_10k.fillna(0)

# A linha 'Liability, Defined Benefit Plan, Noncurrent' nas colunas '2023-12-31' e '2022-12-31' estão zeradas
# As informações estão na linha 'Liability, Defined Benefit Pension Plan, Noncurrent'
df_balance_sheet_10k.loc['Liability, Defined Benefit Plan, Noncurrent', '2023-12-31'] = df_balance_sheet_10k.loc['Liability, Defined Benefit Pension Plan, Noncurrent', '2023-12-31'] 
df_balance_sheet_10k.loc['Liability, Defined Benefit Plan, Noncurrent', '2022-12-31'] = df_balance_sheet_10k.loc['Liability, Defined Benefit Pension Plan, Noncurrent', '2022-12-31'] 

# Removendo as linhas que movemos os seus itens para as linhas corretas
df_balance_sheet_10k = df_balance_sheet_10k.drop('Liability, Defined Benefit Pension Plan, Noncurrent')

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_balance_sheet_10k = df_balance_sheet_10k.drop([
    'us-gaap_AssetsCurrentAbstract',
    'Accounts Receivable, Related Parties, Current',
    'Assets for Plan Benefits, Defined Benefit Plan',
    'Accounts Payable, Related Parties, Current',
    'Due to Affiliate, Noncurrent',
    'Accounts Payable, Related Parties, Noncurrent'
])

# Arrumando e adicionando alguns números no df
df_balance_sheet_10k.loc['Goodwill', '2022-12-31'] = 47
df_balance_sheet_10k.loc['Accounts Payable and Accrued Liabilities, Current', '2022-12-31'] = 3734 

# Renomeando a linha do índice que está sem formato 
df_balance_sheet_10k = df_balance_sheet_10k.rename(index={
    'ceg_RenewableEnergyCreditsCurrent': 'Renewable Energy Credits Current',
    'ceg_AssetRetirementObligationsExcludingSpentNuclearFuelNoncurrent': 'Asset Retirement Obligations Excluding Spent Nuclear Fuel Non current',
    'ceg_PayableToRegulatoryAgreementUnitsNoncurrent': 'Payable To Regulatory Agreement Units Non current',
    'us-gaap_MembersEquity': 'Members Equity',
    'us-gaap_MembersEquityAttributableToNoncontrollingInterest': 'Members Equity Attributable To Non controlling Interest',
    'us-gaap_AccountsReceivableGrossCurrent': 'Accounts Receivable Gross Current',
    'us-gaap_OtherReceivablesGrossCurrent': 'Other Receivables Gross Current',
    'us-gaap_AssetsOfDisposalGroupIncludingDiscontinuedOperationCurrent': 'Assets Of Disposal Group Including Discontinued Operation Current',
    'us-gaap_LiabilitiesOfDisposalGroupIncludingDiscontinuedOperationCurrent': 'Liabilities Of Disposal Group Including Discontinued Operation Current',
    'us-gaap_AssetRetirementObligationsNoncurrent': 'Asset Retirement Obligations Non current',  
})

df_balance_sheet_10k

In [ ]:
# Fazendo o scraping do income statement (10-K)
income_statement_lst = []

for acc_num in df_lst_acc_num_constellation['acc_num']:

    # Income statement
    income_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='income_statement')
    income_statement = f_sec.rename_statement(income_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    income_statement = income_statement.iloc[:, 0]
    income_statement_lst.append(income_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_income_statement_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in income_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_income_statement_10k = pd.concat([df_income_statement_10k, df], axis=1)

# Substituindo os NaN por 0
df_income_statement_10k = df_income_statement_10k.fillna(0)

# A linha 'InterestExpenseNonoperating' nas colunas '2023-12-31' e '2022-12-31' estão zeradas
# As informações estão na linha 'Interest Expense'
df_income_statement_10k.loc['InterestExpenseNonoperating', '2023-12-31'] = df_income_statement_10k.loc['Interest Expense', '2023-12-31'] 
df_income_statement_10k.loc['InterestExpenseNonoperating', '2022-12-31'] = df_income_statement_10k.loc['Interest Expense', '2022-12-31'] 

# Removendo as linhas que movemos os seus itens para as linhas corretas
df_income_statement_10k = df_income_statement_10k.drop('Interest Expense')

# Renomeando a linha do índice que está sem formato 
df_income_statement_10k = df_income_statement_10k.rename(index={
    'ceg_RelatedPartyCostsOperatingAndMaintenance': 'Related Party Costs Operating And Maintenance',
    'ceg_RelatedPartyTransactionInterestExpense': 'Related Party Transaction Interest Expense',
    'us-gaap_RelatedPartyTransactionExpensesFromTransactionsWithRelatedParty': 'Related Party Transaction Expenses From Transactions With Related Party',
    'us-gaap_InterestExpenseRelatedParty': 'Interest Expense Related Party',
    'us-gaap_IncomeLossFromEquityMethodInvestmentsNetOfDividendsOrDistributions': 'Income Loss From Equity Method Investments Net Of Dividends Or Distributions',
})

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_income_statement_10k = df_income_statement_10k.drop('us-gaap_RevenuesAbstract')

df_income_statement_10k

In [ ]:
# Fazendo o scraping do cash flow statement (10-K)
cash_flow_statement_lst = []

for acc_num in df_lst_acc_num_constellation['acc_num']:

    # Cash flow statement
    cash_flow_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='cash_flow_statement')
    cash_flow_statement = f_sec.rename_statement(cash_flow_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    cash_flow_statement = cash_flow_statement.iloc[:, 0]
    cash_flow_statement_lst.append(cash_flow_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_cash_flow_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in cash_flow_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_cash_flow_10k = pd.concat([df_cash_flow_10k, df], axis=1)

# Substituindo os NaN por 0
df_cash_flow_10k = df_cash_flow_10k.fillna(0)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_cash_flow_10k = df_cash_flow_10k.drop([
    'us-gaap_NetCashProvidedByUsedInOperatingActivitiesAbstract',
    'ceg_IncreaseDecreaseInMoneyPoolWithAffiliate',
    'ceg_PaymentsToAcquireNoncontrollingInterests',
    'Impairment of Long-Lived Assets Held-for-use'
])

# Renomeando as linhas do índice que estão sem formato 
df_cash_flow_10k = df_cash_flow_10k.rename(index={
    'ceg_RealizedAndUnrealizedGainLossOnInvestments': 'Realized And Unrealized Gain Loss On Investments',
    'ceg_IncreaseDecreaseInOptionPremiumsCollectedReceived': 'Increase Decrease In Option Premiums Collected Received',
    'ceg_RetirementOfLongTermDebtDueToAffiliate': 'Retirement Of Long Term Debt Due To Affiliate',
    'ceg_ChangeinCapitalExpendituresIncurredButNotYetPaid': 'Change in Capital Expenditures Incurred But Not Yet Paid',
    'ceg_OilAndGasNoncashAssetRetirementObligationIncreaseDecrease': 'Oil And Gas Non cash Asset Retirement Obligation Increase Decrease',
    'us-gaap_PaymentsToAcquireBusinessesGross': 'Payments To Acquire Businesses Gross',
    'us-gaap_GainLossOnSalesOfAssetsAndAssetImpairmentCharges': 'Gain Loss On Sales Of Assets And Asset Impairment Charges',
    'us-gaap_GainLossOnInvestments': 'Gain Loss On Investments',
    'us-gaap_DebtAndEquitySecuritiesUnrealizedGainLoss': 'Debt And Equity Securities Unrealized Gain Loss',
    'ceg_ChangeinReceivablesFromPayablesToAffiliatesNet': 'Change in Receivables From Payables To Affiliates Net',
    'ceg_ChangesInOptionPremiumsCollectedReceived': 'Changes In Option Premiums Collected Received',
    'us-gaap_IncreaseDecreaseInDueToAffiliatesCurrent': 'Increase Decrease In Due To Affiliates Current',
    'us-gaap_PaymentsToAcquireAdditionalInterestInSubsidiaries': 'Payments To Acquire Additional Interest In Subsidiaries',
})

df_cash_flow_10k

In [ ]:
# O 'df_income_statement_10k' não conseguiu puxar os dados da linha 'Weighted Average Number of Shares Outstanding, Basic' e não tem a linha 'Research and Development Expense'

# Linha 'Weighted Average Number of Shares Outstanding, Basic' a ser adicionada 
new_row_shares_basic_is_10k = [
    #2024
    315, 
    #2023
    323, 
    #2022
    328,
]

# Linha 'Weighted Average Number of Shares Outstanding, Diluted' a ser adicionada 
new_row_shares_diluted_is_10k = [
    #2024
    315, 
    #2023
    324, 
    #2022
    329,
]

# Linha 'Research and Development Expense' a ser adicionada 
new_row_rd_is_10k = [
    #2024
    0, 
    #2023
    0, 
    #2022
    0,
]

# Adicionando a nova linha
df_income_statement_10k.loc['Number of Shares - Basic'] = new_row_shares_basic_is_10k
df_income_statement_10k.loc['Number of Shares - Diluted'] = new_row_shares_diluted_is_10k
df_income_statement_10k.loc['Research and Development'] = new_row_rd_is_10k

In [ ]:
# O 'df_balance_sheet_10k' não conseguiu puxar os dados da linha 'Total Shareholders Equity'

# Linha 'Total Shareholders Equity' a ser adicionada 
new_row_bs_10k = [
    #2024
    13166, 
    #2023
    10925, 
    #2022
    11018,
]

# Adicionando a nova linha
df_balance_sheet_10k.loc['Total Shareholders Equity'] = new_row_bs_10k

In [ ]:
# O 'df_cash_flow_10k' não conseguiu puxar os dados das linhas 'Acquisitions of Assets and Businesses', 'Payments of Dividends' e 'Payments for Repurchase of Common Stock'

# Linhas 'Acquisitions of Assets and Businesses' a ser adicionada 
new_acquisitions_div_cf_10k = [
    #2024
    32, 
    #2023
    1690, 
    #2022
    29,
]

# Linhas 'Payments of Dividends' a ser adicionada 
new_row_div_cf_10k = [
    #2024
    444, 
    #2023
    366, 
    #2022
    185,
]

# Linhas 'Payments for Repurchase of Common Stock' a ser adicionada 
new_row_repurchase_cf_10k = [
    #2024
    999, 
    #2023
    992, 
    #2022
    0,
]

# Adicionando as novas linhas
df_cash_flow_10k.loc['Acquisitions of Assets and Businesses'] = new_acquisitions_div_cf_10k
df_cash_flow_10k.loc['Payments of Dividends'] = new_row_div_cf_10k
df_cash_flow_10k.loc['Payments for Repurchase of Common Stock'] = new_row_repurchase_cf_10k

In [ ]:
# Arrumando alguns index dos dfs para ficar padronizado com os index utilizados nas funções p/ calcular os indicadores fundamentalistas

df_income_statement_10k = df_income_statement_10k.rename(index={
    'Operating Income (Loss)': 'Operating Income',
    'InterestExpenseNonoperating': 'Interest Expense',
    'Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest': 'Income Before Provision for Income Taxes',
    'Income Tax Expense (Benefit)': 'Provision for Income Taxes',
    'Net Income (Loss) Attributable to Parent': 'Net Income'
})

df_cash_flow_10k = df_cash_flow_10k.rename(index={
    'Depreciation, Amortization and Accretion, Net': 'Depreciation, Depletion and Amortization',
})

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//energia_eletrica//constellation//ceg_10k.xlsx') as writer:
    df_balance_sheet_10k.to_excel(writer, sheet_name='balance_sheet')
    df_income_statement_10k.to_excel(writer, sheet_name='income_statement')
    df_cash_flow_10k.to_excel(writer, sheet_name='cash_flow')

### Scraping Constellation 10-Q

In [ ]:
# Número dos documentos que contém a DRE, BP e FC 
acc_documents_10q = f_sec.get_filtered_filings(ticker=ticker, ten_k=False, just_accession_numbers=True)

# Trocando os '-' para '' das strings ('0001045810-24-000264' -> '000104581024000264')
dict_acc_num_10q = {k: v.replace('-', '') for k, v in acc_documents_10q.items()}

# Transformando o dicionário em um df
df_lst_acc_num_10q_ceg = pd.DataFrame.from_dict(dict_acc_num_10q, orient='index', columns=['acc_num'])

# Transformando em datetime o index
df_lst_acc_num_10q_ceg.index = pd.to_datetime(df_lst_acc_num_10q_ceg.index, format='%Y-%m-%d')
df_lst_acc_num_10q_ceg

In [ ]:
# Rótulos
labels_dict = f_sec.get_label_dictionary(ticker, headers=f_sec.header)

In [ ]:
# Fazendo o scraping do balance sheet (10-Q)
balance_sheet_lst = []

for acc_num in df_lst_acc_num_10q_ceg['acc_num']:

    # Balance sheet
    balance_sheet = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='balance_sheet')
    balance_sheet = f_sec.rename_statement(balance_sheet, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    balance_sheet = balance_sheet.iloc[:, 0]
    balance_sheet_lst.append(balance_sheet)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_balance_sheet_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in balance_sheet_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_balance_sheet_10q = pd.concat([df_balance_sheet_10q, df], axis=1)

# Substituindo os NaN por 0
df_balance_sheet_10q = df_balance_sheet_10q.fillna(0)

# A linha 'Liability, Defined Benefit Plan, Noncurrent' nas colunas '2024-09-30', '2024-06-30', '2024-03-31', '2023-09-30', '2023-06-30', '2023-03-31', '2022-09-30', '2022-06-30' e '2022-03-31' estão zeradas
# As informações estão na linha 'Liability, Other Postretirement Defined Benefit Plan, Noncurrent'
df_balance_sheet_10q.loc['Liability, Defined Benefit Plan, Noncurrent', '2024-09-30'] = df_balance_sheet_10q.loc['Liability, Other Postretirement Defined Benefit Plan, Noncurrent', '2024-09-30'] 
df_balance_sheet_10q.loc['Liability, Defined Benefit Plan, Noncurrent', '2024-06-30'] = df_balance_sheet_10q.loc['Liability, Other Postretirement Defined Benefit Plan, Noncurrent', '2024-06-30'] 
df_balance_sheet_10q.loc['Liability, Defined Benefit Plan, Noncurrent', '2024-03-31'] = df_balance_sheet_10q.loc['Liability, Other Postretirement Defined Benefit Plan, Noncurrent', '2024-03-31'] 
df_balance_sheet_10q.loc['Liability, Defined Benefit Plan, Noncurrent', '2023-09-30'] = df_balance_sheet_10q.loc['Liability, Other Postretirement Defined Benefit Plan, Noncurrent', '2023-09-30'] 
df_balance_sheet_10q.loc['Liability, Defined Benefit Plan, Noncurrent', '2023-06-30'] = df_balance_sheet_10q.loc['Liability, Other Postretirement Defined Benefit Plan, Noncurrent', '2023-06-30'] 
df_balance_sheet_10q.loc['Liability, Defined Benefit Plan, Noncurrent', '2023-03-31'] = df_balance_sheet_10q.loc['Liability, Other Postretirement Defined Benefit Plan, Noncurrent', '2023-03-31'] 
df_balance_sheet_10q.loc['Liability, Defined Benefit Plan, Noncurrent', '2022-09-30'] = df_balance_sheet_10q.loc['Liability, Other Postretirement Defined Benefit Plan, Noncurrent', '2022-09-30'] 
df_balance_sheet_10q.loc['Liability, Defined Benefit Plan, Noncurrent', '2022-06-30'] = df_balance_sheet_10q.loc['Liability, Other Postretirement Defined Benefit Plan, Noncurrent', '2022-06-30'] 
df_balance_sheet_10q.loc['Liability, Defined Benefit Plan, Noncurrent', '2022-03-31'] = df_balance_sheet_10q.loc['Liability, Other Postretirement Defined Benefit Plan, Noncurrent', '2022-03-31'] 

# A linha 'Accounts Payable and Accrued Liabilities, Current' nas colunas '2023-06-30', '2023-03-31', '2022-09-30', '2022-06-30' e '2022-03-31' estão zeradas
# As informações estão na linha 'Accrued Liabilities, Current'
df_balance_sheet_10q.loc['Accounts Payable and Accrued Liabilities, Current', '2023-06-30'] = df_balance_sheet_10q.loc['Accrued Liabilities, Current', '2023-06-30'] 
df_balance_sheet_10q.loc['Accounts Payable and Accrued Liabilities, Current', '2023-03-31'] = df_balance_sheet_10q.loc['Accrued Liabilities, Current', '2023-03-31'] 
df_balance_sheet_10q.loc['Accounts Payable and Accrued Liabilities, Current', '2022-09-30'] = df_balance_sheet_10q.loc['Accrued Liabilities, Current', '2022-09-30'] 
df_balance_sheet_10q.loc['Accounts Payable and Accrued Liabilities, Current', '2022-06-30'] = df_balance_sheet_10q.loc['Accrued Liabilities, Current', '2022-06-30'] 
df_balance_sheet_10q.loc['Accounts Payable and Accrued Liabilities, Current', '2022-03-31'] = df_balance_sheet_10q.loc['Accrued Liabilities, Current', '2022-03-31'] 

# Removendo as linhas que movemos os seus itens para as linhas corretas
df_balance_sheet_10q = df_balance_sheet_10q.drop([
    'Liability, Other Postretirement Defined Benefit Plan, Noncurrent',
    'Accrued Liabilities, Current'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_balance_sheet_10q = df_balance_sheet_10q.drop([
    'us-gaap_AssetsCurrentAbstract',
    'Assets for Plan Benefits, Defined Benefit Plan',
    'Due to Affiliate, Noncurrent',
    'Accounts Payable, Related Parties, Noncurrent',
    'Accounts Receivable, Related Parties, Current',
    'Accounts Payable, Related Parties, Current'
])

# Renomeando algumas linhas do índice
df_balance_sheet_10q = df_balance_sheet_10q.rename(index={
    'ceg_RenewableEnergyCreditsCurrent': 'Renewable Energy Credits Current',
    'ceg_AssetRetirementObligationsExcludingSpentNuclearFuelNoncurrent': 'Asset Retirement Obligations Excluding Spent Nuclear Fuel Noncurrent',
    'ceg_PayableToRegulatoryAgreementUnitsNoncurrent': 'Payable To Regulatory Agreement Units Noncurrent',
    'us-gaap_MembersEquity': 'Members Equity',
    'us-gaap_MembersEquityAttributableToNoncontrollingInterest': 'Members Equity Attributable To Noncontrolling Interest',
})

df_balance_sheet_10q

In [ ]:
# Fazendo o scraping do income statement (10-Q)
income_statement_lst = []

for acc_num in df_lst_acc_num_10q_ceg['acc_num']:

    # Income statement
    income_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='income_statement')
    income_statement = f_sec.rename_statement(income_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    income_statement = income_statement.iloc[:, 0]
    income_statement_lst.append(income_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_income_statement_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in income_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_income_statement_10q = pd.concat([df_income_statement_10q, df], axis=1)

# Substituindo os NaN por 0
df_income_statement_10q = df_income_statement_10q.fillna(0)

# A linha 'InterestExpenseNonoperating' nas colunas '2024-03-31', '2023-09-30', '2023-06-30', '2023-03-31', '2022-09-30', '2022-06-30' e '2022-03-31' estão zeradas
# As informações estão na linha 'Interest Expense'
df_income_statement_10q.loc['InterestExpenseNonoperating', '2024-03-31'] = df_income_statement_10q.loc['Interest Expense', '2024-03-31'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2023-09-30'] = df_income_statement_10q.loc['Interest Expense', '2023-09-30'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2023-06-30'] = df_income_statement_10q.loc['Interest Expense', '2023-06-30'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2023-03-31'] = df_income_statement_10q.loc['Interest Expense', '2023-03-31'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2022-09-30'] = df_income_statement_10q.loc['Interest Expense', '2022-09-30'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2022-06-30'] = df_income_statement_10q.loc['Interest Expense', '2022-06-30'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2022-03-31'] = df_income_statement_10q.loc['Interest Expense', '2022-03-31'] 

# A linha 'Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest' na coluna '2025-03-31' está zerada
# As informações estão na linha 'Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest'
df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2025-03-31'] = df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2025-03-31'] 

# Removendo as linhas que movemos os seus itens para as linhas corretas
df_income_statement_10q = df_income_statement_10q.drop([
    'Interest Expense',
    'Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_income_statement_10q = df_income_statement_10q.drop([
    'us-gaap_OperatingExpensesAbstract',
    'us-gaap_RevenuesAbstract',
    'Related Party Transaction, Amounts of Transaction',
    'ceg_RelatedPartyCostsOperatingAndMaintenance',
    'ceg_RelatedPartyTransactionInterestExpense',
    'Revenue from Related Parties',
    'Related Party Transaction, Purchases from Related Party'
])

df_income_statement_10q

In [ ]:
# Fazendo o scraping do cash flow statement (10-Q)
cash_flow_statement_lst = []

for acc_num in df_lst_acc_num_10q_ceg['acc_num']:

    # Cash flow statement
    cash_flow_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='cash_flow_statement')
    cash_flow_statement = f_sec.rename_statement(cash_flow_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    cash_flow_statement = cash_flow_statement.iloc[:, 0]
    cash_flow_statement_lst.append(cash_flow_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_cash_flow_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in cash_flow_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_cash_flow_10q = pd.concat([df_cash_flow_10q, df], axis=1)

# Substituindo os NaN por 0
df_cash_flow_10q = df_cash_flow_10q.fillna(0)

# A linha 'Equity Securities, FV-NI, Gain (Loss)' nas colunas '2022-09-30', '2022-06-30' e '2022-03-31' estão zeradas
# As informações estão na linha 'Equity Securities, FV-NI, Unrealized Gain (Loss)'
df_cash_flow_10q.loc['Equity Securities, FV-NI, Gain (Loss)', '2022-09-30'] = df_cash_flow_10q.loc['Equity Securities, FV-NI, Unrealized Gain (Loss)', '2022-09-30'] 
df_cash_flow_10q.loc['Equity Securities, FV-NI, Gain (Loss)', '2022-06-30'] = df_cash_flow_10q.loc['Equity Securities, FV-NI, Unrealized Gain (Loss)', '2022-06-30'] 
df_cash_flow_10q.loc['Equity Securities, FV-NI, Gain (Loss)', '2022-03-31'] = df_cash_flow_10q.loc['Equity Securities, FV-NI, Unrealized Gain (Loss)', '2022-03-31'] 

# A linha 'Repayments of Short-term Debt, Maturing in More than Three Months' nas colunas '2022-06-30' e '2022-03-31' estão zeradas
# As informações estão na linha 'Proceeds from (Repayments of) Short-term Debt, Maturing in More than Three Months'
df_cash_flow_10q.loc['Repayments of Short-term Debt, Maturing in More than Three Months', '2022-06-30'] = df_cash_flow_10q.loc['Proceeds from (Repayments of) Short-term Debt, Maturing in More than Three Months', '2022-06-30'] 
df_cash_flow_10q.loc['Repayments of Short-term Debt, Maturing in More than Three Months', '2022-03-31'] = df_cash_flow_10q.loc['Proceeds from (Repayments of) Short-term Debt, Maturing in More than Three Months', '2022-03-31'] 

# Removendo as linhas que movemos os seus itens para as linhas corretas
df_cash_flow_10q = df_cash_flow_10q.drop([
    'Equity Securities, FV-NI, Unrealized Gain (Loss)',
    'Proceeds from (Repayments of) Short-term Debt, Maturing in More than Three Months'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_cash_flow_10q = df_cash_flow_10q.drop([
    'us-gaap_NetCashProvidedByUsedInOperatingActivitiesAbstract',
    'Impairment of Long-Lived Assets Held-for-use',
    'Increase (Decrease) in Due to Affiliates',
    'ceg_PaymentsToAcquireNoncontrollingInterests'
])

# Renomeando algumas linhas do índice
df_cash_flow_10q = df_cash_flow_10q.rename(index={
    'ceg_RealizedAndUnrealizedGainLossOnInvestments': 'Realized And Unrealized Gain Loss On Investments',
    'ceg_IncreaseDecreaseInOptionPremiumsCollectedReceived': 'Increase Decrease In Option Premiums Collected Received',
    'ceg_ChangeinCapitalExpendituresIncurredButNotYetPaid': 'Change in Capital Expenditures Incurred But Not Yet Paid',
    'ceg_PropertyPlantAndEquipmentPeriodIncreaseDecreaseRelatedToAssetRetirementObligationAdjustment': 'Property Plant And Equipment Period Increase Decrease Related To Asset Retirement Obligation Adjustment',
    'ceg_RetirementOfLongTermDebtDueToAffiliate': 'Retirement Of Long Term Debt Due To Affiliate',
})

df_cash_flow_10q

In [ ]:
# O 'df_income_statement_10q' não conseguiu puxar os dados da linha 'Weighted Average Number of Shares Outstanding, Basic' e não tem a linha 'Research and Development Expense'

# Linha 'Weighted Average Number of Shares Outstanding, Basic' a ser adicionada 
new_row_shares_basic_is_10q = [
    # 2025-09-30
    313,
    # 2025-06-30
    314,
    # 2025-03-31
    313,	
    # 2024-09-30	
    313,	
    # 2024-06-30	
    315,	
    # 2024-03-31	
    317,	
    # 2023-09-30	
    322,	
    # 2023-06-30	
    324,	
    # 2023-03-31	
    328,	
    # 2022-09-30	
    327,	
    # 2022-06-30	
    327,	
    # 2022-03-31
    327,	
]

new_row_shares_diluted_is_10q = [
    # 2025-09-30
    313,
    # 2025-06-30
    314,
    # 2025-03-31
    314,	
    # 2024-09-30	
    314,	
    # 2024-06-30	
    316,	
    # 2024-03-31	
    318,	
    # 2023-09-30	
    323,	
    # 2023-06-30	
    325,	
    # 2023-03-31	
    328,	
    # 2022-09-30	
    328,	
    # 2022-06-30	
    328,	
    # 2022-03-31
    327,	
]

# Linha 'Research and Development Expense' a ser adicionada 
new_row_rd_is_10q = [
    # 2025-09-30
    0,    
    # 2025-06-30
    0,
    # 2025-03-31
    0,	
    # 2024-09-30	
    0,	
    # 2024-06-30	
    0,	
    # 2024-03-31	
    0,	
    # 2023-09-30	
    0,	
    # 2023-06-30	
    0,	
    # 2023-03-31	
    0,	
    # 2022-09-30	
    0,	
    # 2022-06-30	
    0,	
    # 2022-03-31
    0,	
]

# Adicionando a nova linha
df_income_statement_10q.loc['Number of Shares - Basic'] = new_row_shares_basic_is_10q
df_income_statement_10q.loc['Number of Shares - Diluted'] = new_row_shares_diluted_is_10q
df_income_statement_10q.loc['Research and Development'] = new_row_rd_is_10q

In [ ]:
# O 'df_balance_sheet_10q' não conseguiu puxar os dados da linha "Stockholders' Equity Attributable to Parent"

# Linha "Stockholders' Equity Attributable to Parent" a ser adicionada 
new_row_bs_10q = [
    # 2025-09-30,
    14350,
    # 2025-06-30
    13446,
    # 2025-03-31
    12956,	
    # 2024-09-30	
    12570,	
    # 2024-06-30	
    11425,	
    # 2024-03-31	
    11199,	
    # 2023-09-30	
    11666,	
    # 2023-06-30	
    11256,	
    # 2023-03-31	
    10728,	
    # 2022-09-30	
    10803,	
    # 2022-06-30	
    11000,	
    # 2022-03-31
    11105,	
]

# Adicionando a nova linha
df_balance_sheet_10q.loc['Total Shareholders Equity'] = new_row_bs_10q

In [ ]:
# O 'df_cash_flow_10q' não conseguiu puxar os dados das linhas 'Payments of Dividends' e 'Payments for Repurchase of Common Stock'

# Linhas 'Payments of Dividends' a ser adicionada 
new_row_div_cf_10q = [
    # 2025-09-30
    366,
    # 2025-06-30
    244,
    # 2025-03-31
    122,	
    # 2024-09-30	
    333,	
    # 2024-06-30	
    222,	
    # 2024-03-31	
    112,	
    # 2023-09-30	
    277,	
    # 2023-06-30	
    185,	
    # 2023-03-31	
    93,	
    # 2022-09-30	
    139,	
    # 2022-06-30	
    93,	
    # 2022-03-31
    46,	
]

# Linhas 'Payments for Repurchase of Common Stock' a ser adicionada 
new_row_repurchase_cf_10q = [
    # 2025-09-30
    400,
    # 2025-06-30
    400,
    # 2025-03-31
    0,	
    # 2024-09-30	
    999000,	
    # 2024-06-30	
    999000,	
    # 2024-03-31	
    499000,	
    # 2023-09-30	
    750000,	
    # 2023-06-30	
    499000,	
    # 2023-03-31	
    231000,	
    # 2022-09-30	
    0,	
    # 2022-06-30	
    0,	
    # 2022-03-31
    0,	
]

# Adicionando as novas linhas
df_cash_flow_10q.loc['Payments of Dividends'] = new_row_div_cf_10q
df_cash_flow_10q.loc['Payments for Repurchase of Common Stock'] = new_row_repurchase_cf_10q

In [ ]:
# Arrumando alguns index dos dfs para ficar padronizado com os index utilizados nas funções p/ calcular os indicadores fundamentalistas
df_income_statement_10q = df_income_statement_10q.rename(index={
    'Operating Income (Loss)': 'Operating Income',
    'InterestExpenseNonoperating': 'Interest Expense',
    'Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest': 'Income Before Provision for Income Taxes',
    'Income Tax Expense (Benefit)': 'Provision for Income Taxes',
    'Net Income (Loss) Attributable to Parent': 'Net Income'
})

df_cash_flow_10q = df_cash_flow_10q.rename(index={
    'Depreciation, Amortization and Accretion, Net': 'Depreciation, Depletion and Amortization',
})

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//energia_eletrica//constellation//ceg_10q.xlsx') as writer:
    df_balance_sheet_10q.to_excel(writer, sheet_name='balance_sheet')
    df_income_statement_10q.to_excel(writer, sheet_name='income_statement')
    df_cash_flow_10q.to_excel(writer, sheet_name='cash_flow')

### Indicadores Fundamentalistas

#### Indicadores 10-K

In [ ]:
# Lendo os arquivos 10k
ceg_bs = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//energia_eletrica//constellation//ceg_10k.xlsx', 
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

ceg_is = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//energia_eletrica//constellation//ceg_10k.xlsx', 
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

ceg_cf = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//energia_eletrica//constellation//ceg_10k.xlsx', 
    sheet_name='cash_flow', 
    index_col='Unnamed: 0'
)

# Transformando as colunas em datetime
ceg_bs.columns = pd.to_datetime(ceg_bs.columns)
ceg_is.columns = pd.to_datetime(ceg_is.columns)
ceg_cf.columns = pd.to_datetime(ceg_cf.columns)

# Transformando as colunas em apenas o ano
ceg_bs.columns = ceg_bs.columns.year
ceg_is.columns = ceg_is.columns.year
ceg_cf.columns = ceg_cf.columns.year

In [ ]:
# Preços da CEG
ceg_close = yf.download('CEG', auto_adjust=True, multi_level_index=False, start='2022-01-01')['Close']

# Selecionando o último preço do mês de Dezembro de cada ano 
lst_years = [
    '2022', '2023', '2024',
]

last_dec_close = []

for year in lst_years:
    last_close = ceg_close[f'{year}-12'].iloc[-1]
    last_dec_close.append(last_close)

# Transformando em uma série pandas
ceg_close_dec = pd.Series(last_dec_close, index=lst_years)

# Revertendo o index da série p/ ficar igual com os dos indicadores
ceg_close_dec = ceg_close_dec.reindex(index=ceg_close_dec.index[::-1])

# Arredondamento de duas casas decimais
ceg_close_dec = round(ceg_close_dec, 2)

# Não realizou split recentemente (não possui dados de split no site da Investing)

# Transformando o index em int
ceg_close_dec.index = ceg_close_dec.index.astype(int)

ceg_close_dec

In [ ]:
# Criando uma lista só para fazer a indexação em que o ano mais recente está no começo
idx = ceg_bs.columns

# Valor de Mercado
vm = f_eua.indicador_vm(ten_k=True, price=ceg_close_dec, df_is=ceg_is) 

# LPA
lpa = f_eua.indicador_lpa(ten_k=True, df_is=ceg_is, idx=None)

# P/L Damodaran
p_lpa_damodaran = f_eua.indicador_p_lpa_2(ten_k=True, vm=vm, df_is=ceg_is)

# P/L
p_lpa = f_eua.indicador_p_lpa(ten_k=True, price=ceg_close_dec, lpa=lpa, idx=idx)

# L/P
lpa_p = f_eua.indicador_lpa_p(ten_k=True, price=ceg_close_dec, lpa=lpa, idx=idx) 

# Depreciação
depreciacao = f_eua.indicador_depreciacao(ten_k=True, df_cf=ceg_cf, first_quarter=None, idx=idx)

# EBITDA
ebitda = f_eua.indicador_ebitda(ten_k=True, df_is=ceg_is, idx=None, depreciacao=depreciacao)

# Margem Líquida
m_liq = f_eua.indicador_ml(ten_k=True, df_is=ceg_is)

# VPA
vpa = f_eua.indicador_vpa(ten_k=True, df_bs=ceg_bs, df_is=ceg_is)

# P/VPA
p_vpa = f_eua.indicador_p_vpa(ten_k=True, price=ceg_close_dec, vpa=vpa, idx=idx)

# Caixa e Equivalentes
lst_caixa = [
    'Cash and Cash Equivalents, at Carrying Value', 
    'Restricted Cash and Cash Equivalents, Current'
]

caixa = f_eua.indicador_caixa(ten_k=True, df_bs=ceg_bs, lst_itens_caixa=lst_caixa)

# Dívida Bruta
lst_divida = [
    'Short-term Debt', 
    'Long-term Debt and Lease Obligation, Current', 
    'Long-term Debt, Excluding Current Maturities'
]

div_bruta = f_eua.indicador_divida_bruta(ten_k=True, df_bs=ceg_bs, lst_itens_divida=lst_divida)

# Dívida Líquida
div_liq = f_eua.indicador_divida_liquida(ten_k=True, caixa=caixa, div_bruta=div_bruta, idx=idx)

# Dívida Líquida / EBITDA
dl_ebitda = f_eua.indicador_dl_ebitda(ten_k=True, ebitda=ebitda, div_liq=div_liq, idx=None)

# Dívida Líquida / PL
dl_pl = f_eua.indicador_dl_pl(ten_k=True, div_liq=div_liq, df_bs=ceg_bs) 

# EV/EBITDA
ev_ebitda = f_eua.indicador_ev_ebitda(vm=vm, div_liq=div_liq, ebitda=ebitda)

# ROE
roe = f_eua.indicador_roe(ten_k=True, df_is=ceg_is, df_bs=ceg_bs, idx=None)

# ROIC
roic = f_eua.indicador_roic(ten_k=True, df_is=ceg_is, df_bs=ceg_bs, div_bruta=div_bruta, idx=idx)

# FCO
fco = f_eua.indicador_fco(ten_k=True, df_cf=ceg_cf, first_quarter=None) 

# FCI
fci = f_eua.indicador_fci(ten_k=True, df_cf=ceg_cf, first_quarter=None) 

# FCF
fcf = f_eua.indicador_fcf(ten_k=True, df_cf=ceg_cf, first_quarter=None) 

# CAPEX
lst_capex = ['Payments to Acquire Property, Plant, and Equipment']

capex= f_eua.indicador_capex(ten_k=True, df_cf=ceg_cf, first_quarter=None, lst_itens=lst_capex, idx=idx)

# Free Cash Flow
free_cash_flow = f_eua.indicador_free_cash_flow(fco=fco, capex=capex)

# Net CAPEX
net_capex = f_eua.indicador_net_capex(capex=capex, depreciation=depreciacao)

# R&D
rd_expense = f_eua.indicador_rd(ten_k=True, df_is=ceg_is, idx=idx)

# Net CAPEX ajustado
adj_net_capex = f_eua.indicador_adj_net_capex(net_capex=net_capex, rd_expense=rd_expense)

# Working capital
working_capital = f_eua.indicador_working_capital(ten_k=True, df_bs=ceg_bs)

# Change in non cash working capital
lst_itens_change_non_cash_wc = [
    'Deferred Income Taxes and Tax Credits', 
    'Increase (Decrease) in Receivables', 
    'Increase (Decrease) in Due to Related Parties',
    'Increase (Decrease) in Inventories',
    'Increase (Decrease) in Accounts Payable and Accrued Liabilities',
    'Increase Decrease In Option Premiums Collected Received',
    'Increase (Decrease) Cash Collateral from Counterparties',
    'Increase (Decrease) in Income Taxes Payable',
    'Increase (Decrease) in Obligation, Pension and Other Postretirement Benefits',
    'Increase (Decrease) in Other Operating Assets and Liabilities, Net'
]

change_non_cash_wc = f_eua.indicador_change_non_cash_wc(ten_k=True, df_cf=ceg_cf, lst_itens=lst_itens_change_non_cash_wc, first_quarter=None)

# Reinvestment rate
reinvestment_rate = f_eua.indicador_reinvestment_rate(ten_k=True, net_capex=net_capex, change_non_cash_wc=change_non_cash_wc, df_is=ceg_is, idx=idx)

# New borrowing
lst_itens_new_borrowing = [
    'Proceeds from Short-term Debt, Maturing in More than Three Months', 
    'Proceeds from Issuance of Long-term Debt'
]

new_borrowing = f_eua.indicador_new_borrowing(ten_k=True, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=ceg_cf, first_quarter=None, idx=idx)

# Debt paid
lst_itens_debt_paid = [
    'Repayments of Short-term Debt, Maturing in More than Three Months', 
    'Repayments of Long-term Debt'
]

debt_paid = f_eua.indicador_debt_paid(ten_k=True, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=ceg_cf, first_quarter=None, idx=idx)

# Taxes on operating income
taxes_operating_income = f_eua.indicador_taxes_operating_income(ten_k=True, df_is=ceg_is, idx=idx)

# FCFE
fcfe_10q = f_eua.indicador_fcfe(
    ten_k=True, 
    depreciacao=depreciacao, 
    capex=capex, 
    change_non_cash_wc=change_non_cash_wc, 
    new_borrowing=new_borrowing,
    debt_paid=debt_paid,
    df_is=ceg_is,
    idx=idx
)

# FCFF
fcff_10q = f_eua.indicador_fcff(
        ten_k=True, 
        taxes_operating_income=taxes_operating_income,
        depreciacao=depreciacao, 
        capex=capex, 
        change_non_cash_wc=change_non_cash_wc, 
        df_is=ceg_is,
        idx=idx
)

# Dividendo por Ação
dpa = f_eua.indicador_dpa(ten_k=True, df_cf=ceg_cf, df_is=ceg_is, div_nao_acum=None)

# Payout
payout = f_eua.indicador_payout(ten_k=True, df_cf=ceg_cf, df_is=ceg_is, div_nao_acum=None)

# Buyback
buyback = f_eua.indicador_buyback(ten_k=True, df_cf=ceg_cf, first_quarter=None)

# Colocando todos os indicadores em uma lista
lst_indicadores = [
    p_lpa_damodaran,
    p_lpa,
    lpa_p,  
    ebitda, 
    m_liq,
    p_vpa,
    vm,
    div_bruta,
    caixa,
    div_liq,
    dl_ebitda,
    dl_pl,
    ev_ebitda,
    roe,
    roic,
    fco,
    fci,
    fcf,
    free_cash_flow,
    capex,
    net_capex,
    rd_expense,
    adj_net_capex,
    working_capital,
    reinvestment_rate,
    fcfe_10q,
    fcff_10q,
    dpa,
    payout,
    buyback
]

# Juntando todos os indicadores em um df
ceg_indicadores_10k = pd.concat(lst_indicadores, axis=1)

# Nomes das colunas
nomes_col = [
    'P/L Damodaran', 
    'P/L', 
    'L/P', 
    'EBITDA', 
    'Margem Líquida',
    'P/VPA',
    'Valor de Mercado',
    'Dívida Bruta',
    'Caixa e Equivalentes',
    'Dívida Líquida',
    'Dívida Líquida/EBITDA',
    'Dívida Líquida/PL',
    'EV/EBITDA',
    'ROE',
    'ROIC',
    'FCO',
    'FCI',
    'FCF',
    'Free Cash Flow',
    'Capex',
    'Net Capex',
    'R&D',
    'Adj Net Capex',
    'Working Capital',
    'Reinvestment Rate',
    'FCFE',
    'FCFF',
    'DPA',
    'Payout',
    'Buyback'
]

# Atribuindo os novos nomes das colunas do df
ceg_indicadores_10k.columns = nomes_col
ceg_indicadores_10k

#### Indicadores 10-Q

In [ ]:
# Lendo os arquivos 10q
ceg_bs_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//energia_eletrica//constellation//ceg_10q.xlsx', 
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

ceg_is_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//energia_eletrica//constellation//ceg_10q.xlsx', 
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

ceg_cf_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//energia_eletrica//constellation//ceg_10q.xlsx', 
    sheet_name='cash_flow', 
    index_col='Unnamed: 0'
)

# Aplicando o ajuste nas colunas
ceg_bs_10q.columns = [f_sec.ajustar_mes_03_06_09(col) for col in ceg_bs_10q.columns]
ceg_is_10q.columns = [f_sec.ajustar_mes_03_06_09(col) for col in ceg_is_10q.columns]
ceg_cf_10q.columns = [f_sec.ajustar_mes_03_06_09(col) for col in ceg_cf_10q.columns]

In [ ]:
# Preços da CEG
ceg_close = yf.download('CEG', auto_adjust=True, multi_level_index=False, start='2023-01-01')['Close']

# Selecionando o último preço dos meses de Março, Junho e Setembro
lst_years = ['2023', '2024', '2025']
lst_months = ['03', '06', '09']
last_close_months = []

for year in lst_years:
    for month in lst_months:
        last_close = ceg_close[f'{year}-{month}'].dropna().iloc[-1]  # Dropna para lidar com meses sem dados
        last_close_months.append(last_close)

# Criando uma lista em que junte os meses e os anos para que seja o index da serie pandas
lst_idx = []

for year in lst_years:
    for month in lst_months:
        lst_idx.append(f"{month}-{year}")

# Transformando em uma série pandas
ceg_close_months = pd.Series(last_close_months, index=lst_idx)

# Revertendo o index da série p/ ficar igual com os dos indicadores
ceg_close_months = ceg_close_months.reindex(index=ceg_close_months.index[::-1])

# No ano mais atual, como ainda ele não acabou, não consigo fazer o loop com todos meses (Março, Junho e Setembro)
actual_year_int = datetime.now().year  
actual_year_str = str(actual_year_int) 

lst_actual_months = [
    # '03',
    # '06', 
    # '09'
] 

last_close_actual_months = []

for month in lst_actual_months:
    last_close = ceg_close[f'{actual_year_str}-{month}'].dropna().iloc[-1]  # Dropna para lidar com meses sem dados
    last_close_actual_months.append(last_close)

# Criando uma lista em que junte os meses e os anos para que seja o index da serie pandas
lst_idx_actual = []

for month in lst_actual_months:
    lst_idx_actual.append(f"{month}-{actual_year_str}")

# Transformando em uma série pandas
ceg_close_months_atual = pd.Series(last_close_actual_months, index=lst_idx_actual)

# Revertendo o index da série p/ ficar igual com os dos indicadores
ceg_close_months_atual = ceg_close_months_atual.reindex(index=ceg_close_months_atual.index[::-1])

# Juntando as duas series pandas
ceg_close_months = pd.concat([ceg_close_months_atual, ceg_close_months])

# Arredondamento de duas casas decimais
ceg_close_months = round(ceg_close_months, 2)

# Não realizou split recentemente (não possui dados de split no site da Investing)

# Substituindo o index para ficar igual ao dos dados fundamentalistas
ceg_close_months.index = ceg_bs_10q.columns[:-3]

ceg_close_months

In [ ]:
# Criando uma lista só para fazer a indexação em que o ano mais recente está no começo
idx_10q = ceg_bs_10q.columns

# Valor de Mercado
vm_10q = f_eua.indicador_vm(ten_k=False, price=ceg_close_months, df_is=ceg_is_10q)

# LPA
lpa_acum = f_eua.indicador_lpa(ten_k=False, df_is=ceg_is_10q, idx=idx_10q)[1]
lpa_10q = f_eua.indicador_lpa(ten_k=False, df_is=ceg_is_10q, idx=idx_10q)[0]

# P/L Damodaran
p_lpa_10q_damodaran = f_eua.indicador_p_lpa_2(ten_k=False, vm=vm_10q, df_is=ceg_is_10q)

# P/L
p_lpa_10q = f_eua.indicador_p_lpa(ten_k=False, price=ceg_close_months, lpa=lpa_acum, idx=idx_10q)

# L/P
lpa_p_10q = f_eua.indicador_lpa_p(ten_k=False, price=ceg_close_months, lpa=lpa_acum, idx=idx_10q) 

# Depreciação
depreciacao_10q = f_eua.indicador_depreciacao(ten_k=False, df_cf=ceg_cf_10q, first_quarter='03', idx=idx_10q)[0]
depreciacao_10q_sliced = f_eua.indicador_depreciacao(ten_k=False, df_cf=ceg_cf_10q, first_quarter='03', idx=idx_10q)[1]
depreciacao_10q_acum = f_eua.indicador_depreciacao(ten_k=False, df_cf=ceg_cf_10q, first_quarter='03', idx=idx_10q)[2]

# EBITDA
ebitda_10q = f_eua.indicador_ebitda(ten_k=False, df_is=ceg_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[0]
ebitda_10q_sliced = f_eua.indicador_ebitda(ten_k=False, df_is=ceg_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[1]
ebitda_10q_acum = f_eua.indicador_ebitda(ten_k=False, df_is=ceg_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[2]

# Margem Líquida
m_liq_10q = f_eua.indicador_ml(ten_k=False, df_is=ceg_is_10q)

# VPA
vpa_10q = f_eua.indicador_vpa(ten_k=False, df_bs=ceg_bs_10q, df_is=ceg_is_10q)

# P/VPA
p_vpa_10q = f_eua.indicador_p_vpa(ten_k=False, price=ceg_close_months, vpa=vpa_10q, idx=idx_10q)

# Caixa e Equivalentes
lst_caixa = [
    'Cash and Cash Equivalents, at Carrying Value', 
    'Restricted Cash and Cash Equivalents, Current'
]

caixa_10q = f_eua.indicador_caixa(ten_k=False, df_bs=ceg_bs_10q, lst_itens_caixa=lst_caixa)

# Dívida Bruta
lst_divida = [
    'Long-term Debt and Lease Obligation, Current', 
    'Long-term Debt, Excluding Current Maturities'
]

div_bruta_10q = f_eua.indicador_divida_bruta(ten_k=False, df_bs=ceg_bs_10q, lst_itens_divida=lst_divida)

# Dívida Líquida
div_liq_10q = f_eua.indicador_divida_liquida(ten_k=False, caixa=caixa_10q, div_bruta=div_bruta_10q, idx=idx_10q)

# Dívida Líquida / EBITDA
dl_ebitda_10q =  f_eua.indicador_dl_ebitda(ten_k=False, ebitda=ebitda_10q_acum, div_liq=div_liq_10q, idx=idx_10q)

# Dívida Líquida / PL
dl_pl_10q = f_eua.indicador_dl_pl(ten_k=False, div_liq=div_liq_10q, df_bs=ceg_bs_10q) 

# EV/EBITDA
ev_ebitda_10q = f_eua.indicador_ev_ebitda(vm=vm_10q, div_liq=div_liq_10q, ebitda=ebitda_10q_acum)

# ROE
roe_10q = f_eua.indicador_roe(ten_k=False, df_is=ceg_is_10q, df_bs=ceg_bs_10q, idx=idx_10q)

# ROIC 
roic_10q = f_eua.indicador_roic(ten_k=False, df_is=ceg_is_10q, df_bs=ceg_bs_10q, div_bruta=div_bruta_10q, idx=idx_10q)

# FCO
fco_10q = f_eua.indicador_fco(ten_k=False, df_cf=ceg_cf_10q, first_quarter='03') 

# FCI
fci_10q = f_eua.indicador_fci(ten_k=False, df_cf=ceg_cf_10q, first_quarter='03') 

# FCF
fcf_10q = f_eua.indicador_fcf(ten_k=False, df_cf=ceg_cf_10q, first_quarter='03') 

# CAPEX
lst_capex_10q = ['Payments to Acquire Property, Plant, and Equipment']

capex_10q = f_eua.indicador_capex(ten_k=False, df_cf=ceg_cf_10q, first_quarter='03', lst_itens=lst_capex_10q, idx=idx_10q)[0]
capex_10q_sliced = f_eua.indicador_capex(ten_k=False, df_cf=ceg_cf_10q, first_quarter='03', lst_itens=lst_capex_10q, idx=idx_10q)[1]
capex_10q_acum = f_eua.indicador_capex(ten_k=False, df_cf=ceg_cf_10q, first_quarter='03', lst_itens=lst_capex_10q, idx=idx_10q)[2]

# Free Cash Flow
free_cash_flow_10q = f_eua.indicador_free_cash_flow(fco=fco_10q, capex=capex_10q_sliced)

# Net CAPEX
net_capex_10q = f_eua.indicador_net_capex(capex=capex_10q_acum, depreciation=depreciacao_10q_acum)

# R&D
rd_expense_10q = f_eua.indicador_rd(ten_k=False, df_is=ceg_is_10q, idx=idx_10q)[0]
rd_expense_10q_sliced = f_eua.indicador_rd(ten_k=False, df_is=ceg_is_10q, idx=idx_10q)[1]
rd_expense_10q_acum = f_eua.indicador_rd(ten_k=False, df_is=ceg_is_10q, idx=idx_10q)[2]

# Net CAPEX ajustado
adj_net_capex_10q = f_eua.indicador_adj_net_capex(net_capex=net_capex_10q, rd_expense=rd_expense_10q_acum)

# Working capital
working_capital_10q = f_eua.indicador_working_capital(ten_k=False, df_bs=ceg_bs_10q)

# Change in non cash working capital
lst_itens_change_non_cash_wc = [
    'Deferred Income Taxes and Tax Credits',
    'Increase (Decrease) in Receivables',
    'Increase (Decrease) in Inventories',
    'Increase (Decrease) in Accounts Payable and Accrued Liabilities',
    'Increase Decrease In Option Premiums Collected Received',
    'Increase (Decrease) Cash Collateral from Counterparties',
    'Increase (Decrease) in Income Taxes Payable',
    'Increase (Decrease) in Obligation, Pension and Other Postretirement Benefits',
    'Increase (Decrease) in Other Operating Assets and Liabilities, Net'
]

change_non_cash_wc_10q = f_eua.indicador_change_non_cash_wc(ten_k=False, df_cf=ceg_cf_10q, lst_itens=lst_itens_change_non_cash_wc, first_quarter='03')[0]
change_non_cash_wc_10q_sliced = f_eua.indicador_change_non_cash_wc(ten_k=False, df_cf=ceg_cf_10q, lst_itens=lst_itens_change_non_cash_wc, first_quarter='03')[1]

# Reinvestment rate
reinvestment_rate_10q = f_eua.indicador_reinvestment_rate(ten_k=False, net_capex=adj_net_capex_10q, change_non_cash_wc=change_non_cash_wc_10q, df_is=ceg_is_10q, idx=idx_10q)

# New borrowing
lst_itens_new_borrowing = [
    'Proceeds from Short-term Debt, Maturing in More than Three Months', 
    'Proceeds from Issuance of Long-term Debt'
]

new_borrowing_10q = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=ceg_cf_10q, first_quarter='03', idx=idx_10q)[0]
new_borrowing_10q_sliced = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=ceg_cf_10q, first_quarter='03', idx=idx_10q)[1]
new_borrowing_10q_acum = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=ceg_cf_10q, first_quarter='03', idx=idx_10q)[2]

# Debt paid
lst_itens_debt_paid = [
    'Repayments of Short-term Debt, Maturing in More than Three Months', 
    'Repayments of Long-term Debt'
]

debt_paid_10q = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=ceg_cf_10q, first_quarter='03', idx=idx_10q)[0]
debt_paid_10q_sliced = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=ceg_cf_10q, first_quarter='03', idx=idx_10q)[1]
debt_paid_10q_acum = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=ceg_cf_10q, first_quarter='03', idx=idx_10q)[2]

# Taxes on operating income
taxes_operating_income_10q = f_eua.indicador_taxes_operating_income(ten_k=False, df_is=ceg_is_10q, idx=idx_10q)

# FCFE
fcfe_10q = f_eua.indicador_fcfe(
    ten_k=False, 
    depreciacao=depreciacao_10q_acum, 
    capex=capex_10q_acum, 
    change_non_cash_wc=change_non_cash_wc_10q_sliced, 
    new_borrowing=new_borrowing_10q_acum, 
    debt_paid=debt_paid_10q_acum, 
    df_is=ceg_is_10q,
    idx=idx_10q
)

# FCFF
fcff_10q = f_eua.indicador_fcff(
        ten_k=False, 
        taxes_operating_income=taxes_operating_income_10q,
        depreciacao=depreciacao_10q_acum, 
        capex=capex_10q_acum, 
        change_non_cash_wc=change_non_cash_wc_10q_sliced, 
        df_is=ceg_is_10q,
        idx=idx_10q
)

# Dividendo não acumulado
div_nao_acum = f_eua.indicador_div_nao_acum(df_cf=ceg_cf_10q, first_quarter='03')

# Dividendo por Ação
dpa_10q = f_eua.indicador_dpa(ten_k=False, df_cf=ceg_cf_10q, df_is=ceg_is_10q, div_nao_acum=div_nao_acum)

# Payout
payout_10q = f_eua.indicador_payout(ten_k=False, df_cf=ceg_cf_10q, df_is=ceg_is_10q, div_nao_acum=div_nao_acum)

# Buyback
buyback_10q = f_eua.indicador_buyback(ten_k=False, df_cf=ceg_cf_10q,  first_quarter='03')

# Lista com todos os indicadores
lst_indicadores_10q = [
    p_lpa_10q_damodaran,
    p_lpa_10q,
    lpa_p_10q,  
    ebitda_10q_sliced, 
    m_liq_10q,
    p_vpa_10q,
    vm_10q,
    div_bruta_10q,
    caixa_10q,
    div_liq_10q,
    dl_ebitda_10q,
    dl_pl_10q,
    ev_ebitda_10q,
    roe_10q,
    roic_10q,
    fco_10q,
    fci_10q,
    fcf_10q,
    free_cash_flow_10q,
    capex_10q_sliced,
    net_capex_10q,
    rd_expense_10q_sliced,
    adj_net_capex_10q,
    working_capital_10q,
    reinvestment_rate_10q,
    fcfe_10q,
    fcff_10q,
    dpa_10q,
    payout_10q,
    buyback_10q
]

# Juntando todos os indicadores em um df
ceg_indicadores_10q = pd.concat(lst_indicadores_10q, axis=1)

# Nomes das colunas
nomes_col = [
    'P/L Damodaran', 
    'P/L', 
    'L/P', 
    'EBITDA', 
    'Margem Líquida',
    'P/VPA',
    'Valor de Mercado',
    'Dívida Bruta',
    'Caixa e Equivalentes',
    'Dívida Líquida',
    'Dívida Líquida/EBITDA',
    'Dívida Líquida/PL',
    'EV/EBITDA',
    'ROE',
    'ROIC',
    'FCO',
    'FCI',
    'FCF',
    'Free Cash Flow',
    'Capex',
    'Net Capex',
    'R&D',
    'Adj Net Capex',
    'Working Capital',
    'Reinvestment Rate',
    'FCFE',
    'FCFF',
    'DPA',
    'Payout',
    'Buyback'
]

# Atribuindo os novos nomes das colunas do df
ceg_indicadores_10q.columns = nomes_col
ceg_indicadores_10q

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//energia_eletrica//constellation//ceg_indicators.xlsx') as writer:
    ceg_indicadores_10k.to_excel(writer, sheet_name='indicators_10k')
    ceg_indicadores_10q.to_excel(writer, sheet_name='indicators_10q')

## Vistra

* 1Q -> Lançamento em Março (03);
* 2Q -> Lançamento em Junho (06);
* 3Q -> Lançamento em Setembro (09);
* 10K -> Lançamento em Dezembro (12).

In [ ]:
ticker = 'vst'
f_sec.cik_matching_ticker(ticker)

### Scraping Vistra 10-K

In [ ]:
# Número dos documentos que contém a DRE, BP e FC 
acc_documents = f_sec.get_filtered_filings(ticker=ticker, ten_k=True, just_accession_numbers=True)

# Trocando os '-' para '' das strings ('0001045810-24-000264' -> '000104581024000264')
dict_acc_num = {k: v.replace('-', '') for k, v in acc_documents.items()}

# Transformando o dicionário em um df
df_lst_acc_num_vistra = pd.DataFrame.from_dict(dict_acc_num, orient='index', columns=['acc_num'])

# Transformando em datetime o index
df_lst_acc_num_vistra.index = pd.to_datetime(df_lst_acc_num_vistra.index, format='%Y-%m-%d')

# Selecionando os 'acc_num' dos 10-k a partir de 2020 p/ iteração
df_lst_acc_num_vistra = df_lst_acc_num_vistra.loc['2020-12-31':]

df_lst_acc_num_vistra

In [ ]:
# Rótulos
labels_dict = f_sec.get_label_dictionary(ticker, headers=f_sec.header)

In [ ]:
# Fazendo o scraping do balance sheet (10-K)
balance_sheet_lst = []

for acc_num in df_lst_acc_num_vistra['acc_num']:

    # Balance sheet
    balance_sheet = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='balance_sheet')
    balance_sheet = f_sec.rename_statement(balance_sheet, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    balance_sheet = balance_sheet.iloc[:, 0]
    balance_sheet_lst.append(balance_sheet)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_balance_sheet_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in balance_sheet_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_balance_sheet_10k = pd.concat([df_balance_sheet_10k, df], axis=1)

# Substituindo os NaN por 0
df_balance_sheet_10k = df_balance_sheet_10k.fillna(0)

# A linha 'Restricted Cash, Current' nas colunas '2023-12-31', '2022-12-31', '2021-12-31' e '2020-12-31' estão zeradas
# As informações estão na linha 'Restricted Cash and Investments, Current'
df_balance_sheet_10k.loc['Restricted Cash, Current', '2023-12-31'] = df_balance_sheet_10k.loc['Restricted Cash and Investments, Current', '2023-12-31'] 
df_balance_sheet_10k.loc['Restricted Cash, Current', '2022-12-31'] = df_balance_sheet_10k.loc['Restricted Cash and Investments, Current', '2022-12-31'] 
df_balance_sheet_10k.loc['Restricted Cash, Current', '2021-12-31'] = df_balance_sheet_10k.loc['Restricted Cash and Investments, Current', '2021-12-31'] 
df_balance_sheet_10k.loc['Restricted Cash, Current', '2020-12-31'] = df_balance_sheet_10k.loc['Restricted Cash and Investments, Current', '2020-12-31'] 

# A linha 'Restricted Cash, Noncurrent' nas colunas '2023-12-31', '2022-12-31', '2021-12-31' e '2020-12-31' estão zeradas
# As informações estão na linha 'Restricted Cash and Investments, Noncurrent'
df_balance_sheet_10k.loc['Restricted Cash, Noncurrent', '2023-12-31'] = df_balance_sheet_10k.loc['Restricted Cash and Investments, Noncurrent', '2023-12-31'] 
df_balance_sheet_10k.loc['Restricted Cash, Noncurrent', '2022-12-31'] = df_balance_sheet_10k.loc['Restricted Cash and Investments, Noncurrent', '2022-12-31'] 
df_balance_sheet_10k.loc['Restricted Cash, Noncurrent', '2021-12-31'] = df_balance_sheet_10k.loc['Restricted Cash and Investments, Noncurrent', '2021-12-31'] 
df_balance_sheet_10k.loc['Restricted Cash, Noncurrent', '2020-12-31'] = df_balance_sheet_10k.loc['Restricted Cash and Investments, Noncurrent', '2020-12-31'] 
                                                                              
# Removendo as linhas que movemos os seus itens para as linhas corretas
df_balance_sheet_10k = df_balance_sheet_10k.drop([
    'Restricted Cash and Investments, Current',
    'Restricted Cash and Investments, Noncurrent'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_balance_sheet_10k = df_balance_sheet_10k.drop([
    'us-gaap_AssetsCurrentAbstract',
    'Equity Method Investments',
    'Preferred Stock, Shares Outstanding',
    'Preferred Stock, Liquidation Preference Per Share'
])

# Renomeando a linha do índice que está sem formato 
df_balance_sheet_10k = df_balance_sheet_10k.rename(index={
    'vistra_MarginDepositsPostedUnderAffiliateFinancingAgreementAsset': 'Margin Deposits Posted Under Affiliate Financing Agreement Asset',
    'vistra_AccountsReceivableSecuritizationProgramAmountsBorrowed': 'Accounts Receivable Securitization Program Amounts Borrowed',
    'vistra_ForwardRepurchaseObligationToBePaidCurrent': 'Forward Repurchase Obligation To Be Paid Current',
    'vistra_MarginDepositLiabilities': 'Margin Deposit Liabilities',
    'vistra_ForwardRepurchaseObligationToBePaidNoncurrent': 'Forward Repurchase Obligation To Be Paid Noncurrent',
    'vistra_DerivativeAndHedgeLiabilitiesNoncurrent': 'Derivative And Hedge Liabilities Noncurrent',
    'vistra_TaxReceivableAgreementObligationNoncurrent': 'Tax Receivable Agreement Obligation Noncurrent',
})

df_balance_sheet_10k

In [ ]:
# Fazendo o scraping do income statement (10-K)
income_statement_lst = []

for acc_num in df_lst_acc_num_vistra['acc_num']:

    # Income statement
    income_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='income_statement')
    income_statement = f_sec.rename_statement(income_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    income_statement = income_statement.iloc[:, 0]
    income_statement_lst.append(income_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_income_statement_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in income_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_income_statement_10k = pd.concat([df_income_statement_10k, df], axis=1)

# Substituindo os NaN por 0
df_income_statement_10k = df_income_statement_10k.fillna(0)

# A linha 'InterestExpenseNonoperating' nas colunas '2023-12-31', '2022-12-31', '2021-12-31' e '2020-12-31' estão zeradas
# As informações estão na linha 'Interest Expense'
df_income_statement_10k.loc['InterestExpenseNonoperating', '2023-12-31'] = df_income_statement_10k.loc['Interest Expense', '2023-12-31'] 
df_income_statement_10k.loc['InterestExpenseNonoperating', '2022-12-31'] = df_income_statement_10k.loc['Interest Expense', '2022-12-31'] 
df_income_statement_10k.loc['InterestExpenseNonoperating', '2021-12-31'] = df_income_statement_10k.loc['Interest Expense', '2021-12-31'] 
df_income_statement_10k.loc['InterestExpenseNonoperating', '2020-12-31'] = df_income_statement_10k.loc['Interest Expense', '2020-12-31'] 
                                                                              
# Removendo as linhas que movemos os seus itens para as linhas corretas
df_income_statement_10k = df_income_statement_10k.drop('Interest Expense')

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_income_statement_10k = df_income_statement_10k.drop([
    'us-gaap_IncomeStatementAbstract',
    'Income (Loss) from Equity Method Investments'
])

# Renomeando a linha do índice que está sem formato 
df_income_statement_10k = df_income_statement_10k.rename(index={
    'vistra_CostOfFuelPurchasedPowerAndDelivery': 'Cost Of Fuel Purchased Power And Delivery',
    'vistra_ImpactsOfTaxReceivableAgreement': 'Impacts Of Tax Receivable Agreement',
})

# Arrumando a escala da linha 'Weighted Average Number of Shares Outstanding, Basic' e 'Weighted Average Number of Shares Outstanding, Diluted'
df_income_statement_10k.loc['Weighted Average Number of Shares Outstanding, Basic'] = round(df_income_statement_10k.loc['Weighted Average Number of Shares Outstanding, Basic'] / 1000, 0)
df_income_statement_10k.loc['Weighted Average Number of Shares Outstanding, Diluted'] = round(df_income_statement_10k.loc['Weighted Average Number of Shares Outstanding, Diluted'] / 1000, 0)

df_income_statement_10k

In [ ]:
# Fazendo o scraping do cash flow statement (10-K)
cash_flow_statement_lst = []

for acc_num in df_lst_acc_num_vistra['acc_num']:

    # Cash flow statement
    cash_flow_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='cash_flow_statement')
    cash_flow_statement = f_sec.rename_statement(cash_flow_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    cash_flow_statement = cash_flow_statement.iloc[:, 0]
    cash_flow_statement_lst.append(cash_flow_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_cash_flow_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in cash_flow_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_cash_flow_10k = pd.concat([df_cash_flow_10k, df], axis=1)

# Substituindo os NaN por 0
df_cash_flow_10k = df_cash_flow_10k.fillna(0)

# A linha 'vistra_UnrealizedGainLossOnInterestRateSwapDerivatives' nas colunas '2023-12-31', '2022-12-31', '2021-12-31' e	'2020-12-31' estão zeradas
# As informações estão na linha 'vistra_UnrealizedGainLossOnInterestRateSwapsDerivatives'
df_cash_flow_10k.loc['vistra_UnrealizedGainLossOnInterestRateSwapDerivatives', '2023-12-31'] = df_cash_flow_10k.loc['vistra_UnrealizedGainLossOnInterestRateSwapsDerivatives', '2023-12-31'] 
df_cash_flow_10k.loc['vistra_UnrealizedGainLossOnInterestRateSwapDerivatives', '2022-12-31'] = df_cash_flow_10k.loc['vistra_UnrealizedGainLossOnInterestRateSwapsDerivatives', '2022-12-31'] 
df_cash_flow_10k.loc['vistra_UnrealizedGainLossOnInterestRateSwapDerivatives', '2021-12-31'] = df_cash_flow_10k.loc['vistra_UnrealizedGainLossOnInterestRateSwapsDerivatives', '2021-12-31'] 
df_cash_flow_10k.loc['vistra_UnrealizedGainLossOnInterestRateSwapDerivatives', '2020-12-31'] = df_cash_flow_10k.loc['vistra_UnrealizedGainLossOnInterestRateSwapsDerivatives', '2020-12-31'] 

# A linha 'Proceeds from Issuance of Debt' nas colunas '2023-12-31', '2022-12-31', '2021-12-31' e	'2020-12-31' estão zeradas
# As informações estão na linha 'Proceeds from Issuance of Long-term Debt'
df_cash_flow_10k.loc['Proceeds from Issuance of Debt', '2023-12-31'] = df_cash_flow_10k.loc['Proceeds from Issuance of Long-term Debt', '2023-12-31'] 
df_cash_flow_10k.loc['Proceeds from Issuance of Debt', '2022-12-31'] = df_cash_flow_10k.loc['Proceeds from Issuance of Long-term Debt', '2022-12-31'] 
df_cash_flow_10k.loc['Proceeds from Issuance of Debt', '2021-12-31'] = df_cash_flow_10k.loc['Proceeds from Issuance of Long-term Debt', '2021-12-31'] 
df_cash_flow_10k.loc['Proceeds from Issuance of Debt', '2020-12-31'] = df_cash_flow_10k.loc['Proceeds from Issuance of Long-term Debt', '2020-12-31'] 

# A linha 'Repayments of Long-term Debt' nas colunas '2023-12-31', '2022-12-31', '2021-12-31' e	'2020-12-31' estão zeradas
# As informações estão na linha 'Repayment of Long-term Debt, Long-term Lease Obligation, and Capital Security'
df_cash_flow_10k.loc['Repayments of Long-term Debt', '2023-12-31'] = df_cash_flow_10k.loc['Repayment of Long-term Debt, Long-term Lease Obligation, and Capital Security', '2023-12-31'] 
df_cash_flow_10k.loc['Repayments of Long-term Debt', '2022-12-31'] = df_cash_flow_10k.loc['Repayment of Long-term Debt, Long-term Lease Obligation, and Capital Security', '2022-12-31'] 
df_cash_flow_10k.loc['Repayments of Long-term Debt', '2021-12-31'] = df_cash_flow_10k.loc['Repayment of Long-term Debt, Long-term Lease Obligation, and Capital Security', '2021-12-31'] 
df_cash_flow_10k.loc['Repayments of Long-term Debt', '2020-12-31'] = df_cash_flow_10k.loc['Repayment of Long-term Debt, Long-term Lease Obligation, and Capital Security', '2020-12-31'] 

# A linha 'vistra_ProceedsFromRepaymentsOfAccountsReceivableFinancing' nas colunas '2023-12-31', '2022-12-31', '2021-12-31' e '2020-12-31' estão zeradas
# As informações estão na linha 'Proceeds from (Repayments of) Accounts Receivable Securitization'
df_cash_flow_10k.loc['vistra_ProceedsFromRepaymentsOfAccountsReceivableFinancing', '2023-12-31'] = df_cash_flow_10k.loc['Proceeds from (Repayments of) Accounts Receivable Securitization', '2023-12-31'] 
df_cash_flow_10k.loc['vistra_ProceedsFromRepaymentsOfAccountsReceivableFinancing', '2022-12-31'] = df_cash_flow_10k.loc['Proceeds from (Repayments of) Accounts Receivable Securitization', '2022-12-31'] 
df_cash_flow_10k.loc['vistra_ProceedsFromRepaymentsOfAccountsReceivableFinancing', '2021-12-31'] = df_cash_flow_10k.loc['Proceeds from (Repayments of) Accounts Receivable Securitization', '2021-12-31'] 
df_cash_flow_10k.loc['vistra_ProceedsFromRepaymentsOfAccountsReceivableFinancing', '2020-12-31'] = df_cash_flow_10k.loc['Proceeds from (Repayments of) Accounts Receivable Securitization', '2020-12-31'] 

# A linha 'Increase (Decrease) in Accrued Taxes Payable' nas colunas '2023-12-31', '2022-12-31', '2021-12-31' e '2020-12-31' estão zeradas
# As informações estão na linha 'Increase (Decrease) in Property and Other Taxes Payable'
df_cash_flow_10k.loc['Increase (Decrease) in Accrued Taxes Payable', '2023-12-31'] = df_cash_flow_10k.loc['Increase (Decrease) in Property and Other Taxes Payable', '2023-12-31'] 
df_cash_flow_10k.loc['Increase (Decrease) in Accrued Taxes Payable', '2022-12-31'] = df_cash_flow_10k.loc['Increase (Decrease) in Property and Other Taxes Payable', '2022-12-31'] 
df_cash_flow_10k.loc['Increase (Decrease) in Accrued Taxes Payable', '2021-12-31'] = df_cash_flow_10k.loc['Increase (Decrease) in Property and Other Taxes Payable', '2021-12-31'] 
df_cash_flow_10k.loc['Increase (Decrease) in Accrued Taxes Payable', '2020-12-31'] = df_cash_flow_10k.loc['Increase (Decrease) in Property and Other Taxes Payable', '2020-12-31'] 

# A linha 'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect' nas colunas '2022-12-31', '2021-12-31' e '2020-12-31' estão zeradas
# As informações estão na linha 'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Excluding Exchange Rate Effect'
df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect', '2022-12-31'] = df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Excluding Exchange Rate Effect', '2022-12-31'] 
df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect', '2021-12-31'] = df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Excluding Exchange Rate Effect', '2021-12-31'] 
df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect', '2020-12-31'] = df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Excluding Exchange Rate Effect', '2020-12-31'] 

# A linha 'vistra_GainLossOnTaxReceivableAgreementRepurchaseAndTenderOffers' na coluna '2023-12-31' está zerada
# As informações estão na linha 'vistra_GainLossTaxReceivableAgreementSettlement'
df_cash_flow_10k.loc['vistra_GainLossOnTaxReceivableAgreementRepurchaseAndTenderOffers', '2023-12-31'] = df_cash_flow_10k.loc['vistra_GainLossTaxReceivableAgreementSettlement', '2023-12-31'] 

# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_cash_flow_10k = df_cash_flow_10k.drop([
    'vistra_UnrealizedGainLossOnInterestRateSwapsDerivatives',
    'Proceeds from Issuance of Long-term Debt',
    'Repayment of Long-term Debt, Long-term Lease Obligation, and Capital Security',
    'Proceeds from (Repayments of) Accounts Receivable Securitization',
    'Increase (Decrease) in Property and Other Taxes Payable',
    'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Excluding Exchange Rate Effect',
    'vistra_GainLossTaxReceivableAgreementSettlement'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_cash_flow_10k = df_cash_flow_10k.drop([
    'us-gaap_NetCashProvidedByUsedInOperatingActivitiesAbstract',
    'Payments to Acquire Business Two, Net of Cash Acquired',
    'Cash Acquired from Acquisition',
    'vistra_ProceedsFromDebtShortTermInNature',
    'vistra_RepaymentsOfDebtShortTermInNature',
    'Proceeds from Issuance of Preferred Stock and Preference Stock',
    'vistra_ProceedsFromForwardCapacityAgreement',
    'Equity Method Investment, Realized Gain (Loss) on Disposal',
    'Proceeds from Nuclear Fuel'
])

# Arrumando e adicionando alguns números no df
df_cash_flow_10k.loc['Proceeds from Lines of Credit', '2021-12-31'] = 1450
df_cash_flow_10k.loc['Proceeds from Lines of Credit', '2020-12-31'] = 1075
df_cash_flow_10k.loc['Repayments of Lines of Credit', '2021-12-31'] = -1450
df_cash_flow_10k.loc['Repayments of Lines of Credit', '2020-12-31'] = -1425

# Renomeando as linhas do índice que estão sem formato 
df_cash_flow_10k = df_cash_flow_10k.rename(index={
    'vistra_UnrealizedGainLossOnCommodityRelatedDerivatives': 'Unrealized Gain Loss On Commodity Related Derivatives',
    'vistra_UnrealizedGainLossOnInterestRateSwapDerivatives': 'Unrealized Gain Loss On Interest Rate Swap Derivatives',
    'vistra_UnrealizedGainLossFromNuclearDecommissioningTrusts': 'Unrealized Gain Loss From Nuclear Decommissioning Trusts',
    'vistra_TaxReceivableAgreementAccretionExpense': 'Tax Receivable Agreement Accretion Expense',
    'vistra_GainLossOnTaxReceivableAgreementRepurchaseAndTenderOffers': 'Gain Loss On Tax Receivable Agreement Repurchase And Tender Offers',
    'vistra_IncreaseDecreaseInCommodityAndDerivativeAssetsAndLiabilities': 'Increase (Decrease) In Commodity And Derivative Assets And Liabilities',
    'vistra_IncreaseDecreaseInMarginDepositsNet': 'Increase (Decrease) In Margin Deposits Net',
    'vistra_PaymentsForSettlementOfAssetRetirementObligation': 'Payments For Settlement Of Asset Retirement Obligation',
    'vistra_ProceedsFromSaleOfInvestmentTaxCreditSoldUnderAgreements': 'Proceeds From Sale Of Investment Tax Credit Sold Under Agreements',
    'vistra_ProceedsFromRepaymentsOfAccountsReceivableFinancing': 'Proceeds From Repayments Of Accounts Receivable Financing',
    'vistra_PaymentsOfOrdinaryDividendsNoncontrollingAndRedeemableNoncontrollingInterest': 'Payments Of Ordinary Dividends Noncontrolling And Redeemable Noncontrolling Interest',
    'vistra_ReturnOfCapitalTaxReceivableAgreementAndTenderOffer': 'Return Of Capital Tax Receivable Agreement And Tender Offer',
    'vistra_ProceedsFromForwardCapacityAgreement': 'Proceeds From Forward Capacity Agreement',
})

df_cash_flow_10k

In [ ]:
# O 'df_income_statement_10k' não tem a linha 'Research and Development Expense'

# Linha 'Research and Development Expense' a ser adicionada 
new_row_rd_is_10k = [
    #2024
    0, 
    #2023
    0, 
    #2022
    0,
    #2021
    0,
    #2020
    0,
]

# Adicionando a nova linha
df_income_statement_10k.loc['Research and Development'] = new_row_rd_is_10k

In [ ]:
# Arrumando alguns index dos dfs para ficar padronizado com os index utilizados nas funções p/ calcular os indicadores fundamentalistas
df_balance_sheet_10k = df_balance_sheet_10k.rename(index={
    "Stockholders' Equity Attributable to Parent": 'Total Shareholders Equity',
})

df_income_statement_10k = df_income_statement_10k.rename(index={
    'Operating Income (Loss)': 'Operating Income',
    'InterestExpenseNonoperating': 'Interest Expense',
    'Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest': 'Income Before Provision for Income Taxes',
    'Income Tax Expense (Benefit)': 'Provision for Income Taxes',
    'Net Income (Loss) Attributable to Parent': 'Net Income',
    'Weighted Average Number of Shares Outstanding, Basic': 'Number of Shares - Basic',
    'Weighted Average Number of Shares Outstanding, Diluted': 'Number of Shares - Diluted'
})

df_cash_flow_10k = df_cash_flow_10k.rename(index={
    'Depreciation, Amortization and Accretion, Net': 'Depreciation, Depletion and Amortization',
    'Payments of Ordinary Dividends, Common Stock': 'Payments of Dividends',
})

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//energia_eletrica//vistra//vst_10k.xlsx') as writer:
    df_balance_sheet_10k.to_excel(writer, sheet_name='balance_sheet')
    df_income_statement_10k.to_excel(writer, sheet_name='income_statement')
    df_cash_flow_10k.to_excel(writer, sheet_name='cash_flow')

### Scraping Vistra 10-Q

In [ ]:
# Número dos documentos que contém a DRE, BP e FC 
acc_documents_10q = f_sec.get_filtered_filings(ticker=ticker, ten_k=False, just_accession_numbers=True)

# Trocando os '-' para '' das strings ('0001045810-24-000264' -> '000104581024000264')
dict_acc_num_10q = {k: v.replace('-', '') for k, v in acc_documents_10q.items()}

# Transformando o dicionário em um df
df_lst_acc_num_10q_vst = pd.DataFrame.from_dict(dict_acc_num_10q, orient='index', columns=['acc_num'])

# Transformando em datetime o index
df_lst_acc_num_10q_vst.index = pd.to_datetime(df_lst_acc_num_10q_vst.index, format='%Y-%m-%d')

df_lst_acc_num_10q_vst

In [ ]:
# Selecionando os 'acc_num' dos 10-k a partir de 2021 p/ iteração
df_lst_acc_num_10q_vst_sliced = df_lst_acc_num_10q_vst.loc['2021-03-31':]
df_lst_acc_num_10q_vst_sliced

In [ ]:
# Rótulos
labels_dict = f_sec.get_label_dictionary(ticker, headers=f_sec.header)

In [ ]:
# Fazendo o scraping do balance sheet (10-Q)
balance_sheet_lst = []

for acc_num in df_lst_acc_num_10q_vst_sliced['acc_num']:

    # Balance sheet
    balance_sheet = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='balance_sheet')
    balance_sheet = f_sec.rename_statement(balance_sheet, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    balance_sheet = balance_sheet.iloc[:, 0]
    balance_sheet_lst.append(balance_sheet)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_balance_sheet_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in balance_sheet_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_balance_sheet_10q = pd.concat([df_balance_sheet_10q, df], axis=1)

# Substituindo os NaN por 0
df_balance_sheet_10q = df_balance_sheet_10q.fillna(0)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_balance_sheet_10q = df_balance_sheet_10q.drop([
    'us-gaap_AssetsCurrentAbstract',
    'Common Stock, Shares Authorized',
    'Common Stock, Shares, Outstanding',
    'Treasury Stock, Common, Shares',
    'Common Stock, Par or Stated Value Per Share',
    'Redeemable Noncontrolling Interest, Equity, Carrying Amount',
    'vistra_MarginDepositFinancingWithAffiliate'
])

# Renomeando algumas linhas do índice
df_balance_sheet_10q = df_balance_sheet_10q.rename(index={
    'vistra_MarginDepositsPostedUnderAffiliateFinancingAgreementAsset': 'Margin Deposits Posted Under Affiliate Financing Agreement Asset',
    'vistra_AccountsReceivableSecuritizationProgramAmountsBorrowed': 'Accounts Receivable Securitization Program Amounts Borrowed',
    'vistra_ForwardRepurchaseObligationToBePaidCurrent': 'Forward Repurchase Obligation To Be Paid Current',
    'vistra_ForwardRepurchaseObligationToBePaidNoncurrent': 'Forward Repurchase Obligation To Be Paid Noncurrent',
    'vistra_DerivativeAndHedgeLiabilitiesNoncurrent': 'Derivative And Hedge Liabilities Noncurrent',
    'vistra_TaxReceivableAgreementObligationNoncurrent': 'Tax Receivable Agreement Obligation Noncurrent',
})

df_balance_sheet_10q

In [ ]:
# Fazendo o scraping do income statement (10-Q)
income_statement_lst = []

for acc_num in df_lst_acc_num_10q_vst_sliced['acc_num']:

    # Income statement
    income_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='income_statement')
    income_statement = f_sec.rename_statement(income_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    income_statement = income_statement.iloc[:, 0]
    income_statement_lst.append(income_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_income_statement_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in income_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_income_statement_10q = pd.concat([df_income_statement_10q, df], axis=1)

# Substituindo os NaN por 0
df_income_statement_10q = df_income_statement_10q.fillna(0)

# A linha 'InterestExpenseNonoperating' nas colunas '2024-03-31', '2023-09-30', '2023-06-30', '2023-03-31', '2022-09-30', '2022-06-30', '2022-03-31', '2021-09-30', '2021-06-30' e '2021-03-31' estão zeradas
# As informações estão na linha 'Interest Expense'
df_income_statement_10q.loc['InterestExpenseNonoperating', '2024-03-31'] = df_income_statement_10q.loc['Interest Expense', '2024-03-31'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2023-09-30'] = df_income_statement_10q.loc['Interest Expense', '2023-09-30'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2023-06-30'] = df_income_statement_10q.loc['Interest Expense', '2023-06-30'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2023-03-31'] = df_income_statement_10q.loc['Interest Expense', '2023-03-31'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2022-09-30'] = df_income_statement_10q.loc['Interest Expense', '2022-09-30'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2022-06-30'] = df_income_statement_10q.loc['Interest Expense', '2022-06-30'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2022-03-31'] = df_income_statement_10q.loc['Interest Expense', '2022-03-31'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2021-09-30'] = df_income_statement_10q.loc['Interest Expense', '2021-09-30'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2021-06-30'] = df_income_statement_10q.loc['Interest Expense', '2021-06-30'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2021-03-31'] = df_income_statement_10q.loc['Interest Expense', '2021-03-31'] 
                                                                              
# Removendo as linhas que movemos os seus itens para as linhas corretas
df_income_statement_10q = df_income_statement_10q.drop('Interest Expense')

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_income_statement_10q = df_income_statement_10q.drop([
    'us-gaap_IncomeStatementAbstract',
    'Income (Loss) from Equity Method Investments'
])

# Renomeando algumas linhas do índice
df_income_statement_10q = df_income_statement_10q.rename(index={
    'vistra_CostOfFuelPurchasedPowerAndDelivery': 'Cost Of Fuel Purchased Power And Delivery',
    'vistra_ImpactsOfTaxReceivableAgreement': 'Impacts Of Tax Receivable Agreement',
})

# Arrumando a escala das linhas 'Weighted Average Number of Shares Outstanding, Basic' e 'Weighted Average Number of Shares Outstanding, Diluted'
df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Basic'] = round(df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Basic'] / 1000, 0)
df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Diluted'] = round(df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Diluted'] / 1000, 0)

df_income_statement_10q

In [ ]:
# Fazendo o scraping do cash flow statement (10-Q)
cash_flow_statement_lst = []

for acc_num in df_lst_acc_num_10q_vst_sliced['acc_num']:

    # Cash flow statement
    cash_flow_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='cash_flow_statement')
    cash_flow_statement = f_sec.rename_statement(cash_flow_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    cash_flow_statement = cash_flow_statement.iloc[:, 0]
    cash_flow_statement_lst.append(cash_flow_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_cash_flow_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in cash_flow_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_cash_flow_10q = pd.concat([df_cash_flow_10q, df], axis=1)

# Substituindo os NaN por 0
df_cash_flow_10q = df_cash_flow_10q.fillna(0)

# A linha 'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect' nas colunas '2023-09-30', '2023-06-30', '2023-03-31', '2022-09-30', '2022-06-30', '2022-03-31', '2021-09-30', '2021-06-30' e '2021-03-31' estão zeradas
# As informações estão na linha 'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Excluding Exchange Rate Effect'
df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect', '2023-09-30'] = df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Excluding Exchange Rate Effect', '2023-09-30'] 
df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect', '2023-06-30'] = df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Excluding Exchange Rate Effect', '2023-06-30'] 
df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect', '2023-03-31'] = df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Excluding Exchange Rate Effect', '2023-03-31'] 
df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect', '2022-09-30'] = df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Excluding Exchange Rate Effect', '2022-09-30'] 
df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect', '2022-06-30'] = df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Excluding Exchange Rate Effect', '2022-06-30'] 
df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect', '2022-03-31'] = df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Excluding Exchange Rate Effect', '2022-03-31'] 
df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect', '2021-09-30'] = df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Excluding Exchange Rate Effect', '2021-09-30'] 
df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect', '2021-06-30'] = df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Excluding Exchange Rate Effect', '2021-06-30'] 
df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect', '2021-03-31'] = df_cash_flow_10q.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Excluding Exchange Rate Effect', '2021-03-31'] 

# A linha 'Repayments of Long-term Debt' na coluna '2021-03-31' está zerada
# As informações estão na linha 'Repayment of Long-term Debt, Long-term Lease Obligation, and Capital Security'
df_cash_flow_10q.loc['Repayments of Long-term Debt', '2021-03-31'] = df_cash_flow_10q.loc['Repayment of Long-term Debt, Long-term Lease Obligation, and Capital Security', '2021-03-31'] 

# A linha 'Repayments of Short-term Debt' na coluna '2021-09-30' está zerada
# As informações estão na linha 'Repayments of Other Short-term Debt'
df_cash_flow_10q.loc['Repayments of Short-term Debt', '2021-09-30'] = df_cash_flow_10q.loc['Repayments of Other Short-term Debt', '2021-09-30'] 

# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_cash_flow_10q = df_cash_flow_10q.drop([
    'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Excluding Exchange Rate Effect',
    'Repayment of Long-term Debt, Long-term Lease Obligation, and Capital Security',
    'Repayments of Other Short-term Debt'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_cash_flow_10q = df_cash_flow_10q.drop([
    'us-gaap_NetCashProvidedByUsedInOperatingActivitiesAbstract',
    'Proceeds from Issuance of Secured Debt',
    'vistra_RepaymentOfTermLoan',
    'Proceeds from Other Short-term Debt',
    'Equity Method Investment, Realized Gain (Loss) on Disposal',
    'vistra_PaymentsOfOrdinaryDividendsNoncontrollingAndRedeemableNoncontrollingInterest',
    'Gain (Loss) on Sale of Properties',
    'vistra_ProceedsFromForwardCapacityAgreement',
    'Payments of Ordinary Dividends, Noncontrolling Interest'
])

# Renomeando algumas linhas do índice
df_cash_flow_10q = df_cash_flow_10q.rename(index={
    'vistra_GainOnInvoluntaryConversion': 'Gain On Involuntary Conversion',
    'vistra_RedeemableNoncontrollingInterestPrincipalPayment': 'Redeemable Noncontrolling Interest Principal Payment',
    'vistra_UnrealizedGainLossOnCommodityRelatedDerivatives': 'Unrealized Gain Loss On Commodity Related Derivatives',
    'vistra_UnrealizedGainLossOnInterestRateSwapDerivatives': 'Unrealized Gain Loss On Interest Rate Swap Derivatives',
    'vistra_UnrealizedGainLossFromNuclearDecommissioningTrusts': 'Unrealized Gain Loss From Nuclear Decommissioning Trusts',
    'vistra_IncreaseDecreaseInMarginDepositsNet': 'Increase (Decrease) In Margin Deposits, Net',
    'vistra_ProceedsFromRepaymentsOfAccountsReceivableFinancing': 'Proceeds From Repayments Of Accounts Receivable Financing',
    'vistra_ReturnOfCapitalTaxReceivableAgreementAndTenderOffer': 'Return Of Capital Tax Receivable Agreement And Tender Offer',
    'vistra_TaxReceivableAgreementAccretionExpense': 'Tax Receivable Agreement Accretion Expense',
    'vistra_GainLossOnTaxReceivableAgreementRepurchaseAndTenderOffers': 'Gain Loss On Tax Receivable Agreement Repurchase And Tender Offers',
})

df_cash_flow_10q

In [ ]:
# O 'df_income_statement_10q' não tem a linha 'Research and Development Expense'

# Linha 'Research and Development Expense' a ser adicionada 
new_row_rd_is_10q = [
    # 2025-09-30
    0,
    # 2025-06-30
    0,
    # 2025-03-31
    0,	
    # 2024-09-30	
    0,	
    # 2024-06-30	
    0,	
    # 2024-03-31	
    0,	
    # 2023-09-30	
    0,	
    # 2023-06-30	
    0,	
    # 2023-03-31	
    0,	
    # 2022-09-30	
    0,	
    # 2022-06-30	
    0,	
    # 2022-03-31
    0,	
	# 2021-09-30	
	0,
    # 2021-06-30	
	0,
    # 2021-03-31
	0,
]

# Adicionando a nova linha
df_income_statement_10q.loc['Research and Development'] = new_row_rd_is_10q

In [ ]:
# Arrumando alguns index dos dfs para ficar padronizado com os index utilizados nas funções p/ calcular os indicadores fundamentalistas
df_balance_sheet_10q = df_balance_sheet_10q.rename(index={
    "Stockholders' Equity Attributable to Parent": 'Total Shareholders Equity',
})

df_income_statement_10q = df_income_statement_10q.rename(index={
    'Operating Income (Loss)': 'Operating Income',
    'InterestExpenseNonoperating': 'Interest Expense',
    'Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest': 'Income Before Provision for Income Taxes',
    'Income Tax Expense (Benefit)': 'Provision for Income Taxes',
    'Net Income (Loss) Attributable to Parent': 'Net Income',
    'Weighted Average Number of Shares Outstanding, Basic': 'Number of Shares - Basic',
    'Weighted Average Number of Shares Outstanding, Diluted': 'Number of Shares - Diluted'
})

df_cash_flow_10q = df_cash_flow_10q.rename(index={
    'Depreciation, Amortization and Accretion, Net': 'Depreciation, Depletion and Amortization',
    'Payments of Ordinary Dividends, Common Stock': 'Payments of Dividends',
})

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//energia_eletrica//vistra//vst_10q.xlsx') as writer:
    df_balance_sheet_10q.to_excel(writer, sheet_name='balance_sheet')
    df_income_statement_10q.to_excel(writer, sheet_name='income_statement')
    df_cash_flow_10q.to_excel(writer, sheet_name='cash_flow')

### Indicadores Fundamentalistas

#### Indicadores 10-K

In [ ]:
# Lendo os arquivos 10k
vst_bs = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//energia_eletrica//vistra//vst_10k.xlsx', 
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

vst_is = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//energia_eletrica//vistra//vst_10k.xlsx', 
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

vst_cf = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//energia_eletrica//vistra//vst_10k.xlsx', 
    sheet_name='cash_flow', 
    index_col='Unnamed: 0'
)

# Transformando as colunas em datetime
vst_bs.columns = pd.to_datetime(vst_bs.columns)
vst_is.columns = pd.to_datetime(vst_is.columns)
vst_cf.columns = pd.to_datetime(vst_cf.columns)

# Transformando as colunas em apenas o ano
vst_bs.columns = vst_bs.columns.year
vst_is.columns = vst_is.columns.year
vst_cf.columns = vst_cf.columns.year

In [ ]:
# Preços da VST
vst_close = yf.download('VST', auto_adjust=True, multi_level_index=False, start='2020-01-01')['Close']

# Selecionando o último preço do mês de Dezembro de cada ano 
lst_years = [
    '2020', '2021', '2022', '2023', 
    '2024',
]

last_dec_close = []

for year in lst_years:
    last_close = vst_close[f'{year}-12'].iloc[-1]
    last_dec_close.append(last_close)

# Transformando em uma série pandas
vst_close_dec = pd.Series(last_dec_close, index=lst_years)

# Revertendo o index da série p/ ficar igual com os dos indicadores
vst_close_dec = vst_close_dec.reindex(index=vst_close_dec.index[::-1])

# Arredondamento de duas casas decimais
vst_close_dec = round(vst_close_dec, 2)

# Não realizou split recentemente (não possui dados de split no site da Investing)

# Transformando o index em int
vst_close_dec.index = vst_close_dec.index.astype(int)

vst_close_dec

In [ ]:
# Criando uma lista só para fazer a indexação em que o ano mais recente está no começo
idx = vst_bs.columns

# Valor de Mercado - utilizando a função 'indicador_vm_2', porque o nº de ações está em escala diferente
vm = f_eua.indicador_vm_2(ten_k=True, price=vst_close_dec, df_is=vst_is) 

# LPA - utilizando a função 'indicador_lpa_2', porque o nº de ações está em escala diferente
lpa = f_eua.indicador_lpa_2(ten_k=True, df_is=vst_is, idx=None)

# P/L Damodaran
p_lpa_damodaran = f_eua.indicador_p_lpa_2(ten_k=True, vm=vm, df_is=vst_is)

# P/L
p_lpa = f_eua.indicador_p_lpa(ten_k=True, price=vst_close_dec, lpa=lpa, idx=idx)

# L/P
lpa_p = f_eua.indicador_lpa_p(ten_k=True, price=vst_close_dec, lpa=lpa, idx=idx) 

# Depreciação
depreciacao = f_eua.indicador_depreciacao(ten_k=True, df_cf=vst_cf, first_quarter=None, idx=idx)

# EBITDA
ebitda = f_eua.indicador_ebitda(ten_k=True, df_is=vst_is, idx=None, depreciacao=depreciacao)

# Margem Líquida
m_liq = f_eua.indicador_ml(ten_k=True, df_is=vst_is)

# VPA - utilizando a função 'indicador_vpa_2', porque o nº de ações está em escala diferente
vpa = f_eua.indicador_vpa_2(ten_k=True, df_bs=vst_bs, df_is=vst_is)

# P/VPA
p_vpa = f_eua.indicador_p_vpa(ten_k=True, price=vst_close_dec, vpa=vpa, idx=idx)

# Caixa e Equivalentes
lst_caixa = [
    'Cash and Cash Equivalents, at Carrying Value', 
    'Restricted Cash, Current'
]

caixa = f_eua.indicador_caixa(ten_k=True, df_bs=vst_bs, lst_itens_caixa=lst_caixa)

# Dívida Bruta
lst_divida = [
    'Long-term Debt and Lease Obligation, Current', 
    'Long-term Debt and Lease Obligation'
]

div_bruta = f_eua.indicador_divida_bruta(ten_k=True, df_bs=vst_bs, lst_itens_divida=lst_divida)

# Dívida Líquida
div_liq = f_eua.indicador_divida_liquida(ten_k=True, caixa=caixa, div_bruta=div_bruta, idx=idx)

# Dívida Líquida / EBITDA
dl_ebitda = f_eua.indicador_dl_ebitda(ten_k=True, ebitda=ebitda, div_liq=div_liq, idx=None)

# Dívida Líquida / PL
dl_pl = f_eua.indicador_dl_pl(ten_k=True, div_liq=div_liq, df_bs=vst_bs) 

# EV/EBITDA
ev_ebitda = f_eua.indicador_ev_ebitda(vm=vm, div_liq=div_liq, ebitda=ebitda)

# ROE
roe = f_eua.indicador_roe(ten_k=True, df_is=vst_is, df_bs=vst_bs, idx=None)

# ROIC
roic = f_eua.indicador_roic(ten_k=True, df_is=vst_is, df_bs=vst_bs, div_bruta=div_bruta, idx=idx)

# FCO
fco = f_eua.indicador_fco(ten_k=True, df_cf=vst_cf, first_quarter=None) 

# FCI
fci = f_eua.indicador_fci(ten_k=True, df_cf=vst_cf, first_quarter=None) 

# FCF
fcf = f_eua.indicador_fcf(ten_k=True, df_cf=vst_cf, first_quarter=None) 

# CAPEX
lst_capex = ['Payments to Acquire Property, Plant, and Equipment']

capex= f_eua.indicador_capex(ten_k=True, df_cf=vst_cf, first_quarter=None, lst_itens=lst_capex, idx=idx)

# Free Cash Flow
free_cash_flow = f_eua.indicador_free_cash_flow(fco=fco, capex=capex)

# Net CAPEX
net_capex = f_eua.indicador_net_capex(capex=capex, depreciation=depreciacao)

# R&D
rd_expense = f_eua.indicador_rd(ten_k=True, df_is=vst_is, idx=idx)

# Net CAPEX ajustado
adj_net_capex = f_eua.indicador_adj_net_capex(net_capex=net_capex, rd_expense=rd_expense)

# Working capital
working_capital = f_eua.indicador_working_capital(ten_k=True, df_bs=vst_bs)

# Change in non cash working capital
lst_itens_change_non_cash_wc = [
    'Deferred Income Taxes and Tax Credits',
    'Increase (Decrease) in Accounts Receivable',
    'Increase (Decrease) in Inventories',
    'Increase (Decrease) in Accounts Payable',
    'Increase (Decrease) in Other Receivables',	
    'Increase (Decrease) in Interest Payable, Net',	
    'Increase (Decrease) in Accrued Taxes Payable',	
    'Increase (Decrease) in Employee Related Liabilities',
    'Payments For Settlement Of Asset Retirement Obligation',
    'Increase (Decrease) in Deferred Charges',
    'Increase (Decrease) in Other Operating Assets',	
    'Increase (Decrease) in Other Operating Liabilities',
]

change_non_cash_wc = f_eua.indicador_change_non_cash_wc(ten_k=True, df_cf=vst_cf, lst_itens=lst_itens_change_non_cash_wc, first_quarter=None)

# Reinvestment rate
reinvestment_rate = f_eua.indicador_reinvestment_rate(ten_k=True, net_capex=net_capex, change_non_cash_wc=change_non_cash_wc, df_is=vst_is, idx=idx)

# New borrowing
lst_itens_new_borrowing = [
    'Proceeds from Issuance of Debt', 
    'Proceeds from Lines of Credit', 
    'Proceeds from Short-term Debt'
]

new_borrowing = f_eua.indicador_new_borrowing(ten_k=True, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=vst_cf, first_quarter=None, idx=idx)

# Debt paid
lst_itens_debt_paid = [
    'Repayments of Long-term Debt', 
    'Repayments of Lines of Credit', 
    'Repayments of Short-term Debt'
]

debt_paid = f_eua.indicador_debt_paid(ten_k=True, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=vst_cf, first_quarter=None, idx=idx)

# Taxes on operating income
taxes_operating_income = f_eua.indicador_taxes_operating_income(ten_k=True, df_is=vst_is, idx=idx)

# FCFE
fcfe_10q = f_eua.indicador_fcfe(
    ten_k=True, 
    depreciacao=depreciacao, 
    capex=capex, 
    change_non_cash_wc=change_non_cash_wc, 
    new_borrowing=new_borrowing,
    debt_paid=debt_paid,
    df_is=vst_is,
    idx=idx
)

# FCFF
fcff_10q = f_eua.indicador_fcff(
        ten_k=True, 
        taxes_operating_income=taxes_operating_income,
        depreciacao=depreciacao, 
        capex=capex, 
        change_non_cash_wc=change_non_cash_wc, 
        df_is=vst_is,
        idx=idx
)

# Dividendo por Ação - utilizando a função 'indicador_dpa_2', porque o nº de ações está em escala diferente
dpa = f_eua.indicador_dpa_2(ten_k=True, df_cf=vst_cf, df_is=vst_is, div_nao_acum=None)

# Payout
payout = f_eua.indicador_payout(ten_k=True, df_cf=vst_cf, df_is=vst_is, div_nao_acum=None)

# Buyback
buyback = f_eua.indicador_buyback(ten_k=True, df_cf=vst_cf, first_quarter=None)

# Colocando todos os indicadores em uma lista
lst_indicadores = [
    p_lpa_damodaran,
    p_lpa,
    lpa_p,  
    ebitda, 
    m_liq,
    p_vpa,
    vm,
    div_bruta,
    caixa,
    div_liq,
    dl_ebitda,
    dl_pl,
    ev_ebitda,
    roe,
    roic,
    fco,
    fci,
    fcf,
    free_cash_flow,
    capex,
    net_capex,
    rd_expense,
    adj_net_capex,
    working_capital,
    reinvestment_rate,
    fcfe_10q,
    fcff_10q,
    dpa,
    payout,
    buyback
]

# Juntando todos os indicadores em um df
vst_indicadores_10k = pd.concat(lst_indicadores, axis=1)

# Nomes das colunas
nomes_col = [
    'P/L Damodaran', 
    'P/L', 
    'L/P', 
    'EBITDA', 
    'Margem Líquida',
    'P/VPA',
    'Valor de Mercado',
    'Dívida Bruta',
    'Caixa e Equivalentes',
    'Dívida Líquida',
    'Dívida Líquida/EBITDA',
    'Dívida Líquida/PL',
    'EV/EBITDA',
    'ROE',
    'ROIC',
    'FCO',
    'FCI',
    'FCF',
    'Free Cash Flow',
    'Capex',
    'Net Capex',
    'R&D',
    'Adj Net Capex',
    'Working Capital',
    'Reinvestment Rate',
    'FCFE',
    'FCFF',
    'DPA',
    'Payout',
    'Buyback'
]

# Atribuindo os novos nomes das colunas do df
vst_indicadores_10k.columns = nomes_col
vst_indicadores_10k

#### Indicadores 10-Q

In [ ]:
# Lendo os arquivos 10q
vst_bs_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//energia_eletrica//vistra//vst_10q.xlsx', 
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

vst_is_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//energia_eletrica//vistra//vst_10q.xlsx', 
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

vst_cf_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//energia_eletrica//vistra//vst_10q.xlsx', 
    sheet_name='cash_flow', 
    index_col='Unnamed: 0'
)

# Aplicando o ajuste nas colunas
vst_bs_10q.columns = [f_sec.ajustar_mes_03_06_09(col) for col in vst_bs_10q.columns]
vst_is_10q.columns = [f_sec.ajustar_mes_03_06_09(col) for col in vst_is_10q.columns]
vst_cf_10q.columns = [f_sec.ajustar_mes_03_06_09(col) for col in vst_cf_10q.columns]

In [ ]:
# Preços da VST
vst_close = yf.download('VST', auto_adjust=True, multi_level_index=False, start='2022-01-01')['Close']

# Selecionando o último preço dos meses de Março, Junho e Setembro
lst_years = [
    '2022','2023', '2024', '2025'
]
lst_months = ['03', '06', '09']
last_close_months = []

for year in lst_years:
    for month in lst_months:
        last_close = vst_close[f'{year}-{month}'].dropna().iloc[-1]  # Dropna para lidar com meses sem dados
        last_close_months.append(last_close)

# Criando uma lista em que junte os meses e os anos para que seja o index da serie pandas
lst_idx = []

for year in lst_years:
    for month in lst_months:
        lst_idx.append(f"{month}-{year}")

# Transformando em uma série pandas
vst_close_months = pd.Series(last_close_months, index=lst_idx)

# Revertendo o index da série p/ ficar igual com os dos indicadores
vst_close_months = vst_close_months.reindex(index=vst_close_months.index[::-1])

# No ano mais atual, como ainda ele não acabou, não consigo fazer o loop com todos meses (Março, Junho e Setembro)
actual_year_int = datetime.now().year  
actual_year_str = str(actual_year_int) 

lst_actual_months = [
    # '03',
    # '06', 
    # '09'
] 

last_close_actual_months = []

for month in lst_actual_months:
    last_close = vst_close[f'{actual_year_str}-{month}'].dropna().iloc[-1]  # Dropna para lidar com meses sem dados
    last_close_actual_months.append(last_close)

# Criando uma lista em que junte os meses e os anos para que seja o index da serie pandas
lst_idx_actual = []

for month in lst_actual_months:
    lst_idx_actual.append(f"{month}-{actual_year_str}")

# Transformando em uma série pandas
vst_close_months_atual = pd.Series(last_close_actual_months, index=lst_idx_actual)

# Revertendo o index da série p/ ficar igual com os dos indicadores
vst_close_months_atual = vst_close_months_atual.reindex(index=vst_close_months_atual.index[::-1])

# Juntando as duas series pandas
vst_close_months = pd.concat([vst_close_months_atual, vst_close_months])

# Arredondamento de duas casas decimais
vst_close_months = round(vst_close_months, 2)

# Não realizou split recentemente (não possui dados de split no site da Investing)

# Substituindo o index para ficar igual ao dos dados fundamentalistas
vst_close_months.index = vst_bs_10q.columns[:-3]

vst_close_months

In [ ]:
# Criando uma lista só para fazer a indexação em que o ano mais recente está no começo
idx_10q = vst_bs_10q.columns

# Valor de Mercado - utilizando a função 'indicador_vm_2', porque o nº de ações está em escala diferente
vm_10q = f_eua.indicador_vm_2(ten_k=False, price=vst_close_months, df_is=vst_is_10q)

# LPA - utilizando a função 'indicador_lpa_2', porque o nº de ações está em escala diferente
lpa_acum = f_eua.indicador_lpa_2(ten_k=False, df_is=vst_is_10q, idx=idx_10q)[1]
lpa_10q = f_eua.indicador_lpa_2(ten_k=False, df_is=vst_is_10q, idx=idx_10q)[0]

# P/L Damodaran
p_lpa_10q_damodaran = f_eua.indicador_p_lpa_2(ten_k=False, vm=vm_10q, df_is=vst_is_10q)

# P/L
p_lpa_10q = f_eua.indicador_p_lpa(ten_k=False, price=vst_close_months, lpa=lpa_acum, idx=idx_10q)

# L/P
lpa_p_10q = f_eua.indicador_lpa_p(ten_k=False, price=vst_close_months, lpa=lpa_acum, idx=idx_10q) 

# Depreciação
depreciacao_10q = f_eua.indicador_depreciacao(ten_k=False, df_cf=vst_cf_10q, first_quarter='03', idx=idx_10q)[0]
depreciacao_10q_sliced = f_eua.indicador_depreciacao(ten_k=False, df_cf=vst_cf_10q, first_quarter='03', idx=idx_10q)[1]
depreciacao_10q_acum = f_eua.indicador_depreciacao(ten_k=False, df_cf=vst_cf_10q, first_quarter='03', idx=idx_10q)[2]

# EBITDA
ebitda_10q = f_eua.indicador_ebitda(ten_k=False, df_is=vst_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[0]
ebitda_10q_sliced = f_eua.indicador_ebitda(ten_k=False, df_is=vst_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[1]
ebitda_10q_acum = f_eua.indicador_ebitda(ten_k=False, df_is=vst_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[2]

# Margem Líquida
m_liq_10q = f_eua.indicador_ml(ten_k=False, df_is=vst_is_10q)

# VPA - utilizando a função 'indicador_vpa_2', porque o nº de ações está em escala diferente
vpa_10q = f_eua.indicador_vpa_2(ten_k=False, df_bs=vst_bs_10q, df_is=vst_is_10q)

# P/VPA
p_vpa_10q = f_eua.indicador_p_vpa(ten_k=False, price=vst_close_months, vpa=vpa_10q, idx=idx_10q)

# Caixa e Equivalentes
lst_caixa = [
    'Cash and Cash Equivalents, at Carrying Value', 
    'Restricted Cash, Current', 
    'Restricted Cash, Noncurrent'
]

caixa_10q = f_eua.indicador_caixa(ten_k=False, df_bs=vst_bs_10q, lst_itens_caixa=lst_caixa)

# Dívida Bruta
lst_divida = [
    'Long-term Debt and Lease Obligation, Current', 
    'Long-term Debt and Lease Obligation'
]

div_bruta_10q = f_eua.indicador_divida_bruta(ten_k=False, df_bs=vst_bs_10q, lst_itens_divida=lst_divida)

# Dívida Líquida
div_liq_10q = f_eua.indicador_divida_liquida(ten_k=False, caixa=caixa_10q, div_bruta=div_bruta_10q, idx=idx_10q)

# Dívida Líquida / EBITDA
dl_ebitda_10q = f_eua.indicador_dl_ebitda(ten_k=False, ebitda=ebitda_10q_acum, div_liq=div_liq_10q, idx=idx_10q)

# Dívida Líquida / PL
dl_pl_10q = f_eua.indicador_dl_pl(ten_k=False, div_liq=div_liq_10q, df_bs=vst_bs_10q) 

# EV/EBITDA
ev_ebitda_10q = f_eua.indicador_ev_ebitda(vm=vm_10q, div_liq=div_liq_10q, ebitda=ebitda_10q_acum)

# ROE
roe_10q = f_eua.indicador_roe(ten_k=False, df_is=vst_is_10q, df_bs=vst_bs_10q, idx=idx_10q)

# ROIC 
roic_10q = f_eua.indicador_roic(ten_k=False, df_is=vst_is_10q, df_bs=vst_bs_10q, div_bruta=div_bruta_10q, idx=idx_10q)

# FCO
fco_10q = f_eua.indicador_fco(ten_k=False, df_cf=vst_cf_10q, first_quarter='03') 

# FCI
fci_10q = f_eua.indicador_fci(ten_k=False, df_cf=vst_cf_10q, first_quarter='03') 

# FCF
fcf_10q = f_eua.indicador_fcf(ten_k=False, df_cf=vst_cf_10q, first_quarter='03') 

# CAPEX
lst_capex_10q = [
    'Payments to Acquire Property, Plant, and Equipment' , 
    'Payments to Acquire Businesses, Net of Cash Acquired'
]

capex_10q = f_eua.indicador_capex(ten_k=False, df_cf=vst_cf_10q, first_quarter='03', lst_itens=lst_capex_10q, idx=idx_10q)[0]
capex_10q_sliced = f_eua.indicador_capex(ten_k=False, df_cf=vst_cf_10q, first_quarter='03', lst_itens=lst_capex_10q, idx=idx_10q)[1]
capex_10q_acum = f_eua.indicador_capex(ten_k=False, df_cf=vst_cf_10q, first_quarter='03', lst_itens=lst_capex_10q, idx=idx_10q)[2]

# Free Cash Flow
free_cash_flow_10q = f_eua.indicador_free_cash_flow(fco=fco_10q, capex=capex_10q_sliced)

# Net CAPEX
net_capex_10q = f_eua.indicador_net_capex(capex=capex_10q_acum, depreciation=depreciacao_10q_acum)

# R&D
rd_expense_10q = f_eua.indicador_rd(ten_k=False, df_is=vst_is_10q, idx=idx_10q)[0]
rd_expense_10q_sliced = f_eua.indicador_rd(ten_k=False, df_is=vst_is_10q, idx=idx_10q)[1]
rd_expense_10q_acum = f_eua.indicador_rd(ten_k=False, df_is=vst_is_10q, idx=idx_10q)[2]

# Net CAPEX ajustado
adj_net_capex_10q = f_eua.indicador_adj_net_capex(net_capex=net_capex_10q, rd_expense=rd_expense_10q_acum)

# Working capital
working_capital_10q = f_eua.indicador_working_capital(ten_k=False, df_bs=vst_bs_10q)

# Change in non cash working capital
lst_itens_change_non_cash_wc = [
    'Increase (Decrease) In Margin Deposits, Net',
    'Increase (Decrease) in Interest Payable, Net',
    'Increase (Decrease) in Accrued Taxes Payable',
    'Increase (Decrease) in Employee Related Liabilities',
    'Increase (Decrease) in Other Operating Assets and Liabilities, Net',
    'Increase (Decrease) in Other Receivables'
]

change_non_cash_wc_10q = f_eua.indicador_change_non_cash_wc(ten_k=False, df_cf=vst_cf_10q, lst_itens=lst_itens_change_non_cash_wc, first_quarter='03')[0]
change_non_cash_wc_10q_sliced = f_eua.indicador_change_non_cash_wc(ten_k=False, df_cf=vst_cf_10q, lst_itens=lst_itens_change_non_cash_wc, first_quarter='03')[1]

# Reinvestment rate
reinvestment_rate_10q = f_eua.indicador_reinvestment_rate(ten_k=False, net_capex=adj_net_capex_10q, change_non_cash_wc=change_non_cash_wc_10q, df_is=vst_is_10q, idx=idx_10q)

# New borrowing
lst_itens_new_borrowing = [
    'Proceeds from Issuance of Debt', 
    'Proceeds from Short-term Debt', 
    'Proceeds from Lines of Credit'
]

new_borrowing_10q = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=vst_cf_10q, first_quarter='03', idx=idx_10q)[0]
new_borrowing_10q_sliced = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=vst_cf_10q, first_quarter='03', idx=idx_10q)[1]
new_borrowing_10q_acum = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=vst_cf_10q, first_quarter='03', idx=idx_10q)[2]

# Debt paid
lst_itens_debt_paid = [
    'Repayments of Long-term Debt', 
    'Repayments of Lines of Credit', 
    'Repayments of Short-term Debt'
]

debt_paid_10q = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=vst_cf_10q, first_quarter='03', idx=idx_10q)[0]
debt_paid_10q_sliced = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=vst_cf_10q, first_quarter='03', idx=idx_10q)[1]
debt_paid_10q_acum = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=vst_cf_10q, first_quarter='03', idx=idx_10q)[2]

# Taxes on operating income
taxes_operating_income_10q = f_eua.indicador_taxes_operating_income(ten_k=False, df_is=vst_is_10q, idx=idx_10q)

# FCFE
fcfe_10q = f_eua.indicador_fcfe(
    ten_k=False, 
    depreciacao=depreciacao_10q_acum, 
    capex=capex_10q_acum, 
    change_non_cash_wc=change_non_cash_wc_10q_sliced, 
    new_borrowing=new_borrowing_10q_acum, 
    debt_paid=debt_paid_10q_acum, 
    df_is=vst_is_10q,
    idx=idx_10q
)

# FCFF
fcff_10q = f_eua.indicador_fcff(
        ten_k=False, 
        taxes_operating_income=taxes_operating_income_10q,
        depreciacao=depreciacao_10q_acum, 
        capex=capex_10q_acum, 
        change_non_cash_wc=change_non_cash_wc_10q_sliced, 
        df_is=vst_is_10q,
        idx=idx_10q
)

# Dividendo não acumulado
div_nao_acum = f_eua.indicador_div_nao_acum(df_cf=vst_cf_10q, first_quarter='03')

# Dividendo por Ação - utilizando a função 'indicador_dpa_2', porque o nº de ações está em escala diferente
dpa_10q = f_eua.indicador_dpa_2(ten_k=False, df_cf=vst_cf_10q, df_is=vst_is_10q, div_nao_acum=div_nao_acum)

# Payout
payout_10q = f_eua.indicador_payout(ten_k=False, df_cf=vst_cf_10q, df_is=vst_is_10q, div_nao_acum=div_nao_acum)

# Buyback
buyback_10q = f_eua.indicador_buyback(ten_k=False, df_cf=vst_cf_10q,  first_quarter='03')

# Lista com todos os indicadores
lst_indicadores_10q = [
    p_lpa_10q_damodaran,
    p_lpa_10q,
    lpa_p_10q,  
    ebitda_10q_sliced, 
    m_liq_10q,
    p_vpa_10q,
    vm_10q,
    div_bruta_10q,
    caixa_10q,
    div_liq_10q,
    dl_ebitda_10q,
    dl_pl_10q,
    ev_ebitda_10q,
    roe_10q,
    roic_10q,
    fco_10q,
    fci_10q,
    fcf_10q,
    free_cash_flow_10q,
    capex_10q_sliced,
    net_capex_10q,
    rd_expense_10q_sliced,
    adj_net_capex_10q,
    working_capital_10q,
    reinvestment_rate_10q,
    fcfe_10q,
    fcff_10q,
    dpa_10q,
    payout_10q,
    buyback_10q
]

# Juntando todos os indicadores em um df
vst_indicadores_10q = pd.concat(lst_indicadores_10q, axis=1)

# Nomes das colunas
nomes_col = [
    'P/L Damodaran', 
    'P/L', 
    'L/P', 
    'EBITDA', 
    'Margem Líquida',
    'P/VPA',
    'Valor de Mercado',
    'Dívida Bruta',
    'Caixa e Equivalentes',
    'Dívida Líquida',
    'Dívida Líquida/EBITDA',
    'Dívida Líquida/PL',
    'EV/EBITDA',
    'ROE',
    'ROIC',
    'FCO',
    'FCI',
    'FCF',
    'Free Cash Flow',
    'Capex',
    'Net Capex',
    'R&D',
    'Adj Net Capex',
    'Working Capital',
    'Reinvestment Rate',
    'FCFE',
    'FCFF',
    'DPA',
    'Payout',
    'Buyback'
]

# Atribuindo os novos nomes das colunas do df
vst_indicadores_10q.columns = nomes_col
vst_indicadores_10q

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//energia_eletrica//vistra//vst_indicators.xlsx') as writer:
    vst_indicadores_10k.to_excel(writer, sheet_name='indicators_10k')
    vst_indicadores_10q.to_excel(writer, sheet_name='indicators_10q')

## Talen

* 1Q -> Lançamento em Março (03);
* 2Q -> Lançamento em Junho (06);
* 3Q -> Lançamento em Setembro (09);
* 10K -> Lançamento em Dezembro (12).

In [ ]:
ticker = 'tln'
f_sec.cik_matching_ticker(ticker)

### Scraping Talen 10-K

In [ ]:
# Número dos documentos que contém a DRE, BP e FC 
acc_documents = f_sec.get_filtered_filings(ticker=ticker, ten_k=True, just_accession_numbers=True)

# Trocando os '-' para '' das strings ('0001045810-24-000264' -> '000104581024000264')
dict_acc_num = {k: v.replace('-', '') for k, v in acc_documents.items()}

# Transformando o dicionário em um df
df_lst_acc_num_talen = pd.DataFrame.from_dict(dict_acc_num, orient='index', columns=['acc_num'])

# Transformando em datetime o index
df_lst_acc_num_talen.index = pd.to_datetime(df_lst_acc_num_talen.index, format='%Y-%m-%d')

df_lst_acc_num_talen

In [ ]:
# Selecionando os 'acc_num' dos 10-k a partir de 2024 p/ iteração
df_lst_acc_num_talen_sliced = df_lst_acc_num_talen.loc['2024-12-31':]
df_lst_acc_num_talen_sliced

In [ ]:
# Rótulos
labels_dict = f_sec.get_label_dictionary(ticker, headers=f_sec.header)

In [ ]:
# NÃO fiz o loop no 'acc_num', porque só tem um ano (2024) em operação

# Balance sheet
balance_sheet = f_sec.process_one_statement(ticker=ticker, accession_number='000162828025008786', statement_name='balance_sheet')
df_balance_sheet_10k = f_sec.rename_statement(balance_sheet, label_dictionary=labels_dict)

# Substituindo os NaN por 0
df_balance_sheet_10k = df_balance_sheet_10k.fillna(0)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_balance_sheet_10k = df_balance_sheet_10k.drop([
    'us-gaap_AssetsAbstract',
    'Common Stock, Value, Issued'
])

# Renomeando a linha do índice que está sem formato 
df_balance_sheet_10k = df_balance_sheet_10k.rename(index={
    'tln_AssetRetirementObligationAndAccruedEnvironmentalCostsNoncurrent': 'Asset Retirement Obligation And Accrued Environmental Costs Noncurrent',
})

df_balance_sheet_10k

In [ ]:
# NÃO fiz o loop no 'acc_num', porque só tem um ano (2024) em operação

# Income statement
income_statement = f_sec.process_one_statement(ticker=ticker, accession_number='000162828025008786', statement_name='income_statement')
df_income_statement_10k = f_sec.rename_statement(income_statement, label_dictionary=labels_dict)

# O 'acc_num' de 2024 retorna duas colunas a mais ('2022-12-31' e '2023-05-17'), selecionando apenas as colunas '2023-12-31' e '2024-12-31'
df_income_statement_10k = df_income_statement_10k.loc[:, ['2024-12-31', '2023-12-31']]

# O 'acc_num' de 2024 retorna nas últimas três linhas com nomes iguais ('Revenues'), retirando essas linhas
df_income_statement_10k = df_income_statement_10k.iloc[0:-3]

# Substituindo os NaN por 0
df_income_statement_10k = df_income_statement_10k.fillna(0)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_income_statement_10k = df_income_statement_10k.drop([
    'us-gaap_OperatingExpensesAbstract',
    'Restructuring Charges',
    'Reorganization Items',
    'Variable Interest Entity, Initial Consolidation, Gain (Loss)'
])

# Renomeando a linha do índice que está sem formato 
df_income_statement_10k = df_income_statement_10k.rename(index={
    'tln_EnergyExpenses': 'Energy Expenses',
    'tln_OperatingExpenseFuelUsedAndPurchasedPower': 'Operating Expense Fuel Used And Purchased Power',
    'tln_UnrealizedGainLossOnCommodityContractsEnergyExpenses': 'Unrealized Gain Loss On Commodity Contracts Energy Expenses',
    'tln_OperationMaintenanceAndDevelopmentExpense': 'Operation Maintenance And Development Expense',
    'tln_GainLossOnNuclearDecommissioningTrustFundsNet': 'Gain Loss On Nuclear Decommissioning Trust Funds Net',
    'tln_GainLossOnDispositionOfAssetsAndOtherMiscellaneousExpensesNet': 'Gain Loss On Disposition Of Assets And Other Miscellaneous Expenses Net',
})

# Arrumando a escala da linha 'Weighted Average Number of Shares Outstanding, Basic'
df_income_statement_10k.loc['Weighted Average Number of Shares Outstanding, Basic'] = df_income_statement_10k.loc['Weighted Average Number of Shares Outstanding, Basic'] / 1000

df_income_statement_10k

In [ ]:
# NÃO fiz o loop no 'acc_num', porque só tem um ano (2024) em operação

# Cash flow statement
cash_flow_statement = f_sec.process_one_statement(ticker=ticker, accession_number='000162828025008786', statement_name='cash_flow_statement')
df_cash_flow_10k = f_sec.rename_statement(cash_flow_statement, label_dictionary=labels_dict)

# O 'acc_num' de 2024 retorna duas colunas a mais ('2022-12-31' e '2023-05-17'), selecionando apenas as colunas '2023-12-31' e '2024-12-31'
df_cash_flow_10k = df_cash_flow_10k.loc[:, ['2024-12-31', '2023-12-31']]

# Substituindo os NaN por 0
df_cash_flow_10k = df_cash_flow_10k.fillna(0)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_cash_flow_10k = df_cash_flow_10k.drop([
    'us-gaap_NetCashProvidedByUsedInOperatingActivitiesAbstract',
    'tln_ReorganizationItemsCashFlowImpact',
    'Restructuring Charges',
    'Variable Interest Entity, Initial Consolidation, Gain (Loss)',
    'tln_ProceedsFromConsolidationOfSubsidiaries',
    'Repayments of Secured Debt',
    'tln_ProceedsFromExitFinancingsNet',
    'Proceeds from Contributions from Affiliates',
    'Payment for Debt Extinguishment or Debt Prepayment Cost',
    'Payments for Derivative Instrument, Financing Activities',
    'tln_ProceedsFromDebtorInPossessionCreditFacilities',
    'tln_RepaymentsOfPrepetitionDeferredCapacityObligations',
    'tln_PaymentsForPrepetitionInventoryRepurchaseObligationsNet',
    'Proceeds from Lines of Credit',
    'Repayments of Lines of Credit'
])

# Renomeando a linha do índice que está sem formato 
df_cash_flow_10k = df_cash_flow_10k.rename(index={
    'tln_GainLossOnDispositionOfNonCoreAssets': 'Gain Loss On Disposition Of Non Core Assets',
    'tln_RepaymentOfTermLoan2': 'Repayment Of Term Loan (2)',
    'tln_PaymentForSettlementOfRestrictedStockUnits': 'Payment For Settlement Of Restricted Stock Units',
    'tln_RepaymentOfTermLoan': 'Repayment Of Term Loan',
})

df_cash_flow_10k

In [ ]:
# O 'df_income_statement_10k' não tem a linha 'Research and Development Expense'

# Linha 'Research and Development Expense' a ser adicionada 
new_row_rd_is_10k = [
    #2024
    0, 
    #2023
    0, 
]

# Adicionando a nova linha
df_income_statement_10k.loc['Research and Development'] = new_row_rd_is_10k

In [ ]:
# Arrumando alguns index dos dfs para ficar padronizado com os index utilizados nas funções p/ calcular os indicadores fundamentalistas
df_balance_sheet_10k = df_balance_sheet_10k.rename(index={
    "Stockholders' Equity Attributable to Parent": 'Total Shareholders Equity',
})

df_income_statement_10k = df_income_statement_10k.rename(index={
    'Operating Income (Loss)': 'Operating Income',
    'Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest': 'Income Before Provision for Income Taxes',
    'Income Tax Expense (Benefit)': 'Provision for Income Taxes',
    'Net Income (Loss) Available to Common Stockholders, Basic': 'Net Income',
    'Weighted Average Number of Shares Outstanding, Basic': 'Number of Shares - Basic',
    'Weighted Average Number of Shares Outstanding, Diluted': 'Number of Shares - Diluted'
})

df_cash_flow_10k = df_cash_flow_10k.rename(index={
    'Depreciation, Amortization and Accretion, Net': 'Depreciation, Depletion and Amortization',
})

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//energia_eletrica//talen//tln_10k.xlsx') as writer:
    df_balance_sheet_10k.to_excel(writer, sheet_name='balance_sheet')
    df_income_statement_10k.to_excel(writer, sheet_name='income_statement')
    df_cash_flow_10k.to_excel(writer, sheet_name='cash_flow')

### Scraping Talen 10-Q

In [ ]:
# Número dos documentos que contém a DRE, BP e FC 
acc_documents_10q = f_sec.get_filtered_filings(ticker=ticker, ten_k=False, just_accession_numbers=True)

# Trocando os '-' para '' das strings ('0001045810-24-000264' -> '000104581024000264')
dict_acc_num_10q = {k: v.replace('-', '') for k, v in acc_documents_10q.items()}

# Transformando o dicionário em um df
df_lst_acc_num_10q_tln = pd.DataFrame.from_dict(dict_acc_num_10q, orient='index', columns=['acc_num'])

# Transformando em datetime o index
df_lst_acc_num_10q_tln.index = pd.to_datetime(df_lst_acc_num_10q_tln.index, format='%Y-%m-%d')

df_lst_acc_num_10q_tln

In [ ]:
# Selecionando os 'acc_num' dos 10-k a partir de 2024 p/ iteração
df_lst_acc_num_10q_tln_sliced = df_lst_acc_num_10q_tln.loc['2024-06-30':]
df_lst_acc_num_10q_tln_sliced

* Esperar sair mais três trimestres para o 'acc_num' ficar com seis trimestres para funcionar os as funções do 10-q.

### Indicadores Fundamentalistas

#### Indicadores 10-K

In [ ]:
# Lendo os arquivos 10k
tln_bs = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//energia_eletrica//talen//tln_10k.xlsx', 
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

tln_is = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//energia_eletrica//talen//tln_10k.xlsx', 
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

tln_cf = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//energia_eletrica//talen//tln_10k.xlsx', 
    sheet_name='cash_flow', 
    index_col='Unnamed: 0'
)

# Transformando as colunas em datetime
tln_bs.columns = pd.to_datetime(tln_bs.columns)
tln_is.columns = pd.to_datetime(tln_is.columns)
tln_cf.columns = pd.to_datetime(tln_cf.columns)

# Transformando as colunas em apenas o ano
tln_bs.columns = tln_bs.columns.year
tln_is.columns = tln_is.columns.year
tln_cf.columns = tln_cf.columns.year

In [ ]:
# Preços da TLN
tln_close = yf.download('TLN', auto_adjust=True, multi_level_index=False, start='2023-01-01')['Close']

# Selecionando o último preço do mês de Dezembro de cada ano 
lst_years = [
    '2023', '2024'
]

last_dec_close = []

for year in lst_years:
    last_close = tln_close[f'{year}-12'].iloc[-1]
    last_dec_close.append(last_close)

# Transformando em uma série pandas
tln_close_dec = pd.Series(last_dec_close, index=lst_years)

# Revertendo o index da série p/ ficar igual com os dos indicadores
tln_close_dec = tln_close_dec.reindex(index=tln_close_dec.index[::-1])

# Arredondamento de duas casas decimais
tln_close_dec = round(tln_close_dec, 2)

# Não realizou split recentemente (não possui dados de split no site da Investing)

# Transformando o index em int
tln_close_dec.index = tln_close_dec.index.astype(int)

tln_close_dec

In [ ]:
# Criando uma lista só para fazer a indexação em que o ano mais recente está no começo
idx = tln_bs.columns

# Valor de Mercado
vm = f_eua.indicador_vm(ten_k=True, price=tln_close_dec, df_is=tln_is) 

# LPA
lpa = f_eua.indicador_lpa(ten_k=True, df_is=tln_is, idx=None)

# P/L Damodaran
p_lpa_damodaran = f_eua.indicador_p_lpa_2(ten_k=True, vm=vm, df_is=tln_is)

# P/L
p_lpa = f_eua.indicador_p_lpa(ten_k=True, price=tln_close_dec, lpa=lpa, idx=idx)

# L/P
lpa_p = f_eua.indicador_lpa_p(ten_k=True, price=tln_close_dec, lpa=lpa, idx=idx) 

# Depreciação
depreciacao = f_eua.indicador_depreciacao(ten_k=True, df_cf=tln_cf, first_quarter=None, idx=idx)

# EBITDA
ebitda = f_eua.indicador_ebitda(ten_k=True, df_is=tln_is, idx=None, depreciacao=depreciacao)

# Margem Líquida
m_liq = f_eua.indicador_ml(ten_k=True, df_is=tln_is)

# VPA
vpa = f_eua.indicador_vpa(ten_k=True, df_bs=tln_bs, df_is=tln_is)

# P/VPA
p_vpa = f_eua.indicador_p_vpa(ten_k=True, price=tln_close_dec, vpa=vpa, idx=idx)

# Caixa e Equivalentes
lst_caixa = [
    'Cash and Cash Equivalents, at Carrying Value', 
    'Restricted Cash and Cash Equivalents, Current'
]

caixa = f_eua.indicador_caixa(ten_k=True, df_bs=tln_bs, lst_itens_caixa=lst_caixa)

# Dívida Bruta
lst_divida = [
    'Long-term Debt, Current Maturities', 
    'Long-term Debt, Excluding Current Maturities'
]

div_bruta = f_eua.indicador_divida_bruta(ten_k=True, df_bs=tln_bs, lst_itens_divida=lst_divida)

# Dívida Líquida
div_liq = f_eua.indicador_divida_liquida(ten_k=True, caixa=caixa, div_bruta=div_bruta, idx=idx)

# Dívida Líquida / EBITDA
dl_ebitda = f_eua.indicador_dl_ebitda(ten_k=True, ebitda=ebitda, div_liq=div_liq, idx=None)

# Dívida Líquida / PL
dl_pl = f_eua.indicador_dl_pl(ten_k=True, div_liq=div_liq, df_bs=tln_bs) 

# EV/EBITDA
ev_ebitda = f_eua.indicador_ev_ebitda(vm=vm, div_liq=div_liq, ebitda=ebitda)

# ROE
roe = f_eua.indicador_roe(ten_k=True, df_is=tln_is, df_bs=tln_bs, idx=None)

# ROIC
roic = f_eua.indicador_roic(ten_k=True, df_is=tln_is, df_bs=tln_bs, div_bruta=div_bruta, idx=idx)

# FCO
fco = f_eua.indicador_fco(ten_k=True, df_cf=tln_cf, first_quarter=None) 

# FCI
fci = f_eua.indicador_fci(ten_k=True, df_cf=tln_cf, first_quarter=None) 

# FCF
fcf = f_eua.indicador_fcf(ten_k=True, df_cf=tln_cf, first_quarter=None) 

# CAPEX
lst_capex = [
    'Payments to Acquire Property, Plant, and Equipment', 
    'Payments to Acquire Interest in Subsidiaries and Affiliates'
]

capex= f_eua.indicador_capex(ten_k=True, df_cf=tln_cf, first_quarter=None, lst_itens=lst_capex, idx=idx)

# Free Cash Flow
free_cash_flow = f_eua.indicador_free_cash_flow(fco=fco, capex=capex)

# Net CAPEX
net_capex = f_eua.indicador_net_capex(capex=capex, depreciation=depreciacao)

# R&D
rd_expense = f_eua.indicador_rd(ten_k=True, df_is=tln_is, idx=idx)

# Net CAPEX ajustado
adj_net_capex = f_eua.indicador_adj_net_capex(net_capex=net_capex, rd_expense=rd_expense)

# Working capital
working_capital = f_eua.indicador_working_capital(ten_k=True, df_bs=tln_bs)

# Change in non cash working capital
lst_itens_change_non_cash_wc = [
    'Increase (Decrease) in Inventories',
    'Increase (Decrease) in Accounts and Other Receivables',
    'Increase (Decrease) in Other Operating Assets',
    'Increase (Decrease) in Accounts Payable and Accrued Liabilities',
    'Increase (Decrease) in Interest Payable, Net',
    'Increase (Decrease) in Other Operating Liabilities'
]

change_non_cash_wc = f_eua.indicador_change_non_cash_wc(ten_k=True, df_cf=tln_cf, lst_itens=lst_itens_change_non_cash_wc, first_quarter=None)

# Reinvestment rate
reinvestment_rate = f_eua.indicador_reinvestment_rate(ten_k=True, net_capex=net_capex, change_non_cash_wc=change_non_cash_wc, df_is=tln_is, idx=idx)

# New borrowing
lst_itens_new_borrowing = [
    'Proceeds from Issuance of Other Long-term Debt', 
    'Proceeds from Issuance of Secured Debt'
]

new_borrowing = f_eua.indicador_new_borrowing(ten_k=True, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=tln_cf, first_quarter=None, idx=idx)

# Debt paid
lst_itens_debt_paid = [
    'Repayments of Other Long-term Debt', 
    'Repayment Of Term Loan (2)', 
    'Payments of Debt Issuance Costs', 
    'Repayment Of Term Loan'
]

debt_paid = f_eua.indicador_debt_paid(ten_k=True, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=tln_cf, first_quarter=None, idx=idx)

# Taxes on operating income
taxes_operating_income = f_eua.indicador_taxes_operating_income(ten_k=True, df_is=tln_is, idx=idx)

# FCFE
fcfe_10q = f_eua.indicador_fcfe(
    ten_k=True, 
    depreciacao=depreciacao, 
    capex=capex, 
    change_non_cash_wc=change_non_cash_wc, 
    new_borrowing=new_borrowing,
    debt_paid=debt_paid,
    df_is=tln_is,
    idx=idx
)

# FCFF
fcff_10q = f_eua.indicador_fcff(
        ten_k=True, 
        taxes_operating_income=taxes_operating_income,
        depreciacao=depreciacao, 
        capex=capex, 
        change_non_cash_wc=change_non_cash_wc, 
        df_is=tln_is,
        idx=idx
)

# DPA e Payout - Talen não paga dividendo

# Buyback
buyback = f_eua.indicador_buyback(ten_k=True, df_cf=tln_cf, first_quarter=None)

# Colocando todos os indicadores em uma lista
lst_indicadores = [
    p_lpa_damodaran,
    p_lpa,
    lpa_p,  
    ebitda, 
    m_liq,
    p_vpa,
    vm,
    div_bruta,
    caixa,
    div_liq,
    dl_ebitda,
    dl_pl,
    ev_ebitda,
    roe,
    roic,
    fco,
    fci,
    fcf,
    free_cash_flow,
    capex,
    net_capex,
    rd_expense,
    adj_net_capex,
    working_capital,
    reinvestment_rate,
    fcfe_10q,
    fcff_10q,
    buyback
]

# Juntando todos os indicadores em um df
tln_indicadores_10k = pd.concat(lst_indicadores, axis=1)

# Nomes das colunas
nomes_col = [
    'P/L Damodaran', 
    'P/L', 
    'L/P', 
    'EBITDA', 
    'Margem Líquida',
    'P/VPA',
    'Valor de Mercado',
    'Dívida Bruta',
    'Caixa e Equivalentes',
    'Dívida Líquida',
    'Dívida Líquida/EBITDA',
    'Dívida Líquida/PL',
    'EV/EBITDA',
    'ROE',
    'ROIC',
    'FCO',
    'FCI',
    'FCF',
    'Free Cash Flow',
    'Capex',
    'Net Capex',
    'R&D',
    'Adj Net Capex',
    'Working Capital',
    'Reinvestment Rate',
    'FCFE',
    'FCFF',
    'Buyback'
]

# Atribuindo os novos nomes das colunas do df
tln_indicadores_10k.columns = nomes_col
tln_indicadores_10k

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//energia_eletrica//talen//tln_indicators.xlsx') as writer:
    tln_indicadores_10k.to_excel(writer, sheet_name='indicators_10k')

## Oklo

* 1Q -> Lançamento em Março (03);
* 2Q -> Lançamento em Junho (06);
* 3Q -> Lançamento em Setembro (09);
* 10K -> Lançamento em Dezembro (12).

In [ ]:
ticker = 'oklo'
f_sec.cik_matching_ticker(ticker)

### Scraping Oklo 10-K

In [ ]:
# Número dos documentos que contém a DRE, BP e FC 
acc_documents = f_sec.get_filtered_filings(ticker=ticker, ten_k=True, just_accession_numbers=True)

# Trocando os '-' para '' das strings ('0001045810-24-000264' -> '000104581024000264')
dict_acc_num = {k: v.replace('-', '') for k, v in acc_documents.items()}

# Transformando o dicionário em um df
df_lst_acc_num_oklo = pd.DataFrame.from_dict(dict_acc_num, orient='index', columns=['acc_num'])

# Transformando em datetime o index
df_lst_acc_num_oklo.index = pd.to_datetime(df_lst_acc_num_oklo.index, format='%Y-%m-%d')

df_lst_acc_num_oklo

In [ ]:
# Selecionando os 'acc_num' dos 10-k a partir de 2024 p/ iteração
df_lst_acc_num_oklo_sliced = df_lst_acc_num_oklo.loc['2024-12-31':]
df_lst_acc_num_oklo_sliced

In [ ]:
# Rótulos
labels_dict = f_sec.get_label_dictionary(ticker, headers=f_sec.header)

In [ ]:
# NÃO fiz o loop no 'acc_num', porque só tem um ano (2024) em operação

# Balance sheet
balance_sheet = f_sec.process_one_statement(ticker=ticker, accession_number='000162828025014490', statement_name='balance_sheet')
df_balance_sheet_10k = f_sec.rename_statement(balance_sheet, label_dictionary=labels_dict)

# Substituindo os NaN por 0
df_balance_sheet_10k = df_balance_sheet_10k.fillna(0)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_balance_sheet_10k = df_balance_sheet_10k.drop('us-gaap_AssetsCurrentAbstract')

# Renomeando a linha do índice que está sem formato 
df_balance_sheet_10k = df_balance_sheet_10k.rename(index={
    'DebtSecuritiesAvailableForSaleExcludingAccruedInterestCurrent': 'Debt Securities Available For Sale Excluding Accrued Interest Current',
    'DebtSecuritiesAvailableForSaleExcludingAccruedInterestNoncurrent': 'Debt Securities Available For Sale Excluding Accrued Interest Noncurrent',
    'oklo_RightOfFirstRefusalLiability': 'Right Of First Refusal Liability',
})

df_balance_sheet_10k

In [ ]:
# NÃO fiz o loop no 'acc_num', porque só tem um ano (2024) em operação

# Income statement
income_statement = f_sec.process_one_statement(ticker=ticker, accession_number='000162828025014490', statement_name='income_statement')
df_income_statement_10k = f_sec.rename_statement(income_statement, label_dictionary=labels_dict)

# Substituindo os NaN por 0
df_income_statement_10k = df_income_statement_10k.fillna(0)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_income_statement_10k = df_income_statement_10k.drop('us-gaap_OperatingExpensesAbstract')

# Renomeando a linha do índice que está sem formato 
df_income_statement_10k = df_income_statement_10k.rename(index={
    'oklo_FinancialInstrumentsSubjectToMandatoryRedemptionSettlementTermsChangeInFairValueOfShares': 'Financial Instruments Subject To Mandatory Redemption Settlement Terms Change In Fair Value Of Shares',
})

# Arrumando a escala da linha 'Weighted Average Number of Shares Outstanding, Basic'
df_income_statement_10k.loc['Weighted Average Number of Shares Outstanding, Basic'] = round(df_income_statement_10k.loc['Weighted Average Number of Shares Outstanding, Basic'] / 1000000, 2)

df_income_statement_10k

In [ ]:
# NÃO fiz o loop no 'acc_num', porque só tem um ano (2024) em operação

# Cash flow statement
cash_flow_statement = f_sec.process_one_statement(ticker=ticker, accession_number='000162828025014490', statement_name='cash_flow_statement')
df_cash_flow_10k = f_sec.rename_statement(cash_flow_statement, label_dictionary=labels_dict)

# Substituindo os NaN por 0
df_cash_flow_10k = df_cash_flow_10k.fillna(0)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_cash_flow_10k = df_cash_flow_10k.drop([
    'us-gaap_NetCashProvidedByUsedInOperatingActivitiesAbstract',
    'Interest Paid, Including Capitalized Interest, Operating and Investing Activities'
])

# Renomeando a linha do índice que está sem formato 
df_cash_flow_10k = df_cash_flow_10k.rename(index={
    'oklo_FinancialInstrumentsSubjectToMandatoryRedemptionSettlementTermsChangeInFairValueOfShares': 'Financial Instruments Subject To Mandatory Redemption Settlement Terms Change In Fair Value Of Shares',
    'oklo_IncreaseDecreaseInOperatingLeaseRightOfUseAssetsAndLiability': 'Increase (Decrease) In Operating Lease Right Of Use Assets And Liability',
    'oklo_CashAcquiredThroughReverseRecapitalization': 'Cash Acquired Through Reverse Recapitalization',
    'oklo_ProceedsFromRightOfFirstRefusalLiability': 'Proceeds From Right Of First Refusal Liability',
    'oklo_ReclassificationOfDeferredIssuanceCostsInConnectionWithBusinessCombination': 'Reclassification Of Deferred Issuance Costs In Connection With Business Combination',
    'oklo_ReclassificationOfSimpleAgreementsForFutureEquityInConnectionWithBusinessCombination': 'Reclassification Of Simple Agreements For Future Equity In Connection With Business Combination',
    'oklo_DeferredIssuanceCostsIncludedInAccountsPayable': 'Deferred Issuance Costs Included In Accounts Payable',
    'oklo_DeferredIssuanceCostsIncludedInAccruedExpenseAndOther': 'Deferred Issuance Costs Included In Accrued Expense And Other',
})

df_cash_flow_10k

In [ ]:
# O 'df_income_statement_10k' não tem as linhas 'Revenues' e 'Operating Income'

# Linha 'Revenues' a ser adicionada 
new_row_revenues_is_10k = [
    #2024
    0, 
    #2023
    0, 
]

# Linha 'Operating Income' a ser adicionada 
new_row_operating_income_is_10k = [
    #2024
    0, 
    #2023
    0, 
]

# Adicionando as novas linhas
df_income_statement_10k.loc['Revenues'] = new_row_revenues_is_10k
df_income_statement_10k.loc['Operating Income'] = new_row_operating_income_is_10k

In [ ]:
# O 'df_cash_flow_10k' não tem a linha 'Proceeds from Issuance'

# Linha 'Proceeds from Issuance' a ser adicionada 
new_row_proceeds_is_10k = [
    #2024
    0, 
    #2023
    0, 
]

# Adicionando a nova linha
df_cash_flow_10k.loc['Proceeds from Issuance'] = new_row_proceeds_is_10k 

In [ ]:
# Arrumando alguns index dos dfs para ficar padronizado com os index utilizados nas funções p/ calcular os indicadores fundamentalistas
df_balance_sheet_10k = df_balance_sheet_10k.rename(index={
    "Stockholders' Equity Attributable to Parent": 'Total Shareholders Equity',
})

df_income_statement_10k = df_income_statement_10k.rename(index={
    'Research and Development Expense': 'Research and Development',
    'Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest': 'Income Before Provision for Income Taxes',
    'Income Tax Expense (Benefit)': 'Provision for Income Taxes',
    'Net Income (Loss) Attributable to Parent': 'Net Income',
    'Weighted Average Number of Shares Outstanding, Basic': 'Number of Shares - Basic',
})

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//energia_eletrica//oklo//oklo_10k.xlsx') as writer:
    df_balance_sheet_10k.to_excel(writer, sheet_name='balance_sheet')
    df_income_statement_10k.to_excel(writer, sheet_name='income_statement')
    df_cash_flow_10k.to_excel(writer, sheet_name='cash_flow')

### Scraping Oklo 10-Q

In [ ]:
# Número dos documentos que contém a DRE, BP e FC 
acc_documents_10q = f_sec.get_filtered_filings(ticker=ticker, ten_k=False, just_accession_numbers=True)

# Trocando os '-' para '' das strings ('0001045810-24-000264' -> '000104581024000264')
dict_acc_num_10q = {k: v.replace('-', '') for k, v in acc_documents_10q.items()}

# Transformando o dicionário em um df
df_lst_acc_num_10q_oklo = pd.DataFrame.from_dict(dict_acc_num_10q, orient='index', columns=['acc_num'])

# Transformando em datetime o index
df_lst_acc_num_10q_oklo.index = pd.to_datetime(df_lst_acc_num_10q_oklo.index, format='%Y-%m-%d')

df_lst_acc_num_10q_oklo

In [ ]:
# Selecionando os 'acc_num' dos 10-k a partir de 2024 p/ iteração
df_lst_acc_num_10q_oklo_sliced = df_lst_acc_num_10q_oklo.loc['2024-06-30':]
df_lst_acc_num_10q_oklo_sliced

* Esperar sair mais três trimestres para o 'acc_num' ficar com seis trimestres para funcionar os as funções do 10-q.

### Indicadores Fundamentalistas

#### Indicadores 10-K

In [ ]:
# Lendo os arquivos 10k
oklo_bs = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//energia_eletrica//oklo//oklo_10k.xlsx', 
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

oklo_is = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//energia_eletrica//oklo//oklo_10k.xlsx', 
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

oklo_cf = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//energia_eletrica//oklo//oklo_10k.xlsx', 
    sheet_name='cash_flow', 
    index_col='Unnamed: 0'
)

# Transformando as colunas em datetime
oklo_bs.columns = pd.to_datetime(oklo_bs.columns)
oklo_is.columns = pd.to_datetime(oklo_is.columns)
oklo_cf.columns = pd.to_datetime(oklo_cf.columns)

# Transformando as colunas em apenas o ano
oklo_bs.columns = oklo_bs.columns.year
oklo_is.columns = oklo_is.columns.year
oklo_cf.columns = oklo_cf.columns.year

In [ ]:
# Preços da OKLO
oklo_close = yf.download('OKLO', auto_adjust=True, multi_level_index=False, start='2023-01-01')['Close']

# Selecionando o último preço do mês de Dezembro de cada ano 
lst_years = [
    '2023', '2024'
]

last_dec_close = []

for year in lst_years:
    last_close = oklo_close[f'{year}-12'].iloc[-1]
    last_dec_close.append(last_close)

# Transformando em uma série pandas
oklo_close_dec = pd.Series(last_dec_close, index=lst_years)

# Revertendo o index da série p/ ficar igual com os dos indicadores
oklo_close_dec = oklo_close_dec.reindex(index=oklo_close_dec.index[::-1])

# Arredondamento de duas casas decimais
oklo_close_dec = round(oklo_close_dec, 2)

# Não realizou split recentemente (não possui dados de split no site da Investing)

# Transformando o index em int
oklo_close_dec.index = oklo_close_dec.index.astype(int)

oklo_close_dec

In [ ]:
# Criando uma lista só para fazer a indexação em que o ano mais recente está no começo
idx = oklo_bs.columns

# Valor de Mercado
vm = f_eua.indicador_vm(ten_k=True, price=oklo_close_dec, df_is=oklo_is) 

# LPA
lpa = f_eua.indicador_lpa(ten_k=True, df_is=oklo_is, idx=None)

# P/L Damodaran
p_lpa_damodaran = f_eua.indicador_p_lpa_2(ten_k=True, vm=vm, df_is=oklo_is)

# P/L
p_lpa = f_eua.indicador_p_lpa(ten_k=True, price=oklo_close_dec, lpa=lpa, idx=idx)

# L/P
lpa_p = f_eua.indicador_lpa_p(ten_k=True, price=oklo_close_dec, lpa=lpa, idx=idx) 

# Depreciação
depreciacao = f_eua.indicador_depreciacao(ten_k=True, df_cf=oklo_cf, first_quarter=None, idx=idx)

# EBITDA
ebitda = f_eua.indicador_ebitda(ten_k=True, df_is=oklo_is, idx=None, depreciacao=depreciacao)

# Margem Líquida
m_liq = f_eua.indicador_ml(ten_k=True, df_is=oklo_is)

# VPA
vpa = f_eua.indicador_vpa(ten_k=True, df_bs=oklo_bs, df_is=oklo_is)

# P/VPA
p_vpa = f_eua.indicador_p_vpa(ten_k=True, price=oklo_close_dec, vpa=vpa, idx=idx)

# Caixa e Equivalentes
lst_caixa = [
    'Cash and Cash Equivalents, at Carrying Value', 
    'Debt Securities Available For Sale Excluding Accrued Interest Current'
]

caixa = f_eua.indicador_caixa(ten_k=True, df_bs=oklo_bs, lst_itens_caixa=lst_caixa)

# Dívida Bruta
lst_divida = ['Operating Lease, Liability, Noncurrent']

div_bruta = f_eua.indicador_divida_bruta(ten_k=True, df_bs=oklo_bs, lst_itens_divida=lst_divida)

# Dívida Líquida
div_liq = f_eua.indicador_divida_liquida(ten_k=True, caixa=caixa, div_bruta=div_bruta, idx=idx)

# Dívida Líquida / EBITDA
dl_ebitda = f_eua.indicador_dl_ebitda(ten_k=True, ebitda=ebitda, div_liq=div_liq, idx=None)

# Dívida Líquida / PL
dl_pl = f_eua.indicador_dl_pl(ten_k=True, div_liq=div_liq, df_bs=oklo_bs) 

# EV/EBITDA
ev_ebitda = f_eua.indicador_ev_ebitda(vm=vm, div_liq=div_liq, ebitda=ebitda)

# ROE
roe = f_eua.indicador_roe(ten_k=True, df_is=oklo_is, df_bs=oklo_bs, idx=None)

# ROIC
roic = f_eua.indicador_roic(ten_k=True, df_is=oklo_is, df_bs=oklo_bs, div_bruta=div_bruta, idx=idx)

# FCO
fco = f_eua.indicador_fco(ten_k=True, df_cf=oklo_cf, first_quarter=None) 

# FCI
fci = f_eua.indicador_fci(ten_k=True, df_cf=oklo_cf, first_quarter=None) 

# FCF
fcf = f_eua.indicador_fcf(ten_k=True, df_cf=oklo_cf, first_quarter=None) 

# CAPEX
lst_capex = ['Payments to Acquire Property, Plant, and Equipment']

capex= f_eua.indicador_capex(ten_k=True, df_cf=oklo_cf, first_quarter=None, lst_itens=lst_capex, idx=idx)

# Free Cash Flow
free_cash_flow = f_eua.indicador_free_cash_flow(fco=fco, capex=capex)

# Net CAPEX
net_capex = f_eua.indicador_net_capex(capex=capex, depreciation=depreciacao)

# R&D
rd_expense = f_eua.indicador_rd(ten_k=True, df_is=oklo_is, idx=idx)

# Net CAPEX ajustado
adj_net_capex = f_eua.indicador_adj_net_capex(net_capex=net_capex, rd_expense=rd_expense)

# Working capital
working_capital = f_eua.indicador_working_capital(ten_k=True, df_bs=oklo_bs)

# Change in non cash working capital
lst_itens_change_non_cash_wc = [
    'Increase (Decrease) in Prepaid Expense and Other Assets',
    'Increase (Decrease) in Other Noncurrent Assets',
    'Increase (Decrease) in Accounts Payable',
    'Increase (Decrease) in Other Accrued Liabilities',
    'Increase (Decrease) In Operating Lease Right Of Use Assets And Liability'
]

change_non_cash_wc = f_eua.indicador_change_non_cash_wc(ten_k=True, df_cf=oklo_cf, lst_itens=lst_itens_change_non_cash_wc, first_quarter=None)

# Reinvestment rate
reinvestment_rate = f_eua.indicador_reinvestment_rate(ten_k=True, net_capex=net_capex, change_non_cash_wc=change_non_cash_wc, df_is=oklo_is, idx=idx)

# New borrowing
lst_itens_new_borrowing = ['Proceeds from Issuance']

new_borrowing = f_eua.indicador_new_borrowing(ten_k=True, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=oklo_cf, first_quarter=None, idx=idx)

# Debt paid
lst_itens_debt_paid = ['Payments of Stock Issuance Costs']

debt_paid = f_eua.indicador_debt_paid(ten_k=True, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=oklo_cf, first_quarter=None, idx=idx)

# Taxes on operating income
taxes_operating_income = f_eua.indicador_taxes_operating_income(ten_k=True, df_is=oklo_is, idx=idx)

# FCFE
fcfe_10q = f_eua.indicador_fcfe(
    ten_k=True, 
    depreciacao=depreciacao, 
    capex=capex, 
    change_non_cash_wc=change_non_cash_wc, 
    new_borrowing=new_borrowing,
    debt_paid=debt_paid,
    df_is=oklo_is,
    idx=idx
)

# FCFF
fcff_10q = f_eua.indicador_fcff(
        ten_k=True, 
        taxes_operating_income=taxes_operating_income,
        depreciacao=depreciacao, 
        capex=capex, 
        change_non_cash_wc=change_non_cash_wc, 
        df_is=oklo_is,
        idx=idx
)

# DPA e Payout - Oklo não paga dividendo

# Buyback - Oklo não recompra ações

# Colocando todos os indicadores em uma lista
lst_indicadores = [
    p_lpa_damodaran,
    p_lpa,
    lpa_p,  
    ebitda, 
    m_liq,
    p_vpa,
    vm,
    div_bruta,
    caixa,
    div_liq,
    dl_ebitda,
    dl_pl,
    ev_ebitda,
    roe,
    roic,
    fco,
    fci,
    fcf,
    free_cash_flow,
    capex,
    net_capex,
    rd_expense,
    adj_net_capex,
    working_capital,
    reinvestment_rate,
    fcfe_10q,
    fcff_10q,
]

# Juntando todos os indicadores em um df
oklo_indicadores_10k = pd.concat(lst_indicadores, axis=1)

# Nomes das colunas
nomes_col = [
    'P/L Damodaran', 
    'P/L', 
    'L/P', 
    'EBITDA', 
    'Margem Líquida',
    'P/VPA',
    'Valor de Mercado',
    'Dívida Bruta',
    'Caixa e Equivalentes',
    'Dívida Líquida',
    'Dívida Líquida/EBITDA',
    'Dívida Líquida/PL',
    'EV/EBITDA',
    'ROE',
    'ROIC',
    'FCO',
    'FCI',
    'FCF',
    'Free Cash Flow',
    'Capex',
    'Net Capex',
    'R&D',
    'Adj Net Capex',
    'Working Capital',
    'Reinvestment Rate',
    'FCFE',
    'FCFF',
]

# Atribuindo os novos nomes das colunas do df
oklo_indicadores_10k.columns = nomes_col
oklo_indicadores_10k

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//energia_eletrica//oklo//oklo_indicators.xlsx') as writer:
    oklo_indicadores_10k.to_excel(writer, sheet_name='indicators_10k')

## NRG

* 1Q -> Lançamento em Março (03);
* 2Q -> Lançamento em Junho (06);
* 3Q -> Lançamento em Setembro (09);
* 10K -> Lançamento em Dezembro (12).

In [ ]:
ticker = 'nrg'
f_sec.cik_matching_ticker(ticker)

### Scraping NRG 10-K

In [ ]:
# Número dos documentos que contém a DRE, BP e FC 
acc_documents = f_sec.get_filtered_filings(ticker=ticker, ten_k=True, just_accession_numbers=True)

# Trocando os '-' para '' das strings ('0001045810-24-000264' -> '000104581024000264')
dict_acc_num = {k: v.replace('-', '') for k, v in acc_documents.items()}

# Transformando o dicionário em um df
df_lst_acc_num_nrg = pd.DataFrame.from_dict(dict_acc_num, orient='index', columns=['acc_num'])

# Transformando em datetime o index
df_lst_acc_num_nrg.index = pd.to_datetime(df_lst_acc_num_nrg.index, format='%Y-%m-%d')

df_lst_acc_num_nrg

In [ ]:
# Rótulos
labels_dict = f_sec.get_label_dictionary(ticker, headers=f_sec.header)

In [ ]:
# Fazendo o scraping do balance sheet (10-K)
balance_sheet_lst = []

for acc_num in df_lst_acc_num_nrg['acc_num']:

    # Balance sheet
    balance_sheet = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='balance_sheet')
    balance_sheet = f_sec.rename_statement(balance_sheet, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    balance_sheet = balance_sheet.iloc[:, 0]
    balance_sheet_lst.append(balance_sheet)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_balance_sheet_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in balance_sheet_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_balance_sheet_10k = pd.concat([df_balance_sheet_10k, df], axis=1)

# Substituindo os NaN por 0
df_balance_sheet_10k = df_balance_sheet_10k.fillna(0)

# A linha 'Accrued Liabilities, Current' nas colunas '2023-12-31', '2022-12-31', '2021-12-31', '2020-12-31', '2019-12-31', '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'Other Accrued Liabilities, Current'
df_balance_sheet_10k.loc['Accrued Liabilities, Current', '2023-12-31'] = df_balance_sheet_10k.loc['Other Accrued Liabilities, Current', '2023-12-31'] 
df_balance_sheet_10k.loc['Accrued Liabilities, Current', '2022-12-31'] = df_balance_sheet_10k.loc['Other Accrued Liabilities, Current', '2022-12-31'] 
df_balance_sheet_10k.loc['Accrued Liabilities, Current', '2021-12-31'] = df_balance_sheet_10k.loc['Other Accrued Liabilities, Current', '2021-12-31'] 
df_balance_sheet_10k.loc['Accrued Liabilities, Current', '2020-12-31'] = df_balance_sheet_10k.loc['Other Accrued Liabilities, Current', '2020-12-31'] 
df_balance_sheet_10k.loc['Accrued Liabilities, Current', '2019-12-31'] = df_balance_sheet_10k.loc['Other Accrued Liabilities, Current', '2019-12-31'] 
df_balance_sheet_10k.loc['Accrued Liabilities, Current', '2018-12-31'] = df_balance_sheet_10k.loc['Other Accrued Liabilities, Current', '2018-12-31'] 
df_balance_sheet_10k.loc['Accrued Liabilities, Current', '2017-12-31'] = df_balance_sheet_10k.loc['Other Accrued Liabilities, Current', '2017-12-31'] 

# A linha 'nrg_DerivativeLiabilityNotSubjectToMasterNettingArrangementCollateralRightToReclaimCashNotOffset' nas colunas '2021-12-31', '2020-12-31', '2019-12-31', '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'Derivative, Collateral, Right to Reclaim Cash'
df_balance_sheet_10k.loc['nrg_DerivativeLiabilityNotSubjectToMasterNettingArrangementCollateralRightToReclaimCashNotOffset', '2021-12-31'] = df_balance_sheet_10k.loc['Derivative, Collateral, Right to Reclaim Cash', '2021-12-31'] 
df_balance_sheet_10k.loc['nrg_DerivativeLiabilityNotSubjectToMasterNettingArrangementCollateralRightToReclaimCashNotOffset', '2020-12-31'] = df_balance_sheet_10k.loc['Derivative, Collateral, Right to Reclaim Cash', '2020-12-31'] 
df_balance_sheet_10k.loc['nrg_DerivativeLiabilityNotSubjectToMasterNettingArrangementCollateralRightToReclaimCashNotOffset', '2019-12-31'] = df_balance_sheet_10k.loc['Derivative, Collateral, Right to Reclaim Cash', '2019-12-31'] 
df_balance_sheet_10k.loc['nrg_DerivativeLiabilityNotSubjectToMasterNettingArrangementCollateralRightToReclaimCashNotOffset', '2018-12-31'] = df_balance_sheet_10k.loc['Derivative, Collateral, Right to Reclaim Cash', '2018-12-31'] 
df_balance_sheet_10k.loc['nrg_DerivativeLiabilityNotSubjectToMasterNettingArrangementCollateralRightToReclaimCashNotOffset', '2017-12-31'] = df_balance_sheet_10k.loc['Derivative, Collateral, Right to Reclaim Cash', '2017-12-31'] 

# A linha 'Public Utilities, Property, Plant and Equipment, Net' nas colunas '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'Property, Plant and Equipment, Net'
df_balance_sheet_10k.loc['Public Utilities, Property, Plant and Equipment, Net', '2018-12-31'] = df_balance_sheet_10k.loc['Property, Plant and Equipment, Net', '2018-12-31'] 
df_balance_sheet_10k.loc['Public Utilities, Property, Plant and Equipment, Net', '2017-12-31'] = df_balance_sheet_10k.loc['Property, Plant and Equipment, Net', '2017-12-31'] 

# A linha 'Deferred Income Tax Assets, Net' nas colunas '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'Deferred Tax Assets, Net of Valuation Allowance, Noncurrent'
df_balance_sheet_10k.loc['Deferred Income Tax Assets, Net', '2018-12-31'] = df_balance_sheet_10k.loc['Deferred Tax Assets, Net of Valuation Allowance, Noncurrent', '2018-12-31'] 
df_balance_sheet_10k.loc['Deferred Income Tax Assets, Net', '2017-12-31'] = df_balance_sheet_10k.loc['Deferred Tax Assets, Net of Valuation Allowance, Noncurrent', '2017-12-31'] 

# A linha 'Regulated Entity, Other Assets, Noncurrent' nas colunas '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'Assets, Noncurrent'
df_balance_sheet_10k.loc['Regulated Entity, Other Assets, Noncurrent', '2018-12-31'] = df_balance_sheet_10k.loc['Assets, Noncurrent', '2018-12-31'] 
df_balance_sheet_10k.loc['Regulated Entity, Other Assets, Noncurrent', '2017-12-31'] = df_balance_sheet_10k.loc['Assets, Noncurrent', '2017-12-31'] 

# Removendo as linhas que movemos os seus itens para as linhas corretas
df_balance_sheet_10k = df_balance_sheet_10k.drop([
    'Other Accrued Liabilities, Current',
    'Derivative, Collateral, Right to Reclaim Cash',
    'Property, Plant and Equipment, Net',
    'Deferred Tax Assets, Net of Valuation Allowance, Noncurrent',
    'Assets, Noncurrent'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_balance_sheet_10k = df_balance_sheet_10k.drop([
    'us-gaap_AssetsCurrentAbstract',
    'Assets Held-for-sale, Not Part of Disposal Group, Current',
    'nrg_AssetsHeldForSaleNotPartOfDisposalGroupNoncurrent',
    'nrg_LiabilitiesHeldForSaleNotPartOfDisposalGroupOrDiscontinuedOperationsCurrent',
    'nrg_LiabilitiesHeldForSaleNotPartOfDisposalGroupOrDiscontinuedOperationsNonCurrent',
    'Due to Affiliate, Current',
    "Stockholders' Equity Attributable to Noncontrolling Interest",
    'Financing Receivable, after Allowance for Credit Loss, Noncurrent',
    'Derivative, Collateral, Obligation to Return Cash',
    'Interest Payable, Current',
    'Disposal Group, Including Discontinued Operation, Assets, Current',
    'Disposal Group, Including Discontinued Operation, Assets, Noncurrent',
    'Disposal Group, Including Discontinued Operation, Liabilities, Current',
    'Disposal Group, Including Discontinued Operation, Other Liabilities, Noncurrent',
    'Disposal Group, Including Discontinued Operation, Liabilities, Noncurrent',
    'Due from Related Parties, Current',
    'Disposal Group, Including Discontinued Operation, Other Assets, Current',
    'Disposal Group, Including Discontinued Operation, Other Assets, Noncurrent',
    'Accounts Payable, Related Parties, Current',
    'Disposal Group, Including Discontinued Operation, Accrued Liabilities, Current',
    'Liability, Pension and Other Postretirement and Postemployment Benefits, Noncurrent',
    'Off-market Lease, Unfavorable',
    'nrg_ReceivableFromGovernmentGrant',
    'Redeemable Noncontrolling Interest, Equity, Other, Carrying Amount'
])

# Renomeando a linha do índice que está sem formato 
df_balance_sheet_10k = df_balance_sheet_10k.rename(index={
    'nrg_FundsDepositedByCounterparties': 'Funds Deposited By Counterparties',
    'nrg_DerivativeLiabilityNotSubjectToMasterNettingArrangementCollateralRightToReclaimCashNotOffset': 'Derivative Liability Not Subject To Master Netting Arrangement Collateral Right To Reclaim Cash Not Offset',
    'nrg_NuclearDecommissioningTrustFundLiabilityNoncurrent': 'Nuclear Decommissioning Trust Fund Liability Noncurrent',
})

df_balance_sheet_10k

In [ ]:
# Os dados do income statement da NRG de 2019 não estão aparecendo no loop com o 'acc_num'
# Para selecionar os dados de 2019, eu utilizei o 'acc_num' de 2020
income_statement_20_19_18 = f_sec.process_one_statement(ticker=ticker, accession_number='000101387121000005', statement_name='income_statement')
income_statement_20_19_18 = f_sec.rename_statement(income_statement_20_19_18, label_dictionary=labels_dict)
# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
income_statement_20_19_18 = income_statement_20_19_18.drop('us-gaap_RevenuesAbstract')


# Dados do income statement de 2017
income_statement_17 = f_sec.process_one_statement(ticker=ticker, accession_number='000101387118000011', statement_name='income_statement')
income_statement_17 = f_sec.rename_statement(income_statement_17, label_dictionary=labels_dict)
# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
income_statement_17 = income_statement_17.drop('us-gaap_RevenuesAbstract')
# Selecionando a coluna de 2017
income_statement_17 = income_statement_17.loc[:, '2017-12-31']


# Juntando os dfs ('income_statement_20_19_18' e 'income_statement_17')
income_statement_20_19_18_17 = pd.concat([income_statement_20_19_18, income_statement_17], axis=1)

# Substituindo os NaN por 0
income_statement_20_19_18_17 = income_statement_20_19_18_17.fillna(0)

# A linha 'Restructuring Costs' na coluna '2017-12-31' está zerada
# As informações estão na linha 'Reorganization Items'
income_statement_20_19_18_17.loc['Restructuring Costs', '2017-12-31'] = income_statement_20_19_18_17.loc['Reorganization Items', '2017-12-31'] 

# A linha 'Gain (Loss) on Disposition of Other Assets' na coluna '2017-12-31' está zerada
# As informações estão na linha 'Disposal Group, Not Discontinued Operation, Gain (Loss) on Disposal'
income_statement_20_19_18_17.loc['Gain (Loss) on Disposition of Other Assets', '2017-12-31'] = income_statement_20_19_18_17.loc['Disposal Group, Not Discontinued Operation, Gain (Loss) on Disposal', '2017-12-31'] 

# Removendo as linhas que movemos os seus itens para as linhas corretas
income_statement_20_19_18_17 = income_statement_20_19_18_17.drop([
    'Reorganization Items',
    'Disposal Group, Not Discontinued Operation, Gain (Loss) on Disposal'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
income_statement_20_19_18_17 = income_statement_20_19_18_17.drop([
    'Defined Benefit Plan, Net Periodic Benefit Cost (Credit), Gain (Loss) Due to Curtailment',
    'Equity Method Investment, Realized Gain (Loss) on Disposal',
    'Preferred Stock Dividends, Income Statement Impact',
    'nrg_GainonRedemptionofRedeemablePreferredStock',
    'Income (Loss) from Continuing Operations, Per Basic Share',
    'Discontinued Operation, Income (Loss) from Discontinued Operation, Net of Tax, Per Basic Share',
    'Income (Loss) from Continuing Operations, Per Diluted Share',
    'Discontinued Operation, Income (Loss) from Discontinued Operation, Net of Tax, Per Diluted Share'
])	

# # Arrumando e adicionando alguns números no df
income_statement_20_19_18_17.loc['Weighted Average Number of Shares Outstanding, Basic'] = round(income_statement_20_19_18_17.loc['Weighted Average Number of Shares Outstanding, Basic'] / 1000, 0)
income_statement_20_19_18_17.loc['Weighted Average Number of Shares Outstanding, Diluted'] = round(income_statement_20_19_18_17.loc['Weighted Average Number of Shares Outstanding, Diluted'] / 1000, 0)
income_statement_20_19_18_17.loc['Weighted Average Number of Shares Outstanding, Basic', '2017-12-31'] = 317000
income_statement_20_19_18_17.loc['Weighted Average Number of Shares Outstanding, Diluted', '2017-12-31'] = 317000

In [ ]:
# Fazendo o scraping do income statement (10-K)
income_statement_lst = []

for acc_num in df_lst_acc_num_nrg['acc_num']:

    # Income statement
    income_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='income_statement')
    income_statement = f_sec.rename_statement(income_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    income_statement = income_statement.iloc[:, 0]
    income_statement_lst.append(income_statement)

# Juntando todos os dfs que estão em listas em apenas um df
df_income_statement_10k = pd.concat(income_statement_lst, axis=1)

# Substituindo os NaN por 0
df_income_statement_10k = df_income_statement_10k.fillna(0)

# Selecionando as colunas a partir de 2021
df_income_statement_10k = df_income_statement_10k.loc[:, '2021-12-31':]

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_income_statement_10k = df_income_statement_10k.drop([
    'us-gaap_RevenuesAbstract',
    'Other than Temporary Impairment Losses, Investments',
    'Income (Loss) from Discontinued Operations, Net of Tax, Attributable to Parent',
     'Net Income (Loss) Attributable to Noncontrolling Interest',
    'Income (Loss) from Continuing Operations, Per Basic Share',		
    'Discontinued Operation, Income (Loss) from Discontinued Operation, Net of Tax, Per Basic Share',			
    'Income (Loss) from Continuing Operations, Per Diluted Share',
    'Discontinued Operation, Income (Loss) from Discontinued Operation, Net of Tax, Per Diluted Share',			
    'Cost of Revenue',		
    'Restructuring Costs',				
    'Research and Development Expense',				
    'us-gaap_StatementOfIncomeAndComprehensiveIncomeAbstract',			
    'Other Comprehensive Income (Loss), Cash Flow Hedge, Gain (Loss), before Reclassification, after Tax',				
    'Other Comprehensive Income (Loss), Foreign Currency Transaction and Translation Adjustment, Net of Tax',				
    'OCI, Debt Securities, Available-for-Sale, Gain (Loss), after Adjustment and Tax',				
    'Other Comprehensive (Income) Loss, Defined Benefit Plan, after Reclassification Adjustment, after Tax',				
    'Other Comprehensive Income (Loss), Net of Tax',			
    'Comprehensive Income (Loss), Net of Tax, Including Portion Attributable to Noncontrolling Interest',			
    'Comprehensive Income (Loss), Net of Tax, Attributable to Noncontrolling Interest',
    'Comprehensive Income (Loss), Net of Tax, Attributable to Parent',		
    'Reorganization Items',			
    'Related Party Transaction, Other Revenues from Transactions with Related Party',			
    'Disposal Group, Not Discontinued Operation, Gain (Loss) on Disposal',			
    'nrg_GainonRedemptionofRedeemablePreferredStock',			
    'Common Stock, Dividends, Per Share, Declared',			
    'Defined Benefit Plan, Net Periodic Benefit Cost (Credit), Gain (Loss) Due to Curtailment',			
    'Equity Method Investment, Realized Gain (Loss) on Disposal',
    'Earnings Per Share, Basic and Diluted',	
    'Income (Loss) from Continuing Operations, Net of Tax, Including Portion Attributable to Noncontrolling Interest',
    'Net Income (Loss) Available to Common Stockholders, Basic'
])

# Arrumando e adicionando alguns números no df
df_income_statement_10k.loc['Net Income (Loss) Attributable to Parent', '2024-12-31'] = 1125
df_income_statement_10k.loc['Net Income (Loss) Attributable to Parent', '2023-12-31'] = -202
df_income_statement_10k.loc['Net Income (Loss) Attributable to Parent', '2022-12-31'] = 1221000
df_income_statement_10k.loc['Weighted Average Number of Shares Outstanding, Diluted', '2023-12-31'] = 228000

# Renomeando as linhas do índice que estão sem formato 
df_income_statement_10k = df_income_statement_10k.rename(index={
    'Cost of Goods and Service, Excluding Depreciation, Depletion, and Amortization': 'Cost of Revenue',
})


# Juntando os dfs 
df_income_statement_10k_final = pd.concat([df_income_statement_10k, income_statement_20_19_18_17], axis=1)

# Substituindo os NaN por 0
df_income_statement_10k_final = df_income_statement_10k_final.fillna(0)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_income_statement_10k_final = df_income_statement_10k_final.drop([
    'Net Income (Loss) Attributable to Noncontrolling Interest',
    'Related Party Transaction, Other Revenues from Transactions with Related Party',
    'Earnings Per Share, Basic and Diluted',
    'Common Stock, Dividends, Per Share, Declared',
    'Preferred Stock Dividends, Income Statement Impact',
])

# Renomeando a linha do índice que está sem formato 
df_income_statement_10k_final = df_income_statement_10k_final.rename(index={
    'nrg_BusinessCombinationAcquisitionAndIntegrationCosts': 'Business Combination Acquisition And Integration Costs',
})

# Arrumando a escala das linhas 'Weighted Average Number of Shares Outstanding, Basic' e 'Weighted Average Number of Shares Outstanding, Diluted'
df_income_statement_10k_final.loc['Weighted Average Number of Shares Outstanding, Basic', '2017-12-31'] = round(df_income_statement_10k_final.loc['Weighted Average Number of Shares Outstanding, Basic', '2017-12-31']/1000, 2)
df_income_statement_10k_final.loc['Weighted Average Number of Shares Outstanding, Basic', '2018-12-31'] = round(df_income_statement_10k_final.loc['Weighted Average Number of Shares Outstanding, Basic', '2018-12-31']/1000, 2)
df_income_statement_10k_final.loc['Weighted Average Number of Shares Outstanding, Basic', '2019-12-31'] = round(df_income_statement_10k_final.loc['Weighted Average Number of Shares Outstanding, Basic', '2019-12-31']/1000, 2)
df_income_statement_10k_final.loc['Weighted Average Number of Shares Outstanding, Basic', '2020-12-31'] = round(df_income_statement_10k_final.loc['Weighted Average Number of Shares Outstanding, Basic', '2020-12-31']/1000, 2)
df_income_statement_10k_final.loc['Weighted Average Number of Shares Outstanding, Basic', '2023-12-31'] = round(df_income_statement_10k_final.loc['Weighted Average Number of Shares Outstanding, Basic', '2023-12-31']/1000, 2)
df_income_statement_10k_final.loc['Weighted Average Number of Shares Outstanding, Basic', '2024-12-31'] = round(df_income_statement_10k_final.loc['Weighted Average Number of Shares Outstanding, Basic', '2024-12-31']/1000, 2)
df_income_statement_10k_final.loc['Weighted Average Number of Shares Outstanding, Basic', '2021-12-31'] = round(df_income_statement_10k_final.loc['Weighted Average Number of Shares Outstanding, Basic', '2021-12-31']/1000000, 2)
df_income_statement_10k_final.loc['Weighted Average Number of Shares Outstanding, Basic', '2022-12-31'] = round(df_income_statement_10k_final.loc['Weighted Average Number of Shares Outstanding, Basic', '2022-12-31']/1000000, 2)

df_income_statement_10k_final.loc['Weighted Average Number of Shares Outstanding, Diluted', '2017-12-31'] = round(df_income_statement_10k_final.loc['Weighted Average Number of Shares Outstanding, Diluted', '2017-12-31']/1000, 2)
df_income_statement_10k_final.loc['Weighted Average Number of Shares Outstanding, Diluted', '2018-12-31'] = round(df_income_statement_10k_final.loc['Weighted Average Number of Shares Outstanding, Diluted', '2018-12-31']/1000, 2)
df_income_statement_10k_final.loc['Weighted Average Number of Shares Outstanding, Diluted', '2019-12-31'] = round(df_income_statement_10k_final.loc['Weighted Average Number of Shares Outstanding, Diluted', '2019-12-31']/1000, 2)
df_income_statement_10k_final.loc['Weighted Average Number of Shares Outstanding, Diluted', '2020-12-31'] = round(df_income_statement_10k_final.loc['Weighted Average Number of Shares Outstanding, Diluted', '2020-12-31']/1000, 2)
df_income_statement_10k_final.loc['Weighted Average Number of Shares Outstanding, Diluted', '2023-12-31'] = round(df_income_statement_10k_final.loc['Weighted Average Number of Shares Outstanding, Diluted', '2023-12-31']/1000, 2)
df_income_statement_10k_final.loc['Weighted Average Number of Shares Outstanding, Diluted', '2024-12-31'] = round(df_income_statement_10k_final.loc['Weighted Average Number of Shares Outstanding, Diluted', '2024-12-31']/1000, 2)
df_income_statement_10k_final.loc['Weighted Average Number of Shares Outstanding, Diluted', '2021-12-31'] = round(df_income_statement_10k_final.loc['Weighted Average Number of Shares Outstanding, Diluted', '2021-12-31']/1000000, 2)
df_income_statement_10k_final.loc['Weighted Average Number of Shares Outstanding, Diluted', '2022-12-31'] = round(df_income_statement_10k_final.loc['Weighted Average Number of Shares Outstanding, Diluted', '2022-12-31']/1000000, 2)

df_income_statement_10k_final

In [ ]:
# Fazendo o scraping do cash flow statement (10-K)
cash_flow_statement_lst = []

for acc_num in df_lst_acc_num_nrg['acc_num']:

    # Cash flow statement
    cash_flow_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='cash_flow_statement')
    cash_flow_statement = f_sec.rename_statement(cash_flow_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    cash_flow_statement = cash_flow_statement.iloc[:, 0]
    cash_flow_statement_lst.append(cash_flow_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_cash_flow_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in cash_flow_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_cash_flow_10k = pd.concat([df_cash_flow_10k, df], axis=1)

# Substituindo os NaN por 0
df_cash_flow_10k = df_cash_flow_10k.fillna(0)

# A linha 'Depreciation, Amortization and Accretion, Net' na coluna '2017-12-31' está zerada
# As informações estão na linha 'Depreciation, Depletion and Amortization'
df_cash_flow_10k.loc['Depreciation, Amortization and Accretion, Net', '2017-12-31'] = df_cash_flow_10k.loc['Depreciation, Depletion and Amortization', '2017-12-31'] 

# A linha 'nrg_AmortizationOfIntangiblesInTheMoneyContractsAndRetirementOfRECs' nas colunas '2021-12-31', '2020-12-31', '2019-12-31', '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'nrg_AmortizationOfIntangiblesAndOutOfMarketContracts'
df_cash_flow_10k.loc['nrg_AmortizationOfIntangiblesInTheMoneyContractsAndRetirementOfRECs', '2021-12-31'] = df_cash_flow_10k.loc['nrg_AmortizationOfIntangiblesAndOutOfMarketContracts', '2021-12-31'] 
df_cash_flow_10k.loc['nrg_AmortizationOfIntangiblesInTheMoneyContractsAndRetirementOfRECs', '2020-12-31'] = df_cash_flow_10k.loc['nrg_AmortizationOfIntangiblesAndOutOfMarketContracts', '2020-12-31'] 
df_cash_flow_10k.loc['nrg_AmortizationOfIntangiblesInTheMoneyContractsAndRetirementOfRECs', '2019-12-31'] = df_cash_flow_10k.loc['nrg_AmortizationOfIntangiblesAndOutOfMarketContracts', '2019-12-31'] 
df_cash_flow_10k.loc['nrg_AmortizationOfIntangiblesInTheMoneyContractsAndRetirementOfRECs', '2018-12-31'] = df_cash_flow_10k.loc['nrg_AmortizationOfIntangiblesAndOutOfMarketContracts', '2018-12-31'] 
df_cash_flow_10k.loc['nrg_AmortizationOfIntangiblesInTheMoneyContractsAndRetirementOfRECs', '2017-12-31'] = df_cash_flow_10k.loc['nrg_AmortizationOfIntangiblesAndOutOfMarketContracts', '2017-12-31'] 

# A linha 'Increase (Decrease) in Derivative Assets and Liabilities' nas colunas '2021-12-31', '2020-12-31', '2019-12-31', '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'nrg_ChangesInDerivatives'
df_cash_flow_10k.loc['Increase (Decrease) in Derivative Assets and Liabilities', '2021-12-31'] = df_cash_flow_10k.loc['nrg_ChangesInDerivatives', '2021-12-31'] 
df_cash_flow_10k.loc['Increase (Decrease) in Derivative Assets and Liabilities', '2020-12-31'] = df_cash_flow_10k.loc['nrg_ChangesInDerivatives', '2020-12-31'] 
df_cash_flow_10k.loc['Increase (Decrease) in Derivative Assets and Liabilities', '2019-12-31'] = df_cash_flow_10k.loc['nrg_ChangesInDerivatives', '2019-12-31'] 
df_cash_flow_10k.loc['Increase (Decrease) in Derivative Assets and Liabilities', '2018-12-31'] = df_cash_flow_10k.loc['nrg_ChangesInDerivatives', '2018-12-31'] 
df_cash_flow_10k.loc['Increase (Decrease) in Derivative Assets and Liabilities', '2017-12-31'] = df_cash_flow_10k.loc['nrg_ChangesInDerivatives', '2017-12-31'] 

# A linha 'Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents' nas colunas '2021-12-31', '2020-12-31', '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations'
df_cash_flow_10k.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2021-12-31'] = df_cash_flow_10k.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations', '2021-12-31'] 
df_cash_flow_10k.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2020-12-31'] = df_cash_flow_10k.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations', '2020-12-31'] 
df_cash_flow_10k.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2018-12-31'] = df_cash_flow_10k.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations', '2018-12-31'] 
df_cash_flow_10k.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2017-12-31'] = df_cash_flow_10k.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations', '2017-12-31'] 

# A linha 'Payments for (Proceeds from) Derivative Instrument, Financing Activities' nas colunas '2020-12-31', '2019-12-31', '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'Proceeds from (Payments for) Other Financing Activities'
df_cash_flow_10k.loc['Payments for (Proceeds from) Derivative Instrument, Financing Activities', '2020-12-31'] = df_cash_flow_10k.loc['Proceeds from (Payments for) Other Financing Activities', '2020-12-31'] 
df_cash_flow_10k.loc['Payments for (Proceeds from) Derivative Instrument, Financing Activities', '2019-12-31'] = df_cash_flow_10k.loc['Proceeds from (Payments for) Other Financing Activities', '2019-12-31'] 
df_cash_flow_10k.loc['Payments for (Proceeds from) Derivative Instrument, Financing Activities', '2018-12-31'] = df_cash_flow_10k.loc['Proceeds from (Payments for) Other Financing Activities', '2018-12-31'] 
df_cash_flow_10k.loc['Payments for (Proceeds from) Derivative Instrument, Financing Activities', '2017-12-31'] = df_cash_flow_10k.loc['Proceeds from (Payments for) Other Financing Activities', '2017-12-31'] 

# A linha 'nrg_IncreaseDecreaseAccruedExpensesAndOtherCurrentLiabilitiesOperating' nas colunas '2019-12-31', '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'nrg_AccruedExpensesAndOtherCurrentLiabilitiesOperating'
df_cash_flow_10k.loc['nrg_IncreaseDecreaseAccruedExpensesAndOtherCurrentLiabilitiesOperating', '2019-12-31'] = df_cash_flow_10k.loc['nrg_AccruedExpensesAndOtherCurrentLiabilitiesOperating', '2019-12-31'] 
df_cash_flow_10k.loc['nrg_IncreaseDecreaseAccruedExpensesAndOtherCurrentLiabilitiesOperating', '2018-12-31'] = df_cash_flow_10k.loc['nrg_AccruedExpensesAndOtherCurrentLiabilitiesOperating', '2018-12-31'] 
df_cash_flow_10k.loc['nrg_IncreaseDecreaseAccruedExpensesAndOtherCurrentLiabilitiesOperating', '2017-12-31'] = df_cash_flow_10k.loc['nrg_AccruedExpensesAndOtherCurrentLiabilitiesOperating', '2017-12-31'] 

# A linha 'nrg_PaymentsToAcquireAssetsAndBusinessesNetOfCashAcquired' nas colunas '2019-12-31', '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'Payments to Acquire Businesses, Net of Cash Acquired'
df_cash_flow_10k.loc['nrg_PaymentsToAcquireAssetsAndBusinessesNetOfCashAcquired', '2019-12-31'] = df_cash_flow_10k.loc['Payments to Acquire Businesses, Net of Cash Acquired', '2019-12-31'] 
df_cash_flow_10k.loc['nrg_PaymentsToAcquireAssetsAndBusinessesNetOfCashAcquired', '2018-12-31'] = df_cash_flow_10k.loc['Payments to Acquire Businesses, Net of Cash Acquired', '2018-12-31'] 
df_cash_flow_10k.loc['nrg_PaymentsToAcquireAssetsAndBusinessesNetOfCashAcquired', '2017-12-31'] = df_cash_flow_10k.loc['Payments to Acquire Businesses, Net of Cash Acquired', '2017-12-31']

# A linha 'Proceeds from Issuance of Senior Long-term Debt' nas colunas '2019-12-31', '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão nas linhas 'Proceeds from Other Debt' e 'Proceeds from Issuance of Long-term Debt'
df_cash_flow_10k.loc['Proceeds from Issuance of Senior Long-term Debt', '2019-12-31'] = df_cash_flow_10k.loc['Proceeds from Other Debt', '2019-12-31'] 
df_cash_flow_10k.loc['Proceeds from Issuance of Senior Long-term Debt', '2018-12-31'] = df_cash_flow_10k.loc['Proceeds from Issuance of Long-term Debt', '2018-12-31'] 
df_cash_flow_10k.loc['Proceeds from Issuance of Senior Long-term Debt', '2017-12-31'] = df_cash_flow_10k.loc['Proceeds from Issuance of Long-term Debt', '2017-12-31']

# A linha 'nrg_AssetImpairmentChargesIncludingEquityMethodInvestments' nas colunas '2019-12-31', '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'nrg_ImpairmentChargesAndAssetWriteDowns'
df_cash_flow_10k.loc['nrg_AssetImpairmentChargesIncludingEquityMethodInvestments', '2019-12-31'] = df_cash_flow_10k.loc['nrg_ImpairmentChargesAndAssetWriteDowns', '2019-12-31'] 
df_cash_flow_10k.loc['nrg_AssetImpairmentChargesIncludingEquityMethodInvestments', '2018-12-31'] = df_cash_flow_10k.loc['nrg_ImpairmentChargesAndAssetWriteDowns', '2018-12-31'] 
df_cash_flow_10k.loc['nrg_AssetImpairmentChargesIncludingEquityMethodInvestments', '2017-12-31'] = df_cash_flow_10k.loc['nrg_ImpairmentChargesAndAssetWriteDowns', '2017-12-31']

# A linha 'Proceeds from Sale of Other Productive Assets' nas colunas '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'Payments for (Proceeds from) Productive Assets'
df_cash_flow_10k.loc['Proceeds from Sale of Other Productive Assets', '2018-12-31'] = df_cash_flow_10k.loc['Payments for (Proceeds from) Productive Assets', '2018-12-31'] 
df_cash_flow_10k.loc['Proceeds from Sale of Other Productive Assets', '2017-12-31'] = df_cash_flow_10k.loc['Payments for (Proceeds from) Productive Assets', '2017-12-31'] 

# A linha 'Payments to Acquire Debt Securities, Available-for-sale' nas colunas '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'Payments to Acquire Available-for-sale Securities'
df_cash_flow_10k.loc['Payments to Acquire Debt Securities, Available-for-sale', '2018-12-31'] = df_cash_flow_10k.loc['Payments to Acquire Available-for-sale Securities', '2018-12-31'] 
df_cash_flow_10k.loc['Payments to Acquire Debt Securities, Available-for-sale', '2017-12-31'] = df_cash_flow_10k.loc['Payments to Acquire Available-for-sale Securities', '2017-12-31'] 

# A linha 'Proceeds from Sale of Debt Securities, Available-for-sale' nas colunas '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'Proceeds from Sale of Available-for-sale Securities'
df_cash_flow_10k.loc['Proceeds from Sale of Debt Securities, Available-for-sale', '2018-12-31'] = df_cash_flow_10k.loc['Proceeds from Sale of Available-for-sale Securities', '2018-12-31'] 
df_cash_flow_10k.loc['Proceeds from Sale of Debt Securities, Available-for-sale', '2017-12-31'] = df_cash_flow_10k.loc['Proceeds from Sale of Available-for-sale Securities', '2017-12-31'] 

# A linha 'Repayments of Other Debt' nas colunas '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'Proceeds from (Repayments of) Debt'
df_cash_flow_10k.loc['Repayments of Other Debt', '2018-12-31'] = df_cash_flow_10k.loc['Proceeds from (Repayments of) Debt', '2018-12-31'] 
df_cash_flow_10k.loc['Repayments of Other Debt', '2017-12-31'] = df_cash_flow_10k.loc['Proceeds from (Repayments of) Debt', '2017-12-31'] 

# A linha 'Gain (Loss) on Disposition of Assets' nas colunas '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'Gain (Loss) on Disposition of Other Assets'
df_cash_flow_10k.loc['Gain (Loss) on Disposition of Assets', '2018-12-31'] = df_cash_flow_10k.loc['Gain (Loss) on Disposition of Other Assets', '2018-12-31'] 
df_cash_flow_10k.loc['Gain (Loss) on Disposition of Assets', '2017-12-31'] = df_cash_flow_10k.loc['Gain (Loss) on Disposition of Other Assets', '2017-12-31'] 

# A linha 'nrg_ProceedsReceivedReceivableFromGovernmentGrant' na coluna '2021-12-31' está zerada
# As informações estão na linha 'nrg_GainLossOnStormSecuritization'
df_cash_flow_10k.loc['nrg_ProceedsReceivedReceivableFromGovernmentGrant', '2021-12-31'] = df_cash_flow_10k.loc['nrg_GainLossOnStormSecuritization', '2021-12-31'] 

# Removendo as linhas que movemos os seus itens para as linhas corretas
df_cash_flow_10k = df_cash_flow_10k.drop([
    'nrg_AmortizationOfIntangiblesAndOutOfMarketContracts',
    'nrg_ChangesInDerivatives',
    'Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations',
    'Proceeds from (Payments for) Other Financing Activities',
    'Depreciation, Depletion and Amortization',
    'nrg_AccruedExpensesAndOtherCurrentLiabilitiesOperating',
    'Payments to Acquire Businesses, Net of Cash Acquired',
    'Proceeds from Other Debt',
    'Proceeds from Issuance of Long-term Debt',
    'nrg_ImpairmentChargesAndAssetWriteDowns',
    'Payments for (Proceeds from) Productive Assets',
    'Payments to Acquire Available-for-sale Securities',
    'Proceeds from Sale of Available-for-sale Securities',
    'Proceeds from (Repayments of) Debt',
    'Gain (Loss) on Disposition of Other Assets',
    'nrg_GainLossOnStormSecuritization',
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_cash_flow_10k = df_cash_flow_10k.drop([
    'us-gaap_NetCashProvidedByUsedInOperatingActivitiesAbstract',
    'Payments of Distributions to Affiliates',
    'Payments for Repurchase of Redeemable Preferred Stock',
    'Defined Benefit Plan, Net Periodic Benefit Cost (Credit), Gain (Loss) Due to Curtailment',
    'nrg_Proceedsfromsaleofnoncontrollinginterestsinsubsidiaries',
    'Net Cash Provided by (Used in) Operating Activities, Continuing Operations',
    'Cash Provided by (Used in) Operating Activities, Discontinued Operations',
    'Net Cash Provided by (Used in) Investing Activities, Continuing Operations',	
    'Cash Provided by (Used in) Investing Activities, Discontinued Operations',	
    'Net Cash Provided by (Used in) Financing Activities, Continuing Operations',	
    'Cash Provided by (Used in) Financing Activities, Discontinued Operations',
    'Net Cash Provided by (Used in) Discontinued Operations',
    'nrg_OilInventoryLowerOfCostOrMarketAdjustment',
    'Proceeds from Issuance of Preferred Stock and Preference Stock',
    'nrg_ProceedsFromRenewableEnergyGrants',
    'Gain (Loss) on Disposition of Intangible Assets',
    'Increase (Decrease) in Notes Receivables',
    'Payments to Acquire Intangible Assets',
    'nrg_PaymentsToSettleAcquiredDerivativesThatIncludeFinancingElements',
    'Deconsolidation, Gain (Loss), Amount',
    'Cash Divested from Deconsolidation',
    'Nondebtor Reorganization Items, Net (Gain) Loss on Settlement of Other Claims',
    'Proceeds from (Repayments of) Lines of Credit'
])

# Renomeando a linha do índice que está sem formato 
df_cash_flow_10k = df_cash_flow_10k.rename(index={
    'nrg_AmortizationOfNuclearFuel': 'Amortization Of Nuclear Fuel',
    'nrg_GainsLossesOnExtinguishmentOfDebtNonCashPortion': 'Gains Losses On Extinguishment Of Debt Non Cash Portion',
    'nrg_AmortizationOfIntangiblesInTheMoneyContractsAndRetirementOfRECs': 'Amortization Of Intangibles In The Money Contracts And Retirement Of RECs',
    'nrg_AssetImpairmentChargesIncludingEquityMethodInvestments': 'Asset Impairment Charges Including Equity Method Investments',
    'nrg_ChangesInCollateralDepositsSupportingEnergyRiskManagementActivities': 'Changes In Collateral Deposits Supporting Energy Risk Management Activities',
    'nrg_ChangesInNuclearDecommissioningTrustLiability': 'Changes In Nuclear Decommissioning Trust Liability',
    'nrg_ProceedsReceivedReceivableFromGovernmentGrant': 'Proceeds Received Receivable From Government Grant',
    'nrg_IncreaseDecreaseAccruedExpensesAndOtherCurrentLiabilitiesOperating': 'Increase (Decrease) Accrued Expenses And Other Current Liabilities Operating',
    'nrg_PaymentsToAcquireAssetsAndBusinessesNetOfCashAcquired': 'Payments To Acquire Assets And Businesses Net Of Cash Acquired',
    'nrg_PaymentsForProceedsFromIntangibleAssets': 'Payments For Proceeds From Intangible Assets',
    'nrg_ProceedsFromInsuranceRecoveriesForPropertyPlantAndEquipmentNet': 'Proceeds From Insurance Recoveries For Property Plant And Equipment Net',
    'nrg_InvestmentsInProceedsFromSalesOfUnconsolidatedAffiliates': 'Investments In Proceeds From Sales Of Unconsolidated Affiliates',
    'nrg_NetContributionsToDistributionsFromDiscontinuedOperations': 'Net Contributions To Distributions From Discontinued Operations'
})

df_cash_flow_10k

In [ ]:
# Arrumando alguns index dos dfs para ficar padronizado com os index utilizados nas funções p/ calcular os indicadores fundamentalistas
df_balance_sheet_10k = df_balance_sheet_10k.rename(index={
    "Stockholders' Equity, Including Portion Attributable to Noncontrolling Interest": 'Total Shareholders Equity',
})

df_income_statement_10k_final = df_income_statement_10k_final.rename(index={
    'Research and Development Expense': 'Research and Development',
    'Operating Income (Loss)': 'Operating Income',
    'Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest': 'Income Before Provision for Income Taxes',
    'Income Tax Expense (Benefit)': 'Provision for Income Taxes',
    'Net Income (Loss) Attributable to Parent': 'Net Income',
    'Weighted Average Number of Shares Outstanding, Basic': 'Number of Shares - Basic',
    'Weighted Average Number of Shares Outstanding, Diluted': 'Number of Shares - Diluted'
})

df_cash_flow_10k = df_cash_flow_10k.rename(index={
    'Depreciation, Amortization and Accretion, Net': 'Depreciation, Depletion and Amortization',
    'Payments of Ordinary Dividends, Common Stock': 'Payments of Dividends',
})

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//energia_eletrica//nrg//nrg_10k.xlsx') as writer:
    df_balance_sheet_10k.to_excel(writer, sheet_name='balance_sheet')
    df_income_statement_10k_final.to_excel(writer, sheet_name='income_statement')
    df_cash_flow_10k.to_excel(writer, sheet_name='cash_flow')

### Scraping NRG 10-Q

In [ ]:
# Número dos documentos que contém a DRE, BP e FC 
acc_documents_10q = f_sec.get_filtered_filings(ticker=ticker, ten_k=False, just_accession_numbers=True)

# Trocando os '-' para '' das strings ('0001045810-24-000264' -> '000104581024000264')
dict_acc_num_10q = {k: v.replace('-', '') for k, v in acc_documents_10q.items()}

# Transformando o dicionário em um df
df_lst_acc_num_10q_nrg = pd.DataFrame.from_dict(dict_acc_num_10q, orient='index', columns=['acc_num'])

# Transformando em datetime o index
df_lst_acc_num_10q_nrg.index = pd.to_datetime(df_lst_acc_num_10q_nrg.index, format='%Y-%m-%d')

df_lst_acc_num_10q_nrg

In [ ]:
# Selecionando os 'acc_num' dos 10-Q a partir de '2020-03-31'
df_lst_acc_num_10q_nrg_sliced = df_lst_acc_num_10q_nrg.loc['2020-03-31':]
df_lst_acc_num_10q_nrg_sliced

In [ ]:
# Rótulos
labels_dict = f_sec.get_label_dictionary(ticker, headers=f_sec.header)

In [ ]:
# Fazendo o scraping do balance sheet (10-Q)
balance_sheet_lst = []

for acc_num in df_lst_acc_num_10q_nrg_sliced['acc_num']:

    # Balance sheet
    balance_sheet = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='balance_sheet')
    balance_sheet = f_sec.rename_statement(balance_sheet, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    balance_sheet = balance_sheet.iloc[:, 0]
    balance_sheet_lst.append(balance_sheet)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_balance_sheet_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in balance_sheet_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_balance_sheet_10q = pd.concat([df_balance_sheet_10q, df], axis=1)

# Substituindo os NaN por 0
df_balance_sheet_10q = df_balance_sheet_10q.fillna(0)

# A linha 'Long-term Debt and Lease Obligation, Current' nas colunas '2020-03-31', '2020-06-30' e '2020-09-30' estão zeradas
# As informações estão na linha 'Long-term Debt, Current Maturities'
df_balance_sheet_10q.loc['Long-term Debt and Lease Obligation, Current', '2020-09-30'] = df_balance_sheet_10q.loc['Long-term Debt, Current Maturities', '2020-09-30'] 
df_balance_sheet_10q.loc['Long-term Debt and Lease Obligation, Current', '2020-06-30'] = df_balance_sheet_10q.loc['Long-term Debt, Current Maturities', '2020-06-30'] 
df_balance_sheet_10q.loc['Long-term Debt and Lease Obligation, Current', '2020-03-31'] = df_balance_sheet_10q.loc['Long-term Debt, Current Maturities', '2020-03-31'] 

# A linha 'Long-term Debt and Lease Obligation' nas colunas '2020-03-31', '2020-06-30' e '2020-09-30' estão zeradas
# As informações estão na linha 'Long-term Debt, Excluding Current Maturities'
df_balance_sheet_10q.loc['Long-term Debt and Lease Obligation', '2020-09-30'] = df_balance_sheet_10q.loc['Long-term Debt, Excluding Current Maturities', '2020-09-30'] 
df_balance_sheet_10q.loc['Long-term Debt and Lease Obligation', '2020-06-30'] = df_balance_sheet_10q.loc['Long-term Debt, Excluding Current Maturities', '2020-06-30'] 
df_balance_sheet_10q.loc['Long-term Debt and Lease Obligation', '2020-03-31'] = df_balance_sheet_10q.loc['Long-term Debt, Excluding Current Maturities', '2020-03-31'] 

# # Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_balance_sheet_10q = df_balance_sheet_10q.drop('Long-term Debt, Current Maturities')
df_balance_sheet_10q = df_balance_sheet_10q.drop('Long-term Debt, Excluding Current Maturities')

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_balance_sheet_10q = df_balance_sheet_10q.drop([
    'us-gaap_AssetsCurrentAbstract',
    'Common Stock, Shares Authorized',
    'Common Stock, Par or Stated Value Per Share',
    'Disposal Group, Including Discontinued Operation, Liabilities, Noncurrent',
    'Treasury Stock, Common, Shares',
    'nrg_ReceivableFromGovernmentGrant',
    'Redeemable Noncontrolling Interest, Equity, Other, Carrying Amount',
    'Disposal Group, Including Discontinued Operation, Assets, Noncurrent',
    'Disposal Group, Including Discontinued Operation, Liabilities, Current',
    'Disposal Group, Including Discontinued Operation, Assets, Current'
])

# Renomeando algumas linhas do índice
df_balance_sheet_10q = df_balance_sheet_10q.rename(index={
    'nrg_FundsDepositedByCounterparties': 'Funds Deposited By Counterparties',
    'nrg_NuclearDecommissioningTrustFundLiabilityNoncurrent': 'Nuclear Decommissioning Trust Fund Liability Noncurrent',
})

df_balance_sheet_10q

In [ ]:
# Fazendo o scraping do income statement (10-Q)
income_statement_lst = []

for acc_num in df_lst_acc_num_10q_nrg_sliced['acc_num']:

    # Income statement
    income_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='income_statement')
    income_statement = f_sec.rename_statement(income_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    income_statement = income_statement.iloc[:, 0]
    income_statement_lst.append(income_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_income_statement_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in income_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_income_statement_10q = pd.concat([df_income_statement_10q, df], axis=1)

# Substituindo os NaN por 0
df_income_statement_10q = df_income_statement_10q.fillna(0)

# As linhas 'InterestExpenseNonoperating' nas colunas '2024-03-31', '2023-09-30', '2023-06-30', '2023-03-31', '2022-09-30', '2022-06-30', '2022-03-31', '2021-09-30', '2021-06-30', '2021-03-31', '2020-09-30', '2020-06-30' e '2020-03-31' estão zeradas
# As informações estão na linha 'Interest Expense'
df_income_statement_10q.loc['InterestExpenseNonoperating', '2024-03-31'] = df_income_statement_10q.loc['Interest Expense', '2024-03-31']
df_income_statement_10q.loc['InterestExpenseNonoperating', '2023-09-30'] = df_income_statement_10q.loc['Interest Expense', '2023-09-30']
df_income_statement_10q.loc['InterestExpenseNonoperating', '2023-06-30'] = df_income_statement_10q.loc['Interest Expense', '2023-06-30']
df_income_statement_10q.loc['InterestExpenseNonoperating', '2023-03-31'] = df_income_statement_10q.loc['Interest Expense', '2023-03-31']
df_income_statement_10q.loc['InterestExpenseNonoperating', '2022-09-30'] = df_income_statement_10q.loc['Interest Expense', '2022-09-30']
df_income_statement_10q.loc['InterestExpenseNonoperating', '2022-06-30'] = df_income_statement_10q.loc['Interest Expense', '2022-06-30']
df_income_statement_10q.loc['InterestExpenseNonoperating', '2022-03-31'] = df_income_statement_10q.loc['Interest Expense', '2022-03-31']
df_income_statement_10q.loc['InterestExpenseNonoperating', '2021-09-30'] = df_income_statement_10q.loc['Interest Expense', '2021-09-30']
df_income_statement_10q.loc['InterestExpenseNonoperating', '2021-06-30'] = df_income_statement_10q.loc['Interest Expense', '2021-06-30']
df_income_statement_10q.loc['InterestExpenseNonoperating', '2021-03-31'] = df_income_statement_10q.loc['Interest Expense', '2021-03-31']
df_income_statement_10q.loc['InterestExpenseNonoperating', '2020-09-30'] = df_income_statement_10q.loc['Interest Expense', '2020-09-30']
df_income_statement_10q.loc['InterestExpenseNonoperating', '2020-06-30'] = df_income_statement_10q.loc['Interest Expense', '2020-06-30']
df_income_statement_10q.loc['InterestExpenseNonoperating', '2020-03-31'] = df_income_statement_10q.loc['Interest Expense', '2020-03-31']

# A linha 'Cost of Goods and Service, Excluding Depreciation, Depletion, and Amortization' nas colunas '2021-06-30', '2021-03-31', '2020-09-30', '2020-06-30' e '2020-03-31' estão zeradas
# As informações estão na linha 'Cost of Revenue'
df_income_statement_10q.loc['Cost of Goods and Service, Excluding Depreciation, Depletion, and Amortization', '2021-06-30'] = df_income_statement_10q.loc['Cost of Revenue', '2021-06-30'] 
df_income_statement_10q.loc['Cost of Goods and Service, Excluding Depreciation, Depletion, and Amortization', '2021-03-31'] = df_income_statement_10q.loc['Cost of Revenue', '2021-03-31'] 
df_income_statement_10q.loc['Cost of Goods and Service, Excluding Depreciation, Depletion, and Amortization', '2020-09-30'] = df_income_statement_10q.loc['Cost of Revenue', '2020-09-30'] 
df_income_statement_10q.loc['Cost of Goods and Service, Excluding Depreciation, Depletion, and Amortization', '2020-06-30'] = df_income_statement_10q.loc['Cost of Revenue', '2020-06-30'] 
df_income_statement_10q.loc['Cost of Goods and Service, Excluding Depreciation, Depletion, and Amortization', '2020-03-31'] = df_income_statement_10q.loc['Cost of Revenue', '2020-03-31'] 

# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_income_statement_10q = df_income_statement_10q.drop([
    'Cost of Revenue',
    'Interest Expense'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_income_statement_10q = df_income_statement_10q.drop([
    'us-gaap_RevenuesAbstract',
    'Discontinued Operation, Income (Loss) from Discontinued Operation, Net of Tax, Per Diluted Share',
    'Discontinued Operation, Income (Loss) from Discontinued Operation, Net of Tax, Per Basic Share',
    'Income (Loss) from Discontinued Operations, Net of Tax, Attributable to Parent',
    'Income (Loss) from Continuing Operations, Per Basic Share',
    'Income (Loss) from Continuing Operations, Per Diluted Share',
    'Net Income (Loss) Attributable to Noncontrolling Interest',
    'Other than Temporary Impairment Losses, Investments',
    'Restructuring Costs',
    'Income (Loss) from Continuing Operations, Net of Tax, Including Portion Attributable to Noncontrolling Interest'
])

# Arrumando e adicionando alguns números no df
df_income_statement_10q.loc['Net Income (Loss) Attributable to Parent', '2021-03-31'] = -82
df_income_statement_10q.loc['Net Income (Loss) Attributable to Parent', '2021-06-30'] = 1078
df_income_statement_10q.loc['Net Income (Loss) Attributable to Parent', '2021-09-30'] = 1618
df_income_statement_10q.loc['Net Income (Loss) Attributable to Parent', '2022-03-31'] = 1736
df_income_statement_10q.loc['Net Income (Loss) Attributable to Parent', '2022-06-30'] = 513
df_income_statement_10q.loc['Net Income (Loss) Attributable to Parent', '2022-09-30'] = 67
df_income_statement_10q.loc['Net Income (Loss) Attributable to Parent', '2023-03-31'] = -1335
df_income_statement_10q.loc['Net Income (Loss) Attributable to Parent', '2023-06-30'] = 308
df_income_statement_10q.loc['Net Income (Loss) Attributable to Parent', '2023-09-30'] = 343
df_income_statement_10q.loc['Net Income (Loss) Attributable to Parent', '2024-03-31'] = 511
df_income_statement_10q.loc['Net Income (Loss) Attributable to Parent', '2024-06-30'] = 738
df_income_statement_10q.loc['Net Income (Loss) Attributable to Parent', '2024-09-30'] = -767
df_income_statement_10q.loc['Net Income (Loss) Attributable to Parent', '2025-03-31'] = 750
df_income_statement_10q.loc['Net Income (Loss) Attributable to Parent', '2025-06-30'] = -104
df_income_statement_10q.loc['Net Income (Loss) Attributable to Parent', '2025-09-30'] = 152

df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Diluted', '2022-03-31'] = 242
df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Diluted', '2022-06-30'] = 237
df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Diluted', '2022-09-30'] = 235
df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Diluted', '2023-03-31'] = 230

df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Basic', '2022-06-30'] = round(df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Basic', '2022-06-30'] / 1000, 0)
df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Basic', '2022-09-30'] = round(df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Basic', '2022-09-30'] / 1000, 0)
df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Basic', '2023-06-30'] = round(df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Basic', '2023-06-30'] / 1000, 0)
df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Basic', '2023-03-31'] = round(df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Basic', '2023-03-31'] / 1000, 0)
df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Basic', '2023-09-30'] = round(df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Basic', '2023-09-30'] / 1000, 0)
df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Basic', '2024-03-31'] = round(df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Basic', '2024-03-31'] / 1000, 0)
df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Basic', '2024-06-30'] = round(df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Basic', '2024-06-30'] / 1000, 0)
df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Basic', '2024-09-30'] = round(df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Basic', '2024-09-30'] / 1000, 0)
df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Basic', '2025-03-31'] = round(df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Basic', '2025-03-31'] / 1000, 0)
df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Basic', '2025-06-30'] = round(df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Basic', '2025-06-30'] / 1000, 0)
df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Basic', '2025-09-30'] = round(df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Basic', '2025-09-30'] / 1000, 0)

df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Diluted', '2023-06-30'] = round(df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Diluted', '2023-06-30'] / 1000, 0)
df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Diluted', '2023-09-30'] = round(df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Diluted', '2023-09-30'] / 1000, 0)
df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Diluted', '2024-03-31'] = round(df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Diluted', '2024-03-31'] / 1000, 0)
df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Diluted', '2024-06-30'] = round(df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Diluted', '2024-06-30'] / 1000, 0)
df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Diluted', '2024-09-30'] = round(df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Diluted', '2024-09-30'] / 1000, 0)
df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Diluted', '2025-03-31'] = round(df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Diluted', '2025-03-31'] / 1000, 0)
df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Diluted', '2025-06-30'] = round(df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Diluted', '2025-06-30'] / 1000, 0)
df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Diluted', '2025-09-30'] = round(df_income_statement_10q.loc['Weighted Average Number of Shares Outstanding, Diluted', '2025-09-30'] / 1000, 0)

# Renomeando algumas linhas do índice
df_income_statement_10q = df_income_statement_10q.rename(index={
    'nrg_BusinessCombinationAcquisitionAndIntegrationCosts': 'Business Combination Acquisition And Integration Costs',
})

df_income_statement_10q

In [ ]:
# Fazendo o scraping do cash flow statement (10-Q)
cash_flow_statement_lst = []

for acc_num in df_lst_acc_num_10q_nrg_sliced['acc_num']:

    # Cash flow statement
    cash_flow_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='cash_flow_statement')
    cash_flow_statement = f_sec.rename_statement(cash_flow_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    cash_flow_statement = cash_flow_statement.iloc[:, 0]
    cash_flow_statement_lst.append(cash_flow_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_cash_flow_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in cash_flow_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_cash_flow_10q = pd.concat([df_cash_flow_10q, df], axis=1)

# Substituindo os NaN por 0
df_cash_flow_10q = df_cash_flow_10q.fillna(0)

# A linha 'Payments to Acquire Other Productive Assets' nas colunas '2024-09-30', '2024-06-30', '2024-03-31', '2023-09-30', '2023-06-30', '2023-03-31', '2022-09-30', '2022-06-30', '2022-03-31', '2021-09-30', '2021-06-30', '2021-03-31' e '2020-06-30' estão zeradas
# As informações estão nas linhas 'Payments to Acquire Businesses, Net of Cash Acquired' e 'nrg_PaymentsToAcquireAssetsAndBusinessesNetOfCashAcquired'
df_cash_flow_10q.loc['Payments to Acquire Other Productive Assets','2024-09-30'] = df_cash_flow_10q.loc['Payments to Acquire Businesses, Net of Cash Acquired', '2024-09-30']
df_cash_flow_10q.loc['Payments to Acquire Other Productive Assets','2024-06-30'] = df_cash_flow_10q.loc['Payments to Acquire Businesses, Net of Cash Acquired', '2024-06-30']
df_cash_flow_10q.loc['Payments to Acquire Other Productive Assets','2024-03-31'] = df_cash_flow_10q.loc['Payments to Acquire Businesses, Net of Cash Acquired', '2024-03-31']
df_cash_flow_10q.loc['Payments to Acquire Other Productive Assets','2023-09-30'] = df_cash_flow_10q.loc['Payments to Acquire Businesses, Net of Cash Acquired', '2023-09-30']
df_cash_flow_10q.loc['Payments to Acquire Other Productive Assets','2023-06-30'] = df_cash_flow_10q.loc['Payments to Acquire Businesses, Net of Cash Acquired', '2023-06-30']
df_cash_flow_10q.loc['Payments to Acquire Other Productive Assets','2023-03-31'] = df_cash_flow_10q.loc['Payments to Acquire Businesses, Net of Cash Acquired', '2023-03-31']
df_cash_flow_10q.loc['Payments to Acquire Other Productive Assets','2022-09-30'] = df_cash_flow_10q.loc['Payments to Acquire Businesses, Net of Cash Acquired', '2022-09-30']
df_cash_flow_10q.loc['Payments to Acquire Other Productive Assets','2022-06-30'] = df_cash_flow_10q.loc['Payments to Acquire Businesses, Net of Cash Acquired', '2022-06-30']
df_cash_flow_10q.loc['Payments to Acquire Other Productive Assets','2022-03-31'] = df_cash_flow_10q.loc['Payments to Acquire Businesses, Net of Cash Acquired', '2022-03-31']
df_cash_flow_10q.loc['Payments to Acquire Other Productive Assets','2021-09-30'] = df_cash_flow_10q.loc['Payments to Acquire Businesses, Net of Cash Acquired', '2021-09-30']
df_cash_flow_10q.loc['Payments to Acquire Other Productive Assets','2021-06-30'] = df_cash_flow_10q.loc['Payments to Acquire Businesses, Net of Cash Acquired', '2021-06-30']
df_cash_flow_10q.loc['Payments to Acquire Other Productive Assets','2021-03-31'] = df_cash_flow_10q.loc['Payments to Acquire Businesses, Net of Cash Acquired', '2021-03-31']
df_cash_flow_10q.loc['Payments to Acquire Other Productive Assets','2020-09-30'] = df_cash_flow_10q.loc['nrg_PaymentsToAcquireAssetsAndBusinessesNetOfCashAcquired', '2020-09-30']

# A linha 'nrg_GainLossOnSaleAndDispositionOfAssets' nas colunas '2024-09-30', '2024-06-30', '2024-03-31', '2023-09-30', '2023-06-30', '2023-03-31', '2022-09-30', '2022-06-30', '2022-03-31', '2021-09-30', '2021-06-30', '2021-03-31', '2020-09-30', '2020-06-30' e '2020-03-31' estão zeradas
# As informações estão na linha 'Gain (Loss) on Disposition of Assets'
df_cash_flow_10q.loc['nrg_GainLossOnSaleAndDispositionOfAssets', '2024-09-30'] = df_cash_flow_10q.loc['Gain (Loss) on Disposition of Assets', '2024-09-30']
df_cash_flow_10q.loc['nrg_GainLossOnSaleAndDispositionOfAssets', '2024-06-30'] = df_cash_flow_10q.loc['Gain (Loss) on Disposition of Assets', '2024-06-30']
df_cash_flow_10q.loc['nrg_GainLossOnSaleAndDispositionOfAssets', '2024-03-31'] = df_cash_flow_10q.loc['Gain (Loss) on Disposition of Assets', '2024-03-31']
df_cash_flow_10q.loc['nrg_GainLossOnSaleAndDispositionOfAssets', '2023-09-30'] = df_cash_flow_10q.loc['Gain (Loss) on Disposition of Assets', '2023-09-30']
df_cash_flow_10q.loc['nrg_GainLossOnSaleAndDispositionOfAssets', '2023-06-30'] = df_cash_flow_10q.loc['Gain (Loss) on Disposition of Assets', '2023-06-30']
df_cash_flow_10q.loc['nrg_GainLossOnSaleAndDispositionOfAssets', '2023-03-31'] = df_cash_flow_10q.loc['Gain (Loss) on Disposition of Assets', '2023-03-31']
df_cash_flow_10q.loc['nrg_GainLossOnSaleAndDispositionOfAssets', '2022-09-30'] = df_cash_flow_10q.loc['Gain (Loss) on Disposition of Assets', '2022-09-30']
df_cash_flow_10q.loc['nrg_GainLossOnSaleAndDispositionOfAssets', '2022-06-30'] = df_cash_flow_10q.loc['Gain (Loss) on Disposition of Assets', '2022-06-30']
df_cash_flow_10q.loc['nrg_GainLossOnSaleAndDispositionOfAssets', '2022-03-31'] = df_cash_flow_10q.loc['Gain (Loss) on Disposition of Assets', '2022-03-31']
df_cash_flow_10q.loc['nrg_GainLossOnSaleAndDispositionOfAssets', '2021-09-30'] = df_cash_flow_10q.loc['Gain (Loss) on Disposition of Assets', '2021-09-30']
df_cash_flow_10q.loc['nrg_GainLossOnSaleAndDispositionOfAssets', '2021-06-30'] = df_cash_flow_10q.loc['Gain (Loss) on Disposition of Assets', '2021-06-30']
df_cash_flow_10q.loc['nrg_GainLossOnSaleAndDispositionOfAssets', '2021-03-31'] = df_cash_flow_10q.loc['Gain (Loss) on Disposition of Assets', '2021-03-31']
df_cash_flow_10q.loc['nrg_GainLossOnSaleAndDispositionOfAssets', '2020-09-30'] = df_cash_flow_10q.loc['Gain (Loss) on Disposition of Assets', '2020-09-30']
df_cash_flow_10q.loc['nrg_GainLossOnSaleAndDispositionOfAssets', '2020-06-30'] = df_cash_flow_10q.loc['Gain (Loss) on Disposition of Assets', '2020-06-30']
df_cash_flow_10q.loc['nrg_GainLossOnSaleAndDispositionOfAssets', '2020-03-31'] = df_cash_flow_10q.loc['Gain (Loss) on Disposition of Assets', '2020-03-31']

# A linha 'Payments of Ordinary Dividends' nas colunas '2023-06-30', '2023-03-31', '2022-09-30', '2022-06-30', '2022-03-31', '2021-09-30', '2021-06-30', '2021-03-31', '2020-09-30', '2020-06-30' e '2020-03-31' estão zeradas
# As informações estão nas linhas 'Payments of Ordinary Dividends, Common Stock' e 'Payments of Ordinary Dividends, Preferred Stock and Preference Stock'
df_cash_flow_10q.loc['Payments of Ordinary Dividends','2023-06-30'] = df_cash_flow_10q.loc['Payments of Ordinary Dividends, Common Stock','2023-06-30']   
df_cash_flow_10q.loc['Payments of Ordinary Dividends','2023-03-31'] = df_cash_flow_10q.loc['Payments of Ordinary Dividends, Common Stock','2023-03-31']   
df_cash_flow_10q.loc['Payments of Ordinary Dividends','2022-09-30'] = df_cash_flow_10q.loc['Payments of Ordinary Dividends, Common Stock','2022-09-30']  
df_cash_flow_10q.loc['Payments of Ordinary Dividends','2022-06-30'] = df_cash_flow_10q.loc['Payments of Ordinary Dividends, Common Stock','2022-06-30']  
df_cash_flow_10q.loc['Payments of Ordinary Dividends','2022-03-31'] = df_cash_flow_10q.loc['Payments of Ordinary Dividends, Common Stock','2022-03-31']    
df_cash_flow_10q.loc['Payments of Ordinary Dividends','2021-09-30'] = df_cash_flow_10q.loc['Payments of Ordinary Dividends, Common Stock','2021-09-30']   
df_cash_flow_10q.loc['Payments of Ordinary Dividends','2021-06-30'] = df_cash_flow_10q.loc['Payments of Ordinary Dividends, Common Stock','2021-06-30']   
df_cash_flow_10q.loc['Payments of Ordinary Dividends','2021-03-31'] = df_cash_flow_10q.loc['Payments of Ordinary Dividends, Common Stock','2021-03-31']  
df_cash_flow_10q.loc['Payments of Ordinary Dividends','2020-09-30'] = df_cash_flow_10q.loc['Payments of Ordinary Dividends, Preferred Stock and Preference Stock','2020-09-30']   
df_cash_flow_10q.loc['Payments of Ordinary Dividends','2020-06-30'] = df_cash_flow_10q.loc['Payments of Ordinary Dividends, Preferred Stock and Preference Stock','2020-06-30']   
df_cash_flow_10q.loc['Payments of Ordinary Dividends','2020-03-31'] = df_cash_flow_10q.loc['Payments of Ordinary Dividends, Preferred Stock and Preference Stock','2020-03-31']    
  
# A linha 'nrg_GainLossOnAccretionOfAssetRetirementObligations' nas colunas '2024-09-30', '2024-06-30', '2024-03-31', '2023-09-30', '2023-06-30', '2023-03-31', '2022-09-30', '2022-03-31', '2021-09-30', '2021-06-30', '2021-03-31', '2020-09-30', '2020-06-30' e '2020-03-31' estão zeradas
# As informações estão na linha 'Accretion Expense, Including Asset Retirement Obligations'
df_cash_flow_10q.loc['nrg_GainLossOnAccretionOfAssetRetirementObligations', '2024-09-30'] = df_cash_flow_10q.loc['Accretion Expense, Including Asset Retirement Obligations', '2024-09-30']
df_cash_flow_10q.loc['nrg_GainLossOnAccretionOfAssetRetirementObligations', '2024-06-30'] = df_cash_flow_10q.loc['Accretion Expense, Including Asset Retirement Obligations', '2024-06-30']
df_cash_flow_10q.loc['nrg_GainLossOnAccretionOfAssetRetirementObligations', '2024-03-31'] = df_cash_flow_10q.loc['Accretion Expense, Including Asset Retirement Obligations', '2024-03-31']
df_cash_flow_10q.loc['nrg_GainLossOnAccretionOfAssetRetirementObligations', '2023-09-30'] = df_cash_flow_10q.loc['Accretion Expense, Including Asset Retirement Obligations', '2023-09-30']
df_cash_flow_10q.loc['nrg_GainLossOnAccretionOfAssetRetirementObligations', '2023-06-30'] = df_cash_flow_10q.loc['Accretion Expense, Including Asset Retirement Obligations', '2023-06-30']
df_cash_flow_10q.loc['nrg_GainLossOnAccretionOfAssetRetirementObligations', '2023-03-31'] = df_cash_flow_10q.loc['Accretion Expense, Including Asset Retirement Obligations', '2023-03-31']
df_cash_flow_10q.loc['nrg_GainLossOnAccretionOfAssetRetirementObligations', '2022-09-30'] = df_cash_flow_10q.loc['Accretion Expense, Including Asset Retirement Obligations', '2022-09-30'] 
df_cash_flow_10q.loc['nrg_GainLossOnAccretionOfAssetRetirementObligations', '2022-03-31'] = df_cash_flow_10q.loc['Accretion Expense, Including Asset Retirement Obligations', '2022-03-31']
df_cash_flow_10q.loc['nrg_GainLossOnAccretionOfAssetRetirementObligations', '2021-09-30'] = df_cash_flow_10q.loc['Accretion Expense, Including Asset Retirement Obligations', '2021-09-30']
df_cash_flow_10q.loc['nrg_GainLossOnAccretionOfAssetRetirementObligations', '2021-06-30'] = df_cash_flow_10q.loc['Accretion Expense, Including Asset Retirement Obligations', '2021-06-30']
df_cash_flow_10q.loc['nrg_GainLossOnAccretionOfAssetRetirementObligations', '2021-03-31'] = df_cash_flow_10q.loc['Accretion Expense, Including Asset Retirement Obligations', '2021-03-31']
df_cash_flow_10q.loc['nrg_GainLossOnAccretionOfAssetRetirementObligations', '2020-09-30'] = df_cash_flow_10q.loc['Accretion Expense, Including Asset Retirement Obligations', '2020-09-30']
df_cash_flow_10q.loc['nrg_GainLossOnAccretionOfAssetRetirementObligations', '2020-06-30'] = df_cash_flow_10q.loc['Accretion Expense, Including Asset Retirement Obligations', '2020-06-30']
df_cash_flow_10q.loc['nrg_GainLossOnAccretionOfAssetRetirementObligations', '2020-03-31'] = df_cash_flow_10q.loc['Accretion Expense, Including Asset Retirement Obligations', '2020-03-31']

# A linha 'Payments for Repurchase of Common Stock' nas colunas '2020-09-30', '2020-06-30' e '2020-03-31' estão zeradas
# As informações estão na linha 'Payments for Repurchase of Preferred Stock and Preference Stock'
df_cash_flow_10q.loc['Payments for Repurchase of Common Stock', '2020-09-30'] = df_cash_flow_10q.loc['Payments for Repurchase of Preferred Stock and Preference Stock', '2020-09-30']
df_cash_flow_10q.loc['Payments for Repurchase of Common Stock', '2020-06-30'] = df_cash_flow_10q.loc['Payments for Repurchase of Preferred Stock and Preference Stock', '2020-06-30']
df_cash_flow_10q.loc['Payments for Repurchase of Common Stock', '2020-03-31'] = df_cash_flow_10q.loc['Payments for Repurchase of Preferred Stock and Preference Stock', '2020-03-31']

# A linha 'Proceeds from Long-term Lines of Credit' nas colunas '2021-03-31' e '2020-03-31' estão zeradas
# As informações estão na linha 'Proceeds from Lines of Credit'
df_cash_flow_10q.loc['Proceeds from Long-term Lines of Credit', '2021-03-31'] = df_cash_flow_10q.loc['Proceeds from Lines of Credit', '2021-03-31']
df_cash_flow_10q.loc['Proceeds from Long-term Lines of Credit', '2020-03-31'] = df_cash_flow_10q.loc['Proceeds from Lines of Credit', '2020-03-31']

# A linha 'Repayments of Long-term Lines of Credit' nas colunas '2020-09-30' e '2020-06-30' estão zeradas
# As informações estão na linha 'Repayments of Lines of Credit'
df_cash_flow_10q.loc['Repayments of Long-term Lines of Credit', '2020-09-30'] = df_cash_flow_10q.loc['Repayments of Lines of Credit', '2020-09-30']
df_cash_flow_10q.loc['Repayments of Long-term Lines of Credit', '2020-06-30'] = df_cash_flow_10q.loc['Repayments of Lines of Credit', '2020-06-30']

# A linha 'Payments to Acquire Other Productive Assets' na coluna '2025-06-30' está zerada
# As informações estão na linha 'nrg_PaymentsToAcquireBusinessesAndAssetAcquisitionsNet'
df_cash_flow_10q.loc['Payments to Acquire Other Productive Assets', '2025-06-30'] = df_cash_flow_10q.loc['nrg_PaymentsToAcquireBusinessesAndAssetAcquisitionsNet', '2025-06-30']

# A linha 'nrg_GainLossOnAccretionOfAssetRetirementObligations' na coluna '2025-06-30' está zerada
# As informações estão na linha 'Asset Retirement Obligation, Accretion Expense'
df_cash_flow_10q.loc['nrg_GainLossOnAccretionOfAssetRetirementObligations', '2025-06-30'] = df_cash_flow_10q.loc['Asset Retirement Obligation, Accretion Expense', '2025-06-30']


# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_cash_flow_10q = df_cash_flow_10q.drop([
    'Payments to Acquire Businesses, Net of Cash Acquired',
    'nrg_PaymentsToAcquireAssetsAndBusinessesNetOfCashAcquired',
    'Gain (Loss) on Disposition of Assets',
    'Payments of Ordinary Dividends, Common Stock',
    'Payments of Ordinary Dividends, Preferred Stock and Preference Stock',
    'Accretion Expense, Including Asset Retirement Obligations',
    'Payments for Repurchase of Preferred Stock and Preference Stock',
    'Proceeds from Lines of Credit',
    'Repayments of Lines of Credit',
    'nrg_PaymentsToAcquireBusinessesAndAssetAcquisitionsNet',
    'Asset Retirement Obligation, Accretion Expense'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_cash_flow_10q = df_cash_flow_10q.drop([
    'us-gaap_NetCashProvidedByUsedInOperatingActivitiesAbstract',
    'Inventory Write-down',
    'Income (Loss) from Discontinued Operations, Net of Tax, Attributable to Parent',
    'Cash Provided by (Used in) Operating Activities, Discontinued Operations',
    'Proceeds from Collection of Notes Receivable',
    'nrg_NetContributionsToDistributionsFromDiscontinuedOperations',
    'Cash Provided by (Used in) Investing Activities, Discontinued Operations',
    'Cash Provided by (Used in) Financing Activities, Discontinued Operations',
    'Net Cash Provided by (Used in) Discontinued Operations',
    'Payments for (Proceeds from) Other Investing Activities',
    'Income (Loss) from Continuing Operations, Net of Tax, Including Portion Attributable to Noncontrolling Interest',
    'Net Cash Provided by (Used in) Operating Activities, Continuing Operations',
    'nrg_ProceedsFromSaleOfAssetsAndSaleOfDiscontinuedOperationsNet',
    'Net Cash Provided by (Used in) Investing Activities, Continuing Operations',
    'Net Cash Provided by (Used in) Financing Activities, Continuing Operations',
    'nrg_ProceedsFromGovernmentGrant'
])

# Arrumando e adicionando alguns números no df
df_cash_flow_10q.loc['Proceeds from Long-term Lines of Credit', '2023-03-31'] = 1050
df_cash_flow_10q.loc['Proceeds from Long-term Lines of Credit', '2023-06-30'] = 1870
df_cash_flow_10q.loc['Proceeds from Long-term Lines of Credit', '2023-09-30'] = 3020

df_cash_flow_10q.loc['Payments of Debt Issuance Costs', '2020-06-30'] = -1
df_cash_flow_10q.loc['Payments of Debt Issuance Costs', '2020-09-30'] = -24

df_cash_flow_10q.loc['Repayments of Long-term Lines of Credit', '2023-03-31'] = -825
df_cash_flow_10q.loc['Repayments of Long-term Lines of Credit', '2023-06-30'] = -1670
df_cash_flow_10q.loc['Repayments of Long-term Lines of Credit', '2023-09-30'] = -3020

# Renomeando algumas linhas do índice
df_cash_flow_10q = df_cash_flow_10q.rename(index={
    'nrg_AmortizationOfIntangiblesInTheMoneyContractsAndRetirementOfRECs': 'Amortization Of Intangibles In The Money Contracts And Retirement Of RECs',
    'nrg_GainLossOnSaleAndDispositionOfAssets': 'Gain (Loss) On Sale And Disposition Of Assets',
    'nrg_ChangesInCollateralDepositsSupportingEnergyRiskManagementActivities': 'Changes In Collateral Deposits Supporting Energy Risk Management Activities',
    'nrg_PaymentsForProceedsFromIntangibleAssets': 'Payments For Proceeds From Intangible Assets',
    'nrg_ProceedsFromInsuranceRecoveryPropertyPlantAndEquipmentNet': 'Proceeds From Insurance Recovery Property Plant And Equipment, Net',
    'nrg_AssetImpairmentChargesIncludingEquityMethodInvestments': 'Asset Impairment Charges Including Equity Method Investments',
    'nrg_ChangesInNuclearDecommissioningTrustLiability': 'Changes In Nuclear Decommissioning Trust Liability',
    'nrg_GainsLossesOnExtinguishmentOfDebtNonCashPortion': 'Gains (Losses) On Extinguishment Of Debt Non Cash Portion',
    'nrg_InvestmentsInProceedsFromSalesOfUnconsolidatedAffiliates': 'Investments In Proceeds From Sales Of Unconsolidated Affiliates',
    'nrg_AmortizationOfIntangiblesAndOutOfMarketContracts': 'Amortization Of Intangibles And Out Of Market Contracts',
    'nrg_ProceedsFromSaleOfAssetsAndSaleOfDiscontinuedOperationsNet': 'Proceeds From Sale Of Assets And Sale Of Discontinued Operations, Net',
    'nrg_NetContributionsToDistributionsFromDiscontinuedOperations': 'Net Contributions To Distributions From Discontinued Operations',
    'nrg_GainLossOnAccretionOfAssetRetirementObligations': 'Gain (Loss) On Accretion Of Asset Retirement Obligations'

})

df_cash_flow_10q

In [ ]:
# Arrumando alguns index dos dfs para ficar padronizado com os index utilizados nas funções p/ calcular os indicadores fundamentalistas
df_balance_sheet_10q = df_balance_sheet_10q.rename(index={
    "Stockholders' Equity, Including Portion Attributable to Noncontrolling Interest": 'Total Shareholders Equity',
})

df_income_statement_10q = df_income_statement_10q.rename(index={
    'Research and Development Expense': 'Research and Development',
    'Operating Income (Loss)': 'Operating Income',
    'InterestExpenseNonoperating': 'Interest Expense',
    'Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest': 'Income Before Provision for Income Taxes',
    'Income Tax Expense (Benefit)': 'Provision for Income Taxes',
    'Net Income (Loss) Attributable to Parent': 'Net Income',
    'Weighted Average Number of Shares Outstanding, Basic': 'Number of Shares - Basic',
    'Weighted Average Number of Shares Outstanding, Diluted': 'Number of Shares - Diluted'
})

df_cash_flow_10q = df_cash_flow_10q.rename(index={
    'Depreciation, Amortization and Accretion, Net': 'Depreciation, Depletion and Amortization',
    'Payments of Ordinary Dividends': 'Payments of Dividends',
})

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//energia_eletrica//nrg//nrg_10q.xlsx') as writer:
    df_balance_sheet_10q.to_excel(writer, sheet_name='balance_sheet')
    df_income_statement_10q.to_excel(writer, sheet_name='income_statement')
    df_cash_flow_10q.to_excel(writer, sheet_name='cash_flow')

### Indicadores Fundamentalistas

#### Indicadores 10-K

In [ ]:
# Lendo os arquivos 10k
nrg_bs = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//energia_eletrica//nrg//nrg_10k.xlsx', 
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

nrg_is = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//energia_eletrica//nrg//nrg_10k.xlsx', 
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

nrg_cf = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//energia_eletrica//nrg//nrg_10k.xlsx', 
    sheet_name='cash_flow', 
    index_col='Unnamed: 0'
)

# Transformando as colunas em datetime
nrg_bs.columns = pd.to_datetime(nrg_bs.columns)
nrg_is.columns = pd.to_datetime(nrg_is.columns)
nrg_cf.columns = pd.to_datetime(nrg_cf.columns)

# Transformando as colunas em apenas o ano
nrg_bs.columns = nrg_bs.columns.year
nrg_is.columns = nrg_is.columns.year
nrg_cf.columns = nrg_cf.columns.year

In [ ]:
# Preços da NRG
nrg_close = yf.download('NRG', auto_adjust=True, multi_level_index=False, start='2017-01-01')['Close']

# Selecionando o último preço do mês de Dezembro de cada ano 
lst_years = [
    '2017', '2018', '2019', '2020', 
    '2021', '2022', '2023', '2024'
]

last_dec_close = []

for year in lst_years:
    last_close = nrg_close[f'{year}-12'].iloc[-1]
    last_dec_close.append(last_close)

# Transformando em uma série pandas
nrg_close_dec = pd.Series(last_dec_close, index=lst_years)

# Revertendo o index da série p/ ficar igual com os dos indicadores
nrg_close_dec = nrg_close_dec.reindex(index=nrg_close_dec.index[::-1])

# Arredondamento de duas casas decimais
nrg_close_dec = round(nrg_close_dec, 2)

# Não realizou split recentemente (https://br.investing.com/equities/nrg-energy-inc-historical-data-splits)

# Transformando o index em int
nrg_close_dec.index = nrg_close_dec.index.astype(int)

nrg_close_dec

In [ ]:
# Criando uma lista só para fazer a indexação em que o ano mais recente está no começo
idx = nrg_bs.columns

# Valor de Mercado
vm = f_eua.indicador_vm(ten_k=True, price=nrg_close_dec, df_is=nrg_is) 

# LPA
lpa = f_eua.indicador_lpa(ten_k=True, df_is=nrg_is, idx=None)

# P/L Damodaran
p_lpa_damodaran = f_eua.indicador_p_lpa_2(ten_k=True, vm=vm, df_is=nrg_is)

# P/L
p_lpa = f_eua.indicador_p_lpa(ten_k=True, price=nrg_close_dec, lpa=lpa, idx=idx)

# L/P
lpa_p = f_eua.indicador_lpa_p(ten_k=True, price=nrg_close_dec, lpa=lpa, idx=idx) 

# Depreciação
depreciacao = f_eua.indicador_depreciacao(ten_k=True, df_cf=nrg_cf, first_quarter=None, idx=idx)

# EBITDA
ebitda = f_eua.indicador_ebitda(ten_k=True, df_is=nrg_is, idx=None, depreciacao=depreciacao)

# Margem Líquida
m_liq = f_eua.indicador_ml(ten_k=True, df_is=nrg_is)

# VPA
vpa = f_eua.indicador_vpa(ten_k=True, df_bs=nrg_bs, df_is=nrg_is)

# P/VPA
p_vpa = f_eua.indicador_p_vpa(ten_k=True, price=nrg_close_dec, vpa=vpa, idx=idx)

# Caixa e Equivalentes
lst_caixa = [
    'Cash and Cash Equivalents, at Carrying Value',
    'Restricted Cash and Cash Equivalents, Current'
]

caixa = f_eua.indicador_caixa(ten_k=True, df_bs=nrg_bs, lst_itens_caixa=lst_caixa)

# Dívida Bruta
lst_divida = [
    'Long-term Debt and Lease Obligation, Current', 
    'Operating Lease, Liability, Current', 
    'Long-term Debt and Lease Obligation', 
    'Operating Lease, Liability, Noncurrent'
]

div_bruta = f_eua.indicador_divida_bruta(ten_k=True, df_bs=nrg_bs, lst_itens_divida=lst_divida)

# Dívida Líquida
div_liq = f_eua.indicador_divida_liquida(ten_k=True, caixa=caixa, div_bruta=div_bruta, idx=idx)

# Dívida Líquida / EBITDA
dl_ebitda = f_eua.indicador_dl_ebitda(ten_k=True, ebitda=ebitda, div_liq=div_liq, idx=None)

# Dívida Líquida / PL
dl_pl = f_eua.indicador_dl_pl(ten_k=True, div_liq=div_liq, df_bs=nrg_bs) 

# EV/EBITDA
ev_ebitda = f_eua.indicador_ev_ebitda(vm=vm, div_liq=div_liq, ebitda=ebitda)

# ROE
roe = f_eua.indicador_roe(ten_k=True, df_is=nrg_is, df_bs=nrg_bs, idx=None)

# ROIC
roic = f_eua.indicador_roic(ten_k=True, df_is=nrg_is, df_bs=nrg_bs, div_bruta=div_bruta, idx=idx)

# FCO
fco = f_eua.indicador_fco(ten_k=True, df_cf=nrg_cf, first_quarter=None) 

# FCI
fci = f_eua.indicador_fci(ten_k=True, df_cf=nrg_cf, first_quarter=None) 

# FCF
fcf = f_eua.indicador_fcf(ten_k=True, df_cf=nrg_cf, first_quarter=None) 

# CAPEX
lst_capex = [
    'Payments To Acquire Assets And Businesses Net Of Cash Acquired', 
    'Payments to Acquire Property, Plant, and Equipment'
]

capex = f_eua.indicador_capex(ten_k=True, df_cf=nrg_cf, first_quarter=None, lst_itens=lst_capex, idx=idx)

# Free Cash Flow
free_cash_flow = f_eua.indicador_free_cash_flow(fco=fco, capex=capex)

# Net CAPEX
net_capex = f_eua.indicador_net_capex(capex=capex, depreciation=depreciacao)

# R&D
rd_expense = f_eua.indicador_rd(ten_k=True, df_is=nrg_is, idx=idx)

# Net CAPEX ajustado
adj_net_capex = f_eua.indicador_adj_net_capex(net_capex=net_capex, rd_expense=rd_expense)

# Working capital
working_capital = f_eua.indicador_working_capital(ten_k=True, df_bs=nrg_bs)

# Change in non cash working capital
lst_itens_change_non_cash_wc = [
    'Increase (Decrease) in Accounts Receivable', 
    'Increase (Decrease) in Inventories',
    'Increase (Decrease) in Prepaid Expense and Other Assets',
    'Increase (Decrease) in Accounts Payable',
    'Increase (Decrease) Accrued Expenses And Other Current Liabilities Operating',
    'Increase (Decrease) in Other Operating Assets and Liabilities, Net'
]

change_non_cash_wc = f_eua.indicador_change_non_cash_wc(ten_k=True, df_cf=nrg_cf, lst_itens=lst_itens_change_non_cash_wc, first_quarter=None)

# Reinvestment rate
reinvestment_rate = f_eua.indicador_reinvestment_rate(ten_k=True, net_capex=net_capex, change_non_cash_wc=change_non_cash_wc, df_is=nrg_is, idx=idx)

# New borrowing
lst_itens_new_borrowing = [
    'Proceeds from Issuance of Senior Long-term Debt', 
    'Proceeds from Long-term Lines of Credit'
]

new_borrowing = f_eua.indicador_new_borrowing(ten_k=True, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=nrg_cf, first_quarter=None, idx=idx)

# Debt paid
lst_itens_debt_paid = [
    'Repayments of Other Debt', 
    'Payment for Debt Extinguishment or Debt Prepayment Cost', 
    'Repayments of Long-term Lines of Credit'
]

debt_paid = f_eua.indicador_debt_paid(ten_k=True, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=nrg_cf, first_quarter=None, idx=idx)

# Taxes on operating income
taxes_operating_income = f_eua.indicador_taxes_operating_income(ten_k=True, df_is=nrg_is, idx=idx)

# FCFE
fcfe_10q = f_eua.indicador_fcfe(
    ten_k=True, 
    depreciacao=depreciacao, 
    capex=capex, 
    change_non_cash_wc=change_non_cash_wc, 
    new_borrowing=new_borrowing,
    debt_paid=debt_paid,
    df_is=nrg_is,
    idx=idx
)

# FCFF
fcff_10q = f_eua.indicador_fcff(
        ten_k=True, 
        taxes_operating_income=taxes_operating_income,
        depreciacao=depreciacao, 
        capex=capex, 
        change_non_cash_wc=change_non_cash_wc, 
        df_is=nrg_is,
        idx=idx
)

# Dividendo por Ação
dpa = f_eua.indicador_dpa(ten_k=True, df_cf=nrg_cf, df_is=nrg_is, div_nao_acum=None)

# Payout
payout = f_eua.indicador_payout(ten_k=True, df_cf=nrg_cf, df_is=nrg_is, div_nao_acum=None)

# Buyback
buyback = f_eua.indicador_buyback(ten_k=True, df_cf=nrg_cf, first_quarter=None)

# Colocando todos os indicadores em uma lista
lst_indicadores = [
    p_lpa_damodaran,
    p_lpa,
    lpa_p,  
    ebitda, 
    m_liq,
    p_vpa,
    vm,
    div_bruta,
    caixa,
    div_liq,
    dl_ebitda,
    dl_pl,
    ev_ebitda,
    roe,
    roic,
    fco,
    fci,
    fcf,
    free_cash_flow,
    capex,
    net_capex,
    rd_expense,
    adj_net_capex,
    working_capital,
    reinvestment_rate,
    fcfe_10q,
    fcff_10q,
    dpa,
    payout,
    buyback
]

# Juntando todos os indicadores em um df
nrg_indicadores_10k = pd.concat(lst_indicadores, axis=1)

# Nomes das colunas
nomes_col = [
    'P/L Damodaran', 
    'P/L', 
    'L/P', 
    'EBITDA', 
    'Margem Líquida',
    'P/VPA',
    'Valor de Mercado',
    'Dívida Bruta',
    'Caixa e Equivalentes',
    'Dívida Líquida',
    'Dívida Líquida/EBITDA',
    'Dívida Líquida/PL',
    'EV/EBITDA',
    'ROE',
    'ROIC',
    'FCO',
    'FCI',
    'FCF',
    'Free Cash Flow',
    'Capex',
    'Net Capex',
    'R&D',
    'Adj Net Capex',
    'Working Capital',
    'Reinvestment Rate',
    'FCFE',
    'FCFF',
    'DPA',
    'Payout',
    'Buyback'
]

# Atribuindo os novos nomes das colunas do df
nrg_indicadores_10k.columns = nomes_col
nrg_indicadores_10k

#### Indicadores 10-Q

In [ ]:
# Lendo os arquivos 10q
nrg_bs_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//energia_eletrica//nrg//nrg_10q.xlsx', 
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

nrg_is_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//energia_eletrica//nrg//nrg_10q.xlsx', 
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

nrg_cf_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//energia_eletrica//nrg//nrg_10q.xlsx', 
    sheet_name='cash_flow', 
    index_col='Unnamed: 0'
)

# Aplicando o ajuste nas colunas
nrg_bs_10q.columns = [f_sec.ajustar_mes_03_06_09(col) for col in nrg_bs_10q.columns]
nrg_is_10q.columns = [f_sec.ajustar_mes_03_06_09(col) for col in nrg_is_10q.columns]
nrg_cf_10q.columns = [f_sec.ajustar_mes_03_06_09(col) for col in nrg_cf_10q.columns]

In [ ]:
# Preços da NRG
nrg_close = yf.download('NRG', auto_adjust=True, multi_level_index=False, start='2021-01-01')['Close']

# Selecionando o último preço dos meses de Março, Junho e Setembro
lst_years = [
    '2021', '2022', '2023', '2024',
    '2025'
]

lst_months = ['03', '06', '09']
last_close_months = []

for year in lst_years:
    for month in lst_months:
        last_close = nrg_close[f'{year}-{month}'].dropna().iloc[-1]  # Dropna para lidar com meses sem dados
        last_close_months.append(last_close)

# Criando uma lista em que junte os meses e os anos para que seja o index da serie pandas
lst_idx = []

for year in lst_years:
    for month in lst_months:
        lst_idx.append(f"{month}-{year}")

# Transformando em uma série pandas
nrg_close_months = pd.Series(last_close_months, index=lst_idx)

# Revertendo o index da série p/ ficar igual com os dos indicadores
nrg_close_months = nrg_close_months.reindex(index=nrg_close_months.index[::-1])

# No ano mais atual, como ainda ele não acabou, não consigo fazer o loop com todos meses (Março, Junho e Setembro)
actual_year_int = datetime.now().year  
actual_year_str = str(actual_year_int) 

lst_actual_months = [
    # '03',
    # '06', 
    # '09'
] 

last_close_actual_months = []

for month in lst_actual_months:
    last_close = nrg_close[f'{actual_year_str}-{month}'].dropna().iloc[-1]  # Dropna para lidar com meses sem dados
    last_close_actual_months.append(last_close)

# Criando uma lista em que junte os meses e os anos para que seja o index da serie pandas
lst_idx_actual = []

for month in lst_actual_months:
    lst_idx_actual.append(f"{month}-{actual_year_str}")

# Transformando em uma série pandas
nrg_close_months_atual = pd.Series(last_close_actual_months, index=lst_idx_actual)

# Revertendo o index da série p/ ficar igual com os dos indicadores
nrg_close_months_atual = nrg_close_months_atual.reindex(index=nrg_close_months_atual.index[::-1])

# Juntando as duas series pandas
nrg_close_months = pd.concat([nrg_close_months_atual, nrg_close_months])

# Arredondamento de duas casas decimais
nrg_close_months = round(nrg_close_months, 2)

# Não realizou split recentemente (https://br.investing.com/equities/nrg-energy-inc-historical-data-splits)

# Substituindo o index para ficar igual ao dos dados fundamentalistas
nrg_close_months.index = nrg_bs_10q.columns[:-3]

nrg_close_months

In [ ]:
# Criando uma lista só para fazer a indexação em que o ano mais recente está no começo
idx_10q = nrg_bs_10q.columns

# Valor de Mercado
vm_10q = f_eua.indicador_vm(ten_k=False, price=nrg_close_months, df_is=nrg_is_10q)

# LPA
lpa_acum = f_eua.indicador_lpa(ten_k=False, df_is=nrg_is_10q, idx=idx_10q)[1]
lpa_10q = f_eua.indicador_lpa(ten_k=False, df_is=nrg_is_10q, idx=idx_10q)[0]

# P/L Damodaran
p_lpa_10q_damodaran = f_eua.indicador_p_lpa_2(ten_k=False, vm=vm_10q, df_is=nrg_is_10q)

# P/L
p_lpa_10q = f_eua.indicador_p_lpa(ten_k=False, price=nrg_close_months, lpa=lpa_acum, idx=idx_10q)

# L/P
lpa_p_10q = f_eua.indicador_lpa_p(ten_k=False, price=nrg_close_months, lpa=lpa_acum, idx=idx_10q) 

# Depreciação
depreciacao_10q = f_eua.indicador_depreciacao(ten_k=False, df_cf=nrg_cf_10q, first_quarter='03', idx=idx_10q)[0]
depreciacao_10q_sliced = f_eua.indicador_depreciacao(ten_k=False, df_cf=nrg_cf_10q, first_quarter='03', idx=idx_10q)[1]
depreciacao_10q_acum = f_eua.indicador_depreciacao(ten_k=False, df_cf=nrg_cf_10q, first_quarter='03', idx=idx_10q)[2]

# EBITDA
ebitda_10q = f_eua.indicador_ebitda(ten_k=False, df_is=nrg_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[0]
ebitda_10q_sliced = f_eua.indicador_ebitda(ten_k=False, df_is=nrg_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[1]
ebitda_10q_acum = f_eua.indicador_ebitda(ten_k=False, df_is=nrg_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[2]

# Margem Líquida
m_liq_10q = f_eua.indicador_ml(ten_k=False, df_is=nrg_is_10q)

# VPA
vpa_10q = f_eua.indicador_vpa(ten_k=False, df_bs=nrg_bs_10q, df_is=nrg_is_10q)

# P/VPA
p_vpa_10q = f_eua.indicador_p_vpa(ten_k=False, price=nrg_close_months, vpa=vpa_10q, idx=idx_10q)

# Caixa e Equivalentes
lst_caixa = [
    'Cash and Cash Equivalents, at Carrying Value', 
    'Restricted Cash and Cash Equivalents, Current'
]

caixa_10q = f_eua.indicador_caixa(ten_k=False, df_bs=nrg_bs_10q, lst_itens_caixa=lst_caixa)

# Dívida Bruta
lst_divida = [
    'Long-term Debt and Lease Obligation, Current', 
    'Long-term Debt and Lease Obligation', 
    'Operating Lease, Liability, Current',
    'Operating Lease, Liability, Noncurrent'
]

div_bruta_10q = f_eua.indicador_divida_bruta(ten_k=False, df_bs=nrg_bs_10q, lst_itens_divida=lst_divida)

# Dívida Líquida
div_liq_10q = f_eua.indicador_divida_liquida(ten_k=False, caixa=caixa_10q, div_bruta=div_bruta_10q, idx=idx_10q)

# Dívida Líquida / EBITDA
dl_ebitda_10q = f_eua.indicador_dl_ebitda(ten_k=False, ebitda=ebitda_10q_acum, div_liq=div_liq_10q, idx=idx_10q)

# Dívida Líquida / PL
dl_pl_10q = f_eua.indicador_dl_pl(ten_k=False, div_liq=div_liq_10q, df_bs=nrg_bs_10q) 

# EV/EBITDA
ev_ebitda_10q = f_eua.indicador_ev_ebitda(vm=vm_10q, div_liq=div_liq_10q, ebitda=ebitda_10q_acum)

# ROE
roe_10q = f_eua.indicador_roe(ten_k=False, df_is=nrg_is_10q, df_bs=nrg_bs_10q, idx=idx_10q)

# ROIC 
roic_10q = f_eua.indicador_roic(ten_k=False, df_is=nrg_is_10q, df_bs=nrg_bs_10q, div_bruta=div_bruta_10q, idx=idx_10q)

# FCO
fco_10q = f_eua.indicador_fco(ten_k=False, df_cf=nrg_cf_10q, first_quarter='03') 

# FCI
fci_10q = f_eua.indicador_fci(ten_k=False, df_cf=nrg_cf_10q, first_quarter='03') 

# FCF
fcf_10q = f_eua.indicador_fcf(ten_k=False, df_cf=nrg_cf_10q, first_quarter='03') 

# CAPEX
lst_capex_10q = [
    'Payments to Acquire Other Productive Assets', 
    'Payments to Acquire Property, Plant, and Equipment'
]

capex_10q = f_eua.indicador_capex(ten_k=False, df_cf=nrg_cf_10q, first_quarter='03', lst_itens=lst_capex_10q, idx=idx_10q)[0]
capex_10q_sliced = f_eua.indicador_capex(ten_k=False, df_cf=nrg_cf_10q, first_quarter='03', lst_itens=lst_capex_10q, idx=idx_10q)[1]
capex_10q_acum = f_eua.indicador_capex(ten_k=False, df_cf=nrg_cf_10q, first_quarter='03', lst_itens=lst_capex_10q, idx=idx_10q)[2]

# Free Cash Flow
free_cash_flow_10q = f_eua.indicador_free_cash_flow(fco=fco_10q, capex=capex_10q_sliced)

# Net CAPEX
net_capex_10q = f_eua.indicador_net_capex(capex=capex_10q_acum, depreciation=depreciacao_10q_acum)

# R&D
rd_expense_10q = f_eua.indicador_rd(ten_k=False, df_is=nrg_is_10q, idx=idx_10q)[0]
rd_expense_10q_sliced = f_eua.indicador_rd(ten_k=False, df_is=nrg_is_10q, idx=idx_10q)[1]
rd_expense_10q_acum = f_eua.indicador_rd(ten_k=False, df_is=nrg_is_10q, idx=idx_10q)[2]

# Net CAPEX ajustado
adj_net_capex_10q = f_eua.indicador_adj_net_capex(net_capex=net_capex_10q, rd_expense=rd_expense_10q_acum)

# Working capital
working_capital_10q = f_eua.indicador_working_capital(ten_k=False, df_bs=nrg_bs_10q)

# Change in non cash working capital
lst_itens_change_non_cash_wc = [
    'Increase (Decrease) in Derivative Assets and Liabilities',
    'Increase (Decrease) in Income Taxes',
    'Changes In Collateral Deposits Supporting Energy Risk Management Activities',
    'Increase (Decrease) in Other Operating Assets and Liabilities, Net'
]


change_non_cash_wc_10q = f_eua.indicador_change_non_cash_wc(ten_k=False, df_cf=nrg_cf_10q, lst_itens=lst_itens_change_non_cash_wc, first_quarter='03')[0]
change_non_cash_wc_10q_sliced = f_eua.indicador_change_non_cash_wc(ten_k=False, df_cf=nrg_cf_10q, lst_itens=lst_itens_change_non_cash_wc, first_quarter='03')[1]

# Reinvestment rate
reinvestment_rate_10q = f_eua.indicador_reinvestment_rate(ten_k=False, net_capex=adj_net_capex_10q, change_non_cash_wc=change_non_cash_wc_10q, df_is=nrg_is_10q, idx=idx_10q)

# New borrowing
lst_itens_new_borrowing = [
    'Proceeds from Issuance of Debt', 
    'Proceeds from Long-term Lines of Credit', 
    'Proceeds from (Repayments of) Lines of Credit'
]

new_borrowing_10q = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=nrg_cf_10q, first_quarter='03', idx=idx_10q)[0]
new_borrowing_10q_sliced = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=nrg_cf_10q, first_quarter='03', idx=idx_10q)[1]
new_borrowing_10q_acum = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=nrg_cf_10q, first_quarter='03', idx=idx_10q)[2]

# Debt paid
lst_itens_debt_paid = [
    'Payments of Debt Issuance Costs', 
    'Repayments of Other Debt', 
    'Payment for Debt Extinguishment or Debt Prepayment Cost',
    'Repayments of Long-term Lines of Credit'
]

debt_paid_10q = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=nrg_cf_10q, first_quarter='03', idx=idx_10q)[0]
debt_paid_10q_sliced = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=nrg_cf_10q, first_quarter='03', idx=idx_10q)[1]
debt_paid_10q_acum = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=nrg_cf_10q, first_quarter='03', idx=idx_10q)[2]

# Taxes on operating income
taxes_operating_income_10q = f_eua.indicador_taxes_operating_income(ten_k=False, df_is=nrg_is_10q, idx=idx_10q)

# FCFE
fcfe_10q = f_eua.indicador_fcfe(
    ten_k=False, 
    depreciacao=depreciacao_10q_acum, 
    capex=capex_10q_acum, 
    change_non_cash_wc=change_non_cash_wc_10q_sliced, 
    new_borrowing=new_borrowing_10q_acum, 
    debt_paid=debt_paid_10q_acum, 
    df_is=nrg_is_10q,
    idx=idx_10q
)

# FCFF
fcff_10q = f_eua.indicador_fcff(
        ten_k=False, 
        taxes_operating_income=taxes_operating_income_10q,
        depreciacao=depreciacao_10q_acum, 
        capex=capex_10q_acum, 
        change_non_cash_wc=change_non_cash_wc_10q_sliced, 
        df_is=nrg_is_10q,
        idx=idx_10q
)

# Dividendo não acumulado
div_nao_acum = f_eua.indicador_div_nao_acum(df_cf=nrg_cf_10q, first_quarter='03')

# Dividendo por Ação
dpa_10q = f_eua.indicador_dpa(ten_k=False, df_cf=nrg_cf_10q, df_is=nrg_is_10q, div_nao_acum=div_nao_acum)

# Payout
payout_10q = f_eua.indicador_payout(ten_k=False, df_cf=nrg_cf_10q, df_is=nrg_is_10q, div_nao_acum=div_nao_acum)

# Buyback
buyback_10q = f_eua.indicador_buyback(ten_k=False, df_cf=nrg_cf_10q,  first_quarter='03')

# Lista com todos os indicadores
lst_indicadores_10q = [
    p_lpa_10q_damodaran,
    p_lpa_10q,
    lpa_p_10q,  
    ebitda_10q_sliced, 
    m_liq_10q,
    p_vpa_10q,
    vm_10q,
    div_bruta_10q,
    caixa_10q,
    div_liq_10q,
    dl_ebitda_10q,
    dl_pl_10q,
    ev_ebitda_10q,
    roe_10q,
    roic_10q,
    fco_10q,
    fci_10q,
    fcf_10q,
    free_cash_flow_10q,
    capex_10q_sliced,
    net_capex_10q,
    rd_expense_10q_sliced,
    adj_net_capex_10q,
    working_capital_10q,
    reinvestment_rate_10q,
    fcfe_10q,
    fcff_10q,
    dpa_10q,
    payout_10q,
    buyback_10q
]

# Juntando todos os indicadores em um df
nrg_indicadores_10q = pd.concat(lst_indicadores_10q, axis=1)

# Nomes das colunas
nomes_col = [
    'P/L Damodaran', 
    'P/L', 
    'L/P', 
    'EBITDA', 
    'Margem Líquida',
    'P/VPA',
    'Valor de Mercado',
    'Dívida Bruta',
    'Caixa e Equivalentes',
    'Dívida Líquida',
    'Dívida Líquida/EBITDA',
    'Dívida Líquida/PL',
    'EV/EBITDA',
    'ROE',
    'ROIC',
    'FCO',
    'FCI',
    'FCF',
    'Free Cash Flow',
    'Capex',
    'Net Capex',
    'R&D',
    'Adj Net Capex',
    'Working Capital',
    'Reinvestment Rate',
    'FCFE',
    'FCFF',
    'DPA',
    'Payout',
    'Buyback'
]

# Atribuindo os novos nomes das colunas do df
nrg_indicadores_10q.columns = nomes_col
nrg_indicadores_10q

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//energia_eletrica//nrg//nrg_indicators.xlsx') as writer:
    nrg_indicadores_10k.to_excel(writer, sheet_name='indicators_10k')
    nrg_indicadores_10q.to_excel(writer, sheet_name='indicators_10q')

# **Setor de tecnologia**

## Amazon

* 1Q -> Lançamento em Março (03);
* 2Q -> Lançamento em Junho (06);
* 3Q -> Lançamento em Setembro (09);
* 10K -> Lançamento em Dezembro (12).

In [ ]:
ticker = 'amzn'
f_sec.cik_matching_ticker(ticker)

### Scraping Amazon 10-K

In [ ]:
# Número dos documentos que contém a DRE, BP e FC 
acc_documents = f_sec.get_filtered_filings(ticker=ticker, ten_k=True, just_accession_numbers=True)

# Trocando os '-' para '' das strings ('0001045810-24-000264' -> '000104581024000264')
dict_acc_num = {k: v.replace('-', '') for k, v in acc_documents.items()}

# Transformando o dicionário em um df
df_lst_acc_num_amzn = pd.DataFrame.from_dict(dict_acc_num, orient='index', columns=['acc_num'])

# Adicionando os 'acc_num' de 2017 e 2018
df_acc_num = pd.DataFrame(
    {
        'acc_num': [
            '000101872419000004', # 2018
            '000101872418000005', # 2017
        ]
    },
    index=pd.to_datetime([
        '2018-12-31',
        '2017-12-31',
    ])
)

# Concatenando os dfs
df_lst_acc_num_amzn = pd.concat([df_lst_acc_num_amzn, df_acc_num])

# Transformando em datetime o index
df_lst_acc_num_amzn.index = pd.to_datetime(df_lst_acc_num_amzn.index, format='%Y-%m-%d')

df_lst_acc_num_amzn

In [ ]:
# Rótulos
labels_dict = f_sec.get_label_dictionary(ticker, headers=f_sec.header)

In [ ]:
# Fazendo o scraping do balance sheet (10-K)
balance_sheet_lst = []

for acc_num in df_lst_acc_num_amzn['acc_num']:

    # Balance sheet
    balance_sheet = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='balance_sheet')
    balance_sheet = f_sec.rename_statement(balance_sheet, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    balance_sheet = balance_sheet.iloc[:, 0]
    balance_sheet_lst.append(balance_sheet)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_balance_sheet_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in balance_sheet_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_balance_sheet_10k = pd.concat([df_balance_sheet_10k, df], axis=1)

# Substituindo os NaN por 0
df_balance_sheet_10k = df_balance_sheet_10k.fillna(0)

# A linha 'Treasury Stock, Common, Value' nas colunas '2021-12-31', '2020-12-31', '2019-12-31', '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'Treasury Stock, Value'
df_balance_sheet_10k.loc['Treasury Stock, Common, Value', '2021-12-31'] = df_balance_sheet_10k.loc['Treasury Stock, Value', '2021-12-31'] 
df_balance_sheet_10k.loc['Treasury Stock, Common, Value', '2020-12-31'] = df_balance_sheet_10k.loc['Treasury Stock, Value', '2020-12-31'] 
df_balance_sheet_10k.loc['Treasury Stock, Common, Value', '2019-12-31'] = df_balance_sheet_10k.loc['Treasury Stock, Value', '2019-12-31'] 
df_balance_sheet_10k.loc['Treasury Stock, Common, Value', '2018-12-31'] = df_balance_sheet_10k.loc['Treasury Stock, Value', '2018-12-31'] 
df_balance_sheet_10k.loc['Treasury Stock, Common, Value', '2017-12-31'] = df_balance_sheet_10k.loc['Treasury Stock, Value', '2017-12-31'] 

# A linha 'Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization' nas colunas '2019-12-31', '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'Property, Plant and Equipment, Net'
df_balance_sheet_10k.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2019-12-31'] = df_balance_sheet_10k.loc['Property, Plant and Equipment, Net', '2019-12-31'] 
df_balance_sheet_10k.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2018-12-31'] = df_balance_sheet_10k.loc['Property, Plant and Equipment, Net', '2018-12-31'] 
df_balance_sheet_10k.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2017-12-31'] = df_balance_sheet_10k.loc['Property, Plant and Equipment, Net', '2017-12-31'] 

# A linha 'Accrued Liabilities, Current' nas colunas '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'amzn_AccruedLiabilitiesAndOtherCurrent'
df_balance_sheet_10k.loc['Accrued Liabilities, Current', '2018-12-31'] = df_balance_sheet_10k.loc['amzn_AccruedLiabilitiesAndOtherCurrent', '2018-12-31'] 
df_balance_sheet_10k.loc['Accrued Liabilities, Current', '2017-12-31'] = df_balance_sheet_10k.loc['amzn_AccruedLiabilitiesAndOtherCurrent', '2017-12-31'] 

# A linha 'Contract with Customer, Liability, Current' na coluna '2017-12-31' está zerada
# As informações estão na linha 'Deferred Revenue, Current'
df_balance_sheet_10k.loc['Contract with Customer, Liability, Current', '2017-12-31'] = df_balance_sheet_10k.loc['Deferred Revenue, Current', '2017-12-31'] 

# Removendo as linhas que movemos os seus itens para as linhas corretas
df_balance_sheet_10k = df_balance_sheet_10k.drop([
    'Treasury Stock, Value',
    'Property, Plant and Equipment, Net',
    'amzn_AccruedLiabilitiesAndOtherCurrent',
    'Deferred Revenue, Current'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_balance_sheet_10k = df_balance_sheet_10k.drop([
    'us-gaap_AssetsCurrentAbstract',
    'Preferred Stock, Value, Issued'
])

# Arrumando e adicionando alguns números no df
df_balance_sheet_10k.loc['amzn_LeaseLiabilityNoncurrent', '2018-12-31'] = 9650

# Renomeando a linha do índice que está sem formato 
df_balance_sheet_10k = df_balance_sheet_10k.rename(index={
    'amzn_LeaseLiabilityNoncurrent': 'Lease Liability, Noncurrent'
})

df_balance_sheet_10k

In [ ]:
# Fazendo o scraping do income statement (10-K)
income_statement_lst = []

for acc_num in df_lst_acc_num_amzn['acc_num']:

    # Income statement
    income_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='income_statement')
    income_statement = f_sec.rename_statement(income_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    income_statement = income_statement.iloc[:, 0]
    income_statement_lst.append(income_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_income_statement_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in income_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_income_statement_10k = pd.concat([df_income_statement_10k, df], axis=1)

# Substituindo os NaN por 0
df_income_statement_10k = df_income_statement_10k.fillna(0)

# A linha 'InterestExpenseNonoperating' nas colunas '2023-12-31', '2022-12-31', '2021-12-31', '2020-12-31', '2019-12-31', '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão nas linhas 'Interest Expense' 
df_income_statement_10k.loc['InterestExpenseNonoperating', '2023-12-31'] = df_income_statement_10k.loc['Interest Expense', '2023-12-31']
df_income_statement_10k.loc['InterestExpenseNonoperating', '2022-12-31'] = df_income_statement_10k.loc['Interest Expense', '2022-12-31']
df_income_statement_10k.loc['InterestExpenseNonoperating', '2021-12-31'] = df_income_statement_10k.loc['Interest Expense', '2021-12-31']
df_income_statement_10k.loc['InterestExpenseNonoperating', '2020-12-31'] = df_income_statement_10k.loc['Interest Expense', '2020-12-31']
df_income_statement_10k.loc['InterestExpenseNonoperating', '2019-12-31'] = df_income_statement_10k.loc['Interest Expense', '2019-12-31']
df_income_statement_10k.loc['InterestExpenseNonoperating', '2018-12-31'] = df_income_statement_10k.loc['Interest Expense', '2018-12-31']
df_income_statement_10k.loc['InterestExpenseNonoperating', '2017-12-31'] = df_income_statement_10k.loc['Interest Expense', '2017-12-31']

# A linha 'amzn_TechnologyAndInfrastructureExpense' nas colunas '2022-12-31', '2021-12-31', '2020-12-31', '2019-12-31', '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão nas linhas 'amzn_TechnologyAndContentExpense' 
df_income_statement_10k.loc['amzn_TechnologyAndInfrastructureExpense', '2022-12-31'] = df_income_statement_10k.loc['amzn_TechnologyAndContentExpense', '2022-12-31']
df_income_statement_10k.loc['amzn_TechnologyAndInfrastructureExpense', '2021-12-31'] = df_income_statement_10k.loc['amzn_TechnologyAndContentExpense', '2021-12-31']
df_income_statement_10k.loc['amzn_TechnologyAndInfrastructureExpense', '2020-12-31'] = df_income_statement_10k.loc['amzn_TechnologyAndContentExpense', '2020-12-31']
df_income_statement_10k.loc['amzn_TechnologyAndInfrastructureExpense', '2019-12-31'] = df_income_statement_10k.loc['amzn_TechnologyAndContentExpense', '2019-12-31']
df_income_statement_10k.loc['amzn_TechnologyAndInfrastructureExpense', '2018-12-31'] = df_income_statement_10k.loc['amzn_TechnologyAndContentExpense', '2018-12-31']
df_income_statement_10k.loc['amzn_TechnologyAndInfrastructureExpense', '2017-12-31'] = df_income_statement_10k.loc['amzn_TechnologyAndContentExpense', '2017-12-31']

# A linha 'Other Operating Income (Expense), Net' nas colunas '2019-12-31', '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão nas linhas 'Other Cost and Expense, Operating' 
df_income_statement_10k.loc['Other Operating Income (Expense), Net', '2019-12-31'] = df_income_statement_10k.loc['Other Cost and Expense, Operating', '2019-12-31']
df_income_statement_10k.loc['Other Operating Income (Expense), Net', '2018-12-31'] = df_income_statement_10k.loc['Other Cost and Expense, Operating', '2018-12-31']
df_income_statement_10k.loc['Other Operating Income (Expense), Net', '2017-12-31'] = df_income_statement_10k.loc['Other Cost and Expense, Operating', '2017-12-31']

# A linha 'Revenue from Contract with Customer, Excluding Assessed Tax' na coluna '2017-12-31' está zerada
# As informações estão nas linhas 'Revenue, Net (Deprecated 2018-01-31)' 
df_income_statement_10k.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2017-12-31'] = df_income_statement_10k.loc['Revenue, Net (Deprecated 2018-01-31)', '2017-12-31'] 

# Removendo as linhas que movemos os seus itens para as linhas corretas
df_income_statement_10k = df_income_statement_10k.drop([
    'amzn_TechnologyAndContentExpense',
    'Other Cost and Expense, Operating',
    'Revenue, Net (Deprecated 2018-01-31)',
    'Interest Expense'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_income_statement_10k = df_income_statement_10k.drop([
    'us-gaap_CostsAndExpensesAbstract',
    'us-gaap_IncomeStatementAbstract',
    'Sales Revenue, Goods, Net (Deprecated 2018-01-31)',
    'Sales Revenue, Services, Net (Deprecated 2018-01-31)'
])

# Renomeando a linha do índice que está sem formato 
df_income_statement_10k = df_income_statement_10k.rename(index={
    'amzn_FulfillmentExpense': 'Fulfillment Expense',
    'amzn_TechnologyAndInfrastructureExpense': 'Technology and Infrastructure Expense',
    'InterestExpenseNonoperating': 'Interest Expense',
})

# Formatando o número para ficar positivo
df_income_statement_10k.loc['Interest Expense'] = df_income_statement_10k.loc['Interest Expense'] * -1
df_income_statement_10k.loc['Income Tax Expense (Benefit)'] = df_income_statement_10k.loc['Income Tax Expense (Benefit)'] * -1

df_income_statement_10k

In [ ]:
# Fazendo o scraping do cash flow statement (10-K)
cash_flow_statement_lst = []

for acc_num in df_lst_acc_num_amzn['acc_num']:

    # Cash flow statement
    cash_flow_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='cash_flow_statement')
    cash_flow_statement = f_sec.rename_statement(cash_flow_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    cash_flow_statement = cash_flow_statement.iloc[:, 0]
    cash_flow_statement_lst.append(cash_flow_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_cash_flow_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in cash_flow_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_cash_flow_10k = pd.concat([df_cash_flow_10k, df], axis=1)

# Substituindo os NaN por 0
df_cash_flow_10k = df_cash_flow_10k.fillna(0)

# A linha 'amzn_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPaymentsToAcquireNonmarketableSecuritiesAndOther' nas colunas '2022-12-31', '2021-12-31', '2020-12-31', '2019-12-31', '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'amzn_PaymentsToAcquireBusinessesNetOfCashAcquiredAndOther'
df_cash_flow_10k.loc['amzn_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPaymentsToAcquireNonmarketableSecuritiesAndOther', '2022-12-31'] = df_cash_flow_10k.loc['amzn_PaymentsToAcquireBusinessesNetOfCashAcquiredAndOther', '2022-12-31'] 
df_cash_flow_10k.loc['amzn_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPaymentsToAcquireNonmarketableSecuritiesAndOther', '2021-12-31'] = df_cash_flow_10k.loc['amzn_PaymentsToAcquireBusinessesNetOfCashAcquiredAndOther', '2021-12-31'] 
df_cash_flow_10k.loc['amzn_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPaymentsToAcquireNonmarketableSecuritiesAndOther', '2020-12-31'] = df_cash_flow_10k.loc['amzn_PaymentsToAcquireBusinessesNetOfCashAcquiredAndOther', '2020-12-31'] 
df_cash_flow_10k.loc['amzn_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPaymentsToAcquireNonmarketableSecuritiesAndOther', '2019-12-31'] = df_cash_flow_10k.loc['amzn_PaymentsToAcquireBusinessesNetOfCashAcquiredAndOther', '2019-12-31'] 
df_cash_flow_10k.loc['amzn_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPaymentsToAcquireNonmarketableSecuritiesAndOther', '2018-12-31'] = df_cash_flow_10k.loc['amzn_PaymentsToAcquireBusinessesNetOfCashAcquiredAndOther', '2018-12-31'] 
df_cash_flow_10k.loc['amzn_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPaymentsToAcquireNonmarketableSecuritiesAndOther', '2017-12-31'] = df_cash_flow_10k.loc['amzn_PaymentsToAcquireBusinessesNetOfCashAcquiredAndOther', '2017-12-31'] 

# A linha 'Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations' nas colunas '2021-12-31', '2020-12-31', '2019-12-31', '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'Effect of Exchange Rate on Cash and Cash Equivalents'
df_cash_flow_10k.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations', '2021-12-31'] = df_cash_flow_10k.loc['Effect of Exchange Rate on Cash and Cash Equivalents', '2021-12-31'] 
df_cash_flow_10k.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations', '2020-12-31'] = df_cash_flow_10k.loc['Effect of Exchange Rate on Cash and Cash Equivalents', '2020-12-31'] 
df_cash_flow_10k.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations', '2019-12-31'] = df_cash_flow_10k.loc['Effect of Exchange Rate on Cash and Cash Equivalents', '2019-12-31'] 
df_cash_flow_10k.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations', '2018-12-31'] = df_cash_flow_10k.loc['Effect of Exchange Rate on Cash and Cash Equivalents', '2018-12-31'] 
df_cash_flow_10k.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations', '2017-12-31'] = df_cash_flow_10k.loc['Effect of Exchange Rate on Cash and Cash Equivalents', '2017-12-31'] 

# A linha 'Repayments of Long-term Debt' nas colunas '2020-12-31', '2019-12-31', '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'Repayments of Debt' 
df_cash_flow_10k.loc['Repayments of Long-term Debt', '2020-12-31'] = df_cash_flow_10k.loc['Repayments of Debt', '2020-12-31'] 
df_cash_flow_10k.loc['Repayments of Long-term Debt', '2019-12-31'] = df_cash_flow_10k.loc['Repayments of Debt', '2019-12-31'] 
df_cash_flow_10k.loc['Repayments of Long-term Debt', '2018-12-31'] = df_cash_flow_10k.loc['Repayments of Debt', '2018-12-31'] 
df_cash_flow_10k.loc['Repayments of Long-term Debt', '2017-12-31'] = df_cash_flow_10k.loc['Repayments of Debt', '2017-12-31'] 

# A linha 'amzn_ProceedsFromPropertyPlantAndEquipmentSalesAndIncentives' nas colunas '2020-12-31', '2019-12-31', '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'amzn_ProceedsFromRebatesOnPurchasesOfProductiveAssets'
df_cash_flow_10k.loc['amzn_ProceedsFromPropertyPlantAndEquipmentSalesAndIncentives', '2020-12-31'] = df_cash_flow_10k.loc['amzn_ProceedsFromRebatesOnPurchasesOfProductiveAssets', '2020-12-31'] 
df_cash_flow_10k.loc['amzn_ProceedsFromPropertyPlantAndEquipmentSalesAndIncentives', '2019-12-31'] = df_cash_flow_10k.loc['amzn_ProceedsFromRebatesOnPurchasesOfProductiveAssets', '2019-12-31'] 
df_cash_flow_10k.loc['amzn_ProceedsFromPropertyPlantAndEquipmentSalesAndIncentives', '2018-12-31'] = df_cash_flow_10k.loc['amzn_ProceedsFromRebatesOnPurchasesOfProductiveAssets', '2018-12-31'] 
df_cash_flow_10k.loc['amzn_ProceedsFromPropertyPlantAndEquipmentSalesAndIncentives', '2017-12-31'] = df_cash_flow_10k.loc['amzn_ProceedsFromRebatesOnPurchasesOfProductiveAssets', '2017-12-31'] 

# A linha 'Proceeds from Short-term Debt' nas colunas '2022-12-31', '2021-12-31' e '2020-12-31' estão zeradas
# As informações estão na linha 'amzn_ProceedsFromShortTermDebtAndOtherFinancingActivities'
df_cash_flow_10k.loc['Proceeds from Short-term Debt', '2022-12-31'] = df_cash_flow_10k.loc['amzn_ProceedsFromShortTermDebtAndOtherFinancingActivities', '2022-12-31'] 
df_cash_flow_10k.loc['Proceeds from Short-term Debt', '2021-12-31'] = df_cash_flow_10k.loc['amzn_ProceedsFromShortTermDebtAndOtherFinancingActivities', '2021-12-31'] 
df_cash_flow_10k.loc['Proceeds from Short-term Debt', '2020-12-31'] = df_cash_flow_10k.loc['amzn_ProceedsFromShortTermDebtAndOtherFinancingActivities', '2020-12-31']

# A linha 'Repayments of Short-term Debt' nas colunas '2022-12-31', '2021-12-31' e '2020-12-31' estão zeradas
# As informações estão na linha 'amzn_RepaymentsOfShortTermDebtAndOtherFinancingActivities'
df_cash_flow_10k.loc['Repayments of Short-term Debt', '2022-12-31'] = df_cash_flow_10k.loc['amzn_RepaymentsOfShortTermDebtAndOtherFinancingActivities', '2022-12-31'] 
df_cash_flow_10k.loc['Repayments of Short-term Debt', '2021-12-31'] = df_cash_flow_10k.loc['amzn_RepaymentsOfShortTermDebtAndOtherFinancingActivities', '2021-12-31'] 
df_cash_flow_10k.loc['Repayments of Short-term Debt', '2020-12-31'] = df_cash_flow_10k.loc['amzn_RepaymentsOfShortTermDebtAndOtherFinancingActivities', '2020-12-31'] 

# A linha 'amzn_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPaymentsToAcquireNonmarketableSecuritiesAndOther' nas colunas '2019-12-31' e '2018-12-31' estão zeradas
# As informações estão na linha 'Payments to Acquire Businesses, Net of Cash Acquired' 
df_cash_flow_10k.loc['amzn_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPaymentsToAcquireNonmarketableSecuritiesAndOther', '2019-12-31'] = df_cash_flow_10k.loc['Payments to Acquire Businesses, Net of Cash Acquired', '2019-12-31'] 
df_cash_flow_10k.loc['amzn_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPaymentsToAcquireNonmarketableSecuritiesAndOther', '2018-12-31'] = df_cash_flow_10k.loc['Payments to Acquire Businesses, Net of Cash Acquired', '2018-12-31'] 

# A linha 'Finance Lease, Principal Payments' nas colunas '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'Repayments of Long-term Capital Lease Obligations' 
df_cash_flow_10k.loc['Finance Lease, Principal Payments', '2018-12-31'] = df_cash_flow_10k.loc['Repayments of Long-term Capital Lease Obligations', '2018-12-31'] 
df_cash_flow_10k.loc['Finance Lease, Principal Payments', '2017-12-31'] = df_cash_flow_10k.loc['Repayments of Long-term Capital Lease Obligations', '2017-12-31'] 

# A linha 'amzn_RepaymentsOfLongTermFinancingObligations' nas colunas '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'amzn_RepaymentsOfLongTermFinanceLeaseObligations' 
df_cash_flow_10k.loc['amzn_RepaymentsOfLongTermFinancingObligations', '2018-12-31'] = df_cash_flow_10k.loc['amzn_RepaymentsOfLongTermFinanceLeaseObligations', '2018-12-31'] 
df_cash_flow_10k.loc['amzn_RepaymentsOfLongTermFinancingObligations', '2017-12-31'] = df_cash_flow_10k.loc['amzn_RepaymentsOfLongTermFinanceLeaseObligations', '2017-12-31'] 

# A linha 'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents' na coluna '2017-12-31' está zeradas
# As informações estão na linha 'Cash and Cash Equivalents, at Carrying Value'
df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2017-12-31'] = df_cash_flow_10k.loc['Cash and Cash Equivalents, at Carrying Value', '2017-12-31'] 

# A linha 'Payments to Acquire Productive Assets' na coluna '2017-12-31' está zeradas
# As informações estão na linha 'Payments for (Proceeds from) Productive Assets'
df_cash_flow_10k.loc['Payments to Acquire Productive Assets', '2017-12-31'] = df_cash_flow_10k.loc['Payments for (Proceeds from) Productive Assets', '2017-12-31'] 

# A linha 'Increase (Decrease) in Contract with Customer, Liability' na coluna '2017-12-31' está zeradas
# As informações estão na linha 'Increase (Decrease) in Deferred Revenue'
df_cash_flow_10k.loc['Increase (Decrease) in Contract with Customer, Liability', '2017-12-31'] = df_cash_flow_10k.loc['Increase (Decrease) in Deferred Revenue', '2017-12-31'] 

# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_cash_flow_10k = df_cash_flow_10k.drop([
    'amzn_PaymentsToAcquireBusinessesNetOfCashAcquiredAndOther',
    'Effect of Exchange Rate on Cash and Cash Equivalents',
    'Repayments of Debt',
    'amzn_ProceedsFromRebatesOnPurchasesOfProductiveAssets',
    'amzn_ProceedsFromShortTermDebtAndOtherFinancingActivities',
    'amzn_RepaymentsOfShortTermDebtAndOtherFinancingActivities',
    'Payments to Acquire Businesses, Net of Cash Acquired',
    'Repayments of Long-term Capital Lease Obligations',
    'amzn_RepaymentsOfLongTermFinanceLeaseObligations',
    'Cash and Cash Equivalents, at Carrying Value',
    'Payments for (Proceeds from) Productive Assets',
    'Increase (Decrease) in Deferred Revenue'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_cash_flow_10k = df_cash_flow_10k.drop([
    'us-gaap_StatementOfCashFlowsAbstract',
    'amzn_InterestPaidOnCapitalLeaseObligations',
    'amzn_InterestPaidOnLongTermDebt',
    'Operating Lease, Payments',
    'Finance Lease, Interest Payment on Liability',
    'amzn_InterestPaidFinancingObligations',
    'Income Taxes Paid, Net',
    'Right-of-Use Asset Obtained in Exchange for Operating Lease Liability',
    'Lease Obligation Incurred',
    'Right-of-Use Asset Obtained in Exchange for Finance Lease Liability',
    'amzn_PropertyandEquipmentObtainedinExchangeforFinancingObligations',
    'amzn_InterestPaidOnCapitalAndFinanceLeaseObligations',
    'amzn_BuildToSuitLeaseObligationsIncurred',
    'Cash and Cash Equivalents, Period Increase (Decrease)'
])

# Arrumando e adicionando alguns números no df
df_cash_flow_10k.loc['Increase (Decrease) in Other Noncurrent Assets', '2022-12-31'] = -13275
df_cash_flow_10k.loc['Increase (Decrease) in Other Noncurrent Assets', '2021-12-31'] = -9018
df_cash_flow_10k.loc['Proceeds from Short-term Debt', '2019-12-31'] = 1402 
df_cash_flow_10k.loc['Proceeds from Short-term Debt', '2018-12-31'] = 886
df_cash_flow_10k.loc['Proceeds from Short-term Debt', '2017-12-31'] = 16228
df_cash_flow_10k.loc['Repayments of Short-term Debt', '2019-12-31'] = -1518
df_cash_flow_10k.loc['Repayments of Short-term Debt', '2018-12-31'] = -813

# Renomeando a linha do índice que está sem formato 
df_cash_flow_10k = df_cash_flow_10k.rename(index={
    'amzn_ProceedsFromPropertyPlantAndEquipmentSalesAndIncentives': 'Proceeds From Property Plant And Equipment Sales And Incentives',
    'amzn_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPaymentsToAcquireNonmarketableSecuritiesAndOther': 'Payments To Acquire Businesses Net Of Cash Acquired And Payments To Acquire Nonmarketable Securities',
    'amzn_RepaymentsOfLongTermFinancingObligations': 'Repayments Of Long Term Financing Obligations',
})

df_cash_flow_10k

In [ ]:
# Arrumando alguns index dos dfs para ficar padronizado com os index utilizados nas funções p/ calcular os indicadores fundamentalistas
df_balance_sheet_10k = df_balance_sheet_10k.rename(index={
    'Cash and Cash Equivalents, at Carrying Value': 'Cash and Cash Equivalents',
    'Marketable Securities, Current': 'Marketable Securities',
    'Lease Liability, Noncurrent': 'Long-term Lease Liabilities',
    'Long-term Debt, Excluding Current Maturities': 'Long-term Debt',
    "Stockholders' Equity Attributable to Parent": 'Total Shareholders Equity'
})

df_income_statement_10k = df_income_statement_10k.rename(index={
    'Revenue from Contract with Customer, Excluding Assessed Tax': 'Revenues',
    'Technology and Infrastructure Expense': 'Research and Development',
    'Operating Income (Loss)': 'Operating Income',
    'Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest': 'Income Before Provision for Income Taxes',
    'Income Tax Expense (Benefit)': 'Provision for Income Taxes',
    'Net Income (Loss) Attributable to Parent': 'Net Income',
    'Weighted Average Number of Shares Outstanding, Basic': 'Number of Shares - Basic',
    'Weighted Average Number of Shares Outstanding, Diluted': 'Number of Shares - Diluted'
})

df_cash_flow_10k = df_cash_flow_10k.rename(index={
    'Payments to Acquire Productive Assets': 'Purchases Of Property And Equipment And Intangible Assets'
})

In [ ]:
# 'acc_num' entre 2019 e 2025
df_lst_acc_num_amzn_sorted = df_lst_acc_num_amzn.sort_index()
acc_num_list_amzn_sliced = df_lst_acc_num_amzn_sorted.loc['2019':'2025', 'acc_num']
acc_num_list_amzn_sliced

In [ ]:
# 'acc_num' entre 2019 e 2025
df_lst_acc_num_amzn_sorted = df_lst_acc_num_amzn.sort_index()
acc_num_list_amzn_sliced = df_lst_acc_num_amzn_sorted.loc['2019':'2025', 'acc_num']

# Fazendo o scraping do 'current portion of lease liabilities'
current_portion_lease_lst = []

for acc_num in acc_num_list_amzn_sliced:
    # Fazendo o scraping do 'current portion of lease liabilities' (10-K) -> 'Note 4 – Leases' 
    current_portion_lease_10k = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='current_portion_lease')
    current_portion_lease_10k = f_sec.rename_statement(current_portion_lease_10k, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    current_portion_lease_10k = current_portion_lease_10k.iloc[:, 0]
    current_portion_lease_lst.append(current_portion_lease_10k)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_current_portion_lease_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in current_portion_lease_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_current_portion_lease_10k = pd.concat([df_current_portion_lease_10k, df], axis=1)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_current_portion_lease_10k = df_current_portion_lease_10k.drop('us-gaap_LeasesAbstract')

# Renomeando a linha do índice
df_current_portion_lease_10k = df_current_portion_lease_10k.rename(index={
    'amzn_LeaseLiabilityCurrent': 'Current Portion of Lease Liabilities',
})

# Formatando o número para ficar positivo
df_current_portion_lease_10k.loc['Current Portion of Lease Liabilities'] = df_current_portion_lease_10k.loc['Current Portion of Lease Liabilities'] * -1

# Selecionando a linha desejada
df_current_portion_lease_10k = df_current_portion_lease_10k.loc[['Current Portion of Lease Liabilities']]


################################################################################################################################################################
# Nos anos de 2018 e 2017, o 'current portion lease' está separado em dois ('current portion capital lease' e 'current portion finance lease')

# 'acc_num' de 2018
acc_num_2018 = '000101872419000004'

# Fazendo o scraping do 'current portion of capital lease' (10-K) -> 'Note 6 - other long-term liabilities' 
current_portion_capital_lease_10k_18_17 = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num_2018, statement_name='current_portion_capital_lease')
current_portion_capital_lease_10k_18_17 = f_sec.rename_statement(current_portion_capital_lease_10k_18_17, label_dictionary=labels_dict)
# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
current_portion_capital_lease_10k_18_17 = current_portion_capital_lease_10k_18_17.drop('us-gaap_OtherLiabilitiesDisclosureAbstract')
# Renomeando a linha do índice
current_portion_capital_lease_10k_18_17 = current_portion_capital_lease_10k_18_17.rename(index={
    'Capital Lease Obligations, Current': 'Current Portion of Capital Lease',
})
# Formatando o número para ficar positivo
current_portion_capital_lease_10k_18_17.loc['Current Portion of Capital Lease'] = current_portion_capital_lease_10k_18_17.loc['Current Portion of Capital Lease'] * -1
# Selecionando a linha desejada
current_portion_capital_lease_10k_18_17 = current_portion_capital_lease_10k_18_17.loc[['Current Portion of Capital Lease']]

# Fazendo o scraping do 'current portion of finance lease' (10-K) -> 'Note 6 - other long-term liabilities' 
current_portion_finance_lease_10k_18_17 = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num_2018, statement_name='current_portion_finance_lease')
current_portion_finance_lease_10k_18_17 = f_sec.rename_statement(current_portion_finance_lease_10k_18_17, label_dictionary=labels_dict)
# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
current_portion_finance_lease_10k_18_17 = current_portion_finance_lease_10k_18_17.drop('us-gaap_OtherLiabilitiesDisclosureAbstract')
# Renomeando a linha do índice
current_portion_finance_lease_10k_18_17 = current_portion_finance_lease_10k_18_17.rename(index={
    'amzn_FinanceLeaseObligationsCurrent': 'Current Portion of Finance Lease',
})
# Formatando o número para ficar positivo
current_portion_finance_lease_10k_18_17.loc['Current Portion of Finance Lease'] = current_portion_finance_lease_10k_18_17.loc['Current Portion of Finance Lease'] * -1
# Selecionando a linha desejada
current_portion_finance_lease_10k_18_17 = current_portion_finance_lease_10k_18_17.loc[['Current Portion of Finance Lease']]

# Concatenando os dfs
current_portion_lease_10k_18_17 = pd.concat([current_portion_capital_lease_10k_18_17, current_portion_finance_lease_10k_18_17])
# Somando os itens ('current portion capital lease' e 'current portion finance lease') e criando um novo df
df_current_portion_lease_10k_18_17 = pd.DataFrame(current_portion_lease_10k_18_17.sum()).T
# Renomeando o index
df_current_portion_lease_10k_18_17 = df_current_portion_lease_10k_18_17.rename(index={0:'Current Portion of Lease Liabilities'})


################################################################################################################################################################
# Concatenando os dfs
df_current_portion_lease_10k = pd.concat([df_current_portion_lease_10k, df_current_portion_lease_10k_18_17], axis=1)

# Ordenando o df
df_current_portion_lease_10k = df_current_portion_lease_10k[sorted(df_current_portion_lease_10k.columns, reverse=True)]

df_current_portion_lease_10k

In [ ]:
# Fazendo o scraping do 'Current portion of long-term debt' (10-K)
current_portion_debt_lst = []

for acc_num in df_lst_acc_num_amzn['acc_num']:
    # Fazendo o scraping do 'Current portion of long-term debt' (10-K) -> # 'Note 6. Debt' 
    current_portion_debt_10k = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='current_portion_debt')
    current_portion_debt_10k = f_sec.rename_statement(current_portion_debt_10k, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    current_portion_debt_10k = current_portion_debt_10k.iloc[:, 0]
    current_portion_debt_lst.append(current_portion_debt_10k)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_current_portion_debt_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in current_portion_debt_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_current_portion_debt_10k = pd.concat([df_current_portion_debt_10k, df], axis=1)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_current_portion_debt_10k = df_current_portion_debt_10k.drop('us-gaap_DebtInstrumentLineItems')

# Renomeando a linha do índice
df_current_portion_debt_10k = df_current_portion_debt_10k.rename(index={
    'Long-term Debt, Current Maturities': 'Current Portion of Long-term Debt',
})

# Formatando o número para ficar positivo
df_current_portion_debt_10k.loc['Current Portion of Long-term Debt'] = df_current_portion_debt_10k.loc['Current Portion of Long-term Debt'] * -1

# Arrumando a escala de algumas colunas
df_current_portion_debt_10k.loc[:, '2017-12-31'] = df_current_portion_debt_10k.loc[:, '2017-12-31'] / 1000000
df_current_portion_debt_10k.loc[:, '2020-12-31'] = df_current_portion_debt_10k.loc[:, '2020-12-31'] / 1000000
df_current_portion_debt_10k.loc[:, '2021-12-31'] = df_current_portion_debt_10k.loc[:, '2021-12-31'] / 1000000
df_current_portion_debt_10k.loc[:, '2022-12-31'] = df_current_portion_debt_10k.loc[:, '2022-12-31'] / 1000000

# Selecionando a linha desejada
df_current_portion_debt_10k = df_current_portion_debt_10k.loc[['Current Portion of Long-term Debt']]

df_current_portion_debt_10k

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//tecnologia//amazon//amzn_10k.xlsx') as writer:
    df_balance_sheet_10k.to_excel(writer, sheet_name='balance_sheet')
    df_income_statement_10k.to_excel(writer, sheet_name='income_statement')
    df_cash_flow_10k.to_excel(writer, sheet_name='cash_flow')
    df_current_portion_lease_10k.to_excel(writer, sheet_name='current_portion_lease')
    df_current_portion_debt_10k.to_excel(writer, sheet_name='current_portion_debt')


### Scraping Amazon 10-Q

In [ ]:
# Número dos documentos que contém a DRE, BP e FC 
acc_documents_10q = f_sec.get_filtered_filings(ticker=ticker, ten_k=False, just_accession_numbers=True)

# Trocando os '-' para '' das strings ('0001045810-24-000264' -> '000104581024000264')
dict_acc_num_10q = {k: v.replace('-', '') for k, v in acc_documents_10q.items()}

# Transformando o dicionário em um df
df_lst_acc_num_10q_amzn = pd.DataFrame.from_dict(dict_acc_num_10q, orient='index', columns=['acc_num'])

# Transformando em datetime o index
df_lst_acc_num_10q_amzn.index = pd.to_datetime(df_lst_acc_num_10q_amzn.index, format='%Y-%m-%d')

df_lst_acc_num_10q_amzn

In [ ]:
# Selecionando os 'acc_num' dos 10-q a partir de '2020-03-31' p/ iteração
df_lst_acc_num_10q_amzn_sliced = df_lst_acc_num_10q_amzn.loc['2020-03-31':]
df_lst_acc_num_10q_amzn_sliced

In [ ]:
# Rótulos
labels_dict = f_sec.get_label_dictionary(ticker, headers=f_sec.header)

In [ ]:
# Fazendo o scraping do balance sheet (10-Q)
balance_sheet_lst = []

for acc_num in df_lst_acc_num_10q_amzn_sliced['acc_num']:

    # Balance sheet
    balance_sheet = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='balance_sheet')
    balance_sheet = f_sec.rename_statement(balance_sheet, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    balance_sheet = balance_sheet.iloc[:, 0]
    balance_sheet_lst.append(balance_sheet)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_balance_sheet_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in balance_sheet_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_balance_sheet_10q = pd.concat([df_balance_sheet_10q, df], axis=1)

# Substituindo os NaN por 0
df_balance_sheet_10q = df_balance_sheet_10q.fillna(0)

# A linha 'Treasury Stock, Common, Value' nas colunas '2022-06-30', '2022-03-31', '2021-09-30', '2021-06-30', '2021-03-31'. '2020-09-30', '2020-06-30' e '2020-03-31' estão zeradas
# As informações estão na linha 'Treasury Stock, Value' 
df_balance_sheet_10q.loc['Treasury Stock, Common, Value', '2022-06-30'] = df_balance_sheet_10q.loc['Treasury Stock, Value', '2022-06-30'] 
df_balance_sheet_10q.loc['Treasury Stock, Common, Value', '2022-03-31'] = df_balance_sheet_10q.loc['Treasury Stock, Value', '2022-03-31'] 
df_balance_sheet_10q.loc['Treasury Stock, Common, Value', '2021-09-30'] = df_balance_sheet_10q.loc['Treasury Stock, Value', '2021-09-30'] 
df_balance_sheet_10q.loc['Treasury Stock, Common, Value', '2021-06-30'] = df_balance_sheet_10q.loc['Treasury Stock, Value', '2021-06-30'] 
df_balance_sheet_10q.loc['Treasury Stock, Common, Value', '2021-03-31'] = df_balance_sheet_10q.loc['Treasury Stock, Value', '2021-03-31'] 
df_balance_sheet_10q.loc['Treasury Stock, Common, Value', '2020-09-30'] = df_balance_sheet_10q.loc['Treasury Stock, Value', '2020-09-30'] 
df_balance_sheet_10q.loc['Treasury Stock, Common, Value', '2020-06-30'] = df_balance_sheet_10q.loc['Treasury Stock, Value', '2020-06-30'] 
df_balance_sheet_10q.loc['Treasury Stock, Common, Value', '2020-03-31'] = df_balance_sheet_10q.loc['Treasury Stock, Value', '2020-03-31'] 

# A linha 'Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization' nas colunas '2020-09-30', '2020-06-30' e '2020-03-31' estão zeradas
# As informações estão na linha 'Property, Plant and Equipment, Net' 
df_balance_sheet_10q.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2020-09-30'] = df_balance_sheet_10q.loc['Property, Plant and Equipment, Net', '2020-09-30'] 
df_balance_sheet_10q.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2020-06-30'] = df_balance_sheet_10q.loc['Property, Plant and Equipment, Net', '2020-06-30'] 
df_balance_sheet_10q.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2020-03-31'] = df_balance_sheet_10q.loc['Property, Plant and Equipment, Net', '2020-03-31'] 

# # Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_balance_sheet_10q = df_balance_sheet_10q.drop([
    'Treasury Stock, Value',
    'Property, Plant and Equipment, Net'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_balance_sheet_10q = df_balance_sheet_10q.drop([
    'us-gaap_AssetsCurrentAbstract',
    'Preferred Stock, Value, Issued'
])

# Renomeando a linha do índice que está sem formato 
df_balance_sheet_10q = df_balance_sheet_10q.rename(index={
    'amzn_LeaseLiabilityNoncurrent': 'Lease Liability Noncurrent'
})

df_balance_sheet_10q

In [ ]:
# Fazendo o scraping do income statement (10-Q)
income_statement_lst = []

for acc_num in df_lst_acc_num_10q_amzn_sliced['acc_num']:

    # Income statement
    income_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='income_statement')
    income_statement = f_sec.rename_statement(income_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    income_statement = income_statement.iloc[:, 0]
    income_statement_lst.append(income_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_income_statement_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in income_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_income_statement_10q = pd.concat([df_income_statement_10q, df], axis=1)

# Substituindo os NaN por 0
df_income_statement_10q = df_income_statement_10q.fillna(0)

# A linha 'InterestExpenseNonoperating' nas colunas '2024-03-31', '2023-09-30', '2023-06-30', '2023-03-31', '2022-09-30', '2022-06-30', '2022-03-31', '2021-09-30', '2021-06-30', '2021-03-31', '2020-09-30', '2020-06-30' e '2020-03-31' estao zeradas
# As informações estão na linha 'Interest Expense'
df_income_statement_10q.loc['InterestExpenseNonoperating', '2024-03-31'] = df_income_statement_10q.loc['Interest Expense', '2024-03-31'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2023-09-30'] = df_income_statement_10q.loc['Interest Expense', '2023-09-30'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2023-06-30'] = df_income_statement_10q.loc['Interest Expense', '2023-06-30'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2023-03-31'] = df_income_statement_10q.loc['Interest Expense', '2023-03-31'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2022-09-30'] = df_income_statement_10q.loc['Interest Expense', '2022-09-30'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2022-06-30'] = df_income_statement_10q.loc['Interest Expense', '2022-06-30'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2022-03-31'] = df_income_statement_10q.loc['Interest Expense', '2022-03-31'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2021-09-30'] = df_income_statement_10q.loc['Interest Expense', '2021-09-30'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2021-06-30'] = df_income_statement_10q.loc['Interest Expense', '2021-06-30'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2021-03-31'] = df_income_statement_10q.loc['Interest Expense', '2021-03-31'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2020-09-30'] = df_income_statement_10q.loc['Interest Expense', '2020-09-30'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2020-06-30'] = df_income_statement_10q.loc['Interest Expense', '2020-06-30'] 
df_income_statement_10q.loc['InterestExpenseNonoperating', '2020-03-31'] = df_income_statement_10q.loc['Interest Expense', '2020-03-31'] 

# A linha 'amzn_TechnologyAndInfrastructureExpense' nas colunas '2023-03-31', '2022-09-30', '2022-06-30', '2022-03-31', '2021-09-30', '2021-06-30', '2021-03-31', '2020-09-30', '2020-06-30' e '2020-03-31' estao zeradas
# As informações estão na linha 'amzn_TechnologyAndContentExpense'
df_income_statement_10q.loc['amzn_TechnologyAndInfrastructureExpense', '2023-03-31'] = df_income_statement_10q.loc['amzn_TechnologyAndContentExpense', '2023-03-31'] 
df_income_statement_10q.loc['amzn_TechnologyAndInfrastructureExpense', '2022-09-30'] = df_income_statement_10q.loc['amzn_TechnologyAndContentExpense', '2022-09-30'] 
df_income_statement_10q.loc['amzn_TechnologyAndInfrastructureExpense', '2022-06-30'] = df_income_statement_10q.loc['amzn_TechnologyAndContentExpense', '2022-06-30'] 
df_income_statement_10q.loc['amzn_TechnologyAndInfrastructureExpense', '2022-03-31'] = df_income_statement_10q.loc['amzn_TechnologyAndContentExpense', '2022-03-31'] 
df_income_statement_10q.loc['amzn_TechnologyAndInfrastructureExpense', '2021-09-30'] = df_income_statement_10q.loc['amzn_TechnologyAndContentExpense', '2021-09-30'] 
df_income_statement_10q.loc['amzn_TechnologyAndInfrastructureExpense', '2021-06-30'] = df_income_statement_10q.loc['amzn_TechnologyAndContentExpense', '2021-06-30'] 
df_income_statement_10q.loc['amzn_TechnologyAndInfrastructureExpense', '2021-03-31'] = df_income_statement_10q.loc['amzn_TechnologyAndContentExpense', '2021-03-31'] 
df_income_statement_10q.loc['amzn_TechnologyAndInfrastructureExpense', '2020-09-30'] = df_income_statement_10q.loc['amzn_TechnologyAndContentExpense', '2020-09-30'] 
df_income_statement_10q.loc['amzn_TechnologyAndInfrastructureExpense', '2020-06-30'] = df_income_statement_10q.loc['amzn_TechnologyAndContentExpense', '2020-06-30'] 
df_income_statement_10q.loc['amzn_TechnologyAndInfrastructureExpense', '2020-03-31'] = df_income_statement_10q.loc['amzn_TechnologyAndContentExpense', '2020-03-31'] 

# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_income_statement_10q = df_income_statement_10q.drop([
    'Interest Expense',
    'amzn_TechnologyAndContentExpense'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_income_statement_10q = df_income_statement_10q.drop('us-gaap_CostsAndExpensesAbstract')

# Renomeando a linha do índice que está sem formato 
df_income_statement_10q = df_income_statement_10q.rename(index={
    'amzn_TechnologyAndInfrastructureExpense': 'Technology And Infrastructure Expense',
    'InterestExpenseNonoperating': 'Interest Expense',
})

# Formatando o número para ficar positivo
df_income_statement_10q.loc['Interest Expense'] = df_income_statement_10q.loc['Interest Expense'] * -1
df_income_statement_10q.loc['Income Tax Expense (Benefit)'] = df_income_statement_10q.loc['Income Tax Expense (Benefit)'] * -1

df_income_statement_10q

In [ ]:
# Fazendo o scraping do cash flow statement (10-Q)
cash_flow_statement_lst = []

for acc_num in df_lst_acc_num_10q_amzn_sliced['acc_num']:

    # Cash flow statement
    cash_flow_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='cash_flow_statement')
    cash_flow_statement = f_sec.rename_statement(cash_flow_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    cash_flow_statement = cash_flow_statement.iloc[:, 0]
    cash_flow_statement_lst.append(cash_flow_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_cash_flow_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in cash_flow_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_cash_flow_10q = pd.concat([df_cash_flow_10q, df], axis=1)

# Substituindo os NaN por 0
df_cash_flow_10q = df_cash_flow_10q.fillna(0)

# A linha 'Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations' nas colunas '2023-03-31', '2022-09-30', '2022-06-30', '2022-03-31', '2021-09-30', '2021-06-30', '2021-03-31', '2020-09-30', '2020-06-30' e '2020-03-31' estão zeradas
# As informações estão nas linhas 'Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents' e 'Effect of Exchange Rate on Cash and Cash Equivalents'
df_cash_flow_10q.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations', '2022-03-31'] = df_cash_flow_10q.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2022-03-31'] 
df_cash_flow_10q.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations', '2021-09-30'] = df_cash_flow_10q.loc['Effect of Exchange Rate on Cash and Cash Equivalents', '2021-09-30'] 
df_cash_flow_10q.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations', '2021-06-30'] = df_cash_flow_10q.loc['Effect of Exchange Rate on Cash and Cash Equivalents', '2021-06-30'] 
df_cash_flow_10q.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations', '2021-03-31'] = df_cash_flow_10q.loc['Effect of Exchange Rate on Cash and Cash Equivalents', '2021-03-31'] 
df_cash_flow_10q.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations', '2020-09-30'] = df_cash_flow_10q.loc['Effect of Exchange Rate on Cash and Cash Equivalents', '2020-09-30'] 
df_cash_flow_10q.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations', '2020-06-30'] = df_cash_flow_10q.loc['Effect of Exchange Rate on Cash and Cash Equivalents', '2020-06-30'] 
df_cash_flow_10q.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations', '2020-03-31'] = df_cash_flow_10q.loc['Effect of Exchange Rate on Cash and Cash Equivalents', '2020-03-31'] 

# A linha 'amzn_ProceedsFromPropertyPlantAndEquipmentSalesAndIncentives' nas colunas '2021-03-31', '2020-09-30', '2020-06-30' e '2020-03-31' estão zeradas
# As informações estão na linha 'amzn_ProceedsFromRebatesOnPurchasesOfProductiveAssets'
df_cash_flow_10q.loc['amzn_ProceedsFromPropertyPlantAndEquipmentSalesAndIncentives', '2021-03-31'] = df_cash_flow_10q.loc['amzn_ProceedsFromRebatesOnPurchasesOfProductiveAssets', '2021-03-31'] 
df_cash_flow_10q.loc['amzn_ProceedsFromPropertyPlantAndEquipmentSalesAndIncentives', '2020-09-30'] = df_cash_flow_10q.loc['amzn_ProceedsFromRebatesOnPurchasesOfProductiveAssets', '2020-09-30'] 
df_cash_flow_10q.loc['amzn_ProceedsFromPropertyPlantAndEquipmentSalesAndIncentives', '2020-06-30'] = df_cash_flow_10q.loc['amzn_ProceedsFromRebatesOnPurchasesOfProductiveAssets', '2020-06-30'] 
df_cash_flow_10q.loc['amzn_ProceedsFromPropertyPlantAndEquipmentSalesAndIncentives', '2020-03-31'] = df_cash_flow_10q.loc['amzn_ProceedsFromRebatesOnPurchasesOfProductiveAssets', '2020-03-31'] 

# A linha 'Payments to Acquire Businesses, Net of Cash Acquired' na coluna '2023-09-30' está zerada
# As informações estão na linha 'amzn_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPaymentsToAcquireNonmarketableSecuritiesAndOther'
df_cash_flow_10q.loc['Payments to Acquire Businesses, Net of Cash Acquired', '2023-09-30'] = df_cash_flow_10q.loc['amzn_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPaymentsToAcquireNonmarketableSecuritiesAndOther', '2023-09-30'] 

# A linha 'Proceeds from Short-term Debt' na coluna '2020-03-31' está zerada
# As informações estão na linha 'amzn_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPaymentsToAcquireNonmarketableSecuritiesAndOther'
df_cash_flow_10q.loc['Proceeds from Short-term Debt', '2020-03-31'] = df_cash_flow_10q.loc['Repayments of Debt', '2020-03-31'] 


# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_cash_flow_10q = df_cash_flow_10q.drop([
    'Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents',
    'Effect of Exchange Rate on Cash and Cash Equivalents',
    'amzn_ProceedsFromRebatesOnPurchasesOfProductiveAssets',
    'amzn_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPaymentsToAcquireNonmarketableSecuritiesAndOther',
    'Repayments of Debt'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_cash_flow_10q = df_cash_flow_10q.drop([
    'us-gaap_StatementOfCashFlowsAbstract',
    'amzn_InterestPaidOnLongTermDebt',
    'Operating Lease, Payments',
    'Finance Lease, Interest Payment on Liability',
    'amzn_InterestPaidFinancingObligations',
    'Income Taxes Paid, Net',
    'Right-of-Use Asset Obtained in Exchange for Operating Lease Liability',
    'Right-of-Use Asset Obtained in Exchange for Finance Lease Liability',
    'amzn_PropertyandEquipmentObtainedinExchangeforFinancingObligations'
])

# Arrumando e adicionando alguns números no df
df_cash_flow_10q.loc['Repayments of Short-term Debt', '2020-03-31'] = -631
df_cash_flow_10q.loc['Repayments of Long-term Debt', '2020-03-31'] = -36

df_cash_flow_10q.loc['Increase (Decrease) in Other Noncurrent Assets', '2023-09-30'] = -3134
df_cash_flow_10q.loc['Increase (Decrease) in Other Noncurrent Assets', '2023-06-30'] = -3126
df_cash_flow_10q.loc['Increase (Decrease) in Other Noncurrent Assets', '2023-03-31'] = -3203

# Renomeando a linha do índice que está sem formato 
df_cash_flow_10q = df_cash_flow_10q.rename(index={
    'amzn_ProceedsFromPropertyPlantAndEquipmentSalesAndIncentives': 'Proceeds From Property Plant And Equipment Sales And Incentives',
    'amzn_RepaymentsOfLongTermFinancingObligations': 'Repayments Of Long Term Financing Obligations',
})

df_cash_flow_10q

In [ ]:
# Arrumando alguns index dos dfs para ficar padronizado com os index utilizados nas funções p/ calcular os indicadores fundamentalistas
df_balance_sheet_10q = df_balance_sheet_10q.rename(index={
    'Cash and Cash Equivalents, at Carrying Value': 'Cash and Cash Equivalents',
    'Marketable Securities, Current': 'Marketable Securities',
    'Lease Liability Noncurrent': 'Long-term Lease Liabilities',
    'Long-term Debt, Excluding Current Maturities': 'Long-term Debt',
    "Stockholders' Equity Attributable to Parent": 'Total Shareholders Equity'
})

df_income_statement_10q = df_income_statement_10q.rename(index={
    'Revenue from Contract with Customer, Excluding Assessed Tax': 'Revenues',
    'Technology And Infrastructure Expense': 'Research and Development',
    'Operating Income (Loss)': 'Operating Income',
    'Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest': 'Income Before Provision for Income Taxes',
    'Income Tax Expense (Benefit)': 'Provision for Income Taxes',
    'Net Income (Loss) Attributable to Parent': 'Net Income',
    'Weighted Average Number of Shares Outstanding, Basic': 'Number of Shares - Basic',
    'Weighted Average Number of Shares Outstanding, Diluted': 'Number of Shares - Diluted'
})

df_cash_flow_10q = df_cash_flow_10q.rename(index={
    'Payments to Acquire Productive Assets': 'Purchases Of Property And Equipment And Intangible Assets'
})

In [ ]:
# Fazendo o scraping do 'current portion of lease liabilities'
current_portion_lease_lst = []

for acc_num in df_lst_acc_num_10q_amzn_sliced['acc_num']:
    # Fazendo o scraping do 'current portion of lease liabilities' (10-K) -> 'Note 4 – Leases' 
    current_portion_lease_10q = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='current_portion_lease')
    current_portion_lease_10q = f_sec.rename_statement(current_portion_lease_10q, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    current_portion_lease_10q = current_portion_lease_10q.iloc[:, 0]
    current_portion_lease_lst.append(current_portion_lease_10q)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_current_portion_lease_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in current_portion_lease_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_current_portion_lease_10q = pd.concat([df_current_portion_lease_10q, df], axis=1)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_current_portion_lease_10q = df_current_portion_lease_10q.drop('us-gaap_LeasesAbstract')

# Renomeando a linha do índice
df_current_portion_lease_10q = df_current_portion_lease_10q.rename(index={
    'amzn_LeaseLiabilityCurrent': 'Current Portion of Lease Liabilities',
})

# Formatando o número para ficar positivo
df_current_portion_lease_10q.loc['Current Portion of Lease Liabilities'] = df_current_portion_lease_10q.loc['Current Portion of Lease Liabilities'] * -1

# Selecionando a linha desejada
df_current_portion_lease_10q = df_current_portion_lease_10q.loc[['Current Portion of Lease Liabilities']]

df_current_portion_lease_10q

In [ ]:
# Fazendo o scraping do 'Current portion of long-term debt' (10-K)
current_portion_debt_lst = []

for acc_num in df_lst_acc_num_10q_amzn_sliced['acc_num']:
    # Fazendo o scraping do 'Current portion of long-term debt' (10-K) -> # 'Note 6. Debt' 
    current_portion_debt_10q = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='current_portion_debt')
    current_portion_debt_10q = f_sec.rename_statement(current_portion_debt_10q, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    current_portion_debt_10q = current_portion_debt_10q.iloc[:, 0]
    current_portion_debt_lst.append(current_portion_debt_10q)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_current_portion_debt_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in current_portion_debt_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_current_portion_debt_10q = pd.concat([df_current_portion_debt_10q, df], axis=1)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_current_portion_debt_10q = df_current_portion_debt_10q.drop('us-gaap_DebtInstrumentLineItems')

# Renomeando a linha do índice
df_current_portion_debt_10q = df_current_portion_debt_10q.rename(index={
    'Long-term Debt, Current Maturities': 'Current Portion of Long-term Debt',
})

# Formatando o número para ficar positivo
df_current_portion_debt_10q.loc['Current Portion of Long-term Debt'] = df_current_portion_debt_10q.loc['Current Portion of Long-term Debt'] * -1

# Arrumando a escala de algumas colunas
df_current_portion_debt_10q.loc[:, '2020-03-31'] = df_current_portion_debt_10q.loc[:, '2020-03-31'] / 1000000
df_current_portion_debt_10q.loc[:, '2020-06-30'] = df_current_portion_debt_10q.loc[:, '2020-06-30'] / 1000000
df_current_portion_debt_10q.loc[:, '2020-09-30'] = df_current_portion_debt_10q.loc[:, '2020-09-30'] / 1000000
df_current_portion_debt_10q.loc[:, '2021-03-31'] = df_current_portion_debt_10q.loc[:, '2021-03-31'] / 1000000
df_current_portion_debt_10q.loc[:, '2021-06-30'] = df_current_portion_debt_10q.loc[:, '2021-06-30'] / 1000000
df_current_portion_debt_10q.loc[:, '2021-09-30'] = df_current_portion_debt_10q.loc[:, '2021-09-30'] / 1000000
df_current_portion_debt_10q.loc[:, '2022-03-31'] = df_current_portion_debt_10q.loc[:, '2022-03-31'] / 1000000

# Selecionando a linha desejada
df_current_portion_debt_10q = df_current_portion_debt_10q.loc[['Current Portion of Long-term Debt']]

df_current_portion_debt_10q

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//tecnologia//amazon//amzn_10q.xlsx') as writer:
    df_balance_sheet_10q.to_excel(writer, sheet_name='balance_sheet')
    df_income_statement_10q.to_excel(writer, sheet_name='income_statement')
    df_cash_flow_10q.to_excel(writer, sheet_name='cash_flow')
    df_current_portion_lease_10q.to_excel(writer, sheet_name='current_portion_lease')
    df_current_portion_debt_10q.to_excel(writer, sheet_name='current_portion_debt')

### Indicadores Fundamentalistas

#### Indicadores 10-K

In [ ]:
# Lendo os arquivos 10k
amzn_bs = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//amazon//amzn_10k.xlsx', 
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

amzn_is = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//amazon//amzn_10k.xlsx', 
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

amzn_cf = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//amazon//amzn_10k.xlsx', 
    sheet_name='cash_flow', 
    index_col='Unnamed: 0'
)

amzn_current_portion_lease = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//amazon//amzn_10k.xlsx', 
    sheet_name='current_portion_lease', 
    index_col='Unnamed: 0'
)

amzn_current_portion_debt = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//amazon//amzn_10k.xlsx', 
    sheet_name='current_portion_debt', 
    index_col='Unnamed: 0'
)

# Transformando as colunas em datetime
amzn_bs.columns = pd.to_datetime(amzn_bs.columns)
amzn_is.columns = pd.to_datetime(amzn_is.columns)
amzn_cf.columns = pd.to_datetime(amzn_cf.columns)
amzn_current_portion_lease.columns = pd.to_datetime(amzn_current_portion_lease.columns)
amzn_current_portion_debt.columns = pd.to_datetime(amzn_current_portion_debt.columns)

# Transformando as colunas em apenas o ano
amzn_bs.columns = amzn_bs.columns.year
amzn_is.columns = amzn_is.columns.year
amzn_cf.columns = amzn_cf.columns.year
amzn_current_portion_lease.columns = amzn_current_portion_lease.columns.year
amzn_current_portion_debt.columns = amzn_current_portion_debt.columns.year

In [ ]:
# Preços da AMZN
amzn_close = yf.download('AMZN', auto_adjust=True, multi_level_index=False, start='2017-01-01')['Close']

# Selecionando o último preço do mês de Dezembro de cada ano 
lst_years = [
    '2017', '2018', '2019', '2020', 
    '2021', '2022', '2023', '2024',
    '2025',
]

last_dec_close = []

for year in lst_years:
    last_close = amzn_close[f'{year}-12'].iloc[-1]
    last_dec_close.append(last_close)

# Transformando em uma série pandas
amzn_close_dec = pd.Series(last_dec_close, index=lst_years)

# Revertendo o index da série p/ ficar igual com os dos indicadores
amzn_close_dec = amzn_close_dec.reindex(index=amzn_close_dec.index[::-1])

# Arredondamento de duas casas decimais
amzn_close_dec = round(amzn_close_dec, 2)

# Teve um split de 20:1 em 06/06/2022 (https://br.investing.com/equities/amazon-com-inc-historical-data-splits)
amzn_close_dec.loc['2021'] = amzn_close_dec.loc['2021'] * 20
amzn_close_dec.loc['2020'] = amzn_close_dec.loc['2020'] * 20
amzn_close_dec.loc['2019'] = amzn_close_dec.loc['2019'] * 20
amzn_close_dec.loc['2018'] = amzn_close_dec.loc['2018'] * 20
amzn_close_dec.loc['2017'] = amzn_close_dec.loc['2017'] * 20

# Transformando o index em int
amzn_close_dec .index = amzn_close_dec .index.astype(int)

amzn_close_dec

In [ ]:
# Criando uma lista só para fazer a indexação em que o ano mais recente está no começo
idx = amzn_bs.columns

# Valor de Mercado
vm = f_eua.indicador_vm(ten_k=True, price=amzn_close_dec, df_is=amzn_is) 

# LPA
lpa = f_eua.indicador_lpa(ten_k=True, df_is=amzn_is, idx=None)

# P/L Damodaran
p_lpa_damodaran = f_eua.indicador_p_lpa_2(ten_k=True, vm=vm, df_is=amzn_is)

# P/L
p_lpa = f_eua.indicador_p_lpa(ten_k=True, price=amzn_close_dec, lpa=lpa, idx=idx)

# L/P
lpa_p = f_eua.indicador_lpa_p(ten_k=True, price=amzn_close_dec, lpa=lpa, idx=idx) 

# Depreciação
depreciacao = f_eua.indicador_depreciacao(ten_k=True, df_cf=amzn_cf, first_quarter=None, idx=idx)

# EBITDA
ebitda = f_eua.indicador_ebitda(ten_k=True, df_is=amzn_is, idx=None, depreciacao=depreciacao)

# Margem Líquida
m_liq = f_eua.indicador_ml(ten_k=True, df_is=amzn_is)

# VPA
vpa = f_eua.indicador_vpa(ten_k=True, df_bs=amzn_bs, df_is=amzn_is)

# P/VPA
p_vpa = f_eua.indicador_p_vpa(ten_k=True, price=amzn_close_dec, vpa=vpa, idx=idx)

# Caixa e Equivalentes
lst_caixa = ['Cash and Cash Equivalents', 'Marketable Securities']
caixa = f_eua.indicador_caixa(ten_k=True, df_bs=amzn_bs, lst_itens_caixa=lst_caixa)

# Dívida Bruta
lst_divida = ['Long-term Lease Liabilities', 'Long-term Debt']
div_bruta = f_eua.indicador_divida_bruta(ten_k=True, df_bs=amzn_bs, lst_itens_divida=lst_divida)
div_bruta_total = div_bruta.values + amzn_current_portion_lease.values + amzn_current_portion_debt.values
div_bruta_total = pd.Series(div_bruta_total[0], index=idx)

# Dívida Líquida
div_liq = f_eua.indicador_divida_liquida(ten_k=True, caixa=caixa, div_bruta=div_bruta_total, idx=idx)

# Dívida Líquida / EBITDA
dl_ebitda = f_eua.indicador_dl_ebitda(ten_k=True, ebitda=ebitda, div_liq=div_liq, idx=None)

# Dívida Líquida / PL
dl_pl = f_eua.indicador_dl_pl(ten_k=True, div_liq=div_liq, df_bs=amzn_bs) 

# EV/EBITDA
ev_ebitda = f_eua.indicador_ev_ebitda(vm=vm, div_liq=div_liq, ebitda=ebitda)

# ROE
roe = f_eua.indicador_roe(ten_k=True, df_is=amzn_is, df_bs=amzn_bs, idx=None)

# ROIC
roic = f_eua.indicador_roic(ten_k=True, df_is=amzn_is, df_bs=amzn_bs, div_bruta=div_bruta_total, idx=idx)

# FCO
fco = f_eua.indicador_fco(ten_k=True, df_cf=amzn_cf, first_quarter=None) 

# FCI
fci = f_eua.indicador_fci(ten_k=True, df_cf=amzn_cf, first_quarter=None) 

# FCF
fcf = f_eua.indicador_fcf(ten_k=True, df_cf=amzn_cf, first_quarter=None) 

# CAPEX
lst_capex = ['Purchases Of Property And Equipment And Intangible Assets']

capex= f_eua.indicador_capex(ten_k=True, df_cf=amzn_cf, first_quarter=None, lst_itens=lst_capex, idx=idx)

# Free Cash Flow
free_cash_flow = f_eua.indicador_free_cash_flow(fco=fco, capex=capex)

# Net CAPEX
net_capex = f_eua.indicador_net_capex(capex=capex, depreciation=depreciacao)

# R&D
rd_expense = f_eua.indicador_rd(ten_k=True, df_is=amzn_is, idx=idx)

# Net CAPEX ajustado
adj_net_capex = f_eua.indicador_adj_net_capex(net_capex=net_capex, rd_expense=rd_expense)

# Working capital
working_capital = f_eua.indicador_working_capital(ten_k=True, df_bs=amzn_bs)

# Change in non cash working capital
lst_itens_change_non_cash_wc = [
    'Deferred Income Tax Expense (Benefit)',
    'Increase (Decrease) in Inventories',
    'Increase (Decrease) in Accounts Receivable and Other Operating Assets',
    'Increase (Decrease) in Other Noncurrent Assets',
    'Increase (Decrease) in Accounts Payable',
    'Increase (Decrease) in Accrued Liabilities and Other Operating Liabilities',
    'Increase (Decrease) in Contract with Customer, Liability'
]

change_non_cash_wc = f_eua.indicador_change_non_cash_wc(ten_k=True, df_cf=amzn_cf, lst_itens=lst_itens_change_non_cash_wc, first_quarter=None)

# Reinvestment rate
reinvestment_rate = f_eua.indicador_reinvestment_rate(ten_k=True, net_capex=net_capex, change_non_cash_wc=change_non_cash_wc, df_is=amzn_is, idx=idx)

# New borrowing
lst_itens_new_borrowing = ['Proceeds from Short-term Debt', 'Proceeds from Issuance of Long-term Debt']

new_borrowing = f_eua.indicador_new_borrowing(ten_k=True, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=amzn_cf, first_quarter=None, idx=idx)

# Debt paid
lst_itens_debt_paid = [
    'Repayments of Short-term Debt', 
    'Repayments of Long-term Debt', 
    'Finance Lease, Principal Payments', 
    'Repayments Of Long Term Financing Obligations'
]

debt_paid = f_eua.indicador_debt_paid(ten_k=True, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=amzn_cf, first_quarter=None, idx=idx)

# Taxes on operating income
taxes_operating_income = f_eua.indicador_taxes_operating_income(ten_k=True, df_is=amzn_is, idx=idx)

# FCFE
fcfe_10k = f_eua.indicador_fcfe(
    ten_k=True, 
    depreciacao=depreciacao, 
    capex=capex, 
    change_non_cash_wc=change_non_cash_wc, 
    new_borrowing=new_borrowing,
    debt_paid=debt_paid,
    df_is=amzn_is,
    idx=idx
)

# FCFF
fcff_10k = f_eua.indicador_fcff(
        ten_k=True, 
        taxes_operating_income=taxes_operating_income,
        depreciacao=depreciacao, 
        capex=capex, 
        change_non_cash_wc=change_non_cash_wc, 
        df_is=amzn_is,
        idx=idx
)

# DPA e Payout - Amazon não paga dividendo

# Buyback
buyback = f_eua.indicador_buyback(ten_k=True, df_cf=amzn_cf, first_quarter=None)

# Colocando todos os indicadores em uma lista
lst_indicadores = [
    p_lpa_damodaran,
    p_lpa,
    lpa_p,  
    ebitda, 
    m_liq,
    p_vpa,
    vm,
    div_bruta_total,
    caixa,
    div_liq,
    dl_ebitda,
    dl_pl,
    ev_ebitda,
    roe,
    roic,
    fco,
    fci,
    fcf,
    free_cash_flow,
    capex,
    net_capex,
    rd_expense,
    adj_net_capex,
    working_capital,
    reinvestment_rate,
    fcfe_10k,
    fcff_10k,
    buyback
]

# Juntando todos os indicadores em um df
amzn_indicadores_10k = pd.concat(lst_indicadores, axis=1)

# Nomes das colunas
nomes_col = [
    'P/L Damodaran', 
    'P/L', 
    'L/P', 
    'EBITDA', 
    'Margem Líquida',
    'P/VPA',
    'Valor de Mercado',
    'Dívida Bruta',
    'Caixa e Equivalentes',
    'Dívida Líquida',
    'Dívida Líquida/EBITDA',
    'Dívida Líquida/PL',
    'EV/EBITDA',
    'ROE',
    'ROIC',
    'FCO',
    'FCI',
    'FCF',
    'Free Cash Flow',
    'Capex',
    'Net Capex',
    'R&D',
    'Adj Net Capex',
    'Working Capital',
    'Reinvestment Rate',
    'FCFE',
    'FCFF',
    'Buyback'
]

# Atribuindo os novos nomes das colunas do df
amzn_indicadores_10k.columns = nomes_col
amzn_indicadores_10k

#### Indicadores 10-Q

In [ ]:
# Lendo os arquivos 10q
amzn_bs_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//amazon//amzn_10q.xlsx',
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

amzn_is_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//amazon//amzn_10q.xlsx',
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

amzn_cf_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//amazon//amzn_10q.xlsx',
    sheet_name='cash_flow',  
    index_col='Unnamed: 0'
)

amzn_current_portion_lease_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//amazon//amzn_10q.xlsx', 
    sheet_name='current_portion_lease', 
    index_col='Unnamed: 0'
)

amzn_current_portion_debt_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//amazon//amzn_10q.xlsx', 
    sheet_name='current_portion_debt', 
    index_col='Unnamed: 0'
)

# Aplicando o ajuste nas colunas
amzn_bs_10q.columns = [f_sec.ajustar_mes_03_06_09(col) for col in amzn_bs_10q.columns]
amzn_is_10q.columns = [f_sec.ajustar_mes_03_06_09(col) for col in amzn_is_10q.columns]
amzn_cf_10q.columns = [f_sec.ajustar_mes_03_06_09(col) for col in amzn_cf_10q.columns]
amzn_current_portion_lease_10q.columns = [f_sec.ajustar_mes_03_06_09(col) for col in amzn_current_portion_lease_10q.columns]
amzn_current_portion_debt_10q.columns = [f_sec.ajustar_mes_03_06_09(col) for col in amzn_current_portion_debt_10q.columns]

In [ ]:
# Preços da AMZN
amzn_close = yf.download('AMZN', auto_adjust=True, multi_level_index=False, start='2021-01-01')['Close']

# Selecionando o último preço dos meses de Março, Junho e Setembro
lst_years = [
    '2021', '2022', '2023', '2024',
    '2025',
]
lst_months = ['03', '06', '09']
last_close_months = []

for year in lst_years:
    for month in lst_months:
        last_close = amzn_close[f'{year}-{month}'].dropna().iloc[-1]  # Dropna para lidar com meses sem dados
        last_close_months.append(last_close)

# Criando uma lista em que junte os meses e os anos para que seja o index da serie pandas
lst_idx = []

for year in lst_years:
    for month in lst_months:
        lst_idx.append(f"{month}-{year}")

# Transformando em uma série pandas
amzn_close_months = pd.Series(last_close_months, index=lst_idx)

# Revertendo o index da série p/ ficar igual com os dos indicadores
amzn_close_months = amzn_close_months.reindex(index=amzn_close_months.index[::-1])

# No ano mais atual, como ainda ele não acabou, não consigo fazer o loop com todos meses (Março, Junho e Setembro)
actual_year_int = datetime.now().year  
actual_year_str = str(actual_year_int) 

lst_actual_months = [
    # '03',
    # '06', 
    # '09'
] 

last_close_actual_months = []

for month in lst_actual_months:
    last_close = amzn_close[f'{actual_year_str}-{month}'].dropna().iloc[-1]  # Dropna para lidar com meses sem dados
    last_close_actual_months.append(last_close)

# Criando uma lista em que junte os meses e os anos para que seja o index da serie pandas
lst_idx_actual = []

for month in lst_actual_months:
    lst_idx_actual.append(f"{month}-{actual_year_str}")

# Transformando em uma série pandas
amzn_close_months_atual = pd.Series(last_close_actual_months, index=lst_idx_actual)

# Revertendo o index da série p/ ficar igual com os dos indicadores
amzn_close_months_atual = amzn_close_months_atual.reindex(index=amzn_close_months_atual.index[::-1])

# Juntando as duas series pandas
amzn_close_months = pd.concat([amzn_close_months_atual, amzn_close_months])

# Arredondamento de duas casas decimais
amzn_close_months = round(amzn_close_months, 2)

# Teve um split de 20:1 em 06/06/2022 (https://br.investing.com/equities/amazon-com-inc-historical-data-splits)
amzn_close_months.loc['03-2022'] = amzn_close_months.loc['03-2022'] * 20
amzn_close_months.loc['09-2021'] = amzn_close_months.loc['09-2021'] * 20
amzn_close_months.loc['06-2021'] = amzn_close_months.loc['06-2021'] * 20
amzn_close_months.loc['03-2021'] = amzn_close_months.loc['03-2021'] * 20

# Substituindo o index para ficar igual ao dos dados fundamentalistas
amzn_close_months.index = amzn_bs_10q.columns[:-3]

amzn_close_months

In [ ]:
# Criando uma lista só para fazer a indexação em que o ano mais recente está no começo
idx_10q = amzn_bs_10q.columns

# Valor de Mercado
vm_10q = f_eua.indicador_vm(ten_k=False, price=amzn_close_months, df_is=amzn_is_10q)

# LPA
lpa_acum = f_eua.indicador_lpa(ten_k=False, df_is=amzn_is_10q, idx=idx_10q)[1]
lpa_10q = f_eua.indicador_lpa(ten_k=False, df_is=amzn_is_10q, idx=idx_10q)[0]

# P/L Damodaran
p_lpa_10q_damodaran = f_eua.indicador_p_lpa_2(ten_k=False, vm=vm_10q, df_is=amzn_is_10q)

# P/L
p_lpa_10q = f_eua.indicador_p_lpa(ten_k=False, price=amzn_close_months, lpa=lpa_acum, idx=idx_10q)

# L/P
lpa_p_10q = f_eua.indicador_lpa_p(ten_k=False, price=amzn_close_months, lpa=lpa_acum, idx=idx_10q) 

# Depreciação
depreciacao_10q = f_eua.indicador_depreciacao(ten_k=False, df_cf=amzn_cf_10q, first_quarter='03', idx=idx_10q)[0]
depreciacao_10q_sliced = f_eua.indicador_depreciacao(ten_k=False, df_cf=amzn_cf_10q, first_quarter='03', idx=idx_10q)[1]
depreciacao_10q_acum = f_eua.indicador_depreciacao(ten_k=False, df_cf=amzn_cf_10q, first_quarter='03', idx=idx_10q)[2]

# EBITDA
ebitda_10q = f_eua.indicador_ebitda(ten_k=False, df_is=amzn_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[0]
ebitda_10q_sliced = f_eua.indicador_ebitda(ten_k=False, df_is=amzn_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[1]
ebitda_10q_acum = f_eua.indicador_ebitda(ten_k=False, df_is=amzn_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[2]

# Margem Líquida
m_liq_10q = f_eua.indicador_ml(ten_k=False, df_is=amzn_is_10q)

# VPA
vpa_10q = f_eua.indicador_vpa(ten_k=False, df_bs=amzn_bs_10q, df_is=amzn_is_10q)

# P/VPA
p_vpa_10q = f_eua.indicador_p_vpa(ten_k=False, price=amzn_close_months, vpa=vpa_10q, idx=idx_10q)

# Caixa e Equivalentes
lst_caixa = ['Cash and Cash Equivalents', 'Marketable Securities']
caixa_10q = f_eua.indicador_caixa(ten_k=False, df_bs=amzn_bs_10q, lst_itens_caixa=lst_caixa)

# Dívida Bruta
lst_divida = ['Long-term Lease Liabilities', 'Long-term Debt']
div_bruta_10q = f_eua.indicador_divida_bruta(ten_k=False, df_bs=amzn_bs_10q, lst_itens_divida=lst_divida)
div_bruta_total_10q = div_bruta_10q.values + amzn_current_portion_lease_10q.iloc[:, 0:-3].values + amzn_current_portion_debt_10q.iloc[:, 0:-3].values
div_bruta_total_10q = pd.Series(div_bruta_total_10q[0], index=idx_10q[0:-3])

# Dívida Líquida
div_liq_10q = f_eua.indicador_divida_liquida(ten_k=False, caixa=caixa_10q, div_bruta=div_bruta_total_10q, idx=idx_10q)

# Dívida Líquida / EBITDA
dl_ebitda_10q = f_eua.indicador_dl_ebitda(ten_k=False, ebitda=ebitda_10q_acum, div_liq=div_liq_10q, idx=idx_10q)

# Dívida Líquida / PL
dl_pl_10q = f_eua.indicador_dl_pl(ten_k=False, div_liq=div_liq_10q, df_bs=amzn_bs_10q) 

# EV/EBITDA
ev_ebitda_10q = f_eua.indicador_ev_ebitda(vm=vm_10q, div_liq=div_liq_10q, ebitda=ebitda_10q_acum)

# ROE
roe_10q = f_eua.indicador_roe(ten_k=False, df_is=amzn_is_10q, df_bs=amzn_bs_10q, idx=idx_10q)

# ROIC 
roic_10q = f_eua.indicador_roic(ten_k=False, df_is=amzn_is_10q, df_bs=amzn_bs_10q, div_bruta=div_bruta_total_10q, idx=idx_10q)

# FCO
fco_10q = f_eua.indicador_fco(ten_k=False, df_cf=amzn_cf_10q, first_quarter='03') 

# FCI
fci_10q = f_eua.indicador_fci(ten_k=False, df_cf=amzn_cf_10q, first_quarter='03') 

# FCF
fcf_10q = f_eua.indicador_fcf(ten_k=False, df_cf=amzn_cf_10q, first_quarter='03') 

# CAPEX
lst_capex_10q = ['Purchases Of Property And Equipment And Intangible Assets']

capex_10q = f_eua.indicador_capex(ten_k=False, df_cf=amzn_cf_10q, first_quarter='03', lst_itens=lst_capex_10q, idx=idx_10q)[0]
capex_10q_sliced = f_eua.indicador_capex(ten_k=False, df_cf=amzn_cf_10q, first_quarter='03', lst_itens=lst_capex_10q, idx=idx_10q)[1]
capex_10q_acum = f_eua.indicador_capex(ten_k=False, df_cf=amzn_cf_10q, first_quarter='03', lst_itens=lst_capex_10q, idx=idx_10q)[2]

# Free Cash Flow
free_cash_flow_10q = f_eua.indicador_free_cash_flow(fco=fco_10q, capex=capex_10q_sliced)

# Net CAPEX
net_capex_10q = f_eua.indicador_net_capex(capex=capex_10q_acum, depreciation=depreciacao_10q_acum)

# R&D
rd_expense_10q = f_eua.indicador_rd(ten_k=False, df_is=amzn_is_10q, idx=idx_10q)[0]
rd_expense_10q_sliced = f_eua.indicador_rd(ten_k=False, df_is=amzn_is_10q, idx=idx_10q)[1]
rd_expense_10q_acum = f_eua.indicador_rd(ten_k=False, df_is=amzn_is_10q, idx=idx_10q)[2]

# Net CAPEX ajustado
adj_net_capex_10q = f_eua.indicador_adj_net_capex(net_capex=net_capex_10q, rd_expense=rd_expense_10q_acum)

# Working capital
working_capital_10q = f_eua.indicador_working_capital(ten_k=False, df_bs=amzn_bs_10q)

# Change in non cash working capital
lst_itens_change_non_cash_wc = [
    'Deferred Income Tax Expense (Benefit)',
    'Increase (Decrease) in Inventories',
    'Increase (Decrease) in Accounts Receivable and Other Operating Assets',
    'Increase (Decrease) in Other Noncurrent Assets',
    'Increase (Decrease) in Accounts Payable',
    'Increase (Decrease) in Accrued Liabilities and Other Operating Liabilities',
    'Increase (Decrease) in Contract with Customer, Liability'
]

change_non_cash_wc_10q = f_eua.indicador_change_non_cash_wc(ten_k=False, df_cf=amzn_cf_10q, lst_itens=lst_itens_change_non_cash_wc, first_quarter='03')[0]
change_non_cash_wc_10q_sliced = f_eua.indicador_change_non_cash_wc(ten_k=False, df_cf=amzn_cf_10q, lst_itens=lst_itens_change_non_cash_wc, first_quarter='03')[1]

# Reinvestment rate
reinvestment_rate_10q = f_eua.indicador_reinvestment_rate(ten_k=False, net_capex=adj_net_capex_10q, change_non_cash_wc=change_non_cash_wc_10q, df_is=amzn_is_10q, idx=idx_10q)

# New borrowing
lst_itens_new_borrowing = ['Proceeds from Short-term Debt', 'Proceeds from Issuance of Long-term Debt']

new_borrowing_10q = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=amzn_cf_10q, first_quarter='03', idx=idx_10q)[0]
new_borrowing_10q_sliced = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=amzn_cf_10q, first_quarter='03', idx=idx_10q)[1]
new_borrowing_10q_acum = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=amzn_cf_10q, first_quarter='03', idx=idx_10q)[2]

# Debt paid
lst_itens_debt_paid = [
    'Repayments of Short-term Debt', 
    'Repayments of Long-term Debt', 
    'Finance Lease, Principal Payments', 
    'Repayments Of Long Term Financing Obligations'
]

debt_paid_10q = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=amzn_cf_10q, first_quarter='03', idx=idx_10q)[0]
debt_paid_10q_sliced = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=amzn_cf_10q, first_quarter='03', idx=idx_10q)[1]
debt_paid_10q_acum = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=amzn_cf_10q, first_quarter='03', idx=idx_10q)[2]

# Taxes on operating income
taxes_operating_income_10q = f_eua.indicador_taxes_operating_income(ten_k=False, df_is=amzn_is_10q, idx=idx_10q)

# FCFE
fcfe_10q = f_eua.indicador_fcfe(
    ten_k=False, 
    depreciacao=depreciacao_10q_acum, 
    capex=capex_10q_acum, 
    change_non_cash_wc=change_non_cash_wc_10q_sliced, 
    new_borrowing=new_borrowing_10q_acum, 
    debt_paid=debt_paid_10q_acum, 
    df_is=amzn_is_10q,
    idx=idx_10q
)

# FCFF
fcff_10q = f_eua.indicador_fcff(
        ten_k=False, 
        taxes_operating_income=taxes_operating_income_10q,
        depreciacao=depreciacao_10q_acum, 
        capex=capex_10q_acum, 
        change_non_cash_wc=change_non_cash_wc_10q_sliced, 
        df_is=amzn_is_10q,
        idx=idx_10q
)

# DPA e Payout - Amazon não paga dividendo

# Buyback
buyback_10q = f_eua.indicador_buyback(ten_k=False, df_cf=amzn_cf_10q,  first_quarter='03')

# Lista com todos os indicadores
lst_indicadores_10q = [
    p_lpa_10q_damodaran,
    p_lpa_10q,
    lpa_p_10q,  
    ebitda_10q_sliced, 
    m_liq_10q,
    p_vpa_10q,
    vm_10q,
    div_bruta_total_10q,
    caixa_10q,
    div_liq_10q,
    dl_ebitda_10q,
    dl_pl_10q,
    ev_ebitda_10q,
    roe_10q,
    roic_10q,
    fco_10q,
    fci_10q,
    fcf_10q,
    free_cash_flow_10q,
    capex_10q_sliced,
    net_capex_10q,
    rd_expense_10q_sliced,
    adj_net_capex_10q,
    working_capital_10q,
    reinvestment_rate_10q,
    fcfe_10q,
    fcff_10q,
    buyback_10q
]

# Juntando todos os indicadores em um df
amzn_indicadores_10q = pd.concat(lst_indicadores_10q, axis=1)

# Nomes das colunas
nomes_col = [
    'P/L Damodaran', 
    'P/L', 
    'L/P', 
    'EBITDA', 
    'Margem Líquida',
    'P/VPA',
    'Valor de Mercado',
    'Dívida Bruta',
    'Caixa e Equivalentes',
    'Dívida Líquida',
    'Dívida Líquida/EBITDA',
    'Dívida Líquida/PL',
    'EV/EBITDA',
    'ROE',
    'ROIC',
    'FCO',
    'FCI',
    'FCF',
    'Free Cash Flow',
    'Capex',
    'Net Capex',
    'R&D',
    'Adj Net Capex',
    'Working Capital',
    'Reinvestment Rate',
    'FCFE',
    'FCFF',
    'Buyback'
]

# Atribuindo os novos nomes das colunas do df
amzn_indicadores_10q.columns = nomes_col

# Preenchendo os NaN por zero
amzn_indicadores_10q = amzn_indicadores_10q.fillna(0)

amzn_indicadores_10q

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//tecnologia//amazon//amzn_indicators.xlsx') as writer:
    amzn_indicadores_10k.to_excel(writer, sheet_name='indicators_10k')
    amzn_indicadores_10q.to_excel(writer, sheet_name='indicators_10q')

## Apple

* 1Q -> Lançamento em Dezembro (12);
* 2Q -> Lançamento em Março (03);
* 3Q -> Lançamento em Junho (06);
* 10K -> Lançamento em Setembro (09).

In [ ]:
ticker = 'aapl'
f_sec.cik_matching_ticker(ticker)

### Scraping Apple 10-K

In [ ]:
# Número dos documentos que contém a DRE, BP e FC 
acc_documents = f_sec.get_filtered_filings(ticker=ticker, ten_k=True, just_accession_numbers=True)

# Trocando os '-' para '' das strings ('0001045810-24-000264' -> '000104581024000264')
dict_acc_num = {k: v.replace('-', '') for k, v in acc_documents.items()}

# Transformando o dicionário em um df
df_lst_acc_num_aapl = pd.DataFrame.from_dict(dict_acc_num, orient='index', columns=['acc_num'])

# Transformando em datetime o index
df_lst_acc_num_aapl.index = pd.to_datetime(df_lst_acc_num_aapl.index, format='%Y-%m-%d')

# Selecionando os 'acc_num' dos 10-k a partir de 2017 p/ iteração
df_lst_acc_num_aapl = df_lst_acc_num_aapl.loc['2017-09-30':]

df_lst_acc_num_aapl

In [ ]:
# Rótulos
labels_dict = f_sec.get_label_dictionary(ticker, headers=f_sec.header)

In [ ]:
# Fazendo o scraping do balance sheet (10-K)
balance_sheet_lst = []

for acc_num in df_lst_acc_num_aapl['acc_num']:

    # Balance sheet
    balance_sheet = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='balance_sheet')
    balance_sheet = f_sec.rename_statement(balance_sheet, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    balance_sheet = balance_sheet.iloc[:, 0]
    balance_sheet_lst.append(balance_sheet)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_balance_sheet_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in balance_sheet_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_balance_sheet_10k = pd.concat([df_balance_sheet_10k, df], axis=1)

# Substituindo os NaN por 0
df_balance_sheet_10k = df_balance_sheet_10k.fillna(0)

# A linha 'Marketable Securities, Current' nas colunas '2018-09-29'	e '2017-09-30' estão zeradas
# As informações estão na linha 'Available-for-sale Securities, Current'
df_balance_sheet_10k.loc['Marketable Securities, Current', '2018-09-29'] = df_balance_sheet_10k.loc['Available-for-sale Securities, Current', '2018-09-29'] 
df_balance_sheet_10k.loc['Marketable Securities, Current', '2017-09-30'] = df_balance_sheet_10k.loc['Available-for-sale Securities, Current', '2017-09-30'] 

# A linha 'Marketable Securities, Noncurrent' nas colunas '2018-09-29'	e '2017-09-30' estão zeradas
# As informações estão na linha 'Available-for-sale Securities, Noncurrent'
df_balance_sheet_10k.loc['Marketable Securities, Noncurrent', '2018-09-29'] = df_balance_sheet_10k.loc['Available-for-sale Securities, Noncurrent', '2018-09-29'] 
df_balance_sheet_10k.loc['Marketable Securities, Noncurrent', '2017-09-30'] = df_balance_sheet_10k.loc['Available-for-sale Securities, Noncurrent', '2017-09-30'] 

# Removendo as linhas que movemos os seus itens para as linhas corretas
df_balance_sheet_10k = df_balance_sheet_10k.drop([
    'Available-for-sale Securities, Current',
    'Available-for-sale Securities, Noncurrent'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_balance_sheet_10k = df_balance_sheet_10k.drop('us-gaap_AssetsCurrentAbstract')

df_balance_sheet_10k

In [ ]:
# Fazendo o scraping do income statement (10-K)
income_statement_lst = []

for acc_num in df_lst_acc_num_aapl['acc_num']:

    # Income statement
    income_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='income_statement')
    income_statement = f_sec.rename_statement(income_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    income_statement = income_statement.iloc[:, 0]
    income_statement_lst.append(income_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_income_statement_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in income_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_income_statement_10k = pd.concat([df_income_statement_10k, df], axis=1)

# Substituindo os NaN por 0
df_income_statement_10k = df_income_statement_10k.fillna(0)

# A linha 'Revenue from Contract with Customer, Excluding Assessed Tax' nas colunas '2018-09-29' e '2017-09-30' estão zeradas
# As informações estão nas linhas 'Revenues' e 'Revenue, Net (Deprecated 2018-01-31)'
df_income_statement_10k.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2018-09-29'] = df_income_statement_10k.loc['Revenues', '2018-09-29'] 
df_income_statement_10k.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2017-09-30'] = df_income_statement_10k.loc['Revenue, Net (Deprecated 2018-01-31)', '2017-09-30'] 

# Removendo as linhas que movemos os seus itens para as linhas corretas
df_income_statement_10k = df_income_statement_10k.drop([
    'Revenues',
    'Revenue, Net (Deprecated 2018-01-31)'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_income_statement_10k = df_income_statement_10k.drop([
    'us-gaap_OperatingExpensesAbstract',
    'us-gaap_IncomeStatementAbstract',
    'Common Stock, Dividends, Per Share, Declared'
])

df_income_statement_10k

In [ ]:
# Fazendo o scraping do cash flow statement (10-K)
cash_flow_statement_lst = []

for acc_num in df_lst_acc_num_aapl['acc_num']:

    # Cash flow statement
    cash_flow_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='cash_flow_statement')
    cash_flow_statement = f_sec.rename_statement(cash_flow_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    cash_flow_statement = cash_flow_statement.iloc[:, 0]
    cash_flow_statement_lst.append(cash_flow_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_cash_flow_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in cash_flow_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_cash_flow_10k = pd.concat([df_cash_flow_10k, df], axis=1)

# Substituindo os NaN por 0
df_cash_flow_10k = df_cash_flow_10k.fillna(0)

# A linha 'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents' nas colunas '2018-09-29' e '2017-09-30' estão zeradas
# As informações estão na linha 'Cash and Cash Equivalents, at Carrying Value'
df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2018-09-29'] = df_cash_flow_10k.loc['Cash and Cash Equivalents, at Carrying Value', '2018-09-29'] 
df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2017-09-30'] = df_cash_flow_10k.loc['Cash and Cash Equivalents, at Carrying Value', '2017-09-30'] 

# A linha 'Payments to Acquire Debt Securities, Available-for-sale' nas colunas '2018-09-29' e '2017-09-30' estão zeradas
# As informações estão na linha 'Payments to Acquire Available-for-sale Securities'
df_cash_flow_10k.loc['Payments to Acquire Debt Securities, Available-for-sale', '2018-09-29'] = df_cash_flow_10k.loc['Payments to Acquire Available-for-sale Securities', '2018-09-29'] 
df_cash_flow_10k.loc['Payments to Acquire Debt Securities, Available-for-sale', '2017-09-30'] = df_cash_flow_10k.loc['Payments to Acquire Available-for-sale Securities', '2017-09-30'] 

# A linha 'Proceeds from Sale of Debt Securities, Available-for-sale' nas colunas '2018-09-29' e '2017-09-30' estão zeradas
# As informações estão na linha 'Proceeds from Sale of Available-for-sale Securities	'
df_cash_flow_10k.loc['Proceeds from Sale of Debt Securities, Available-for-sale', '2018-09-29'] = df_cash_flow_10k.loc['Proceeds from Sale of Available-for-sale Securities', '2018-09-29'] 
df_cash_flow_10k.loc['Proceeds from Sale of Debt Securities, Available-for-sale', '2017-09-30'] = df_cash_flow_10k.loc['Proceeds from Sale of Available-for-sale Securities', '2017-09-30'] 

# A linha 'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect' nas colunas '2018-09-29' e '2017-09-30' estão zeradas
# As informações estão na linha 'Cash and Cash Equivalents, Period Increase (Decrease)'
df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect', '2018-09-29'] = df_cash_flow_10k.loc['Cash and Cash Equivalents, Period Increase (Decrease)', '2018-09-29'] 
df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect', '2017-09-30'] = df_cash_flow_10k.loc['Cash and Cash Equivalents, Period Increase (Decrease)', '2017-09-30'] 

# A linha 'Depreciation, Depletion and Amortization' na coluna '2017-09-30' está zerada
# As informações estão na linha 'Depreciation, Amortization and Accretion, Net'
df_cash_flow_10k.loc['Depreciation, Depletion and Amortization', '2017-09-30'] = df_cash_flow_10k.loc['Depreciation, Amortization and Accretion, Net', '2017-09-30'] 

# A linha 'Interest Paid, Excluding Capitalized Interest, Operating Activities' na coluna '2017-09-30' está zerada
# As informações estão na linha 'Interest Paid, Including Capitalized Interest, Operating and Investing Activities'
df_cash_flow_10k.loc['Interest Paid, Excluding Capitalized Interest, Operating Activities', '2017-09-30'] = df_cash_flow_10k.loc['Interest Paid, Including Capitalized Interest, Operating and Investing Activities', '2017-09-30'] 

# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_cash_flow_10k = df_cash_flow_10k.drop([
    'Cash and Cash Equivalents, at Carrying Value',
    'Payments to Acquire Available-for-sale Securities',
    'Proceeds from Sale of Available-for-sale Securities',
    'Cash and Cash Equivalents, Period Increase (Decrease)',
    'Depreciation, Amortization and Accretion, Net',
    'Interest Paid, Including Capitalized Interest, Operating and Investing Activities'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_cash_flow_10k = df_cash_flow_10k.drop('us-gaap_StatementOfCashFlowsAbstract')

# Arrumando e adicionando alguns números no df
df_cash_flow_10k.loc['Increase (Decrease) in Contract with Customer, Liability', '2018-09-29'] = -3000
df_cash_flow_10k.loc['Increase (Decrease) in Contract with Customer, Liability', '2017-09-30'] = -593000
df_cash_flow_10k.loc['Proceeds from Sale and Maturity of Other Investments', '2017-09-30'] = 126000

df_cash_flow_10k

In [ ]:
# Arrumando alguns index dos dfs para ficar padronizado com os index utilizados nas funções p/ calcular os indicadores fundamentalistas
df_balance_sheet_10k = df_balance_sheet_10k.rename(index={
    'Cash and Cash Equivalents, at Carrying Value': 'Cash and Cash Equivalents',
    'Long-term Debt, Current Maturities': 'Long-term Debt, Current',
    'Long-term Debt, Excluding Current Maturities': 'Long-term Debt, Noncurrent',
    "Stockholders' Equity Attributable to Parent": 'Total Shareholders Equity'
})

df_income_statement_10k = df_income_statement_10k.rename(index={
    'Revenue from Contract with Customer, Excluding Assessed Tax': 'Revenues',
    'Research and Development Expense': 'Research and Development',
    'Operating Income (Loss)': 'Operating Income',
    'Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest': 'Income Before Provision for Income Taxes',
    'Income Tax Expense (Benefit)': 'Provision for Income Taxes',
    'Net Income (Loss) Attributable to Parent': 'Net Income',
    'Weighted Average Number of Shares Outstanding, Basic': 'Number of Shares - Basic',
    'Weighted Average Number of Shares Outstanding, Diluted': 'Number of Shares - Diluted',
})

df_cash_flow_10k = df_cash_flow_10k.rename(index={
    'Payments to Acquire Property, Plant, and Equipment': 'Purchases Of Property And Equipment And Intangible Assets'
})

In [ ]:
# Selecionando o 'acc_num' do ano de 2023 - Apple tem o custo 'interest expense' em 2023. Em 2024 e 2025, ela não possui esse custo 
acc_num_2023 = '000032019323000106'

# Fazendo o scraping do financial statement details (10-K) -> 'Note 6 – Consolidated Financial Statement Details' -> 'Other Income/(Expense), Net' -> 'Interest expense''
interest_expense_10k = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num_2023, statement_name='interest_expense')
interest_expense_10k = f_sec.rename_statement(interest_expense_10k, label_dictionary=labels_dict)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
interest_expense_10k = interest_expense_10k.drop('us-gaap_OrganizationConsolidationAndPresentationOfFinancialStatementsAbstract')

# Formatando o número para ficar positivo
interest_expense_10k.loc['Interest Expense'] = interest_expense_10k.loc['Interest Expense'] * -1

# Selecionando a linha desejada
interest_expense_10k = interest_expense_10k.loc[['Interest Expense']]

interest_expense_10k

In [ ]:
# 'acc_num' entre 2020 e 2025
df_lst_acc_num_aapl_sorted = df_lst_acc_num_aapl.sort_index()
acc_num_list_aapl_sliced = df_lst_acc_num_aapl_sorted.loc['2020':'2025', 'acc_num']

# Fazendo o scraping do leases (10-K)
lease_lst = []

for acc_num in acc_num_list_aapl_sliced:

# Fazendo o scraping do leases (10-K) -> 'Note 8 – Leases' -> 'Total lease liabilities'
    leases_10k = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='leases')
    leases_10k = f_sec.rename_statement(leases_10k, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    leases_10k = leases_10k.iloc[:, 0]
    lease_lst.append(leases_10k)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_leases_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in lease_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_leases_10k = pd.concat([df_leases_10k, df], axis=1)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_leases_10k = df_leases_10k.drop('us-gaap_AssetsAndLiabilitiesLesseeAbstract')

# Renomeando a linha do índice
df_leases_10k = df_leases_10k.rename(index={
    'aapl_OperatingandFinanceLeaseLiability': 'Total Lease Liabilities',
})

# Selecionando a linha desejada
df_leases_10k = df_leases_10k.loc[['Total Lease Liabilities']]

####################################################################################################################

# 'acc_num' entre 2017 e 2019
acc_num_list_aapl_sliced_2 = df_lst_acc_num_aapl_sorted.loc['2017':'2019', 'acc_num']

# Fazendo o scraping do leases (10-K)
lease_lst = []

for acc_num in acc_num_list_aapl_sliced_2:

# Fazendo o scraping do leases (10-K) -> 'Note 8 – Leases' -> 'Total lease liabilities'
    leases_10k_17_18_19 = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='leases')
    leases_10k_17_18_19 = f_sec.rename_statement(leases_10k_17_18_19, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    leases_10k_17_18_19 = leases_10k_17_18_19.iloc[:, 0]
    lease_lst.append(leases_10k_17_18_19)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_leases_10k_17_18_19 = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in lease_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_leases_10k_17_18_19 = pd.concat([df_leases_10k_17_18_19, df], axis=1)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_leases_10k_17_18_19 = df_leases_10k_17_18_19.drop('us-gaap_OperatingLeasesFutureMinimumPaymentsDueAbstract')

# Renomeando a linha do índice
df_leases_10k_17_18_19 = df_leases_10k_17_18_19.rename(index={
    'Operating Leases, Future Minimum Payments Due': 'Total Lease Liabilities',
})

# Selecionando a linha desejada
df_leases_10k_17_18_19 = df_leases_10k_17_18_19.loc[['Total Lease Liabilities']]

####################################################################################################################

# Contatenando os dfs
df_leases_10k = pd.concat([df_leases_10k, df_leases_10k_17_18_19], axis=1)

# Ordenando o df
df_leases_10k = df_leases_10k[sorted(df_leases_10k.columns, reverse=True)]

df_leases_10k

In [ ]:
# Selecionando o 'acc_num' do ano mais recente
last_acc_num = df_lst_acc_num_aapl.iloc[0][0]

# Fazendo o scraping do financial statement details (10-K) -> 'Note 6 – Consolidated Financial Statement Details' -> 'Other non-current assets'
cross_holding_10k = f_sec.process_one_statement(ticker=ticker, accession_number=last_acc_num, statement_name='cross_holding')
cross_holding_10k = f_sec.rename_statement(cross_holding_10k, label_dictionary=labels_dict)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
cross_holding_10k = cross_holding_10k.drop('us-gaap_OrganizationConsolidationAndPresentationOfFinancialStatementsAbstract')

# Renomeando a linha do índice
cross_holding_10k = cross_holding_10k.rename(index={
    'Other Assets, Miscellaneous, Noncurrent': 'Other Noncurrent Assets',

})

# Selecionando a linha desejada
cross_holding_10k = cross_holding_10k.loc[['Other Noncurrent Assets']]

cross_holding_10k

In [ ]:
# Fazendo o scraping do restricted stock units (10-K) -> 'Note 11 – Share-Based Compensation' -> 'Number of RSUs'
rsu_10k = f_sec.process_one_statement(ticker=ticker, accession_number=last_acc_num, statement_name='rsu')
rsu_10k = f_sec.rename_statement(rsu_10k, label_dictionary=labels_dict)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
rsu_10k = rsu_10k.drop('us-gaap_ShareBasedCompensationArrangementByShareBasedPaymentAwardEquityInstrumentsOtherThanOptionsNonvestedRollForward')

# Resetando o index do df, porque ele possui vários index igauis
rsu_10k = rsu_10k.reset_index(drop=True)

# Renomeando a linha do índice
rsu_10k = rsu_10k.rename(index={
    4: 'RSUs',
})

# Selecionando a linha desejada
rsu_10k = rsu_10k.loc[['RSUs']]

rsu_10k

In [ ]:
# Fazendo o scraping das receitas por segmento geográfico
geographic_revenue_10k = f_sec.process_one_statement(ticker=ticker, accession_number=last_acc_num, statement_name='geographic_revenue')
geographic_revenue_10k = f_sec.rename_statement(geographic_revenue_10k, label_dictionary=labels_dict)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
geographic_revenue_10k = geographic_revenue_10k.drop('us-gaap_SegmentReportingInformationLineItems')

# Resetando o index do df, porque ele possui vários index igauis
geographic_revenue_10k = geographic_revenue_10k.reset_index(drop=True)

# Renomeado o index do df
geographic_revenue_10k = geographic_revenue_10k.rename(index={
    0: 'Net Sales - Total',
    5: 'Operating Income - Total',
    7: 'Net Sales - America',
    10: 'Operating Income - America',
    11: 'Net Sales - Europe',
    14: 'Operating Income - Europe',
    15: 'Net Sales - Greater China',
    18: 'Operating Income - Greater China',
    19: 'Net Sales - Japan',
    22: 'Operating Income - Japan',
    23: 'Net Sales - Rest of Asia Pacific',
    26: 'Operating Income - Rest of Asia Pacific',
})

geographic_revenue_10k

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//tecnologia//apple//aapl_10k.xlsx') as writer:
    df_balance_sheet_10k.to_excel(writer, sheet_name='balance_sheet')
    df_income_statement_10k.to_excel(writer, sheet_name='income_statement')
    df_cash_flow_10k.to_excel(writer, sheet_name='cash_flow')
    interest_expense_10k.to_excel(writer, sheet_name='interest_expense')
    df_leases_10k.to_excel(writer, sheet_name='leases')
    cross_holding_10k.to_excel(writer, sheet_name='cross_holding')
    rsu_10k.to_excel(writer, sheet_name='num_shares')
    geographic_revenue_10k.to_excel(writer, sheet_name='geographic_revenue')

### Scraping Apple 10-Q

In [ ]:
# Número dos documentos que contém a DRE, BP e FC 
acc_documents_10q = f_sec.get_filtered_filings(ticker=ticker, ten_k=False, just_accession_numbers=True)

# Trocando os '-' para '' das strings ('0001045810-24-000264' -> '000104581024000264')
dict_acc_num_10q = {k: v.replace('-', '') for k, v in acc_documents_10q.items()}

# Transformando o dicionário em um df
df_lst_acc_num_10q_aapl = pd.DataFrame.from_dict(dict_acc_num_10q, orient='index', columns=['acc_num'])

# Transformando em datetime o index
df_lst_acc_num_10q_aapl.index = pd.to_datetime(df_lst_acc_num_10q_aapl.index, format='%Y-%m-%d')

df_lst_acc_num_10q_aapl

In [ ]:
# Selecionando os 'acc_num' dos 10-q a partir de '2019-12-28' p/ iteração
df_lst_acc_num_10q_aapl_sliced = df_lst_acc_num_10q_aapl.loc['2019-12-28':]
df_lst_acc_num_10q_aapl_sliced

In [ ]:
# Rótulos
labels_dict = f_sec.get_label_dictionary(ticker, headers=f_sec.header)

In [ ]:
# Fazendo o scraping do balance sheet (10-Q)
balance_sheet_lst = []

for acc_num in df_lst_acc_num_10q_aapl_sliced['acc_num']:

    # Balance sheet
    balance_sheet = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='balance_sheet')
    balance_sheet = f_sec.rename_statement(balance_sheet, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    balance_sheet = balance_sheet.iloc[:, 0]
    balance_sheet_lst.append(balance_sheet)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_balance_sheet_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in balance_sheet_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_balance_sheet_10q = pd.concat([df_balance_sheet_10q, df], axis=1)

# Substituindo os NaN por 0
df_balance_sheet_10q = df_balance_sheet_10q.fillna(0)

# A linha 'Commercial Paper' nas colunas '2020-06-27' e '2020-03-28' estão zeradas
# As informações estão na linha 'Other Short-term Borrowings'
df_balance_sheet_10q.loc['Commercial Paper', '2020-03-28'] = df_balance_sheet_10q.loc['Other Short-term Borrowings', '2020-03-28'] 
df_balance_sheet_10q.loc['Commercial Paper', '2020-06-27'] = df_balance_sheet_10q.loc['Other Short-term Borrowings', '2020-06-27'] 

# # Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_balance_sheet_10q = df_balance_sheet_10q.drop('Other Short-term Borrowings')

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_balance_sheet_10q = df_balance_sheet_10q.drop([
    'us-gaap_AssetsCurrentAbstract',
    'Common Stock, Shares, Outstanding'
])

df_balance_sheet_10q

In [ ]:
# Fazendo o scraping do income statement (10-Q)
income_statement_lst = []

for acc_num in df_lst_acc_num_10q_aapl_sliced['acc_num']:

    # Income statement
    income_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='income_statement')
    income_statement = f_sec.rename_statement(income_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    income_statement = income_statement.iloc[:, 0]
    income_statement_lst.append(income_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_income_statement_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in income_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_income_statement_10q = pd.concat([df_income_statement_10q, df], axis=1)

# Substituindo os NaN por 0
df_income_statement_10q = df_income_statement_10q.fillna(0)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_income_statement_10q = df_income_statement_10q.drop('us-gaap_OperatingExpensesAbstract')

df_income_statement_10q

In [ ]:
# Fazendo o scraping do cash flow statement (10-Q)
cash_flow_statement_lst = []

for acc_num in df_lst_acc_num_10q_aapl_sliced['acc_num']:

    # Cash flow statement
    cash_flow_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='cash_flow_statement')
    cash_flow_statement = f_sec.rename_statement(cash_flow_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    cash_flow_statement = cash_flow_statement.iloc[:, 0]
    cash_flow_statement_lst.append(cash_flow_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_cash_flow_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in cash_flow_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_cash_flow_10q = pd.concat([df_cash_flow_10q, df], axis=1)

# Substituindo os NaN por 0
df_cash_flow_10q = df_cash_flow_10q.fillna(0)

# # Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_cash_flow_10q = df_cash_flow_10q.drop('us-gaap_StatementOfCashFlowsAbstract')

df_cash_flow_10q

In [ ]:
# Arrumando alguns index dos dfs para ficar padronizado com os index utilizados nas funções p/ calcular os indicadores fundamentalistas
df_balance_sheet_10q = df_balance_sheet_10q.rename(index={
    'Cash and Cash Equivalents, at Carrying Value': 'Cash and Cash Equivalents',
    'Long-term Debt, Current Maturities': 'Long-term Debt, Current',
    'Long-term Debt, Excluding Current Maturities': 'Long-term Debt, Noncurrent',
    "Stockholders' Equity Attributable to Parent": 'Total Shareholders Equity'
})

df_income_statement_10q = df_income_statement_10q.rename(index={
    'Revenue from Contract with Customer, Excluding Assessed Tax': 'Revenues',
    'Research and Development Expense': 'Research and Development',
    'Operating Income (Loss)': 'Operating Income',
    'Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest': 'Income Before Provision for Income Taxes',
    'Income Tax Expense (Benefit)': 'Provision for Income Taxes',
    'Net Income (Loss) Attributable to Parent': 'Net Income',
    'Weighted Average Number of Shares Outstanding, Basic': 'Number of Shares - Basic',
    'Weighted Average Number of Shares Outstanding, Diluted': 'Number of Shares - Diluted',
})

df_cash_flow_10q = df_cash_flow_10q.rename(index={
    'Payments to Acquire Property, Plant, and Equipment': 'Purchases Of Property And Equipment And Intangible Assets'
})

* Não preciso fazer o scraping do financial statement details (10-Q) -> 'Note 6 – Consolidated Financial Statement Details' -> 'Other Income/(Expense), Net' -> 'Interest expense', porque essa info não é publicada;
* Não preciso fazer o scraping do leases (10-Q) -> 'Note 8 – Leases' -> 'Total lease liabilities', porque essa info não é publicada;
* Não preciso fazer o scraping do financial statement details (10-Q) -> 'Note 6 – Consolidated Financial Statement Details' -> 'Other non-current assets', porque essa info não é publicada;
* Não preciso fazer o scraping das geographic revenues (10-Q) -> 'Segment Operating Performance', porque uso a informação do 10-K;

In [ ]:
# Selecionando o 'acc_num' do trimestre mais recente
last_acc_num_10q = df_lst_acc_num_10q_aapl_sliced.iloc[0].item()

# Fazendo o scraping do restricted stock units -> 'Note 11 – Share-Based Compensation' -> 'Number of RSUs'
rsu_10q = f_sec.process_one_statement(ticker=ticker, accession_number=last_acc_num_10q, statement_name='rsu')
rsu_10q = f_sec.rename_statement(rsu_10q, label_dictionary=labels_dict)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
rsu_10q = rsu_10q.drop('us-gaap_ShareBasedCompensationArrangementByShareBasedPaymentAwardEquityInstrumentsOtherThanOptionsNonvestedRollForward')

# Resetando o index do df, porque ele possui vários index igauis
rsu_10q = rsu_10q.reset_index(drop=True)

# Renomeando a linha do índice
rsu_10q = rsu_10q.rename(index={
    4: 'RSUs',
})

# Selecionando a linha desejada
rsu_10q = rsu_10q.loc[['RSUs']]

rsu_10q

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//tecnologia//apple//aapl_10q.xlsx') as writer:
    df_balance_sheet_10q.to_excel(writer, sheet_name='balance_sheet')
    df_income_statement_10q.to_excel(writer, sheet_name='income_statement')
    df_cash_flow_10q.to_excel(writer, sheet_name='cash_flow')
    rsu_10q.to_excel(writer, sheet_name='num_shares')

### Indicadores Fundamentalistas

#### Indicadores 10-K

In [ ]:
# Lendo os arquivos 10k
aapl_bs = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//apple//aapl_10k.xlsx', 
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

aapl_is = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//apple//aapl_10k.xlsx', 
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

aapl_cf = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//apple//aapl_10k.xlsx', 
    sheet_name='cash_flow', 
    index_col='Unnamed: 0'
)

aapl_leases = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//apple//aapl_10k.xlsx', 
    sheet_name='leases', 
    index_col='Unnamed: 0'
)

# Transformando as colunas em datetime
aapl_bs.columns = pd.to_datetime(aapl_bs.columns)
aapl_is.columns = pd.to_datetime(aapl_is.columns)
aapl_cf.columns = pd.to_datetime(aapl_cf.columns)
aapl_leases.columns = pd.to_datetime(aapl_leases.columns)

# Transformando as colunas em apenas o ano
aapl_bs.columns = aapl_bs.columns.year
aapl_is.columns = aapl_is.columns.year
aapl_cf.columns = aapl_cf.columns.year
aapl_leases.columns = aapl_leases.columns.year

In [ ]:
# Preços da AAPL
aapl_close = yf.download('AAPL', auto_adjust=True, multi_level_index=False, start='2017-01-01')['Close']

# Selecionando o último preço do mês de Dezembro de cada ano 
lst_years = [
    '2017', '2018', '2019', '2020', 
    '2021', '2022', '2023', '2024', 
    '2025',
]

last_sep_close = []

for year in lst_years:
    last_close = aapl_close[f'{year}-09'].iloc[-1]
    last_sep_close.append(last_close)

# Transformando em uma série pandas
aapl_close_sep = pd.Series(last_sep_close, index=lst_years)

# Revertendo o index da série p/ ficar igual com os dos indicadores
aapl_close_sep = aapl_close_sep.reindex(index=aapl_close_sep.index[::-1])

# Arredondamento de duas casas decimais
aapl_close_sep = round(aapl_close_sep, 2)

# Teve um split de 4:1 em 31/08/2020 (https://br.investing.com/equities/apple-computer-inc-historical-data-splits)
aapl_close_sep.loc['2019'] = aapl_close_sep.loc['2019'] * 4
aapl_close_sep.loc['2018'] = aapl_close_sep.loc['2018'] * 4
aapl_close_sep.loc['2017'] = aapl_close_sep.loc['2017'] * 4

# Transformando o index em int
aapl_close_sep .index = aapl_close_sep .index.astype(int)

aapl_close_sep

In [ ]:
# Criando uma lista só para fazer a indexação em que o ano mais recente está no começo
idx = aapl_bs.columns

# Valor de Mercado - utilizando a função 'indicador_vm_2', porque o nº de ações está em escala diferente
vm = f_eua.indicador_vm_2(ten_k=True, price=aapl_close_sep, df_is=aapl_is) 

# LPA - utilizando a função 'indicador_lpa_2', porque o nº de ações está em escala diferente
lpa = f_eua.indicador_lpa_2(ten_k=True, df_is=aapl_is, idx=None)

# P/L Damodaran
p_lpa_damodaran = f_eua.indicador_p_lpa_2(ten_k=True, vm=vm, df_is=aapl_is)

# P/L
p_lpa = f_eua.indicador_p_lpa(ten_k=True, price=aapl_close_sep, lpa=lpa, idx=idx)

# L/P
lpa_p = f_eua.indicador_lpa_p(ten_k=True, price=aapl_close_sep, lpa=lpa, idx=idx) 

# Depreciação
depreciacao = f_eua.indicador_depreciacao(ten_k=True, df_cf=aapl_cf, first_quarter=None, idx=idx)

# EBITDA
ebitda = f_eua.indicador_ebitda(ten_k=True, df_is=aapl_is, idx=None, depreciacao=depreciacao)

# Margem Líquida
m_liq = f_eua.indicador_ml(ten_k=True, df_is=aapl_is)

# VPA - utilizando a função 'indicador_vpa_2', porque o nº de ações está em escala diferente
vpa = f_eua.indicador_vpa_2(ten_k=True, df_bs=aapl_bs, df_is=aapl_is)

# P/VPA
p_vpa = f_eua.indicador_p_vpa(ten_k=True, price=aapl_close_sep, vpa=vpa, idx=idx)

# Caixa e Equivalentes
lst_caixa = [
    'Cash and Cash Equivalents', 
    'Marketable Securities, Current', 
    'Marketable Securities, Noncurrent'
]

caixa = f_eua.indicador_caixa(ten_k=True, df_bs=aapl_bs, lst_itens_caixa=lst_caixa)

# Dívida Bruta
lst_divida = [
    'Commercial Paper', 
    'Long-term Debt, Current', 
    'Long-term Debt, Noncurrent'
]

div_bruta = f_eua.indicador_divida_bruta(ten_k=True, df_bs=aapl_bs, lst_itens_divida=lst_divida)
div_bruta_total = div_bruta.values + aapl_leases.values
div_bruta_total = pd.Series(div_bruta_total[0], index=idx)

# Dívida Líquida
div_liq = f_eua.indicador_divida_liquida(ten_k=True, caixa=caixa, div_bruta=div_bruta_total, idx=idx)

# Dívida Líquida / EBITDA
dl_ebitda = f_eua.indicador_dl_ebitda(ten_k=True, ebitda=ebitda, div_liq=div_liq, idx=None)

# Dívida Líquida / PL
dl_pl = f_eua.indicador_dl_pl(ten_k=True, div_liq=div_liq, df_bs=aapl_bs) 

# EV/EBITDA
ev_ebitda = f_eua.indicador_ev_ebitda(vm=vm, div_liq=div_liq, ebitda=ebitda)

# ROE
roe = f_eua.indicador_roe(ten_k=True, df_is=aapl_is, df_bs=aapl_bs, idx=None)

# ROIC
roic = f_eua.indicador_roic(ten_k=True, df_is=aapl_is, df_bs=aapl_bs, div_bruta=div_bruta_total, idx=idx)

# FCO
fco = f_eua.indicador_fco(ten_k=True, df_cf=aapl_cf, first_quarter=None) 

# FCI
fci = f_eua.indicador_fci(ten_k=True, df_cf=aapl_cf, first_quarter=None) 

# FCF
fcf = f_eua.indicador_fcf(ten_k=True, df_cf=aapl_cf, first_quarter=None) 

# CAPEX
lst_capex = ['Purchases Of Property And Equipment And Intangible Assets']

capex= f_eua.indicador_capex(ten_k=True, df_cf=aapl_cf, first_quarter=None, lst_itens=lst_capex, idx=idx)

# Free Cash Flow
free_cash_flow = f_eua.indicador_free_cash_flow(fco=fco, capex=capex)

# Net CAPEX
net_capex = f_eua.indicador_net_capex(capex=capex, depreciation=depreciacao)

# R&D
rd_expense = f_eua.indicador_rd(ten_k=True, df_is=aapl_is, idx=idx)

# Net CAPEX ajustado
adj_net_capex = f_eua.indicador_adj_net_capex(net_capex=net_capex, rd_expense=rd_expense)

# Working capital
working_capital = f_eua.indicador_working_capital(ten_k=True, df_bs=aapl_bs)

# Change in non cash working capital
lst_itens_change_non_cash_wc = [
    'Increase (Decrease) in Accounts Receivable', 
    'Increase (Decrease) in Other Receivables', 
    'Increase (Decrease) in Inventories',
    'Increase (Decrease) in Other Operating Assets',
    'Increase (Decrease) in Accounts Payable',
    'Increase (Decrease) in Other Operating Liabilities'
]

change_non_cash_wc = f_eua.indicador_change_non_cash_wc(ten_k=True, df_cf=aapl_cf, lst_itens=lst_itens_change_non_cash_wc, first_quarter=None)

# Reinvestment rate
reinvestment_rate = f_eua.indicador_reinvestment_rate(ten_k=True, net_capex=net_capex, change_non_cash_wc=change_non_cash_wc, df_is=aapl_is, idx=idx)

# New borrowing
lst_itens_new_borrowing = ['Proceeds from Issuance of Long-term Debt']

new_borrowing = f_eua.indicador_new_borrowing(ten_k=True, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=aapl_cf, first_quarter=None, idx=idx)

# Debt paid
lst_itens_debt_paid = ['Repayments of Long-term Debt', 'Proceeds from (Repayments of) Commercial Paper']

debt_paid = f_eua.indicador_debt_paid(ten_k=True, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=aapl_cf, first_quarter=None, idx=idx)

# Taxes on operating income
taxes_operating_income = f_eua.indicador_taxes_operating_income(ten_k=True, df_is=aapl_is, idx=idx)

# FCFE
fcfe_10k = f_eua.indicador_fcfe(
    ten_k=True, 
    depreciacao=depreciacao, 
    capex=capex, 
    change_non_cash_wc=change_non_cash_wc, 
    new_borrowing=new_borrowing,
    debt_paid=debt_paid,
    df_is=aapl_is,
    idx=idx
)

# FCFF
fcff_10k = f_eua.indicador_fcff(
        ten_k=True, 
        taxes_operating_income=taxes_operating_income,
        depreciacao=depreciacao, 
        capex=capex, 
        change_non_cash_wc=change_non_cash_wc, 
        df_is=aapl_is,
        idx=idx
)

# Dividendo por Ação - utilizando a função 'indicador_dpa_2', porque o nº de ações está em escala diferente
dpa = f_eua.indicador_dpa_2(ten_k=True, df_cf=aapl_cf, df_is=aapl_is, div_nao_acum=None)

# Payout
payout = f_eua.indicador_payout(ten_k=True, df_cf=aapl_cf, df_is=aapl_is, div_nao_acum=None)

# Buyback
buyback = f_eua.indicador_buyback(ten_k=True, df_cf=aapl_cf, first_quarter=None)

# Colocando todos os indicadores em uma lista
lst_indicadores = [
    p_lpa_damodaran,
    p_lpa,
    lpa_p,  
    ebitda, 
    m_liq,
    p_vpa,
    vm,
    div_bruta_total,
    caixa,
    div_liq,
    dl_ebitda,
    dl_pl,
    ev_ebitda,
    roe,
    roic,
    fco,
    fci,
    fcf,
    free_cash_flow,
    capex,
    net_capex,
    rd_expense,
    adj_net_capex,
    working_capital,
    reinvestment_rate,
    fcfe_10k,
    fcff_10k,
    dpa,
    payout,
    buyback
]

# Juntando todos os indicadores em um df
aapl_indicadores_10k = pd.concat(lst_indicadores, axis=1)

# Nomes das colunas
nomes_col = [
    'P/L Damodaran', 
    'P/L', 
    'L/P', 
    'EBITDA', 
    'Margem Líquida',
    'P/VPA',
    'Valor de Mercado',
    'Dívida Bruta',
    'Caixa e Equivalentes',
    'Dívida Líquida',
    'Dívida Líquida/EBITDA',
    'Dívida Líquida/PL',
    'EV/EBITDA',
    'ROE',
    'ROIC',
    'FCO',
    'FCI',
    'FCF',
    'Free Cash Flow',
    'Capex',
    'Net Capex',
    'R&D',
    'Adj Net Capex',
    'Working Capital',
    'Reinvestment Rate',
    'FCFE',
    'FCFF',
    'DPA',
    'Payout',
    'Buyback'
]

# Atribuindo os novos nomes das colunas do df
aapl_indicadores_10k.columns = nomes_col
aapl_indicadores_10k

#### Indicadores 10-Q

In [ ]:
# Lendo os arquivos 10q
aapl_bs_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//apple//aapl_10q.xlsx',
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

aapl_is_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//apple//aapl_10q.xlsx',
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

aapl_cf_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//apple//aapl_10q.xlsx',
    sheet_name='cash_flow',  
    index_col='Unnamed: 0'
)

# Aplicando o ajuste nas colunas
aapl_bs_10q.columns = [f_sec.ajustar_mes_12_03_06_aapl(col) for col in aapl_bs_10q.columns]
aapl_is_10q.columns = [f_sec.ajustar_mes_12_03_06_aapl(col) for col in aapl_is_10q.columns]
aapl_cf_10q.columns = [f_sec.ajustar_mes_12_03_06_aapl(col) for col in aapl_cf_10q.columns]

In [ ]:
# Preços da AAPL
aapl_close = yf.download('AAPL', auto_adjust=True, multi_level_index=False, start='2020-01-01')['Close']

# Selecionando o último preço dos meses de Março, Junho e Setembro
lst_years = [
    '2020', '2021', '2022', '2023', 
    '2024', '2025',
]

lst_months = ['12', '03', '06']
last_close_months = []

for year in lst_years:
    for month in lst_months:
        last_close = aapl_close[f'{year}-{month}'].dropna().iloc[-1]  # Dropna para lidar com meses sem dados
        last_close_months.append(last_close)

# Criando uma lista em que junte os meses e os anos para que seja o index da serie pandas
lst_idx = []

for year in lst_years:
    for month in lst_months:
        lst_idx.append(f"{month}-{year}")

# Transformando em uma série pandas
aapl_close_months = pd.Series(last_close_months, index=lst_idx)

# Retirando as datas ('03-2020' e '06-2020')
aapl_close_months = aapl_close_months.drop(index=['03-2020', '06-2020'])

# Revertendo o index da série p/ ficar igual com os dos indicadores
aapl_close_months = aapl_close_months.reindex(index=aapl_close_months.index[::-1])

# No ano mais atual, como ainda ele não acabou, não consigo fazer o loop com todos meses (Março, Junho e Setembro)
actual_year_int = datetime.now().year  
actual_year_str = str(actual_year_int) 

lst_actual_months = [
    # '12',
    # '03',
    # '06', 
] 

last_close_actual_months = []

for month in lst_actual_months:
    last_close = aapl_close[f'{actual_year_str}-{month}'].dropna().iloc[-1]  # Dropna para lidar com meses sem dados
    last_close_actual_months.append(last_close)

# Criando uma lista em que junte os meses e os anos para que seja o index da serie pandas
lst_idx_actual = []

for month in lst_actual_months:
    lst_idx_actual.append(f"{month}-{actual_year_str}")

# Transformando em uma série pandas
aapl_close_months_atual = pd.Series(last_close_actual_months, index=lst_idx_actual)

# Revertendo o index da série p/ ficar igual com os dos indicadores
aapl_close_months_atual = aapl_close_months_atual.reindex(index=aapl_close_months_atual.index[::-1])

# Juntando as duas series pandas
aapl_close_months = pd.concat([aapl_close_months_atual, aapl_close_months])

# Arredondamento de duas casas decimais
aapl_close_months = round(aapl_close_months, 2)

# Teve um split de 4:1 em 31/08/2020 (https://br.investing.com/equities/apple-computer-inc-historical-data-splits)

# Substituindo o index para ficar igual ao dos dados fundamentalistas
aapl_close_months.index = aapl_bs_10q.columns[:-3]

aapl_close_months

In [ ]:
# Criando uma lista só para fazer a indexação em que o ano mais recente está no começo
idx_10q = aapl_bs_10q.columns

# Valor de Mercado - utilizando a função 'indicador_vm_2', porque o nº de ações está em escala diferente
vm_10q = f_eua.indicador_vm_2(ten_k=False, price=aapl_close_months, df_is=aapl_is_10q)

# LPA - utilizando a função 'indicador_lpa_2', porque o nº de ações está em escala diferente
lpa_acum = f_eua.indicador_lpa_2(ten_k=False, df_is=aapl_is_10q, idx=idx_10q)[1]
lpa_10q = f_eua.indicador_lpa_2(ten_k=False, df_is=aapl_is_10q, idx=idx_10q)[0]

# P/L Damodaran
p_lpa_10q_damodaran = f_eua.indicador_p_lpa_2(ten_k=False, vm=vm_10q, df_is=aapl_is_10q)

# P/L
p_lpa_10q = f_eua.indicador_p_lpa(ten_k=False, price=aapl_close_months, lpa=lpa_acum, idx=idx_10q)

# L/P
lpa_p_10q = f_eua.indicador_lpa_p(ten_k=False, price=aapl_close_months, lpa=lpa_acum, idx=idx_10q) 

# Depreciação
depreciacao_10q = f_eua.indicador_depreciacao(ten_k=False, df_cf=aapl_cf_10q, first_quarter='12', idx=idx_10q)[0]
depreciacao_10q_sliced = f_eua.indicador_depreciacao(ten_k=False, df_cf=aapl_cf_10q, first_quarter='12', idx=idx_10q)[1]
depreciacao_10q_acum = f_eua.indicador_depreciacao(ten_k=False, df_cf=aapl_cf_10q, first_quarter='12', idx=idx_10q)[2]

# EBITDA
ebitda_10q = f_eua.indicador_ebitda(ten_k=False, df_is=aapl_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[0]
ebitda_10q_sliced = f_eua.indicador_ebitda(ten_k=False, df_is=aapl_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[1]
ebitda_10q_acum = f_eua.indicador_ebitda(ten_k=False, df_is=aapl_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[2]

# Margem Líquida
m_liq_10q = f_eua.indicador_ml(ten_k=False, df_is=aapl_is_10q)

# VPA - utilizando a função 'indicador_vpa_2', porque o nº de ações está em escala diferente
vpa_10q = f_eua.indicador_vpa_2(ten_k=False, df_bs=aapl_bs_10q, df_is=aapl_is_10q)

# P/VPA
p_vpa_10q = f_eua.indicador_p_vpa(ten_k=False, price=aapl_close_months, vpa=vpa_10q, idx=idx_10q)

# Caixa e Equivalentes
lst_caixa = [
    'Cash and Cash Equivalents', 
    'Marketable Securities, Current', 
    'Marketable Securities, Noncurrent'
]

caixa_10q = f_eua.indicador_caixa(ten_k=False, df_bs=aapl_bs_10q, lst_itens_caixa=lst_caixa)

# Dívida Bruta - não somei o custo dos 'leases', porque ele não é divulgado no 10q
lst_divida = [
    'Commercial Paper', 
    'Long-term Debt, Current', 
    'Long-term Debt, Noncurrent'
]

div_bruta_10q = f_eua.indicador_divida_bruta(ten_k=False, df_bs=aapl_bs_10q, lst_itens_divida=lst_divida)

# Dívida Líquida
div_liq_10q = f_eua.indicador_divida_liquida(ten_k=False, caixa=caixa_10q, div_bruta=div_bruta_10q, idx=idx_10q)

# Dívida Líquida / EBITDA
dl_ebitda_10q = f_eua.indicador_dl_ebitda(ten_k=False, ebitda=ebitda_10q_acum, div_liq=div_liq_10q, idx=idx_10q)

# Dívida Líquida / PL
dl_pl_10q = f_eua.indicador_dl_pl(ten_k=False, div_liq=div_liq_10q, df_bs=aapl_bs_10q) 

# EV/EBITDA
ev_ebitda_10q = f_eua.indicador_ev_ebitda(vm=vm_10q, div_liq=div_liq_10q, ebitda=ebitda_10q_acum)

# ROE
roe_10q = f_eua.indicador_roe(ten_k=False, df_is=aapl_is_10q, df_bs=aapl_bs_10q, idx=idx_10q)

# ROIC 
roic_10q = f_eua.indicador_roic(ten_k=False, df_is=aapl_is_10q, df_bs=aapl_bs_10q, div_bruta=div_bruta_10q, idx=idx_10q)

# FCO
fco_10q = f_eua.indicador_fco(ten_k=False, df_cf=aapl_cf_10q, first_quarter='12') 

# FCI
fci_10q = f_eua.indicador_fci(ten_k=False, df_cf=aapl_cf_10q, first_quarter='12') 

# FCF
fcf_10q = f_eua.indicador_fcf(ten_k=False, df_cf=aapl_cf_10q, first_quarter='12') 

# CAPEX
lst_capex_10q = ['Purchases Of Property And Equipment And Intangible Assets']

capex_10q = f_eua.indicador_capex(ten_k=False, df_cf=aapl_cf_10q, first_quarter='12', lst_itens=lst_capex_10q, idx=idx_10q)[0]
capex_10q_sliced = f_eua.indicador_capex(ten_k=False, df_cf=aapl_cf_10q, first_quarter='12', lst_itens=lst_capex_10q, idx=idx_10q)[1]
capex_10q_acum = f_eua.indicador_capex(ten_k=False, df_cf=aapl_cf_10q, first_quarter='12', lst_itens=lst_capex_10q, idx=idx_10q)[2]

# Free Cash Flow
free_cash_flow_10q = f_eua.indicador_free_cash_flow(fco=fco_10q, capex=capex_10q_sliced)

# Net CAPEX
net_capex_10q = f_eua.indicador_net_capex(capex=capex_10q_acum, depreciation=depreciacao_10q_acum)

# R&D
rd_expense_10q = f_eua.indicador_rd(ten_k=False, df_is=aapl_is_10q, idx=idx_10q)[0]
rd_expense_10q_sliced = f_eua.indicador_rd(ten_k=False, df_is=aapl_is_10q, idx=idx_10q)[1]
rd_expense_10q_acum = f_eua.indicador_rd(ten_k=False, df_is=aapl_is_10q, idx=idx_10q)[2]

# Net CAPEX ajustado
adj_net_capex_10q = f_eua.indicador_adj_net_capex(net_capex=net_capex_10q, rd_expense=rd_expense_10q_acum)

# Working capital
working_capital_10q = f_eua.indicador_working_capital(ten_k=False, df_bs=aapl_bs_10q)

# Change in non cash working capital
lst_itens_change_non_cash_wc = [
    'Increase (Decrease) in Accounts Receivable',
    'Increase (Decrease) in Other Receivables',
    'Increase (Decrease) in Inventories',
    'Increase (Decrease) in Other Operating Assets',
    'Increase (Decrease) in Accounts Payable',
    'Increase (Decrease) in Other Operating Liabilities'
]

change_non_cash_wc_10q = f_eua.f_eua.indicador_change_non_cash_wc(ten_k=False, df_cf=aapl_cf_10q, lst_itens=lst_itens_change_non_cash_wc, first_quarter='12')[0]
change_non_cash_wc_10q_sliced = f_eua.f_eua.indicador_change_non_cash_wc(ten_k=False, df_cf=aapl_cf_10q, lst_itens=lst_itens_change_non_cash_wc, first_quarter='12')[1]

# Reinvestment rate
reinvestment_rate_10q = f_eua.indicador_reinvestment_rate(ten_k=False, net_capex=adj_net_capex_10q, change_non_cash_wc=change_non_cash_wc_10q, df_is=aapl_is_10q, idx=idx_10q)

# New borrowing
lst_itens_new_borrowing = ['Proceeds from Issuance of Long-term Debt']

new_borrowing_10q = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=aapl_cf_10q, first_quarter='12', idx=idx_10q)[0]
new_borrowing_10q_sliced = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=aapl_cf_10q, first_quarter='12', idx=idx_10q)[1]
new_borrowing_10q_acum = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=aapl_cf_10q, first_quarter='12', idx=idx_10q)[2]

# Debt paid
lst_itens_debt_paid = ['Repayments of Long-term Debt', 'Proceeds from (Repayments of) Commercial Paper']

debt_paid_10q = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=aapl_cf_10q, first_quarter='12', idx=idx_10q)[0]
debt_paid_10q_sliced = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=aapl_cf_10q, first_quarter='12', idx=idx_10q)[1]
debt_paid_10q_acum = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=aapl_cf_10q, first_quarter='12', idx=idx_10q)[2]

# Taxes on operating income
taxes_operating_income_10q = f_eua.indicador_taxes_operating_income(ten_k=False, df_is=aapl_is_10q, idx=idx_10q)

# FCFE
fcfe_10q = f_eua.indicador_fcfe(
    ten_k=False, 
    depreciacao=depreciacao_10q_acum, 
    capex=capex_10q_acum, 
    change_non_cash_wc=change_non_cash_wc_10q_sliced, 
    new_borrowing=new_borrowing_10q_acum, 
    debt_paid=debt_paid_10q_acum, 
    df_is=aapl_is_10q,
    idx=idx_10q
)

# FCFF
fcff_10q = f_eua.indicador_fcff(
        ten_k=False, 
        taxes_operating_income=taxes_operating_income_10q,
        depreciacao=depreciacao_10q_acum, 
        capex=capex_10q_acum, 
        change_non_cash_wc=change_non_cash_wc_10q_sliced, 
        df_is=aapl_is_10q,
        idx=idx_10q
)

# Dividendo não acumulado
div_nao_acum = f_eua.indicador_div_nao_acum(df_cf=aapl_cf_10q, first_quarter='12')

# Dividendo por Ação - utilizando a função 'indicador_dpa_2', porque o nº de ações está em escala diferente
dpa_10q = f_eua.indicador_dpa_2(ten_k=False, df_cf=aapl_cf_10q, df_is=aapl_is_10q, div_nao_acum=div_nao_acum)

# Payout
payout_10q = f_eua.indicador_payout(ten_k=False, df_cf=aapl_cf_10q, df_is=aapl_is_10q, div_nao_acum=div_nao_acum)

# Buyback
buyback_10q = f_eua.indicador_buyback(ten_k=False, df_cf=aapl_cf_10q,  first_quarter='12')

# Lista com todos os indicadores
lst_indicadores_10q = [
    p_lpa_10q_damodaran,
    p_lpa_10q,
    lpa_p_10q,  
    ebitda_10q_sliced, 
    m_liq_10q,
    p_vpa_10q,
    vm_10q,
    div_bruta_10q,
    caixa_10q,
    div_liq_10q,
    dl_ebitda_10q,
    dl_pl_10q,
    ev_ebitda_10q,
    roe_10q,
    roic_10q,
    fco_10q,
    fci_10q,
    fcf_10q,
    free_cash_flow_10q,
    capex_10q_sliced,
    net_capex_10q,
    rd_expense_10q_sliced,
    adj_net_capex_10q,
    working_capital_10q,
    reinvestment_rate_10q,
    fcfe_10q,
    fcff_10q,
    dpa_10q,
    payout_10q,
    buyback_10q
]

# Juntando todos os indicadores em um df
aapl_indicadores_10q = pd.concat(lst_indicadores_10q, axis=1)

# Nomes das colunas
nomes_col = [
    'P/L Damodaran', 
    'P/L', 
    'L/P', 
    'EBITDA', 
    'Margem Líquida',
    'P/VPA',
    'Valor de Mercado',
    'Dívida Bruta',
    'Caixa e Equivalentes',
    'Dívida Líquida',
    'Dívida Líquida/EBITDA',
    'Dívida Líquida/PL',
    'EV/EBITDA',
    'ROE',
    'ROIC',
    'FCO',
    'FCI',
    'FCF',
    'Free Cash Flow',
    'Capex',
    'Net Capex',
    'R&D',
    'Adj Net Capex',
    'Working Capital',
    'Reinvestment Rate',
    'FCFE',
    'FCFF',
    'DPA',
    'Payout',
    'Buyback'
]

# Atribuindo os novos nomes das colunas do df
aapl_indicadores_10q.columns = nomes_col
aapl_indicadores_10q

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//tecnologia//apple//aapl_indicators.xlsx') as writer:
    aapl_indicadores_10k.to_excel(writer, sheet_name='indicators_10k')
    aapl_indicadores_10q.to_excel(writer, sheet_name='indicators_10q')

## Google

* 1Q -> Lançamento em Março (03);
* 2Q -> Lançamento em Junho (06);
* 3Q -> Lançamento em Setembro (09);
* 10K -> Lançamento em Dezembro (12).

In [ ]:
ticker = 'googl'
f_sec.cik_matching_ticker(ticker)

### Scraping Google 10-K

In [ ]:
# Número dos documentos que contém a DRE, BP e FC 
acc_documents = f_sec.get_filtered_filings(ticker=ticker, ten_k=True, just_accession_numbers=True)

# Trocando os '-' para '' das strings ('0001045810-24-000264' -> '000104581024000264')
dict_acc_num = {k: v.replace('-', '') for k, v in acc_documents.items()}

# Transformando o dicionário em um df
df_lst_acc_num_googl = pd.DataFrame.from_dict(dict_acc_num, orient='index', columns=['acc_num'])

# Adicionando os 'acc_num' de 2017, 2018, 2019, 2020 e 2021
df_acc_num = pd.DataFrame(
    {
        'acc_num': [
            '000165204422000019', # 2021
            '000165204421000010', # 2020
            '000165204420000008', # 2019
            '000165204419000004', # 2018
            '000165204418000007', # 2017
        ]
    },
    index=pd.to_datetime([
        '2021-12-31', 
        '2020-12-31',
        '2019-12-31',
        '2018-12-31',
        '2017-12-31',
    ])
)

# Concatenando os dfs
df_lst_acc_num_googl = pd.concat([df_lst_acc_num_googl, df_acc_num])

# Transformando em datetime o index
df_lst_acc_num_googl.index = pd.to_datetime(df_lst_acc_num_googl.index, format='%Y-%m-%d')

df_lst_acc_num_googl

In [ ]:
# Rótulos
labels_dict = f_sec.get_label_dictionary(ticker, headers=f_sec.header)

In [ ]:
# Fazendo o scraping do balance sheet (10-K)
balance_sheet_lst = []

for acc_num in df_lst_acc_num_googl['acc_num']:

    # Balance sheet
    balance_sheet = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='balance_sheet')
    balance_sheet = f_sec.rename_statement(balance_sheet, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    balance_sheet = balance_sheet.iloc[:, 0]
    balance_sheet_lst.append(balance_sheet)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_balance_sheet_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in balance_sheet_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_balance_sheet_10k = pd.concat([df_balance_sheet_10k, df], axis=1)

# Substituindo os NaN por 0
df_balance_sheet_10k = df_balance_sheet_10k.fillna(0)

# A linha 'Marketable Securities, Current' na coluna '2017-12-31' está zerada
# As informações estão na linha 'Available-for-sale Securities, Current'
df_balance_sheet_10k.loc['Marketable Securities, Current', '2017-12-31'] = df_balance_sheet_10k.loc['Available-for-sale Securities, Current', '2017-12-31'] 

# A linha 'Contract with Customer, Liability, Noncurrent' na coluna '2017-12-31' está zerada
# As informações estão na linha 'Deferred Revenue, Noncurrent'
df_balance_sheet_10k.loc['Contract with Customer, Liability, Noncurrent', '2017-12-31'] = df_balance_sheet_10k.loc['Deferred Revenue, Noncurrent', '2017-12-31'] 

# A linha 'Accrued Income Taxes, Noncurrent' na coluna '2017-12-31' está zerada
# As informações estão na linha 'Liability for Uncertainty in Income Taxes, Noncurrent'
df_balance_sheet_10k.loc['Accrued Income Taxes, Noncurrent', '2017-12-31'] = df_balance_sheet_10k.loc['Liability for Uncertainty in Income Taxes, Noncurrent', '2017-12-31'] 

# A linha 'Contract with Customer, Liability, Current' na coluna '2017-12-31' está zerada
# As informações estão na linha 'Deferred Revenue, Current'
df_balance_sheet_10k.loc['Contract with Customer, Liability, Current', '2017-12-31'] = df_balance_sheet_10k.loc['Deferred Revenue, Current', '2017-12-31'] 

# A linha 'Property, Plant and Equipment, Net' nas colunas '2019-12-31' e '2025-12-31' estão zeradas
# As informações estão na linha 'goog_PropertyPlantandEquipmentandFinanceLeaseRightofUseAssetafterAccumulatedDepreciationandAmortization' e 'Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization'
df_balance_sheet_10k.loc['Property, Plant and Equipment, Net', '2019-12-31'] = df_balance_sheet_10k.loc['goog_PropertyPlantandEquipmentandFinanceLeaseRightofUseAssetafterAccumulatedDepreciationandAmortization', '2019-12-31'] 
df_balance_sheet_10k.loc['Property, Plant and Equipment, Net', '2025-12-31'] = df_balance_sheet_10k.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2025-12-31'] 

# A linha 'Long-term Debt and Lease Obligation' na coluna '2025-12-31' está zerada
# As informações estão na linha 'Long-term Debt, Excluding Current Maturities'
df_balance_sheet_10k.loc['Long-term Debt and Lease Obligation', '2025-12-31'] = df_balance_sheet_10k.loc['Long-term Debt, Excluding Current Maturities', '2025-12-31'] 

# Removendo as linhas que movemos os seus itens para as linhas corretas
df_balance_sheet_10k = df_balance_sheet_10k.drop([
    'Available-for-sale Securities, Current',
    'Deferred Revenue, Noncurrent',
    'Liability for Uncertainty in Income Taxes, Noncurrent',
    'Deferred Revenue, Current',
    'goog_PropertyPlantandEquipmentandFinanceLeaseRightofUseAssetafterAccumulatedDepreciationandAmortization',
    'Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization',
    'Long-term Debt, Excluding Current Maturities',
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_balance_sheet_10k = df_balance_sheet_10k.drop([
    'us-gaap_AssetsCurrentAbstract',
    'Convertible Preferred Stock, Nonredeemable or Redeemable, Issuer Option, Value',
    'Common Stock, Shares Authorized',
    'Preferred Stock, Shares Authorized',
    'Preferred Stock, Par or Stated Value Per Share'
])

# Renomeando a linha do índice que está sem formato 
df_balance_sheet_10k = df_balance_sheet_10k.rename(index={
    'goog_AccruedRevenueShare': 'Accrued Revenue Share'
})

df_balance_sheet_10k

In [ ]:
# Fazendo o scraping do income statement (10-K)
income_statement_lst = []

for acc_num in df_lst_acc_num_googl['acc_num']:

    # Income statement
    income_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='income_statement')
    income_statement = f_sec.rename_statement(income_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    income_statement = income_statement.iloc[:, 0]
    income_statement_lst.append(income_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_income_statement_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in income_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_income_statement_10k = pd.concat([df_income_statement_10k, df], axis=1)

# Substituindo os NaN por 0
df_income_statement_10k = df_income_statement_10k.fillna(0)

# A linha 'Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest' nas colunas '2020-12-31', '2019-12-31', '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest'
df_income_statement_10k.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2020-12-31'] = df_income_statement_10k.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2020-12-31'] 
df_income_statement_10k.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2019-12-31'] = df_income_statement_10k.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2019-12-31'] 
df_income_statement_10k.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2018-12-31'] = df_income_statement_10k.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2018-12-31'] 
df_income_statement_10k.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2017-12-31'] = df_income_statement_10k.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2017-12-31'] 

# A linha 'Revenue from Contract with Customer, Excluding Assessed Tax' nas colunas '2021-12-31', '2020-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'Revenues'
df_income_statement_10k.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2025-12-31'] = df_income_statement_10k.loc['Revenues', '2025-12-31'] 
df_income_statement_10k.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2021-12-31'] = df_income_statement_10k.loc['Revenues', '2021-12-31'] 
df_income_statement_10k.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2020-12-31'] = df_income_statement_10k.loc['Revenues', '2020-12-31'] 
df_income_statement_10k.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2017-12-31'] = df_income_statement_10k.loc['Revenues', '2017-12-31'] 

# Removendo as linhas que movemos os seus itens para as linhas corretas
df_income_statement_10k = df_income_statement_10k.drop([
    'Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest',
    'Revenues'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_income_statement_10k = df_income_statement_10k.drop([
    'us-gaap_IncomeStatementAbstract',
    'us-gaap_CostsAndExpensesAbstract',
    'Preferred Stock Dividends and Other Adjustments',
    'Loss Contingency, Loss in Period',
    'Net Income (Loss) Available to Common Stockholders, Basic'
])

df_income_statement_10k

In [ ]:
# Fazendo o scraping do cash flow statement (10-K)
cash_flow_statement_lst = []

for acc_num in df_lst_acc_num_googl['acc_num']:

    # Cash flow statement
    cash_flow_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='cash_flow_statement')
    cash_flow_statement = f_sec.rename_statement(cash_flow_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    cash_flow_statement = cash_flow_statement.iloc[:, 0]
    cash_flow_statement_lst.append(cash_flow_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_cash_flow_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in cash_flow_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_cash_flow_10k = pd.concat([df_cash_flow_10k, df], axis=1)

# Substituindo os NaN por 0
df_cash_flow_10k = df_cash_flow_10k.fillna(0)

# # A linha 'Depreciation' nas colunas '2022-12-31', '2021-12-31', '2020-12-31', '2019-12-31', '2018-12-31' e '2017-12-31' estão zeradas
# # As informações estão na linha 'goog_DepreciationAndImpairmentOnDispositionOfPropertyAndEquipment'
df_cash_flow_10k.loc['Depreciation', '2022-12-31'] = df_cash_flow_10k.loc['goog_DepreciationAndImpairmentOnDispositionOfPropertyAndEquipment', '2022-12-31'] 
df_cash_flow_10k.loc['Depreciation', '2021-12-31'] = df_cash_flow_10k.loc['goog_DepreciationAndImpairmentOnDispositionOfPropertyAndEquipment', '2021-12-31'] 
df_cash_flow_10k.loc['Depreciation', '2020-12-31'] = df_cash_flow_10k.loc['goog_DepreciationAndImpairmentOnDispositionOfPropertyAndEquipment', '2020-12-31'] 
df_cash_flow_10k.loc['Depreciation', '2019-12-31'] = df_cash_flow_10k.loc['goog_DepreciationAndImpairmentOnDispositionOfPropertyAndEquipment', '2019-12-31'] 
df_cash_flow_10k.loc['Depreciation', '2018-12-31'] = df_cash_flow_10k.loc['goog_DepreciationAndImpairmentOnDispositionOfPropertyAndEquipment', '2018-12-31'] 
df_cash_flow_10k.loc['Depreciation', '2017-12-31'] = df_cash_flow_10k.loc['goog_DepreciationAndImpairmentOnDispositionOfPropertyAndEquipment', '2017-12-31'] 

# # A linha 'Payments for (Proceeds from) Other Investing Activities' nas colunas '2018-12-31' e '2017-12-31' estão zeradas
# # As informações estão na linha 'Proceeds from Sale of Property, Plant, and Equipment'
df_cash_flow_10k.loc['Payments for (Proceeds from) Other Investing Activities', '2018-12-31'] = df_cash_flow_10k.loc['Proceeds from Sale of Property, Plant, and Equipment', '2018-12-31'] 
df_cash_flow_10k.loc['Payments for (Proceeds from) Other Investing Activities', '2017-12-31'] = df_cash_flow_10k.loc['Proceeds from Sale of Property, Plant, and Equipment', '2017-12-31'] 

# # A linha 'Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents' nas colunas '2018-12-31' e '2017-12-31' estão zeradas
# # As informações estão na linha 'Effect of Exchange Rate on Cash and Cash Equivalents'
df_cash_flow_10k.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2018-12-31'] = df_cash_flow_10k.loc['Effect of Exchange Rate on Cash and Cash Equivalents', '2018-12-31'] 
df_cash_flow_10k.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2017-12-31'] = df_cash_flow_10k.loc['Effect of Exchange Rate on Cash and Cash Equivalents', '2017-12-31'] 

# A linha 'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect' nas colunas '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'Cash and Cash Equivalents, Period Increase (Decrease)'
df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect', '2018-12-31'] = df_cash_flow_10k.loc['Cash and Cash Equivalents, Period Increase (Decrease)', '2018-12-31'] 
df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect', '2017-12-31'] = df_cash_flow_10k.loc['Cash and Cash Equivalents, Period Increase (Decrease)', '2017-12-31'] 

# A linha 'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents' nas colunas '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'Cash and Cash Equivalents, at Carrying Value'
df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2018-12-31'] = df_cash_flow_10k.loc['Cash and Cash Equivalents, at Carrying Value', '2018-12-31'] 
df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2017-12-31'] = df_cash_flow_10k.loc['Cash and Cash Equivalents, at Carrying Value', '2017-12-31'] 

# # A linha 'Increase (Decrease) in Contract with Customer, Liability' na coluna '2017-12-31' está zeradas
# # As informações estão na linha 'Increase (Decrease) in Deferred Revenue'
df_cash_flow_10k.loc['Increase (Decrease) in Contract with Customer, Liability', '2017-12-31'] = df_cash_flow_10k.loc['Increase (Decrease) in Deferred Revenue', '2017-12-31'] 

# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_cash_flow_10k = df_cash_flow_10k.drop([
    'goog_DepreciationAndImpairmentOnDispositionOfPropertyAndEquipment',
    'Proceeds from Sale of Property, Plant, and Equipment',
    'Effect of Exchange Rate on Cash and Cash Equivalents',
    'Cash and Cash Equivalents, Period Increase (Decrease)',
    'Cash and Cash Equivalents, at Carrying Value',
    'Increase (Decrease) in Deferred Revenue'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_cash_flow_10k = df_cash_flow_10k.drop([
    'us-gaap_NetCashProvidedByUsedInOperatingActivitiesAbstract',
    'Increase (Decrease) in Collateral Held under Securities Lending',
    'Proceeds from (Payments for) Securities Purchased under Agreements to Resell'
])

# Arrumando e adicionando alguns números no df
df_cash_flow_10k.loc['Debt and Equity Securities, Gain (Loss)', '2017-12-31'] = 37

# Renomeando a linha do índice que está sem formato 
df_cash_flow_10k = df_cash_flow_10k.rename(index={
    'goog_IncreaseDecreaseInAccruedRevenueShare': 'Increase (Decrease) in Accrued Revenue Share',
    'goog_AcquisitionsNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets': 'Acquisitions Net Of Cash Acquired And Purchases Of Intangible And Other Assets',
    'goog_NetProceedsPaymentsRelatedToStockBasedAwardActivities': 'Net Proceeds Payments Related To Stock Based Award Activities',
    'goog_AmortizationAndImpairmentOfIntangibleAssets': 'Amortization And Impairment Of Intangible Assets',
    'goog_GainLossOnMarketableAndNonMarketableInvestmentsNet': 'Gain Loss On Marketable And Non Marketable Investments, Net',
})

df_cash_flow_10k

In [ ]:
# Arrumando alguns index dos dfs para ficar padronizado com os index utilizados nas funções p/ calcular os indicadores fundamentalistas
df_balance_sheet_10k = df_balance_sheet_10k.rename(index={
    'Cash and Cash Equivalents, at Carrying Value': 'Cash and Cash Equivalents',
    'Marketable Securities, Current': 'Marketable Securities',
    'Other Long-term Investments': 'Non-marketable Securities',
    'Long-term Debt and Lease Obligation': 'Long-term Debt',
    'Operating Lease, Liability, Noncurrent': 'Operating Lease Liabilities',
    "Stockholders' Equity Attributable to Parent": 'Total Shareholders Equity'
})

df_income_statement_10k = df_income_statement_10k.rename(index={
    'Revenue from Contract with Customer, Excluding Assessed Tax': 'Revenues',
    'Research and Development Expense': 'Research and Development',
    'Operating Income (Loss)': 'Operating Income',
    'Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest': 'Income Before Provision for Income Taxes',
    'Income Tax Expense (Benefit)': 'Provision for Income Taxes',
    'Net Income (Loss) Attributable to Parent': 'Net Income'
})

df_cash_flow_10k = df_cash_flow_10k.rename(index={
    'Depreciation': 'Depreciation, Depletion and Amortization',
    'Payments to Acquire Property, Plant, and Equipment': 'Purchases Of Property And Equipment And Intangible Assets'
})

In [ ]:
# Selecionando o 'acc_num' do ano mais recente
last_acc_num = df_lst_acc_num_googl.iloc[0].item()

# Fazendo o scraping do interest expense (10-K) -> 'Other Income (Expense), Net' 
interest_expense_10k = f_sec.process_one_statement(ticker=ticker, accession_number=last_acc_num, statement_name='interest_expense')
interest_expense_10k = f_sec.rename_statement(interest_expense_10k, label_dictionary=labels_dict)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
interest_expense_10k = interest_expense_10k.drop('goog_BalanceSheetComponentsDisclosureAbstract')

# Renomeando a linha do índice que está sem formato 
interest_expense_10k = interest_expense_10k.rename(index={
    'InterestExpenseNonoperating': 'Interest Expense',
})

# Formatando o número para ficar positivo
interest_expense_10k.loc['Interest Expense'] = interest_expense_10k.loc['Interest Expense'] * -1

# Selecionando a linha desejada
interest_expense_10k = interest_expense_10k.loc[['Interest Expense']]

interest_expense_10k

In [ ]:
# 'acc_num' de 2024 - eu não coloquei no loop for abaixo, porque ele tem um df com três colunas ('2024-09-30', '2024-12-31' e '2023-12-31')
acc_num_2024 = '000165204425000014'

# Fazendo o scraping do leases (10-K) -> 'Note 8 – Leases' -> 'Total lease liabilities'
leases_10k_2024 = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num_2024, statement_name='leases')
leases_10k_2024 = f_sec.rename_statement(leases_10k_2024, label_dictionary=labels_dict)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
leases_10k_2024 = leases_10k_2024.drop('goog_BalanceSheetComponentsDisclosureAbstract')
leases_10k_2024 = leases_10k_2024.drop('Loss Contingency Accrual, Payments')

# Renomeando a linha do índice
leases_10k_2024 = leases_10k_2024.rename(index={
    'Operating Lease, Liability, Current': 'Current Operating Lease Liabilities',
})

# Selecionando a linha e a coluna desejada
leases_10k_2024 = leases_10k_2024.loc[['Current Operating Lease Liabilities']]
leases_10k_2024 = leases_10k_2024.loc[:, '2024-12-31']

##############################################################################################

# 'acc_num' entre 2023 e 2019
df_lst_acc_num_googl_sorted = df_lst_acc_num_googl.sort_index()
acc_num_list_googl_sliced = df_lst_acc_num_googl_sorted.loc['2019':'2023', 'acc_num']

# Fazendo o scraping do leases (10-K)
leases_lst = []

for acc_num in acc_num_list_googl_sliced:

    # Fazendo o scraping do leases (10-K) -> 'Note 8 – Leases' -> 'Total lease liabilities'
    leases_10k = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='leases')
    leases_10k = f_sec.rename_statement(leases_10k, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    leases_10k = leases_10k.iloc[:, 0]
    leases_lst.append(leases_10k)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_leases_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in leases_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_leases_10k = pd.concat([df_leases_10k, df], axis=1)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_leases_10k = df_leases_10k.drop('goog_BalanceSheetComponentsDisclosureAbstract')

# Renomeando a linha do índice
df_leases_10k = df_leases_10k.rename(index={
    'Operating Lease, Liability, Current': 'Current Operating Lease Liabilities',
})

# Selecionando a linha desejada
df_leases_10k = df_leases_10k.loc[['Current Operating Lease Liabilities']]

# Criando as colunas dos anos de 2017, 2018, 2024 e 2025
df_leases_10k['2017-12-31'] = 0
df_leases_10k['2018-12-31'] = 0	
df_leases_10k['2024-12-31'] = leases_10k_2024
df_leases_10k['2025-12-31'] = 0

# Ordenando o df
df_leases_10k = df_leases_10k[sorted(df_leases_10k.columns, reverse=True)]

df_leases_10k

In [ ]:
# 'acc_num' entre 2023 e 2019
df_lst_acc_num_googl_sorted = df_lst_acc_num_googl.sort_index()
acc_num_list_googl_sliced = df_lst_acc_num_googl_sorted.loc['2020':'2023', 'acc_num']

# Fazendo o scraping do leases (10-K)
current_portion_lease_lst = []

for acc_num in acc_num_list_googl_sliced:

    # Fazendo o scraping do long-term debt (10-K) -> # 'Note 6. Debt' -> 'Long-Term Debt' -> 'Current portion of future finance lease payments'
    current_portion_lease_10k = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='current_portion_lease')
    current_portion_lease_10k = f_sec.rename_statement(current_portion_lease_10k, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    current_portion_lease_10k = current_portion_lease_10k.iloc[:, 0]
    current_portion_lease_lst.append(current_portion_lease_10k)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_current_portion_lease_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in current_portion_lease_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_current_portion_lease_10k = pd.concat([df_current_portion_lease_10k, df], axis=1)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_current_portion_lease_10k = df_current_portion_lease_10k.drop('us-gaap_LongTermDebtNoncurrentAbstract')

# Renomeando a linha do índice
df_current_portion_lease_10k = df_current_portion_lease_10k.rename(index={
    'Finance Lease, Liability, Current': 'Current Portion of Future Finance Lease Payments',
})

# Formatando o número para ficar positivo
df_current_portion_lease_10k.loc['Current Portion of Future Finance Lease Payments'] = df_current_portion_lease_10k.loc['Current Portion of Future Finance Lease Payments'] * -1

# Selecionando a linha desejada
df_current_portion_lease_10k = df_current_portion_lease_10k.loc[['Current Portion of Future Finance Lease Payments']]

# Criando as colunas dos anos de 2017, 2018, 2019, 2024 e 2025
df_current_portion_lease_10k['2017-12-31'] = 0
df_current_portion_lease_10k['2018-12-31'] = 0	
df_current_portion_lease_10k['2019-12-31'] = 115
df_current_portion_lease_10k['2024-12-31'] = 0
df_current_portion_lease_10k['2025-12-31'] = 0

# Ordenando o df
df_current_portion_lease_10k = df_current_portion_lease_10k[sorted(df_current_portion_lease_10k.columns, reverse=True)]

df_current_portion_lease_10k

In [ ]:
# Pegando os dados dos nº de ações de todo o período para usar esses dados na parte dos indicadores fundamentalistas

# 'acc_num' de 2025
acc_num_2025 = '000165204426000018'

# Fazendo o scraping do restricted stock units (10-K) -> 'Note 11 – Share-Based Compensation' -> 'Number of RSUs'
shares_10k_25 = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num_2025, statement_name='rsu')
shares_10k_25 = f_sec.rename_statement(shares_10k_25, label_dictionary=labels_dict)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
shares_10k_25 = shares_10k_25.drop('us-gaap_NetIncomeLossAvailableToCommonStockholdersBasicAbstract')

# Resetando o index do df, porque ele possui vários index iguais
shares_10k_25 = shares_10k_25.reset_index(drop=True)

# Selecionando as linhas desejadas e a coluna '2025-12-31'
shares_10k_25 = shares_10k_25.iloc[[5, 11, 15, 27], [0]]

# Renomeando a linha do índice
shares_10k_25 = shares_10k_25.rename(index={
    5: 'RSUs',
    11: 'Class A',
    15: 'Class B',
    27: 'Class C',
})

##################################################

# 'acc_num' de 2024
acc_num_2024 = '000165204425000014'

# Fazendo o scraping do restricted stock units (10-K) -> 'Note 11 – Share-Based Compensation' -> 'Number of RSUs'
shares_10k_24_23_22 = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num_2024, statement_name='rsu')
shares_10k_24_23_22 = f_sec.rename_statement(shares_10k_24_23_22, label_dictionary=labels_dict)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
shares_10k_24_23_22 = shares_10k_24_23_22.drop('us-gaap_NetIncomeLossAvailableToCommonStockholdersBasicAbstract')

# Resetando o index do df, porque ele possui vários index iguais
shares_10k_24_23_22 = shares_10k_24_23_22.reset_index(drop=True)

# Selecionando as linhas desejadas
shares_10k_24_23_22 = shares_10k_24_23_22.iloc[[5, 11, 15, 26]]

# Renomeando a linha do índice
shares_10k_24_23_22 = shares_10k_24_23_22.rename(index={
    5: 'RSUs',
    11: 'Class A',
    15: 'Class B',
    26: 'Class C',
})

##################################################

# 'acc_num' de 2021
acc_num_2021 = '000165204422000019'

# Fazendo o scraping do restricted stock units (10-K) -> 'Note 11 – Share-Based Compensation' -> 'Number of RSUs'
shares_10k_21_20_19 = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num_2021, statement_name='rsu')
shares_10k_21_20_19 = f_sec.rename_statement(shares_10k_21_20_19, label_dictionary=labels_dict)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
shares_10k_21_20_19 = shares_10k_21_20_19.drop('us-gaap_WeightedAverageNumberOfSharesOutstandingBasicAbstract')

# Resetando o index do df, porque ele possui vários index iguais
shares_10k_21_20_19 = shares_10k_21_20_19.reset_index(drop=True)

# Selecionando as linhas desejadas
shares_10k_21_20_19 = shares_10k_21_20_19.iloc[[3, 7, 8, 14]]

# Renomeando a linha do índice
shares_10k_21_20_19 = shares_10k_21_20_19.rename(index={
    8: 'RSUs',
    3: 'Class A',
    7: 'Class B',
    14: 'Class C',
})

##################################################

# 'acc_num' de 2018
acc_num_2018 = '000165204419000004'

# Fazendo o scraping do restricted stock units (10-K) -> 'Note 11 – Share-Based Compensation' -> 'Number of RSUs'
shares_10k_18_17 = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num_2018, statement_name='rsu')
shares_10k_18_17 = f_sec.rename_statement(shares_10k_18_17, label_dictionary=labels_dict)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
shares_10k_18_17 = shares_10k_18_17.drop('us-gaap_WeightedAverageNumberOfSharesOutstandingBasicAbstract')

# Resetando o index do df, porque ele possui vários index iguais
shares_10k_18_17 = shares_10k_18_17.reset_index(drop=True)

# Selecionando as linhas desejadas
shares_10k_18_17 = shares_10k_18_17.iloc[[3, 9, 10, 19]]

# Renomeando a linha do índice
shares_10k_18_17 = shares_10k_18_17.rename(index={
    10: 'RSUs',
    3: 'Class A',
    9: 'Class B',
    19: 'Class C',
})

# Removendo a coluna de 2016
shares_10k_18_17 = shares_10k_18_17.drop(columns='2016-12-31')

##################################################

# Concatenando os dfs
shares_10k = pd.concat([
    shares_10k_25,
    shares_10k_24_23_22, 
    shares_10k_21_20_19,
    shares_10k_18_17
], axis=1)

shares_10k

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//tecnologia//google//googl_10k.xlsx') as writer:
    df_balance_sheet_10k.to_excel(writer, sheet_name='balance_sheet')
    df_income_statement_10k.to_excel(writer, sheet_name='income_statement')
    df_cash_flow_10k.to_excel(writer, sheet_name='cash_flow')
    interest_expense_10k.to_excel(writer, sheet_name='interest_expense')
    df_leases_10k.to_excel(writer, sheet_name='leases')
    df_current_portion_lease_10k.to_excel(writer, sheet_name='current_portion_lease')
    shares_10k.to_excel(writer, sheet_name='num_shares')

### Scraping Google 10-Q

In [ ]:
# Número dos documentos que contém a DRE, BP e FC 
acc_documents_10q = f_sec.get_filtered_filings(ticker=ticker, ten_k=False, just_accession_numbers=True)

# Trocando os '-' para '' das strings ('0001045810-24-000264' -> '000104581024000264')
dict_acc_num_10q = {k: v.replace('-', '') for k, v in acc_documents_10q.items()}

# Transformando o dicionário em um df
df_lst_acc_num_10q_googl = pd.DataFrame.from_dict(dict_acc_num_10q, orient='index', columns=['acc_num'])

# Adicionando os 'acc_num' de 1Q, 2Q e 3Q dos anos de 2020 e 2021
df_acc_num = pd.DataFrame(
    {
        'acc_num': [
            '000165204421000057', # 3Q 2021
            '000165204421000047', # 2Q 2021
            '000165204421000020', # 1Q 2021
            '000165204420000050', # 3Q 2020
            '000165204420000032', # 2Q 2020
            '000165204420000021', # 1Q 2020
        ]
    },
    index=pd.to_datetime([
        '2021-09-30', 
        '2021-06-30',
        '2021-03-31',
        '2020-09-30', 
        '2020-06-30',
        '2020-03-31',

    ])
)

# Concatenando os dfs
df_lst_acc_num_10q_googl = pd.concat([df_lst_acc_num_10q_googl, df_acc_num])

# Transformando em datetime o index
df_lst_acc_num_10q_googl.index = pd.to_datetime(df_lst_acc_num_10q_googl.index, format='%Y-%m-%d')

df_lst_acc_num_10q_googl

In [ ]:
# Rótulos
labels_dict = f_sec.get_label_dictionary(ticker, headers=f_sec.header)

In [ ]:
# Fazendo o scraping do balance sheet (10-Q)
balance_sheet_lst = []

for acc_num in df_lst_acc_num_10q_googl['acc_num']:

    # Balance sheet
    balance_sheet = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='balance_sheet')
    balance_sheet = f_sec.rename_statement(balance_sheet, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    balance_sheet = balance_sheet.iloc[:, 0]
    balance_sheet_lst.append(balance_sheet)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_balance_sheet_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in balance_sheet_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_balance_sheet_10q = pd.concat([df_balance_sheet_10q, df], axis=1)

# Substituindo os NaN por 0
df_balance_sheet_10q = df_balance_sheet_10q.fillna(0)

# A linha 'Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization' nas colunas '2025-03-31', '2024-09-30', '2024-06-30', '2024-03-31' e '2020-03-31' estão zeradas
# As informações estão nas linhas 'Property, Plant and Equipment, Net' e 'goog_PropertyPlantandEquipmentandFinanceLeaseRightofUseAssetafterAccumulatedDepreciationandAmortization'
df_balance_sheet_10q.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2025-03-31'] = df_balance_sheet_10q.loc['Property, Plant and Equipment, Net', '2025-03-31'] 
df_balance_sheet_10q.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2024-09-30'] = df_balance_sheet_10q.loc['Property, Plant and Equipment, Net', '2024-09-30'] 
df_balance_sheet_10q.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2024-06-30'] = df_balance_sheet_10q.loc['Property, Plant and Equipment, Net', '2024-06-30'] 
df_balance_sheet_10q.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2024-03-31'] = df_balance_sheet_10q.loc['Property, Plant and Equipment, Net', '2024-03-31'] 
df_balance_sheet_10q.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2020-03-31'] = df_balance_sheet_10q.loc['goog_PropertyPlantandEquipmentandFinanceLeaseRightofUseAssetafterAccumulatedDepreciationandAmortization', '2020-03-31'] 

# A linha 'Long-term Debt and Lease Obligation' nas colunas '2025-09-30' e '2025-06-30' estão zeradas
# As informações estão na linha 'Long-term Debt, Excluding Current Maturities' 
df_balance_sheet_10q.loc['Long-term Debt and Lease Obligation', '2025-09-30'] = df_balance_sheet_10q.loc['Long-term Debt, Excluding Current Maturities', '2025-09-30'] 
df_balance_sheet_10q.loc['Long-term Debt and Lease Obligation', '2025-06-30'] = df_balance_sheet_10q.loc['Long-term Debt, Excluding Current Maturities', '2025-06-30'] 

# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_balance_sheet_10q = df_balance_sheet_10q.drop([
    'Property, Plant and Equipment, Net',
    'goog_PropertyPlantandEquipmentandFinanceLeaseRightofUseAssetafterAccumulatedDepreciationandAmortization',
    'Long-term Debt, Excluding Current Maturities'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_balance_sheet_10q = df_balance_sheet_10q.drop([
    'us-gaap_AssetsCurrentAbstract',
    'Convertible Preferred Stock, Nonredeemable or Redeemable, Issuer Option, Value',
    'Preferred Stock, Par or Stated Value Per Share',
    'Preferred Stock, Shares Authorized',
    'Common Stock, Shares Authorized',
    'Common Stock, Shares, Issued'
])

# Renomeando a linha do índice que está sem formato 
df_balance_sheet_10q = df_balance_sheet_10q.rename(index={
    'goog_AccruedRevenueShare': 'Accrued Revenue Share'
})

df_balance_sheet_10q

In [ ]:
# Fazendo o scraping do income statement (10-Q)
income_statement_lst = []

for acc_num in df_lst_acc_num_10q_googl['acc_num']:

    # Income statement
    income_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='income_statement')
    income_statement = f_sec.rename_statement(income_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    income_statement = income_statement.iloc[:, 0]
    income_statement_lst.append(income_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_income_statement_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in income_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_income_statement_10q = pd.concat([df_income_statement_10q, df], axis=1)

# Substituindo os NaN por 0
df_income_statement_10q = df_income_statement_10q.fillna(0)

# A linha 'Revenues' nas colunas '2025-03-31', '2024-09-30', '2024-06-30', '2024-03-31', '2023-09-30', '2023-06-30', '2023-03-31', '2022-09-30', '2022-06-30', '2022-03-31', '2021-09-30', '2021-06-30', '2021-03-31', '2020-09-30', '2020-06-30' e '2020-03-31' estao zeradas
# As informações estão na linha 'Revenue from Contract with Customer, Excluding Assessed Tax'
df_income_statement_10q.loc['Revenues', '2025-03-31'] = df_income_statement_10q.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2025-03-31'] 
df_income_statement_10q.loc['Revenues', '2024-09-30'] = df_income_statement_10q.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2024-09-30'] 
df_income_statement_10q.loc['Revenues', '2024-06-30'] = df_income_statement_10q.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2024-06-30'] 
df_income_statement_10q.loc['Revenues', '2024-03-31'] = df_income_statement_10q.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2024-03-31'] 
df_income_statement_10q.loc['Revenues', '2023-09-30'] = df_income_statement_10q.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2023-09-30'] 
df_income_statement_10q.loc['Revenues', '2023-06-30'] = df_income_statement_10q.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2023-06-30'] 
df_income_statement_10q.loc['Revenues', '2023-03-31'] = df_income_statement_10q.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2023-03-31'] 
df_income_statement_10q.loc['Revenues', '2022-09-30'] = df_income_statement_10q.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2022-09-30'] 
df_income_statement_10q.loc['Revenues', '2022-06-30'] = df_income_statement_10q.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2022-06-30'] 
df_income_statement_10q.loc['Revenues', '2022-03-31'] = df_income_statement_10q.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2022-03-31'] 
df_income_statement_10q.loc['Revenues', '2021-09-30'] = df_income_statement_10q.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2021-09-30'] 
df_income_statement_10q.loc['Revenues', '2021-06-30'] = df_income_statement_10q.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2021-06-30'] 
df_income_statement_10q.loc['Revenues', '2021-03-31'] = df_income_statement_10q.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2021-03-31'] 
df_income_statement_10q.loc['Revenues', '2020-09-30'] = df_income_statement_10q.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2020-09-30'] 
df_income_statement_10q.loc['Revenues', '2020-06-30'] = df_income_statement_10q.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2020-06-30'] 
df_income_statement_10q.loc['Revenues', '2020-03-31'] = df_income_statement_10q.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2020-03-31'] 

# A linha 'Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest' nas colunas '2021-06-30', '2021-03-31', '2020-09-30', '2020-06-30' e '2020-03-31' estão zeradas
# As informações estão na linha 'Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest'
df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2021-06-30'] = df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2021-06-30'] 
df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2021-03-31'] = df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2021-03-31'] 
df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2020-09-30'] = df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2020-09-30'] 
df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2020-06-30'] = df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2020-06-30'] 
df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2020-03-31'] = df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2020-03-31'] 

# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_income_statement_10q = df_income_statement_10q.drop([
    'Revenue from Contract with Customer, Excluding Assessed Tax',
    'Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_income_statement_10q = df_income_statement_10q.drop([
    'us-gaap_IncomeStatementAbstract',
    'Loss Contingency, Loss in Period'
])

df_income_statement_10q

In [ ]:
# Fazendo o scraping do cash flow statement (10-Q)
cash_flow_statement_lst = []

for acc_num in df_lst_acc_num_10q_googl['acc_num']:

    # Cash flow statement
    cash_flow_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='cash_flow_statement')
    cash_flow_statement = f_sec.rename_statement(cash_flow_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    cash_flow_statement = cash_flow_statement.iloc[:, 0]
    cash_flow_statement_lst.append(cash_flow_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_cash_flow_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in cash_flow_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_cash_flow_10q = pd.concat([df_cash_flow_10q, df], axis=1)

# Substituindo os NaN por 0
df_cash_flow_10q = df_cash_flow_10q.fillna(0)

# A linha 'Depreciation' nas colunas '2023-09-30', '2023-06-30', '2023-03-31', '2022-09-30', '2022-06-30', '2022-03-31', '2021-09-30', '2021-06-30', '2021-03-31', '2020-09-30', '2020-06-30' e '2020-03-31' estão zeradas
# As informações estão na linha 'goog_DepreciationAndImpairmentOnDispositionOfPropertyAndEquipment'
df_cash_flow_10q.loc['Depreciation', '2023-09-30'] = df_cash_flow_10q.loc['goog_DepreciationAndImpairmentOnDispositionOfPropertyAndEquipment', '2023-09-30'] 
df_cash_flow_10q.loc['Depreciation', '2023-06-30'] = df_cash_flow_10q.loc['goog_DepreciationAndImpairmentOnDispositionOfPropertyAndEquipment', '2023-06-30'] 
df_cash_flow_10q.loc['Depreciation', '2023-03-31'] = df_cash_flow_10q.loc['goog_DepreciationAndImpairmentOnDispositionOfPropertyAndEquipment', '2023-03-31'] 
df_cash_flow_10q.loc['Depreciation', '2022-09-30'] = df_cash_flow_10q.loc['goog_DepreciationAndImpairmentOnDispositionOfPropertyAndEquipment', '2022-09-30'] 
df_cash_flow_10q.loc['Depreciation', '2022-06-30'] = df_cash_flow_10q.loc['goog_DepreciationAndImpairmentOnDispositionOfPropertyAndEquipment', '2022-06-30'] 
df_cash_flow_10q.loc['Depreciation', '2022-03-31'] = df_cash_flow_10q.loc['goog_DepreciationAndImpairmentOnDispositionOfPropertyAndEquipment', '2022-03-31'] 
df_cash_flow_10q.loc['Depreciation', '2021-09-30'] = df_cash_flow_10q.loc['goog_DepreciationAndImpairmentOnDispositionOfPropertyAndEquipment', '2021-09-30'] 
df_cash_flow_10q.loc['Depreciation', '2021-06-30'] = df_cash_flow_10q.loc['goog_DepreciationAndImpairmentOnDispositionOfPropertyAndEquipment', '2021-06-30'] 
df_cash_flow_10q.loc['Depreciation', '2021-03-31'] = df_cash_flow_10q.loc['goog_DepreciationAndImpairmentOnDispositionOfPropertyAndEquipment', '2021-03-31'] 
df_cash_flow_10q.loc['Depreciation', '2020-09-30'] = df_cash_flow_10q.loc['goog_DepreciationAndImpairmentOnDispositionOfPropertyAndEquipment', '2020-09-30'] 
df_cash_flow_10q.loc['Depreciation', '2020-06-30'] = df_cash_flow_10q.loc['goog_DepreciationAndImpairmentOnDispositionOfPropertyAndEquipment', '2020-06-30'] 
df_cash_flow_10q.loc['Depreciation', '2020-03-31'] = df_cash_flow_10q.loc['goog_DepreciationAndImpairmentOnDispositionOfPropertyAndEquipment', '2020-03-31'] 

# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_cash_flow_10q = df_cash_flow_10q.drop('goog_DepreciationAndImpairmentOnDispositionOfPropertyAndEquipment')

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_cash_flow_10q = df_cash_flow_10q.drop('us-gaap_NetCashProvidedByUsedInOperatingActivitiesAbstract')

# Renomeando a linha do índice que está sem formato 
df_cash_flow_10q = df_cash_flow_10q.rename(index={
    'goog_IncreaseDecreaseInAccruedRevenueShare': 'Increase (Decrease) in Accrued Revenue Share',
    'goog_AcquisitionsNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets': 'Acquisitions Net Of Cash Acquired And Purchases Of Intangible And Other Assets',
    'goog_NetProceedsPaymentsRelatedToStockBasedAwardActivities': 'Net Proceeds Payments Related To Stock Based Award Activities',
    'goog_AmortizationAndImpairmentOfIntangibleAssets': 'Amortization And Impairment Of Intangible Assets',
})

df_cash_flow_10q

In [ ]:
# Arrumando alguns index dos dfs para ficar padronizado com os index utilizados nas funções p/ calcular os indicadores fundamentalistas
df_balance_sheet_10q = df_balance_sheet_10q.rename(index={
    'Cash and Cash Equivalents, at Carrying Value': 'Cash and Cash Equivalents',
    'Marketable Securities, Current': 'Marketable Securities',
    'Other Long-term Investments': 'Non-marketable Securities',
    'Long-term Debt and Lease Obligation': 'Long-term Debt',
    'Operating Lease, Liability, Noncurrent': 'Operating Lease Liabilities',
    "Stockholders' Equity Attributable to Parent": 'Total Shareholders Equity'
})

df_income_statement_10q = df_income_statement_10q.rename(index={
    'Research and Development Expense': 'Research and Development',
    'Operating Income (Loss)': 'Operating Income',
    'Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest': 'Income Before Provision for Income Taxes',
    'Income Tax Expense (Benefit)': 'Provision for Income Taxes',
    'Net Income (Loss) Attributable to Parent': 'Net Income'
})

df_cash_flow_10q = df_cash_flow_10q.rename(index={
    'Payments to Acquire Property, Plant, and Equipment': 'Purchases Of Property And Equipment And Intangible Assets',
    'Depreciation': 'Depreciation, Depletion and Amortization',
})

In [ ]:
# Selecionando o 'acc_num' do trimestre mais recente
last_acc_num_10q = df_lst_acc_num_10q_googl.iloc[0][0]

# Fazendo o scraping do interest expense -> 'Other Income (Expense), Net'
interest_expense_10q = f_sec.process_one_statement(ticker=ticker, accession_number=last_acc_num_10q, statement_name='interest_expense')
interest_expense_10q = f_sec.rename_statement(interest_expense_10q, label_dictionary=labels_dict)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
interest_expense_10q = interest_expense_10q.drop('goog_BalanceSheetComponentsDisclosureAbstract')

# Renomeando a linha do índice que está sem formato 
interest_expense_10q = interest_expense_10q.rename(index={
    'InterestExpenseNonoperating': 'Interest Expense',
})

# Formatando o número para ficar positivo
interest_expense_10q.loc['Interest Expense'] = interest_expense_10q.loc['Interest Expense'] * -1

# Selecionando a linha desejada
interest_expense_10q = interest_expense_10q.loc[['Interest Expense']]

# Selecionando a 1º e 2º coluna do df que são os nº não acumulados
interest_expense_10q = interest_expense_10q.iloc[:, [0,1]]

interest_expense_10q

In [ ]:
# 'acc_num' de 2Q2025
acc_num_2Q2025 = '000165204425000062'

# Fazendo o scraping do restricted stock units -> 'Note 11 – Share-Based Compensation' -> 'Number of RSUs'
shares_10q_2Q2025 = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num_2Q2025, statement_name='rsu')
shares_10q_2Q2025 = f_sec.rename_statement(shares_10q_2Q2025, label_dictionary=labels_dict)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
shares_10q_2Q2025 = shares_10q_2Q2025.drop('us-gaap_NetIncomeLossAvailableToCommonStockholdersBasicAbstract')

# Resetando o index do df, porque ele possui vários index iguais
shares_10q_2Q2025 = shares_10q_2Q2025.reset_index(drop=True)

# Selecionando as linhas desejadas
shares_10q_2Q2025 = shares_10q_2Q2025.iloc[[11, 15, 18, 26]]

# Renomeando a linha do índice
shares_10q_2Q2025 = shares_10q_2Q2025.rename(index={
    18: 'RSUs',
    11: 'Class A',
    15: 'Class B',
    26: 'Class C',
})

# Selecionando a 1º e 2º coluna do df que são os nº não acumulados
shares_10q_2Q2025 = shares_10q_2Q2025.iloc[:, [0,1]]

shares_10q_2Q2025

In [ ]:
# 'acc_num' de 3Q2025
acc_num_3Q2025 = '000165204425000091'

# Fazendo o scraping do restricted stock units -> 'Note 11 – Share-Based Compensation' -> 'Number of RSUs'
shares_10q_3Q2025 = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num_3Q2025, statement_name='rsu')
shares_10q_3Q2025 = f_sec.rename_statement(shares_10q_3Q2025, label_dictionary=labels_dict)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
shares_10q_3Q2025 = shares_10q_3Q2025.drop('us-gaap_NetIncomeLossAvailableToCommonStockholdersBasicAbstract')

# Resetando o index do df, porque ele possui vários index iguais
shares_10q_3Q2025 = shares_10q_3Q2025.reset_index(drop=True)

# Selecionando as linhas desejadas
shares_10q_3Q2025 = shares_10q_3Q2025.iloc[[11, 15, 18, 26]]

# Renomeando a linha do índice
shares_10q_3Q2025 = shares_10q_3Q2025.rename(index={
    18: 'RSUs',
    11: 'Class A',
    15: 'Class B',
    26: 'Class C',
})

# Selecionando a 1º e 2º coluna do df que são os nº não acumulados
shares_10q_3Q2025 = shares_10q_3Q2025.iloc[:, [0,1]]

shares_10q_3Q2025

In [ ]:
# Dados dos nº de ações (Class A, B, C e RSUs) de 2025 e 2024
# 'acc_num' de 1Q2025
acc_num_1Q2025 = '000165204425000043'

# Fazendo o scraping do restricted stock units -> 'Note 11 – Share-Based Compensation' -> 'Number of RSUs'
shares_10q_1Q2025 = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num_1Q2025, statement_name='rsu')
shares_10q_1Q2025 = f_sec.rename_statement(shares_10q_1Q2025, label_dictionary=labels_dict)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
shares_10q_1Q2025 = shares_10q_1Q2025.drop('us-gaap_NetIncomeLossAvailableToCommonStockholdersBasicAbstract')

# Resetando o index do df, porque ele possui vários index iguais
shares_10q_1Q2025 = shares_10q_1Q2025.reset_index(drop=True)

# Selecionando as linhas desejadas
shares_10q_1Q2025 = shares_10q_1Q2025.iloc[[11, 15, 17, 26]]

# Renomeando a linha do índice
shares_10q_1Q2025 = shares_10q_1Q2025.rename(index={
    17: 'RSUs',
    11: 'Class A',
    15: 'Class B',
    26: 'Class C',
})

# Os dfs dos 'acc_num' de 2Q2025 e 3Q2025 tinham tamanhos iguais
lst_acc_2025 = [
    '000165204425000062',  # 2Q2023
    '000165204425000091',  # 3Q2025
]

shares_lst_2025 = []

for acc_num in lst_acc_2025:
    # Fazendo o scraping do restricted stock units -> 'Note 11 – Share-Based Compensation' -> 'Number of RSUs'
    shares_10q_2025 = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='rsu')
    shares_10q_2025 = f_sec.rename_statement(shares_10q_2025, label_dictionary=labels_dict)
    # Selecionando a 1º e 2º coluna do df que são os nº não acumulados
    shares_10q_2025 = shares_10q_2025.iloc[:, [0,1]]
    # Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
    shares_10q_2025 = shares_10q_2025.drop('us-gaap_NetIncomeLossAvailableToCommonStockholdersBasicAbstract')
    # Resetando o index do df, porque ele possui vários index iguais
    shares_10q_2025 = shares_10q_2025.reset_index(drop=True)
    shares_lst_2025.append(shares_10q_2025)

# Concatenando os dfs
df_shares_10q_2025 = pd.concat(shares_lst_2025, ignore_index=True, axis=1)

# Selecionando as linhas desejadas
df_shares_10q_2025 = df_shares_10q_2025.iloc[[11, 15, 18, 26]]

# Renomeando a linha do índice
df_shares_10q_2025 = df_shares_10q_2025.rename(index={
    18: 'RSUs',
    11: 'Class A',
    15: 'Class B',
    26: 'Class C',
})

df_shares_10q_2025 = df_shares_10q_2025.rename(columns={
    0: '2025-06-30',
    1: '2024-06-30',
    2: '2025-09-30',
    3: '2024-09-30',
})

# Concatenando os dfs de 1Q2023
df_shares_10q_2025 = pd.concat([df_shares_10q_2025, shares_10q_1Q2025], axis=1)

# Transformando as colunas em datetime
df_shares_10q_2025.columns = pd.to_datetime(df_shares_10q_2025.columns, format='%Y-%m-%d')

# Ordenando as colunas
df_shares_10q_2025 = df_shares_10q_2025[sorted(df_shares_10q_2025.columns, reverse=True)]

###################################################################################################################
###################################################################################################################

# Dados dos nº de ações (Class A, B, C e RSUs) de 2023 e 2022
# 'acc_num' de 1Q2023
acc_num_1Q2023 = '000165204423000045'

# Fazendo o scraping do restricted stock units -> 'Note 11 – Share-Based Compensation' -> 'Number of RSUs'
shares_10q_1Q2023 = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num_1Q2023, statement_name='rsu')
shares_10q_1Q2023 = f_sec.rename_statement(shares_10q_1Q2023, label_dictionary=labels_dict)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
shares_10q_1Q2023 = shares_10q_1Q2023.drop('us-gaap_WeightedAverageNumberOfSharesOutstandingBasicAbstract')

# Resetando o index do df, porque ele possui vários index iguais
shares_10q_1Q2023 = shares_10q_1Q2023.reset_index(drop=True)

# Selecionando as linhas desejadas
shares_10q_1Q2023 = shares_10q_1Q2023.iloc[[3, 7, 9, 13]]

# Renomeando a linha do índice
shares_10q_1Q2023 = shares_10q_1Q2023.rename(index={
    9: 'RSUs',
    3: 'Class A',
    7: 'Class B',
    13: 'Class C',
})


# Os dfs dos 'acc_num' de 2Q2023 e 3Q2023 tinham tamanhos iguais
lst_acc_2023 = [
    '000165204423000070',  # 2Q2023
    '000165204423000094',  # 3Q2023
]

shares_lst_2023 = []

for acc_num in lst_acc_2023:
    # Fazendo o scraping do restricted stock units -> 'Note 11 – Share-Based Compensation' -> 'Number of RSUs'
    shares_10q_2023 = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='rsu')
    shares_10q_2023 = f_sec.rename_statement(shares_10q_2023, label_dictionary=labels_dict)
    # Selecionando a 1º e 2º coluna do df que são os nº não acumulados
    shares_10q_2023 = shares_10q_2023.iloc[:, [0,1]]
    # Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
    shares_10q_2023 = shares_10q_2023.drop('us-gaap_WeightedAverageNumberOfSharesOutstandingBasicAbstract')
    # Resetando o index do df, porque ele possui vários index iguais
    shares_10q_2023 = shares_10q_2023.reset_index(drop=True)
    shares_lst_2023.append(shares_10q_2023)

# Concatenando os dfs
df_shares_10q_2023 = pd.concat(shares_lst_2023, ignore_index=True, axis=1)

# Selecionando as linhas desejadas
df_shares_10q_2023 = df_shares_10q_2023.iloc[[3, 7, 10, 15]]

# Renomeando a linha do índice e as colunas
df_shares_10q_2023 = df_shares_10q_2023.rename(index={
    10: 'RSUs',
    3: 'Class A',
    7: 'Class B',
    15: 'Class C',
})

df_shares_10q_2023 = df_shares_10q_2023.rename(columns={
    0: '2023-06-30',
    1: '2022-06-30',
    2: '2023-09-30',
    3: '2022-09-30',
})

# Concatenando os dfs de 1Q2023
df_shares_10q_2023 = pd.concat([df_shares_10q_2023, shares_10q_1Q2023], axis=1)

# Transformando as colunas em datetime
df_shares_10q_2023.columns = pd.to_datetime(df_shares_10q_2023.columns, format='%Y-%m-%d')

# Ordenando as colunas
df_shares_10q_2023 = df_shares_10q_2023[sorted(df_shares_10q_2023.columns, reverse=True)]

###################################################################################################################
###################################################################################################################

# Dados dos nº de ações (Class A, B, C e RSUs) de 2021 e 2020
# 'acc_num' de 1Q2021
acc_num_1Q2021 = '000165204421000020'

# Fazendo o scraping do restricted stock units -> 'Note 11 – Share-Based Compensation' -> 'Number of RSUs'
shares_10q_1Q2021 = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num_1Q2021, statement_name='rsu')
shares_10q_1Q2021 = f_sec.rename_statement(shares_10q_1Q2021, label_dictionary=labels_dict)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
shares_10q_1Q2021 = shares_10q_1Q2021.drop('us-gaap_WeightedAverageNumberOfSharesOutstandingBasicAbstract')

# Resetando o index do df, porque ele possui vários index iguais
shares_10q_1Q2021 = shares_10q_1Q2021.reset_index(drop=True)

# Selecionando as linhas desejadas
shares_10q_1Q2021 = shares_10q_1Q2021.iloc[[3, 7, 9, 14]]

# Renomeando a linha do índice
shares_10q_1Q2021 = shares_10q_1Q2021.rename(index={
    9: 'RSUs',
    3: 'Class A',
    7: 'Class B',
    14: 'Class C',
})


# Os dfs dos 'acc_num' de 2Q2021 e 3Q2021 tinham tamanhos iguais
lst_acc_2021 = [
    '000165204421000047',  # 2Q2021
    '000165204421000057',  # 3Q2021
]

shares_lst_2021 = []

for acc_num in lst_acc_2021:
    # Fazendo o scraping do restricted stock units -> 'Note 11 – Share-Based Compensation' -> 'Number of RSUs'
    shares_10q_2021 = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='rsu')
    shares_10q_2021 = f_sec.rename_statement(shares_10q_2021, label_dictionary=labels_dict)
    # Selecionando a 1º e 2º coluna do df que são os nº não acumulados
    shares_10q_2021 = shares_10q_2021.iloc[:, [0,1]]
    # Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
    shares_10q_2021 = shares_10q_2021.drop('us-gaap_WeightedAverageNumberOfSharesOutstandingBasicAbstract')
    # Resetando o index do df, porque ele possui vários index iguais
    shares_10q_2021 = shares_10q_2021.reset_index(drop=True)
    shares_lst_2021.append(shares_10q_2021)

# Concatenando os dfs
df_shares_10q_2021 = pd.concat(shares_lst_2021, ignore_index=True, axis=1)

# Selecionando as linhas desejadas
df_shares_10q_2021 = df_shares_10q_2021.iloc[[3, 7, 10, 17]]

# Renomeando a linha do índice e as colunas
df_shares_10q_2021 = df_shares_10q_2021.rename(index={
    10: 'RSUs',
    3: 'Class A',
    7: 'Class B',
    17: 'Class C',
})

df_shares_10q_2021 = df_shares_10q_2021.rename(columns={
    0: '2021-06-30',
    1: '2020-06-30',
    2: '2021-09-30',
    3: '2020-09-30',
})

# Concatenando os dfs de 1Q2021
df_shares_10q_2021 = pd.concat([df_shares_10q_2021, shares_10q_1Q2021], axis=1)

# Transformando as colunas em datetime
df_shares_10q_2021.columns = pd.to_datetime(df_shares_10q_2021.columns, format='%Y-%m-%d')

# Ordenando as colunas
df_shares_10q_2021 = df_shares_10q_2021[sorted(df_shares_10q_2021.columns, reverse=True)]

###################################################################################################################
####################################################################################################################

# Concatenando todos os dfs
df_shares_10q = pd.concat([
    df_shares_10q_2021,
    df_shares_10q_2023,
    df_shares_10q_2025
], axis=1)

# Ordenando as colunas
df_shares_10q = df_shares_10q[sorted(df_shares_10q.columns, reverse=True)]

# Arrumando e adicionando alguns números no df
df_shares_10q.loc['Class A', '2022-03-31'] = 300478 		
df_shares_10q.loc['Class B', '2022-03-31'] = 44535 
df_shares_10q.loc['Class C', '2022-03-31'] = 315158 
df_shares_10q.loc['RSUs', '2022-03-31'] = 5

df_shares_10q

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//tecnologia//google//googl_10q.xlsx') as writer:
    df_balance_sheet_10q.to_excel(writer, sheet_name='balance_sheet')
    df_income_statement_10q.to_excel(writer, sheet_name='income_statement')
    df_cash_flow_10q.to_excel(writer, sheet_name='cash_flow')
    interest_expense_10q.to_excel(writer, sheet_name='interest_expense')
    df_shares_10q.to_excel(writer, sheet_name='num_shares')

### Indicadores Fundamentalistas

#### Indicadores 10-K

In [ ]:
# Lendo os arquivos 10k
googl_bs = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//google//googl_10k.xlsx', 
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

googl_is = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//google//googl_10k.xlsx', 
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

googl_cf = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//google//googl_10k.xlsx', 
    sheet_name='cash_flow', 
    index_col='Unnamed: 0'
)

googl_leases = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//google//googl_10k.xlsx', 
    sheet_name='leases', 
    index_col='Unnamed: 0'
)

googl_current_portion_lease = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//google//googl_10k.xlsx', 
    sheet_name='current_portion_lease', 
    index_col='Unnamed: 0'
)

googl_shares = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//google//googl_10k.xlsx', 
    sheet_name='num_shares',
    index_col='Unnamed: 0'
)

# Transformando as colunas em datetime
googl_bs.columns = pd.to_datetime(googl_bs.columns)
googl_is.columns = pd.to_datetime(googl_is.columns)
googl_cf.columns = pd.to_datetime(googl_cf.columns)
googl_leases.columns = pd.to_datetime(googl_leases.columns)
googl_current_portion_lease.columns = pd.to_datetime(googl_current_portion_lease.columns)
googl_shares.columns = pd.to_datetime(googl_shares.columns)

# Transformando as colunas em apenas o ano
googl_bs.columns = googl_bs.columns.year
googl_is.columns = googl_is.columns.year
googl_cf.columns = googl_cf.columns.year
googl_leases.columns = googl_leases.columns.year
googl_current_portion_lease.columns = googl_current_portion_lease.columns.year
googl_shares.columns = googl_shares.columns.year

# Removendo a linha 'RSUs'
googl_shares = googl_shares.drop(index='RSUs')
# Fazendo a soma entre o nº de ações das classes A, B e C
googl_shares_total = googl_shares.sum()
# Transformando em um df
df_googl_shares_total = pd.DataFrame(googl_shares_total).T
# Transformando em float
df_googl_shares_total = df_googl_shares_total.astype(float)
# Arrumando a escala das colunas
df_googl_shares_total.loc[:, 2021] = df_googl_shares_total.loc[:, 2021].values[0] / 1000
df_googl_shares_total.loc[:, 2020] = df_googl_shares_total.loc[:, 2020].values[0] / 1000
df_googl_shares_total.loc[:, 2019] = df_googl_shares_total.loc[:, 2019].values[0] / 1000
df_googl_shares_total.loc[:, 2018] = df_googl_shares_total.loc[:, 2018].values[0] / 1000
df_googl_shares_total.loc[:, 2017] = df_googl_shares_total.loc[:, 2017].values[0] / 1000
# Renomeando o index
df_googl_shares_total = df_googl_shares_total.rename({0: 'Number of Shares - Basic'})
# Arrendondamento das casas decimais
df_googl_shares_total = df_googl_shares_total.round(2)
# Concatenando os dados de nº de ações no income statement
googl_is = pd.concat([googl_is, df_googl_shares_total])

In [ ]:
# Preços da GOOGL
googl_close = yf.download('GOOGL', auto_adjust=True, multi_level_index=False, start='2017-01-01')['Close']

# Selecionando o último preço do mês de Dezembro de cada ano 
lst_years = [
    '2017', '2018', '2019', '2020', 
    '2021', '2022', '2023', '2024',
    '2025'
]

last_dec_close = []

for year in lst_years:
    last_close = googl_close[f'{year}-12'].iloc[-1]
    last_dec_close.append(last_close)

# Transformando em uma série pandas
googl_close_dec = pd.Series(last_dec_close, index=lst_years)

# Revertendo o index da série p/ ficar igual com os dos indicadores
googl_close_dec = googl_close_dec.reindex(index=googl_close_dec.index[::-1])

# Arredondamento de duas casas decimais
googl_close_dec = round(googl_close_dec, 2)

# Teve um split de 20:1 em 18/07/2022 (https://br.investing.com/equities/google-inc-historical-data-splits)
googl_close_dec.loc['2021'] = googl_close_dec.loc['2021'] * 20
googl_close_dec.loc['2020'] = googl_close_dec.loc['2020'] * 20
googl_close_dec.loc['2019'] = googl_close_dec.loc['2019'] * 20
googl_close_dec.loc['2018'] = googl_close_dec.loc['2018'] * 20
googl_close_dec.loc['2017'] = googl_close_dec.loc['2017'] * 20

# Transformando o index em int
googl_close_dec .index = googl_close_dec .index.astype(int)

googl_close_dec

In [ ]:
# Criando uma lista só para fazer a indexação em que o ano mais recente está no começo
idx = googl_bs.columns

# Valor de Mercado
vm = f_eua.indicador_vm(ten_k=True, price=googl_close_dec, df_is=googl_is) 

# LPA
lpa = f_eua.indicador_lpa(ten_k=True, df_is=googl_is, idx=None)

# P/L Damodaran
p_lpa_damodaran = f_eua.indicador_p_lpa_2(ten_k=True, vm=vm, df_is=googl_is)

# P/L
p_lpa = f_eua.indicador_p_lpa(ten_k=True, price=googl_close_dec, lpa=lpa, idx=idx)

# L/P
lpa_p = f_eua.indicador_lpa_p(ten_k=True, price=googl_close_dec, lpa=lpa, idx=idx) 

# Depreciação
depreciacao = f_eua.indicador_depreciacao(ten_k=True, df_cf=googl_cf, first_quarter=None, idx=idx)

# EBITDA
ebitda = f_eua.indicador_ebitda(ten_k=True, df_is=googl_is, idx=None, depreciacao=depreciacao)

# Margem Líquida
m_liq = f_eua.indicador_ml(ten_k=True, df_is=googl_is)

# VPA
vpa = f_eua.indicador_vpa(ten_k=True, df_bs=googl_bs, df_is=googl_is)

# P/VPA
p_vpa = f_eua.indicador_p_vpa(ten_k=True, price=googl_close_dec, vpa=vpa, idx=idx)

# Caixa e Equivalentes
lst_caixa = ['Cash and Cash Equivalents', 'Marketable Securities']
caixa = f_eua.indicador_caixa(ten_k=True, df_bs=googl_bs, lst_itens_caixa=lst_caixa)

# Dívida Bruta
lst_divida = ['Long-term Debt', 'Operating Lease Liabilities']
div_bruta = f_eua.indicador_divida_bruta(ten_k=True, df_bs=googl_bs, lst_itens_divida=lst_divida)
div_bruta_total = div_bruta.values + googl_leases.values + googl_current_portion_lease.values
div_bruta_total = pd.Series(div_bruta_total[0], index=idx)

# Dívida Líquida
div_liq = f_eua.indicador_divida_liquida(ten_k=True, caixa=caixa, div_bruta=div_bruta_total, idx=idx)

# Dívida Líquida / EBITDA
dl_ebitda = f_eua.indicador_dl_ebitda(ten_k=True, ebitda=ebitda, div_liq=div_liq, idx=None)

# Dívida Líquida / PL
dl_pl = f_eua.indicador_dl_pl(ten_k=True, div_liq=div_liq, df_bs=googl_bs) 

# EV/EBITDA
ev_ebitda = f_eua.indicador_ev_ebitda(vm=vm, div_liq=div_liq, ebitda=ebitda)

# ROE
roe = f_eua.indicador_roe(ten_k=True, df_is=googl_is, df_bs=googl_bs, idx=None)

# ROIC
roic = f_eua.indicador_roic(ten_k=True, df_is=googl_is, df_bs=googl_bs, div_bruta=div_bruta_total, idx=idx)

# FCO
fco = f_eua.indicador_fco(ten_k=True, df_cf=googl_cf, first_quarter=None) 

# FCI
fci = f_eua.indicador_fci(ten_k=True, df_cf=googl_cf, first_quarter=None) 

# FCF
fcf = f_eua.indicador_fcf(ten_k=True, df_cf=googl_cf, first_quarter=None) 

# CAPEX
lst_capex = ['Purchases Of Property And Equipment And Intangible Assets']

capex= f_eua.indicador_capex(ten_k=True, df_cf=googl_cf, first_quarter=None, lst_itens=lst_capex, idx=idx)

# Free Cash Flow
free_cash_flow = f_eua.indicador_free_cash_flow(fco=fco, capex=capex)

# Net CAPEX
net_capex = f_eua.indicador_net_capex(capex=capex, depreciation=depreciacao)

# R&D
rd_expense = f_eua.indicador_rd(ten_k=True, df_is=googl_is, idx=idx)

# Net CAPEX ajustado
adj_net_capex = f_eua.indicador_adj_net_capex(net_capex=net_capex, rd_expense=rd_expense)

# Working capital
working_capital = f_eua.indicador_working_capital(ten_k=True, df_bs=googl_bs)

# Change in non cash working capital
lst_itens_change_non_cash_wc = [
    'Deferred Income Taxes and Tax Credits',
    'Increase (Decrease) in Accounts Receivable',
    'Increase (Decrease) in Income Taxes',
    'Increase (Decrease) in Other Operating Assets',
    'Increase (Decrease) in Accounts Payable',
    'Increase (Decrease) in Accrued Liabilities',
    'Increase (Decrease) in Accrued Revenue Share',
    'Increase (Decrease) in Contract with Customer, Liability'
]

change_non_cash_wc = f_eua.indicador_change_non_cash_wc(ten_k=True, df_cf=googl_cf, lst_itens=lst_itens_change_non_cash_wc, first_quarter=None)

# Reinvestment rate
reinvestment_rate = f_eua.indicador_reinvestment_rate(ten_k=True, net_capex=net_capex, change_non_cash_wc=change_non_cash_wc, df_is=googl_is, idx=idx)

# New borrowing
lst_itens_new_borrowing = ['Proceeds from Debt, Net of Issuance Costs']

new_borrowing = f_eua.indicador_new_borrowing(ten_k=True, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=googl_cf, first_quarter=None, idx=idx)

# Debt paid
lst_itens_debt_paid = ['Repayments of Debt and Lease Obligation']

debt_paid = f_eua.indicador_debt_paid(ten_k=True, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=googl_cf, first_quarter=None, idx=idx)

# Taxes on operating income
taxes_operating_income = f_eua.indicador_taxes_operating_income(ten_k=True, df_is=googl_is, idx=idx)

# FCFE
fcfe_10k = f_eua.indicador_fcfe(
    ten_k=True, 
    depreciacao=depreciacao, 
    capex=capex, 
    change_non_cash_wc=change_non_cash_wc, 
    new_borrowing=new_borrowing,
    debt_paid=debt_paid,
    df_is=googl_is,
    idx=idx
)

# FCFF
fcff_10k = f_eua.indicador_fcff(
        ten_k=True, 
        taxes_operating_income=taxes_operating_income,
        depreciacao=depreciacao, 
        capex=capex, 
        change_non_cash_wc=change_non_cash_wc, 
        df_is=googl_is,
        idx=idx
)

# Dividendo por Ação
dpa = f_eua.indicador_dpa(ten_k=True, df_cf=googl_cf, df_is=googl_is, div_nao_acum=None)

# Payout
payout = f_eua.indicador_payout(ten_k=True, df_cf=googl_cf, df_is=googl_is, div_nao_acum=None)

# Buyback
buyback = f_eua.indicador_buyback(ten_k=True, df_cf=googl_cf, first_quarter=None)

# Colocando todos os indicadores em uma lista
lst_indicadores = [
    p_lpa_damodaran,
    p_lpa,
    lpa_p,  
    ebitda, 
    m_liq,
    p_vpa,
    vm,
    div_bruta_total,
    caixa,
    div_liq,
    dl_ebitda,
    dl_pl,
    ev_ebitda,
    roe,
    roic,
    fco,
    fci,
    fcf,
    free_cash_flow,
    capex,
    net_capex,
    rd_expense,
    adj_net_capex,
    working_capital,
    reinvestment_rate,
    fcfe_10k,
    fcff_10k,
    dpa,
    payout,
    buyback
]

# Juntando todos os indicadores em um df
googl_indicadores_10k = pd.concat(lst_indicadores, axis=1)

# Nomes das colunas
nomes_col = [
    'P/L Damodaran', 
    'P/L', 
    'L/P', 
    'EBITDA', 
    'Margem Líquida',
    'P/VPA',
    'Valor de Mercado',
    'Dívida Bruta',
    'Caixa e Equivalentes',
    'Dívida Líquida',
    'Dívida Líquida/EBITDA',
    'Dívida Líquida/PL',
    'EV/EBITDA',
    'ROE',
    'ROIC',
    'FCO',
    'FCI',
    'FCF',
    'Free Cash Flow',
    'Capex',
    'Net Capex',
    'R&D',
    'Adj Net Capex',
    'Working Capital',
    'Reinvestment Rate',
    'FCFE',
    'FCFF',
    'DPA',
    'Payout',
    'Buyback'
]

# Atribuindo os novos nomes das colunas do df
googl_indicadores_10k.columns = nomes_col
googl_indicadores_10k

#### Indicadores 10-Q

In [ ]:
# Lendo os arquivos 10q
googl_bs_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//google//googl_10q.xlsx',
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

googl_is_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//google//googl_10q.xlsx',
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

googl_cf_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//google//googl_10q.xlsx',
    sheet_name='cash_flow',  
    index_col='Unnamed: 0'
)

googl_shares_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//google//googl_10q.xlsx',
    sheet_name='num_shares',  
    index_col='Unnamed: 0'
)

# Aplicando o ajuste nas colunas
googl_bs_10q.columns = [f_sec.ajustar_mes_03_06_09(col) for col in googl_bs_10q.columns]
googl_is_10q.columns = [f_sec.ajustar_mes_03_06_09(col) for col in googl_is_10q.columns]
googl_cf_10q.columns = [f_sec.ajustar_mes_03_06_09(col) for col in googl_cf_10q.columns]
googl_shares_10q.columns = [f_sec.ajustar_mes_03_06_09(col) for col in googl_shares_10q.columns]

# Removendo a linha 'RSUs'
googl_shares_10q = googl_shares_10q.drop(index='RSUs')
# Fazendo a soma entre o nº de ações das classes A, B e C
googl_shares_total_10q = googl_shares_10q.sum()
# Transformando em um df
df_googl_shares_total_10q = pd.DataFrame(googl_shares_total_10q).T
# Transformando em float
df_googl_shares_total_10q = df_googl_shares_total_10q.astype(float)
# Arrumando a escala das colunas
df_googl_shares_total_10q.loc[:, '2022-06-30'] = df_googl_shares_total_10q.loc[:, '2022-06-30'].values[0] / 10    # Eu dividi por 10 para ficar na mesma escala que os outros trimestres
df_googl_shares_total_10q.loc[:, '2022-03-31'] = df_googl_shares_total_10q.loc[:, '2022-03-31'].values[0] / 1000
df_googl_shares_total_10q.loc[:, '2021-09-30'] = df_googl_shares_total_10q.loc[:, '2021-09-30'].values[0] / 1000
df_googl_shares_total_10q.loc[:, '2021-06-30'] = df_googl_shares_total_10q.loc[:, '2021-06-30'].values[0] / 1000
df_googl_shares_total_10q.loc[:, '2021-03-31'] = df_googl_shares_total_10q.loc[:, '2021-03-31'].values[0] / 1000
df_googl_shares_total_10q.loc[:, '2020-09-30'] = df_googl_shares_total_10q.loc[:, '2020-09-30'].values[0] / 1000
df_googl_shares_total_10q.loc[:, '2020-06-30'] = df_googl_shares_total_10q.loc[:, '2020-06-30'].values[0] / 1000
df_googl_shares_total_10q.loc[:, '2020-03-31'] = df_googl_shares_total_10q.loc[:, '2020-03-31'].values[0] / 1000
# Renomeando o index
df_googl_shares_total_10q = df_googl_shares_total_10q.rename({0: 'Number of Shares - Basic'})
# Arrendondamento das casas decimais
df_googl_shares_total_10q = df_googl_shares_total_10q.round(2)
# Concatenando os dados de nº de ações no income statement
googl_is_10q = pd.concat([googl_is_10q, df_googl_shares_total_10q])

In [ ]:
# Preços da GOOGL
googl_close = yf.download('GOOGL', auto_adjust=True, multi_level_index=False, start='2021-01-01')['Close']

# Selecionando o último preço dos meses de Março, Junho e Setembro
lst_years = [
    '2021', '2022', '2023', '2024',
    '2025',
]
lst_months = ['03', '06', '09']
last_close_months = []

for year in lst_years:
    for month in lst_months:
        last_close = googl_close[f'{year}-{month}'].dropna().iloc[-1]  # Dropna para lidar com meses sem dados
        last_close_months.append(last_close)

# Criando uma lista em que junte os meses e os anos para que seja o index da serie pandas
lst_idx = []

for year in lst_years:
    for month in lst_months:
        lst_idx.append(f"{month}-{year}")

# Transformando em uma série pandas
googl_close_months = pd.Series(last_close_months, index=lst_idx)

# Revertendo o index da série p/ ficar igual com os dos indicadores
googl_close_months = googl_close_months.reindex(index=googl_close_months.index[::-1])

# No ano mais atual, como ainda ele não acabou, não consigo fazer o loop com todos meses (Março, Junho e Setembro)
actual_year_int = datetime.now().year  
actual_year_str = str(actual_year_int) 

lst_actual_months = [
    # '03',
    # '06', 
    # '09'
] 

last_close_actual_months = []

for month in lst_actual_months:
    last_close = googl_close[f'{actual_year_str}-{month}'].dropna().iloc[-1]  # Dropna para lidar com meses sem dados
    last_close_actual_months.append(last_close)

# Criando uma lista em que junte os meses e os anos para que seja o index da serie pandas
lst_idx_actual = []

for month in lst_actual_months:
    lst_idx_actual.append(f"{month}-{actual_year_str}")

# Transformando em uma série pandas
googl_close_months_atual = pd.Series(last_close_actual_months, index=lst_idx_actual)

# Revertendo o index da série p/ ficar igual com os dos indicadores
googl_close_months_atual = googl_close_months_atual.reindex(index=googl_close_months_atual.index[::-1])

# Juntando as duas series pandas
googl_close_months = pd.concat([googl_close_months_atual, googl_close_months])

# Arredondamento de duas casas decimais
googl_close_months = round(googl_close_months, 2)

# Teve um split de 20:1 em 18/07/2022 (https://br.investing.com/equities/google-inc-historical-data-splits)
googl_close_months.loc['06-2022'] = googl_close_months.loc['06-2022'] * 20
googl_close_months.loc['03-2022'] = googl_close_months.loc['03-2022'] * 20
googl_close_months.loc['09-2021'] = googl_close_months.loc['09-2021'] * 20
googl_close_months.loc['06-2021'] = googl_close_months.loc['06-2021'] * 20
googl_close_months.loc['03-2021'] = googl_close_months.loc['03-2021'] * 20

# Substituindo o index para ficar igual ao dos dados fundamentalistas
googl_close_months.index = googl_bs_10q.columns[:-3]

googl_close_months

In [ ]:
# Criando uma lista só para fazer a indexação em que o ano mais recente está no começo
idx_10q = googl_bs_10q.columns

# Valor de Mercado
vm_10q = f_eua.indicador_vm(ten_k=False, price=googl_close_months, df_is=googl_is_10q)

# LPA
lpa_acum = f_eua.indicador_lpa(ten_k=False, df_is=googl_is_10q, idx=idx_10q)[1]
lpa_10q = f_eua.indicador_lpa(ten_k=False, df_is=googl_is_10q, idx=idx_10q)[0]

# P/L Damodaran
p_lpa_10q_damodaran = f_eua.indicador_p_lpa_2(ten_k=False, vm=vm_10q, df_is=googl_is_10q)

# P/L
p_lpa_10q = f_eua.indicador_p_lpa(ten_k=False, price=googl_close_months, lpa=lpa_acum, idx=idx_10q)

# L/P
lpa_p_10q = f_eua.indicador_lpa_p(ten_k=False, price=googl_close_months, lpa=lpa_acum, idx=idx_10q) 

# Depreciação
depreciacao_10q = f_eua.indicador_depreciacao(ten_k=False, df_cf=googl_cf_10q, first_quarter='03', idx=idx_10q)[0]
depreciacao_10q_sliced = f_eua.indicador_depreciacao(ten_k=False, df_cf=googl_cf_10q, first_quarter='03', idx=idx_10q)[1]
depreciacao_10q_acum = f_eua.indicador_depreciacao(ten_k=False, df_cf=googl_cf_10q, first_quarter='03', idx=idx_10q)[2]

# EBITDA
ebitda_10q = f_eua.indicador_ebitda(ten_k=False, df_is=googl_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[0]
ebitda_10q_sliced = f_eua.indicador_ebitda(ten_k=False, df_is=googl_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[1]
ebitda_10q_acum = f_eua.indicador_ebitda(ten_k=False, df_is=googl_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[2]

# Margem Líquida
m_liq_10q = f_eua.indicador_ml(ten_k=False, df_is=googl_is_10q)

# VPA
vpa_10q = f_eua.indicador_vpa(ten_k=False, df_bs=googl_bs_10q, df_is=googl_is_10q)

# P/VPA
p_vpa_10q = f_eua.indicador_p_vpa(ten_k=False, price=googl_close_months, vpa=vpa_10q, idx=idx_10q)

# Caixa e Equivalentes
lst_caixa = ['Cash and Cash Equivalents', 'Marketable Securities']
caixa_10q = f_eua.indicador_caixa(ten_k=False, df_bs=googl_bs_10q, lst_itens_caixa=lst_caixa)

# Dívida Bruta - não somei o custo dos 'leases' e 'current portion lease', porque eles não são divulgados no 10q
lst_divida = ['Long-term Debt', 'Operating Lease Liabilities']
div_bruta_10q = f_eua.indicador_divida_bruta(ten_k=False, df_bs=googl_bs_10q, lst_itens_divida=lst_divida)

# Dívida Líquida
div_liq_10q = f_eua.indicador_divida_liquida(ten_k=False, caixa=caixa_10q, div_bruta=div_bruta_10q, idx=idx_10q)

# Dívida Líquida / EBITDA
dl_ebitda_10q =  f_eua.indicador_dl_ebitda(ten_k=False, ebitda=ebitda_10q_acum, div_liq=div_liq_10q, idx=idx_10q)

# Dívida Líquida / PL
dl_pl_10q = f_eua.indicador_dl_pl(ten_k=False, div_liq=div_liq_10q, df_bs=googl_bs_10q) 

# EV/EBITDA
ev_ebitda_10q = f_eua.indicador_ev_ebitda(vm=vm_10q, div_liq=div_liq_10q, ebitda=ebitda_10q_acum)

# ROE
roe_10q = f_eua.indicador_roe(ten_k=False, df_is=googl_is_10q, df_bs=googl_bs_10q, idx=idx_10q)

# ROIC 
roic_10q = f_eua.indicador_roic(ten_k=False, df_is=googl_is_10q, df_bs=googl_bs_10q, div_bruta=div_bruta_10q, idx=idx_10q)

# FCO
fco_10q = f_eua.indicador_fco(ten_k=False, df_cf=googl_cf_10q, first_quarter='03') 

# FCI
fci_10q = f_eua.indicador_fci(ten_k=False, df_cf=googl_cf_10q, first_quarter='03') 

# FCF
fcf_10q = f_eua.indicador_fcf(ten_k=False, df_cf=googl_cf_10q, first_quarter='03') 

# CAPEX
lst_capex_10q = ['Purchases Of Property And Equipment And Intangible Assets']

capex_10q = f_eua.indicador_capex(ten_k=False, df_cf=googl_cf_10q, first_quarter='03', lst_itens=lst_capex_10q, idx=idx_10q)[0]
capex_10q_sliced = f_eua.indicador_capex(ten_k=False, df_cf=googl_cf_10q, first_quarter='03', lst_itens=lst_capex_10q, idx=idx_10q)[1]
capex_10q_acum = f_eua.indicador_capex(ten_k=False, df_cf=googl_cf_10q, first_quarter='03', lst_itens=lst_capex_10q, idx=idx_10q)[2]

# Free Cash Flow
free_cash_flow_10q = f_eua.indicador_free_cash_flow(fco=fco_10q, capex=capex_10q_sliced)

# Net CAPEX
net_capex_10q = f_eua.indicador_net_capex(capex=capex_10q_acum, depreciation=depreciacao_10q_acum)

# R&D
rd_expense_10q = f_eua.indicador_rd(ten_k=False, df_is=googl_is_10q, idx=idx_10q)[0]
rd_expense_10q_sliced = f_eua.indicador_rd(ten_k=False, df_is=googl_is_10q, idx=idx_10q)[1]
rd_expense_10q_acum = f_eua.indicador_rd(ten_k=False, df_is=googl_is_10q, idx=idx_10q)[2]

# Net CAPEX ajustado
adj_net_capex_10q = f_eua.indicador_adj_net_capex(net_capex=net_capex_10q, rd_expense=rd_expense_10q_acum)

# Working capital
working_capital_10q = f_eua.indicador_working_capital(ten_k=False, df_bs=googl_bs_10q)

# Change in non cash working capital
lst_itens_change_non_cash_wc = [
    'Deferred Income Taxes and Tax Credits',
    'Increase (Decrease) in Accounts Receivable',
    'Increase (Decrease) in Income Taxes',
    'Increase (Decrease) in Other Operating Assets',
    'Increase (Decrease) in Accounts Payable',
    'Increase (Decrease) in Accrued Liabilities',
    'Increase (Decrease) in Accrued Revenue Share',
    'Increase (Decrease) in Contract with Customer, Liability'
]

change_non_cash_wc_10q = f_eua.indicador_change_non_cash_wc(ten_k=False, df_cf=googl_cf_10q, lst_itens=lst_itens_change_non_cash_wc, first_quarter='03')[0]
change_non_cash_wc_10q_sliced = f_eua.indicador_change_non_cash_wc(ten_k=False, df_cf=googl_cf_10q, lst_itens=lst_itens_change_non_cash_wc, first_quarter='03')[1]

# Reinvestment rate
reinvestment_rate_10q = f_eua.indicador_reinvestment_rate(ten_k=False, net_capex=adj_net_capex_10q, change_non_cash_wc=change_non_cash_wc_10q, df_is=googl_is_10q, idx=idx_10q)

# New borrowing
lst_itens_new_borrowing = ['Proceeds from Debt, Net of Issuance Costs']

new_borrowing_10q = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=googl_cf_10q, first_quarter='03', idx=idx_10q)[0]
new_borrowing_10q_sliced = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=googl_cf_10q, first_quarter='03', idx=idx_10q)[1]
new_borrowing_10q_acum = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=googl_cf_10q, first_quarter='03', idx=idx_10q)[2]

# Debt paid
lst_itens_debt_paid = ['Repayments of Debt and Lease Obligation']

debt_paid_10q = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=googl_cf_10q, first_quarter='03', idx=idx_10q)[0]
debt_paid_10q_sliced = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=googl_cf_10q, first_quarter='03', idx=idx_10q)[1]
debt_paid_10q_acum = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=googl_cf_10q, first_quarter='03', idx=idx_10q)[2]

# Taxes on operating income
taxes_operating_income_10q = f_eua.indicador_taxes_operating_income(ten_k=False, df_is=googl_is_10q, idx=idx_10q)

# FCFE
fcfe_10q = f_eua.indicador_fcfe(
    ten_k=False, 
    depreciacao=depreciacao_10q_acum, 
    capex=capex_10q_acum, 
    change_non_cash_wc=change_non_cash_wc_10q_sliced, 
    new_borrowing=new_borrowing_10q_acum, 
    debt_paid=debt_paid_10q_acum, 
    df_is=googl_is_10q,
    idx=idx_10q
)

# FCFF
fcff_10q = f_eua.indicador_fcff(
        ten_k=False, 
        taxes_operating_income=taxes_operating_income_10q,
        depreciacao=depreciacao_10q_acum, 
        capex=capex_10q_acum, 
        change_non_cash_wc=change_non_cash_wc_10q_sliced, 
        df_is=googl_is_10q,
        idx=idx_10q
)

# Dividendo não acumulado
div_nao_acum = f_eua.indicador_div_nao_acum(df_cf=googl_cf_10q, first_quarter='03')

# Dividendo por Ação
dpa_10q = f_eua.indicador_dpa(ten_k=False, df_cf=googl_cf_10q, df_is=googl_is_10q, div_nao_acum=div_nao_acum)

# Payout
payout_10q = f_eua.indicador_payout(ten_k=False, df_cf=googl_cf_10q, df_is=googl_is_10q, div_nao_acum=div_nao_acum)

# Buyback
buyback_10q = f_eua.indicador_buyback(ten_k=False, df_cf=googl_cf_10q,  first_quarter='03')

# Lista com todos os indicadores
lst_indicadores_10q = [
    p_lpa_10q_damodaran,
    p_lpa_10q,
    lpa_p_10q,  
    ebitda_10q_sliced, 
    m_liq_10q,
    p_vpa_10q,
    vm_10q,
    div_bruta_10q,
    caixa_10q,
    div_liq_10q,
    dl_ebitda_10q,
    dl_pl_10q,
    ev_ebitda_10q,
    roe_10q,
    roic_10q,
    fco_10q,
    fci_10q,
    fcf_10q,
    free_cash_flow_10q,
    capex_10q_sliced,
    net_capex_10q,
    rd_expense_10q_sliced,
    adj_net_capex_10q,
    working_capital_10q,
    reinvestment_rate_10q,
    fcfe_10q,
    fcff_10q,
    dpa_10q,
    payout_10q,
    buyback_10q
]

# Juntando todos os indicadores em um df
googl_indicadores_10q = pd.concat(lst_indicadores_10q, axis=1)

# Nomes das colunas
nomes_col = [
    'P/L Damodaran', 
    'P/L', 
    'L/P', 
    'EBITDA', 
    'Margem Líquida',
    'P/VPA',
    'Valor de Mercado',
    'Dívida Bruta',
    'Caixa e Equivalentes',
    'Dívida Líquida',
    'Dívida Líquida/EBITDA',
    'Dívida Líquida/PL',
    'EV/EBITDA',
    'ROE',
    'ROIC',
    'FCO',
    'FCI',
    'FCF',
    'Free Cash Flow',
    'Capex',
    'Net Capex',
    'R&D',
    'Adj Net Capex',
    'Working Capital',
    'Reinvestment Rate',
    'FCFE',
    'FCFF',
    'DPA',
    'Payout',
    'Buyback'
]

# Atribuindo os novos nomes das colunas do df
googl_indicadores_10q.columns = nomes_col

# Preenchendo os NaN por zero
googl_indicadores_10q = googl_indicadores_10q.fillna(0)

googl_indicadores_10q

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//tecnologia//google//googl_indicators.xlsx') as writer:
    googl_indicadores_10k.to_excel(writer, sheet_name='indicators_10k')
    googl_indicadores_10q.to_excel(writer, sheet_name='indicators_10q')

## Meta

* 1Q -> Lançamento em Março (03);
* 2Q -> Lançamento em Junho (06);
* 3Q -> Lançamento em Setembro (09);
* 10K -> Lançamento em Dezembro (12).

In [ ]:
ticker = 'meta'
f_sec.cik_matching_ticker(ticker)

### Scraping Meta 10-K

In [ ]:
# Número dos documentos que contém a DRE, BP e FC 
acc_documents = f_sec.get_filtered_filings(ticker=ticker, ten_k=True, just_accession_numbers=True)

# Trocando os '-' para '' das strings ('0001045810-24-000264' -> '000104581024000264')
dict_acc_num = {k: v.replace('-', '') for k, v in acc_documents.items()}

# Transformando o dicionário em um df
df_lst_acc_num_meta = pd.DataFrame.from_dict(dict_acc_num, orient='index', columns=['acc_num'])

# Adicionando os 'acc_num' de 2017, 2018, 2019, 2020, 2021, 2022 e 2023
df_acc_num = pd.DataFrame(
    {
        'acc_num': [
            '000132680124000012', # 2023
            '000132680123000013', # 2022
            '000132680122000018', # 2021
            '000132680121000014', # 2020
            '000132680120000013', # 2019
            '000132680119000009', # 2018
            '000132680118000009', # 2017
        ]
    },
    index=pd.to_datetime([
        '2023-12-31', 
        '2022-12-31', 
        '2021-12-31', 
        '2020-12-31',
        '2019-12-31',
        '2018-12-31',
        '2017-12-31',
    ])
)

# Concatenando os dfs
df_lst_acc_num_meta = pd.concat([df_lst_acc_num_meta, df_acc_num])

# Transformando em datetime o index
df_lst_acc_num_meta.index = pd.to_datetime(df_lst_acc_num_meta.index, format='%Y-%m-%d')

df_lst_acc_num_meta

In [ ]:
# Rótulos
labels_dict = f_sec.get_label_dictionary(ticker, headers=f_sec.header)

In [ ]:
# Fazendo o scraping do balance sheet (10-K)
balance_sheet_lst = []

for acc_num in df_lst_acc_num_meta['acc_num']:

    # Balance sheet
    balance_sheet = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='balance_sheet')
    balance_sheet = f_sec.rename_statement(balance_sheet, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    balance_sheet = balance_sheet.iloc[:, 0]
    balance_sheet_lst.append(balance_sheet)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_balance_sheet_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in balance_sheet_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_balance_sheet_10k = pd.concat([df_balance_sheet_10k, df], axis=1)

# Substituindo os NaN por 0
df_balance_sheet_10k = df_balance_sheet_10k.fillna(0)

# A linha 'Marketable Securities, Current' nas colunas '2021-12-31', '2020-12-31', '2019-12-31' e '2018-12-31' estão zeradas
# As informações estão na linha 'Debt Securities, Available-for-sale, Current'
df_balance_sheet_10k.loc['Marketable Securities, Current', '2021-12-31'] = df_balance_sheet_10k.loc['Debt Securities, Available-for-sale, Current', '2021-12-31'] 
df_balance_sheet_10k.loc['Marketable Securities, Current', '2020-12-31'] = df_balance_sheet_10k.loc['Debt Securities, Available-for-sale, Current', '2020-12-31'] 
df_balance_sheet_10k.loc['Marketable Securities, Current', '2019-12-31'] = df_balance_sheet_10k.loc['Debt Securities, Available-for-sale, Current', '2019-12-31'] 
df_balance_sheet_10k.loc['Marketable Securities, Current', '2018-12-31'] = df_balance_sheet_10k.loc['Debt Securities, Available-for-sale, Current', '2018-12-31'] 

# A linha 'Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization' nas colunas '2019-12-31', '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão nas linhas 'fb_PropertyPlantAndEquipmentAndFinanceLeaseRightofUseAssetAfterAccumulatedDepreciationAndAmortization' e 'Property, Plant and Equipment, Net'
df_balance_sheet_10k.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2019-12-31'] = df_balance_sheet_10k.loc['fb_PropertyPlantAndEquipmentAndFinanceLeaseRightofUseAssetAfterAccumulatedDepreciationAndAmortization', '2019-12-31'] 
df_balance_sheet_10k.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2018-12-31'] = df_balance_sheet_10k.loc['Property, Plant and Equipment, Net', '2018-12-31'] 
df_balance_sheet_10k.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2017-12-31'] = df_balance_sheet_10k.loc['Property, Plant and Equipment, Net', '2017-12-31'] 


# A linha 'meta_NonmarketableEquitySecuritiesCarryingValue' nas colunas '2021-12-31' e '2020-12-31' estão zeradas
# As informações estão nas linhas 'fb_NonmarketableEquitySecuritiesCarryingValue' e 'Equity Securities without Readily Determinable Fair Value, Amount'
df_balance_sheet_10k.loc['meta_NonmarketableEquitySecuritiesCarryingValue', '2021-12-31'] = df_balance_sheet_10k.loc['fb_NonmarketableEquitySecuritiesCarryingValue', '2021-12-31'] 
df_balance_sheet_10k.loc['meta_NonmarketableEquitySecuritiesCarryingValue', '2020-12-31'] = df_balance_sheet_10k.loc['Equity Securities without Readily Determinable Fair Value, Amount', '2020-12-31'] 

# A linha 'Accounts Payable, Trade, Current' nas colunas '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão nas linhas 'Accounts Payable, Other, Current'
df_balance_sheet_10k.loc['Accounts Payable, Trade, Current', '2018-12-31'] = df_balance_sheet_10k.loc['Accounts Payable, Other, Current', '2018-12-31'] 
df_balance_sheet_10k.loc['Accounts Payable, Trade, Current', '2017-12-31'] = df_balance_sheet_10k.loc['Accounts Payable, Other, Current', '2017-12-31'] 

# A linha 'Contract with Customer, Liability, Current' na coluna '2021-12-31' e '2017-12-31' estão zeradas
# As informações estão nas linhas 'fb_ContractWithCustomerLiabilityAndUnusedDeposits'
df_balance_sheet_10k.loc['Contract with Customer, Liability, Current', '2021-12-31'] = df_balance_sheet_10k.loc['fb_ContractWithCustomerLiabilityAndUnusedDeposits', '2021-12-31'] 
df_balance_sheet_10k.loc['Contract with Customer, Liability, Current', '2017-12-31'] = df_balance_sheet_10k.loc['Deferred Revenue and Credits, Current', '2017-12-31'] 

# Removendo as linhas que movemos os seus itens para as linhas corretas
df_balance_sheet_10k = df_balance_sheet_10k.drop([
    'Debt Securities, Available-for-sale, Current',
    'fb_PropertyPlantAndEquipmentAndFinanceLeaseRightofUseAssetAfterAccumulatedDepreciationAndAmortization',
    'Property, Plant and Equipment, Net',
    'fb_NonmarketableEquitySecuritiesCarryingValue',
    'Equity Securities without Readily Determinable Fair Value, Amount',
    'Accounts Payable, Other, Current',
    'fb_ContractWithCustomerLiabilityAndUnusedDeposits',
    'Deferred Revenue and Credits, Current'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_balance_sheet_10k = df_balance_sheet_10k.drop([
    'us-gaap_AssetsCurrentAbstract',
    'Common Stock, Value, Issued'
])

# Arrumando e adicionando alguns números no df
df_balance_sheet_10k.loc['Accrued Income Taxes, Noncurrent', '2023-12-31'] = 7514
df_balance_sheet_10k.loc['meta_NonmarketableEquitySecuritiesCarryingValue', '2019-12-31'] = 86

# Renomeando a linha do índice que está sem formato 
df_balance_sheet_10k = df_balance_sheet_10k.rename(index={
    'meta_NonmarketableEquitySecuritiesCarryingValue': 'Non-marketable Equity Securities Carrying Value',
})

df_balance_sheet_10k

In [ ]:
# Fazendo o scraping do income statement (10-K)
income_statement_lst = []

for acc_num in df_lst_acc_num_meta['acc_num']:

    # Income statement
    income_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='income_statement')
    income_statement = f_sec.rename_statement(income_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    income_statement = income_statement.iloc[:, 0]
    income_statement_lst.append(income_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_income_statement_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in income_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_income_statement_10k = pd.concat([df_income_statement_10k, df], axis=1)

# Substituindo os NaN por 0
df_income_statement_10k = df_income_statement_10k.fillna(0)

# A linha 'Revenue from Contract with Customer, Excluding Assessed Tax' na coluna '2017-12-31' está zerada
# As informações estão na linha 'Revenues'
df_income_statement_10k.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2017-12-31'] = df_income_statement_10k.loc['Revenues', '2017-12-31'] 

# Removendo as linhas que movemos os seus itens para as linhas corretas
df_income_statement_10k = df_income_statement_10k.drop('Revenues')

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_income_statement_10k = df_income_statement_10k.drop([
    'us-gaap_IncomeStatementAbstract',
    'us-gaap_CostsAndExpensesAbstract',
    'Undistributed Earnings (Loss) Allocated to Participating Securities, Basic',
    'Net Income (Loss) Available to Common Stockholders, Basic'
])

df_income_statement_10k

In [ ]:
# Fazendo o scraping do cash flow statement (10-K)
cash_flow_statement_lst = []

for acc_num in df_lst_acc_num_meta['acc_num']:

    # Cash flow statement
    cash_flow_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='cash_flow_statement')
    cash_flow_statement = f_sec.rename_statement(cash_flow_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    cash_flow_statement = cash_flow_statement.iloc[:, 0]
    cash_flow_statement_lst.append(cash_flow_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_cash_flow_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in cash_flow_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_cash_flow_10k = pd.concat([df_cash_flow_10k, df], axis=1)

# Substituindo os NaN por 0
df_cash_flow_10k = df_cash_flow_10k.fillna(0)

# A linha 'Deferred Income Taxes and Tax Credits' nas colunas '2023-12-31', '2022-12-31', '2021-12-31', '2020-12-31', '2019-12-31', '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'Deferred Income Tax Expense (Benefit)'
df_cash_flow_10k.loc['Deferred Income Taxes and Tax Credits', '2023-12-31'] = df_cash_flow_10k.loc['Deferred Income Tax Expense (Benefit)', '2023-12-31'] 
df_cash_flow_10k.loc['Deferred Income Taxes and Tax Credits', '2022-12-31'] = df_cash_flow_10k.loc['Deferred Income Tax Expense (Benefit)', '2022-12-31'] 
df_cash_flow_10k.loc['Deferred Income Taxes and Tax Credits', '2021-12-31'] = df_cash_flow_10k.loc['Deferred Income Tax Expense (Benefit)', '2021-12-31'] 
df_cash_flow_10k.loc['Deferred Income Taxes and Tax Credits', '2020-12-31'] = df_cash_flow_10k.loc['Deferred Income Tax Expense (Benefit)', '2020-12-31'] 
df_cash_flow_10k.loc['Deferred Income Taxes and Tax Credits', '2019-12-31'] = df_cash_flow_10k.loc['Deferred Income Tax Expense (Benefit)', '2019-12-31'] 
df_cash_flow_10k.loc['Deferred Income Taxes and Tax Credits', '2018-12-31'] = df_cash_flow_10k.loc['Deferred Income Tax Expense (Benefit)', '2018-12-31'] 
df_cash_flow_10k.loc['Deferred Income Taxes and Tax Credits', '2017-12-31'] = df_cash_flow_10k.loc['Deferred Income Tax Expense (Benefit)', '2017-12-31'] 

# A linha 'meta_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets' nas colunas '2021-12-31', '2020-12-31', '2019-12-31', '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'fb_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets'
df_cash_flow_10k.loc['meta_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets', '2021-12-31'] = df_cash_flow_10k.loc['fb_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets', '2021-12-31'] 
df_cash_flow_10k.loc['meta_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets', '2020-12-31'] = df_cash_flow_10k.loc['fb_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets', '2020-12-31'] 
df_cash_flow_10k.loc['meta_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets', '2019-12-31'] = df_cash_flow_10k.loc['fb_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets', '2019-12-31'] 
df_cash_flow_10k.loc['meta_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets', '2018-12-31'] = df_cash_flow_10k.loc['fb_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets', '2018-12-31'] 
df_cash_flow_10k.loc['meta_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets', '2017-12-31'] = df_cash_flow_10k.loc['fb_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets', '2017-12-31'] 

# A linha 'Payments to Acquire Marketable Securities' nas colunas '2023-12-31', '2022-12-31' e '2021-12-31' estão zeradas
# As informações estão na linha 'Payments to Acquire Debt Securities, Available-for-sale'
df_cash_flow_10k.loc['Payments to Acquire Marketable Securities', '2023-12-31'] = df_cash_flow_10k.loc['Payments to Acquire Debt Securities, Available-for-sale', '2023-12-31'] 
df_cash_flow_10k.loc['Payments to Acquire Marketable Securities', '2022-12-31'] = df_cash_flow_10k.loc['Payments to Acquire Debt Securities, Available-for-sale', '2022-12-31'] 
df_cash_flow_10k.loc['Payments to Acquire Marketable Securities', '2021-12-31'] = df_cash_flow_10k.loc['Payments to Acquire Debt Securities, Available-for-sale', '2021-12-31'] 

# A linha 'Increase (Decrease) in Accounts Payable, Trade' nas colunas '2018-12-31' e '2017-12-31' estão zeradas
# As informações estão na linha 'Increase (Decrease) in Accounts Payable'
df_cash_flow_10k.loc['Increase (Decrease) in Accounts Payable, Trade', '2018-12-31'] = df_cash_flow_10k.loc['Increase (Decrease) in Accounts Payable', '2018-12-31'] 
df_cash_flow_10k.loc['Increase (Decrease) in Accounts Payable, Trade', '2017-12-31'] = df_cash_flow_10k.loc['Increase (Decrease) in Accounts Payable', '2017-12-31'] 

# A linha 'meta_AcquisitionOfBusinessAndIntangibleAssets' nas colunas '2023-12-31' e '2022-12-31' estão zeradas
# As informações estão na linha 'meta_AcquisitionIncurredButNotYetPaid'
df_cash_flow_10k.loc['meta_AcquisitionOfBusinessAndIntangibleAssets', '2023-12-31'] = df_cash_flow_10k.loc['meta_AcquisitionIncurredButNotYetPaid', '2023-12-31'] 
df_cash_flow_10k.loc['meta_AcquisitionOfBusinessAndIntangibleAssets', '2022-12-31'] = df_cash_flow_10k.loc['meta_AcquisitionIncurredButNotYetPaid', '2022-12-31'] 

# A linha 'Impairment of Long-Lived Assets Held-for-use' na coluna '2022-12-31' está zerada
# As informações estão na linha 'us-gaap_RestructuringCharges'
df_cash_flow_10k.loc['Impairment of Long-Lived Assets Held-for-use', '2022-12-31'] = df_cash_flow_10k.loc['us-gaap_RestructuringCharges', '2022-12-31'] 

# A linha 'Increase (Decrease) in Other Noncurrent Liabilities' na coluna '2017-12-31' está zerada
# As informações estão na linha 'Increase (Decrease) in Other Operating Liabilities'
df_cash_flow_10k.loc['Increase (Decrease) in Other Noncurrent Liabilities', '2017-12-31'] = df_cash_flow_10k.loc['Increase (Decrease) in Other Operating Liabilities', '2017-12-31'] 

# A linha 'Proceeds from Sale and Maturity of Marketable Securities' na coluna '2023-12-31' está zerada
# As informações estão na linha 'Proceeds from Sale and Maturity of Debt Securities, Available-for-sale'
df_cash_flow_10k.loc['Proceeds from Sale and Maturity of Marketable Securities', '2023-12-31'] = df_cash_flow_10k.loc['Proceeds from Sale and Maturity of Debt Securities, Available-for-sale', '2023-12-31'] 

# A linha 'Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents' na coluna '2017-12-31' está zerada
# As informações estão na linha 'Effect of Exchange Rate on Cash and Cash Equivalents'
df_cash_flow_10k.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2017-12-31'] = df_cash_flow_10k.loc['Effect of Exchange Rate on Cash and Cash Equivalents', '2017-12-31'] 

# A linha 'meta_RepurchaseOfCommonStockInAccruedExpensesAndOtherCurrentLiabilities' na coluna '2021-12-31' está zerada
# As informações estão na linha 'fb_RepurchaseOfCommonStockInAccruedExpensesAndOtherCurrentLiabilities'
df_cash_flow_10k.loc['meta_RepurchaseOfCommonStockInAccruedExpensesAndOtherCurrentLiabilities', '2021-12-31'] = df_cash_flow_10k.loc['fb_RepurchaseOfCommonStockInAccruedExpensesAndOtherCurrentLiabilities', '2021-12-31'] 

# A linha 'meta_OtherSignificantNoncashTransaction' na coluna '2021-12-31' está zerada
# As informações estão na linha 'fb_OtherSignificantNoncashTransaction'
df_cash_flow_10k.loc['meta_OtherSignificantNoncashTransaction', '2021-12-31'] = df_cash_flow_10k.loc['fb_OtherSignificantNoncashTransaction', '2021-12-31'] 

# A linha 'Proceeds from Sale of Debt Securities, Available-for-sale' na coluna '2017-12-31' está zerada
# As informações estão na linha 'Proceeds from Sale of Available-for-sale Securities'
df_cash_flow_10k.loc['Proceeds from Sale of Debt Securities, Available-for-sale', '2017-12-31'] = df_cash_flow_10k.loc['Proceeds from Sale of Available-for-sale Securities', '2017-12-31'] 

# A linha 'Increase (Decrease) in Contract with Customer, Liability' na coluna '2017-12-31' está zerada
# As informações estão na linha 'Increase (Decrease) in Deferred Revenue and Customer Advances and Deposits'
df_cash_flow_10k.loc['Increase (Decrease) in Contract with Customer, Liability', '2017-12-31'] = df_cash_flow_10k.loc['Increase (Decrease) in Deferred Revenue and Customer Advances and Deposits', '2017-12-31'] 

# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_cash_flow_10k = df_cash_flow_10k.drop([
    'Deferred Income Tax Expense (Benefit)',
    'fb_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets',
    'Payments to Acquire Debt Securities, Available-for-sale',
    'Increase (Decrease) in Accounts Payable',
    'meta_AcquisitionIncurredButNotYetPaid',
    'us-gaap_RestructuringCharges',
    'Increase (Decrease) in Other Operating Liabilities',
    'Proceeds from Sale and Maturity of Debt Securities, Available-for-sale',
    'Effect of Exchange Rate on Cash and Cash Equivalents',
    'fb_RepurchaseOfCommonStockInAccruedExpensesAndOtherCurrentLiabilities',
    'fb_OtherSignificantNoncashTransaction',
    'Proceeds from Sale of Available-for-sale Securities',
    'Increase (Decrease) in Deferred Revenue and Customer Advances and Deposits'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_cash_flow_10k = df_cash_flow_10k.drop([
    'us-gaap_NetCashProvidedByUsedInOperatingActivitiesAbstract',
    'fb_RepaymentsOfCapitalLeaseAndOtherFinancingObligations',
    'Restricted Cash and Cash Equivalents, Current',
    'Restricted Cash and Cash Equivalents, Noncurrent',
    'Notes Issued',
    'Interest Paid, Including Capitalized Interest, Operating and Investing Activities',
    'fb_TaxBenefitfromSharebasedCompensationOperatingActivities',
    'meta_PaymentsForEquitySecuritiesWithoutReadilyDeterminableFairValue',
    'fb_IncreaseDecreaseInPrepaidsAndLiabilitiesRelatedToPropertyAndEquipmentAdditions',
    'fb_IncreaseDecreaseinPrepaidsandLiabilitiesRelatedtoPropertyandEquipmentAdditions',
    'fb_SettlementOfContingentConsiderationLiability',
    'fb_IncreaseDecreaseInRestrictedCashAndDeposits',
    'Cash and Cash Equivalents, Period Increase (Decrease)',
    'Equity Securities, FV-NI, Gain (Loss)',
    'fb_ChangeinUnsettledRepurchasesofCommonStock',
    'fb_AcquisitionIncurredButNotYetPaid',
    'fb_PaymentsForEquitySecuritiesWithoutReadilyDeterminableFairValue',
    'Restricted Cash and Cash Equivalents',
    'meta_OtherSignificantNoncashTransaction'
])

# Arrumando e adicionando alguns números no df
df_cash_flow_10k.loc['Proceeds from Sale and Maturity of Marketable Securities', '2022-12-31'] = 13158
df_cash_flow_10k.loc['Payments for (Proceeds from) Other Investing Activities', '2017-12-31'] = -13
df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect', '2017-12-31'] = -905
df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2017-12-31'] = 9109
df_cash_flow_10k.loc['Cash and Cash Equivalents, at Carrying Value', '2020-12-31'] = 17576
df_cash_flow_10k.loc['Cash and Cash Equivalents, at Carrying Value', '2019-12-31'] = 19079
df_cash_flow_10k.loc['Cash and Cash Equivalents, at Carrying Value', '2018-12-31'] = 10019
df_cash_flow_10k.loc['Capital Expenditures Incurred but Not yet Paid', '2018-12-31'] = 1955

# Renomeando a linha do índice que está sem formato 
df_cash_flow_10k = df_cash_flow_10k.rename(index={
    'meta_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets': 'Payments To Acquire Businesses Net Of Cash Acquired And Purchases Of Intangible And Other Assets',
    'meta_AcquisitionOfBusinessAndIntangibleAssets': 'Acquisition Of Business And Intangible Assets',
    'meta_RepurchaseOfCommonStockInAccruedExpensesAndOtherCurrentLiabilities': 'Repurchase Of Common Stock In Accrued Expenses And Other Current Liabilities',
    'meta_UnrealizedGainLossOnMarketableAndNonmarketableEquityInvestments': 'Unrealized Gain Loss On Marketable And Nonmarketable Equity Investments',
    'meta_PaymentsForHeldForSaleAssets': 'Payments For Held For Sale Assets',
    'meta_ProceedsFromOneTimeVentureDistribution': 'Proceeds From One Time Venture Distribution'
})

df_cash_flow_10k

In [ ]:
# Arrumando alguns index dos dfs para ficar padronizado com os index utilizados nas funções p/ calcular os indicadores fundamentalistas
df_balance_sheet_10k = df_balance_sheet_10k.rename(index={
    'Cash and Cash Equivalents, at Carrying Value': 'Cash and Cash Equivalents',
    'Marketable Securities, Current': 'Marketable Securities',
    'Long-term Debt, Excluding Current Maturities': 'Long-term Debt',
    "Stockholders' Equity Attributable to Parent": 'Total Shareholders Equity'
})

df_income_statement_10k = df_income_statement_10k.rename(index={
    'Revenue from Contract with Customer, Excluding Assessed Tax': 'Revenues',
    'Research and Development Expense': 'Research and Development',
    'Operating Income (Loss)': 'Operating Income',
    'Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest': 'Income Before Provision for Income Taxes',
    'Income Tax Expense (Benefit)': 'Provision for Income Taxes',
    'Net Income (Loss) Attributable to Parent': 'Net Income',
    'Weighted Average Number of Shares Outstanding, Basic': 'Number of Shares - Basic',
    'Weighted Average Number of Shares Outstanding, Diluted': 'Number of Shares - Diluted'
})

df_cash_flow_10k = df_cash_flow_10k.rename(index={
    'Payments to Acquire Property, Plant, and Equipment': 'Purchases Of Property And Equipment And Intangible Assets'
})

In [ ]:
# Selecionando o 'acc_num' do ano mais recente
last_acc_num = df_lst_acc_num_meta.iloc[0].item()

# Fazendo o scraping do interent expense (10-K) -> 'Note 14. Interest and Other Income (Expense), Net'
interest_expense_10k = f_sec.process_one_statement(ticker=ticker, accession_number=last_acc_num, statement_name='interest_expense')
interest_expense_10k = f_sec.rename_statement(interest_expense_10k, label_dictionary=labels_dict)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
interest_expense_10k = interest_expense_10k.drop('us-gaap_OtherIncomeAndExpensesAbstract')

# Renomeando a linha do índice que está sem formato 
interest_expense_10k = interest_expense_10k.rename(index={
    'InterestExpenseNonoperating': 'Interest Expense',
})

# Formatando o número para ficar positivo
interest_expense_10k.loc['Interest Expense'] = interest_expense_10k.loc['Interest Expense'] * -1

# Selecionando a linha desejada
interest_expense_10k = interest_expense_10k.loc[['Interest Expense']]

interest_expense_10k

In [ ]:
# 'acc_num' de 2024
acc_num_2024 = '000132680125000017'

# Fazendo o scraping do restructuring (10-K) -> 'Note 3. Restructuring'
restructuring_pretax_charges_10k = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num_2024, statement_name='restructuring_pretax_charges')
restructuring_pretax_charges_10k = f_sec.rename_statement(restructuring_pretax_charges_10k, label_dictionary=labels_dict)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
restructuring_pretax_charges_10k = restructuring_pretax_charges_10k.drop('us-gaap_RestructuringPlanAxis=meta_A2022RestructuringMember')

# Resetando o index do df, porque ele possui vários index iguais
restructuring_pretax_charges_10k = restructuring_pretax_charges_10k.reset_index(drop=True)

# Selecionando as linhas desejadas
restructuring_pretax_charges_10k = restructuring_pretax_charges_10k.iloc[[2]]

# Renomeando a linha do índice
restructuring_pretax_charges_10k = restructuring_pretax_charges_10k.rename(index={
    2: 'Restructuring Pre-tax Charges',
})

# Criando a coluna do ano de 2025
restructuring_pretax_charges_10k['2025-12-31'] = 0

# Ordenando o df
restructuring_pretax_charges_10k = restructuring_pretax_charges_10k[sorted(restructuring_pretax_charges_10k.columns, reverse=True)]

restructuring_pretax_charges_10k

In [ ]:
# 'acc_num' entre 2019 e 2025
df_lst_acc_num_meta_sorted = df_lst_acc_num_meta.sort_index()
acc_num_list_meta_sliced = df_lst_acc_num_meta_sorted.loc['2019':'2025', 'acc_num']

# Fazendo o scraping do leases (10-K)
leases_lst = []

for acc_num in acc_num_list_meta_sliced:

    # Fazendo o scraping do leases (10-K) -> 'Note 8. Leases'
    leases_10k = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='leases')
    leases_10k = f_sec.rename_statement(leases_10k, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    leases_10k = leases_10k.iloc[:, 0]
    leases_lst.append(leases_10k)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_leases_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in leases_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_leases_10k = pd.concat([df_leases_10k, df], axis=1)

# Renomeando a linha do índice
df_leases_10k = df_leases_10k.rename(index={
    'Operating Lease, Liability': 'Present Value of Lease Liabilities - Operating Leases',
    'Finance Lease, Liability': 'Present Value of Lease Liabilities - Finance Leases',
})

# Selecionando as linhas desejadas
df_leases_10k = df_leases_10k.loc[[
    'Present Value of Lease Liabilities - Operating Leases',
    'Present Value of Lease Liabilities - Finance Leases'
]]

#####################################################################################

# 'acc_num' de 2018
acc_num_2018 = '000132680119000009'

# Fazendo o scraping do leases (10-K) -> 'Note 8. Leases'
leases_10k_2018 = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num_2018, statement_name='leases')
leases_10k_2018 = f_sec.rename_statement(leases_10k_2018, label_dictionary=labels_dict)

# Selecionando a coluna desejada 
leases_10k_2018 = leases_10k_2018.iloc[:, [1]]

# Renomeando a linha do índice
leases_10k_2018 = leases_10k_2018.rename(index={
    'Operating Leases, Future Minimum Payments Due': 'Present Value of Lease Liabilities - Operating Leases'
})

# Criando uma linha nova
leases_10k_2018.loc['Present Value of Lease Liabilities - Finance Leases'] = 0

# Selecionando as linhas desejadas
leases_10k_2018 = leases_10k_2018.loc[[
    'Present Value of Lease Liabilities - Operating Leases', 
    'Present Value of Lease Liabilities - Finance Leases'
]]

#####################################################################################

# 'acc_num' de 2017
acc_num_2017 = '000132680118000009'

# Fazendo o scraping do leases (10-K) -> 'Note 8. Leases'
leases_10k_2017 = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num_2017, statement_name='leases')
leases_10k_2017 = f_sec.rename_statement(leases_10k_2017, label_dictionary=labels_dict)

# Selecionando a coluna desejada 
leases_10k_2017 = leases_10k_2017.iloc[:, [0]]

# Renomeando a linha do índice
leases_10k_2017 = leases_10k_2017.rename(index={
    'Operating Leases, Future Minimum Payments Due': 'Present Value of Lease Liabilities - Operating Leases'
})

# Criando uma linha nova
leases_10k_2017.loc['Present Value of Lease Liabilities - Finance Leases'] = 0

# Selecionando as linhas desejadas
leases_10k_2017 = leases_10k_2017.loc[[
    'Present Value of Lease Liabilities - Operating Leases', 
    'Present Value of Lease Liabilities - Finance Leases'
]]

#####################################################################################

# Concatenando os dfs
df_leases_10k = pd.concat([df_leases_10k, leases_10k_2018, leases_10k_2017], axis=1)

# Ordenando o df
df_leases_10k = df_leases_10k[sorted(df_leases_10k.columns, reverse=True)]

df_leases_10k

In [ ]:
# Fazendo o scraping do restricted stock units (10-K) -> 'Note 4. Earnings per Share'
rsu_10k = f_sec.process_one_statement(ticker=ticker, accession_number=last_acc_num, statement_name='rsu')
rsu_10k = f_sec.rename_statement(rsu_10k, label_dictionary=labels_dict)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
rsu_10k = rsu_10k.drop('us-gaap_NetIncomeLossAvailableToCommonStockholdersBasicAbstract')

# Renomeando a linha do índice
rsu_10k = rsu_10k.rename(index={
    'Incremental Common Shares Attributable to Dilutive Effect of Share-based Payment Arrangements': 'RSUs',
})

# Selecionando a linha desejada
rsu_10k = rsu_10k.loc[['RSUs']]

rsu_10k

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//tecnologia//meta//meta_10k.xlsx') as writer:
    df_balance_sheet_10k.to_excel(writer, sheet_name='balance_sheet')
    df_income_statement_10k.to_excel(writer, sheet_name='income_statement')
    df_cash_flow_10k.to_excel(writer, sheet_name='cash_flow')
    interest_expense_10k.to_excel(writer, sheet_name='interest_expense')
    restructuring_pretax_charges_10k.to_excel(writer, sheet_name='restructuring_pretax_charges')
    df_leases_10k.to_excel(writer, sheet_name='leases')
    rsu_10k.to_excel(writer, sheet_name='num_shares')

### Scraping Meta 10-Q

In [ ]:
# Número dos documentos que contém a DRE, BP e FC 
acc_documents_10q = f_sec.get_filtered_filings(ticker=ticker, ten_k=False, just_accession_numbers=True)

# Trocando os '-' para '' das strings ('0001045810-24-000264' -> '000104581024000264')
dict_acc_num_10q = {k: v.replace('-', '') for k, v in acc_documents_10q.items()}

# Transformando o dicionário em um df
df_lst_acc_num_10q_meta = pd.DataFrame.from_dict(dict_acc_num_10q, orient='index', columns=['acc_num'])

# Adicionando os 'acc_num' de 1Q, 2Q e 3Q dos anos de 2020, 2021, 2022 e 2023
df_acc_num = pd.DataFrame(
    {
        'acc_num': [
            '000132680123000103', # 3Q 2023
            '000132680123000093', # 2Q 2023
            '000132680123000067', # 1Q 2023
            '000132680122000108', # 3Q 2022
            '000132680122000082', # 2Q 2022
            '000132680122000057', # 1Q 2022
            '000132680121000065', # 3Q 2021
            '000132680121000049', # 2Q 2021
            '000132680121000033', # 1Q 2021
            '000132680120000084', # 3Q 2020
            '000132680120000076', # 2Q 2020
            '000132680120000048', # 1Q 2020
        ]
    },
    index=pd.to_datetime([
        '2023-09-30', 
        '2023-06-30',
        '2023-03-31',
        '2022-09-30', 
        '2022-06-30',
        '2022-03-31',
        '2021-09-30', 
        '2021-06-30',
        '2021-03-31',
        '2020-09-30', 
        '2020-06-30',
        '2020-03-31',

    ])
)

# Concatenando os dfs
df_lst_acc_num_10q_meta = pd.concat([df_lst_acc_num_10q_meta, df_acc_num])

# Transformando em datetime o index
df_lst_acc_num_10q_meta.index = pd.to_datetime(df_lst_acc_num_10q_meta.index, format='%Y-%m-%d')

df_lst_acc_num_10q_meta

In [ ]:
# Rótulos
labels_dict = f_sec.get_label_dictionary(ticker, headers=f_sec.header)

In [ ]:
# Fazendo o scraping do balance sheet (10-Q)
balance_sheet_lst = []

for acc_num in df_lst_acc_num_10q_meta['acc_num']:

    # Balance sheet
    balance_sheet = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='balance_sheet')
    balance_sheet = f_sec.rename_statement(balance_sheet, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    balance_sheet = balance_sheet.iloc[:, 0]
    balance_sheet_lst.append(balance_sheet)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'
# Tenho que usar a função 'keep=first'
df_balance_sheet_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in balance_sheet_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_balance_sheet_10q = pd.concat([df_balance_sheet_10q, df], axis=1)

# Substituindo os NaN por 0
df_balance_sheet_10q = df_balance_sheet_10q.fillna(0)

# A linha 'Marketable Securities, Current' nas colunas '2022-03-31', '2021-09-30', '2021-06-30', '2021-03-31', '2020-09-30', '2020-06-30' e '2020-03-31' estão zeradas
# As informações estão na linha 'Debt Securities, Available-for-sale, Current' 
df_balance_sheet_10q.loc['Marketable Securities, Current', '2022-03-31'] = df_balance_sheet_10q.loc['Debt Securities, Available-for-sale, Current', '2022-03-31'] 
df_balance_sheet_10q.loc['Marketable Securities, Current', '2021-09-30'] = df_balance_sheet_10q.loc['Debt Securities, Available-for-sale, Current', '2021-09-30'] 
df_balance_sheet_10q.loc['Marketable Securities, Current', '2021-06-30'] = df_balance_sheet_10q.loc['Debt Securities, Available-for-sale, Current', '2021-06-30'] 
df_balance_sheet_10q.loc['Marketable Securities, Current', '2021-03-31'] = df_balance_sheet_10q.loc['Debt Securities, Available-for-sale, Current', '2021-03-31'] 
df_balance_sheet_10q.loc['Marketable Securities, Current', '2020-09-30'] = df_balance_sheet_10q.loc['Debt Securities, Available-for-sale, Current', '2020-09-30'] 
df_balance_sheet_10q.loc['Marketable Securities, Current', '2020-06-30'] = df_balance_sheet_10q.loc['Debt Securities, Available-for-sale, Current', '2020-06-30'] 
df_balance_sheet_10q.loc['Marketable Securities, Current', '2020-03-31'] = df_balance_sheet_10q.loc['Debt Securities, Available-for-sale, Current', '2020-03-31'] 

# A linha 'meta_ContractWithCustomerLiabilityAndUnusedDeposits' nas colunas '2021-09-30', '2021-06-30', '2021-03-31', '2020-09-30', '2020-06-30' e '2020-03-31' estão zeradas
# As informações estão na linha 'fb_ContractWithCustomerLiabilityAndUnusedDeposits' 
df_balance_sheet_10q.loc['meta_ContractWithCustomerLiabilityAndUnusedDeposits', '2021-09-30'] = df_balance_sheet_10q.loc['fb_ContractWithCustomerLiabilityAndUnusedDeposits', '2021-09-30'] 
df_balance_sheet_10q.loc['meta_ContractWithCustomerLiabilityAndUnusedDeposits', '2021-06-30'] = df_balance_sheet_10q.loc['fb_ContractWithCustomerLiabilityAndUnusedDeposits', '2021-06-30'] 
df_balance_sheet_10q.loc['meta_ContractWithCustomerLiabilityAndUnusedDeposits', '2021-03-31'] = df_balance_sheet_10q.loc['fb_ContractWithCustomerLiabilityAndUnusedDeposits', '2021-03-31'] 
df_balance_sheet_10q.loc['meta_ContractWithCustomerLiabilityAndUnusedDeposits', '2020-09-30'] = df_balance_sheet_10q.loc['Contract with Customer, Liability, Current', '2020-09-30'] 
df_balance_sheet_10q.loc['meta_ContractWithCustomerLiabilityAndUnusedDeposits', '2020-06-30'] = df_balance_sheet_10q.loc['Contract with Customer, Liability, Current', '2020-06-30'] 
df_balance_sheet_10q.loc['meta_ContractWithCustomerLiabilityAndUnusedDeposits', '2020-03-31'] = df_balance_sheet_10q.loc['Contract with Customer, Liability, Current', '2020-03-31'] 

# A linha 'meta_NonmarketableEquitySecuritiesCarryingValue' nas colunas '2021-09-30', '2021-06-30', '2021-03-31' e '2020-09-30' estão zeradas
# As informações estão na linha 'Equity Securities without Readily Determinable Fair Value, Amount' 
df_balance_sheet_10q.loc['meta_NonmarketableEquitySecuritiesCarryingValue', '2021-09-30'] = df_balance_sheet_10q.loc['Equity Securities without Readily Determinable Fair Value, Amount', '2021-09-30'] 
df_balance_sheet_10q.loc['meta_NonmarketableEquitySecuritiesCarryingValue', '2021-06-30'] = df_balance_sheet_10q.loc['Equity Securities without Readily Determinable Fair Value, Amount', '2021-06-30'] 
df_balance_sheet_10q.loc['meta_NonmarketableEquitySecuritiesCarryingValue', '2021-03-31'] = df_balance_sheet_10q.loc['Equity Securities without Readily Determinable Fair Value, Amount', '2021-03-31'] 
df_balance_sheet_10q.loc['meta_NonmarketableEquitySecuritiesCarryingValue', '2020-09-30'] = df_balance_sheet_10q.loc['Equity Securities without Readily Determinable Fair Value, Amount', '2020-09-30'] 

# A linha 'Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization' nas colunas '2020-09-30', '2020-06-30' e '2020-03-31' estão zeradas
# As informações estão na linha 'Property, Plant and Equipment, Net' 
df_balance_sheet_10q.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2020-09-30'] = df_balance_sheet_10q.loc['Property, Plant and Equipment, Net', '2020-09-30'] 
df_balance_sheet_10q.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2020-06-30'] = df_balance_sheet_10q.loc['Property, Plant and Equipment, Net', '2020-06-30'] 
df_balance_sheet_10q.loc['Property, Plant, and Equipment and Finance Lease Right-of-Use Asset, after Accumulated Depreciation and Amortization', '2020-03-31'] = df_balance_sheet_10q.loc['Property, Plant and Equipment, Net', '2020-03-31'] 

# A linha 'Accounts Payable, Other, Current' nas colunas '2020-09-30', '2020-06-30' e '2020-03-31' estão zeradas
# As informações estão na linha 'Accounts Payable and Other Accrued Liabilities, Current' 
df_balance_sheet_10q.loc['Accounts Payable, Other, Current', '2020-09-30'] = df_balance_sheet_10q.loc['Accounts Payable and Other Accrued Liabilities, Current', '2020-09-30'] 
df_balance_sheet_10q.loc['Accounts Payable, Other, Current', '2020-06-30'] = df_balance_sheet_10q.loc['Accounts Payable and Other Accrued Liabilities, Current', '2020-06-30'] 
df_balance_sheet_10q.loc['Accounts Payable, Other, Current', '2020-03-31'] = df_balance_sheet_10q.loc['Accounts Payable and Other Accrued Liabilities, Current', '2020-03-31'] 

# A linha 'Accounts Payable, Trade, Current' nas colunas '2020-09-30' e '2020-06-30' estão zeradas
# As informações estão na linha 'Accounts Payable, Current' 
df_balance_sheet_10q.loc['Accounts Payable, Trade, Current', '2020-09-30'] = df_balance_sheet_10q.loc['Accounts Payable, Current', '2020-09-30'] 
df_balance_sheet_10q.loc['Accounts Payable, Trade, Current', '2020-06-30'] = df_balance_sheet_10q.loc['Accounts Payable, Current', '2020-06-30'] 

# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_balance_sheet_10q = df_balance_sheet_10q.drop([
    'Debt Securities, Available-for-sale, Current',
    'fb_ContractWithCustomerLiabilityAndUnusedDeposits',
    'Contract with Customer, Liability, Current',
    'Equity Securities without Readily Determinable Fair Value, Amount',
    'Property, Plant and Equipment, Net',
    'Accounts Payable and Other Accrued Liabilities, Current',
    'Accounts Payable, Current'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_balance_sheet_10q = df_balance_sheet_10q.drop([
    'us-gaap_AssetsCurrentAbstract',
    'Common Stock, Value, Issued'
])

# Renomeando a linha do índice que está sem formato 
df_balance_sheet_10q = df_balance_sheet_10q.rename(index={
    'meta_NonmarketableEquitySecuritiesCarryingValue': 'Nonmarketable Equity Securities Carrying Value',
    'meta_ContractWithCustomerLiabilityAndUnusedDeposits': 'Contract With Customer Liability And Unused Deposits',
})

df_balance_sheet_10q

In [ ]:
# Fazendo o scraping do income statement (10-Q)
income_statement_lst = []

for acc_num in df_lst_acc_num_10q_meta['acc_num']:

    # Income statement
    income_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='income_statement')
    income_statement = f_sec.rename_statement(income_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    income_statement = income_statement.iloc[:, 0]
    income_statement_lst.append(income_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_income_statement_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in income_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_income_statement_10q = pd.concat([df_income_statement_10q, df], axis=1)

# Substituindo os NaN por 0
df_income_statement_10q = df_income_statement_10q.fillna(0)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_income_statement_10q = df_income_statement_10q.drop([
    'us-gaap_IncomeStatementAbstract',
    'us-gaap_CostsAndExpensesAbstract'
])

df_income_statement_10q

In [ ]:
# Fazendo o scraping do cash flow statement (10-Q)
cash_flow_statement_lst = []

for acc_num in df_lst_acc_num_10q_meta['acc_num']:

    # Cash flow statement
    cash_flow_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='cash_flow_statement')
    cash_flow_statement = f_sec.rename_statement(cash_flow_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    cash_flow_statement = cash_flow_statement.iloc[:, 0]
    cash_flow_statement_lst.append(cash_flow_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_cash_flow_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in cash_flow_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_cash_flow_10q = pd.concat([df_cash_flow_10q, df], axis=1)

# Substituindo os NaN por 0
df_cash_flow_10q = df_cash_flow_10q.fillna(0)

# A linha 'Proceeds from Sale and Maturity of Debt Securities, Available-for-sale' nas colunas '2022-09-30', '2022-06-30', '2022-03-31', '2021-09-30', '2021-06-30', '2021-03-31', '2020-09-30', '2020-06-30' e '2020-03-31' estão zeradas
# As informações estão na linha 'Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale'
df_cash_flow_10q.loc['Proceeds from Sale and Maturity of Debt Securities, Available-for-sale', '2022-09-30'] = df_cash_flow_10q.loc['Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale', '2022-09-30'] 
df_cash_flow_10q.loc['Proceeds from Sale and Maturity of Debt Securities, Available-for-sale', '2022-06-30'] = df_cash_flow_10q.loc['Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale', '2022-06-30'] 
df_cash_flow_10q.loc['Proceeds from Sale and Maturity of Debt Securities, Available-for-sale', '2022-03-31'] = df_cash_flow_10q.loc['Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale', '2022-03-31'] 
df_cash_flow_10q.loc['Proceeds from Sale and Maturity of Debt Securities, Available-for-sale', '2021-09-30'] = df_cash_flow_10q.loc['Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale', '2021-09-30'] 
df_cash_flow_10q.loc['Proceeds from Sale and Maturity of Debt Securities, Available-for-sale', '2021-06-30'] = df_cash_flow_10q.loc['Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale', '2021-06-30'] 
df_cash_flow_10q.loc['Proceeds from Sale and Maturity of Debt Securities, Available-for-sale', '2021-03-31'] = df_cash_flow_10q.loc['Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale', '2021-03-31'] 
df_cash_flow_10q.loc['Proceeds from Sale and Maturity of Debt Securities, Available-for-sale', '2020-09-30'] = df_cash_flow_10q.loc['Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale', '2020-09-30'] 
df_cash_flow_10q.loc['Proceeds from Sale and Maturity of Debt Securities, Available-for-sale', '2020-06-30'] = df_cash_flow_10q.loc['Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale', '2020-06-30'] 
df_cash_flow_10q.loc['Proceeds from Sale and Maturity of Debt Securities, Available-for-sale', '2020-03-31'] = df_cash_flow_10q.loc['Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale', '2020-03-31'] 

# A linha 'meta_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets' nas colunas '2021-09-30', '2021-06-30', '2020-09-30', '2020-06-30' e '2020-03-31' estão zeradas
# As informações estão na linha 'fb_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets' 
df_cash_flow_10q.loc['meta_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets', '2021-09-30'] = df_cash_flow_10q.loc['fb_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets', '2021-09-30'] 
df_cash_flow_10q.loc['meta_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets', '2021-06-30'] = df_cash_flow_10q.loc['fb_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets', '2021-06-30'] 
df_cash_flow_10q.loc['meta_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets', '2020-09-30'] = df_cash_flow_10q.loc['fb_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets', '2020-09-30'] 
df_cash_flow_10q.loc['meta_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets', '2020-06-30'] = df_cash_flow_10q.loc['fb_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets', '2020-06-30'] 
df_cash_flow_10q.loc['meta_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets', '2020-03-31'] = df_cash_flow_10q.loc['fb_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets', '2020-03-31'] 

# A linha 'meta_AcquisitionOfBusinessAndIntangibleAssets' nas colunas '2024-03-31', '2023-09-30', '2023-06-30', '2023-03-31' e '2022-03-31' estão zeradas
# As informações estão nas linhas 'meta_PaymentsToAcquireBusinessesAndIntangibleAssets' e 'meta_AcquisitionOfBusinessesInAccruedExpensesAndOtherCurrentLiabilitiesAndOtherLiabilities'
df_cash_flow_10q.loc['meta_AcquisitionOfBusinessAndIntangibleAssets', '2024-03-31'] = df_cash_flow_10q.loc['meta_AcquisitionIncurredButNotYetPaid', '2024-03-31'] 
df_cash_flow_10q.loc['meta_AcquisitionOfBusinessAndIntangibleAssets', '2023-09-30'] = df_cash_flow_10q.loc['meta_AcquisitionIncurredButNotYetPaid', '2023-09-30'] 
df_cash_flow_10q.loc['meta_AcquisitionOfBusinessAndIntangibleAssets', '2023-06-30'] = df_cash_flow_10q.loc['meta_AcquisitionIncurredButNotYetPaid', '2023-06-30'] 
df_cash_flow_10q.loc['meta_AcquisitionOfBusinessAndIntangibleAssets', '2023-03-31'] = df_cash_flow_10q.loc['meta_AcquisitionIncurredButNotYetPaid', '2023-03-31'] 
df_cash_flow_10q.loc['meta_AcquisitionOfBusinessAndIntangibleAssets', '2022-03-31'] = df_cash_flow_10q.loc['meta_AcquisitionIncurredButNotYetPaid', '2022-03-31'] 
df_cash_flow_10q.loc['meta_AcquisitionOfBusinessAndIntangibleAssets', '2022-09-30'] = df_cash_flow_10q.loc['meta_AcquisitionOfBusinessesInAccruedExpensesAndOtherCurrentLiabilitiesAndOtherLiabilities', '2022-09-30'] 

# A linha 'meta_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets' nas colunas '2022-09-30', '2022-06-30', '2022-03-31', '2021-09-30', '2021-06-30', '2021-03-31', '2020-09-30', '2020-06-30' e '2020-03-31' estão zeradas
# As informações estão na linha 'meta_PaymentsToAcquireBusinessesAndIntangibleAssets'
df_cash_flow_10q.loc['meta_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets', '2022-09-30'] = df_cash_flow_10q.loc['meta_PaymentsToAcquireBusinessesAndIntangibleAssets', '2022-09-30'] 
df_cash_flow_10q.loc['meta_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets', '2022-06-30'] = df_cash_flow_10q.loc['meta_PaymentsToAcquireBusinessesAndIntangibleAssets', '2022-06-30'] 
df_cash_flow_10q.loc['meta_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets', '2022-03-31'] = df_cash_flow_10q.loc['meta_PaymentsToAcquireBusinessesAndIntangibleAssets', '2022-03-31'] 

# A linha 'Increase (Decrease) in Contract with Customer, Liability' nas colunas '2022-09-30', '2022-06-30' e '2022-03-31' estão zeradas
# As informações estão na linha 'meta_IncreaseDecreaseInContractWithCustomerLiabilityAndUnusedDeposits'
df_cash_flow_10q.loc['Increase (Decrease) in Contract with Customer, Liability', '2022-09-30'] = df_cash_flow_10q.loc['meta_IncreaseDecreaseInContractWithCustomerLiabilityAndUnusedDeposits', '2022-09-30'] 
df_cash_flow_10q.loc['Increase (Decrease) in Contract with Customer, Liability', '2022-06-30'] = df_cash_flow_10q.loc['meta_IncreaseDecreaseInContractWithCustomerLiabilityAndUnusedDeposits', '2022-06-30'] 
df_cash_flow_10q.loc['Increase (Decrease) in Contract with Customer, Liability', '2022-03-31'] = df_cash_flow_10q.loc['meta_IncreaseDecreaseInContractWithCustomerLiabilityAndUnusedDeposits', '2022-03-31'] 

# A linha 'Increase (Decrease) in Other Accounts Payable' nas colunas '2020-09-30' e '2020-06-30' estão zeradas
# As informações estão na linha 'Increase (Decrease) in Accounts Payable and Other Operating Liabilities'
df_cash_flow_10q.loc['Increase (Decrease) in Other Accounts Payable', '2020-09-30'] = df_cash_flow_10q.loc['Increase (Decrease) in Accounts Payable and Other Operating Liabilities', '2020-09-30'] 
df_cash_flow_10q.loc['Increase (Decrease) in Other Accounts Payable', '2020-06-30'] = df_cash_flow_10q.loc['Increase (Decrease) in Accounts Payable and Other Operating Liabilities', '2020-06-30'] 

# A linha 'Income Taxes Paid, Net' nas colunas '2021-09-30' e '2021-06-30' estão zeradas
# As informações estão na linha 'Income Taxes Paid'
df_cash_flow_10q.loc['Income Taxes Paid, Net', '2021-09-30'] = df_cash_flow_10q.loc['Income Taxes Paid', '2021-09-30'] 
df_cash_flow_10q.loc['Income Taxes Paid, Net', '2021-06-30'] = df_cash_flow_10q.loc['Income Taxes Paid', '2021-06-30'] 

# A linha 'Payments to Acquire Debt Securities, Available-for-sale' na coluna '2020-03-31' está zerada
# As informações estão na linha 'Payments to Acquire Marketable Securities'
df_cash_flow_10q.loc['Payments to Acquire Debt Securities, Available-for-sale', '2020-03-31'] = df_cash_flow_10q.loc['Payments to Acquire Marketable Securities', '2020-03-31'] 

# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_cash_flow_10q = df_cash_flow_10q.drop([
    'Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale',
    'fb_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets',
    'meta_AcquisitionIncurredButNotYetPaid',
    'meta_AcquisitionOfBusinessesInAccruedExpensesAndOtherCurrentLiabilitiesAndOtherLiabilities',
    'meta_PaymentsToAcquireBusinessesAndIntangibleAssets',
    'meta_IncreaseDecreaseInContractWithCustomerLiabilityAndUnusedDeposits',
    'Increase (Decrease) in Accounts Payable and Other Operating Liabilities',
    'Income Taxes Paid',
    'Payments to Acquire Marketable Securities'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_cash_flow_10q = df_cash_flow_10q.drop([
    'us-gaap_NetCashProvidedByUsedInOperatingActivitiesAbstract',
    'Restricted Cash and Cash Equivalents, Current',
    'Restricted Cash and Cash Equivalents, Noncurrent',
    'meta_SettlementOfConversionNotesWithMarketableEquitySecurities',
    'meta_SettlementOfConvertibleNotesInExchangeOfEquitySecuritiesInOtherCurrentAssets',
    'fb_NoncashOrPartNoncashAcquisitionAndOtherInvestmentsOtherLiabilitiesAssumed',
    'Noncash or Part Noncash Acquisition, Other Liabilities Assumed',
    'meta_NonMarketableEquityInvestmentsInAccruedExpensesAndOtherCurrentLiabilities',
    'fb_RepurchaseOfCommonStockInAccruedExpensesAndOtherCurrentLiabilities',
    'fb_OtherSignificantNoncashTransaction',
    'meta_OtherSignificantNoncashTransaction',
    'fb_AcquisitionIncurredButNotYetPaid',
    'fb_PaymentsForEquitySecuritiesWithoutReadilyDeterminableFairValue'
])

# Renomeando a linha do índice que está sem formato 
df_cash_flow_10q = df_cash_flow_10q.rename(index={
    'meta_PaymentsForHeldForSaleAssets': 'Payments For Held For Sale Assets',
    'meta_PaymentsToAcquireBusinessesNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets': 'Payments To Acquire Businesses Net Of Cash Acquired And Purchases Of Intangible And Other Assets',
    'meta_AcquisitionOfBusinessAndIntangibleAssets': 'Acquisition Of Business And Intangible Assets',
    'meta_RepurchaseOfCommonStockInAccruedExpensesAndOtherCurrentLiabilities': 'Repurchase Of Common Stock In Accrued Expenses And Other Current Liabilities',
})

df_cash_flow_10q

In [ ]:
# Arrumando alguns index dos dfs para ficar padronizado com os index utilizados nas funções p/ calcular os indicadores fundamentalistas
df_balance_sheet_10q = df_balance_sheet_10q.rename(index={
    'Cash and Cash Equivalents, at Carrying Value': 'Cash and Cash Equivalents',
    'Marketable Securities, Current': 'Marketable Securities',
    'Long-term Debt, Excluding Current Maturities': 'Long-term Debt',
    'Operating Lease, Liability, Current': 'Operating Lease, Current',
    'Operating Lease, Liability, Noncurrent': 'Operating Lease, Noncurrent',
    "Stockholders' Equity Attributable to Parent": 'Total Shareholders Equity'
})

df_income_statement_10q = df_income_statement_10q.rename(index={
    'Revenue from Contract with Customer, Excluding Assessed Tax': 'Revenues',
    'Research and Development Expense': 'Research and Development',
    'Operating Income (Loss)': 'Operating Income',
    'Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest': 'Income Before Provision for Income Taxes',
    'Income Tax Expense (Benefit)': 'Provision for Income Taxes',
    'Net Income (Loss) Attributable to Parent': 'Net Income',
    'Weighted Average Number of Shares Outstanding, Basic': 'Number of Shares - Basic',
    'Weighted Average Number of Shares Outstanding, Diluted': 'Number of Shares - Diluted'
})

df_cash_flow_10q = df_cash_flow_10q.rename(index={
    'Payments to Acquire Property, Plant, and Equipment': 'Purchases Of Property And Equipment And Intangible Assets'
})

Os dados abaixo não aparecem no 10-Q pela função 'get_statement_file_names_in_filing_summary' - eu vou ter quer ADICIONAR NA MÃO:
* 'Interest expense';
* 'RSUs'.


In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//tecnologia//meta//meta_10q.xlsx') as writer:
    df_balance_sheet_10q.to_excel(writer, sheet_name='balance_sheet')
    df_income_statement_10q.to_excel(writer, sheet_name='income_statement')
    df_cash_flow_10q.to_excel(writer, sheet_name='cash_flow')

### Indicadores Fundamentalistas

#### Indicadores 10-K

In [ ]:
# Lendo os arquivos 10k
meta_bs = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//meta//meta_10k.xlsx', 
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

meta_is = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//meta//meta_10k.xlsx', 
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

meta_cf = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//meta//meta_10k.xlsx', 
    sheet_name='cash_flow', 
    index_col='Unnamed: 0'
)

meta_leases = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//meta//meta_10k.xlsx', 
    sheet_name='leases', 
    index_col='Unnamed: 0'
)

# Transformando as colunas em datetime
meta_bs.columns = pd.to_datetime(meta_bs.columns)
meta_is.columns = pd.to_datetime(meta_is.columns)
meta_cf.columns = pd.to_datetime(meta_cf.columns)
meta_leases.columns = pd.to_datetime(meta_leases.columns)

# Transformando as colunas em apenas o ano
meta_bs.columns = meta_bs.columns.year
meta_is.columns = meta_is.columns.year
meta_cf.columns = meta_cf.columns.year
meta_leases.columns = meta_leases.columns.year

In [ ]:
# Preços da META
meta_close = yf.download('META', auto_adjust=True, multi_level_index=False, start='2017-01-01')['Close']

# Selecionando o último preço do mês de Dezembro de cada ano 
lst_years = [
    '2017', '2018', '2019', '2020', 
    '2021', '2022', '2023', '2024',
    '2025',
]

last_dec_close = []

for year in lst_years:
    last_close = meta_close[f'{year}-12'].iloc[-1]
    last_dec_close.append(last_close)

# Transformando em uma série pandas
meta_close_dec = pd.Series(last_dec_close, index=lst_years)

# Revertendo o index da série p/ ficar igual com os dos indicadores
meta_close_dec = meta_close_dec.reindex(index=meta_close_dec.index[::-1])

# Arredondamento de duas casas decimais
meta_close_dec = round(meta_close_dec, 2)

# Não realizou split recentemente (não possui dados de split no site da Investing)

# Transformando o index em int
meta_close_dec .index = meta_close_dec .index.astype(int)

meta_close_dec

In [ ]:
# Criando uma lista só para fazer a indexação em que o ano mais recente está no começo
idx = meta_bs.columns

# Valor de Mercado
vm = f_eua.indicador_vm(ten_k=True, price=meta_close_dec, df_is=meta_is) 

# LPA
lpa = f_eua.indicador_lpa(ten_k=True, df_is=meta_is, idx=None)

# P/L Damodaran
p_lpa_damodaran = f_eua.indicador_p_lpa_2(ten_k=True, vm=vm, df_is=meta_is)

# P/L
p_lpa = f_eua.indicador_p_lpa(ten_k=True, price=meta_close_dec, lpa=lpa, idx=idx)

# L/P
lpa_p = f_eua.indicador_lpa_p(ten_k=True, price=meta_close_dec, lpa=lpa, idx=idx) 

# Depreciação
depreciacao = f_eua.indicador_depreciacao(ten_k=True, df_cf=meta_cf, first_quarter=None, idx=idx)

# EBITDA
ebitda = f_eua.indicador_ebitda(ten_k=True, df_is=meta_is, idx=None, depreciacao=depreciacao)

# Margem Líquida
m_liq = f_eua.indicador_ml(ten_k=True, df_is=meta_is)

# VPA
vpa = f_eua.indicador_vpa(ten_k=True, df_bs=meta_bs, df_is=meta_is)

# P/VPA
p_vpa = f_eua.indicador_p_vpa(ten_k=True, price=meta_close_dec, vpa=vpa, idx=idx)

# Caixa e Equivalentes
lst_caixa = ['Cash and Cash Equivalents', 'Marketable Securities']
caixa = f_eua.indicador_caixa(ten_k=True, df_bs=meta_bs, lst_itens_caixa=lst_caixa)

# Dívida Bruta
lst_divida = ['Long-term Debt']
div_bruta = f_eua.indicador_divida_bruta(ten_k=True, df_bs=meta_bs, lst_itens_divida=lst_divida)
div_bruta_total = div_bruta.values + meta_leases.sum().values
div_bruta_total = pd.Series(div_bruta_total, index=idx)


# Dívida Líquida
div_liq = f_eua.indicador_divida_liquida(ten_k=True, caixa=caixa, div_bruta=div_bruta_total, idx=idx)

# Dívida Líquida / EBITDA
dl_ebitda = f_eua.indicador_dl_ebitda(ten_k=True, ebitda=ebitda, div_liq=div_liq, idx=None)

# Dívida Líquida / PL
dl_pl = f_eua.indicador_dl_pl(ten_k=True, div_liq=div_liq, df_bs=meta_bs) 

# EV/EBITDA
ev_ebitda = f_eua.indicador_ev_ebitda(vm=vm, div_liq=div_liq, ebitda=ebitda)

# ROE
roe = f_eua.indicador_roe(ten_k=True, df_is=meta_is, df_bs=meta_bs, idx=None)

# ROIC
roic = f_eua.indicador_roic(ten_k=True, df_is=meta_is, df_bs=meta_bs, div_bruta=div_bruta_total, idx=idx)

# FCO
fco = f_eua.indicador_fco(ten_k=True, df_cf=meta_cf, first_quarter=None) 

# FCI
fci = f_eua.indicador_fci(ten_k=True, df_cf=meta_cf, first_quarter=None) 

# FCF
fcf = f_eua.indicador_fcf(ten_k=True, df_cf=meta_cf, first_quarter=None) 

# CAPEX
lst_capex = ['Purchases Of Property And Equipment And Intangible Assets']

capex= f_eua.indicador_capex(ten_k=True, df_cf=meta_cf, first_quarter=None, lst_itens=lst_capex, idx=idx)

# Free Cash Flow
free_cash_flow = f_eua.indicador_free_cash_flow(fco=fco, capex=capex)

# Net CAPEX
net_capex = f_eua.indicador_net_capex(capex=capex, depreciation=depreciacao)

# R&D
rd_expense = f_eua.indicador_rd(ten_k=True, df_is=meta_is, idx=idx)

# Net CAPEX ajustado
adj_net_capex = f_eua.indicador_adj_net_capex(net_capex=net_capex, rd_expense=rd_expense)

# Working capital
working_capital = f_eua.indicador_working_capital(ten_k=True, df_bs=meta_bs)

# Change in non cash working capital
lst_itens_change_non_cash_wc = [
    'Deferred Income Taxes and Tax Credits',
    'Increase (Decrease) in Accounts Receivable',
    'Increase (Decrease) in Prepaid Expense and Other Assets',
    'Increase (Decrease) in Other Operating Assets',
    'Increase (Decrease) in Accounts Payable, Trade',
    'Increase (Decrease) in Accrued Liabilities',
    'Increase (Decrease) in Other Noncurrent Liabilities'
]

change_non_cash_wc = f_eua.indicador_change_non_cash_wc(ten_k=True, df_cf=meta_cf, lst_itens=lst_itens_change_non_cash_wc, first_quarter=None)

# Reinvestment rate
reinvestment_rate = f_eua.indicador_reinvestment_rate(ten_k=True, net_capex=net_capex, change_non_cash_wc=change_non_cash_wc, df_is=meta_is, idx=idx)

# New borrowing
lst_itens_new_borrowing = ['Proceeds from Issuance of Long-term Debt']

new_borrowing = f_eua.indicador_new_borrowing(ten_k=True, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=meta_cf, first_quarter=None, idx=idx)

# Debt paid
lst_itens_debt_paid = ['Finance Lease, Principal Payments']

debt_paid = f_eua.indicador_debt_paid(ten_k=True, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=meta_cf, first_quarter=None, idx=idx)

# Taxes on operating income
taxes_operating_income = f_eua.indicador_taxes_operating_income(ten_k=True, df_is=meta_is, idx=idx)

# FCFE
fcfe_10k = f_eua.indicador_fcfe(
    ten_k=True, 
    depreciacao=depreciacao, 
    capex=capex, 
    change_non_cash_wc=change_non_cash_wc, 
    new_borrowing=new_borrowing,
    debt_paid=debt_paid,
    df_is=meta_is,
    idx=idx
)

# FCFF
fcff_10k = f_eua.indicador_fcff(
        ten_k=True, 
        taxes_operating_income=taxes_operating_income,
        depreciacao=depreciacao, 
        capex=capex, 
        change_non_cash_wc=change_non_cash_wc, 
        df_is=meta_is,
        idx=idx
)

# Dividendo por Ação
dpa = f_eua.indicador_dpa(ten_k=True, df_cf=meta_cf, df_is=meta_is, div_nao_acum=None)

# Payout
payout = f_eua.indicador_payout(ten_k=True, df_cf=meta_cf, df_is=meta_is, div_nao_acum=None)

# Buyback
buyback = f_eua.indicador_buyback(ten_k=True, df_cf=meta_cf, first_quarter=None)

# Colocando todos os indicadores em uma lista
lst_indicadores = [
    p_lpa_damodaran,
    p_lpa,
    lpa_p,  
    ebitda, 
    m_liq,
    p_vpa,
    vm,
    div_bruta_total,
    caixa,
    div_liq,
    dl_ebitda,
    dl_pl,
    ev_ebitda,
    roe,
    roic,
    fco,
    fci,
    fcf,
    free_cash_flow,
    capex,
    net_capex,
    rd_expense,
    adj_net_capex,
    working_capital,
    reinvestment_rate,
    fcfe_10k,
    fcff_10k,
    dpa,
    payout,
    buyback
]

# Juntando todos os indicadores em um df
meta_indicadores_10k = pd.concat(lst_indicadores, axis=1)

# Nomes das colunas
nomes_col = [
    'P/L Damodaran', 
    'P/L', 
    'L/P', 
    'EBITDA', 
    'Margem Líquida',
    'P/VPA',
    'Valor de Mercado',
    'Dívida Bruta',
    'Caixa e Equivalentes',
    'Dívida Líquida',
    'Dívida Líquida/EBITDA',
    'Dívida Líquida/PL',
    'EV/EBITDA',
    'ROE',
    'ROIC',
    'FCO',
    'FCI',
    'FCF',
    'Free Cash Flow',
    'Capex',
    'Net Capex',
    'R&D',
    'Adj Net Capex',
    'Working Capital',
    'Reinvestment Rate',
    'FCFE',
    'FCFF',
    'DPA',
    'Payout',
    'Buyback'
]

# Atribuindo os novos nomes das colunas do df
meta_indicadores_10k.columns = nomes_col
meta_indicadores_10k

#### Indicadores 10-Q

In [ ]:
# Lendo os arquivos 10q
meta_bs_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//meta//meta_10q.xlsx',
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

meta_is_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//meta//meta_10q.xlsx',
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

meta_cf_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//meta//meta_10q.xlsx',
    sheet_name='cash_flow',  
    index_col='Unnamed: 0'
)

# Aplicando o ajuste nas colunas
meta_bs_10q.columns = [f_sec.ajustar_mes_03_06_09(col) for col in meta_bs_10q.columns]
meta_is_10q.columns = [f_sec.ajustar_mes_03_06_09(col) for col in meta_is_10q.columns]
meta_cf_10q.columns = [f_sec.ajustar_mes_03_06_09(col) for col in meta_cf_10q.columns]

In [ ]:
# Preços da META
meta_close = yf.download('META', auto_adjust=True, multi_level_index=False, start='2021-01-01')['Close']

# Selecionando o último preço dos meses de Março, Junho e Setembro
lst_years = [
    '2021', '2022', '2023', '2024',
    '2025'
]
lst_months = ['03', '06', '09']
last_close_months = []

for year in lst_years:
    for month in lst_months:
        last_close = meta_close[f'{year}-{month}'].dropna().iloc[-1]  # Dropna para lidar com meses sem dados
        last_close_months.append(last_close)

# Criando uma lista em que junte os meses e os anos para que seja o index da serie pandas
lst_idx = []

for year in lst_years:
    for month in lst_months:
        lst_idx.append(f"{month}-{year}")

# Transformando em uma série pandas
meta_close_months = pd.Series(last_close_months, index=lst_idx)

# Revertendo o index da série p/ ficar igual com os dos indicadores
meta_close_months = meta_close_months.reindex(index=meta_close_months.index[::-1])

# No ano mais atual, como ainda ele não acabou, não consigo fazer o loop com todos meses (Março, Junho e Setembro)
actual_year_int = datetime.now().year  
actual_year_str = str(actual_year_int) 

lst_actual_months = [
    # '03',
    # '06', 
    # '09'
] 

last_close_actual_months = []

for month in lst_actual_months:
    last_close = meta_close[f'{actual_year_str}-{month}'].dropna().iloc[-1]  # Dropna para lidar com meses sem dados
    last_close_actual_months.append(last_close)

# Criando uma lista em que junte os meses e os anos para que seja o index da serie pandas
lst_idx_actual = []

for month in lst_actual_months:
    lst_idx_actual.append(f"{month}-{actual_year_str}")

# Transformando em uma série pandas
meta_close_months_atual = pd.Series(last_close_actual_months, index=lst_idx_actual)

# Revertendo o index da série p/ ficar igual com os dos indicadores
meta_close_months_atual = meta_close_months_atual.reindex(index=meta_close_months_atual.index[::-1])

# Juntando as duas series pandas
meta_close_months = pd.concat([meta_close_months_atual, meta_close_months])

# Arredondamento de duas casas decimais
meta_close_months = round(meta_close_months, 2)

# Não realizou split recentemente (não possui dados de split no site da Investing)

# Substituindo o index para ficar igual ao dos dados fundamentalistas
meta_close_months.index = meta_bs_10q.columns[:-3]

meta_close_months

In [ ]:
# Criando uma lista só para fazer a indexação em que o ano mais recente está no começo
idx_10q = meta_bs_10q.columns

# Valor de Mercado
vm_10q = f_eua.indicador_vm(ten_k=False, price=meta_close_months, df_is=meta_is_10q)

# LPA
lpa_acum = f_eua.indicador_lpa(ten_k=False, df_is=meta_is_10q, idx=idx_10q)[1]
lpa_10q = f_eua.indicador_lpa(ten_k=False, df_is=meta_is_10q, idx=idx_10q)[0]

# P/L Damodaran
p_lpa_10q_damodaran = f_eua.indicador_p_lpa_2(ten_k=False, vm=vm_10q, df_is=meta_is_10q)

# P/L
p_lpa_10q = f_eua.indicador_p_lpa(ten_k=False, price=meta_close_months, lpa=lpa_acum, idx=idx_10q)

# L/P
lpa_p_10q = f_eua.indicador_lpa_p(ten_k=False, price=meta_close_months, lpa=lpa_acum, idx=idx_10q) 

# Depreciação
depreciacao_10q = f_eua.indicador_depreciacao(ten_k=False, df_cf=meta_cf_10q, first_quarter='03', idx=idx_10q)[0]
depreciacao_10q_sliced = f_eua.indicador_depreciacao(ten_k=False, df_cf=meta_cf_10q, first_quarter='03', idx=idx_10q)[1]
depreciacao_10q_acum = f_eua.indicador_depreciacao(ten_k=False, df_cf=meta_cf_10q, first_quarter='03', idx=idx_10q)[2]

# EBITDA
ebitda_10q = f_eua.indicador_ebitda(ten_k=False, df_is=meta_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[0]
ebitda_10q_sliced = f_eua.indicador_ebitda(ten_k=False, df_is=meta_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[1]
ebitda_10q_acum = f_eua.indicador_ebitda(ten_k=False, df_is=meta_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[2]

# Margem Líquida
m_liq_10q = f_eua.indicador_ml(ten_k=False, df_is=meta_is_10q)

# VPA
vpa_10q = f_eua.indicador_vpa(ten_k=False, df_bs=meta_bs_10q, df_is=meta_is_10q)

# P/VPA
p_vpa_10q = f_eua.indicador_p_vpa(ten_k=False, price=meta_close_months, vpa=vpa_10q, idx=idx_10q)

# Caixa e Equivalentes
lst_caixa = ['Cash and Cash Equivalents', 'Marketable Securities']
caixa_10q = f_eua.indicador_caixa(ten_k=False, df_bs=meta_bs_10q, lst_itens_caixa=lst_caixa)

# Dívida Bruta - não somei o custo dos 'leases', porque eles não são divulgados no 10q
lst_divida = ['Long-term Debt', 'Operating Lease, Current', 'Operating Lease, Noncurrent']
div_bruta_10q = f_eua.indicador_divida_bruta(ten_k=False, df_bs=meta_bs_10q, lst_itens_divida=lst_divida)

# Dívida Líquida
div_liq_10q = f_eua.indicador_divida_liquida(ten_k=False, caixa=caixa_10q, div_bruta=div_bruta_10q, idx=idx_10q)

# Dívida Líquida / EBITDA
dl_ebitda_10q = f_eua.indicador_dl_ebitda(ten_k=False, ebitda=ebitda_10q_acum, div_liq=div_liq_10q, idx=idx_10q)

# Dívida Líquida / PL
dl_pl_10q = f_eua.indicador_dl_pl(ten_k=False, div_liq=div_liq_10q, df_bs=meta_bs_10q) 

# EV/EBITDA
ev_ebitda_10q = f_eua.indicador_ev_ebitda(vm=vm_10q, div_liq=div_liq_10q, ebitda=ebitda_10q_acum)

# ROE
roe_10q = f_eua.indicador_roe(ten_k=False, df_is=meta_is_10q, df_bs=meta_bs_10q, idx=idx_10q)

# ROIC 
roic_10q = f_eua.indicador_roic(ten_k=False, df_is=meta_is_10q, df_bs=meta_bs_10q, div_bruta=div_bruta_10q, idx=idx_10q)

# FCO
fco_10q = f_eua.indicador_fco(ten_k=False, df_cf=meta_cf_10q, first_quarter='03') 

# FCI
fci_10q = f_eua.indicador_fci(ten_k=False, df_cf=meta_cf_10q, first_quarter='03') 

# FCF
fcf_10q = f_eua.indicador_fcf(ten_k=False, df_cf=meta_cf_10q, first_quarter='03') 

# CAPEX
lst_capex_10q = ['Purchases Of Property And Equipment And Intangible Assets']

capex_10q = f_eua.indicador_capex(ten_k=False, df_cf=meta_cf_10q, first_quarter='03', lst_itens=lst_capex_10q, idx=idx_10q)[0]
capex_10q_sliced = f_eua.indicador_capex(ten_k=False, df_cf=meta_cf_10q, first_quarter='03', lst_itens=lst_capex_10q, idx=idx_10q)[1]
capex_10q_acum = f_eua.indicador_capex(ten_k=False, df_cf=meta_cf_10q, first_quarter='03', lst_itens=lst_capex_10q, idx=idx_10q)[2]

# Free Cash Flow
free_cash_flow_10q = f_eua.indicador_free_cash_flow(fco=fco_10q, capex=capex_10q_sliced)

# Net CAPEX
net_capex_10q = f_eua.indicador_net_capex(capex=capex_10q_acum, depreciation=depreciacao_10q_acum)

# R&D
rd_expense_10q = f_eua.indicador_rd(ten_k=False, df_is=meta_is_10q, idx=idx_10q)[0]
rd_expense_10q_sliced = f_eua.indicador_rd(ten_k=False, df_is=meta_is_10q, idx=idx_10q)[1]
rd_expense_10q_acum = f_eua.indicador_rd(ten_k=False, df_is=meta_is_10q, idx=idx_10q)[2]

# Net CAPEX ajustado
adj_net_capex_10q = f_eua.indicador_adj_net_capex(net_capex=net_capex_10q, rd_expense=rd_expense_10q_acum)

# Working capital
working_capital_10q = f_eua.indicador_working_capital(ten_k=False, df_bs=meta_bs_10q)

# Change in non cash working capital
lst_itens_change_non_cash_wc = [
    'Deferred Income Tax Expense (Benefit)',
    'Increase (Decrease) in Accounts Receivable',
    'Increase (Decrease) in Prepaid Expense and Other Assets',
    'Increase (Decrease) in Other Operating Assets',
    'Increase (Decrease) in Accounts Payable, Trade',
    'Increase (Decrease) in Accrued Liabilities',
    'Increase (Decrease) in Other Noncurrent Liabilities'
]

change_non_cash_wc_10q = f_eua.indicador_change_non_cash_wc(ten_k=False, df_cf=meta_cf_10q, lst_itens=lst_itens_change_non_cash_wc, first_quarter='03')[0]
change_non_cash_wc_10q_sliced = f_eua.indicador_change_non_cash_wc(ten_k=False, df_cf=meta_cf_10q, lst_itens=lst_itens_change_non_cash_wc, first_quarter='03')[1]

# Reinvestment rate
reinvestment_rate_10q = f_eua.indicador_reinvestment_rate(ten_k=False, net_capex=adj_net_capex_10q, change_non_cash_wc=change_non_cash_wc_10q, df_is=meta_is_10q, idx=idx_10q)

# New borrowing
lst_itens_new_borrowing = ['Proceeds from Issuance of Long-term Debt']

new_borrowing_10q = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=meta_cf_10q, first_quarter='03', idx=idx_10q)[0]
new_borrowing_10q_sliced = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=meta_cf_10q, first_quarter='03', idx=idx_10q)[1]
new_borrowing_10q_acum = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=meta_cf_10q, first_quarter='03', idx=idx_10q)[2]

# Debt paid
lst_itens_debt_paid = ['Finance Lease, Principal Payments']

debt_paid_10q = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=meta_cf_10q, first_quarter='03', idx=idx_10q)[0]
debt_paid_10q_sliced = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=meta_cf_10q, first_quarter='03', idx=idx_10q)[1]
debt_paid_10q_acum = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=meta_cf_10q, first_quarter='03', idx=idx_10q)[2]

# Taxes on operating income
taxes_operating_income_10q = f_eua.indicador_taxes_operating_income(ten_k=False, df_is=meta_is_10q, idx=idx_10q)

# FCFE
fcfe_10q = f_eua.indicador_fcfe(
    ten_k=False, 
    depreciacao=depreciacao_10q_acum, 
    capex=capex_10q_acum, 
    change_non_cash_wc=change_non_cash_wc_10q_sliced, 
    new_borrowing=new_borrowing_10q_acum, 
    debt_paid=debt_paid_10q_acum, 
    df_is=meta_is_10q,
    idx=idx_10q
)

# FCFF
fcff_10q = f_eua.indicador_fcff(
        ten_k=False, 
        taxes_operating_income=taxes_operating_income_10q,
        depreciacao=depreciacao_10q_acum, 
        capex=capex_10q_acum, 
        change_non_cash_wc=change_non_cash_wc_10q_sliced, 
        df_is=meta_is_10q,
        idx=idx_10q
)

# Dividendo não acumulado
div_nao_acum = f_eua.indicador_div_nao_acum(df_cf=meta_cf_10q, first_quarter='03')

# Dividendo por Ação
dpa_10q = f_eua.indicador_dpa(ten_k=False, df_cf=meta_cf_10q, df_is=meta_is_10q, div_nao_acum=div_nao_acum)

# Payout
payout_10q = f_eua.indicador_payout(ten_k=False, df_cf=meta_cf_10q, df_is=meta_is_10q, div_nao_acum=div_nao_acum)

# Buyback
buyback_10q = f_eua.indicador_buyback(ten_k=False, df_cf=meta_cf_10q,  first_quarter='03')

# Lista com todos os indicadores
lst_indicadores_10q = [
    p_lpa_10q_damodaran,
    p_lpa_10q,
    lpa_p_10q,  
    ebitda_10q_sliced, 
    m_liq_10q,
    p_vpa_10q,
    vm_10q,
    div_bruta_10q,
    caixa_10q,
    div_liq_10q,
    dl_ebitda_10q,
    dl_pl_10q,
    ev_ebitda_10q,
    roe_10q,
    roic_10q,
    fco_10q,
    fci_10q,
    fcf_10q,
    free_cash_flow_10q,
    capex_10q_sliced,
    net_capex_10q,
    rd_expense_10q_sliced,
    adj_net_capex_10q,
    working_capital_10q,
    reinvestment_rate_10q,
    fcfe_10q,
    fcff_10q,
    dpa_10q,
    payout_10q,
    buyback_10q
]

# Juntando todos os indicadores em um df
meta_indicadores_10q = pd.concat(lst_indicadores_10q, axis=1)

# Nomes das colunas
nomes_col = [
    'P/L Damodaran', 
    'P/L', 
    'L/P', 
    'EBITDA', 
    'Margem Líquida',
    'P/VPA',
    'Valor de Mercado',
    'Dívida Bruta',
    'Caixa e Equivalentes',
    'Dívida Líquida',
    'Dívida Líquida/EBITDA',
    'Dívida Líquida/PL',
    'EV/EBITDA',
    'ROE',
    'ROIC',
    'FCO',
    'FCI',
    'FCF',
    'Free Cash Flow',
    'Capex',
    'Net Capex',
    'R&D',
    'Adj Net Capex',
    'Working Capital',
    'Reinvestment Rate',
    'FCFE',
    'FCFF',
    'DPA',
    'Payout',
    'Buyback'
]

# Atribuindo os novos nomes das colunas do df
meta_indicadores_10q.columns = nomes_col

# Preenchendo os NaN por zero
meta_indicadores_10q = meta_indicadores_10q.fillna(0)

meta_indicadores_10q

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//tecnologia//meta//meta_indicators.xlsx') as writer:
    meta_indicadores_10k.to_excel(writer, sheet_name='indicators_10k')
    meta_indicadores_10q.to_excel(writer, sheet_name='indicators_10q')

## Microsoft

* 1Q -> Lançamento em Setembro (09);
* 2Q -> Lançamento em Dezembro (12);
* 3Q -> Lançamento em Março (03);
* 10K -> Lançamento em Junho (06).

In [ ]:
ticker = 'msft'
f_sec.cik_matching_ticker(ticker)

### Scraping Microsoft 10-K

In [ ]:
# Número dos documentos que contém a DRE, BP e FC 
acc_documents = f_sec.get_filtered_filings(ticker=ticker, ten_k=True, just_accession_numbers=True)

# Trocando os '-' para '' das strings ('0001045810-24-000264' -> '000104581024000264')
dict_acc_num = {k: v.replace('-', '') for k, v in acc_documents.items()}

# Transformando o dicionário em um df
df_lst_acc_num_msft = pd.DataFrame.from_dict(dict_acc_num, orient='index', columns=['acc_num'])

# Adicionando os 'acc_num' de 2017 e 2018
df_acc_num = pd.DataFrame(
    {
        'acc_num': [
            '000156459018019062', # 2018
            '000156459017014900', # 2017
        ]
    },
    index=pd.to_datetime([
        '2018-06-30',
        '2017-06-30',
    ])
)

# Concatenando os dfs
df_lst_acc_num_msft = pd.concat([df_lst_acc_num_msft, df_acc_num])

# Transformando em datetime o index
df_lst_acc_num_msft.index = pd.to_datetime(df_lst_acc_num_msft.index, format='%Y-%m-%d')

df_lst_acc_num_msft

In [ ]:
# Rótulos
labels_dict = f_sec.get_label_dictionary(ticker, headers=f_sec.header)

In [ ]:
# Fazendo o scraping do balance sheet (10-K)
balance_sheet_lst = []

for acc_num in df_lst_acc_num_msft['acc_num']:

    # Balance sheet
    balance_sheet = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='balance_sheet')
    balance_sheet = f_sec.rename_statement(balance_sheet, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    balance_sheet = balance_sheet.iloc[:, 0]
    balance_sheet_lst.append(balance_sheet)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_balance_sheet_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in balance_sheet_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_balance_sheet_10k = pd.concat([df_balance_sheet_10k, df], axis=1)

# Substituindo os NaN por 0
df_balance_sheet_10k = df_balance_sheet_10k.fillna(0)

# A linha 'Deferred Income Tax Liabilities, Net' nas colunas '2019-06-30', '2018-06-30' e '2017-06-30' estão zeradas
# As informações estão na linha 'Deferred Tax Liabilities, Net, Noncurrent'
df_balance_sheet_10k.loc['Deferred Income Tax Liabilities, Net', '2019-06-30'] = df_balance_sheet_10k.loc['Deferred Tax Liabilities, Net, Noncurrent', '2019-06-30'] 
df_balance_sheet_10k.loc['Deferred Income Tax Liabilities, Net', '2018-06-30'] = df_balance_sheet_10k.loc['Deferred Tax Liabilities, Net, Noncurrent', '2018-06-30'] 
df_balance_sheet_10k.loc['Deferred Income Tax Liabilities, Net', '2017-06-30'] = df_balance_sheet_10k.loc['Deferred Tax Liabilities, Net, Noncurrent', '2017-06-30'] 

# A linha 'Short-term Investments' nas colunas '2018-06-30' e '2017-06-30' estão zeradas
# As informações estão na linha 'Available-for-sale Securities, Current'
df_balance_sheet_10k.loc['Short-term Investments', '2018-06-30'] = df_balance_sheet_10k.loc['Available-for-sale Securities, Current', '2018-06-30'] 
df_balance_sheet_10k.loc['Short-term Investments', '2017-06-30'] = df_balance_sheet_10k.loc['Available-for-sale Securities, Current', '2017-06-30'] 

# A linha 'Commercial Paper' na coluna '2017-06-30' está zerada
# As informações estão na linha 'Short-term Debt'
df_balance_sheet_10k.loc['Commercial Paper', '2017-06-30'] = df_balance_sheet_10k.loc['Short-term Debt', '2017-06-30'] 

# Removendo as linhas que movemos os seus itens para as linhas corretas
df_balance_sheet_10k = df_balance_sheet_10k.drop([
    'Deferred Tax Liabilities, Net, Noncurrent',
    'Available-for-sale Securities, Current',
    'Short-term Debt'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_balance_sheet_10k = df_balance_sheet_10k.drop([
    'us-gaap_AssetsCurrentAbstract',
    'Deferred Revenue, Current',
    'Deferred Revenue, Noncurrent',
    'Deposits Received for Securities Loaned, at Carrying Value'
])

# Arrumando e adicionando alguns números no df
df_balance_sheet_10k.loc['Operating Lease, Right-of-Use Asset', '2017-06-30'] = 6555
df_balance_sheet_10k.loc['Contract with Customer, Liability, Current', '2017-06-30'] = 24013
df_balance_sheet_10k.loc['Accrued Income Taxes, Noncurrent', '2017-06-30'] = 13485
df_balance_sheet_10k.loc['Contract with Customer, Liability, Noncurrent', '2017-06-30'] = 2643
df_balance_sheet_10k.loc['Operating Lease, Liability, Noncurrent', '2017-06-30'] = 5372

df_balance_sheet_10k

In [ ]:
# Fazendo o scraping do income statement (10-K)
income_statement_lst = []

for acc_num in df_lst_acc_num_msft['acc_num']:

    # Income statement
    income_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='income_statement')
    income_statement = f_sec.rename_statement(income_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    income_statement = income_statement.iloc[:, 0]
    income_statement_lst.append(income_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_income_statement_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in income_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_income_statement_10k = pd.concat([df_income_statement_10k, df], axis=1)

# Substituindo os NaN por 0
df_income_statement_10k = df_income_statement_10k.fillna(0)

# A linha 'Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest' nas colunas '2021-06-30', '2020-06-30', '2019-06-30', '2018-06-30' e '2017-06-30' estão zeradas
# As informações estão na linha 'Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest'
df_income_statement_10k.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2021-06-30'] = df_income_statement_10k.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2021-06-30'] 
df_income_statement_10k.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2020-06-30'] = df_income_statement_10k.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2020-06-30'] 
df_income_statement_10k.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2019-06-30'] = df_income_statement_10k.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2019-06-30'] 
df_income_statement_10k.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2018-06-30'] = df_income_statement_10k.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2018-06-30'] 
df_income_statement_10k.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2017-06-30'] = df_income_statement_10k.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2017-06-30'] 

# A linha 'Revenue from Contract with Customer, Excluding Assessed Tax' na coluna '2017-06-30' está zerada
# As informações estão na linha 'Sales Revenue, Goods, Net (Deprecated 2018-01-31)'
df_income_statement_10k.loc['Revenue from Contract with Customer, Excluding Assessed Tax', '2017-06-30'] = df_income_statement_10k.loc['Sales Revenue, Goods, Net (Deprecated 2018-01-31)', '2017-06-30'] 

# A linha 'Cost of Goods and Services Sold' na coluna '2017-06-30' está zerada
# As informações estão na linha 'msft_SalesRevenueServicesAndOtherNet'
df_income_statement_10k.loc['Cost of Goods and Services Sold', '2017-06-30'] = df_income_statement_10k.loc['msft_SalesRevenueServicesAndOtherNet', '2017-06-30'] 

# Removendo as linhas que movemos os seus itens para as linhas corretas
df_income_statement_10k = df_income_statement_10k.drop([
    'Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest',
    'Sales Revenue, Goods, Net (Deprecated 2018-01-31)',
    'msft_SalesRevenueServicesAndOtherNet'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_income_statement_10k = df_income_statement_10k.drop([
    'us-gaap_EarningsPerShareAbstract',
    'Restructuring Charges',
    'msft_ImpairmentAndRestructuringExpenses',
    'Common Stock, Dividends, Per Share, Declared',
    'us-gaap_SalesRevenueNetAbstract',
    'Revenue, Net (Deprecated 2018-01-31)',
    'Cost of Goods Sold (Deprecated 2018-01-31)',
    'Cost of Revenue',
    'msft_ImpairmentIntegrationAndRestructuringExpenses',
    'msft_CostOfServicesAndOther'
])

df_income_statement_10k

In [ ]:
# Fazendo o scraping do cash flow statement (10-K)
cash_flow_statement_lst = []

for acc_num in df_lst_acc_num_msft['acc_num']:

    # Cash flow statement
    cash_flow_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='cash_flow_statement')
    cash_flow_statement = f_sec.rename_statement(cash_flow_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    cash_flow_statement = cash_flow_statement.iloc[:, 0]
    cash_flow_statement_lst.append(cash_flow_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_cash_flow_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in cash_flow_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_cash_flow_10k = pd.concat([df_cash_flow_10k, df], axis=1)

# Substituindo os NaN por 0
df_cash_flow_10k = df_cash_flow_10k.fillna(0)

# A linha 'Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations' nas colunas '2021-06-30', '2020-06-30', '2019-06-30', '2018-06-30' e '2017-06-30' estão zeradas
# As informações estão na linha 'Effect of Exchange Rate on Cash and Cash Equivalents'
df_cash_flow_10k.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations', '2021-06-30'] = df_cash_flow_10k.loc['Effect of Exchange Rate on Cash and Cash Equivalents', '2021-06-30'] 
df_cash_flow_10k.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations', '2020-06-30'] = df_cash_flow_10k.loc['Effect of Exchange Rate on Cash and Cash Equivalents', '2020-06-30'] 
df_cash_flow_10k.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations', '2019-06-30'] = df_cash_flow_10k.loc['Effect of Exchange Rate on Cash and Cash Equivalents', '2019-06-30'] 
df_cash_flow_10k.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations', '2018-06-30'] = df_cash_flow_10k.loc['Effect of Exchange Rate on Cash and Cash Equivalents', '2018-06-30'] 
df_cash_flow_10k.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations', '2017-06-30'] = df_cash_flow_10k.loc['Effect of Exchange Rate on Cash and Cash Equivalents', '2017-06-30'] 

# A linha 'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect' nas colunas '2019-06-30', '2018-06-30' e '2017-06-30' estão zeradas
# As informações estão na linha 'Cash and Cash Equivalents, Period Increase (Decrease)'
df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect', '2019-06-30'] = df_cash_flow_10k.loc['Cash and Cash Equivalents, Period Increase (Decrease)', '2019-06-30'] 
df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect', '2018-06-30'] = df_cash_flow_10k.loc['Cash and Cash Equivalents, Period Increase (Decrease)', '2018-06-30'] 
df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect', '2017-06-30'] = df_cash_flow_10k.loc['Cash and Cash Equivalents, Period Increase (Decrease)', '2017-06-30'] 

# A linha 'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents' nas colunas '2019-06-30', '2018-06-30' e '2017-06-30' estão zeradas
# As informações estão na linha 'Cash and Cash Equivalents, at Carrying Value'
df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2019-06-30'] = df_cash_flow_10k.loc['Cash and Cash Equivalents, at Carrying Value', '2019-06-30'] 
df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2018-06-30'] = df_cash_flow_10k.loc['Cash and Cash Equivalents, at Carrying Value', '2018-06-30'] 
df_cash_flow_10k.loc['Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents', '2017-06-30'] = df_cash_flow_10k.loc['Cash and Cash Equivalents, at Carrying Value', '2017-06-30'] 

# A linha 'msft_ProceedsFromInvestments' nas colunas '2018-06-30' e '2017-06-30' estão zeradas
# As informações estão na linha 'Proceeds from Sale of Available-for-sale Securities'
df_cash_flow_10k.loc['msft_ProceedsFromInvestments', '2018-06-30'] = df_cash_flow_10k.loc['Proceeds from Sale of Available-for-sale Securities', '2018-06-30'] 
df_cash_flow_10k.loc['msft_ProceedsFromInvestments', '2017-06-30'] = df_cash_flow_10k.loc['Proceeds from Sale of Available-for-sale Securities', '2017-06-30'] 

# A linha 'Payments for (Proceeds from) Other Investing Activities' nas colunas '2018-06-30' e '2017-06-30' estão zeradas
# As informações estão na linha 'Increase (Decrease) in Collateral Held under Securities Lending'
df_cash_flow_10k.loc['Payments for (Proceeds from) Other Investing Activities', '2018-06-30'] = df_cash_flow_10k.loc['Increase (Decrease) in Collateral Held under Securities Lending', '2018-06-30'] 
df_cash_flow_10k.loc['Payments for (Proceeds from) Other Investing Activities', '2017-06-30'] = df_cash_flow_10k.loc['Increase (Decrease) in Collateral Held under Securities Lending', '2017-06-30'] 

# A linha 'Payments of Debt Restructuring Costs' na coluna '2020-06-30' está zerada
# As informações estão na linha 'Payment for Debt Extinguishment or Debt Prepayment Cost'
df_cash_flow_10k.loc['Payments of Debt Restructuring Costs', '2020-06-30'] = df_cash_flow_10k.loc['Payment for Debt Extinguishment or Debt Prepayment Cost', '2020-06-30'] 

# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_cash_flow_10k = df_cash_flow_10k.drop([
    'Effect of Exchange Rate on Cash and Cash Equivalents',
    'Cash and Cash Equivalents, Period Increase (Decrease)',
    'Cash and Cash Equivalents, at Carrying Value',
    'Proceeds from Sale of Available-for-sale Securities',
    'Increase (Decrease) in Collateral Held under Securities Lending',
    'Payment for Debt Extinguishment or Debt Prepayment Cost'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_cash_flow_10k = df_cash_flow_10k.drop([
    'us-gaap_NetCashProvidedByUsedInOperatingActivitiesAbstract',
    'Asset Impairment Charges',
    'Net Cash Provided by (Used in) Operating Activities, Continuing Operations',
    'Net Cash Provided by (Used in) Financing Activities, Continuing Operations',
    'Net Cash Provided by (Used in) Investing Activities, Continuing Operations',
    'us-gaap_NetCashProvidedByUsedInOperatingActivitiesContinuingOperationsAbstract'
])

# Arrumando e adicionando alguns números no df
df_cash_flow_10k.loc['Increase (Decrease) in Contract with Customer, Liability', '2017-06-30'] = 3820
df_cash_flow_10k.loc['Increase (Decrease) in Income Taxes Payable', '2017-06-30'] = 1792
df_cash_flow_10k.loc['Net Cash Provided by (Used in) Operating Activities', '2017-06-30'] = 39507
df_cash_flow_10k.loc['Net Cash Provided by (Used in) Financing Activities', '2017-06-30'] = 8408
df_cash_flow_10k.loc['Net Cash Provided by (Used in) Investing Activities', '2017-06-30'] = -46781

# Renomeando a linha do índice que está sem formato 
df_cash_flow_10k = df_cash_flow_10k.rename(index={
    'msft_DepreciationAmortizationAndOther': 'Depreciation Amortization And Other',
    'msft_GainLossOnInvestmentsAndDerivativeInstruments': 'Gain Loss On Investments And Derivative Instruments',
    'msft_AcquisitionsNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets': 'Acquisitions Net Of Cash Acquired And Purchases Of Intangible And Other Assets',
    'msft_ProceedsFromInvestments': 'Proceeds From Investments',
})

df_cash_flow_10k

In [ ]:
# Arrumando alguns index dos dfs para ficar padronizado com os index utilizados nas funções p/ calcular os indicadores fundamentalistas
df_balance_sheet_10k = df_balance_sheet_10k.rename(index={
    'Cash and Cash Equivalents, at Carrying Value': 'Cash and Cash Equivalents',
    'Long-term Investments': 'Equity and Other Investments',
    'Long-term Debt, Current Maturities': 'Current Portion of Long-term Debt',
    'Long-term Debt, Excluding Current Maturities': 'Long-term debt',
    "Stockholders' Equity Attributable to Parent": 'Total Shareholders Equity'
})

df_income_statement_10k = df_income_statement_10k.rename(index={
    'Revenue from Contract with Customer, Excluding Assessed Tax': 'Revenues',
    'Research and Development Expense': 'Research and Development',
    'Operating Income (Loss)': 'Operating Income',
    'Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest': 'Income Before Provision for Income Taxes',
    'Income Tax Expense (Benefit)': 'Provision for Income Taxes',
    'Net Income (Loss) Attributable to Parent': 'Net Income',
    'Weighted Average Number of Shares Outstanding, Basic': 'Number of Shares - Basic',
    'Weighted Average Number of Shares Outstanding, Diluted': 'Number of Shares - Diluted'
})

df_cash_flow_10k = df_cash_flow_10k.rename(index={
    'Depreciation Amortization And Other': 'Depreciation, Depletion and Amortization',
    'Property, Plant and Equipment, Net': 'Purchases Of Property And Equipment And Intangible Assets',
    'Payments of Ordinary Dividends, Common Stock': 'Payments of Dividends'
})

In [ ]:
# Selecionando o 'acc_num' do ano mais recente
last_acc_num = df_lst_acc_num_msft.iloc[0][0]

# Fazendo o scraping do interest expense (10-K) -> 'Other Income (Expense), Net' -> 'Interest expense'
interest_expense_10k = f_sec.process_one_statement(ticker=ticker, accession_number=last_acc_num, statement_name='interest_expense')
interest_expense_10k = f_sec.rename_statement(interest_expense_10k, label_dictionary=labels_dict)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
interest_expense_10k = interest_expense_10k.drop('us-gaap_OtherIncomeAndExpensesAbstract')

# Renomeando a linha do índice que está sem formato 
interest_expense_10k = interest_expense_10k.rename(index={
    'InterestExpenseNonoperating': 'Interest Expense',
})

# Formatando o número para ficar positivo
interest_expense_10k.loc['Interest Expense'] = interest_expense_10k.loc['Interest Expense'] * -1

# Selecionando a linha desejada
interest_expense_10k = interest_expense_10k.loc[['Interest Expense']]

interest_expense_10k

In [ ]:
# 'acc_num' entre 2018 e 2025
df_lst_acc_num_msft_sorted = df_lst_acc_num_msft.sort_index()
acc_num_list_msft_sliced = df_lst_acc_num_msft_sorted.loc['2018':'2025', 'acc_num']

# Fazendo o scraping do leases (10-K)
leases_lst = []

for acc_num in acc_num_list_msft_sliced:

    # Fazendo o scraping do leases (10-K) -> 'Note 8 – Leases' -> 'Total lease liabilities'
    leases_10k = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='leases')
    leases_10k = f_sec.rename_statement(leases_10k, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    leases_10k = leases_10k.iloc[:, 0]
    leases_lst.append(leases_10k)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_leases_10k = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in leases_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_leases_10k = pd.concat([df_leases_10k, df], axis=1)
    
# Renomeando as linhas do índice
df_leases_10k = df_leases_10k.rename(index={
    'Operating Lease, Liability': 'Total Operating Lease Liabilities',
    'Finance Lease, Liability': 'Total Finance Lease Liabilities',

})

# Selecionando as linhas desejadas
df_leases_10k = df_leases_10k.loc[['Total Operating Lease Liabilities', 'Total Finance Lease Liabilities']]

# Criando a coluna do ano de 2017 
df_leases_10k.loc['Total Operating Lease Liabilities', '2017-06-30'] = 6795
df_leases_10k.loc['Total Finance Lease Liabilities', '2017-06-30'] = 2538

# Ordenando o df
df_leases_10k = df_leases_10k[sorted(df_leases_10k.columns, reverse=True)]

df_leases_10k

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//tecnologia//microsoft//msft_10k.xlsx') as writer:
    df_balance_sheet_10k.to_excel(writer, sheet_name='balance_sheet')
    df_income_statement_10k.to_excel(writer, sheet_name='income_statement')
    df_cash_flow_10k.to_excel(writer, sheet_name='cash_flow')
    interest_expense_10k.to_excel(writer, sheet_name='interest_expense')
    df_leases_10k.to_excel(writer, sheet_name='leases')

### Scraping Microsoft 10-Q

In [ ]:
# Número dos documentos que contém a DRE, BP e FC 
acc_documents_10q = f_sec.get_filtered_filings(ticker=ticker, ten_k=False, just_accession_numbers=True)

# Trocando os '-' para '' das strings ('0001045810-24-000264' -> '000104581024000264')
dict_acc_num_10q = {k: v.replace('-', '') for k, v in acc_documents_10q.items()}

# Transformando o dicionário em um df
df_lst_acc_num_10q_msft = pd.DataFrame.from_dict(dict_acc_num_10q, orient='index', columns=['acc_num'])

# Transformando em datetime o index
df_lst_acc_num_10q_msft.index = pd.to_datetime(df_lst_acc_num_10q_msft.index, format='%Y-%m-%d')

df_lst_acc_num_10q_msft

In [ ]:
# Selecionando os 'acc_num' dos 10-q a partir de '2019-09-30' p/ iteração
df_lst_acc_num_10q_msft_sliced = df_lst_acc_num_10q_msft.loc['2019-09-30':]
df_lst_acc_num_10q_msft_sliced

In [ ]:
# Rótulos
labels_dict = f_sec.get_label_dictionary(ticker, headers=f_sec.header)

In [ ]:
# Fazendo o scraping do balance sheet (10-Q)
balance_sheet_lst = []

for acc_num in df_lst_acc_num_10q_msft_sliced['acc_num']:

    # Balance sheet
    balance_sheet = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='balance_sheet')
    balance_sheet = f_sec.rename_statement(balance_sheet, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    balance_sheet = balance_sheet.iloc[:, 0]
    balance_sheet_lst.append(balance_sheet)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_balance_sheet_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in balance_sheet_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_balance_sheet_10q = pd.concat([df_balance_sheet_10q, df], axis=1)

# Substituindo os NaN por 0
df_balance_sheet_10q = df_balance_sheet_10q.fillna(0)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_balance_sheet_10q = df_balance_sheet_10q.drop('us-gaap_AssetsCurrentAbstract')

df_balance_sheet_10q

In [ ]:
# Fazendo o scraping do income statement (10-Q)
income_statement_lst = []

for acc_num in df_lst_acc_num_10q_msft_sliced['acc_num']:

    # Income statement
    income_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='income_statement')
    income_statement = f_sec.rename_statement(income_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    income_statement = income_statement.iloc[:, 0]
    income_statement_lst.append(income_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_income_statement_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in income_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_income_statement_10q = pd.concat([df_income_statement_10q, df], axis=1)

# Substituindo os NaN por 0
df_income_statement_10q = df_income_statement_10q.fillna(0)

# A linha 'Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest' nas colunas '2021-09-30', '2021-03-31', '2020-12-31', '2020-09-30', '2020-03-31', '2019-12-31' e '2019-09-30' estao zeradas
# As informações estão na linha 'Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest'
df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2021-09-30'] = df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2021-09-30'] 
df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2021-03-31'] = df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2021-03-31'] 
df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2020-12-31'] = df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2020-12-31'] 
df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2020-09-30'] = df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2020-09-30'] 
df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2020-03-31'] = df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2020-03-31'] 
df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2019-12-31'] = df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2019-12-31'] 
df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest', '2019-09-30'] = df_income_statement_10q.loc['Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest', '2019-09-30'] 

# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_income_statement_10q = df_income_statement_10q.drop([
    'Income (Loss) from Continuing Operations before Equity Method Investments, Income Taxes, Noncontrolling Interest',
    'us-gaap_EarningsPerShareAbstract'
])

df_income_statement_10q

In [ ]:
# Fazendo o scraping do cash flow statement (10-Q)
cash_flow_statement_lst = []

for acc_num in df_lst_acc_num_10q_msft_sliced['acc_num']:

    # Cash flow statement
    cash_flow_statement = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='cash_flow_statement')
    cash_flow_statement = f_sec.rename_statement(cash_flow_statement, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    cash_flow_statement = cash_flow_statement.iloc[:, 0]
    cash_flow_statement_lst.append(cash_flow_statement)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_cash_flow_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in cash_flow_statement_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_cash_flow_10q = pd.concat([df_cash_flow_10q, df], axis=1)

# Substituindo os NaN por 0
df_cash_flow_10q = df_cash_flow_10q.fillna(0)

# A linha 'Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations' nas colunas '2022-03-31', '2021-12-31', '2021-09-30', '2021-03-31', '2020-12-31', '2020-09-30', '2020-03-31', '2019-12-31' e '2019-09-30' estão zeradas
# As informações estão na linha 'Effect of Exchange Rate on Cash and Cash Equivalents'
df_cash_flow_10q.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations', '2022-03-31'] = df_cash_flow_10q.loc['Effect of Exchange Rate on Cash and Cash Equivalents', '2022-03-31'] 
df_cash_flow_10q.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations', '2021-12-31'] = df_cash_flow_10q.loc['Effect of Exchange Rate on Cash and Cash Equivalents', '2021-12-31'] 
df_cash_flow_10q.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations', '2021-09-30'] = df_cash_flow_10q.loc['Effect of Exchange Rate on Cash and Cash Equivalents', '2021-09-30'] 
df_cash_flow_10q.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations', '2021-03-31'] = df_cash_flow_10q.loc['Effect of Exchange Rate on Cash and Cash Equivalents', '2021-03-31'] 
df_cash_flow_10q.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations', '2020-12-31'] = df_cash_flow_10q.loc['Effect of Exchange Rate on Cash and Cash Equivalents', '2020-12-31'] 
df_cash_flow_10q.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations', '2020-09-30'] = df_cash_flow_10q.loc['Effect of Exchange Rate on Cash and Cash Equivalents', '2020-09-30'] 
df_cash_flow_10q.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations', '2020-03-31'] = df_cash_flow_10q.loc['Effect of Exchange Rate on Cash and Cash Equivalents', '2020-03-31'] 
df_cash_flow_10q.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations', '2019-12-31'] = df_cash_flow_10q.loc['Effect of Exchange Rate on Cash and Cash Equivalents', '2019-12-31'] 
df_cash_flow_10q.loc['Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Including Disposal Group and Discontinued Operations', '2019-09-30'] = df_cash_flow_10q.loc['Effect of Exchange Rate on Cash and Cash Equivalents', '2019-09-30'] 

# A linha 'Proceeds from (Repayments of) Short-term Debt, Maturing in Three Months or Less' na coluna '2023-09-30' está zerada
# As informações estão na linha 'Proceeds from Short-term Debt, Maturing in Three Months or Less'
df_cash_flow_10q.loc['Proceeds from (Repayments of) Short-term Debt, Maturing in Three Months or Less', '2023-09-30'] = df_cash_flow_10q.loc['Proceeds from Short-term Debt, Maturing in Three Months or Less', '2023-09-30'] 

# Removendo as linhas que já que movemos os seus itens para as linhas corretas
df_cash_flow_10q = df_cash_flow_10q.drop([
    'Proceeds from Short-term Debt, Maturing in Three Months or Less',
    'Effect of Exchange Rate on Cash and Cash Equivalents'
])

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
df_cash_flow_10q = df_cash_flow_10q.drop('us-gaap_NetCashProvidedByUsedInOperatingActivitiesAbstract')

# Renomeando a linha do índice que está sem formato 
df_cash_flow_10q = df_cash_flow_10q.rename(index={
    'msft_DepreciationAmortizationAndOther': 'Depreciation Amortization And Other',
    'msft_GainLossOnInvestmentsAndDerivativeInstruments': 'Gain Loss On Investments And Derivative Instruments',
    'msft_AcquisitionsNetOfCashAcquiredAndPurchasesOfIntangibleAndOtherAssets': 'Acquisitions Net Of Cash Acquired And Purchases Of Intangible And Other Assets',
    'msft_ProceedsFromInvestments': 'Proceeds From Investments',
})

df_cash_flow_10q

In [ ]:
# Arrumando alguns index dos dfs para ficar padronizado com os index utilizados nas funções p/ calcular os indicadores fundamentalistas
df_balance_sheet_10q = df_balance_sheet_10q.rename(index={
    'Cash and Cash Equivalents, at Carrying Value': 'Cash and Cash Equivalents',
    'Long-term Investments': 'Equity and Other Investments',
    'Long-term Debt, Current Maturities': 'Current Portion of Long-term Debt',
    'Long-term Debt, Excluding Current Maturities': 'Long-term debt',
    "Stockholders' Equity Attributable to Parent": 'Total Shareholders Equity'
})

df_income_statement_10q = df_income_statement_10q.rename(index={
    'Revenue from Contract with Customer, Excluding Assessed Tax': 'Revenues',
    'Research and Development Expense': 'Research and Development',
    'Operating Income (Loss)': 'Operating Income',
    'Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest': 'Income Before Provision for Income Taxes',
    'Income Tax Expense (Benefit)': 'Provision for Income Taxes',
    'Net Income (Loss) Attributable to Parent': 'Net Income',
    'Weighted Average Number of Shares Outstanding, Basic': 'Number of Shares - Basic',
    'Weighted Average Number of Shares Outstanding, Diluted': 'Number of Shares - Diluted'
})

df_cash_flow_10q = df_cash_flow_10q.rename(index={
    'Depreciation Amortization And Other': 'Depreciation, Depletion and Amortization',
    'Payments to Acquire Property, Plant, and Equipment': 'Purchases Of Property And Equipment And Intangible Assets',
    'Payments of Ordinary Dividends, Common Stock': 'Payments of Dividends'
})

In [ ]:
# Selecionando o 'acc_num' do trimestre mais recente
last_acc_num_10q = df_lst_acc_num_10q_msft_sliced.iloc[0].item()

# Fazendo o scraping do interest expense -> 'Other Income (Expense), Net' -> 'Interest expense'
interest_expense_10q = f_sec.process_one_statement(ticker=ticker, accession_number=last_acc_num_10q, statement_name='interest_expense')
interest_expense_10q = f_sec.rename_statement(interest_expense_10q, label_dictionary=labels_dict)

# Removendo as linhas que não possuem nenhuma informação (NaN ou zeros)
interest_expense_10q = interest_expense_10q.drop('us-gaap_OtherIncomeAndExpensesAbstract')

# Renomeando a linha do índice que está sem formato 
interest_expense_10q = interest_expense_10q.rename(index={
    'InterestExpenseNonoperating': 'Interest Expense',
})

# Formatando o número para ficar positivo
interest_expense_10q.loc['Interest Expense'] = interest_expense_10q.loc['Interest Expense'] * -1

# Selecionando a linha desejada
interest_expense_10q = interest_expense_10q.loc[['Interest Expense']]

# Selecionando a 1º e 2º coluna do df que são os nº não acumulados
interest_expense_10q = interest_expense_10q.iloc[:, [0,1]]

interest_expense_10q

In [ ]:
# Fazendo o scraping do leases (10-K)
leases_lst = []

for acc_num in  df_lst_acc_num_10q_msft_sliced['acc_num']:

    # Fazendo o scraping do leases (10-K) -> 'Note 8 – Leases' -> 'Total lease liabilities'
    leases_10q = f_sec.process_one_statement(ticker=ticker, accession_number=acc_num, statement_name='leases')
    leases_10q = f_sec.rename_statement(leases_10q, label_dictionary=labels_dict)
    # Selecionando apenas a primeira coluna do df
    leases_10q = leases_10q.iloc[:, 0]
    leases_lst.append(leases_10q)

# Juntando todos os dfs que estão em listas em apenas um df
# Para não dar o erro na hora da concatenação 'InvalidIndexError: Reindexing only valid with uniquely valued Index objects'ArithmeticError
# Tenho que usar a função 'keep=first'
df_leases_10q = pd.DataFrame()

# Iterando sobre cada DataFrame na lista e concatene-os ao DataFrame resultante
for df in leases_lst:
    df = df[~df.index.duplicated(keep='first')]  # Removendo os rótulos duplicados se necessário
    df_leases_10q = pd.concat([df_leases_10q, df], axis=1)
    
# Renomeando as linhas do índice
df_leases_10q = df_leases_10q.rename(index={
    'Operating Lease, Liability': 'Total Operating Lease Liabilities',
    'Finance Lease, Liability': 'Total Finance Lease Liabilities',

})

# Selecionando as linhas desejadas
df_leases_10q = df_leases_10q.loc[['Total Operating Lease Liabilities', 'Total Finance Lease Liabilities']]

# Ordenando o df
df_leases_10q = df_leases_10q[sorted(df_leases_10q.columns, reverse=True)]

df_leases_10q

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//tecnologia//microsoft//msft_10q.xlsx') as writer:
    df_balance_sheet_10q.to_excel(writer, sheet_name='balance_sheet')
    df_income_statement_10q.to_excel(writer, sheet_name='income_statement')
    df_cash_flow_10q.to_excel(writer, sheet_name='cash_flow')
    interest_expense_10q.to_excel(writer, sheet_name='interest_expense')
    df_leases_10q.to_excel(writer, sheet_name='leases')

### Indicadores Fundamentalistas

#### Indicadores 10-K

In [ ]:
# Lendo os arquivos 10k
msft_bs = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//microsoft//msft_10k.xlsx', 
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

msft_is = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//microsoft//msft_10k.xlsx', 
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

msft_cf = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//microsoft//msft_10k.xlsx', 
    sheet_name='cash_flow', 
    index_col='Unnamed: 0'
)

msft_leases = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//microsoft//msft_10k.xlsx', 
    sheet_name='leases', 
    index_col='Unnamed: 0'
)

# Transformando as colunas em datetime
msft_bs.columns = pd.to_datetime(msft_bs.columns)
msft_is.columns = pd.to_datetime(msft_is.columns)
msft_cf.columns = pd.to_datetime(msft_cf.columns)
msft_leases.columns = pd.to_datetime(msft_leases.columns)

# Transformando as colunas em apenas o ano
msft_bs.columns = msft_bs.columns.year
msft_is.columns = msft_is.columns.year
msft_cf.columns = msft_cf.columns.year
msft_leases.columns = msft_leases.columns.year

In [ ]:
# Preços da MSFT
msft_close = yf.download('MSFT', auto_adjust=True, multi_level_index=False, start='2017-01-01')['Close']

# Selecionando o último preço do mês de Dezembro de cada ano 
lst_years = [
    '2017', '2018', '2019', '2020', 
    '2021', '2022', '2023', '2024', 
    '2025'
]

last_dec_close = []

for year in lst_years:
    last_close = msft_close[f'{year}-06'].iloc[-1]
    last_dec_close.append(last_close)

# Transformando em uma série pandas
msft_close_jun = pd.Series(last_dec_close, index=lst_years)

# Revertendo o index da série p/ ficar igual com os dos indicadores
msft_close_jun = msft_close_jun.reindex(index=msft_close_jun.index[::-1])

# Arredondamento de duas casas decimais
msft_close_jun = round(msft_close_jun, 2)

# Não realizou split recentemente (https://br.investing.com/equities/microsoft-corp-historical-data-splits)

# Transformando o index em int
msft_close_jun .index = msft_close_jun .index.astype(int)

msft_close_jun

In [ ]:
# Criando uma lista só para fazer a indexação em que o ano mais recente está no começo
idx = msft_bs.columns

# Valor de Mercado
vm = f_eua.indicador_vm(ten_k=True, price=msft_close_jun, df_is=msft_is) 

# LPA
lpa = f_eua.indicador_lpa(ten_k=True, df_is=msft_is, idx=None)

# P/L Damodaran
p_lpa_damodaran = f_eua.indicador_p_lpa_2(ten_k=True, vm=vm, df_is=msft_is)

# P/L
p_lpa = f_eua.indicador_p_lpa(ten_k=True, price=msft_close_jun, lpa=lpa, idx=idx)

# L/P
lpa_p = f_eua.indicador_lpa_p(ten_k=True, price=msft_close_jun, lpa=lpa, idx=idx) 

# Depreciação
depreciacao = f_eua.indicador_depreciacao(ten_k=True, df_cf=msft_cf, first_quarter=None, idx=idx)

# EBITDA
ebitda = f_eua.indicador_ebitda(ten_k=True, df_is=msft_is, idx=None, depreciacao=depreciacao)

# Margem Líquida
m_liq = f_eua.indicador_ml(ten_k=True, df_is=msft_is)

# VPA
vpa = f_eua.indicador_vpa(ten_k=True, df_bs=msft_bs, df_is=msft_is)

# P/VPA
p_vpa = f_eua.indicador_p_vpa(ten_k=True, price=msft_close_jun, vpa=vpa, idx=idx)

# Caixa e Equivalentes
lst_caixa = ['Cash and Cash Equivalents', 'Short-term Investments']
caixa = f_eua.indicador_caixa(ten_k=True, df_bs=msft_bs, lst_itens_caixa=lst_caixa)

# Dívida Bruta
lst_divida = ['Current Portion of Long-term Debt', 'Long-term debt']
div_bruta = f_eua.indicador_divida_bruta(ten_k=True, df_bs=msft_bs, lst_itens_divida=lst_divida)
div_bruta_total = div_bruta.values + msft_leases.sum().values
div_bruta_total = pd.Series(div_bruta_total, index=idx)

# Dívida Líquida
div_liq = f_eua.indicador_divida_liquida(ten_k=True, caixa=caixa, div_bruta=div_bruta_total, idx=idx)

# Dívida Líquida / EBITDA
dl_ebitda = f_eua.indicador_dl_ebitda(ten_k=True, ebitda=ebitda, div_liq=div_liq, idx=None)

# Dívida Líquida / PL
dl_pl = f_eua.indicador_dl_pl(ten_k=True, div_liq=div_liq, df_bs=msft_bs) 

# EV/EBITDA
ev_ebitda = f_eua.indicador_ev_ebitda(vm=vm, div_liq=div_liq, ebitda=ebitda)

# ROE
roe = f_eua.indicador_roe(ten_k=True, df_is=msft_is, df_bs=msft_bs, idx=None)

# ROIC
roic = f_eua.indicador_roic(ten_k=True, df_is=msft_is, df_bs=msft_bs, div_bruta=div_bruta_total, idx=idx)

# FCO
fco = f_eua.indicador_fco(ten_k=True, df_cf=msft_cf, first_quarter=None) 

# FCI
fci = f_eua.indicador_fci(ten_k=True, df_cf=msft_cf, first_quarter=None) 

# FCF
fcf = f_eua.indicador_fcf(ten_k=True, df_cf=msft_cf, first_quarter=None) 

# CAPEX
lst_capex = ['Payments to Acquire Property, Plant, and Equipment']

capex= f_eua.indicador_capex(ten_k=True, df_cf=msft_cf, first_quarter=None, lst_itens=lst_capex, idx=idx)

# Free Cash Flow
free_cash_flow = f_eua.indicador_free_cash_flow(fco=fco, capex=capex)

# Net CAPEX
net_capex = f_eua.indicador_net_capex(capex=capex, depreciation=depreciacao)

# R&D
rd_expense = f_eua.indicador_rd(ten_k=True, df_is=msft_is, idx=idx)

# Net CAPEX ajustado
adj_net_capex = f_eua.indicador_adj_net_capex(net_capex=net_capex, rd_expense=rd_expense)

# Working capital
working_capital = f_eua.indicador_working_capital(ten_k=True, df_bs=msft_bs)

# Change in non cash working capital
lst_itens_change_non_cash_wc = [
    'Deferred Income Tax Expense (Benefit)',
    'Increase (Decrease) in Accounts Receivable',
    'Increase (Decrease) in Inventories',
    'Increase (Decrease) in Other Current Assets',
    'Increase (Decrease) in Other Noncurrent Assets',
    'Increase (Decrease) in Accounts Payable',
    'Increase (Decrease) in Contract with Customer, Liability',
    'Increase (Decrease) in Income Taxes Payable',
    'Increase (Decrease) in Other Current Liabilities',
    'Increase (Decrease) in Other Noncurrent Liabilities'
]

change_non_cash_wc = f_eua.indicador_change_non_cash_wc(ten_k=True, df_cf=msft_cf, lst_itens=lst_itens_change_non_cash_wc, first_quarter=None)

# Reinvestment rate
reinvestment_rate = f_eua.indicador_reinvestment_rate(ten_k=True, net_capex=net_capex, change_non_cash_wc=change_non_cash_wc, df_is=msft_is, idx=idx)

# New borrowing
lst_itens_new_borrowing = ['Proceeds from Debt, Maturing in More than Three Months']

new_borrowing = f_eua.indicador_new_borrowing(ten_k=True, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=msft_cf, first_quarter=None, idx=idx)

# Debt paid
lst_itens_debt_paid = [
    'Repayments of Debt, Maturing in More than Three Months', 
    'Proceeds from (Repayments of) Short-term Debt, Maturing in Three Months or Less'
]

debt_paid = f_eua.indicador_debt_paid(ten_k=True, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=msft_cf, first_quarter=None, idx=idx)

# Taxes on operating income
taxes_operating_income = f_eua.indicador_taxes_operating_income(ten_k=True, df_is=msft_is, idx=idx)

# FCFE
fcfe_10k = f_eua.indicador_fcfe(
    ten_k=True, 
    depreciacao=depreciacao, 
    capex=capex, 
    change_non_cash_wc=change_non_cash_wc, 
    new_borrowing=new_borrowing,
    debt_paid=debt_paid,
    df_is=msft_is,
    idx=idx
)

# FCFF
fcff_10k = f_eua.indicador_fcff(
        ten_k=True, 
        taxes_operating_income=taxes_operating_income,
        depreciacao=depreciacao, 
        capex=capex, 
        change_non_cash_wc=change_non_cash_wc, 
        df_is=msft_is,
        idx=idx
)

# Dividendo por Ação
dpa = f_eua.indicador_dpa(ten_k=True, df_cf=msft_cf, df_is=msft_is, div_nao_acum=None)

# Payout
payout = f_eua.indicador_payout(ten_k=True, df_cf=msft_cf, df_is=msft_is, div_nao_acum=None)

# Buyback
buyback = f_eua.indicador_buyback(ten_k=True, df_cf=msft_cf, first_quarter=None)

# Colocando todos os indicadores em uma lista
lst_indicadores = [
    p_lpa_damodaran,
    p_lpa,
    lpa_p,  
    ebitda, 
    m_liq,
    p_vpa,
    vm,
    div_bruta_total,
    caixa,
    div_liq,
    dl_ebitda,
    dl_pl,
    ev_ebitda,
    roe,
    roic,
    fco,
    fci,
    fcf,
    free_cash_flow,
    capex,
    net_capex,
    rd_expense,
    adj_net_capex,
    working_capital,
    reinvestment_rate,
    fcfe_10k,
    fcff_10k,
    dpa,
    payout,
    buyback
]

# Juntando todos os indicadores em um df
msft_indicadores_10k = pd.concat(lst_indicadores, axis=1)

# Nomes das colunas
nomes_col = [
    'P/L Damodaran', 
    'P/L', 
    'L/P', 
    'EBITDA', 
    'Margem Líquida',
    'P/VPA',
    'Valor de Mercado',
    'Dívida Bruta',
    'Caixa e Equivalentes',
    'Dívida Líquida',
    'Dívida Líquida/EBITDA',
    'Dívida Líquida/PL',
    'EV/EBITDA',
    'ROE',
    'ROIC',
    'FCO',
    'FCI',
    'FCF',
    'Free Cash Flow',
    'Capex',
    'Net Capex',
    'R&D',
    'Adj Net Capex',
    'Working Capital',
    'Reinvestment Rate',
    'FCFE',
    'FCFF',
    'DPA',
    'Payout',
    'Buyback'
]

# Atribuindo os novos nomes das colunas do df
msft_indicadores_10k.columns = nomes_col
msft_indicadores_10k

#### Indicadores 10-Q

In [ ]:
# Lendo os arquivos 10q
msft_bs_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//microsoft//msft_10q.xlsx',
    sheet_name='balance_sheet', 
    index_col='Unnamed: 0'
)

msft_is_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//microsoft//msft_10q.xlsx',
    sheet_name='income_statement', 
    index_col='Unnamed: 0'
)

msft_cf_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//microsoft//msft_10q.xlsx',
    sheet_name='cash_flow',  
    index_col='Unnamed: 0'
)

msft_leases_10q = pd.read_excel(
    'C://B3//analise-fundamentalista//analise_eua//tecnologia//microsoft//msft_10q.xlsx',
    sheet_name='leases',  
    index_col='Unnamed: 0'
)

# Aplicando o ajuste nas colunas
msft_bs_10q.columns = [f_sec.ajustar_mes_12_03_06(col) for col in msft_bs_10q.columns]
msft_is_10q.columns = [f_sec.ajustar_mes_12_03_06(col) for col in msft_is_10q.columns]
msft_cf_10q.columns = [f_sec.ajustar_mes_12_03_06(col) for col in msft_cf_10q.columns]
msft_leases_10q.columns = [f_sec.ajustar_mes_12_03_06(col) for col in msft_leases_10q.columns]

In [ ]:
# Preços da MSFT
msft_close = yf.download('MSFT', auto_adjust=True, multi_level_index=False, start='2020-01-01')['Close']

# Selecionando o último preço dos meses de Março, Junho e Dezembro
lst_years = [
    '2020', '2021', '2022', '2023', 
    '2024', '2025',
]
lst_months = ['03', '09', '12']
last_close_months = []

for year in lst_years:
    for month in lst_months:
        last_close = msft_close[f'{year}-{month}'].dropna().iloc[-1]  # Dropna para lidar com meses sem dados
        last_close_months.append(last_close)

# Criando uma lista em que junte os meses e os anos para que seja o index da serie pandas
lst_idx = []

for year in lst_years:
    for month in lst_months:
        lst_idx.append(f"{month}-{year}")

# Transformando em uma série pandas
msft_close_months = pd.Series(last_close_months, index=lst_idx)

# Revertendo o index da série p/ ficar igual com os dos indicadores
msft_close_months = msft_close_months.reindex(index=msft_close_months.index[::-1])

# Retirando o preço de '03-2020' 
msft_close_months = msft_close_months.drop(index='03-2020')


# No ano mais atual, como ainda ele não acabou, não consigo fazer o loop com todos meses (Março, Junho e Dezembro)
actual_year_int = datetime.now().year  
actual_year_str = str(actual_year_int) 

lst_actual_months = [
    # '03',
    # '09',
    # '12'
] 

last_close_actual_months = []

for month in lst_actual_months:
    last_close = msft_close[f'{actual_year_str}-{month}'].dropna().iloc[-1]  # Dropna para lidar com meses sem dados
    last_close_actual_months.append(last_close)

# Criando uma lista em que junte os meses e os anos para que seja o index da serie pandas
lst_idx_actual = []

for month in lst_actual_months:
    lst_idx_actual.append(f"{month}-{actual_year_str}")

# Transformando em uma série pandas
msft_close_months_atual = pd.Series(last_close_actual_months, index=lst_idx_actual)

# Revertendo o index da série p/ ficar igual com os dos indicadores
msft_close_months_atual = msft_close_months_atual.reindex(index=msft_close_months_atual.index[::-1])

# Juntando as duas series pandas
msft_close_months = pd.concat([msft_close_months_atual, msft_close_months])

# Arredondamento de duas casas decimais
msft_close_months = round(msft_close_months, 2)

# Não realizou split recentemente (https://br.investing.com/equities/microsoft-corp-historical-data-splits)

# Substituindo o index para ficar igual ao dos dados fundamentalistas
msft_close_months.index = msft_bs_10q.columns[:-3]

msft_close_months

In [ ]:
# Criando uma lista só para fazer a indexação em que o ano mais recente está no começo
idx_10q = msft_bs_10q.columns

# Valor de Mercado
vm_10q = f_eua.indicador_vm(ten_k=False, price=msft_close_months, df_is=msft_is_10q)

# LPA
lpa_acum = f_eua.indicador_lpa(ten_k=False, df_is=msft_is_10q, idx=idx_10q)[1]
lpa_10q = f_eua.indicador_lpa(ten_k=False, df_is=msft_is_10q, idx=idx_10q)[0]

# P/L Damodaran
p_lpa_10q_damodaran = f_eua.indicador_p_lpa_2(ten_k=False, vm=vm_10q, df_is=msft_is_10q)

# P/L
p_lpa_10q = f_eua.indicador_p_lpa(ten_k=False, price=msft_close_months, lpa=lpa_acum, idx=idx_10q)

# L/P
lpa_p_10q = f_eua.indicador_lpa_p(ten_k=False, price=msft_close_months, lpa=lpa_acum, idx=idx_10q) 

# Depreciação
depreciacao_10q = f_eua.indicador_depreciacao(ten_k=False, df_cf=msft_cf_10q, first_quarter='09', idx=idx_10q)[0]
depreciacao_10q_sliced = f_eua.indicador_depreciacao(ten_k=False, df_cf=msft_cf_10q, first_quarter='09', idx=idx_10q)[1]
depreciacao_10q_acum = f_eua.indicador_depreciacao(ten_k=False, df_cf=msft_cf_10q, first_quarter='09', idx=idx_10q)[2]

# EBITDA
ebitda_10q = f_eua.indicador_ebitda(ten_k=False, df_is=msft_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[0]
ebitda_10q_sliced = f_eua.indicador_ebitda(ten_k=False, df_is=msft_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[1]
ebitda_10q_acum = f_eua.indicador_ebitda(ten_k=False, df_is=msft_is_10q, idx=idx_10q, depreciacao=depreciacao_10q)[2]

# Margem Líquida
m_liq_10q = f_eua.indicador_ml(ten_k=False, df_is=msft_is_10q)

# VPA
vpa_10q = f_eua.indicador_vpa(ten_k=False, df_bs=msft_bs_10q, df_is=msft_is_10q)

# P/VPA
p_vpa_10q = f_eua.indicador_p_vpa(ten_k=False, price=msft_close_months, vpa=vpa_10q, idx=idx_10q)

# Caixa e Equivalentes
lst_caixa = ['Cash and Cash Equivalents', 'Short-term Investments']
caixa_10q = f_eua.indicador_caixa(ten_k=False, df_bs=msft_bs_10q, lst_itens_caixa=lst_caixa)

# Dívida Bruta
lst_divida = ['Current Portion of Long-term Debt', 'Long-term debt']
div_bruta_10q = f_eua.indicador_divida_bruta(ten_k=False, df_bs=msft_bs_10q, lst_itens_divida=lst_divida)
div_bruta_total_10q = div_bruta_10q.values + msft_leases_10q.sum().values[0:-3]
div_bruta_total_10q = pd.Series(div_bruta_total_10q, index=idx_10q[0:-3])

# Dívida Líquida
div_liq_10q = f_eua.indicador_divida_liquida(ten_k=False, caixa=caixa_10q, div_bruta=div_bruta_total_10q, idx=idx_10q)

# Dívida Líquida / EBITDA
dl_ebitda_10q = f_eua.indicador_dl_ebitda(ten_k=False, ebitda=ebitda_10q_acum, div_liq=div_liq_10q, idx=idx_10q)

# Dívida Líquida / PL
dl_pl_10q = f_eua.indicador_dl_pl(ten_k=False, div_liq=div_liq_10q, df_bs=msft_bs_10q) 

# EV/EBITDA
ev_ebitda_10q = f_eua.indicador_ev_ebitda(vm=vm_10q, div_liq=div_liq_10q, ebitda=ebitda_10q_acum)

# ROE
roe_10q = f_eua.indicador_roe(ten_k=False, df_is=msft_is_10q, df_bs=msft_bs_10q, idx=idx_10q)

# ROIC 
roic_10q = f_eua.indicador_roic(ten_k=False, df_is=msft_is_10q, df_bs=msft_bs_10q, div_bruta=div_bruta_total_10q, idx=idx_10q)

# FCO
fco_10q = f_eua.indicador_fco(ten_k=False, df_cf=msft_cf_10q, first_quarter='09') 

# FCI
fci_10q = f_eua.indicador_fci(ten_k=False, df_cf=msft_cf_10q, first_quarter='09') 

# FCF
fcf_10q = f_eua.indicador_fcf(ten_k=False, df_cf=msft_cf_10q, first_quarter='09') 

# CAPEX
lst_capex_10q = ['Purchases Of Property And Equipment And Intangible Assets']

capex_10q = f_eua.indicador_capex(ten_k=False, df_cf=msft_cf_10q, first_quarter='09', lst_itens=lst_capex_10q, idx=idx_10q)[0]
capex_10q_sliced = f_eua.indicador_capex(ten_k=False, df_cf=msft_cf_10q, first_quarter='09', lst_itens=lst_capex_10q, idx=idx_10q)[1]
capex_10q_acum = f_eua.indicador_capex(ten_k=False, df_cf=msft_cf_10q, first_quarter='09', lst_itens=lst_capex_10q, idx=idx_10q)[2]

# Free Cash Flow
free_cash_flow_10q = f_eua.indicador_free_cash_flow(fco=fco_10q, capex=capex_10q_sliced)

# Net CAPEX
net_capex_10q = f_eua.indicador_net_capex(capex=capex_10q_acum, depreciation=depreciacao_10q_acum)

# R&D
rd_expense_10q = f_eua.indicador_rd(ten_k=False, df_is=msft_is_10q, idx=idx_10q)[0]
rd_expense_10q_sliced = f_eua.indicador_rd(ten_k=False, df_is=msft_is_10q, idx=idx_10q)[1]
rd_expense_10q_acum = f_eua.indicador_rd(ten_k=False, df_is=msft_is_10q, idx=idx_10q)[2]

# Net CAPEX ajustado
adj_net_capex_10q = f_eua.indicador_adj_net_capex(net_capex=net_capex_10q, rd_expense=rd_expense_10q_acum)

# Working capital
working_capital_10q = f_eua.indicador_working_capital(ten_k=False, df_bs=msft_bs_10q)

# Change in non cash working capital
lst_itens_change_non_cash_wc = [
    'Deferred Income Tax Expense (Benefit)',
    'Increase (Decrease) in Accounts Receivable',
    'Increase (Decrease) in Inventories',
    'Increase (Decrease) in Other Current Assets',
    'Increase (Decrease) in Other Noncurrent Assets',
    'Increase (Decrease) in Accounts Payable',
    'Increase (Decrease) in Contract with Customer, Liability',
    'Increase (Decrease) in Income Taxes Payable',
    'Increase (Decrease) in Other Current Liabilities',
    'Increase (Decrease) in Other Noncurrent Liabilities'
]

change_non_cash_wc_10q = f_eua.indicador_change_non_cash_wc(ten_k=False, df_cf=msft_cf_10q, lst_itens=lst_itens_change_non_cash_wc, first_quarter='09')[0]
change_non_cash_wc_10q_sliced = f_eua.indicador_change_non_cash_wc(ten_k=False, df_cf=msft_cf_10q, lst_itens=lst_itens_change_non_cash_wc, first_quarter='09')[1]

# Reinvestment rate
reinvestment_rate_10q = f_eua.indicador_reinvestment_rate(ten_k=False, net_capex=adj_net_capex_10q, change_non_cash_wc=change_non_cash_wc_10q, df_is=msft_is_10q, idx=idx_10q)

# New borrowing
lst_itens_new_borrowing = ['Proceeds from Debt, Maturing in More than Three Months']

new_borrowing_10q = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=msft_cf_10q, first_quarter='09', idx=idx_10q)[0]
new_borrowing_10q_sliced = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=msft_cf_10q, first_quarter='09', idx=idx_10q)[1]
new_borrowing_10q_acum = f_eua.indicador_new_borrowing(ten_k=False, lst_itens_new_borrowing=lst_itens_new_borrowing, df_cf=msft_cf_10q, first_quarter='09', idx=idx_10q)[2]

# Debt paid
lst_itens_debt_paid = ['Proceeds from (Repayments of) Short-term Debt, Maturing in Three Months or Less', 'Repayments of Debt, Maturing in More than Three Months']

debt_paid_10q = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=msft_cf_10q, first_quarter='09', idx=idx_10q)[0]
debt_paid_10q_sliced = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=msft_cf_10q, first_quarter='09', idx=idx_10q)[1]
debt_paid_10q_acum = f_eua.indicador_debt_paid(ten_k=False, lst_itens_debt_paid=lst_itens_debt_paid, df_cf=msft_cf_10q, first_quarter='09', idx=idx_10q)[2]

# Taxes on operating income
taxes_operating_income_10q = f_eua.indicador_taxes_operating_income(ten_k=False, df_is=msft_is_10q, idx=idx_10q)

# FCFE
fcfe_10q = f_eua.indicador_fcfe(
    ten_k=False, 
    depreciacao=depreciacao_10q_acum, 
    capex=capex_10q_acum, 
    change_non_cash_wc=change_non_cash_wc_10q_sliced, 
    new_borrowing=new_borrowing_10q_acum, 
    debt_paid=debt_paid_10q_acum, 
    df_is=msft_is_10q,
    idx=idx_10q
)

# FCFF
fcff_10q = f_eua.indicador_fcff(
        ten_k=False, 
        taxes_operating_income=taxes_operating_income_10q,
        depreciacao=depreciacao_10q_acum, 
        capex=capex_10q_acum, 
        change_non_cash_wc=change_non_cash_wc_10q_sliced, 
        df_is=msft_is_10q,
        idx=idx_10q
)

# Dividendo não acumulado
div_nao_acum = f_eua.indicador_div_nao_acum(df_cf=msft_cf_10q, first_quarter='09')

# Dividendo por Ação
dpa_10q = f_eua.indicador_dpa(ten_k=False, df_cf=msft_cf_10q, df_is=msft_is_10q, div_nao_acum=div_nao_acum)

# Payout
payout_10q = f_eua.indicador_payout(ten_k=False, df_cf=msft_cf_10q, df_is=msft_is_10q, div_nao_acum=div_nao_acum)

# Buyback
buyback_10q = f_eua.indicador_buyback(ten_k=False, df_cf=msft_cf_10q,  first_quarter='09')

# Lista com todos os indicadores
lst_indicadores_10q = [
    p_lpa_10q_damodaran,
    p_lpa_10q,
    lpa_p_10q,  
    ebitda_10q_sliced, 
    m_liq_10q,
    p_vpa_10q,
    vm_10q,
    div_bruta_total_10q,
    caixa_10q,
    div_liq_10q,
    dl_ebitda_10q,
    dl_pl_10q,
    ev_ebitda_10q,
    roe_10q,
    roic_10q,
    fco_10q,
    fci_10q,
    fcf_10q,
    free_cash_flow_10q,
    capex_10q_sliced,
    net_capex_10q,
    rd_expense_10q_sliced,
    adj_net_capex_10q,
    working_capital_10q,
    reinvestment_rate_10q,
    fcfe_10q,
    fcff_10q,
    dpa_10q,
    payout_10q,
    buyback_10q
]

# Juntando todos os indicadores em um df
msft_indicadores_10q = pd.concat(lst_indicadores_10q, axis=1)

# Nomes das colunas
nomes_col = [
    'P/L Damodaran', 
    'P/L', 
    'L/P', 
    'EBITDA', 
    'Margem Líquida',
    'P/VPA',
    'Valor de Mercado',
    'Dívida Bruta',
    'Caixa e Equivalentes',
    'Dívida Líquida',
    'Dívida Líquida/EBITDA',
    'Dívida Líquida/PL',
    'EV/EBITDA',
    'ROE',
    'ROIC',
    'FCO',
    'FCI',
    'FCF',
    'Free Cash Flow',
    'Capex',
    'Net Capex',
    'R&D',
    'Adj Net Capex',
    'Working Capital',
    'Reinvestment Rate',
    'FCFE',
    'FCFF',
    'DPA',
    'Payout',
    'Buyback'
]

# Atribuindo os novos nomes das colunas do df
msft_indicadores_10q.columns = nomes_col

# Preenchendo os NaN por zero
msft_indicadores_10q = msft_indicadores_10q.fillna(0)

msft_indicadores_10q

In [ ]:
# Criando o arquivo excel em que cada informação está em uma aba diferente
with pd.ExcelWriter('C://B3//analise-fundamentalista//analise_eua//tecnologia//microsoft//msft_indicators.xlsx') as writer:
    msft_indicadores_10k.to_excel(writer, sheet_name='indicators_10k')
    msft_indicadores_10q.to_excel(writer, sheet_name='indicators_10q')